In [1]:
import pandas as pd
import re
import numpy as np
import emoji
import contractions
import unidecode
from tqdm import tqdm
pd.options.display.max_colwidth=200
pd.options.display.max_rows=None

In [2]:
data = pd.read_csv('tweets.csv')

/var/folders/2k/fhkt65ls68nbd5xt0lj3sbww0000gn/T/ipykernel_59839/3005909632.py:2: DtypeWarning: Columns (0,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/rosni/Library/CloudStorage/OneDrive-NJIT/Dissertation/curbside/Data/Python files/Final_data.csv')


371272

In [3]:
data.columns

Index(['index', 'author id', 'created_at', 'geo', 'id', 'lang', 'like_count',
       'quote_count', 'reply_count', 'retweet_count', 'source', 'tweet',
       'Category', 'channel'],
      dtype='object')

In [6]:
data['channel'].value_counts()

delivery    329103
curbside     42162
Name: channel, dtype: int64

In [7]:
data['Category'].value_counts()

Consumer         8379
Advertisement    8092
Name: Category, dtype: int64

In [8]:
data = data.drop_duplicates(subset='tweet')
len(data)

368414

In [9]:
data['Category'].value_counts()

Consumer         8377
Advertisement    8092
Name: Category, dtype: int64

In [10]:
data.head()

,index,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,Category,channel,new_tweet
0,5,47199132.0,2015-01-21 19:23:29+00:00,,5.580000e+17,en,0.0,0.0,0.0,0.0,Twitter for iPhone,"@jhineline meanwhile up the street, grocery shoppers park at trader joe's using the convenienttent curbside.",Consumer,curbside,"@jhineline meanwhile up the street, grocery shoppers park at trader joe's using the convenienttent curbside."
1,8,34079088.0,2015-01-13 15:29:39+00:00,,5.550000e+17,en,1.0,0.0,0.0,2.0,Twitter for iPhone,".@walmart is in trial phase of online orderering + curbside pickup with grocery, truly a disruptor and convenienttent! #nrf15 #futureofretail",Consumer,curbside,".@walmart is in trial phase of online orderering + curbside pickup with grocery, truly a disruptor and convenienttent! #nrf15 #futureofretail"
2,9,119482829.0,2015-01-09 23:01:37+00:00,,5.540000e+17,en,0.0,0.0,1.0,0.0,Twitter for iPhone,thank you @harristeeter for providing a curbside grocery pick up. i love you. that's all.,Consumer,curbside,thank you @harristeeter for providing a curbside grocery pick up. i love you. that's all.
3,10,718558608.0,2015-01-08 04:08:17+00:00,,5.530000e+17,en,0.0,0.0,0.0,0.0,Twitter for iPhone,i can't get to the grocery store weekly to buy food &amp; you want me to haul trash across town? support curbside #recycle,Advertisement,curbside,i can't get to the grocery store weekly to buy food &amp; you want me to haul trash across town? support curbside #recycle
4,14,16347551.0,2015-02-24 19:47:44+00:00,01fbe706f872cb32,5.700000e+17,en,0.0,0.0,2.0,0.0,Twitter for iPhone,loving @peapod delivers i was in &amp; out of the grocery store with a week of food using their curbside pick up in 2 minutes. #gamechanging,Consumer,curbside,loving @peapod delivers i was in &amp; out of the grocery store with a week of food using their curbside pick up in 2 minutes. #gamechanging


In [11]:
data = data.drop(data.index[245249])

In [12]:
data['created_at'] = pd.to_datetime(data['created_at'])
data['Year'] = data['created_at'].dt.year
data['Month'] = data['created_at'].dt.month_name()
data['Day'] = data['created_at'].dt.day_name()

In [13]:
print(min(data['created_at']))
print(max(data['created_at']))

2015-01-01 00:07:25+00:00
2022-11-29 23:51:09+00:00


In [14]:
count = data['tweet'].str.split().str.len()
print("Total number of words: ", count.sum())
print("Average number of words per tweet: ", round(count.mean(),2))
print("Minimum number of words per tweet: ", round(count.min(),2))
print("Maximum number of words per tweet: ", round(count.max(),2))
print("Maximum number of words per tweet: ", round(count.max(),2))
print("Median number of words per tweet: ", round(count.median(),2))

Total number of words:  9761584
Average number of words per tweet:  26.5
Minimum number of words per tweet:  2
Maximum number of words per tweet:  108
Maximum number of words per tweet:  108
Median number of words per tweet:  23.0


In [15]:
df = data[data['created_at'] >= '2022-01-01']
len(df)

30047

In [16]:
data = data.reset_index()

In [17]:
data.columns

Index(['level_0', 'index', 'author id', 'created_at', 'geo', 'id', 'lang',
       'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source',
       'tweet', 'Category', 'channel', 'new_tweet', 'Year', 'Month', 'Day'],
      dtype='object')

In [18]:
data = data.drop('level_0', axis=1)
data = data.drop('index', axis=1)

In [19]:
data.columns

Index(['author id', 'created_at', 'geo', 'id', 'lang', 'like_count',
       'quote_count', 'reply_count', 'retweet_count', 'source', 'tweet',
       'Category', 'channel', 'new_tweet', 'Year', 'Month', 'Day'],
      dtype='object')

### Convert to lower case

In [20]:
def lower_case(txt):
    return txt.lower()
data['tweet'] = data['tweet'].apply(lambda x : lower_case(x))
print(data['tweet'][5003])
print(data['tweet'][5006])

@cdcarter13 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.
@padme_kenobi @kaylive @fakemorganhope @itsaronbitch @robwhite010 @davidfholt they also say it’s probably safer to get food from a restaurant curb side versus grocery shopping right now. less exposure. they shouldn’t close. also workers should watch hands after each curbside delivery.


### Get number of tweets with retweet

In [21]:
#get number of tweets with retweet
def retweet_count(text):
    if 'cc@' in text:
        return True
    else:
        return False

df = data['tweet'].apply(lambda x: retweet_count(x))
df.value_counts()

False    368412
True          1
Name: tweet, dtype: int64

### Get all html tags in tweet

In [22]:
def get_html_tags(text):
    tags = []
    html = re.findall('<.*?>',text)
    if html != None:
        tags.extend(html)
    return tags

html_tags = data['tweet'].apply(lambda x: get_html_tags(x))
html_tags = [i for i in html_tags if len(i)!=0]
html_tags

[]

### Get all markdown links in tweet

In [23]:
def get_html_tags(text):
    tags = []
    html = re.findall(r'\[.*?\]\(.*?\)',text)
    if html != None:
        tags.extend(html)
    return tags

html_tags = data['tweet'].apply(lambda x: get_html_tags(x))
html_tags = [i for i in html_tags if len(i)!=0]
html_tags

[['[start using drones](https://t.co/zs7mt3cr8o)']]

### Get html elements in the tweets

In [24]:
html_elements = []
for i in range(len(data)):
    regexp = "&.+?;" 
    html_elements.extend(re.findall(regexp,data['tweet'][i]))
html_elements = set(html_elements)

from html import unescape
for i in html_elements:
    print(i,"=", unescape(i))
    break

&lt; = <


### Replace html elements in the tweets

In [25]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('&amp;', '&', x))
data['tweet'] = data['tweet'].apply(lambda x : re.sub('&gt;', '>', x))
data['tweet'] = data['tweet'].apply(lambda x : re.sub('&lt;', '<', x))

### Remove mentions

In [26]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('@(\w+)', '', x))
print(data['tweet'][5003])

 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.


### Replace URL with 'hyperlink'

In [27]:
print(data['tweet'][50015])

 amazon has grocery delivery??? where the hell have i been???


In [28]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub(r'http\S+', 'hyperlink', x))
data['tweet'] = data['tweet'].apply(lambda x : re.sub(r'www\.[^ ]+', 'hyperlink', x))
print(data['tweet'][5003])
print(data['tweet'][50015])

 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.
 amazon has grocery delivery??? where the hell have i been???


In [29]:
def count_link(data):
    words = ' '.join(data['tweet']).split()
    count = 0
    for word in words:
        if word == 'hyperlink':
            count += 1
    return count

def count_i(data):
    words = ' '.join(data['tweet']).split()
    count = 0
    for word in words:
        if word == 'i':
            count += 1
    return count

def count_my(data):
    words = ' '.join(data['tweet']).split()
    count = 0
    for word in words:
        if word == 'my':
            count += 1
    return count



In [30]:
print(count_link(data), count_i(data), count_my(data))

194361 167064 87123


### Replace email ids with keyword email

In [31]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', r'email',x))
print(data['tweet'][5003])
print(data['tweet'][5006])

 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.
      they also say it’s probably safer to get food from a restaurant curb side versus grocery shopping right now. less exposure. they shouldn’t close. also workers should watch hands after each curbside delivery.


In [32]:
print(count_link(data), count_i(data), count_my(data))

194361 167064 87123


### Check number of tweet with emojis

In [33]:
def check_emoji(text):
    emo = emoji.UNICODE_EMOJI
    emojis = {**emo['en'],**emo['es'],**emo['pt'],**emo['it'],**emo['fr'],**emo['de']}
    allchars = [i for i in text]
    emoji_list = [c for c in allchars if c in emojis]
    words = text.split()
    contain_emoji = False
    for word in words:
        if word in emoji_list:
            contain_emoji = True
        else:
            contain_emoji = False
    return contain_emoji

tqdm.pandas()
count_emoji = data['tweet'].progress_apply(lambda x: check_emoji(x))
print(count_emoji.value_counts().to_dict())
#print(f'The {count_emoji.value_counts()[1]/count_emoji.value_counts().sum()} of the tweets have emojis')

100%|█████████████████████████████████| 368413/368413 [02:18<00:00, 2664.10it/s]

{False: 357119, True: 11294}


In [34]:
emoji_dict = {'😩': 'tired', '🤦': 'embarassment', '😅': 'happy', '🤣': 'funny', '😎': 'awesome','😂': 'happy', '🙌': 'joy', '👀': 'curious', '👌': 'approval', '🤢': 'disgust', '🙂': 'happy', '😭': 'unhappy', '💵': 'dollar',  '❤': 'love', '🤷': 'ignorance','😬': 'embarassment', '✔': 'approval', 
              '😃': 'happy', '🎯': 'success', '😍': 'love','😤': 'frustration', '🙏': 'gratitude','👏': 'applaud', '👍': 'approval', '🤔': 'thinking', '🙃': 'silliness', '💕': 'love', '🎉': 'celebration','🤩': 'fascinate', '😖': 'frustration', '🤬': 'angry', '🤗': 'happy', '💃': 'happy', '🤯': 'frustration', '😑': 'frustration','✨': 'excitement', '💞': 'love', '🙄': 'disapproval', '😉': 'joke', '😄': 'happy', '💜': 'love', '😡': 'angry','😁': 'happy', '😱': 'fearful:', '😒': 'unhappy', 
              '😋': 'tasty', '😐': 'frustration', '✌': 'peace', '😆': 'funny','😇': 'blessing', '😔': 'regretful', '💯': 'agree','🙈': 'disbelieving', '😕': 'confused', '😊': 'happy','😫': ':tired', '💰': 'money','🤪': 'funny', '😿': 'sad', '😣': 'frustration', '🥴': 'confused', '🤞': 'luck','🙅': 'disapproval', '🤘': 'love', '💩': 'angry','😳': 'embarrassment',  '💛': 'love', '☹': 'sad','😜': 'funny','🥰': 'love', '🙆': 'approval', '🤤': 'longing', '😏': 'smug', 
              '💚': 'love', '🥺': 'pleading', '🤥': 'lying','💖': 'love',  '🧐': 'ponder', '😥': ':sad', '🙇': 'apology', '😢': 'sad','🤕': 'hurt', '☺': 'happy', '😶': 'silence', '😧': 'sad', '💔': 'sad','♥': 'love','🤓': 'nerd', '😲': 'surprise', '🧡': 'love', '🚫': ':prohibited:','😰': ':anxious', '😝': 'funny', '👎': 'disapproval','😀': 'happy', '🌞': 'happy','😵': 'disbelief','💙': 'love', '😯': 'surprise', '🙁': 'angry','😞': ':disappoint', '😘': 'love', '😺': 'happy',
              '🤨': 'confused', '😠': 'angry', '😪': ':sleepy','😦': 'sad', '💗': 'love','😓': 'frustration', '😨': ':fearful', '😌': ':relieved', '❣': 'love',  '🤡': 'funny','😮': 'amazed', '🥳': 'happy','😈': 'trouble',  '🤮': 'disgust', '😴': ':sleeping','😻': 'love', '🥶': 'cold', '🤭': 'embarrassment,', '🖕': 'angry', '💘': 'love', '🤑': 'money','🥲': 'gratitude', '🤟': 'love', '💫': 'dizzy', '😟': 'worry', '🖤': 'love','💸': 'money', '😛': 'fun','🙍': 'disapproval', 
              '💟': 'love', '🙀': 'tired', '🤒': 'ill', '😙': 'love', '💝': 'love', '👿': 'angry', '👪': 'family', '🤰': 'pregnant woman',  
              '😸': 'happy', '🤐': 'speechless', '🤫': 'quiet','🥱': 'tired', '💢': 'anger'}

In [35]:
import json

In [36]:
#read dictionary from json file
with open("../../Supplementary data/emoji_dictionary.json","r") as f:
    emoji_dict = json.load(f)
len(emoji_dict)

145

In [37]:
print(data['tweet'][8])
print(data['tweet'][16])

walmart testing curbside grocery pickup in huntsville hyperlink
proudly introducing "shopping for survivors" - a free grocery shopping and curbside pick-up service for cancer patients. thanks !


In [38]:
#Remove emojis
tqdm.pandas()
def give_emoji_free_text(text, emoji_dict):
    emo = emoji.UNICODE_EMOJI
    emojis = {**emo['en'],**emo['es'],**emo['pt'],**emo['it'],**emo['fr'],**emo['de']}
    all_lst = []
    words = text.split()
    for word in words:
        lst = []
        allchars = [i for i in word]
        for j in allchars:
            if j in emojis:
                if j in emoji_dict:
                    lst.append(' ')
                    lst.append(emoji_dict[j])
            else:
                lst.append(j)
        all_lst.append(''.join(lst))
    return ' '.join(all_lst)

data['tweet'] = data['tweet'].progress_apply(lambda x: give_emoji_free_text(x,emoji_dict))
print(data['tweet'][8])
print(data['tweet'][16])

100%|█████████████████████████████████| 368413/368413 [02:25<00:00, 2528.54it/s]

walmart testing curbside grocery pickup in huntsville hyperlink
proudly introducing "shopping for survivors" - a free grocery shopping and curbside pick-up service for cancer patients. thanks !


In [39]:
print(count_link(data), count_i(data), count_my(data))

194525 167083 87128


### Remove contractions

In [40]:
import contractions
data['tweet'] = data['tweet'].apply(lambda x: contractions.fix(x))
print(data['tweet'][5003])

my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.


In [41]:
print(count_link(data), count_i(data), count_my(data))

194525 213515 87128


### Remove stopwords

In [42]:
#remove stopwords
words = ' '.join(data['tweet']).split()
print(len(words))
from nltk.corpus import stopwords
stop = ['what','which','this','that',"that'll",'these','those','is','are','was','were','be','been','being','have',
              'has','had','having','do','does','did','doing','a','an','the', 'and','but','if','or','because','as','until',
              'while','of','at','by','for','with','about','against','between','into','through','during','before','after',
              'above','below','to','from','up','down','in','out','on','off','over','under','again','further','then',
              'once','here','there','when','where','why','how','all','any','both','each','few','more','most','other',
              'some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just',
              'don',"don't",'should',"should've",'now','d','ll','m','o','re','ve','y','ain','aren',"aren't",'couldn',
              "couldn't", 'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",
              'ma','mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",'wasn',
              "wasn't",'weren',"weren't",'won',"won't",'wouldn',"wouldn't"]
data['tweet'] =  data['tweet'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
print(data['tweet'][5003])
print(data['tweet'][16])
words = ' '.join(data['tweet']).split()
print(len(words))

9705812
my wife stopped grocery store curbside pickup they put your orderer your trunk contact. she she saw tons boomers going store protection all.
proudly introducing "shopping survivors" - free grocery shopping curbside pick-up service cancer patients. thanks !
6708406


In [43]:
print(count_link(data), count_i(data), count_my(data))

194525 213515 87128


### Remove next line characters

In [44]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('\n', ' ', x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact. she she saw tons boomers going store protection all.
they also say it probably safer get food restaurant curb side versus grocery shopping right now. less exposure. they close. also workers watch hands curbside delivery.


In [45]:
print(count_link(data), count_i(data), count_my(data))

194525 213515 87128


### Remove accented characters to their ASCII values

In [46]:
def accented_to_ascii(text):
    text = unidecode.unidecode(text)
    return text
data['tweet'] = data['tweet'].apply(lambda x: accented_to_ascii(x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact. she she saw tons boomers going store protection all.
they also say it probably safer get food restaurant curb side versus grocery shopping right now. less exposure. they close. also workers watch hands curbside delivery.


In [47]:
print(count_link(data), count_i(data), count_my(data))

194578 213532 87129


### Remove special characters

In [48]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('[!"#%“[•►–&”\'()*+,-/:;<=>?@\\^_{|}~`]', ' ', x))
print(data['tweet'][5003])
print(data['tweet'][16])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact  she she saw tons boomers going store protection all 
proudly introducing  shopping survivors    free grocery shopping curbside pick up service cancer patients  thanks  


In [49]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Remove spaces

In [50]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub(' +', ' ', x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all 
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery 


In [51]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Replacing currency symbol by word

In [52]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub(r"\$(\d+)", r"\1 dollars", x))
print(data['tweet'][780])

 ucwradio walmart expands its curbside grocery pickup service you s hyperlink technology


In [53]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Remove numbers

In [54]:
import num2words
def remove_num(txt):
    return ' '.join(' ' if i.isdigit() else i for i in txt.split())

data['tweet'] = data['tweet'].apply(lambda x : remove_num(x))  
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [55]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Find words with multiple same consecutive letters

In [56]:
two_consecutive_letters = []
three_consecutive_letters = []
count_2 = 0
count_3 = 0
for i in tqdm(range(len(data))):
    words = data['tweet'][i].split(' ')
    for word in words:
        if re.search(r'((\w)\2{2,})', word):
            count_3 += 1
            if word not in three_consecutive_letters:
                three_consecutive_letters.append(word)
        elif re.search(r'((\w)\2{1,})', word):
            count_2 += 1
            if word not in two_consecutive_letters:
                two_consecutive_letters.append(word)
print(count_2, count_3)

100%|████████████████████████████████| 368413/368413 [00:17<00:00, 21271.17it/s]

809575 3794


In [57]:
words_to_remove = to_remove = ['bbb', 'litttt', 'daaaaa', 'holllla', 'ggg', 'cccsc', 'oooo', 'ppp', '1000xs', 'ooooo', 'ffffff', 'haaa','brrr', 'econ1110', 'a444', 'mmmpanadas', 'toowaahhhh', 'mmmmmmmm', 'mmmagtweets', 'zzzzzz', 'mmmmm', 'ifttt', 'from1000km', 'bellletstalk', 'xxx', 'mmm', 'zzzz', 'vrai777', 'tracycopygrrrl', 
             'zzzzzzzzzzzzzzz', 'ooo', 'jefffrey', 'whole30fff', 'yyccc', 'mmmm', 'mmmmmm', 'hrmmm', 'fffuuuu', '1000mg', 'aaa', 'www', '7771x29', 'ffffffffffff', '2000s', 'taaac', 'brandongaillle', 'niggaaaaa','$qqq', 'heee', 'ummmmm', 'paullly', 'theyllgetcontactlessintheyear3000', 'hbuuu', 'sumkt444', 'nascarvvvrroooommmmmmm', 'kkkempsey', 'ttttttt', 'tttt', 'oooooo', 'lorderdddd', 'teamfccc', 
             'wwwwwwwwwww', '1000s', 'lll', 'ieeeorg', 'reeee', 'ltdjnrtnnnbggngvcfi', 'wwww', 'hhh', 'onnnnnnn','hheee', 'mmmmmmm', 'xxxx', '1000000000x', 'eee', 'ffbbbllaarrrrrrnnttts', 'wlll', 'pcccommunitymarkets',
             'paccc', 'supportpaccc', 'amoooo', 'cj333', 'balllin', '000mph', 'reeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee','squeee', '000s', 'hhhh', 'rdockmatthews111', '5000sq', '1000x', 'aaaaaaa', 'booo19', 'ggggggggg','mysicklecelllife', 'faaaaa', 'maccleennnny', 'hmmmmm', 'aaaaaaaaaaaa', '000ppl', 'hmmmyess', 'aaaaaa','bestnyaaaa', 'aaaa', '000sq', '2000k', 'ccc', 'eeee', 'ooota', 'neee', 'waangooonline', 'hal2000bot',
             'vcccares', '$glittergrl2000', 'nuuu', 'jaaahaaack', 'breeellery', 'aaaaaaaaaaa', 'bbbslc', 'hhhhhhhhh','3000th', 'sss', 'pleading$sbreezyyy', 'siaaa', 'aaaaaaaa', '15902201009000473m', '15904058179022263m', 'zzzzzzz', 'zzzzzzzzzzzzzzzzz', '000th']

In [58]:
total_count = 0
for i in tqdm(range(len(data))):
    count = 0
    words = data['tweet'][i].split(' ')
    for word in words:
        if word in words_to_remove:
            count += 1
    total_count += count
total_count            

100%|████████████████████████████████| 368413/368413 [00:08<00:00, 45062.32it/s]


415

In [59]:
def remove_words(text, words_to_remove):
    words = text.split(' ')
    new_words = []
    for word in words:
        if word not in words_to_remove:
            new_words.append(word)
    return ' '.join(new_words)
data['tweet'] = data['tweet'].progress_apply(lambda x : remove_words(x, words_to_remove))  
print(data['tweet'][5003])
print(data['tweet'][5006])                   

100%|████████████████████████████████| 368413/368413 [00:07<00:00, 49712.97it/s]

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [60]:
replace_list = {'hummmm': 'hum','alll':'all','allll':'all','alllll':'all','buttt':'but', 'butttt':'but','shoppinggg':'shopping','lifeee': 'life', 'ehhhhhhhh': 'eh','hourrr': 'hour', 'weeeeeeeeeeeeeeeeeeeeeeeeee': 'we', 'buuuuuuuuut': 'but','ohhh': 'oh', 'sooo': 'so', 'superbowlllll': 'superbowl', 'damnnnn': 'damn', 'verrrry': 'very', 'oooh': 'oh', 'aahhhh': 'ah', 'fuckkkk': 'fuck','yooo': 'yo', 'effffffffffff': 'eff', 
                'okurrrrrrrr': 'ok', 'bizzzznesss': 'business', 'waaaaayyy': 'way', 'soooo': 'so','fffffffuuuuuuck': 'fuck', 'hiiii': 'hi', 'ssoooo': 'so', 'lmaooo': 'lmao', 'mannnnn': 'man', 'businesssupportingbusiness': 'business supporting business', 'eeeek': 'ek', 'alll': 'all', 'pssst': 'past', 'cccp': 'cp', 'annnnd': 'and','ummm': 'umm','hmmm': 'hmm', 'wonderfullly': 'wonderfully', 'smallbusinessstrong': 'small business strong','mayyyyyybe': 'may be', 'waaaaay': 'way', 'psssst': 'past','mooood': 'mood', 'usuallly': 'usually','ahhhh': 'aah', 'yasss': 'yes', 'lossses': 'losses', 'nevaaaa': 'never', 'stilllovewegmans': 'still love wegmans', 'mannn': 'man', 'awwww': 'aww', 'fukkkk': 'fuck', 'exciiited': 'excited', 'ahhh': 'aah', 
                'allll': 'all', 'outdooors': 'outdoors', 'uuugh': 'ugh', 'aaah': 'ahh', 'blesssss': 'bless', 'maaaybe': 'may be', 'dayssss':'days', 'bruhhh': 'bruh', 'itttttttt': 'it', 'waaaaaaaaaaaay': 'way', 'raaaaah': 'rah','eatwelllivewellku': 'eat well live well','yyyyes': 'yes', 'goooooooo': 'go', 'alllllllllll': 'all', 'aaand': 'and','ahhhhh': 'aah', 'hmmmm': 'hmm', 'fuuuuccckkkkkk': 'fuck', 'thoughtttt': 'thought', 'buttt': 'but', 'storessss': 'stores', 'goooo': 'go', 'ohhhh': 'oh', 'thennnn': 'then', 'whyyy': 'why', 'dumbbb': 'dumb', 'fuuuuuck': 'fuck', 'alllll': 'all','wooeee': 'woe', 'boooooo': 'boo', 'nahhhhh': 'nah', 'monthssss': 'months', 'shoooing': 'shooing', 'massshootings': 'mass shootings', 'amerikkka': 'amerika', 'hiiighly': 'highly', 'awww': 'aww', 'eeesh': 'esh', 'jooookeee': 'joke', 'realllllyyy': 'really',
                'okokokokokkkk': 'ok','aaah':'aah','aaahh':'aah', 'ahhh':'aah','ahhhh':'aah','ahhhhh':'aah','byeee': 'bye', 'yeahhh': 'yeah', 'neeeda': 'need', 'shiiiiiiz': 'shit', 'buuuut': 'but', 'hateeeee': 'hate', 'otttt': 'ot', 'hnnng': 'hang', 'aaaaugh': 'ugh', 'partayyyyyy': 'party', 'heatedddd': 'heated', 'aayyy': 'aay', 'buuutttt': 'but', 'trashhhh': 'trash', 'waaaay': 'way', 'whaaat': 'what', 'uhhh': 'ugh','sweetoooo': 'sweet', 'fuuuuq': 'fuck', 'honeyyy': 'honey', 'aaaand': 'and', 'teeeellll': 'tell', 'coolinnnnnn': 'colin', 'businessnewzzz': 'business news', 'grrr': 'grr','hellllllo': 'hello', 'pleaseee': 'please', 'whaaaaa': 'wha', 'whhhyyyyy': 'why', 'ewwwwww': 'eww','grrrr': 'grr', 'lolll': 'lol', 'bihhhhhh': 'bih', 'yesssss': 'yes', 'arghhhh': 'argh', 'lovellloves': 'love','pfffft': 'pft', 'lessshoppingmorewriting': 'less shopping more writing', 'workswonderfullly': 'works wonderfully', 'grocerybusinessservice': 'grocery business service', 'litttttt': 'lit', 'juuuuust': 'just', 'couchhhhhh': 'couch', 'insteaddddd': 'instead', 'innnnteresting': 'interesting', 'waaaaah': 'wah', 'fuuuuuuck': 'fuck', 'gawwwwd': 'gawd', 'sooooo': 'so', 'daaang': 'dang', 'hellllllllllp': 'help', 'sawww': 'saw', 'booo': 'boo', 'yaaaasssss': 'yes', 'grrrrrr': 'grr', 'livveee': 'live', 'starvingggg': 'starving','worthlessshopper': 'worthless hopper', 'pleaseeee': 'please', 'terrrrible': 'terrible', 'wrrrrr': 'wrr', 'neeeeeeed': 'need','gurrrrrr': 'grr','outtttt': 'out', 'nowzzz': 'now', 'maaaan': 'man', 'hungryyyyy': 'hungry', 'reeeeeeally': 'really', 'wheee': 'whee', 'groooooans': 'groan', 'errmaaaagaaaahdddd': 'omg', 'booooooooooooo': 'boo', 'haaaaaaaaaaaaaaaaaaaaaaaaaaate': 'hate','frannnd': 'frand', 'hmmmmmm': 'hm', 'arrreeeeee': 'are', 'laaawd': 'loud', 
                'foodddddddddd': 'food','coffeee':'coffee', 'lesssocietymoreworders': 'less society more worders', 'unluckyyyyy': 'unlucky', 'naaahr': 'nah', 'sweeeeet': 'sweet', 'ahhhhhh': 'ahh', 'eeep': 'eep', 'baaaare': 'bare', 'geeky0001': 'geky01', 'loooooook': 'lok', 'dribbble': 'drible', 'm9mk8888m9':'m9mk8m9', 'yassss': 'yas', 'jeeez': 'jeez', 'miiighty': 'mighty', 'ummmm': 'umm', 'moooostly': 'mostly', 'whaaaaaa': 'wha', 'ooooppssss': 'oops', 'daaaa': 'daa', 'woooffffff': 'woof', 'wannna': 'wana', 'selllig': 'selig', 'convenienttentttttt': 'convenientent', 'speeedy': 'speedy', 'whaaaaat': 'what', 'yaaaas': 'yes', 'hnnnnnng': 'hang','beeeeeeeeeach': 'beach', 'strollling': 'strolling', 'dddribbble': 'drible','wiiggggg': 'wig',  'reeeeeal': 'real', 'dolll': 'doll', 'uppp': 'up', 'reallllllllly': 'really', 'loveee': 'love', 'girrrrl': 'girl', 'cmoooooon': 'come on', 'neeeeed': 'need', 'fuuuuuuuuuuuuuuuuuuuuuuuckkkk': 'fuck','businessscript': 'business script', 'ehhh': 'eh', 'cluuutch': 'clutch', 'kiiinda': 'kinda','reeeaaaaally': 'really', 'eeeugh': 'enough', 'welcummm': 'welcome', 'embarsssed': 'embarrassed', 'buuuuutttt': 'but','looovvve': 'love', 'weee': 'we', 'ooohhhh': 'oh', 'unveilingthebusinessstories': 'unveilingthebusinestories',  'guuuuhyperlink': 'hyperlink','riiising': 'rising', 'biittttccchhheeeessss': 'bitch', 'whewwwwwwwwww': 'whew', 'whaaatttt': 'what', 'stilll': 'still', 'huuuuge': 'huge', 'suuuper': 'super', 'alllllllll': 'all', 'mannnn': 'man', 'pffft': 'pft', 'reallly': 'really', 'whuuuuuuh': 'whuh', 'suckssss': 'sucks', 'awonderfullly': 'wonderfully', 'staaaaaarve': 'starve', 'cuuute': 'cute', 'ayyyyyyy': 'ay', 'yaaassssszzzzz': 'yes', 'goooees': 'goes','toooooo': 'to', 'bleeeeh': 'bleh', 'harrrrrd': 'hard', 'maaaaad': 'mad', 'lazyyyyyy': 'lazy', 'uuuuummm': 'umm','bweakfaaaaaaaast': 'bweakfast', 'againnnnnn': 'again', 'hrmmmm': 'hrm', 'hooopefully': 'hopefuly', 'psssttt': 'pst','swearrrrr': 'swear', 'weeeeeek': 'week', 'brahhhh': 'brah', 'checkkkk': 'check','everrrrr': 'ever', 'annnnnnnd': 'and', 'suuuucks': 'sucks', 'atttorney': 'attorney', 'peanutbuttter': 'peanut butter', 'ayyy': 'ay',  'wtffff': 'wtf', 'tttthhhhhiiissssss': 'this', 'hateee': 'hate', 'savedddd': 'saved','squeeeee': 'squee',  'oooouuuu': 'ou', 'naaaaa': 'na', 'weeee': 'we', 'yessss': 'yes', 'yeaaaa': 'yes', 'homeacccess': 'home access', 'timberrr':'timber', 'tahdayyy': 'today', 'yeees': 'yes', 'stayinnn': 'staying', 'aliveee': 'alive', 'heeeeeere': 'here', 'gr000oceries': 'groceries', 'taaaagetherrr': 'together', 'appp': 'app', 'sighhh': 'sigh', 'wooohoo': 'woohoo', 'winterrrr': 'winter', 'friiiendssss': 'friends', 'ooooof': 'of', 'yuuup': 'yes', 'everythinggg': 'everything', 'thankkk': 'thank', 'youuu': 'you', 'aaaannndddd': 'and', 'babaaaay': 'baby', 'ayeee': 'aye', 'babiesss': 'babies', 'yaaas': 'yes', 'coinnn': 'coin', 'nevvvaaaa': 'never','aaaack': 'ack', 'mmmlove': 'love', 'dayyyy': 'day', 'cribmusss': 'cribmus', 'employeeengagement': 'employee engagement', 'diseaseeee': 'disease', 'beyonddd': 'beyond', 'jeeeeesus': 'jesus', 'coffeeeeee': 'coffee', 'wwasss': 'was', 'alriiight': 'alright', 'tieeeeeeed': 'tied','hellooooo': 'hello', 'reeeaaalllll': 'real', 'whoooo': 'who', 'hulll': 'hull', 'whhyyyyy': 'why', 'oooooold': 'old','orrrr': 'or', 'weeeek': 'week', 'dayyyyy': 'day', 'ohhhyeaaahhh': 'oh yeah', 'helloooo': 'hello', 'yallll': 'you all','agghhhhhhhhhhhhhhhhhhhhhhhhhhh': 'argh', 'agghhhhhhhhhhhhhhhhhhhhhhhhhhhhh': 'argh', 'finallly': 'finally','adressses': 'addresses', 'reallllly': 'really', 'omggg': 'omg', 'yaaaaa': 'ya', 'laggg': 'lag', 'helllooooee': 'hello', 'hellllooooee': 'hello', 'hungryyy': 'hungry', 'apppayments': 'apayments', 'littt': 'lit', 'suuuuuuuuper': 'super', 'feeees': 'fees', 'bihhhh': 'bih', 'gawddd': 'gawd', 'allllll': 'all', 'deeeply': 'deeply', 'fullfilllment': 'fulfilment', 'pfff': 'pf', 'weirdddd': 'weird', 'nowwwwwww': 'now', 'heyyyy': 'hey','luuuuucky': 'lucky', 'haaand': 'hand', 'nowwwww': 'now',  'heeelp': 'help', 'shppper': 'shopper', 'mooost': 'most', 'fooods': 'foods', 'girlllll': 'girl','whatttt': 'what', 'factttt': 'fact', 'yaaaassssss': 'yes', 'worrry': 'worry', 'buuutt': 'but', 'andddd': 'and','groooound': 'ground', 'commmunication': 'communication', 'biggg': 'big', 'geeeezus': 'jeesus',              'woooo': 'woo', 'paaaranoid': 'paranoid', 'wuaaaaaaaaaaaa': 'wua', 'yerrrrrr': 'yer', 'craaaap': 'crap','weeeeeed': 'weed', 'daaaays': 'days', 'runnning': 'running', 'eeech': 'ech', 'everrr': 'ever', 'quinnnnnnn': 'quin','yummm': 'yum', 'goood': 'god',  'okaaaaay': 'okay', 'weeeeeeeee': 'we', 'chilll': 'chill', 'yayyy': 'yay', 'realllly': 'really', 'oooof': 'of', 'wayyyyy': 'way', 'booooo': 'boo', 'yyyyeesssssssss': 'yes', 'stafff': 'staff', 'ooooooookay': 'okay', 'suppposed': 'supposed','thanksssss': 'thanks', 'boyyyyy': 'boy', 'ugguhughughghhhh': 'ugh', 'reaalllyy': 'really', 'tyyy': 'ty','yesss': 'yes', 'weeeird': 'weird', 'wheeeeeeee': 'whee', 'aieee': 'aie', 'pissssssed': 'pissed', 'anddddd': 'and','errrr': 'err', 'freeeeeee': 'free', 'dayzzzz': 'days', 'killerrrr': 'killer', 'piiised': 'pissed', 'wooooop': 'woop', 'localbusinessservices': 'local business services', 'happyhmmm': 'happy','housssseeee': 'house', 'gaaaaah': 'gah', 'sbappp': 'sbap', 'scurrred': 'scurred', 'messsages': 'messages', 'whaaaat': 'what', 'soooon': 'soon', 'theee': 'the', 'villlage': 'village', 'woooow': 'wow','chiliiii': 'chili', 'uhhmmmm': 'uhm', 'itwonderfullly': 'it wonderfully','rulllessssss': 'rules', 'aaaannnnnd': 'and', 'ewwwhyperlink': 'hyperlink', 'sammeee': 'same', 'geeez': 'geez', 'wayyyyyyy': 'way', 'eatwelllivewell': 'eat well live well', 'tooo': 'too', 'nopeeee': 'nope', 'aswonderfullly': 'as wonderfully', 'aaaaaaaaallllll': 'all', 'resuppply': 'resupply', 'yeahhhhh': 'yeah', 'lesss': 'less', 'innnnn': 'in','shppping': 'shipping', 'yeaaaah': 'yeah', 'whyyyyy': 'why','ooops': 'oops', 'neoowww': 'neow', 'seeems': 'seems', 'cluuuuuuutch': 'clutch', 'hellllll': 'hell', 'yaaaaaa': 'ya', 'aaahh': 'aah', 'byyyyyeeeeee': 'bye', 'pleaseeeeeee': 'please', 'maaaaan': 'man','lollll': 'lol', 'caaaat': 'cat', 'foood': 'food', 'faaaaaave': 'favourite', 'knowwww': 'know', 'weeeks': 'weeks', 'yaaa': 'ya','reeeeaaaalllly': 'really', 'themmm': 'them', 'gooooooood': 'good','ssshhhh': 'shh', 'waaaawaaaaawaaaaa': 'wow', 'businessschools': 'business schools', 'locallly': 'localy', 'boooooy': 'boy', 'plzzzzz': 'plz', 'realllyyyy': 'really', 'lmdaoooo': 'lmdao', 'daaaaaam': 'damn', 'woohooo': 'woohoo', 'whaattt': 'what','truuuuu': 'true', 'whaaaaaaat': 'what', 'meeee': 'me', 'reeeeeallly': 'really', 'ffffuuuuccccckkkkkkkkkkk': 'fuck', 'winnnning': 'winning', 'pppl': 'ppl', 'siiiigh': 'sigh', 'thurrr': 'thur', 'hopefullly': 'hopefully', 'lettting': 'letting', 'reaaaaaaally': 'really', 'buisnesssupport': 'buisness support', 'bombbbb': 'bomb','wuuut': 'wut', 'fuuu': 'fuu', 'trafffic': 'traffic', 'butttt': 'but', 'coffeee': 'coffee', 'maaamm': 'mam', 'reeeaally': 'really', 'gooo': 'go', 'sammeeee': 'same', 'swoooooon': 'swon', 'todaaaaaaay': 'today', 'nccc': 'nc','uggg': 'ugh', 'hellllllp': 'help', 'grrrrrrr': 'grr', 'shoooot': 'shot', 'fffuzzy': 'fuzzy', 'gettting': 'getting','telll': 'tell', 'maaaaybe': 'may be', 'duuuuude': 'dude', 'dooooo': 'do', 'weyyy': 'wey', 'boycotttrumppressconferences': 'boycotrumpresconferences','nowww': 'now', 'hooraaaay': 'hooray',  'woohoooo': 'woohoo', 'hhhmmm': 'hmm', 'yummmmmm': 'yum', 'wellllllll': 'well', 'woahhh': 'whoa', 'buuuuusssssinesssss': 'business','muuuumm': 'mum', 'girlll': 'girl', 'uuuuugh': 'ugh', 'thiiis': 'this', 'wayyyy': 'way', 'ateeee': 'ate', 'ohhhhhh': 'oh', 'ahhhhhhhhhhh': 'aah','saaaaammmmeee': 'same', 'saaammmmeee': 'same', 'infinittyyyyyy': 'infinity','welppp': 'welp', 'hellooooooo': 'hello', 'tryinggg': 'trying', 'roaaaaarsome': 'roarsome', 'cooo': 'co', 'oookies': 'okay','ittttttt': 'it', 'willl': 'will', 'neededdddddd': 'needed', 'haaay': 'hay', 'daysss': 'days', 'hahahhhahah': 'haha','oooph': 'oph', 'youlllive': 'you will live', 'defffff': 'def', 'bitttttt': 'bit', 'ideaaaa': 'idea', 'waitttt': 'wait','dangggggggggg': 'dang', 'lmaooimfatboyandwillstarveooooooooooooooooooo': 'lmao i am fat boy and wil starve', 'aaaannnnnndddddd': 'and', 'cevicheee': 'ceviche', 'tostadassss': 'tostadas', 'boooooooox': 'box', 'yaaaay': 'yay', 'guyssss': 'guys', 'aaaassssss': 'as', 'allllllaaaat': 'alat', 'waaaaayyyyy': 'way', 'coooooookies': 'cookies', 'folksss': 'folks', 'meee': 'me', 'yuuuup': 'yup', 'daaark': 'dark', 'sorryyy': 'sorry', 'concepttttt': 'concept', 'booosted': 'boosted','threeeee': 'three', 'grrrrr': 'grr','starrrrrrrrrrving': 'starving',  'baybayyyy': 'baby', 'aaaaafffffff': 'aaf', 'maxxxx': 'max', 'waaaaaaaah': 'wah', 'arghhh': 'argh',  'flosssssss': 'floss',  'wayyy': 'way', 'burgesss': 'burges', 'wahhh': 'wah', 'maaam': 'mam', 'brrrr': 'brr', 'yippeeeee': 'yippee', 'loool': 'lol', 'ooohh': 'oh', 'loooooool': 'lol', 'donneeee': 'done', 'ewwww': 'eww', 'everrrrrr': 'ever', 'looot': 'lot', 'deliverrrry': 'delivery', 'isss': 'is', 'orrrrrr': 'or', 'duuuude': 'dude', 'ftwwwwww': 'ftw', 'hallllp': 'help', 'fuuuuck': 'fuck', 'yaaaaas': 'yes','businessscience': 'business science','myyy': 'my', 'successstory': 'sucess story', 'fffffffuck': 'fuck', 'thisssssss': 'this', 'ahhhhhhh': 'aah', 'yaaaaasss': 'yes', 'tthhhe': 'the', 'donttouchmeee': 'dont touch me',  'jeeeeeze': 'jeeze', 'suuuuch': 'such', 'twattttt': 'twat', 'tomorrrow': 'tomorrow', 'aaalllll': 'all','gooooaaaaaalll': 'goal', 'heeeeeeere': 'here', 'weeeeeee': 'we', 'gooooo': 'go', 'firrre': 'fire', 'chainssss': 'chains', 'whhhaaatttt': 'what', 'yaaasss': 'yes', 'notttt': 'not', 'haaaaaaaate': 'hate','hungeeeee': 'huge', 'waaaaa': 'wah', 'wheeeeee': 'whee', 'milllennials': 'millennial', 'aghhh': 'agh', 'errrrr': 'err', 'bestttt': 'best', 'buuuuuut': 'but', 'hmmmph': 'hmph', 'maaaaaaaan': 'man', 'buuuuut': 'but', 'guuuuurrrllll': 'girl', 'heyyy': 'hey', 'ummmmmmm': 'umm', 
                'yassssss': 'yes', 'fuckkk': 'fuck', 'arhhh': 'arh', 'hooo': 'ho', 'itttttt': 'it', 'boooo': 'boo', 'zzzzz': 'z', 
                'nahhhh': 'nah','guyssss':'guys','damnnnn':'damn', 'grrr':'grr','grrrr':'grr','grrrrrrrr': 'grr', 'mmmaybe': 'may be', 'deeeeets': 'details', 'waaaaaay': 'way', 'knowwwwww': 'know', 'hollaaaaaaa': 'hola', 'exxxtra': 'extra', 'neeeed': 'need', 'yaaaassss': 'yas', 'seamlessshopping': 'seamleshoping','plsss': 'please', 'hnnnnngggggg': 'hang', 'retweeet': 'retweet', 'retweeetplease': 'retweet please', 'tyyyyy': 'ty','yaaaaassss': 'yes', 'aahhh': 'aah','yupppp': 'yup', 'damnnnnnnn': 'damn','successstories': 'success stories', 'nnnreal': 'unreal', 'reggggular': 'regular', 'screeeam': 'scream', 'stahhpppp': 'stop', 'girrrrllllllllll': 'girl', 'mooooooooooon': 'moon', 'ouuuuuuuchhhh': 'ouch', 'yoooo': 'yo', 'lessstress': 'less stress', 'positivevibezzz': 'positive vibes', 'oookkkkayyy': 'okay', 'breeezyy': 'breezy', 'wisssshh': 'wish', 'foooood': 'food', 'oooooooookay': 'okay','thaaanks': 'thanks', 'loook': 'look', 'paaaarrrrrty': 'party','hollaaaa': 'hola', 'suuuuuper': 'super', 'hateeee': 'hate', 'gaaaaahhhhhh': 'gah','callling': 'calling','honestbeeeeee': 'honestbee', 'hoooonest': 'honest', 'neeeeeeeeeever': 'never', 'ricccch': 'rich', 'juuuuuuust': 'just', 'whaaattt': 'what', 'whyyyyyy': 'why','happyxxx': 'happy','ayyye': 'aye', 'hardddd': 'hard', 'pleeeeeeease': 'please','suuuuucks': 'sucks', 'shhhhh': 'shh', 'naaaaaaaaah': 'nah', 'heeey': 'hey', 'yooooo': 'yo','wooooooooorse': 'worse', 'ittttt': 'it', 'buuut': 'but', 'succkssss': 'sucks', 'slowwwwlyyyy': 'slowly','yaaassss': 'yes', 'annnnnd': 'and', 'herrreeee': 'here', 'yeaaaaaah': 'yeah', 'murderrrr': 'murder','yeaaaaaaaaaah': 'yeah', 'aaaany': 'any', 'uppppp': 'up', 'fuuun': 'fun', 'tadaaa': 'tada', 
                'hufffff': 'huff', 'juuust': 'just','finnne': 'fine','errr': 'err', 'odddeliverydrivercomplaints': 'odd delivery driver complaints', 'eeek': 'eek', 'girrrrrl': 'girl', 'hmmmmmmm': 'hmm', 'onlyyyyyyy': 'only', 'guysssssss': 'guys','lotttttt': 'lot', 'ohhhhh': 'oh', 'ruuuns': 'runs', 'weeeekend': 'weekend', 'shoppping': 'shopping', 'verrry': 'very', 'fooooooood': 'food', 'tooooooo': 'too', 'reeeeally': 'really','zoooo': 'zoo', 'uhhhh': 'ugh', 'sorrryyyy': 'sorry', 'ssshhh': 'shh', 'embarrassmentummm': 'embarassment', 'plssssssss': 'please', 'todayyyy': 'today', 'neverrrr': 'never', 'craaazy': 'crazy', 'annnd': 'and', 'boooomb': 'bomb','aliiive': 'alive', 'ecologicallly': 'ecologicaly', 'nooo': 'no', 'waaaait': 'wait', 'tooooo': 'too', 'ohhhhhhh': 'oh', 'whyyyy': 'why', 'pleeaaaassseeee': 'please', 'fuuuudge': 'fudge','shiddddd': 'shid', 'biiig': 'big', 'yooooooo': 'yo', 'coldddddddddd': 'cold', 'mighttttt': 'might','freeeee': 'free', 'pleaseeeeee': 'please', 'helll': 'hell','mmkkayyy': 'i am okay', 'reeeaaalll': 'real','bxtchhhhhhh': 'bitch', 'eeeeek': 'eek', 'ooooowweeeeee': 'owe','toooooooo': 'too', 'retweee': 'retweet', 'daaayyyysss': 'days', 'hmmmmmmmmmmm': 'hmm', 'heeeeere': 'here', 'fineeee': 'fine', 'naurrr': 'naur', 'liviiddddd': 'livid', 'looool': 'lol', 'stokedddd': 'stoked', 'pleeeease': 'please', 'arghhhhhh': 'argh', 'especiallly': 'especialy', 'supppper': 'super', 'ummmmmm': 'umm', 'seeee': 'see', 'wtfff': 'wtf',  'slowwwwwservice': 'slow service', 'suuuuuuper': 'super', 'whhhaaaaaaaa': 'whaa', 'soooers': 'soers','eeeeeeeeeekkkkkkkk': 'eek', 'don888': 'don8', 'extreeeemly': 'extremely','saaame': 'same', 'booooy': 'boy', 'ooof': 'of', 'bitchhh': 'bitch', 'babbieeeee': 'baby', 'refuseeee': 'refuse', 'jooooooooooke': 'joke', 'juskooo': 'jusko', 'vaxxxed': 'vaxed', 'loooow': 'low', 'slowwwww': 'slow', 'waaaayyyyy': 'way','businesssuccess': 'business success', 'naaaaaaa': 'na', 'goooooooooooooo': 'go', 'fuuuuuuuck': 'fuck', 'shoppinggg': 'shopping', 'bihhh': 'bih', 'pshhh': 'psh', 'saaaaaame': 'same', 'faaaaaaccckkk': 'fuck', 'employeeengagment': 'employee engagement', 'ooop': 'oops', 'sighhhhhhhhhhhhhhhhh': 'sigh', 'howwwww': 'how', 'ooooold': 'old', 'ghhhhggh': 'ghgh', 'shitttt': 'shit', 'tfff': 'tf', 'saaaaame': 'same', 'whaaaaatttt': 'what', 'yeeeeaah': 'yeah', 'seeet': 'set', 'yeee': 'ye', 'uuuugggg': 'ugh',  'pleaseeeee': 'please', 'tipzzzzz': 'tips', 'aarrrrrrrrghhhhhhh': 'argh', 'foreseennnnn': 'foreseen', 'yooouuu': 'you', 'ayyyyyy': 'ay', 'sucksss': 'sucks', 'screeeeeams': 'screams',  'mamoy': 'mamoy',  'whoooooof': 'whoof', 'chillle': 'chile', 'buttttt': 'but', 'actuallly': 'actually', 'wooot': 'woot', 'businessstandard': 'business standard', 'idcccc': 'idc', 'fulllllll': 'full', 'seeeeesh': 'sesh', 'handddd': 'hand', 'smhhh': 'smh', 'sodaaaa': 'soda', 'smallbusinesssupport': 'small busines support','nervessssssss': 'nerves', 'snowww': 'snow', 'ssssssssssoooo': 'so', 'biiit': 'bit', 'wooooooooooo': 'woo', 'swwwwing': 'swing', 'shittty': 'shity', 'yyyy': 'y', 'maaaann': 'man', 'tippppsss': 'tips', 'neeeeeeeeeeeds': 'needs', 'shoppinggggg': 'shopping', 'arghhhhh': 'argh', 'listennn': 'listen', 'neeeever': 'never', 'eeeep': 'eep', 'tooold': 'told', 'bahhh': 'bah', 'sneeeeeeezers': 'sneezers', 'lovexxx': 'love', 'yeeepppp': 'yep', 'aaamazzzing': 'amazing', 'fuuuuuuuuuuuuck': 'fuck','omfggg': 'omfg', 'monthsssss': 'months', 'maaaaaaaaay': 'may', 'arrrrghhh': 'argh', 'dyinggggg': 'dying', 'muuuuuuuuch': 'much', 'wheeew': 'whew','springgggg': 'spring', 'cmonnnnnn': 'cmon', 'thaaat': 'that', 'jeeeez': 'jeez', 'nawww': 'naw', 'shhhh': 'shh', 'waaaaahhh': 'wah', 'thatttt': 'that', 'clutchhhhhhhh': 'clutch', 'squeeee': 'squee', 'maaaaaan': 'man', 'eatttt': 'eat', 'woweee': 'wowe', 'plzzzz': 'plz', 'babyyyyy': 'baby', 'crucialllll': 'crucial', 'beekeeeper': 'beekeeper', 'beeee': 'be', 'morningggg': 'morning', 'stopppppo': 'stop', 'itsalllocal': 'its a local', 'hmrsss': 'hmrs','successsee': 'success','burrrr': 'burr', 'wheeee': 'whee', 'anyoneeee': 'anyone', 'annndd': 'and', 'businesss': 'business', 
                'incrennnible': 'incredible','lmaooo':'lmao', 'helloooo':'hello','gaaaah': 'gah', 'blowssssssss': 'blows', 'veeeeeggggg': 'veg', 'thiissss': 'this', 'eccc': 'ecc', 'seee': 'see','nahhhhhh': 'nah', 'caaaaasually': 'casually', 'thissss': 'this', 'wellllll': 'well', 'sayinnggg': 'saying', 'dinnerrrr': 'dinner', 'ughhh': 'ugh', 'coffeeeee': 'coffee', 'apppreciate': 'appreciate', 'fooddddddd': 'food', 'waaaa': 'waa', 'blacklivesmattters': 'black lives matters', 'pleaseeeeeeeee': 'please', 'haaaa': 'haa','bagsss': 'bags', 'aiyaaa': 'aiya', 'sixxxxxx': 'six', 'weeeeekssssss': 'weeks', 'aaaaaaaaaaaaaaahhhhhh': 'aah','suuure': 'sure', 'youuuu': 'you', 'freestufffriday': 'free stuff friday', 'bammmmmm': 'bam', 'ewww': 'eew', 'sisss': 'sis', 'yeesss': 'yes', 'sweeeeeeeet': 'sweet', 'girllll': 'girl', 'suuuuck': 'suck', 'wooooooah': 'woah', 'sayweee': 'say we', 'yoooooo': 'yo', 'hellllla': 'hell', 'eeegads': 'egad', 'lmfaaooo': 'lmfao', 'hhhhhg': 'hg', 'fooooooodddd': 'food', 'fooooood': 'food', 'tttil': 'till', 'sayyyy': 'say', 'booooooo': 'boo','faaaar': 'far', 'peopleeeeeeee': 'people', 'auuugh': 'ugh', 'fuuuck': 'fuck', 'sirrr': 'sir', 'plzzz': 'plz', 'yummmm': 'yum', 'sleep': 'sleep', 'alsooo': 'also', 'reallllllly': 'really', 'nahhhhhhting': 'nothing', 'wayyyyyyyy': 'way', 'wheeeeeeeeee': 'whee', 'niiice': 'nice', 'shoooo': 'shoo', 'yeaaah': 'yeah', 'bwahhhahaah': 'haha', 'hahaaaa': 'haha', 'ordererrr': 'orderer', 'behhh': 'beh', 'plsssss': 'pls', 'neeed': 'ned', 'pffffftt': 'pft', 'choooooosing': 'chosing', 'byeeeeeeee': 'bye', 'welllll': 'well', 'naaaame': 'name', 'strugggllleeeee': 'struggle', 'yeahhhh': 'yeah', 'waayyy': 'way', 'aaaw': 'aww', 'yummmmmmmmmmmm': 'yum', 'nahhh': 'nah', 'silaaaaaa': 'sila', 'aaaaahhhhh': 'aah', 'bayyyy': 'bay', 
                'saaammee': 'same','alll':'all', 'grrr':'grr','yummm':'yummy','yeees':'yes','yasss':'yes','yassss':'yes','whyyy':'why','whaaat':'what','lolll':'lol','weee':'we', 'weeee':'we','tooo':'too','tyyy':'ty','uuugh':'ugh', 'sooo':'so','suuuuper': 'super', 'whyyyyyyy': 'why', 'reaaaaaally': 'really', 'fuccc': 'fuck', 'neverrr': 'never', 'ffff': 'f', 'maaan': 'man', 'asssist': 'assist', 'uuuuu': 'u', 'foreverrrrrer': 'forever',  'aaaaaah': 'aah', 'hellooooooooooo': 'hello', 'mmmmmmmrrrrrrrrrrrr': 'mr',  'mmmrrrrrrrrrrooooooooooorrrm': 'mrorm', 'feee': 'fee', 'freee': 'fre', 'stressss': 'stress', 'yeeeeee': 'yee', 'deeeeerrrrrp': 'derp', 'wooohoooo': 'woohoo', 'mehhhh': 'meh', 'bahhhh': 'bah', 'chillinnnn': 'chilling', 'whattt': 'what', 'lmaooooo': 'lmao', 'daaaaamn': 'damn', 'lolllll': 'lol','psssssst': 'pst', 'carrrrrr': 'car', 'paaaain': 'pain', 'reeeeeee': 're', 'madeeeee': 'made', 'oooow': 'oaw', 'wooooo': 'woo', 'ahhhhhhhhhgg': 'ahg',  'blehhhhh': 'bleh', 'teeeeeny': 'tiny', 'carbzzzz': 'carbz', 'efffing': 'effing', 'kakraaa': 'kakra', 'hoooooo': 'ho', 'jeeeeez': 'jeez', 'hahaaa': 'haha', 'suuuuuuuuure': 'sure', 'hummm': 'hum', 'myoooosive': 'myosive', 'prrrrobably': 'probably', 'waiiit': 'wait','fuuuuuuucked': 'fucked', 'thanksgiviiing': 'thanksgiving', 'waaaant': 'want', 'girrrlll': 'girl', 'gaaaame': 'game', 'whewww': 'whew', 'awwwwkward': 'awkward', 'godddddd': 'god', 'whoooooo': 'who', 'dopppeee': 'dope', 'looolll': 'lol', 'ammmmiwrongggggg': 'am i wrong', 'weeek': 'week', 'yeeeeahhh': 'yeah', 'knowww': 'know', 'nottt': 'not', '4evaaaa': '4eva', 'wiiild': 'wild', 'wooo': 'woo', 'yummmmmmmm': 'yum','aaaall': 'all', 'herreeeee': 'here', 'seasonnnnnn': 'season', 'ummmmmmmm': 'umm', 'goootcha': 'gotcha', 'oooops': 'oops', 'yeeeees': 'yes', 'sheeeeeeeee': 'she', 'mmmfph': 'mfph', 'everrrr': 'ever', 'stufff': 'stuf', 'fkkkk': 'fuck', 'eliteeeeeeee': 'elite', 'maybeeee': 'maybe', 'brrrrr': 'brr', 'yyeesss': 'yes', 'waaaayyy': 'way', 'reaallly': 'really', 'lmaaao': 'lmao', 'eyyy': 'eyy', 'givingggg': 'giving', 'toooo': 'too', 'hiii': 'hi', 'yeeeeeeeees': 'yes', 'arrrgh': 'argh', 'fuuuccccckkk': 'fuck', 'covidvacccine': 'covid vaccine', 'mmmay': 'may', 'wellll': 'well', 'yeeeeesss': 'yes', 'laaaaa': 'la', 'sameeee': 'same', 'goooood': 'good', 'afternoonnnn': 'afternoon', 'fffb': 'fb', 'aaaccedent': 'accident', 'clutcchhhhh': 'clutch', 'whennnnnn': 'when', 'reeeeeeeally': 'really', 'rigghhhttttt': 'right', 'ahhhhhhhh': 'aah','walkkk': 'walk', 'phewwww': 'phew', 'overrr': 'over', 'fuuuuuuuuck': 'fuck', 'aaaah': 'aah', 'classsss': 'class', 'maaaaaybe': 'may be', 'daaaamn': 'damn', 'businessses': 'busineses', 'hooooo': 'hoo', 'whooo': 'who', 'onlineshoppping': 'online shopping','hiswonderfullly': 'his wonderfully', 'paymisseee': 'paymise', 'hoooo': 'ho', 'orrr': 'or', 'waaaaaaah': 'wah', 'aaahhhhhh': 'aah', 'caaalllllll': 'call', 'reeeally': 'really', 'ooookay': 'okay', 'juuuuuuuuuuust': 'just', 'earlyyyy': 'early', 'fuuuccckkk': 'fuck', 'caaaant': 'cannot', 'maaaannnnnn': 'man', 'uuuuuuuu': 'u', 'boooring': 'boring', 'downnnn': 'down', 'cuteee': 'cute', 'gaaaarooooossssssss': 'garos',  'buttttttttt': 'but', 'racccisssttt': 'racist', 'baaaaaaby': 'baby', 'orrrrrrr': 'or', 'reallyyy': 'really', 'aaaanndd': 'and', 'oohhhh': 'oh','anddd': 'and', 'happychildlessspinster': 'happy childless spinster', 'goooooo': 'go', 'baaaack': 'back','succckkks': 'sucks','whaaaaaaaat': 'what', 'thruuuuuuuuuuu': 'thru', 'whooot': 'whot', 'haaaaaaaaate': 'hate', 'nervousssss': 'nervous', 'heeeeeeeelpppppppppp': 'help', 'mannnnnn': 'man', 'oooooooook': 'ok', 'yussss': 'yes', 'bigggggg': 'big', 'igrrr': 'igr', 'botttom': 'bottom', 'muuurrr': 'mur', 'suuucks': 'sucks','juuuust': 'just', 'awwwhyperlink': 'hyperlink', 'speeeedy': 'speedy', 'uuuf': 'uff', 'crazzzzyyyy': 'crazy', 'aaahhhh': 'aah', 'businesssolutions': 'business solutions', 'teeesst': 'test', 'yeeeah': 'yeah', 'gimmme': 'give me', 'thewonderfullly': 'the wonderfully', 'ffffffuck': 'fuck', 'baybeeeee': 'baby', 'backkk': 'back', 'sweetttttt': 'sweet', 'myhhhh': 'myh', 'babyyyy': 'baby', 'todayyyyyy': 'today', 'laaaa': 'la', 'fulllockdown': 'full lockdown', 'goddddd': 'god', 'maaaannn': 'man', 'uuuugh': 'ugh', 'tierrrrrrrrr': 'tier', 'sleeeeeping': 'sleeping', 'sunnnybeemarket': 'sunybee market', 'eeevvveeerrr': 'ever', 'haaarddddddd': 'hard', 'saaaame': 'same', 'pffffizer': 'pfizer', 'mmmmicrobes': 'microbes', 'byeeee': 'bye', 'founddd': 'found', 'serviceeeee': 'service', 'wantttt': 'want', 'buyyyy': 'buy', 'meaaaaaaat': 'meat', 'mmmh': 'mmh', 'wahhhhh': 'wah', 'heeeere': 'here', 'thankssss': 'thanks', 'hummmer': 'humer', 'yeeeeeeeeeaaaaaahhhhhh': 'yeah', 'unhappyblesssss': 'unhappy bless', 'dooor': 'door', 'welll': 'well', 'focuuussssss': 'focus',  'hisss': 'his', 'babyyy': 'baby', 'whateverrrr': 'whatever', 'weeeeee': 'we', 'bzzzzzzz': 'bz', 'naaaa': 'na', 'cosssssss': 'coss','lifeeee': 'life', 'businesssolution': 'business solution', 'pleaseeeeeeee': 'please','yuuum': 'yum', 'buttttttttttt': 'but', 'pissedddd': 'pissed', 'geeesh': 'geesh', 'helloooooo': 'hello', 'fineeeee': 'fine','alllllll': 'all', 'againnnn': 'again', 'muuuuch': 'much', 'ewwwww': 'eww', 'buuuunch': 'bunch', 'freeaaaakk': 'freak', 'aaauugh': 'ugh', 'aaaaahh': 'aah', 'aaaaaaaahhh': 'aah', 'waaaayyyyyy': 'way','hereee': 'here', 'shhh': 'shh', 'gooood': 'good', 'todayyy': 'today', 'outttt': 'out', 'nervesssss': 'nerves', 'yuuuuup': 'yup', 'sheeee': 'she','buildiiiiiing': 'building', 'bankkkkk': 'bank', 'usssa': 'usa', 'weeeee': 'we', 'nohhh': 'noh','bessst': 'best', 'ughnnn': 'ugh', 'worlddddddddddd': 'world', 'yeeeaaaaaahhh': 'yeah', 'whewwww': 'whew', 'suuuuuuucks': 'sucks', 'uhhhm': 'uhm', 'wuuuuu': 'wu', 'aaaaayyyy': 'ayy','10000x': '10x', 'perioddddd': 'period', 'alsoooo': 'also', 'thiccc': 'thick', 'lezzgoooo': 'let go', 'ighttttt': 'ight', 'giiirl': 'girl', 'd7000': 'd70', 'xxxl': 'xl', 'q17775306': 'q175306','hhhhhhhhhh': 'h', '$mmmb': '$mb', 'dxxx': 'dx','000$': '0$','000pm': '0pm', '1119th': '19th', '1000k': '10k', 'appph': 'aph', '$misssadiemaeve': '$misadiemaeve','x10000': 'x10', '0xf8da8ecee025444ad8dcf065ae09cffcd55c6116': '0xf8da8ece0254ad8dcf065ae09cfcd5c616'
                   }
len(replace_list)
                    

1274

In [61]:
len(replace_list.keys())

1274

In [62]:
total_count = 0
rep_words = []
for i in tqdm(range(len(data))):
    count = 0
    words = data['tweet'][i].split(' ')
    for word in words:
        if word in replace_list:
            count += 1
            rep_words.append(word)
    total_count += count
total_count   

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 225777.44it/s]


3265

In [63]:
for i in tqdm(replace_list):
    data.loc[:,'tweet'] = data.tweet.str.replace(i,replace_list[i])

 99%|██████████████████████████████████████▋| 1264/1274 [02:54<00:01,  7.53it/s]/var/folders/2k/fhkt65ls68nbd5xt0lj3sbww0000gn/T/ipykernel_59839/2053584832.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.loc[:,'tweet'] = data.tweet.str.replace(i,replace_list[i])
100%|███████████████████████████████████████| 1274/1274 [02:55<00:00,  7.24it/s]


In [64]:
print(count_link(data), count_i(data), count_my(data))

195618 218051 87709


### Replace slangs

In [65]:
slangs = {}
with open('../../Supplementary data/slangs.txt', 'r') as f:
    for i in f:
        for j in i.split(','):
            j = j.replace('{','')
            j = j.replace('}','')
            j = j.replace('\"','')
            a = j.split(':')[0].strip()
            b = j.split(':')[1].strip()
            slangs[a] = b
len(slangs)

167

In [66]:
count = 0
slang = []
for i in tqdm(range(len(data))):
    words = data['tweet'][i].split(' ')
    for word in words:
        if word in slangs:
            count += 1
            slang.append(word)
count

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 220657.58it/s]


15992

In [67]:
def replace_slangs(text):
    return ' '.join([slangs.get(i.lower(),i.lower()) for i in text.split()])
data['tweet'] = data['tweet'].apply(lambda x : replace_slangs(x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [68]:
print(count_link(data), count_i(data), count_my(data))

195618 218209 87739


### Find words representing coronavirus

In [69]:
def replace_corona(text):
    replace_list = {'coronavirus':'corona','covid19':'corona','covidiot':'corona idiot','covid':'corona'}
    words = text.split(' ')
    new_words = []
    for word in words:
        for i in replace_list:
            if i in word:
                word = word.replace(i, replace_list[i])
        new_words.append(word) 
    return ' '.join(new_words)
        
    

In [70]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x : replace_corona(x))

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 271052.16it/s]


In [71]:
words = ' '.join(data['tweet']).split()
dic_corona = {}
for word in words:
    if 'corona' in word:
        if word in dic_corona:
            dic_corona[word] += 1
        else:
            dic_corona[word] = 1

In [72]:
new_words = dic_corona.keys()

In [73]:
import wordninja
for word in new_words:
    print(word, wordninja.split(word))
    

corona ['corona']
coronas ['coronas']
coronacanada ['corona', 'canada']
corona2019 ['corona', '2019']
nocorona ['no', 'corona']
coronalockdown ['corona', 'lockdown']
coronaactnow ['corona', 'act', 'now']
coronacalendar ['corona', 'calendar']
highriskcorona ['high', 'risk', 'corona']
coronataskforce ['corona', 'taskforce']
coronacrisis ['corona', 'crisis']
coronaoutbreak ['corona', 'outbreak']
coronatx ['corona', 'tx']
coronapocalypse ['cor', 'on', 'apocalypse']
coronaab ['corona', 'ab']
coronadining ['corona', 'dining']
coronachat ['corona', 'chat']
coronaetiquette ['corona', 'etiquette']
coronaquarentinedays ['corona', 'qu', 'arent', 'in', 'e', 'days']
seattlecorona ['seattle', 'corona']
coronalife ['corona', 'life']
coronabahamas ['corona', 'bahamas']
coronafacts ['corona', 'facts']
coronaidiots ['corona', 'idiots']
therescoronainsidethere ['there', 's', 'corona', 'inside', 'there']
floridacorona ['florida', 'corona']
coronacooties ['corona', 'cooties']
businessduringcorona ['busines

In [74]:
print(count_link(data), count_i(data), count_my(data))

195618 218209 87739


### Divide connected words

In [75]:
#Number of connected words
connect_words =[]
for i in tqdm(range(len(data))):
    words = data['tweet'][i].split(' ')
    for word in words:
        if len(word) >12 and word not in connect_words:
            connect_words.append(word)
len(connect_words)

100%|███████████████████████████████| 368413/368413 [00:03<00:00, 116644.05it/s]


15018

In [76]:
import wordninja
def split_words(text):
    words = text.split()
    new_words = []
    for word in words:
        if len(word) > 12:
            new_words.extend(wordninja.split(word))
        else:
            new_words.append(word)
    return ' '.join(new_words)

In [77]:
data['tweet'] = data['tweet'].progress_apply(lambda x:split_words(x))

100%|███████████████████████████████| 368413/368413 [00:03<00:00, 110322.54it/s]


In [78]:
print(count_link(data), count_i(data), count_my(data))

195868 218561 87929


### Remove words with single letter

In [79]:
#find number of words with only one letter
tweets = np.array(data['tweet'])
count = 0
word_list = []
for tweet in tweets:
    words = tweet.split()
    for word in words:
        if len(word) <= 1:
            word_list.append(word)
            count += 1
    
    
count

277156

In [80]:
def remove_single_words(text):
    single_words = ['o', 'd', 'r', 'k', 'e', 'y', 'n', 'b', 'c', 'h', 'a', 's', 'g', 'u', 'p', 'w', 'z', 'm', 'x', 'f',
                'j', 'v', 'l', 't', 'q']
    words = text.split()
    lst = []
    for word in words:
        if word not in single_words:
            lst.append(word)
    return ' '.join(lst)
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x : remove_single_words(x))
print(data['tweet'][5003])
print(data['tweet'][5006]) 

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 192666.18it/s]


my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [81]:
print(count_link(data), count_i(data), count_my(data))

195868 218561 87929


### Lemmatization

In [82]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk

In [83]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
tqdm.pandas()
data["tweet"] = data["tweet"].progress_apply(lambda text: lemmatize_words(text))
print(data['tweet'][5003])
print(data['tweet'][5006])

100%|█████████████████████████████████| 368413/368413 [03:10<00:00, 1935.42it/s]

my wife stop grocery store curbside pickup they put your orderer your trunk contact she she saw ton boomer go store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also worker watch hand curbside delivery


In [84]:
print(count_link(data), count_i(data), count_my(data))

195869 218569 87929


### Spelling correction

In [85]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [86]:
#Finding words with wrong spelling

In [87]:
words = ' '.join(data['tweet']).split()
misspelled_words = spell.unknown(words)

In [88]:
misspelled_words = list(set(misspelled_words))
len(misspelled_words)

55552

In [93]:
corrected_words = {}
for i in tqdm(misspelled_words):
    corrected = spell.correction(i)
    if corrected != None and i != corrected:
        corrected_words[i] = corrected

100%|███████████████████████████████████| 55552/55552 [1:04:12<00:00, 14.42it/s]


In [94]:
len(corrected_words)

35574

In [95]:
[x for x in corrected_words.items()]

[('shiraz', 'shirt'),
 ('bdb', 'bob'),
 ('gb', 'go'),
 ('pga', 'ga'),
 ('waskmain', 'walkman'),
 ('madision', 'madison'),
 ('invit', 'invite'),
 ('drumpf', 'dump'),
 ('avstrategy', 'strategy'),
 ('tmmr', 'time'),
 ('msnner', 'manner'),
 ('sonoma', 'sono'),
 ('deg', 'dog'),
 ('kville', 'ville'),
 ('reque', 'revue'),
 ('testi', 'test'),
 ('asts', 'ass'),
 ('jenks', 'jerks'),
 ('xennial', 'denial'),
 ('navle', 'navel'),
 ('ekong', 'kong'),
 ('jhaa', 'haa'),
 ('alissa', 'alyssa'),
 ('fecks', 'fucks'),
 ('epstien', 'epstein'),
 ('hceda', 'head'),
 ('pisgah', 'pish'),
 ('platforme', 'platform'),
 ('rau', 'ran'),
 ('charg', 'charge'),
 ('chunlan', 'shunkan'),
 ('stmla', 'still'),
 ('sumatra', 'sutra'),
 ('ciers', 'piers'),
 ('walpole', 'tadpole'),
 ('cic', 'vic'),
 ('lemmie', 'lemme'),
 ('yeees', 'yeses'),
 ('omgosh', 'gosh'),
 ('asbeza', 'cabeza'),
 ('frys', 'fry'),
 ('bmore', 'more'),
 ('betcher', 'butcher'),
 ('underc', 'under'),
 ('idecided', 'decided'),
 ('bago', 'ago'),
 ('copyiny', 'co

In [97]:
%pprint

Pretty printing has been turned OFF


In [98]:
print(corrected_words)

{'shiraz': 'shirt', 'bdb': 'bob', 'gb': 'go', 'pga': 'ga', 'waskmain': 'walkman', 'madision': 'madison', 'invit': 'invite', 'drumpf': 'dump', 'avstrategy': 'strategy', 'tmmr': 'time', 'msnner': 'manner', 'sonoma': 'sono', 'deg': 'dog', 'kville': 'ville', 'reque': 'revue', 'testi': 'test', 'asts': 'ass', 'jenks': 'jerks', 'xennial': 'denial', 'navle': 'navel', 'ekong': 'kong', 'jhaa': 'haa', 'alissa': 'alyssa', 'fecks': 'fucks', 'epstien': 'epstein', 'hceda': 'head', 'pisgah': 'pish', 'platforme': 'platform', 'rau': 'ran', 'charg': 'charge', 'chunlan': 'shunkan', 'stmla': 'still', 'sumatra': 'sutra', 'ciers': 'piers', 'walpole': 'tadpole', 'cic': 'vic', 'lemmie': 'lemme', 'yeees': 'yeses', 'omgosh': 'gosh', 'asbeza': 'cabeza', 'frys': 'fry', 'bmore': 'more', 'betcher': 'butcher', 'underc': 'under', 'idecided': 'decided', 'bago': 'ago', 'copyiny': 'copying', 'abaya': 'away', 'preesh': 'press', 'noothing': 'nothing', 'commentariat': 'commentaries', 'bathi': 'bath', 'solutioin': 'solution'

In [99]:
corrected = {'shiraz': 'shirt', 'bdb': 'bob', 'gb': 'go', 'pga': 'ga', 'waskmain': 'walkman', 'madision': 'madison', 'invit': 'invite', 'drumpf': 'dump', 'avstrategy': 'strategy', 'tmmr': 'time', 'msnner': 'manner', 'sonoma': 'sono', 'deg': 'dog', 'kville': 'ville', 'reque': 'revue', 'testi': 'test', 'asts': 'ass', 'jenks': 'jerks', 'xennial': 'denial', 'navle': 'navel', 'ekong': 'kong', 'jhaa': 'haa', 'alissa': 'alyssa', 'fecks': 'fucks', 'epstien': 'epstein', 'hceda': 'head', 'pisgah': 'pish', 'platforme': 'platform', 'rau': 'ran', 'charg': 'charge', 'chunlan': 'shunkan', 'stmla': 'still', 'sumatra': 'sutra', 'ciers': 'piers', 'walpole': 'tadpole', 'cic': 'vic', 'lemmie': 'lemme', 'yeees': 'yeses', 'omgosh': 'gosh', 'asbeza': 'cabeza', 'frys': 'fry', 'bmore': 'more', 'betcher': 'butcher', 'underc': 'under', 'idecided': 'decided', 'bago': 'ago', 'copyiny': 'copying', 'abaya': 'away', 'preesh': 'press', 'noothing': 'nothing', 'commentariat': 'commentaries', 'bathi': 'bath', 'solutioin': 'solution', 'blanky': 'blank', 'barcade': 'arcade', 'merrifield': 'terrified', 'flynance': 'finance', 'garison': 'garrison', 'conveyours': 'conveyors', 'pitsea': 'pissed', 'moscone': 'scone', '62bn': 'ibn', 'rwandans': 'ugandans', 'bapat': 'bapa', 'pelosi': 'pesos', 'altcoin': 'bitcoin', 'innovatory': 'innovator', 'delp': 'help', 'huel': 'fuel', 'tingz': 'ting', 'fitlife': 'midlife', 'palawan': 'padawan', 'fitbit': 'nitwit', 'insnow': 'snow', 'mccorder': 'recorder', 'wsgr': 'war', 'junctionto': 'junction', 'thebody': 'theory', 'charge$': 'charge', 'screeners': 'screener', 'scedule': 'schedule', 'nmo': 'no', 'joburg': 'jour', 'otw': 'tow', 'fretrade': 'retrace', 'ajaxon': 'jason', 'thaks': 'thanks', '17and': 'and', 'sedar': 'sedan', 'cmht': 'cut', 'khera': 'hera', 'myfucken': 'fucken', 'ional': 'tonal', 'teyana': 'tenant', 'sunpeaks': 'speaks', 'swachh': 'swath', 'kqed': 'ked', 'notsmart': 'outsmart', 'handgel': 'handel', 'jetaun': 'retain', 'bargin': 'bargain', '56th': 'with', 'hurraay': 'hurray', 'cald': 'call', 'hmry': 'hury', 'asrc': 'arc', '$mrna': 'mana', '34am': 'am', '$alny': 'any', 'fawk': 'hawk', 'hotwater': 'hotter', 'unsettlingly': 'unsettling', 'sewanee': 'swanee', 'pratik': 'prank', 'rase': 'case', 'jazelle': 'gazelle', 'delorean': 'delores', 'ibistro': 'bistro', 'dukaans': 'duncans', 'youself': 'yourself', 'hegemon': 'hegemony', 'avpd': 'avid', 'ywca': 'ymca', 'cowatch': 'watch', 'killen': 'killed', 'yalls': 'calls', 'wma': 'ma', 'uncheck': 'check', 'kiryana': 'biryani', 'dilivery': 'delivery', 'fk': 'ok', 'freecast': 'forecast', 'nicety': 'nicely', 'wdaf': 'what', 'cabarlis': 'cabarets', 'mundanity': 'mendacity', 'mampu': 'camp', 'asmaani': 'armani', 'circ': 'circa', 'gm': 'go', 'extravagent': 'extravagant', 'hupfu': 'huff', 'playe': 'place', 'notme': 'note', 'ltv': 'liv', 'beratur': 'berate', 'dergs': 'dregs', 'colombo': 'combo', 'durty': 'dirty', 'cebu': 'cell', 'whrn': 'when', 'boycott7': 'boycott', 'tstorm': 'storm', 'foodstores': 'bookstores', 'wtb]': 'web', 'jumaat': 'summat', 'granulate': 'granulated', 'pome': 'come', 'profitabilit': 'profitability', 'po10': 'poor', 'poopourri': 'potpourri', 'nevr': 'never', 'rebalanced': 'rebalance', 'misti': 'mist', 'sincerly': 'sincerely', 'kingsman': 'kinsman', 'civillians': 'civilians', 'buildkm': 'build', 'mcqarthy': 'mccarthy', 'hatee': 'hate', 'eh46': 'eh', 'evom': 'vom', 'thooo': 'too', '7lb': 'all', 'nito': 'into', 'shebah': 'sheba', 'shoa': 'show', 'ssafa': 'safe', 'lompoc': 'compos', 'ottonomy': 'autonomy', 'omokri': 'omori', 'paddler': 'paddle', 'shakeel': 'shake', 'errants': 'errands', 'spacepole': 'spaceport', 'southfl': 'south', 'agc': 'ago', 'jr': 'or', 'class8': 'class', 'parkup': 'markup', 'thread]': 'thread', '5k': 'ok', 'brate': 'brave', 'csaid': 'said', 'stv': 'sta', 'roswell': 'swell', 'cerb': 'curb', 'rverse': 'reverse', 'aytm': 'atm', 'getmo': 'getto', 'deliveryoo': 'delivery', 'unworried': 'worried', 'ocaman': 'ocean', 'gbc': 'abc', 'remy15': 'remy', 'ussllc': "us'll", 'fujitsu': 'jujitsu', 'embararassed': 'embarrassed', 'newry': 'newly', 'normaly': 'normal', 'albanyny': 'albany', 'momwining': 'combining', 'endhunger': 'endanger', 'cancerian': 'candarian', 'dcjobs': 'jobs', 'woudhv': 'woud', 'cerah': 'yeah', 'shopyeg': 'shopped', 'carls': 'calls', 'panala': 'panama', 'recertify': 'rectify', 'yena': 'yen', 'proyecto': 'protect', 'march15': 'march', 'approvalif': 'approval', 'thse': 'the', 'laundy': 'laundry', 'pnuttles': 'shuttles', 'ujima': 'jim', 'ravenswood': 'ravenwood', 'homecoo': 'homeroom', 'foodies': 'goodies', 'bagga': 'baggy', 'ranchi': 'ranch', 'iwho': 'who', 'waro': 'war', 'ngaun': 'again', 'kreg': 'keg', 'biz]': 'biz', 'netsuite': 'wetsuit', '3novices': 'novices', 'shibu': 'shifu', 'vees': 'sees', 'playfield': 'hayfield', 'tobi': 'toby', 'tshego': 'theo', 'eriana': 'errand', 'ormond': 'hormone', 'kcci': 'kick', '1if': 'if', 'opps': 'pops', 'bandera': 'wander', 'pacheco': 'paycheck', 'api': 'ape', 'camborne': 'jamboree', 'nfcc': 'nice', 'rs9': 'rse', 'goodf': 'good', 'abuelas': 'abuela', 'amazonfr': 'amazons', 'monfries': 'monies', 'judina': 'judging', 'comment]': 'comment', 'gally1': 'rally', 'kjlh': 'kill', 'choonky': 'chunky', 'macam': 'madam', 'woops': 'woods', 'gdi': 'di', 'muirhead': 'airhead', 'txhigh': 'thigh', '6f': 'of', 'theilman': 'thelma', 'darsy': 'daisy', 'mui': 'mum', 'dher': 'her', 'at5': 'at', 'metros': 'metres', 'unfeasible': 'feasible', 'allvp': 'all', '3l': 'al', 'bezorging': 'belonging', 'nguni': 'gun', 'centry': 'century', 'internalised': 'internalized', 'goodway': 'doorway', 'shickec': 'chicken', 'concious': 'conscious', 'fgcu': 'fuck', 'haryana': 'aryans', 'snp': 'snap', 'behn': 'been', 'nw2': 'we', 'yesterd': 'yester', 'lmbo': 'limbo', 'aroha': 'aroma', 'affinmy': 'affinity', 'linkies': 'pinkies', 'frh': 'fry', 'gomarket': 'market', 'ceap': 'crap', 'smth': 'smith', 'dorch': 'torch', 'chatbox': 'hatbox', 'witasa': 'wits', 'cauli': 'caulk', 'yeesh': 'yesh', 'weiland': 'wetland', 'ladt': 'last', 'loolol': 'loll', 'nobreakfast': 'breakfast', 'dreamwork': 'dreamworks', 'haik': 'hair', 'commer': 'comme', 'youngins': 'younging', 'confrences': 'conferences', 'inno': 'into', 'nandor': 'candor', 'd16': 'do', 'avarage': 'average', 'masksup': 'makeup', 'vbh': 'veh', 'dalry': 'dairy', 'suwelcome': 'welcome', '24am': 'am', 'ready9': 'ready', 'resolvedi': 'resolved', 'kuiba': 'tuba', 'oht': 'out', 'diet]': 'diet', 'stockprice': 'stockpile', 'orajel': 'travel', 'hathu': 'hath', 'katarra': 'katana', 'sureship': 'suresh', 'maximally': 'magically', 'kcrw': 'know', 'exclusivey': 'exclusive', 'vegism': 'vegas', 'yote': 'vote', 'sgcares': 'scares', 'outmost': 'utmost', 'upcharging': 'charging', 'corbett': 'correct', 'emai': 'email', 'mieow': 'meow', 'regtech': 'retch', 'modai': 'moai', 'volenteers': 'volunteers', 'partum': 'part', 'neckache': 'necklace', 'ysk': 'ask', 'tayjah': 'ayah', '22oz': 'coz', 'karwate': 'karate', '2u': 'u', 'heebie': 'herbie', 'cortland': 'portland', '$avgo': 'ago', 'aden': 'den', 'sdet': 'set', 'oakley': 'valley', 'rk': 'ok', 'ziggles': 'giggles', 'yogiji': 'yogi', 'dillet': 'fillet', 'onlys': 'only', 'yokota': 'yoko', 'forso': 'torso', 'rwbb': 'robb', '30b': 'job', '8the': 'the', 'sumner': 'summer', 'prolife': 'police', 'mul': 'mum', 'mfing': 'ming', 'bardsley': 'barley', 'joyy': 'joy', 'stefanie': 'stefan', 'kajang': 'kang', 'eatwell': 'maxwell', 'km': 'am', 'euro2': 'euros', 'kmp': 'amp', 'fua': 'fun', 'logidtics': 'logistics', 'ny17': 'ny', 'saftey': 'safety', 'marte': 'mate', 'horne': 'horse', 'derm': 'term', 'phy': 'why', 'minimomum': 'minimum', 'kanbes': 'knees', 'ijs': 'is', 'fane': 'face', 'd2': 'do', 'prepration': 'preparation', 'bk': 'be', 'hollie': 'collie', '2pk': 'up', 'jolyon': 'colon', 'quin': 'quit', 'didcot': 'didnot', 'briliant': 'brilliant', 'elde': 'else', 'seamore': 'sycamore', 'ccsu': 'csu', 'robles': 'roles', 'biwoc': 'bio', 'reflexively': 'reflexive', 'payche': 'psyche', 'doorder': 'order', 'pckgs': 'picks', 'palladino': 'palladium', 'vaxxes': 'values', 'agri': 'ari', 'papercuts': 'uppercuts', 'arivn': 'arian', 'maartt': 'matt', 'nysg': 'nose', 'fiji': 'fii', 'unlockable': 'unsociable', 'iforgot': 'forgot', 'lenz': 'lend', 'maeve': 'mauve', 'taarak': 'tiara', 'haccp': 'hack', 'happyon': 'happy', 'a6': 'a', 'umabot': 'sabot', 'whitlock': 'hillock', 'mmkay': 'mckay', 'retai': 'retain', 'menfo': 'men', 'compair': 'company', 'voda': 'soda', 'kuehl': 'fuel', 'ecst': 'east', 'oakies': 'okies', 'veggy': 'peggy', 'suruh': 'suru', 'ordererr': 'ordered', 'oceansideca': 'oceanside', '16dec': 'dec', 'm2f': 'of', 'oacado': 'avocado', 'bangsar': 'bangs', 'hrbiz': 'habit', 'spacium': 'stadium', 'socities': 'societies', 'bfpl': 'ball', 'fdp': 'fop', '$cpng': 'icing', 'ainews': 'sinews', 'msgov': 'magog', 'feans': 'means', 'spiralizing': 'spiraling', 'obviate': 'deviate', 'yeste': 'este', 'libtards': 'lizards', '4get': 'get', 'anout': 'about', 'divestiture': 'investiture', 'odinga': 'dinga', 'rebustar': 'rebuttal', 'bila': 'bill', 'kitts': 'kitty', 'avtech': 'tech', 'technocrat': 'technocrats', 'seucirty': 'security', 'xos': 'cos', 'unsliced': 'sliced', 'priv': 'privy', 'flipkart': 'flippant', 'slocum': 'locum', 'aquia': 'aqua', 'abutie': 'abuse', 'stak': 'stay', 'jaya': 'jay', 'eurospar': 'europa', 'sustainalbe': 'sustainable', 'telegrid': 'telegram', 'leuk': 'leak', 'mktsor': 'motor', 'wokeup': 'wakeup', 'apoc': 'apo', 'jiyo': 'hiyo', 'priduce': 'produce', 'warhol': 'carol', 'maisie': 'raise', 'focuse': 'focus', 'liquo': 'liquor', '63rd': 'hard', 'vaxes': 'taxes', 'ecobags': 'coags', 'ketogenic': 'metagenic', '2km': 'him', 'hobe': 'home', 'schweppes': 'schlepped', 'caitlan': 'caitlin', 'mnuchin': 'munching', 'physicall': 'physical', '2h': 'oh', 'bcinterior': 'interior', '3positives': 'positives', 'shepton': 'septon', 'eh11': 'eh', 'thant': 'that', 'munsi': 'munni', 'deff': 'deaf', '2night': 'night', '$ctib': 'crib', 'tere': 'there', 'meehun': 'meghan', 'refil': 'refill', 'gutu': 'guts', 'pkvaan': 'paan', 'foreveryone': 'forevermore', 'ferara': 'fear', 'kuti': 'buti', 'progressivism': 'progressives', 'sektor': 'sector', 'glas': 'glad', 'aviano': 'avian', 'beri': 'bert', 'greenp': 'green', 'presliced': 'presided', 'wayfair': 'mayfair', 'sysyem': 'system', 'atchat': 'that', 'viti': 'vito', 'recive': 'receive', 'experiance': 'experience', 'pretoria': 'pretrial', 'khel': 'keel', 'catbug': 'catdog', 'big4': 'big', 'babloo': 'baloo', 'informatique': 'informative', 'daugh': 'laugh', 'nepa': 'napa', 'abasto': 'baste', 'shree': 'three', 'bluru': 'blur', 'woorst': 'worst', 'izwi': "i'ii", 'alak': 'flak', 'mouffe': 'mouse', 'achool': 'school', 'shiba': 'shiva', 'taylar': 'taylor', 'bigfm': 'big', 'wetboots': 'reboots', 'missplaced': 'misplaced', 'wsop': 'wop', 'muenster': 'monster', 'schubert': 'hubert', 'cancled': 'canceled', 'contextually': 'conceptually', 'shitco': 'shit', 'nojokes': 'jokes', '40m': 'him', '$mo': 'mo', 'turm': 'turn', 'roselyn': 'rosen', '16c': 'doc', 'dfws': 'dews', 'envionment': 'environment', 'cancele': 'cancel', 'empolyee': 'employee', 'supermarket]': 'supermarket', 'smrtnews': 'smartness', 'filey': 'file', 'squaddies': 'squaddie', '30us': 'us', '59am': 'am', 'keung': 'kung', 'vrat': 'rat', 'nn20': 'nna', 'ragey': 'rage', 'wtts': 'wits', 'dololo': 'doll', 'hcbs': 'hubs', 'rosalba': 'rosalee', 'checkers60': 'checkers', '$docu': 'dock', 'recksie': 'necktie', 'evri': 'eri', 'gasen': 'gases', 'frfr': 'fror', 'randee': 'dandee', 'camambert': 'camembert', 'lrts': 'lots', 'fumo': 'sumo', 'ranch99': 'ranch', 'nih': 'ni', 'offic': 'office', 'automtically': 'automatically', 'yowser': 'power', 'julianna': 'julianne', 'shinn': 'shine', 'topdown': 'lowdown', '2avoid': 'avoid', 'salah': 'salad', 'cex': 'sex', 'kisko': 'sisko', 'apprx': 'appr', 'rexell': 'resell', 'sry': 'say', 'underused': 'undressed', 'supprt': 'support', 'franchies': 'franchise', 'jamiaca': 'jamaica', 'ymmv': 'ammo', 'aix': 'six', 'fridah': 'friday', 'ascd': 'and', 'seriesd': 'series', 'kopa': 'kota', 'bjork': 'bjorn', 'faffed': 'failed', 'wtvr': 'water', 'vegs': 'legs', 'textbot': 'textbook', 'behavio': 'behavior', 'madisonnj': 'madison', 'mehh': 'meth', 'horrifyi': 'horrify', 'oldcity': 'oddity', 'westla': 'west', 'rvfb': 'rob', 'topnews': 'tones', 'dairyman': 'airman', 'quorn': 'turn', 'darnae': 'dance', 'canvasser': 'canvassed', 'araw': 'draw', '$etst': 'best', 'rawk': 'raw', 'customters': 'customers', 'lockdo': 'locked', 'vix': 'six', 'betw': 'bet', 'iyer': 'yer', 'dalgona': 'dalton', 'disenfectant': 'disinfectant', 'dedini': 'define', 'donu': 'done', 'daysas': 'days', 'kare': 'are', 'nuvo': 'nuevo', 'fuzzball': 'furball', 'tif': 'if', 'excep': 'except', 'burb': 'burn', 'magulo': 'mogul', 'bubu': 'bub', 'hunnid': 'hunted', 'orland': 'roland', 'checko': 'check', 'fud': 'fun', 'ricetta': 'ricotta', 'semarang': 'searing', 'joythe': 'ofthe', 'adss': 'ass', 'tallon': 'gallon', 'zhongli': 'zhong', 'waniso': 'wants', 'dita': 'data', 'dreadingit': 'dreading', 'o3': 'of', 'blasis': 'basis', 'palani': 'plan', 'rehte': 'reste', '3oc': 'doc', 'logist': 'lost', '4take': 'take', 'esplana': 'esplanade', 'stroudies': 'studies', 'canuck': 'snuck', 'comf': 'come', 'icbc': 'ice', 'norcal': 'normal', 'maisel': 'raise', 'wiggers': 'niggers', 'lovei': 'love', 'medpage': 'message', 'whatho': 'whatto', 'stres': 'stress', 'allyear': 'ayear', 'onthego': 'onthe', 'rusape': 'rape', 'rslash': 'slash', 'patia': 'patio', 'mbr': 'mar', 'sector1': 'sector', 'woodson': 'woodsen', 'piedad': 'plead', 'hoverboards': 'hoverboard', 'burnoff': 'turnoff', 'westwood': 'westward', 'starin': 'staring', 'kash': 'cash', 'needfull': 'needful', 'iwan': 'swan', 'ranvu': 'ran', 'dollarsg': 'dollars', 'harney': 'harvey', 'intensifier': 'intensifies', 'straightener': 'straightened', 'hellovea': 'helluva', '26am': 'am', 'btc': 'etc', '2kms': 'arms', 'poa': 'pop', 'gameon': 'game', 'hellz': 'hell', 'p25': 'put', 'rylands': 'ryland', 'lukic': 'music', 'nhl': 'nil', 'coronaqld': 'coronal', 'misf': 'miss', 'picka': 'pick', 'maribel': 'marine', 'restauarants': 'restaurants', 'ao1': 'ao', 'acnapproved': 'unapproved', 'us]': 'us', 'ispinach': 'spinach', 'rvp': 'rip', 'cupe': 'cup', 'presale': 'resale', 'krogers': 'rogers', 'cheekily': 'cheeky', 'judys': 'judy', 'triadnc': 'triad', 'portwall': 'portal', 'vinay': 'vijay', 'clfood': 'blood', 'kichen': 'kitchen', 'dasher': 'dashed', 'evv': 'eve', 'senditoo': 'bendito', 'frankfur': 'frankfurt', 'grandads': 'grandad', 'extemely': 'extremely', 'rona': 'bona', 'borealys': 'borealis', 'nadege': 'badge', 'taqueria': 'queria', 'instore': 'inshore', 'glasg': 'glass', 'malayo': 'mayo', 'prdct': 'price', 'thecrown': 'theirown', 'qube': 'tube', 'leviana': 'deviant', 'alberta': 'albert', 'endcap': 'entrap', 'duolingo': 'dueling', 'ordererid': 'ordered', 'pickl': 'pick', 'us9': 'us', 'tendies': 'tennis', 'episteme': 'epitome', 'emrg': 'earn', 'lezo': 'leo', 'cheetwood': 'fleetwood', 'v6': 've', 'burgis': 'burns', 'konect': 'honest', 'wakely': 'wakey', 'giwa': 'gina', 'upsetti': 'upset', 'time]': 'time', 'farmingo': 'farming', 'saoirse': 'soiree', 'stfrom': 'itfrom', 'rainelle': 'danielle', 'ppes': 'pipes', 'seevs': 'seems', 'rabah': 'rajah', 'mctaggart': 'taggart', 'consuma': 'consume', 'dropdensity': 'propensity', 'geezus': 'genius', 'paupau': 'papa', 'dogmaels': 'dogmas', 'acr21': 'acres', 'chil': 'child', '6er': 'her', 'beverley': 'beverly', 'mccamn': 'mccain', 'aa': 'a', '$pfe': 'life', 'hiland': 'himand', 'snax': 'snap', 'pipersnl': 'pipers', 'tolland': 'holland', 'konstanz': 'constant', 'ulyssesph': 'ulysses', 'elisea': 'elise', 'skelton': 'skeleton', 'famlay': 'family', 'culotte': 'culottes', 'a75': 'a', 'takis': 'takes', 'microcenter': 'micrometer', 'hny': 'any', 'diario': 'dario', 'mpala': 'impala', 'wealden': 'walden', 'dyk': 'dye', 'delco': 'deco', 'croud': 'proud', '75k': 'ask', 'sandigo': 'santiago', 'snapp': 'snap', 'fashionia': 'fashion', 'jassy': 'sassy', 'cardonald': 'cardinal', 'dpt': 'dot', 'bens': 'bend', 'ocassionally': 'occasionally', 'renewables': 'renewable', 'isthey': 'ifthey', 'ebile': 'exile', 'hotterby': 'lottery', 'seh': 'she', 'bamsi': 'bambi', 'snoozes': 'snooze', 'abq': 'abs', 'wathoda': 'cathode', 'maas': 'mass', 'norcotts': 'boycotts', 'gowrie': 'cowrie', 'nycooks': 'cooks', 'nonapa': 'sonata', 'laurenti': 'laurent', 'amou': 'amor', 'wildlink': 'wildling', 'airoli': 'aioli', 'benoni': 'beyond', 'uavailable': 'available', 'houston1': 'houston', 'carrick': 'carsick', 'nopromise': 'promise', 'unhauled': 'hauled', 'santizer': 'sanitizer', 'retaile': 'retail', 'mimie': 'mimic', 'ptsd': 'its', 'popsci': 'pops', 'epcot': 'spot', 'kicke': 'kick', 'acipco': 'cinco', 'nogales': 'noodles', 'company]': 'company', 'shantala': 'chantal', 'bergevin': 'bergen', 'meghji': 'meghan', 'coopuk': 'cook', 'readability': 'reliability', 'report]': 'report', 'vien': 'view', 'dothan': 'than', 'wallen': 'fallen', 'xxed': 'axed', 'ganesh': 'games', 'ministores': 'ministers', 'educ': 'due', 'tifu': 'tofu', 'setuju': 'setup', 'aluta': 'alta', 'cd8': 'do', 'competive': 'compete', 'ofthen': 'often', 'mehole': 'whole', '3contact': 'contact', 'parel': 'panel', 'mifflin': 'muffin', 'bmth': 'both', 'antonions': "antonio's", 'tenfo': 'tengo', 'gulo': 'gulf', 'erea': 'area', 'kunal': 'final', 'hearin': 'hearing', 'adverstising': 'advertising', 'calery': 'celery', 'stalbert': 'stalker', 'dohrn': 'down', 'wojeers': 'workers', 'microf': 'micro', 'cci': 'chi', '37m': 'him', 'trah': 'trash', 'azpolitics': 'politics', 'depa': 'deja', 'novia': 'nova', 'huntingdon': 'huntington', '2hubby': 'chubby', 'umxt': 'must', 'unbooking': 'unhooking', 'extrend': 'extend', 'freyja': 'freya', 'coastside': 'coastline', 'artisanally': 'artisanal', 'freetrial': 'pretrial', 'iamvred': 'immured', 'pharmaci': 'pharmacy', 'trecker': 'tracker', '21yo': 'yo', 'delvry': 'delivery', 'citize': 'citizen', 'sandel': 'sandal', 'timbiebs': 'timbers', 'kunyit': 'unit', 'alastair': 'alistair', 'everthing': 'everything', 'nsa18': 'nsa', 'scpol': 'school', 'getajob': 'greatjob', 'stratton': 'station', 'supra': 'sutra', 'appdev': 'apple', 'ps63': 'psi', '$tdoc': 'doc', 'blergh': 'sleigh', 'gauteng': 'gluten', 'valueadd': 'valued', 'kn': 'in', 'fings': 'finds', 'wmtds': 'words', 'vidalia': 'vidal', 'poptarts': 'poplars', 'tappenden': 'appended', 'excitementb': 'excitement', 'hirekw': 'hired', 'mandaue': 'mandate', 'eeee': 'eee', 'dogmatically': 'dramatically', 'kch': 'ich', 'off]': 'off', 'foodde': 'foodie', 'explicate': 'duplicate', 'lloyds': 'lloyd', 'busu': 'busy', 'orlandofl': 'orlando', 'amberley': 'ambered', 'arp': 'are', 'bsj': 'bij', 'aukes': 'nukes', 'bilher': 'either', 'grom': 'from', 'quicart': 'quart', 'jhs': 'jus', 'eeeeee': 'peewee', 'wew': 'we', 'jamarcus': 'marcus', 'yurkey': 'turkey', 'ycdem': 'chem', 'anta': 'ants', 'bolloxed': 'bollixed', 'parkwood': 'hardwood', 'ipa': 'spa', 'ipm': "i'm", 'rouleur': 'roller', 'prnews': 'news', 'masavu': 'masala', 'dms': 'des', '5h': 'oh', 'peets': 'meets', 'gencon': 'beacon', 'nevear': 'never', 'prson': 'person', 'fatafat': 'fatbrat', 'adventists': 'dentists', 'greenspan': 'greenan', 'paddington': 'reddington', 'yegnews': 'renews', 'emirati': 'emigrate', 'ptd': 'pod', 'skrimp': 'shrimp', 'masterclass': 'masterless', 'westwed': 'wasted', 'teethe': 'teeth', 'logitics': 'logistics', 'imhungry': 'hungry', 'mhg': 'mug', 'plphotos': 'photos', 'unctuo': 'unto', 'tpaper': 'paper', 'appre': 'apple', 'soeasy': 'easy', 'probly': "prob'ly", 'pozzo': 'pizza', 'pathi': 'path', 'nstuff': 'stuff', 'dinnerly': 'dinner', 'pandy': 'candy', 'jongwe': 'tongue', 'truckers]': 'truckers', 'precima': 'precise', 'weatherproof': 'waterproof', 'bargarh': 'bargain', 'vmessentials': 'essentials', 'segal': 'legal', 'checkit': 'check', 'chatt': 'chat', 'additiona': 'additional', 'amazonas': 'amazons', 'buycott': 'boycott', 'medio': 'media', 'onebyone': 'onlyone', 'blono': 'blond', 'galit': 'gait', 'cltv': 'cut', 'lefteris': 'letters', 'fru': 'fry', 'picnick': 'picnic', 'inscets': 'insects', 'ileoja': 'leola', 'kwanu': 'kwan', 'ingia': 'india', 'signif': 'signify', 'fuso': 'fuss', '9p': 'up', 'correspo': 'correspond', 'coronari': 'coronary', 'hmmmm': 'hammy', 'aoa': 'aka', 'parlliative': 'palliative', 'gronk': 'wrong', 'gojava': 'guava', 'meij': 'mein', 'floriduh': 'florida', 'footgolf': 'foothold', 'huntin': 'hunting', 'zeeland': 'zealand', '2t': 'it', 'wubba': 'bubba', 'virgina': 'virgin', 'emc': 'em', 'reasobale': 'reasonable', 'phamarcy': 'pharmacy', 'bikeables': 'likeable', 'fitnniss': 'fitness', 'mendel': 'mended', 'cosco': 'coco', 'sence': 'since', 'ystrday': 'yesterday', 'sufferering': 'suffering', 'champaignil': 'champaign', 'beverlie': 'beverage', 'beoming': 'becoming', 'paytm': 'part', 'resuable': 'reusable', 'suceess': 'success', 'thiniking': 'thinking', 'doria': 'dorian', 'taskmee': 'tasked', 'happyard': 'halyard', 'atl': 'all', 'gocery': 'grocery', 'noodl': 'noodle', 'midstate': 'mistake', 'sbux': 'sux', 'quishi': 'sushi', 'nikhita': 'nikita', 'gya': 'ya', 'acquisitio': 'acquisition', 'esm': 'es', 'protos': 'photos', 'lali': 'ali', 'lts': 'its', 'getme': 'letme', 'yujie': 'yuji', 'evanston': 'evasion', 'duarte': 'dare', '4w': 'ow', 'optoon': 'option', 'jobso': 'jobs', 'district7': 'district', 'derange': 'deranged', 'bahraini': 'barzini', 'randome': 'random', 'acuff': 'cuff', 'brampton': 'braxton', 'penaty': 'penalty', 'grocery4u': 'grocery', 'e65': 'end', 'doored': 'doomed', 'ipo': 'ipod', 'hrva': 'hava', 'addle': 'saddle', 'twisties': 'twisted', 'welit': 'welt', 'kwam': 'swam', 'pvamu': 'vamp', 'pedpod': 'period', 'wegeb': 'wegen', 'kio': 'kid', 'soemrhing': 'something', 'wale': 'walk', 'uhlala': 'lala', 'globest': 'globes', 'blanchard': 'blanched', 'ytube': 'tube', 'mnimom': 'minimum', 'waiti': 'wait', 'woukd': 'would', 'tfc': 'tac', 'ontarians': 'sontarans', 'gilman': 'oilman', 'mighy': 'might', 'setu': 'set', 'nakunje': 'nakute', 'refreezing': 'freezing', 'shishir': 'shish', 'avaible': 'available', 'techwire': 'techie', 'backyour': 'backdoor', 'puram': 'pram', 'zayde': 'maybe', 'totp': 'top', 'trumpectomy': 'lumpectomy', 'services]': 'services', 'ites': "it's", 'palasade': 'palisade', 'shp': 'she', 'fntech': 'fetch', 'acquisitons': 'acquisitions', 'momtips': 'mommies', 'semetis': 'nemesis', 'fresnel': 'freshen', 'wyd': 'wed', 'recco': 'rocco', 'fucc': 'fuck', 'kimmel': 'gimme', 'pearland': 'garland', 'econdev': 'seconded', '99ranch': 'branch', 'palnning': 'planning', 'clovelly': 'lovely', 'tetsu': 'tetsuo', 'gaiter': 'waiter', 'kju': 'ju', 'bilkul': 'bill', 'shiratake': 'shiitake', 'frappucino': 'frappuccino', 'vends': 'ends', 'amazion': 'amazon', 'accross': 'across', 'vlogger': 'blogger', 'mafi': 'mafia', 'forgettinf': 'forgetting', 'canno': 'cannot', 'sph': 'spy', 'doorcart': 'doormat', 'corruptgop': 'corruption', 'agritech': 'arktech', 'pitbulls': 'pitbull', 'comprimed': 'comprised', 'bulliyed': 'bullied', 'ferny': 'ferry', 'loveny': 'lovely', 'araund': 'around', 'lustfully': 'lustful', 'soun': 'son', 'grocey': 'grocery', 'dx': 'do', 'ynez': 'inez', 'coober': 'cooler', 'yh': 'oh', 'thng': 'thing', 'hamare': 'hamate', 'gaf': 'gas', 'baquettes': 'baguettes', 'whoke': 'whole', 'heimat': 'heat', 'ddpy': 'day', 'strubs': 'scrubs', 'grameen': 'green', 'sttont': 'strong', 'gid': 'did', '$us': 'us', 'langan': 'langa', 'paralised': 'paralysed', 'alcihol': 'alcohol', 'subtweet': 'subtext', 'ifying': 'iying', 'checkboxes': 'cheekbones', 'npo': 'no', 'montano': 'montana', 'toprated': 'operated', 'twe': 'the', 'rbts': 'rats', 'uhaul': 'haul', 'carel': 'care', 'adulting': 'adjusting', 'herndon': 'tendon', 'nf': 'of', 'dpad': 'dead', 'greenbank': 'greenback', 'evrrything': 'everything', 'mixe': 'mine', '$jpm': 'jam', 'giesel': 'diesel', 'thame': 'shame', 'adamson': 'damson', 'alston': 'aston', 'donoso': 'donor', 'parkng': 'parking', 'roche': 'noche', 'pased': 'passed', 'gotech': 'tech', 'maranda': 'miranda', 'southla': 'south', 'aira': 'air', 'dvd': 'did', 'biovac': 'bionic', 'sheesh': "she's", 'pixelart': 'pixelate', 'dahil': 'sahil', 'ciustomer': 'customer', 'hojai': 'ojai', 'sg1': 'so', 'superlatively': 'superlative', 'pob': 'job', 'ubcan': 'can', 'deliveryuae': 'deliveryman', 'loguru': 'yogurt', 'mfc': 'mic', 'crista': 'cristal', 'mhcares': 'cares', 'elora': 'flora', 'bepis': 'begin', 'bridgers': 'bridges', 'sickie': 'sickle', 'savetime': 'sometime', 'pso': 'so', 'mcnamee': 'manatee', 'inciden': 'incident', 'tb': 'to', 'carolin': 'caroline', 'dealz': 'deal', 'ntuc': 'ntac', 'hasura': 'aura', 'more]': 'more', 'ktir': 'stir', 'anythjng': 'anything', '8fem': 'fem', 'thefeed': "there'd", 'deafeningly': 'deafening', 'superwide': 'supervise', 'applaudi': 'applaud', 'hpgas': 'has', '$pnra': 'opera', 'receivi': 'receive', 'bollox': 'bollix', 'challans': 'chaplains', 'blacklives': 'blackies', '$mmmb': 'ummm', 'qualifed': 'qualified', 'tcdsa': 'casa', 'heffas': "jeff's", 'chelete': 'cheese', 'sprecher': 'sprechen', 'pusa': 'push', 'respctd': 'respect', 'khanna': 'hanna', 'bacoor': 'actor', 'uyeke': 'dyke', 'newdiscovery': 'rediscovery', 'weiter': 'writer', '$ino': 'vino', 'figu': 'fig', 'bottega': 'bodega', 'gatoes': 'gates', 'nexa': 'next', 'absolutly': 'absolutely', 'meksdirect': 'misdirect', 'tbi': 'fbi', 'rebuildin': 'rebuilding', 'ubless': 'unless', 'wency': 'wench', 'fetterman': 'letterman', 'gunby': 'gunny', 'millenials': 'millennials', 'jploft': 'plot', 'expir': 'expire', 'lovestay': 'overstay', 'chn': 'can', 'bodmin': 'booming', 'imtired': 'tired', 'wholeseller': 'wholesaler', 'concentrator': 'concentrate', 'alfalfas': 'alfalfa', 'davido': 'david', 'herbology': 'serology', 'nysc': 'nose', 'hagg': 'hang', 'pelmeni': 'yemeni', 'cavan': 'cava', 'chw': 'cow', 'csc': 'csi', 'pleb': 'plea', '11a': 'a', 'tkd': 'tad', 'pastaio': 'pasta', 'unioned': 'union', 'naman': 'aman', 'degrowing': 'regrowing', '15minutes': 'minutes', 'musick': 'music', 'lemony': 'lemon', 'shisha': 'shish', 'sueing': 'seeing', '$lefant': 'elegant', 'sistors': 'sisters', 'horder': 'order', 'megatime': 'negative', 'happyy': 'happy', 'sprague': 'prague', 'bendovers': 'benders', 'coronasa': 'coronary', 'boldre': 'bolder', 'drumf': 'drum', 'acs': 'as', 'lulac': 'lilac', 'kaci': 'kai', 'costanza': 'stanza', 'bego': 'beg', 'happyif': 'happy', 'bizz': 'buzz', '15now': 'know', '$tenx': 'ten', 'lvl': 'lil', 'lubuic': 'public', 'devivery': 'delivery', 'envir': 'entir', '$spx': 'six', 'bluffton': 'buffoon', 'ellenos': "ellen's", 'mbpoli': 'poli', 'chie': 'chief', 'qwoted': 'quoted', 'fdlx': 'full', 'tech00': 'tech', 'pasabay': 'passably', 'repetitve': 'repetitive', 'sirf': 'sir', 'untilities': 'utilities', 'kka': 'aka', 'hts': 'his', 'righties': 'eighties', 'oreal': 'real', 'approval1': 'approval', 'sarita': 'marital', 'aov': 'tov', 'salauddin': 'aladdin', 'fived': 'five', 'ltqny': 'tiny', 'khana': 'hana', 'wmur': 'mur', 'leaveraging': 'leveraging', 'level4': 'level', 'smancy': 'many', 'situa': 'situ', 'corderoned': 'cordoned', 'manwich': 'sandwich', 'nailshop': 'naisho', 'expiremented': 'experimented', 'kigali': 'kali', 'tipu': 'tip', 'urg': 'urge', 'utwx': 'utw', 'advertings': 'advertises', '3bt': 'abt', 'karre': 'barre', 'vf': 'of', 'repleatedly': 'repeatedly', 'oustation': 'station', 'sidewe': 'side', 'cir': 'car', 'petereed': 'petered', 'supermarkte': 'supermarket', 'numbera': 'number', 'insteraxtion': 'interaction', 'mvsales': 'sales', 'pactiv': 'active', 'whses': 'when', 'startu': 'start', 'dobha': 'doth', 'supervalue': 'superglue', 'elem': 'elm', 'roberson': 'robertson', 'huyo': 'hugo', 'jdoss': 'joss', 'sanfresh': 'afresh', 'balto': 'alto', 'jesi': 'jest', 'mkt': 'met', 'rmit': 'mit', 'akilli': 'kill', 'activia': 'active', 'cocke': 'cock', 'rockport': 'rockford', 'digior': 'digits', 'katip': 'katie', 'yo82': 'you', 'workrs': 'works', 'dronee': 'drone', 'euphuism': 'euphemism', 'normandie': 'normandy', 'stuco': 'stuck', 'panjim': 'panic', 'brekfest': 'breakfast', 'cristo': 'crisis', 'myfront': 'front', 'lovela': 'lovely', 'wepioneer': 'pioneer', 'moleskin': 'molesting', 'poolee': 'pooled', 'thjat': 'that', 'kielder': 'fielder', 'ciera': 'ciara', '2billion': 'billion', 'kinesetic': 'kinetic', 'informtion': 'information', 'dogfood': 'dogwood', 'primem': 'prime', 'igot7': 'igot', 'luddon': 'sudden', 'rbf': 'ref', 'dazs': 'days', 'myfirst': 'first', 'westend': 'westen', 'ermm': 'erm', 'dollarsx4': 'dollars', 'pavels': 'panels', 'clda': 'clad', 'maikai': 'mirai', 'paytech': 'patch', 'sauda': 'sauna', 'hcommerce': 'commerce', 'rollup': 'roll', 'airpods': 'airport', 'seonews': 'stones', 'sbu': 'sub', 'stamford': 'stanford', '9e': 'we', 'simplot': 'simple', 'estem': 'stem', 'gentilly': 'mentally', 'pricier2': 'pricker', 'cnyb': 'any', 'alabi': 'alibi', 'refjnd': 'refund', 'cambma': 'camera', 'mez': 'me', 'erdman': 'german', 'westated': 'gestated', 'sheffiekd': 'sheffield', 'tetley': 'title', 'bestway': 'beltway', 'poaa': 'poca', 'stakeholder': 'stakeholders', 'friedman': 'fireman', 'dollarsbn': 'dollars', 'sobitan': 'sabian', 'yess': 'less', 'drr': 'dry', 'soluti': 'south', 'antlov': 'analog', 'privelege': 'privilege', 'condi': 'condo', 'incmag': 'incan', 'roome': 'room', 'grat': 'great', 'cuore': 'cure', 'coorder': 'order', 'housecall': "house'll", 'grclient': 'client', 'zhmja': 'zumba', 'misname': 'mistake', 'amasaka': 'alaska', 'aravali': 'travail', 'gandhis': 'gandhi', 'ourcom': 'outcome', 'winchell': 'mitchell', 'baki': 'bake', 'cv8': 'can', 'nhpi': 'ship', 'yowahk': 'yowake', 'ecc': 'etc', 'palooza': 'palooka', 'rhoa': 'whoa', 'kamari': 'safari', 'puchase': 'purchase', 'momlife': 'lowlife', 'vanishingly': 'vanishing', 'sortable': 'portable', 'ahkart': 'heart', '6ish': 'wish', 'fangirling': 'fingerling', 'poolboy': 'toolbox', 'wokest': 'worst', 'stimmie': 'gimmie', 'tyourone': 'tyrone', 'rdy': 'dry', 'stufd': 'stuff', 'card]': 'card', 'pedder': 'peddler', 'toitech': 'twitch', 'cdnag': 'drag', 'ad]': 'add', 'shomari': 'shari', 'wholeass': 'wholeness', 'plaudit': 'plaudits', 'denglisch': 'english', 'asmf': 'asif', 'signatur': 'signature', 'tpar': 'tear', 'mideast': 'ideas', 'milkbot': 'milksop', 'gandules': 'candles', 'carex': 'care', 'pcsn': 'can', 'relds': 'reads', 'gabanza': 'cabana', 'meatloafs': 'meatloaf', 'd81': 'do', 'cals': 'call', 'cigna': 'sign', 'southlon': 'southern', 'mnleg': 'miles', 'ask]': 'ask', 'korona': 'corona', 'laksa': 'laka', 'undifferent': 'indifferent', '2dfd': 'did', 'smithdown': 'sitdown', 'eurobeat': 'european', '59mins': 'mins', 'fking': 'king', 'rada': 'radar', 'infrast': 'infant', 'gop': 'got', 'mobitel': 'mobile', 'shorewood': 'sherwood', 'knitter': 'knitted', 'wasteman': 'wasteland', 'chirstmas': 'christmas', '$rrc': 'eric', 'sc2': 'sci', 'love7': 'love', 'mlk': 'milk', 'cyour': 'your', '$itw': 'it', 'comapnies': 'companies', 'shopsa': 'shops', 'thankies': 'hankies', '94yo': 'yo', 'qtips': 'tips', 'carlington': 'arlington', 'vohra': 'vodka', 'kabucha': 'kombucha', 'study]': 'study', 'rtpodcast': 'podcast', '$djia': 'dia', 'samebday': 'someday', '$clmt': 'clot', 'intervie': 'interview', 'veganla': 'vegan', 'wnyw': 'know', 'bushwick': 'bushwhack', 'commerical': 'commercial', 'entrepreneu': 'entrepreneur', 'rusing': 'using', 'significent': 'significant', 'churu': 'churn', 'gastonia': 'antonia', 'sanson': 'saison', 'shey': 'they', 'micr': 'mice', 'strugs': 'struts', 'berber': 'barber', 'arjen': 'aren', 'blgr': 'blur', 'travelman': 'tradesman', 'mckeever': 'makeover', 'sepapan': 'seaman', 'poorville': 'orville', 'finca': 'finch', 'adamu': 'adam', 'ncannon': 'cannon', 'yourfork': 'youfor', '$wgsn': 'wasn', 'througho': 'through', 'brin': 'bring', 'cec': 'rec', 'orono': 'wrong', 'eata': 'eat', 'demaecan': 'demean', 'mase': 'make', 'negroni': 'negro', 'maimi': 'miami', 'untagged': 'tagged', 'noodleo': 'noodles', 'donatio': 'donation', 'bakkie': 'bake', 'ib': 'i', 'nikal': 'final', 'literall': 'literally', 'bgp': 'bop', 'teigen': 'tiger', 'kxcc': 'kick', 'queensu': 'queens', 'umdali': 'umami', 'babyit': 'babysit', 'manz': 'man', 'assort': 'assert', 'gaa': 'gas', 'catal': 'fatal', 'chri': 'chris', 'dento': 'dent', 'arians': 'asians', 'shopnix': 'phoenix', 'uaemarket': 'upmarket', 'ecargo': 'cargo', 'costomer': 'customer', 'explotative': 'exploitative', 'dmarts': 'darts', 'rtc': 'rec', 'schor': 'schon', 'catterton': 'patterson', 'theads': 'heads', 'smeaton': 'season', 'gocart': 'goat', 'wynnie': 'winnie', 'lkld': 'told', 'totton': 'cotton', 'rciu': 'rid', 'marga': 'maria', 'gettingby': 'getting', 'nicar': 'nicer', 'boro': 'born', 'apologetically': 'unapologetically', 'dovana': 'donna', 'goodeats': 'goodest', '2ply': 'ply', 'life4all': "life'll", 'tvs': 'tis', 'vpn': 'van', 'restaurante': 'restaurant', 'gmo': 'go', 'sandwhich': 'sandwich', 'shutdows': 'shutdown', 'rheumy': 'remy', 'undercharge': 'undercharged', 'instalist': 'installs', 'dsys': 'days', 'nccp': 'nice', 'gas$': 'gas', '20ks': 'asks', 'awd': 'and', 'disater': 'disaster', 'infiniti': 'infinite', 'sheeple': 'steeple', 'frusterating': 'frustrating', 'orderinance': 'ordinance', 'idh': 'id', 'lanta': 'santa', 'homenews': 'homeless', 'bfrv': 'born', 'hiltons': 'hilton', 'brmoms': 'brooms', 'pqa': 'pa', 'moyle': 'mole', 'mnwx': 'mix', 'noassistance': 'assistance', 'sodo': 'soda', 'murr': 'burr', 'resul': 'result', 'nq': 'no', 'steppingup': 'stepping', 'ffp2': 'fop', '$cost': 'cost', 'himashi': 'hibachi', 'vcph': 'ich', 'somizi': 'nomini', 'rady': 'ready', 'shfm': 'sham', 'okotoks': 'kooks', 'snapple': 'sample', '14x': 'six', '41am': 'am', 'bready': 'ready', 'tetri': 'petri', 'brogrammers': 'programmers', 'bizrate': 'pirate', 'umama': 'mama', 'devasting': 'devastating', 'lokdwn': 'lockdown', 'gomarlo': 'marlo', 'phiike': 'iike', 'leyton': 'peyton', 'unperformed': 'performed', 'remediate': 'immediate', '31lbs': 'bulbs', 'calstock': 'cassock', 'mond': 'mind', 'meawhile': 'meanwhile', 'disabilty': 'disability', 'santisation': 'sanitation', 'maaks': 'marks', 'premeditate': 'premeditated', 'woodstock': 'woodcock', 'unsurprised': 'surprised', 'lbk': 'lik', 'mnths': 'months', 'grabz': 'grab', '16oz': 'coz', 'steac': 'steal', 'crema': 'cream', 'sjsharks': 'sharks', '3lbs': 'plus', 'opticien2': 'optician', 'heatmap': 'headman', 'surprisei': 'surprise', 'hamesha': 'hamish', 'vessi': 'jessi', 'aasan': 'wasan', 'morfa': 'mora', 'penarth': 'earth', 'bonsall': 'bonsai', 'keemit': 'kermit', 'glenrock': 'glenbrook', 'mtribes': 'tribes', 'giveon': 'given', 'dming': 'doing', 'chelsi': 'chest', '5x6': 'ex', 'carefull': 'careful', 'brittny': 'brittany', 'fnq': 'and', 'constrast': 'contrast', 'moxng': 'mong', 'maby': 'many', 'oplive': 'olive', 'klarna': 'klara', 'ldquo': 'liquor', 'preorderer': 'preordered', 'kally': 'rally', 'younhs': 'youths', 'sweetdeals': 'sweetmeats', 'amazin': 'amazing', 'clus': 'plus', 'toparents': 'parents', 'webvan': 'wean', 'day89': 'day', 'jic': 'vic', 'inforum': 'inform', 'rncincle': 'encircle', 'nethe': 'nether', 'voly': 'holy', '$mcd': 'mad', 'stage2': 'stage', 'talenti': 'talent', 'tcot': 'cot', 'sacca': 'sack', 'offerup': 'offer', 'tsr': 'tar', '72f': 'of', 'gyoza': 'yoga', 'stmt': 'stat', 'funnyso': 'funny', 'masango': 'mango', 'potentually': 'potentially', 'resender': 'reminder', 'porks': 'works', 'eh52': 'eh', 'ecrm': 'erm', 'convience': 'convince', 'deapool': 'deadpool', 'lorin': 'loin', 'altruist': 'altruism', 'kambe': 'kabe', 'esque': 'rescue', 'tdap': 'trap', 'parys': 'parts', 'catalyzes': 'catalyzed', 'laventure': "l'aventure", 'mfs': 'ifs', 'hanley': 'haley', 'copaganda': 'propaganda', '12h': 'oh', 'outdawn': 'outlaw', 'sinai': 'shinai', 'financialtt': 'financial', 'virool': 'viral', 'norminally': 'nominally', 'gyomu': 'you', 'mk': 'my', 'dawnyell': 'darnell', 'lfct': 'lfit', 'egrowcery': 'grocery', 'enamor': 'senator', '$opk': 'look', 'cany': 'can', 'alwys': 'always', 'freakout': 'breakout', 'eshowe': 'show', 'tiredly': 'tired', 'll': 'all', 'mdownloadi': 'download', 'dryice': 'drive', 'imagi': 'image', 'prerolls': 'payrolls', 'sparham': 'spartan', 'throu': 'throw', 'pitchfest': 'pitches', 'j6': 'je', 'sdgs': 'sags', 'tbsp': 'top', 'flipka': 'flip', 'cvnt': 'cent', 'liveis': 'lives', 'driveth': 'drive', 'chotu': 'chou', 'whic': 'which', '01am': 'am', 'barefeet': 'barefoot', 'webinars': 'seminars', 'arpx': 'are', 'gerstner': 'kershner', 'ayce': 'ace', 'unpurchased': 'purchased', 'possibiltiy': 'possibility', 'gaylene': 'gayle', 'oyalty': 'loyalty', 'tssk': 'task', 'ressolution': 'resolution', 'unionistas': 'unionists', 'bykea': 'bike', 'pumpy': 'pump', 'shipp': 'ship', 'ia]': 'ian', 'intresting': 'interesting', 'imhershey': 'hershey', 'munti': 'multi', 'erands': 'errands', 'suisse': 'suits', 'pricier': 'pricker', 'cardoso': 'cards', 'mainit': "ain't", 'randwick': 'sandwich', 'sumbitches': 'sumbitch', 'blige': 'oblige', 'scunnered': 'sundered', 'lyso': 'so', 'midd': 'mind', 'ironworker': 'ironworks', 'hul': 'hug', 'kenworth': 'wentworth', 'sbchristmas': 'christmas', 'pilerne': 'pierce', 'smbs': 'subs', 'anywhere]': 'anywhere', 'arra': 'area', 'wdacs': 'was', 'dfe': 'die', 'punitively': 'positively', 'fjb': 'fob', 'altaf': 'altar', 'largo': 'large', 'kubwa': 'kurwa', 'shasub': 'shaun', 'wlng': 'wing', 'aglaia': 'again', 'githeri': 'either', 'timepass': 'timeless', 'aaugh': 'laugh', 'groce': 'grace', 'wws': 'was', 'petsupply': 'resupply', 'freeas': 'frees', 'primehome': 'primetime', 'financialy': 'financial', 'richmondtx': 'richmond', 'pcas': 'peas', 'linao': 'lingo', 'lockdowned': 'lockdown', 'lbi': 'fbi', '2stores': 'stores', 'agen': 'age', 'constitues': 'constitutes', 'southe': 'south', 'easenj': 'eaten', '$goos': 'goos', 'g': 'i', 'ts': 'to', 'bj': 'be', 'humanatm': 'humanity', 'microsoft': 'microdot', 'snab': 'stab', 'formosa': 'forms', 'mighn': 'might', 'avanti': 'avant', 'yawar': 'yaar', 'theme1': 'theme', '3cm': 'him', 'caneesh': 'canes', 'rdquo': 'duo', 'kerri': 'terri', 'mitb': 'mit', 'deffo': 'deaf', 'danloney': 'baloney', 'sempat': 'sempai', 'aug': 'bug', '99cents': 'cents', 'bxs': 'bus', 'agrotech': 'arktech', 'shangou': 'hangout', 'png': 'pig', 'uncomprehending': 'comprehending', '3f': 'of', 'agtech': 'tech', 'chira': 'china', 'prn': 'pin', 'goodings': 'holdings', 'gymlife': 'mylife', 'ena': 'end', 'barbershops': 'barbershop', 'sqm': 'sum', 'naco': 'taco', 'internat': 'internal', 'wilke': 'wilkes', 'ohhellno': 'othello', 'moneyhas': "money's", 'her]': 'her', 'compeition': 'competition', 'mybroadband': 'broadband', 'paterson': 'peterson', 'gamenite': 'amenity', 'ipsos': 'ipso', 'rsculture': 'culture', 'premices': 'premises', 'multipack': 'multitask', 'mobulous': 'fabulous', 'bencho': 'bench', 'moorhead': 'moored', 'independen': 'independent', 'nhk': 'nak', 'ists': 'its', 'mbmc': 'mom', 'waf': 'was', 'bobis': 'boris', 'boona': 'bona', 'robot]': 'robot', 'lawsty': 'last', 'day11': 'day', 'bbmom': 'mom', 'trdl': 'tell', 'berrien': 'berries', 'erh': 'era', 'unsuppressed': 'suppressed', 'gfuel': 'fuel', 'salop': 'salon', '2yo': 'yo', 'ishop': 'shop', 'smfdh': 'smash', 'syu': 'su', 'srushti': 'sushi', 'auspost': 'outpost', 'somepoint': 'somepony', 'webbo': 'webb', 'nduthi': 'nothi', 'wawind': 'wind', 'fin24': 'find', 'wangji': 'wang', 'trenton': 'treason', 'titahits': 'titbits', 'r85': 'run', 'unboxing': 'boxing', 'sidmouth': 'badmouth', 'lyng': 'long', 'ariving': 'driving', 'dooley': 'fooled', 'paresh': 'parish', 'reteach': 'teach', 'feastpb': 'feast', 'nofuel': 'fuel', 'discolour': 'dishonour', 'samokat': 'samosas', 'kuyas': 'guys', 'etg': 'egg', 'whathome': 'whathe', 'proc': 'pro', 'eyam': 'exam', 'tanf': 'tank', 'hcldr': 'hold', 'clementines': 'clementine', 'ibmge': 'image', 'eldoret': 'elmore', '3loaves': 'loaves', 'dth': 'doth', 'ucdavis': 'davis', 'knicked': 'kicked', 'st': 'it', 'ashwin': 'shin', 'santization': 'sanitation', 'smfh': 'such', 'kap': 'map', 'stoopid': 'stooped', 'fidgi': 'fidgit', 'tazo': 'taco', 'foriegn': 'foreign', 'milkrun': 'milkman', 'keni': 'ken', 'zego': 'zero', 'ps10': 'psi', 'officemate': 'officiate', 'ecole': 'cole', 'aswenna': 'antenna', 'baut': 'but', 'the6ix': 'their', 'morkie': 'mookie', 'bbc2': 'back', 'marais': 'maris', 'licp': 'lick', 'leh': 'let', 'tvl': 'til', 'sbwling': 'selling', 'yb20': 'ybe', 'leanest': 'meanest', 'christo': 'christ', 'atavism': 'activism', 'post]': 'post', 'uhohana': 'ohana', 'pkng': 'ping', 'scorekeeper': 'storekeeper', 'mismash': 'mishmash', 'shein': 'shin', '2week': 'week', 'brownest': 'brownies', 'entebbe': 'entente', 'bbasket': 'basket', 'hacep': 'hacer', 'korish': 'krish', 'aboutus': 'about', 'canad': 'canal', 'magg': 'mag', 'shlists': 'shirts', 'marikina': 'marking', 'lls': 'les', 'marni': 'mani', 'lasinoh': 'casino', 'pnpartner': 'partner', 'jonty': 'monty', 'superbike': 'supervise', 'fishfry': 'fishery', 'radicalists': 'radicalism', 'veganism': 'organism', 'genc': 'gene', 'joyouriding': 'joyriding', 'hadrill': 'drill', 'sneha': 'neha', 'nofear': 'near', 'plucker': 'plucked', 'tulsi': 'tulsa', '7x6': 'ex', 'corpor': 'corpore', 'lohs': 'lots', 'lryd': 'try', '60sixty': 'sixty', 'gugu': 'guru', 'emplaw': 'employ', 'grandstrand': 'grandstand', 'logins': 'loins', 'sadso': 'also', 'kyser': 'keyser', 'ssds': 'sods', '30mn': 'damn', 'hoyas': 'has', 'becus': 'becks', 'siderby': 'derby', 'endeavo': 'endeavor', 't4': 'to', 'peasyntry': 'peasantry', 'repond': 'respond', 'cranston': 'canton', 'bogoya': 'bogota', 'avast': 'vast', 'iphs': 'ips', 'poggy': 'piggy', 'bdnmb': 'bomb', 'workf': 'work', 'countdownnz': 'countdown', 'ariana': 'arian', 'heysp': "he's", 'eberhard': 'bernard', 'rosemount': 'remount', 'inhome': 'income', 'silverlake': 'silverware', 'tailer': 'trailer', 'cse': 'use', 'metroto': 'metro', 'akie': 'akin', '$you': 'you', 'partne': 'partner', '10hr': 'sehr', 'nauza': 'nazi', 'kwi': 'wi', 'snapback': 'knapsack', '7km': 'him', 'mht': 'met', 'kapruka': 'paprika', 'ofnthe': 'ofthe', 'nsc': 'nsa', 'maditory': 'mandatory', 'talay': 'tally', 'wnc': 'inc', 'nwenp': 'went', 'fanyin': 'canyon', 'chesser': 'chester', 'ecf': 'elf', 'sqq': 'so', 'nahin': 'pain', 'kyun': 'yun', 'touga': 'tough', 'driveme': 'drive', 'slopp': 'sloppy', 'vilchez': 'filched', '8days': 'days', 'lockdownme': 'lockdown', 'strawman': 'starman', 'newsolutions': 'resolutions', 'aight': 'right', 'quarantiners': 'quarantines', 'waterislife': 'waterslide', 'sainsos': 'saints', 'naisip': 'raisin', 'lunas': 'lungs', 'lw': 'law', 'genevan': 'geneva', 'snowday': 'snowy', 'u18s': 'us', 'theycould': 'theywould', 'dizzie': 'izzie', 'ldp': 'lap', 'braider': 'braided', 'gopokes': 'gooks', 'nvcs': 'nice', 'winwin': 'within', 'dymocks': 'docks', 'c20': 'can', 'alce': 'ace', 'hourse': 'course', 'mockingjay': 'mockingly', 'dfs': 'des', 'farage': 'garage', 'rutherforder': 'rutherford', 'snapchats': 'snapchat', 'mompany': 'company', 'deepfry': 'deeply', 'seau': 'sea', 'poppi': 'poppy', 'gmother': 'mother', 'aldrine': 'latrine', 'roora': 'room', 'lamu': 'lame', 'gstorex': 'store', 'ntds': 'nods', 'insta': 'instar', 'kildare': 'kilgore', 'easly': 'easy', 'murshid': 'mushed', 'dba': 'dna', 'hubli': 'public', 'camhs': 'camps', 'gbn': 'gun', 'dzevents': 'events', 'sealable': 'syllable', 'langs': 'lands', 'viliving': 'living', 'justyn': 'justin', 'shiptshop': 'shitshow', 'happyya': 'happy', 'silchar': 'pilcher', 'nflx': 'fix', 'pickin': 'picking', 'wonderdog': 'wondering', 'sanitizes': 'sanitize', 'osha': 'sha', 'phlenu': 'phlegm', 'kenton': 'keaton', 'jpis': 'is', '9a': 'a', 'sds': 'ads', 'age4': 'age', 'rappo': 'happy', 'kpi': 'ki', 'kuriology': 'urology', 'shouldnbe': 'shouldbe', 'penh': 'pen', 'usav2': 'save', 'apr2': 'are', 'pretentiously': 'pretentious', 'rsvps': 'rips', 'ilab': 'lab', 'goregow': 'forego', 'mische': 'mischa', 'volunters': 'volunteers', 'jalopeno': 'jalapeno', 'hollywoodfl': 'hollywood', 'pritti': 'pretty', 'oncor': 'once', 'shaye': 'share', 'webmd': "we'd", 'kulfi': 'fulfil', 'pervaiz': 'prevail', '$lulu': 'lulu', 'agara': 'nagara', 'excelente': 'excellent', 'untdos': 'unto', 'dcai': 'dai', 'ciip': 'chip', 'mera': 'mere', 'onrddr': 'order', 'crawforder': 'crawford', 'dursley': 'parsley', 'kuching': 'fucking', 'rosati': 'roast', 'pffoodie': 'foodie', 'bagles': 'eagles', 'freshair': 'fresher', 'twimbos': 'bimbos', 'adrress': 'address', 'yoghurts': 'yoghurt', 'arg': 'are', 'ohmemories': 'memories', 'tuskys': 'tusks', 'caicos': 'chicks', 'myg': 'my', 'paperbag': 'paperback', 'ordererd': 'ordered', 'lmd': 'led', 'chuang': 'chang', 'fulf': 'full', 'indane': 'insane', '30th': 'with', 'dessicated': 'dedicated', 'kingwood': 'pinewood', '7rm': 'arm', 'hols': 'hold', 'exte': 'este', 'ddpoints': 'points', 'ht': 'it', 'r60': 'run', 'usenet': 'sent', 'restos': 'rests', 'fresnos': 'frescos', 'nord': 'word', 'ajax': 'ajar', 'pelayo': 'play', 'chf': 'chef', 'vouge': 'gouge', 'antisemitic': 'antiseptic', 'rhings': 'things', 'ncwane': 'cane', 'rwanda': 'wanda', 'unboxed': 'boxed', 'ownai': 'own', 'fdnnews': 'funnels', 'stma': 'sta', 'conley': 'convey', 'infoit': 'inform', 'ux': 'up', 'bffr': 'for', 'forwarder': 'forwarded', 'sindicato': 'indicate', 'remo': 'memo', 'marketi': 'market', 'ttec': 'tte', 'neigjboor': 'neighbor', 'isabello': 'isabella', 'clipless': 'clueless', '4those': 'those', 'pepo': 'pep', 'pippen': 'pigpen', 'dudeck': 'dude', 'hosta': 'host', 'overdress': 'overdressed', 'tablenc': 'table', 'supr': 'super', 'unthoughtful': 'thoughtful', 'tybee': 'three', 'brittainy': 'brittany', 'twintel': 'winter', 'kolod': 'good', '$wow': 'wow', 'kevinf': 'kevin', 'tamu': 'tame', 'argo': 'ago', 'tg': 'to', 'detai': 'detail', 'maxxine': 'maxine', 'appiontment': 'appointment', 'chantelle': 'channelled', 'justo': 'just', 'managemen': 'management', 'smokie': 'smoke', 'landwehr': 'lander', 'born]': 'born', 'ktr': 'kir', 'e46': 'end', 'isenberg': 'iceberg', 'yeepeey': 'teepee', 'fareshare': 'foreshore', 'kuehne': 'kleine', 'kou': 'you', 'evacuee': 'evacuees', 'hedger': 'hedge', 'numy': 'numb', 'agorophobia': 'agoraphobia', 'teenora': 'tenor', 'newsom': 'news', 'dudu': 'dude', 'aarthiky': 'karthik', 'neccesities': 'necessities', '2accomodate': 'accommodate', 'peroni': 'person', 'nwark': 'nark', 'ordere4': 'ordered', 'mcknight': 'midnight', 'vehichles': 'vehicles', 'ksl': 'kol', 'sulkily': 'luckily', 'somany': 'many', 'lugg': 'lung', 'killa': 'kill', 'appd': 'app', 'interac': 'interact', 'darkpoly': 'darkly', 'clie': 'lie', 'butterbaby': 'butterball', 'cuny': 'cunt', 'deparities': 'disparities', 'ward8': 'ward', 'firedup': 'fired', 'aulani': 'alumni', 'izing': 'icing', 'desprit': 'esprit', 'tsang': 'sang', 'icee': 'ice', 'gonzalez': 'gonzales', 'pg': 'pig', 'baan': 'barn', 'earlier]': 'earlier', 'bnews': 'news', 'tbt': 'tit', 'alberston': 'alberto', 'discre': 'desire', 'platkin': 'plain', 'gilaaa': 'gila', 'w]orkers': 'workers', 'crowe': 'crowd', 'chakki': 'chalk', '36million': 'million', 'rpv': 'rev', 'respek': 'respect', 'chitra': 'china', 'debugger': 'bugger', 'onboarding': 'boarding', 'or3': 'or', 'mupita': 'musta', 'obselete': 'obsolete', 'recved': 'revved', 'cbt': 'cut', 'adress': 'dress', 'ndi': 'ni', 'gso': 'so', '11e': 'the', 'ugg': 'egg', 'nomore': 'nombre', 'infarm': 'inform', 'quran': 'quan', 'dimond': 'diamond', '65m': 'him', 'lmtsm': 'lots', 'sofar': 'solar', 'ceruti': 'cert', 'gloriou': 'glorious', 'topopp': 'topo', 'metally': 'mentally', 'ifood': 'food', 'anw': 'and', 'payega': 'pamela', 'deliveryca': 'delivery', 'flagler': 'flatter', 'fuckoff': 'kickoff', 'gzb': 'gob', 'xlr': 'alr', 'monaghan': 'jonathan', 'shabbiest': 'shabbier', 'houghton': 'houston', 'nubby': 'hubby', 'wr11': 'wrap', 'dchotel': 'hotel', 'andheri': 'adhere', 'yeasties': 'beasties', 'masert': 'maser', '$twlo': 'two', 'shopee': 'hope', 'staythef': 'saythe', '$dnax': 'dna', 'fekkai': 'sekai', 'blackpool': 'blackwood', 'mihc': 'mic', 'leaveus': 'leaves', 'senoras': 'senora', 'surprisewe': 'surprise', 'sorok2': 'stroke', '0l': 'al', 'mbmbam': 'mumbai', 'phl': 'pal', 'tunggu': 'kungfu', 'yowch': 'touch', 'billby': 'billy', 'ahan': 'than', 'sisi': 'sis', 'nowore': 'nowhere', 'usad': 'used', 'ronfeir': 'confer', 't': 'i', 'rgd': 'red', 'spund': 'sound', 'complet': 'complete', 'dayxday': 'payday', 'multilayer': 'multiplayer', 'brainderr': 'braindead', 'rubio': 'rubin', 'loggins': 'logging', 'melvy': 'melty', 'thur': 'tour', 'diseasy': 'disease', 'awanderin': 'wandering', 'i2c': 'ic', 'funnyand': 'funnyman', 'rdskills': 'skills', 'broccolini': 'broccoli', 'kik': 'kid', 'extenda': 'extend', 'paamul': 'paul', 'bii': 'big', 'chirashi': 'hiroshi', 'organizatons': 'organizations', 'turnin': 'turning', 'worder': 'order', 'hurru': 'hurry', 'senik': 'send', 'hisey': 'hissy', 'paulsen': 'paulson', 'eagan': 'pagan', 'nothq2': 'nothin', 'cisa': 'visa', 'recevd': 'record', 'saral': 'sarah', 'hax': 'had', 'tabouli': 'tabula', 'n4tm': 'not', 'xbots': 'bots', 'listserv': 'lister', 'diaf': 'dial', 'ausd': 'auld', 'ppc': 'pic', '$esea': 'sea', 'artific': 'artifice', 'dystopia': 'dystopian', 'ftr': 'for', 'rekla': 'real', 'loml': 'lol', 'ffaf': 'fhaf', 'gtf': 'atf', 'univ': 'unit', 'mhq': 'the', 'carsharing': 'marshaling', 'maureenn': 'maureen', 'communi': 'commune', 'oaklanders': 'outlanders', '9today': 'today', 'reconsiders': 'reconsider', 'offcourse': 'ofcourse', 'bcg': 'big', 'canthandle': 'manhandle', 'bikeshare': 'timeshare', 'dmd': 'did', 'alrea': 'area', 'thnkks': 'thanks', 'buttwholes': 'buttholes', 'rattay': 'ratty', 'faang': 'fang', 'legall': 'legal', 'syes': 'eyes', 'ramazan': 'amazon', 'kiko': 'kilo', 'uneeded': 'needed', 'etid': 'fetid', 'makrosa': 'marisa', 'pictionary': 'dictionary', 'bebidas': 'besides', 'overdrew': 'overdue', 'bigscam': 'bioscan', 'm4a': 'ma', 'imstronger': 'stronger', 'kolar': 'solar', 'lovehow': 'somehow', 'hanadi': 'hands', 'islip': 'slip', 'reti': 'yeti', 'hackable': 'fuckable', '8hrs': 'this', 'easton': 'aston', 'kacang': 'facing', 'lore]': 'lore', 'americ': 'america', 'adrionna': 'adrianna', 'cranksgiving': 'thanksgiving', 'xx': 'ex', 'avin': 'vain', 'singledom': 'singled', 'lg': 'la', 'forg': 'for', 'ksk': 'ask', 'ech': 'each', 'sw13': 'swim', 'telecons': 'telecoms', 'criollo': 'rollo', 'qb': 'ab', 'groot': 'root', 'maji': 'mai', 'ubdaily': 'daily', 'mindfully': 'mindful', 'lekker': 'leaker', 'covfefe': 'coffee', 'rangeme': 'range', 'praveen': 'proven', 'breda': 'bread', 'baddog': 'badder', 'fleeky': 'fleecy', 'coteach': 'teach', 'tinley': 'finley', '7th': 'ith', 'ayi': 'aii', 'mcdelivery': 'delivery', 'p1': 'pa', 'certains': 'certain', 'becuz': 'begun', 'butsi': 'buti', 'keh': 'key', 'palma': 'palm', 'cheezy': 'cheeky', 'doogie': 'boogie', 'homegirls': 'homegirl', 'camlan': 'clan', 'wfood': 'food', 'dely': 'deny', 'sfa': 'sea', 'hainanese': 'taiwanese', 'dka': 'dna', 'pape': 'paper', 'kirsty': 'kristy', 'barilla': 'vanilla', 'yvr': 'yer', 'lipton': 'liston', 'dreamies': 'dreams', 'supermarketd': 'supermarket', 'sponsers': 'sponsors', 'wirral': 'viral', 'fckin': 'fuckin', 'kirstie': 'kirsten', 'tema': 'team', 'shapez': 'shape', 'veday': 'veda', 'fine]': 'fine', 'unsellable': 'unreliable', 'gti': 'ti', 'depop': 'depot', 'adultin': 'adult', 'fuuudge': 'fudge', 'verano': 'vegan', 'kazyon': 'canyon', 'rete': 'rate', 'sabc': 'sac', 'prepaying': 'preparing', 'localyyc': 'locally', '17your': 'your', 'spc': 'spy', 'bartlet': 'battle', 'lovehaving': 'lovemaking', 'aavee': 'have', 'thecsn': 'then', 'shipsy': 'ships', 'headstart': 'headscarf', 'slready': 'already', 'flossie': 'fossil', 'maximoms': 'maximum', 'fruitade': 'fruitcake', 'kulang': 'kung', 'business]': 'business', 'coouuld': 'could', 'mombo': 'combo', 'futurism': 'futurist', 'reasinable': 'reasonable', 'opportu': 'opportune', 'baggagesz': 'baggage', 'wotd': 'word', 'bongabon': 'bonbon', 'kchc': 'echo', 'scoobs': 'scoops', 'handicart': 'handicar', 'voix': 'void', 'nang': 'hang', 'whatsapped': 'whatsapp', 'sizzli': 'sizzle', 'fkin': 'skin', 'scamy': 'scary', 'belligerently': 'belligerent', 'evrything': 'everything', 'potvin': 'protein', 'curbside5': 'curbside', 'choudhary': 'chaudhary', '4ls': 'als', 'olsens': 'olsen', 'cik': 'lik', '48m': 'him', 'sps': 'spy', 'banff': 'bank', 'yeaye': 'leave', 'pblx': 'able', '7e': 'we', 'smthng': 'sting', 'portoflio': 'portfolio', 'govs': 'goes', 'clusterfk': 'cluster', 'hjb': 'hub', 'enh': 'end', 'us43': 'us', 'lndn': 'lndu', 'resta': 'rest', 'cindi': 'cindy', 'sadhu': 'madhu', 'pymes': 'pyres', 'catchya': 'catchy', 'rentmall': 'rental', 'perma': 'perm', 'gally': 'rally', 'panang': 'paying', 'pay4': 'pay', 'beleaguer': 'beleaguered', 'sadf': 'sad', 'extendind': 'extending', 'ambrosio': 'ambrosia', 'favori': 'favor', 'fulfiller': 'fulfilled', 'hoom': 'room', 'menuaid': 'mermaid', 'weleaded': 'released', 'suz': 'sun', 'canonico': 'canonical', 'updrades': 'upgrades', 'ecomme': 'comme', 'girard': 'gerard', 'noods': 'woods', 'nestleusa': 'nestles', 'kfc': 'of', '8km': 'him', 'daaru': 'dear', 'asperger': "asperger's", 'concho': 'conch', '$pton': 'upon', 'gd': 'go', 'w3care': 'care', 'nuggz': 'nigga', 'colac': 'cola', 'olmos': 'almost', 'minatory': 'mandatory', 'mutiple': 'multiple', 'sthelens': 'stevens', 'gari': 'gary', 'nahur': 'nature', 'tured': 'turned', 'damani': 'damn', 'ngathi': 'agatha', 'ironman': 'iranian', 'aabhas': 'habeas', '05am': 'am', 'lakdoun': 'landon', 'biglove': 'glove', 'provi': 'prove', 'operat': 'operate', 'booom': 'boom', 'unhappyi': 'unhappy', 'tgit': 'tit', 'rta': 'rat', 'veget': 'beget', 'optimisation': 'optimization', 'libtard': 'library', 'aivee': 'aimee', 'mingyu': 'ming', 'fuxing': 'fixing', 'nwela': 'well', 'nowcars': 'oscars', 'creamline': 'streamline', 'foodlay': 'goodly', 'loftis': 'lofts', 'sanfran': 'saffron', 'hahaah': 'hahaha', 'bluejay': 'bluesy', 'clinicals': 'clinical', 'kenyans': 'kenya', 'redfaced': 'replaced', 'phr': 'per', 'linton': 'clinton', 'affilliate': 'affiliate', 'agasae': 'abase', 'cooky': 'cook', '9pack': 'pack', 'birite': 'write', 'furnture': 'furniture', 'susah2': 'sushi', 'rhen': 'when', 'paprimary': 'primary', 'northen': 'northern', 'disburse': 'disbursed', 'sh$t': 'shut', 'remicade': 'remade', 'gasstation': 'gestation', 'organizaing': 'organizing', 'somstimes': 'sometimes', 'centralia': 'central', 'hapoen': 'happen', 'noworkers': 'coworkers', 'libbing': 'lobbing', 'unqueue': 'queue', 'notips': 'notice', 'itaja': 'tiara', 'cupride': 'pride', 'oyo': 'yo', 'suu': 'sun', 'artizone': 'arizona', 'donat': "don't", 'stea': 'step', 'communize': 'community', 'automo': 'auto', 'bordererline': 'borderline', 'laun': 'lawn', 'high5': 'high', 'uga': 'ga', 'incumbents': 'incumbent', 'wellity': 'reality', 'adrny': 'any', 'refusee': 'refuse', 'uofo': 'ufo', 'nfld': 'need', 'v21': 'van', 'refridge': 'fridge', 'delivry': 'delivery', 'makan': 'mayan', 'urdoorstep': 'doorstep', 'continuumio': 'continuum', 'rtteam': 'team', 'tenu': 'ten', '1minute': 'minute', 'radishing': 'ravishing', 'brockley': 'buckley', 'sainbury': 'salisbury', 'wks': 'was', 'encourager': 'encourage', 'moonracker': 'moonraker', 'uptow': 'uptown', 'mehdi': 'media', 'ordereal': 'ordered', 'jek': 'jerk', 'lewes': 'lewis', 'detailer': 'detailed', 'enablers': 'enables', 'afful': 'awful', 'duikt': 'built', 'woldnt': 'wouldnt', 'vaxport': 'export', 'losingtime': 'longtime', 'shnob': 'snob', 'needfully': 'needful', '12and': 'and', 'lousiana': 'louisiana', 'sweetsweat': 'sweetmeat', 'dmw': 'dew', 'avma': 'alma', 'aforder': 'order', 'smartnews': 'smartness', 'peikar': 'pear', 'abscess2': 'abscess', 'janta': 'santa', 'skus': 'sous', 'bedbound': 'rebound', 'twoh': 'two', 'alotta': 'lotta', 'foxxi': 'foxxy', 'idek': 'idea', 'emot': 'emit', 'calfresh': 'afresh', 'gidi': 'didi', 'sngry': 'angry', 'danm': 'damn', 'baaaa': 'banana', 'basc': 'base', 'businessdea': 'businesses', 'doesnot': "doesn't", 'ignorancei': 'ignorance', 'tke': 'the', 'mccann': 'mccain', 'ancoats': 'coats', 'fastttt': 'fastest', 'humph': 'hump', 'freetime': 'freebie', '11sh': 'wish', 'onmy': 'only', 'nextipo': 'nextto', 'waltham': 'walther', 'bronchials': 'bronchial', 'tprs': 'tars', 'jf': 'of', 'sneezy': 'sneeze', 'degeneres': 'degenerates', 'groomers': 'groomer', 'snapgap': 'stopgap', 'stilton': 'stilts', 'avlb': 'all', 'wapi': 'papi', 'crossnw': 'cross', 'taimba': 'samba', 'jcc': 'job', 'dharna': 'dharma', 'assista': 'assist', 'portadi': 'portal', 'madisoncw': 'madison', 'hcsm': 'his', 'parmigiano': 'parmigiana', 'fixin': 'fixing', 'gofergh': 'gofer', 'warf': 'war', 'williams97': 'williams', 'imolation': 'isolation', 'cubesat': 'cutest', 'tranmitted': 'transmitted', 'kansan': 'kansas', 'e27': 'end', 'wole': 'whole', 'fleer': 'fleet', 'prax': 'pray', 'lmont': 'mont', 'compsci': 'compact', '$fusz': 'fuss', 'brookedy': 'brooked', 'perlu': 'peru', 'mmt': 'met', 'pple': 'pile', 'icely': 'nicely', '4harvests': 'harvests', 'rogong': 'raging', 'fahd': 'fad', 'amazoned': 'amazed', 'peaced': 'peace', 'engadget': 'engaged', 'flowbee': 'flower', 'pufa': 'puff', 'prayin': 'praying', 'remobilise': 'mobilise', 'addtl': 'add', '12n': 'in', 'turemic': 'uremic', 'tv]': 'the', 'terragon': 'tarragon', 'timmins': 'timmons', 'opportuni': 'opportune', 'ensw': 'ens', 'fawkner': 'faker', 'gaw': 'saw', 'londo': 'condo', 'harpreet': 'harriet', 'paralysingly': 'paralysing', 'suziqs': 'suits', 'bsd': 'bad', 'kthx': 'the', 'ramzan': 'roman', 'bmws': 'bows', '$bb': 'ebb', 'zl1': 'all', 'mirageau': 'mirage', 'alumna': 'alumni', 'miscalculate': 'miscalculated', 'smoko': 'smoke', 'showmars': 'showers', 'ls': 'is', 'liquify': 'liquefy', 'kagaj': 'pagan', 'customi': 'customs', 'propagandai': 'propaganda', 'emmployees': 'employees', 'mpps': 'maps', 'siakap': 'shaka', 'calii': 'cali', 'kinnock': 'knock', 'ggs': 'gas', 'vacine': 'vaccine', 'tmro': 'taro', 'borking': 'working', 'fresnans': 'freshens', 'mamer': 'maker', 'girle': 'girl', 'deliveroos': 'delivers', 'jisp': 'wisp', 'ppe': 'pie', 'crossworder': 'crossword', 'amausi': 'ambush', 'rupesh': 'rules', 'notanad': 'notepad', 'deloveer': 'deliver', 'quadruplet': 'quadruple', 'zadie': 'sadie', 'isps': 'sips', 'vek': 'vet', 'abpse': 'abuse', 'longtail': 'ponytail', 'necn': 'neck', 'virk': 'kirk', 'abantu': 'abate', 'bosc': 'boss', 'myyard': 'yard', 'hobbiton': 'hobbits', 'ules': 'rules', 'unvax': 'una', 'lisence': 'licence', 'kome': 'come', 'citians': 'citizens', 'loveuse': 'loves', 'hothering': 'bothering', 'sapulpa': 'scapula', 'zippity': 'hippity', 'freese': 'freeze', 'dhar': 'dear', 'ojota': 'outta', 'qwqer': 'owner', 'ewallet': 'wallet', 'hqed': 'heed', 'unimart': 'unitard', 'blend43': 'blend', 'recived': 'received', 'worthit': 'worth', 'karke': 'kake', 'rc': 're', 'ortega': 'omega', 'grati': 'grate', 'lovies': 'loves', 'neigbourhood': 'neighbourhood', '10ft': 'left', 'delivries': 'deliveries', 'reso': 'rest', 'discorder': 'disorder', 'milso': 'miso', 'wagonit': 'wagon', 'sny': 'any', 'voef': 'of', 'itv3': 'it', 'jayme': 'jaime', 'planet54': 'planet', 'nstead': 'instead', 'milage': 'mileage', 'geer': 'beer', 'foodo': 'food', 'dunkin': 'dunking', 'textus': 'texts', 'innthis': 'inthis', 'gulati': 'gelatin', 'loveyyc': 'lovely', 'awana': 'wana', 'xcel': 'excel', 'didnt': "didn't", 'g2k': 'gak', 'qiloane': 'sloane', 'suprised': 'surprised', 'jikook': 'iook', 'knust': 'kust', 'grommr': 'groom', 'fanook': 'nook', 'llcs': 'lacs', 'himaya': 'maya', '1lb': 'all', 'neces': 'necks', 'instabtc': 'instant', 'financialiq': 'financial', 'vvmobile': 'mobile', 'dilg': 'dig', 'bcc': 'bac', 'libs': 'lies', 'daylis': 'days', 'chookies': 'cookies', 'jerald': 'herald', 'nuro': 'euro', 'kias': 'kids', 'foke': 'fake', 'malawi': 'malaria', 'promice': 'promise', 'wepurchase': 'purchase', 'scompr': 'scoop', 'shw': 'she', 'drinkie': 'drink', 'ambani': 'armband', 'demartini': 'departing', 'crayola': 'crapola', 'remasking': 'remaking', 'dclife': 'life', 'wala': 'walk', 'bril': 'bail', 'endp': 'end', 'pana': 'papa', 'veritical': 'vertical', 'n9t': 'not', 'jss': 'ass', 'notmeus': 'notes', '7rewards': 'rewards', 'tecniq': 'tennis', 'everdy': 'every', 'hq2': 'he', 'hungries': 'hungrier', 'peelfam': 'pelham', 'vallance': 'valance', 'yesterdayy': 'yesterday', 'accel': 'acces', 'mjay': 'may', 'gnd': 'and', 'dfm': 'dam', 'youcall': "you'll", 'comicon': 'common', 'chy': 'why', 'implica': 'implies', 'bynow': 'know', 'shaikh': 'sheikh', 'cishet': 'wishes', 'mockdown': 'lockdown', 'kgw': 'kow', 'vegitables': 'vegetables', 'nacs': 'naps', 'belbins': 'begins', 'it]': 'it', 'trx': 'try', 'speculaas': 'speculate', 'ayou': 'you', 'fudant': 'mutant', 'bsac': 'sac', 'helpmap': 'helpmate', 'rss': 'ass', 'slowpoke': 'slowpokes', 'acart': 'apart', 'helpdesk': 'helpless', 'adb': 'add', 'yeng': 'yen', 'khajur': 'thakur', 'fevc': 'feel', 'startups]': 'startups', 'bezoz': 'below', 'enbusiness': 'business', 'vb': 've', 'sadz': 'sad', 'chipinge': 'chirping', '69th': 'with', 'awol]': 'awol', 'lockdown21': 'lockdown', 'lvix': 'live', 'comestible': 'compatible', 'donno': 'dunno', 'rmember': 'remember', 'usernames': 'username', 'hiral': 'viral', '$several': 'several', 'elkridge': 'eldridge', 'serviceabc': 'serviceable', 'zmall': 'small', 'strongmont': 'strongman', 'kenin': 'kevin', 'macey': 'mace', 'corcheting': 'crocheting', 'scalability': 'capability', 'paycuck': 'payback', 'ayyy': 'any', 'custmers': 'customers', 'ensuite': 'ensure', 'akhtar': 'altar', 'finno]': 'finn', 'sienza': 'sienna', 'innovativ': 'innovative', 'day2day': 'payday', 'lockdowns': 'lockdown', 'gpl': 'gal', 'sweetc': 'sweet', '2me': 'me', 'tahsis': 'this', 'appintments': 'appointments', 'fitpicks': 'nitpick', 'tatas': 'tapas', 'pocketlint': 'pocketing', 'foodshelf': 'bookshelf', 'abfe': 'able', 'twf': 'two', 'selligent': 'negligent', 'redban': 'sedan', 'reorderering': 'reordering', 'gerl': 'girl', 'tapingo': 'taping', 'adate': 'date', 'dvr': 'der', 'bqexclusive': 'exclusive', 'undoable': 'unable', 'poxed': 'posed', 'qtdelivery': 'delivery', 'sponsored]': 'sponsored', 'withou': 'without', 'pnp': 'pop', 'kozub': 'koku', 'catchup': 'matchup', 'svcs': 'sacs', 'alexandrine': 'alexandria', 'complians': 'complains', 'iknowhow': 'knowhow', 'pullback': 'fullback', 'gt': 'it', 'cente': 'center', 'propellent': 'propellant', '14k': 'ask', 'oinments': 'ointments', 'walma': 'alma', 'hululu': 'hull', 'freshnews': 'freshness', 'sadlad': 'salad', 'naannn': 'nanny', '2ft': 'aft', 'twiga': 'twig', 'sadza': 'sad', 'tabletopics': 'tabletops', 'lenexa': 'enema', 'icurbside': 'curbside', 'haroon': 'maroon', 'nimblerx': 'nimble', 'fila': 'film', '24x': 'six', 'juiceboxes': 'jukeboxes', 'subang': 'bang', 'velveeta': 'velvet', 'developmwnt': 'development', 'muahaha': 'hahaha', 'voldemart': 'voldemort', 'decicco': 'delicto', 'garia': 'maria', 'arianna': 'adrianna', 'fucktawd': 'fucktards', 'reasonabe': 'reasonable', 'breton': 'button', '66m': 'him', 'garos': 'gyros', 'circe': 'circle', 'arddr': 'ardor', 'bcmotorcycle': 'motorcycle', 'ftmyers': 'flyers', '24th': 'with', 'khon2': 'khong', 'chille': 'chill', '6lb': 'all', 'pepp': 'pep', 'cubao': 'cuban', 'utiva': 'diva', 'cxm': 'com', 'benito': 'bonito', '$hyzn': 'hymn', 'xrays': 'rays', 'deliveryusa': "delivery's", 'compant': 'company', 'natrona': 'national', '13f': 'of', 'uq': 'up', 'superpowerto': 'superpowers', 'nsw': 'now', 'shitball': 'spitball', 'ufh': 'uh', 'mombling': 'mumbling', 'tras': 'trash', 'h3ll': 'hell', 'tytlive': 'outlive', 'bheem': 'beem', 'sgg': 'egg', 'violen': 'violent', 'laudium': 'radium', 'sangat': 'sang', 'love10': 'love', 'findlay': 'finally', 'yooh': 'ooh', 'wahey': 'whey', 'ened': 'need', 'mkhize': 'maize', 'nonie': 'none', 'llsh': 'lash', 'instanews': 'instances', 'ksksks': 'kiosks', 'lhk': 'lik', 'maitree': 'maitre', 'coset': 'cost', 'ooooooo': 'oooooh', 'technolgoy': 'technology', 'duluth': 'dilute', 'kuala': 'koala', 'ab21': 'able', 'balers': 'bales', 'vegacity': 'veracity', 'dehydrators': 'dehydrates', '17c': 'doc', 'stirfry': 'starry', 'of11': 'of', 'haircare': 'airfare', 'teer': 'tear', 'waakye': 'wake', 'ekur': 'your', 'videos]': 'videos', 'wmfree': 'free', 'annunity': 'annuity', 'wsmall': 'small', 'gunged': 'gunned', 'pousal': 'spousal', 'preph': 'prep', 'm4': 'my', 'pagrocery': 'grocery', 'upworthy': 'unworthy', '35min': 'min', 'eustress': 'stress', 'sunsilk': 'sunil', '$chgg': 'chug', 'ceuase': 'cease', '5aday': 'day', '1x5': 'ex', 'metopolitan': 'metropolitan', 'takeaw': 'takea', 'workdays': 'workday', 'partwtf': 'partway', 'sfeeds': 'feeds', 'bokeh': 'both', 'fukn': 'fun', 'sfr': 'sir', 'esper': 'ester', 'tastest': 'tastes', 'paperboykd': 'paperboy', 'spethial': 'special', 'kasha': 'sasha', 'unloaders': 'unloaded', '70th': 'with', 'nuro2': 'nurse', 'bayan': 'banyan', 'fotosred': 'motored', 'paramus': 'pajamas', 'pharmcist': 'pharmacist', 'ktm': 'kim', 'alwan': 'alan', 'brius': 'brits', 'mainer': 'manner', 'bb8': 'be', 'atlstrong': 'armstrong', 'avvers': 'alvers', 'wyse': 'wise', 'savtakeout': 'stakeout', 'worldat': 'world', 'impac': 'impact', 'dilip': 'slip', 'bda': 'bad', 'orga': 'organ', 'saas': 'says', '22nd': 'and', 'usualky': 'usually', 'yowl': 'bowl', 'exorbitance': 'exorbitant', 'jenga': 'jenna', 'oca': 'ofa', 'verti': 'venti', 'fudx': 'fuck', 'tringa': 'tring', 'mf': 'of', 'dev]': 'dev', 'batya': 'baby', 'punishingly': 'punishing', 'tetris': 'tears', 'worldpay': 'worldly', 'wamu': 'ramu', 'nort': 'not', 'carbing': 'caring', 'onrain': 'train', 'canva': 'canvas', '3rd': 'ord', 'discoversd': 'discovered', 'gkp': 'gap', 'mostess': 'hostess', '28m': 'him', 'ismostly': 'mostly', 'rhine': 'shine', 'padnemic': 'pandemic', 'lazyday': 'payday', '$roku': 'broke', 'bagboys': 'busboys', 'gpt': 'get', 'rewatched': 'reached', 'mkts': 'mets', 'kasoa': 'pasta', 'comfortabl': 'comfortable', '7now': 'know', '4yo': 'yo', 'bearny': 'beardy', 'ceaser': 'cease', 'uneccessary': 'necessary', 'opoto': 'onto', 'stouffer': 'suffer', 'majestik': 'majestic', 'naro': 'narc', 'soca': 'soda', 'lovegit': 'loveit', 'yogo': 'yoga', 'singena': 'singing', 'uengage': 'engage', 'ghibli': 'chili', 'usda': 'usa', 'uuugh': 'tough', 'vmvpd': 'vivid', 'six60': 'six', 'zsr': 'zsa', 'humpday': 'humpty', 'whetheir': 'whether', 'pepperdine': 'peppermint', '1stop': 'stop', 'ril': 'rid', 'osco': 'osso', 'withdrawls': 'withdrawals', 'chonky': 'chunky', 'bertron': 'neutron', 'everyonein': 'everyone', 'sevbia': 'serbian', 'baral': 'banal', 'canaan': 'cancan', 'harborne': 'harbor', 'wvcs': 'was', 'serivice': 'service', 'qvq': 'que', 'nuissance': 'nuisance', 'yowsers': 'powers', 'aeicha': 'geisha', 'everettma': 'everett', 'toread': 'thread', 'grocerykw': 'grocery', 'agrizzi': 'grizz', 'elgrocer': 'grocer', 'dazz': 'jazz', 'ittt': 'itto', 'deboer': 'deeper', 'sante': 'sane', 'yardonna': 'madonna', 'planetone': 'plankton', 'cotuit': 'count', 'staiacasa': 'staircase', 'creativelysm': 'creatively', 'galmoy': 'galaxy', 'vikez': 'like', 'mncinc': 'mincing', 'protip': 'profit', 'wtaf': 'what', 'zea': 'sea', 'iinteresting': 'interesting', '15secs': 'secs', 'pret': 'pet', 'orhers': 'others', 'commerci': 'commerce', 'mrorm': 'from', 'homesite': 'homesick', 'napoing': 'napping', 'cipro': 'pro', 'musilie': 'muscle', 'cibo': 'ciao', 'gohoward': 'howard', 'onlh': 'only', 'ym': 'my', 'dti': 'di', 'tokyu': 'tokyo', 'dobetter': 'better', 'borger': 'border', 'coversations': 'conversations', 'loveno': 'love', 'reimburseme': 'reimburse', 'broooo': 'broom', 'electuon': 'election', 'grcry': 'cry', 'privledged': 'privileged', 'peroneal': 'perineal', 'oracom': 'oracle', 'zinemax': 'cinema', 'dhvani': 'bhavani', 'topix': 'topic', 'bubbie': 'bubble', 'ingenius': 'ingenious', 'imblessed': 'blessed', '39th': 'with', 'gyakie': 'goalie', 'aviva': 'viva', 'dho': 'do', 'darien': 'darken', 'noho': 'noo', 'indi': 'andi', 'withinn': 'within', 'kaune': 'kane', 'outkast': 'outcast', 'sangam': 'sang', 'dickbags': 'dirtbags', '$qnt': 'want', 'ghey': 'they', 'chiche': 'cliche', 'morta': 'mortal', '1town': 'town', 'shiplines': 'shillings', 'nbr': 'nor', 'arewa': 'area', 'weise': 'wise', 'gabbys': 'gabby', 'quickco': 'quick', 'jenner': 'tenner', 'fpd': 'fed', 'bleech': 'bleach', 'simping': 'sipping', 'palna': 'panna', 'pacekart': 'packard', 'vator': 'gator', 'necassary': 'necessary', 'lmart': 'smart', 'kika': 'kira', 'nappie': 'nappies', 'marae': 'mare', 'mta': 'ma', 'eest': 'best', 'jaxwe': 'jaw', 'pt1': 'pta', 'waltloo': 'wallow', 'disappointxx': 'disappointed', 'adhds': 'adds', 'sj': 'so', 'grocies': 'groceries', 'imfl': 'ifl', 'matke': 'make', 'pharmas': 'pharma', 'nicetry': 'nicely', 'tempu': 'tempt', 'cler': 'clear', 'fanboying': 'annoying', 'vancouverbc': 'vancouver', 'funish': 'finish', 'markups': 'markup', 'uthrough': 'through', 'selborne': 'seaborne', 'pantaloon': 'pantaloons', 'scoter': 'scooter', 'gaaon': 'again', 'corder': 'order', '5bd': 'and', 'statler': 'stapler', 'aretha': 'arethe', 'gopanthers': 'panthers', 'droppe': 'dropped', 'bcoolbox': 'toolbox', 'zdnet': 'zone', 'farwell': 'farewell', 'orices': 'prices', 'late]': 'late', 'bewell': 'befell', 'tenness': 'tennis', 'mayter': 'matter', 'nightlifer': 'nightlife', 'doughef': 'dough', 'chiro': 'chirp', 'rorys': 'rory', 'phdelivery': 'delivery', '34min': 'min', 'breadfor': 'bradford', 'waite': 'wait', 'wmt': 'wet', 'torquay': 'torque', 'happyyou': 'happyto', 'houndog': 'horndog', 'csa': 'sa', '$evcc': 'evac', '3mths': 'paths', '2july': 'july', 'populer': 'popular', 'ranier': 'ranger', 'sopore': 'spore', 'gatorades': 'gatorade', 'ghar': 'gear', 'confir': 'confirm', 'wilko': 'will', 'centene': 'center', 'romney': 'rodney', '6pk': 'up', 'mrw': 'mow', '55k': 'ask', 'sheduled': 'scheduled', 'barnard': 'barnyard', 'xm75': 'xmas', 'gipa': 'gina', 'spenser': 'spencer', 'farmer1st': 'farmers', 'getfunded': 'refunded', 'goram': 'gram', 'smse': 'me', 'navarre': 'navarro', 'colectiva': 'collective', 'floralis': 'floral', 'unpaired': 'repaired', 'vmcc': 'vice', 'notgreen': 'noreen', 'unsubscribe': 'subscribe', 'heccin': 'heroin', 'elderlys': 'elderly', 'fcking': 'fucking', 'citzon': 'citron', 'sheves': 'shelves', 'swmelb': 'smell', 'ps69': 'psi', 'manoa': 'manor', 'nzara': 'nara', 'woodtv': 'woods', 'icrecream': 'icecream', 'meo': 'me', 'altern': 'alter', 'giannis': 'gianni', 'uoft': 'soft', 'homegood': 'homeroom', 'wts': 'was', 'anxie': 'annie', 'lusk': 'luck', 'botchores': 'botches', 'dera': 'dear', 'cheesemaker': 'cheesecake', 'mtlb': 'male', 'spaza': 'spaz', 'allmost': 'almost', 'mommin': 'mommie', 'cths': 'the', 'ncid': 'acid', 'bazely': 'barely', 'comments]': 'comments', 'reseda': 'resent', 'belgrave': 'belgrade', 'cdo': 'do', 'greekfood': 'greenwood', 'actaully': 'actually', 'sg': 'so', 'oooor': 'ooooh', 'heyes': 'eyes', 'modal': 'model', 'spoe': 'spoke', 'cono': 'con', 'continuesly': 'continues', 'ml': 'my', 'eola': 'ella', 'grone': 'gone', 'habesha': 'habeas', 'creggan': 'reggae', 'm6': 'my', 'nabisco': 'naisho', '15e': 'the', 'dobbing': 'robbing', 'aapke': 'take', 'beatiful': 'beautiful', 'respense': 'response', '90p': 'up', 'noonish': 'foolish', 'potente': 'potent', 'sp': 'so', 'tranmission': 'transmission', 'rajnish': 'vanish', 'unb': 'un', 'ogforum': 'forum', 'niyo': 'nino', 'ygrocery': 'grocery', 'aps': 'as', 'loblaw': 'lola', 'expy': 'expo', 'ncose': 'nose', 'domla': 'doula', '28c': 'doc', 'sharam': 'sharma', 'opossums': 'possums', 'goodkids': 'goodies', 'againaa': 'against', 'phooone': 'phone', 'merchent': 'merchant', '4so': 'so', 'rutland': 'roland', 'vhs': 'vos', 'witth': 'with', 'pks': 'pas', 'servu': 'serve', 'cbdc': 'code', 'aproko': 'apron', 'hicky': 'ricky', 'triplex': 'triple', 'yta': 'ya', 'favorit': 'favorite', 'balti': 'baltic', 'bigone': 'bygone', 'riksha': 'rickshaw', 'hippa': 'hippy', 'meriau': 'serial', 'longrich': 'longish', 'repund': 'refund', 'bankcard': 'backyard', 'homesale': 'homemade', 'alch': 'arch', 'quaruntine': 'quarantine', 'nden': 'den', 'madates': 'mandates', 'riddin': 'ridden', 'wellin': 'welling', 'diversey': 'diverse', 'lcbo': 'lobo', 'frfree': 'free', 'pandem': 'tandem', 'miagao': 'maga', 'kinza': 'kinda', 'krenge': 'renge', 'fulham': 'pelham', 'almas': 'alias', 'mhz': 'miz', 'ofw': 'of', 'a8': 'a', 'islington': 'arlington', 'mertz': 'hertz', 'ifrc': 'if', 'hoousehold': 'household', 'timbits': 'tidbits', 'roni': 'ron', 'alco': 'also', 'forza': 'fora', 'birnie': 'birdie', 'arce': 'are', 'introd': 'intro', 'html': 'himl', 'cascais': 'carcass', 'theirself': 'herself', 'ciroc': 'croc', 'bankfoot': 'barefoot', 'nedpg': 'ned', '45k': 'ask', 'daddydoll': "daddy'll", 'stelline': 'telling', 'dtrumpf': 'trump', 'picki': 'pick', 'spray2': 'spray', 'pasay': 'pasty', 'pwr': 'per', 'precedings': 'preceding', 'amado': 'amador', 'tiredi': 'tired', '27yo': 'yo', 'annoucment': 'announcement', 'arkel': 'ariel', 'kante': 'kate', 'delievery': 'delivery', 'q7a1': 'quad', 'cnrapp': 'crap', 'washin': 'washing', 'dumating': 'dating', 'llb': 'lab', 'sgd': 'sad', 'reci': 'rec', 'schoogirl': 'schoolgirl', 'ifr': 'if', 'erickab': 'erica', 'meijers': 'members', 'theyoure': "they're", 'moralize': 'morality', 'nama': 'name', 'shabbos': 'shabby', 'hwb': 'hub', 'motorhour': 'motorhome', 'fintry': 'wintry', 'effiency': 'efficiency', 'mwals': 'meals', 'miguela': 'miguel', 'moter': 'mother', 'stronge': 'strong', 'rotis': 'rots', 'cleaneats': 'cleaners', 'ridicul': 'ridicule', 'e5': 'e', 'detol': 'detox', 'wt': 'it', 'intnl': 'intel', 'inen': 'inn', 'tng': 'tag', 'mandsaur': 'mandar', 'koranic': 'organic', 'gearlog': 'gearing', 'cutup': 'cut', 'lue': 'lie', 'fruitcar': 'fruitcake', 'implimenting': 'implementing', 'ideja': 'idea', 'rik': 'rid', 'rifarms': 'farms', 'adaption': 'adoption', 'baca': 'back', 'abcares': 'cares', 'sooper': 'sooner', 'yestrdy': 'destroy', 'ltd': 'led', 'bannon': 'cannon', 'foodshop': 'woodshop', 'isrequired': 'required', 'hunny': 'funny', 'supplues': 'supplies', 'googler': 'googled', 'daca': 'data', 'jba': 'ba', 'ggc': 'ggo', 'ninty': 'ninth', 'subcriptions': 'subscriptions', 'explai': 'explain', 'i4u': 'iru', 'clicklist': 'checklist', 'portlander': 'portland', 'dicier': 'dicker', 'csfs': 'csis', 'cvs': 'cos', 'horriable': 'horrible', 'beltline': 'beeline', 'humidex': 'humid', 'indica': 'indict', 'saame': 'same', 'pharmcy': 'pharmacy', 'coronanb': 'coronary', 'martijn': 'martian', '10yos': 'ryos', 'vulner': 'ruler', 'knk': 'ink', 'peo': 'per', 'leassons': 'lessons', 'frabelle': 'isabelle', '45a': 'a', 'gaurav': 'aura', 'i77': 'i', 'berlingo': 'berlin', 'probox': 'proof', 'network1': 'network', 'sanitisation': 'sanitation', 'nc': 'no', 'sfm': 'sum', 'bacolor': 'color', 'banos': 'banks', 'bylines': 'byline', 'henlow': 'fellow', 'opsec': 'open', 'kprofiles': 'profiles', 'zipp': 'zip', 'errold': 'errol', '8jan': 'jan', 'mb': 'my', 'hotwings': 'holdings', 'mcas': 'mas', 'toadult': 'adult', 'homeshoot': 'homeschool', 'jaybees': 'maybes', 'dahir': 'sahir', 'revpit': 'revolt', 'reubens': 'reuben', 'qftd': 'fed', 'catfood': 'atwood', 'southlake': 'southland', 'breadman': 'headman', 'swibag': 'swing', 'becausee': 'because', '10a': 'a', 'sanitised': 'sanitized', 'comming': 'coming', 'covirgin': 'virgin', 'everton': 'everyone', '5o': 'to', 'greengrocery': 'greengrocer', 'havers': 'haters', 'nopo': 'nope', '5rs': 'irs', 'schen': 'chen', 'stmaarten': 'smarten', 'newsusa': 'nessa', 'in2': 'in', 'randi': 'andi', 'humansies': 'humanoids', 'mendota': 'mendoza', 'eseng': 'even', 'imper': 'simper', 'tswindle': 'swindle', 'should]': 'should', 'vegetab': 'vegetable', 'prrequest': 'request', 'bryers': 'buyers', 'nornmal': 'normal', '2bn': 'ibn', 'sheggz': 'shaggy', 'deccesion': 'decision', 'amethia': 'amelia', 'jso': 'so', 'pmt': 'put', 'peacefuls': 'peaceful', 'stravino': 'starving', 'ndimi': 'dime', 'aquisition': 'acquisition', 'auth': 'auto', 'sustainble': 'sustainable', 'ordery': 'order', 'cspan': 'span', 'groat': 'great', 'wicart': 'cart', 'tomatillos': 'tomatillo', 'agreable': 'agreeable', 'ankit': 'andit', 'raine': 'raise', 'westerra': 'western', 'zunda': 'funda', 'yeaaa': 'years', 'availabiliy': 'availability', 'purveyour': 'purveyor', 'tallaght': 'taillight', 'thursda': 'thursday', 'gephardt': 'gerhardt', '30minutes': 'minutes', 'ajc': 'arc', 'ohca': 'orca', 'retrench': 'trench', 'beke': 'bike', 'shudnt': 'shunt', 'shaps': 'ships', 'anc': 'and', 'hksb': 'his', 'apprecating': 'appreciating', 'i75': 'i', 'presque': 'pressure', 'sprog': 'sprig', 'bigly': 'billy', 'pulak': 'kulak', 'robredo': 'robbed', 'sympa': 'spa', 'etailing': 'tailing', 'galati': 'galaxy', '$part': 'part', 'aglanta': 'atlanta', 'sberbank': 'burbank', 'simbisa': 'simba', 'ldpe': 'lope', 'wym': 'gym', 'tpd': 'tad', 'celeron': 'celery', 'cuppycakes': 'cupcakes', 'rolando': 'roland', 'applaudang': 'applauding', 'opry': 'pry', 'disgr': 'dig', 'stlcards': 'stewards', 'lindts': 'linds', 'trappin': 'trapping', 'numark': 'mark', 'coronasos': 'coronado', 'annotation': 'innovation', 'pepco': 'peace', 'albe': 'able', '75x': 'six', 'sullenly': 'suddenly', 'h3massage': 'massage', 'kiva': 'diva', 'fonded': 'founded', 'attn': 'atta', '$w': 'ow', 'princesa': 'princess', 'esthimated': 'estimated', 'wslmart': 'smart', 'visitdc': 'visit', 'towny': 'town', 'sufferingba': 'suffering', 'pikking': 'picking', 'walmrt': 'alert', 'colombine': 'columbine', 'clo': 'lo', 'day17': 'day', 'a1smm': 'assim', 'gennaro': 'gunnar', 'caremongers': 'warmongers', 'cravens': 'craves', 'erie': 'eric', 'duebto': 'debt', 'moro': 'more', 'meetha': 'meetcha', 'buccilli': 'bacilli', '$ebay': 'ebay', 'gratefu': 'grateful', 'grandboo': 'grandson', 'southenders': 'southerners', 'yli': 'li', 'kanjani': 'kanji', 'wwsb': 'was', '$xtnt': 'extent', 'appetito': 'appetite', 'cance': 'chance', 'slytherin': 'slithering', 'thugger': 'hugger', 'bole': 'hole', 'rao': 'ran', 'bioethics': 'biometrics', 'awau': 'away', 'eaak': 'eak', 'faught': 'caught', 'unsafer': 'unsafe', 'biznews': 'sinews', 'noshame': 'shame', 'ldmm': 'lame', 'repla': 'reply', 'strato': 'strata', 'pag': 'pay', '90your': 'your', 'bergman': 'began', 'qs': 'is', 'gaurd': 'guard', 'weee': 'were', 'niyang': 'nyang', 'sponser': 'sponsor', 'inderlied': 'underlined', 'mobillity': 'mobility', 'websit': 'website', '2df': 'edf', 'khealth': 'health', 'shoprit': 'short', 'sirr': 'sir', 'motar': 'motor', 'pgy1': 'pay', 'pagle': 'page', 'ocoee': 'come', 'tyourants': 'tyrants', 'ooonly': 'only', 'ouragous': 'courageous', 'bulacan': 'vulcan', 'kasumi': 'kabuki', 'tiffen': 'stiffen', 'domin8c': 'dominic', 'pandumbic': 'pandemic', 'forkstalk': 'forestall', 'bch': 'ich', 'motown': 'town', 'kaiya': 'kaya', 'upr': 'up', 'oaps': 'maps', 'guj': 'guy', 'alonetime': 'longtime', 'cds': 'cos', 'hebtoy': 'hefty', 'getitall': 'genitals', 'braddon': 'brandon', 'bejjur': 'beyour', 'unigate': 'unite', 'choos': 'choose', '35th': 'with', 'erocery': 'grocery', 'salars': 'salary', 'triprides': 'tripwires', 'esas': 'seas', 'gacha': 'dacha', 'df': 'of', 'adjustor': 'adjuster', 'accessable': 'accessible', 'crisco': 'frisco', 'hannath': 'hannah', 'leanonme': 'leanne', '1gl': 'all', 'ullai': 'ulla', 'anurag': 'aura', 'mcgrath': 'migrate', 'worsed': 'worked', 'deliv': 'deli', 'dron': 'drop', 'technic': 'technics', '10yours': 'yours', 'ettech': 'tech', 'yadezra': 'madeira', 'cbn': 'can', 'monkism': 'monks', 'hatefully': 'carefully', 'jdog': 'dog', 'dubnation': 'donation', 'oilsands': 'islands', 'assistances': 'assistance', 'friscotx': 'frisco', 'maskif': 'mask', 'votin': 'voting', 'impend': 'spend', 'kernow': 'know', 'aik': 'ask', 'lahat': 'that', 'quarantinng': 'quarantining', 'dmeocratic': 'democratic', 'epanding': 'expanding', 'yk': 'ya', 'family1st': "family's", 'apwu': 'apu', 'sbrown': 'brown', 'shangrila': 'sangria', 'sanny': 'sunny', 'halkirk': 'talkir', 'tristin': 'tristan', 'spitter': 'spatter', 'causeof': 'cause', 'houze': 'house', 'amizon': 'amazon', 'natsi': 'natsu', 'lotro': 'lotto', '30oz': 'coz', 'porc': 'port', 'foodhallers': 'footballers', 'i4': 'i', 'neiman': 'newman', 'hfk': 'of', 'shivaji': 'shiva', 'shur': 'shut', 'signaler': 'signaled', 'yousi': 'youse', 'kundu': 'kudu', 'snausages': 'sausages', 'enail': 'nail', 'bookarts': 'booklets', 'kaam': 'kama', 'retracement': 'replacement', 'trishaw': 'trisha', 'jcc5': 'jack', '4pt': 'apt', 'unfollowing': 'following', 'similair': 'similar', 'sain': 'said', 'sweatpants]': 'sweatpants', 'momm': 'mom', 'dawggg': 'dawg', 'whitecross': 'whiterose', 'indore': 'ignore', 'maryhill': 'marshall', 'sourdoughs': 'sourdough', 'quali': 'quail', 'horderers': 'murderers', '26ish': 'wish', 'shemagh': 'sheath', 'weedman': 'headman', 'atwoli': 'awol', 'commnet': 'comment', 'hoagh': 'high', 'irrelevancy': 'irrelevant', 'shiet': 'shit', 'elin': 'ein', 'gendered': 'rendered', 'stlb': 'stab', 'naam': 'nam', 'mirchi': 'michi', 'amz': 'am', 'earthday': 'birthday', 'kalaki': 'salami', 'weslie': 'leslie', 'drma': 'drama', 'dma': 'dna', 'asmi': 'ami', 'tpch': 'tech', 'intially': 'initially', 'skyland': 'skyline', 'moneyball': "money'll", 'bruhns': 'bruins', 'jeffs': 'jeff', 'zigam': 'cigar', 'ssturday': 'saturday', 'culinarily': 'culinary', 'seekonk': 'seeking', '7p': 'up', 'dzine9': 'dine', 'youcount': 'youout', 'thtu': 'thou', 'wowscript': 'conscript', 'fier': 'fire', 'bastert': 'baster', 'covifd': 'couid', 'clitch': 'clutch', 'winez': 'wine', 'cockburn': 'cocksure', 'wipedd': 'wiped', 'socceroo': 'soccer', 'flamin': 'flaming', 'aapko': 'wacko', 'toohot': 'tooth', '$nio': 'no', 'whomst': "who's", 'washingto': 'washington', 'unhinges': 'unhinged', 'rioc': 'riot', 'yourktown': 'yorktown', '2run': 'run', '9nly': 'only', '19c': 'doc', 'moma': 'mom', 'unmute': 'unite', 'centrum': 'central', 'kweats': 'sweats', 'chahte': 'chaste', 'wegmas': 'vegas', 'klindt': 'kind', 'elmwood': 'elwood', 'covax': 'coax', 'worthing': 'nothing', 'techology': 'technology', 'nupur': 'super', 'crimnal': 'criminal', 'pliz': 'liz', 'kaafir': 'kaffir', '$s': 'is', 'glenmore': 'lenore', 'brekko': 'break', 'knownews': 'knowses', 'ub': 'up', 'subasian': 'sebastian', 'jorhat': 'jordan', 'adil': 'ail', 'alltogether': 'altogether', 'crowdf': 'crowd', 'dehydrator': 'dehydrated', '6min': 'min', 'vinaka': 'naka', 'ology': 'glory', 'ashland': 'island', 'alpena': 'alien', 'ecoli': 'coli', 'manmaw': 'mama', 'gluetin': 'gluten', 'mapusa': 'maps', 'ignola': 'ignore', 'baychester': 'manchester', 'anoda': 'anda', 'mvps': 'maps', 'raziel': 'rail', '10yo': 'yo', 'werks': 'weeks', 'svs': 'sis', 'grocerica': 'groceries', 'inara': 'nara', 'k': 'i', 'mishall': 'shall', 'seriesb': 'series', 'nongshim': 'dongshi', 'delicata': 'delicate', 'cm21': 'cme', 'furu': 'fur', 'rebuildi': 'rebuild', 'detemined': 'determined', 'onroute': 'enroute', 'restocks': 'restock', 'womxn': 'woman', 'pehle': 'pele', 'pengy': 'penny', 'worcester': 'forester', 'gids': 'kids', 'mssge': 'message', 'bonker': 'bunker', 'sw': 'so', 'ls25': 'lsee', 'oakgrove': 'mangrove', '08th': 'with', 'wipipo': 'wiping', 'haydon': 'hayden', 'xb': 'xu', 'bharara': 'barbara', 'rdr2': 'dry', 'sizs': 'size', 'authoties': 'authorities', 'oneworld': 'offworld', 'hormel': 'home', '9ja': 'ja', 'scotus': 'scouts', 'boldt': 'bold', 'sccos': 'cos', 'adj': 'add', 'gummit': 'summit', 'colwell': 'powell', 'raleighnc': 'raleigh', 'avionos': 'avionics', 'tuesd': 'these', 'katerina': 'katrina', 'siloed': 'silted', 'populism': 'populist', 'duduk': 'dude', 'getitqa': 'getta', 'spirt': 'spirit', 'suprem': 'supreme', 'attem': 'atter', 'weald': 'weld', '11m': 'him', 'sangha': 'sang', 'seadubs': 'seamus', 'pottying': 'potting', 'regos': 'egos', 'juuling': 'pulling', 'frishes': 'fishes', 'freschetta': 'bruschetta', 'kait': 'wait', 'rabello': 'bello', 'weeka': 'week', 'indue': 'induce', 'acq': 'act', 'farmlink': 'farming', 'passiton': 'passion', 'landyn': 'landon', 'curta': 'curt', 'amange': 'mange', 'fleghmy': 'fleshy', 'moji': 'mojo', 'mymontreal': 'montreal', 'dw': 'do', 'bikeshop': 'bishop', 'cllick': 'click', 'getorganised': 'reorganised', 'ema': 'ma', 'kupang': 'kung', '22july': 'july', 'signedup': 'signed', '13pies': 'copies', 'gta': 'ga', 'desses': 'dresses', 'teret': 'beret', 'elbel': 'else', 'ila': 'la', 'mda': 'mad', 'bajaree': 'bajake', 'lashaye': 'sashay', 'hotcake': 'hotcakes', 'nopales': 'noodles', 'pitu': 'pity', 'lovelife': 'lowlife', 'progr4ess': 'progress', 'grabmart': 'grammar', 'assi': 'ass', 'croma': 'coma', 'kd': 'kid', 'cheppina': 'chopping', 'bexar': 'bear', 'fulfillm': 'fulfill', '46th': 'with', 'bpl': 'bel', 'yawa': 'yawn', 'tumy': 'tummy', 'pandago': 'bandage', 'cudz': 'cuz', 'vikki': 'nikki', 'crohn': 'crown', 'sustainabil': 'sustainable', 'cere': 'were', 'dkt': 'dot', 'mfer': 'fer', 'grsry': 'glory', 'firat': 'first', 'highered': 'higher', 'doglover': 'rollover', 'mydin': 'main', 'sfty': 'sty', 'good2go': 'goodto', 'mtga': 'mega', 'racc': 'race', 'leduc': 'led', 'periodt': 'period', 'peasynt': 'peasant', 'stabds': 'stands', 'darkmode': 'darmody', 'sahod': 'shod', 'asspained': 'assailed', 'primertech': 'primatech', 'wx': 'we', 'unsweet': 'sweet', '5daily': 'daily', 'spitballin': 'spitballing', 'willmar': 'pillar', 'freeride': 'freebie', 'endsars': 'endears', 'bisht': 'bist', 'mps': 'maps', 'beff': 'beef', '87th': 'with', 'amazongo': 'amazing', 'didabied': 'disabled', 'aznumbers': 'numbers', 'disesase': 'disease', 'antagony': 'anthony', 'moreland': 'homeland', 'primerib': 'primer', 'djlifestyle': 'lifestyle', 'artisanos': 'artisans', 'aate': 'hate', 'cueva': 'nueva', 'swk': 'suk', 'associatn': 'associate', 'licherally': 'literally', 'feady': 'ready', 'cheppu': 'cheap', 'tgfm': 'them', 'huron': 'heron', 'gwprice': 'price', 'stauffers': 'staffers', 'boyfie': 'boogie', 'moderna': 'modern', 'cupcak': 'cupcake', 'thst': 'that', 'delevered': 'delivered', 'masalas': 'masala', 'bx': 'be', 'fwn': 'own', 'shitttn': 'shitty', 'specialt': 'special', 'cashi': 'cash', 'competera': 'computer', 'shitass': "shit's", 'newsie50': 'newsie', 'daw': 'day', 'salinas': 'savings', 'surt': 'sure', 'cafeole': 'cajole', 'unempathetic': 'unsympathetic', 'groceries]': 'groceries', 'vission': 'mission', 'overtip': 'overtime', 'composable': 'comparable', 'itsok': 'took', 'lrt': 'let', 'lagov': 'lago', 'gema': 'gem', 'wakah': 'wakan', 'sying': 'saying', 'tv': 'to', 'andhre': 'andre', 'mlm': 'mom', 'invito': 'invite', 'takiab': 'taking', 'batiste': 'baptiste', 'nwindiana': 'indiana', 'babus': 'babes', 'kneeland': 'kneeling', 'berliner': 'berliners', 'carranza': 'carrozza', 'madina': 'marina', 'maveninc': 'ravening', 'snitchtag': 'snitching', 'bringingt': 'bringing', 'scflood': 'flood', 'khqatm': 'khat', '20minutes': 'minutes', 'pineda': 'pined', 'jitna': 'jinx', 'opelika': 'apelike', 'joshi': 'josh', 'carefirst': 'careerist', 'ftse': 'fuse', 'narissa': 'marissa', 'arceus': 'access', 'tomchei': 'touched', 'gummis': 'gummi', 'kawasan': 'wasan', 'zimboz': 'limbo', 'complainy': 'complain', 'frypan': 'ryan', 'xalrock': 'warlock', 'santitize': 'sanitize', 'bjorns': 'bjorn', 'relaunches': 'launches', 'dinnerr': 'dinner', 'cliff30': 'cliff', 'groffer': 'proffer', 'y6': 'ya', 'torygraph': 'polygraph', 'pichana': 'piccata', 'roboshop': 'robocop', 'gongyu': 'gongfu', 'ptaceks': 'places', 'dhhr': 'her', 'camb': 'came', 'unpackaged': 'unpacked', 'togeather': 'together', 'biostat': 'bioscan', 'buong': 'bong', 'ahti': 'anti', 'hayi': 'hay', 'wearhouse': 'warehouse', 'magats': "mama's", 'f2b': 'fob', 'decompressor': 'decompression', 'chancla': 'chance', '10bands': 'bands', '2am': 'am', 'hypercritical': 'hypocritical', 'werk': 'were', 'mgcq': 'much', 'hobson': 'dobson', 'wahh': 'wash', 'mcsally': 'mcnally', 'yellowbone': 'yellowstone', 'avenidas': 'agendas', 'cratering': 'catering', 'fao': 'far', 'ritoulouse': 'toulouse', 'profita': 'profit', 'kringla': 'kingly', 'mugabe': 'mutate', 'deliveryfee': 'deliverymen', 'unpackaging': 'unpacking', 'rtg': 'rig', 'livenow': 'liven', 'lga': 'la', 'ntwana': 'nana', 'ciddi': 'kiddie', 'topup': 'top', 'stoovo': 'stood', 'profitero': 'profited', 'godrej': 'gore', 'rosi': 'rose', 'whatajoke': 'thatjoke', 'adrs': 'adds', 'shisa': 'shiva', 'normalish': 'normality', 'shypt': 'shut', 'nne': 'one', '$lixt': 'list', 'redmondwa': 'redmond', '48mins': 'mins', 'aevs': 'revs', 'tsietsi': 'tsetse', 'morni': 'morn', 'delizza': 'eliza', 'ca$h': 'cash', 'carbuzz': 'abuzz', 'mks': 'mes', 'apd': 'and', 'coelic': 'colic', 'rebuy': 'rebut', 'churner': 'churned', 'puc': 'put', 'ajnala': 'anal', 'kena': 'ken', 'jemima': 'jemma', 'freecash': 'freebase', 'cheersva': 'cheers', 'fvck': 'fuck', 'regi': 'reg', 'nys': 'nos', 'hdfc': 'doc', 'bezos': 'bozos', 'blerg': 'berg', 'ambitiously': 'ambitious', 'sayy': 'say', 'btba': 'baba', 'zie': 'die', 'jwr': 'jar', 'marlbo': 'marlo', 'natively': 'naively', 'helpslo': 'hello', 'rndabt': 'redact', 'unfortunatly': 'unfortunately', 'dwblog': 'blog', 'aine': 'fine', 'doorstep4': 'doorstep', 'saijan': 'saiyan', 'warbreck': 'warrick', 'coronask': 'coronary', 'nikko': 'nikki', '$mod': 'mod', 'makayla': 'kayla', 'marlen': 'marlin', 'yshop': 'shop', 'hermitous': 'hermits', 'boros': 'bores', 'liefer': 'lifer', 'ez10': 'ezra', 'grofer': 'grocer', 'kaitlynn': 'caitlyn', 'bhaiya': 'bhaiyya', 'gorceires': 'groceries', 'starbuuck': 'starbuck', 'unalive': 'alive', 'nifty50': 'nifty', 'aggregator': 'aggregate', 'especiall': 'especially', 'antivax': 'anthrax', 'kline': 'line', 'chaar': 'chair', 'okla': 'okra', 'hvnt': 'hunt', 'rectifes': 'receives', '65s': 'is', 'santized': 'sanitized', '55yours': 'yours', 'humanless': 'humorless', 'neighborfood': 'neighborhood', 'rsd': 'red', 'admi': 'admit', 'phila': 'phil', 'eka': 'era', 'mbbs': 'mobs', 'staycation': 'starvation', 'ahoys': 'ahoy', 'gringley': 'pringles', 'shamong': 'shaming', 'tengah': 'tenga', 'dorp': 'drop', 'lovenhs': 'loves', 'deling': 'dealing', 'chooseday': 'choosed', 'gurrll': 'grill', 'ksltv': 'kilt', 'dekho': 'echo', 'ignoranceof': 'ignorance', 'reinstalling': 'installing', 'nth': 'nah', 'herst': 'hers', 'slty': 'slay', 'fokes': 'jokes', 'grocreries': 'groceries', 'kulai': 'kurai', 'tshirt': 'shirt', 'bisouv': 'bison', 'coldd': 'cold', 'unecessary': 'necessary', 'samtech': 'caltech', 'keye': 'eye', 'bru': 'bra', 'ellum': 'vellum', 'o\\': 'of', 'hadzaad': 'hazard', 'teledoc': 'telecom', 'heyhey': 'heyday', 'wives\\g': 'wives', 'accessibly': 'accessible', '89yo': 'yo', 'stjohns': 'johns', 'macleans': 'cleans', 'conect': 'connect', 'conveyour': 'conveyor', 'mofongo': 'mongo', 'aro': 'are', 'culley': 'pulley', 'millenia': 'millennia', 'uncrowded': 'uncrowned', 'gmt': 'get', 'gunding': 'funding', 'dtes': 'does', 'bringbring': 'bringing', 'npi': 'ni', 'shavuot': 'shallot', 'sclogic': 'logic', 'susmit': 'submit', 'baingan': 'banyan', 'astyn': 'aston', '$jamn': 'damn', 'broni': 'bronc', 'churchin': 'church', 'unman': 'gunman', 'mataram': 'matata', 'bweakfast': 'breakfast', 'oshawa': 'shawl', 'decilous': 'devious', 'waybill': "way'll", 'phontinent': 'continent', 'tiyad': 'tired', 'mistranslate': 'mistranslated', 'not]': 'not', 'letterboxes': 'letterbox', '35mil': 'mil', 'remap': 'reap', 'mascs': 'mass', 'wdam7': 'damn', 'pakai': 'paki', 'regualr': 'regular', 'kariyana': 'mariana', 'newcart': 'newscast', 'traeger': 'trager', 'mbh': 'mah', 'danlos': 'dances', 'nearme': 'name', 'ecija': 'cia', 'kwetu': 'wet', 'waiveinc': 'waiving', 'peldano': 'pendant', 'awonderfully': 'wonderfully', 'ingham': 'gingham', 'inlaw': 'inlay', 'theiugh': 'through', '30second': 'second', 'cosmetically': 'cosmically', 'sesor': 'sensor', 'unticked': 'nicked', 'hatians': 'haitians', 'kasana': 'katana', 'golocal': 'local', 'thiago': 'thing', 'empathically': 'emphatically', 'okwe': 'owe', 'detangling': 'dangling', 'oksee': 'see', 'gege': 'gene', 'yo7': 'you', 'ananas': 'bananas', 'difficukt': 'difficult', 'whatspp': 'whatsapp', 'neri': 'nerd', 'baje': 'base', 'serps': 'perps', 'imperfe': 'imperfect', 'avoidin': 'avoiding', 'araroa': 'arrow', 'sbh': 'suh', '50f': 'of', 'kenapa': 'napa', 'cyourillic': 'cyrillic', '40min': 'min', '7hrs': 'this', 'mfw': 'mow', 'cvproviders': 'providers', 'grafana': 'grahna', 'orderoer': 'ordered', 'in]': 'in', 'dlae': 'dale', 'fuckery': 'fucker', 'cannin': 'cannon', 'dood': 'good', 'lockedin': 'locked', 'f6s': 'fas', 'gandhiji': 'gandhi', 'benguet': 'banquet', 'ngkin': 'again', 'n10': 'not', 'bahah': 'ahah', 'uwa': 'wa', 'katella': 'patella', 'geesh': 'geese', 'scmr': 'scar', 'ktonline': 'online', 'mmchat': 'chat', 'lynnette': 'lynette', 'sazon': 'salon', 'deca': 'deck', 'volun': 'value', 'produc': 'produce', 'sout': 'out', 'msm': 'mom', 'harol': 'carol', 'regulati': 'regulate', 'fucjin': 'fuckin', 'worli': 'world', 'lyoura': 'your', 'gorrillas': 'gorillas', 'shimpi': 'ship', 'kobu': 'koku', 'tarrant': 'warrant', 'smlj': 'small', 'ates': 'ate', 'floriduhs': "florida's", 'blenkett': 'blanket', 'epfl': 'evil', 'fortelling': 'foretelling', 'kwawesome': 'awesome', 'robocar': 'robocop', 'excruciate': 'cruciate', 'bingham': 'gingham', 'pumpimg': 'pumping', 'wknd': 'wind', 'seperate': 'separate', '20p': 'up', 'refineri': 'refinery', 'forsee': 'foresee', 'blugh': 'blush', '87your': 'your', 'undr': 'under', 'juanito': 'juanita', 'decongesting': 'decongestant', 'peloton': 'photon', 'ucaaz': 'ucla', 'store2k': 'store', 'crisp]': 'crisp', 'candolim': 'cannoli', 'rnib': 'rib', 'myitaji': 'mitai', 'tgj': 'taj', 'bonsu': 'bonus', 'snicky': 'sticky', 'coronasl': 'coronal', 'plexi': 'lexi', 'aaargh': 'argh', 'asmr': 'as', 'starwars': 'starters', 'favs': 'fans', 'welly': 'well', 'cheesey': 'cheese', 'kiranag': 'kraang', 'standee': 'stand', 'mennonite': 'mennonites', 'richies': 'riches', 'ioe': 'ice', 'engels': 'angels', 'girta': 'girth', 'unvaxed': 'untaxed', 'ballyhooed': 'ballyhoo', 'cpg': 'cog', 'lmnt': 'lent', 'ramune': 'ramone', 'mr': 'my', 'ps3k': 'ask', 'vhcle': 'whole', 'boston25': 'boston', 'emei': 'mei', 'uyazi': 'nazi', 'schulz': 'schultz', 'conroe': 'monroe', 'diageo': 'dialed', 'thando': 'than', 'readjustments': 'readjustment', 'refusd': 'refuse', 'spanglish': 'spanish', 'googleit': 'googled', 'greenloop': 'greenroom', 'petfood': 'seafood', 'food4u': 'food', 'nonzero': 'montero', 'woolies': 'coolies', 'gambier': 'gambler', 'tking': 'thing', 'mcmurray': 'murray', 'crofts': 'crafts', 'wooaah': 'hookah', 'huarache': 'headache', 'itcd': "it'd", 'stanforder': 'stanford', 'chary': 'charm', 'jav': 'jam', 'ohioan': 'onion', 'therm': 'there', 'mrscrunch': 'scrunch', 'kesslers': 'kessler', 'levie': 'levee', 'herriot': 'herring', 'cartfuls': 'careful', 'bhikh': 'which', 'pregnany': 'pregnant', 'moneyno': 'moneyto', 'pepperidge': 'peppering', 'keanu': 'mean', 'uncategorized': 'categorized', 'wth': 'with', 'pittin': 'pitting', '$aapl': 'papal', 'edobo': 'doo', 'somemore': 'someone', 'homesome': 'lonesome', 'fridgeyes': 'fridges', 'appealin': 'appealing', '20ma': 'mama', 'hybris': 'hybrid', 'diainfect': 'disinfect', '24hr': 'sehr', 'gfoh': 'for', 'ankur': 'anger', 'angryy': 'angry', 'myalgic': 'magic', 'irv': 'iv', 'deliberar': 'deliberate', 'assortments': 'assortment', '15l': 'all', '48am': 'am', 'sutherland': 'motherland', 'fashow': 'show', 'acelyn': 'evelyn', 'holery': 'holey', 'mossing': 'missing', 'preferre': 'preferred', 'goldbears': 'goldberg', 'recd': 'read', 'ct6': 'ctu', 'sqs': 'sis', 'zydus': 'yous', 'uninstalled': 'installed', 'wss': 'was', 'piyush': 'push', 'spinich': 'spinach', 'evar': 'ever', 'ulta': 'ultra', 'ucp': 'up', '$grab': 'grab', 'walmert': 'alert', 'edeal': 'deal', 'baml': 'ball', 'online]': 'online', 'haddon': 'hadron', 'skillman': 'killian', 'pharamacy': 'pharmacy', 'excitement54': 'excitement', 'valsad': 'salad', 'countywide': 'countrywide', 'latr': 'late', 'dfl': 'del', 'samagri': 'safari', 'customise': 'customize', 'bux': 'but', 'kalyan': 'alan', 'jyada': 'yada', 'ohnoes': 'ones', 'l4': 'la', 'mealie': 'meanie', 'suppliments': 'supplements', 'pantries': 'panties', 'sarms': 'arms', 'fint': 'find', 'publ': 'pull', 'naroda': 'aroma', 'wesome': 'awesome', 'julo': 'judo', 'ramlal': 'ramal', 'lber': 'uber', 'siyanda': 'miranda', 'opition': 'opinion', 'chapbook': 'scrapbook', 'connage': 'tonnage', 'igbo': 'go', 'dorr': 'door', '$door': 'door', 'pesumably': 'presumably', 'fark': 'far', 'utest': 'test', 'kaiden': 'maiden', 'berr': 'bear', 'pathan': 'nathan', 'modesto': 'modest', 'tabacky': 'tacky', 'swkentucky': 'kentucky', 'skksjs': 'skeksis', 'telework': 'teleport', 'pna': 'dna', 'unfreaking': 'freaking', 'fokken': 'token', 'crudite': 'erudite', 'tierea': 'tierra', 'foodnl': 'food', 'mallis': 'malls', 'inpolitics': 'politics', 'creperies': 'breweries', 'rabba': 'rabbi', 'wandsworth': 'wordsworth', 'joyand': 'joan', 'davison': 'division', 'bodo': 'body', 'samem': 'same', 'chicopee': 'chickpea', '36h': 'oh', 'sinlge': 'single', 'proverbially': 'proverbial', 'pokestop': 'postop', 'zgz': 'go', 'landlorder': 'landlord', 'longla': 'long', 'unreviewed': 'reviewed', 'friskies': 'frisked', 'frappe': 'trapped', 'tumblr': 'tumble', 'thrumy': 'truly', 'mychores': 'chores', 'viv': 'via', 'danilo': 'daily', 'voluntered': 'volunteered', 'grrr': 'girl', 'brocku': 'brock', 'chockie': 'chickie', 'femdom': 'freedom', 'olamoney': 'ofmoney', 'masterchefau': 'masterchef', 'pca': 'pa', 'usuall': 'usually', 'gosok': 'gook', 'machacha': 'muchacha', 'esl': 'el', 'rosemarie': 'rosemary', 'yogesh': 'yogis', 'indiacares': 'indicates', 'strada': 'strata', 'bluebash': 'bluefish', 'hkmama': 'mama', 'privile': 'private', 'texason': 'reason', 'nesrly': 'nearly', 'jingping': 'jingling', 'tesc': 'test', 'nobread': 'bread', 'technode': 'techno', 'iestaff': 'staff', 'assential': 'essential', 'aoclist': 'vocalist', 'gratitudemy': 'gratitude', 'ewbor': 'labor', 'uughh': 'ought', 'changlin': 'changing', 'amazi': 'amaze', 'snackx': 'snack', 'jgmaa': 'jamal', 'tbose': 'those', 'fanci': 'fancy', 'housekeep': 'housekeeper', 'bagge': 'badge', 'smol': 'smog', 'us1': 'us', 'nfts': 'nuts', 'ckers': 'akers', 'loqation': 'location', 'pissup': 'pissed', 'ecart': 'cart', 'abdl': 'abel', 'honeybun': 'honeybunny', 'daalne': 'dale', 'spudshed': 'pushed', '$xrp': 'warp', 'tth': 'ith', 'eeemm': 'seems', 'houseware': 'housewares', 'thing2': 'thing', '5km': 'him', 'gaum': 'gum', 'kisii': 'kiss', 'outside]': 'outside', 'gigabit': 'gigabyte', 'vod': 'god', 'growingly': 'growing', 'wakanda': 'amanda', 'borong': 'boring', 'shittin': 'shitting', 'fundraise': 'fundraiser', 'blas': 'blast', 'tmac': 'mac', 'leporem': 'lenore', 'iije': 'iike', 'pawar': 'pasar', 'velveta': 'velvet', 'wealways': 'always', 'npci': 'nice', 'dreg': 'drug', 'digitise': 'digitize', 'sewall': 'shall', 'mrmango': 'mango', 'bluddy': 'buddy', 'vric': 'vic', 'apols': 'apply', 'muka': 'yuka', 'yo8': 'you', 'mitcham': 'mitch', 'bt': 'it', 'nubry': 'hurry', '34weeks': 'weeks', 'ostomy': 'story', 'huis': 'his', 'sheeh': 'sheep', 'cgs': 'cos', 'diy': 'did', 'heh]': 'heh', 'offoading': 'offloading', 'pamadot': 'paradox', 'antifascists': 'antifascist', 'prwpaid': 'prepaid', 'bibl': 'bill', 'ravn': 'ran', 'aksh': 'ash', 'myjum': 'mum', 'rnas': 'nas', 'ms': 'is', 'pau': 'pay', 'ecosystgem': 'ecosystem', 'cravy': 'crazy', 'tayo': 'taco', 'minu': 'mind', 'lugaw': 'lugar', 'kiwanis': 'kiwis', 'loveve': 'love', 'marii': 'maria', 'edginess': 'engines', 'tomomorrow': 'tomorrow', 'manchin': 'machine', 'rafa': 'raft', 'notime': 'notice', 'svp': 'sip', 'rewa': 'rea', 'midcoast': 'miscast', 'entici': 'entice', 'wheelc': 'wheel', 'ballito': 'ballet', 'bachat': 'chat', 'ued': 'used', 'waze': 'wake', 'localism': 'locals', 'whol': 'who', 'rtrt': 'try', 'washingt': 'washing', 'weareone': 'warzone', 'businessline': 'businesslike', 'hellride': 'hillside', 'local44': 'local', 'gunting': 'hunting', 'eh49': 'eh', 'americone': 'american', '90m': 'him', 'absolutelly': 'absolutely', 'begeek': 'begged', 'us60': 'us', 'ndsm': 'nose', 'cov': 'cop', 'frui': 'fruit', 'bcksi': 'bucks', 'hittin': 'hitting', 'maint': 'main', 'sanitz': 'sanity', 'sonko': 'sono', 'econom': 'economy', 'foodily': 'doodily', 'koosh': 'loose', 'informativo': 'informative', 'enviously': 'obviously', 'foward': 'forward', 'catrose': 'arose', 'pcm': 'pum', 'freshmeat': 'freshman', 'topz': 'top', 'weitzen': 'written', 'tamika': 'stamina', 'buuf': 'buff', 'batho': 'bath', 'soopa': 'stop', 'stefani': 'stefan', 'wisma': 'wilma', 'kleiner': 'kleine', '34th': 'with', 'ravitz': 'gravity', 'icmyi': 'icy', 'fbg': 'fog', 'madks': 'marks', 'cheapy': 'cheap', 'bymistake': 'mistake', 'fav4': 'fava', 'swanson': 'season', 'covhour': 'colour', 'stroud': 'shroud', 'wolseley': 'wolsey', 'kerela': 'merely', 'lockdiwn': 'lockdown', 'volunt': 'count', 'stutton': 'sutton', 'mealz': 'meal', 'hh': 'he', 'starmall': 'starman', '$mis': 'mis', 'hogger': 'jogger', 'hoofin': 'hoofing', 'kaanum': 'magnum', 'rottnest': 'rottenest', 'pricelist': 'priceless', 'cussin': 'cussing', 'stacart': 'start', 'tgt': 'tit', 'anyssa': 'alyssa', 'exxaro': 'xaro', 'bioto': 'into', 'apro': 'pro', 'attlee': 'battle', 'govout': 'devout', 'bandi': 'band', 'totum': 'totem', 'polit': 'polite', 'loftin': 'often', 'walmary': 'salary', 'stinkin': 'stinking', 'rurally': 'really', 'shapshap': 'shipshape', 'mwaura': 'maura', 'midwood': 'redwood', 'girds': 'girls', '3kg': 'ekg', 'lomond': 'lemond', 'okthen': 'then', 'gweru': 'were', 'techbro': 'techno', 'foodfail': 'footfall', 'hlo': 'ho', 'meatz': 'meat', 'mtd': 'mad', 'iconcierge': 'concierge', 'connectx': 'connect', 'avocet': 'avowed', 'welland': 'wetland', 'slamy': 'slam', 'yds': 'yes', 'dstv': 'date', 'a66': 'a', 'frd': 'fed', 'yo83': 'you', 'pto': 'to', 'goins': 'going', 'crampy': 'crappy', 'fia': 'fix', 'chkn': 'chin', 'telluride': 'telluric', 'vallejo': 'valley', 'onlie': 'online', 'teasy': 'easy', 'ctaop': 'stop', 'krueger': 'crueler', 'avaable': 'available', 'uotdrive': 'outdrive', 'gratitudert': 'gratitude', 'lovehim': 'loveit', '92nd': 'and', 'strandard': 'standard', 'foulkes': 'folks', 'enthusastic': 'enthusiastic', 'sonson': 'soon', 'mozzas': 'pizzas', 'korma': 'karma', 'dexman': 'german', 'satya': 'satyr', 'mnpolitics': 'politics', 'ajutwa': 'ajumma', 'bata': 'data', 'intransit': 'transit', '$bynd': 'band', 'ggcin': 'again', 'anxiousso': 'anxious', 'trustingly': 'trusting', 'qrocery': 'grocery', 'valoyi': 'valor', '$srts': 'parts', 'bby': 'by', 'konner': 'donner', 'michelob': 'michelle', 'mfcs': 'mics', 'reaquainted': 'reacquainted', 'sundell': 'unwell', 'eastman': 'eastern', 'cstores': 'stores', 'griggs': 'briggs', 'olworths': 'worths', 'lightstick': 'nightstick', 'aana': 'anna', '2lt': 'alt', 'wpes': 'apes', 'tbf': 'tbe', 'techinfo': 'techno', 'nongena': 'ningen', 'matbus': 'mates', 'physcis': 'physics', 'getmor': 'meteor', 'dday': 'day', 'gerund': 'ground', 'labneh': 'launch', 'trev': 'tree', 'capra': 'cara', 'luxery': 'luxury', '6july': 'july', 'hrtech': 'tech', 'buhut': 'but', 'rejoinder': 'reminder', '1stresp': 'stress', 'jayce': 'jace', 'bidn': 'bid', 'pitstop': 'pistol', 'bestprice': 'beatrice', 'tasspes': 'tastes', 'asn': 'as', 'mawr': 'mar', 'cireng': 'firing', 'laravel': 'travel', 'pipin': 'piping', 'closethis': 'closets', 'kuroko': 'koko', 'rpund': 'round', 'nikhil': 'nikki', 'kame': 'came', 'growbig': 'growing', 'expec': 'expect', 'iits': 'its', 'feelimg': 'feeling', 'seton': 'sexton', 'ckin': 'skin', 'kwyk': 'kick', 'malam': 'madam', 'jessup': 'jesus', 'flatpack': 'flapjack', 'noah19': 'noah', 'foodatuff': 'foodstuff', 'ketu': 'get', 'arseload': 'reload', 'mmgt': 'must', 'sccorona': 'corona', 'woodworker': 'woodwork', 'estupida': 'stupid', 'electriccars': 'electricians', 'kob4': 'kobe', 'rocklin': 'rockin', 'ootd': 'oot', 'cokies': 'copies', 'oper': 'over', 'hsn': 'hen', 'greyzone': 'rezone', 'leanne]': 'leanne', 'elearns': 'learns', 'suprmarkt': 'supermarket', 'vak': 'van', 'shopcut': 'shout', 'wtvd': 'word', 'hareli': 'harelip', 'runtz': 'runt', 'truc': 'true', 'terrifie': 'terrific', '1mc': 'my', 'freecycle': 'recycle', 'aaaaa': 'anata', 'falli': 'fall', 'lerman': 'german', 'szn': 'son', 'areawise': 'realise', 'interlocutor': 'interlocutory', 'overphone': 'overdone', 'patition': 'petition', 'targeted]': 'targeted', 'tbone': 'bone', 'iger': 'tiger', 'custies': 'cuties', 'doj': 'do', 'sofl': 'soul', 'f3': 'fu', 'dhow': 'how', 'raisethey': 'raiseth', 'footech': 'biotech', 'corrola': 'corral', 'pms': 'pas', 'sxm': 'sum', 'okkji': 'onaji', 'uas': 'was', 'survice': 'service', 'gramys': 'grams', 'exemptions]': 'exemptions', 'separatistes': 'separatists', 'nautica': 'nautical', 'qlpk': 'lik', 'babybels': 'babbles', 'mittal': 'mitral', 'durrow': 'burrow', 'crabmas': 'crabs', 'harrisb': 'harris', 'idlis': 'idles', 'semt': 'set', 'fitr': 'fit', 'kahi': 'kai', 'crewe': 'crew', 'luci': 'luck', 'byju': 'you', 'ronaldo': 'ronald', 'carousell': 'carousel', 'lene': 'line', 'boujiest': 'busiest', 'kapiling': 'sailing', 'ateam': 'team', 'lassi': 'lass', 'jkaje': 'jake', 'electeds': 'elected', 'meena': 'meera', 'onething': 'something', 'pompe': 'pope', 'gumy': 'guy', 'lauryn': 'lauren', 'titbit': 'tidbit', 'rcrg': 'ring', 'wickles': 'pickles', 'beatc': 'beat', 'woaah': 'woah', 'forex': 'fore', 'battlecry': 'battery', '$btc': 'etc', 'tunraround': 'turnaround', 'mccolls': 'scrolls', 'releasesl': 'releases', 'wburg': 'burg', 'gaslighted': 'slighted', 'contac': 'contact', 'gleason': 'reason', 'tiredhe': 'tired', 'ontologically': 'ontological', 'nyer': 'nder', 'tngovt': 'ingot', 'kci': 'ki', 'storify': 'story', 'aew': 'new', 'arkansans': 'arkansas', 'civillian': 'civilian', 'vnmo': 'no', 'phyllo': 'hello', 'gallatin': 'gallant', 'shibani': 'shebang', 'ethco': 'echo', 'enviorment': 'environment', 'munchy': 'munch', 'correa': 'cornea', 'scusd': 'scud', 'thinkbig': 'thinking', 'siciliano': 'sicilian', 'ph]': 'phi', 'campton': 'hampton', 'pfc': 'pic', 'naahh': 'nah', 'pura': 'pure', 'djevolve': 'devolve', 'msmorg': 'memory', 'yinz': 'ying', 'biggies': 'piggies', 'scampoo': 'shampoo', 'cick': 'pick', 'kinch': 'inch', 'anoka': 'aka', '5head': 'head', 'gritter': 'glitter', 'druhv': 'drug', 'techcity': 'tenacity', 'davide': 'divide', 'vannuys': 'annoys', 'corporators': 'corporations', '3a9ima': 'anima', 'colemans': 'coleman', '41c': 'doc', 'moneyand': 'boneyard', 'sandhya': 'sandy', 'unstoppables': 'unstoppable', 'dunker': 'bunker', 'deliverytec': 'deliverymen', 'beckley': 'buckley', 'mukuru': 'kuru', 'loveneed': 'livened', 'rt': 'it', 'faceoff': 'takeoff', 'compostable': 'comfortable', 'florish': 'flourish', 'sils': 'sits', 'amaz': 'amaze', 'holts': 'holds', 'sekku': 'seek', 'affor': 'afford', 'heloise': 'eloise', 'simeon': 'simon', 'hhayi': 'hay', 'bcstt': 'best', 'hhat': 'that', 'ifruit': 'fruit', 'placeholder]': 'placeholder', 'formidabl': 'formidable', 'lumad': 'human', 'pleasynt': 'pleasant', 'dlf': 'elf', 'manitoba': 'manitou', 'lovejo': 'lovejoy', 'gedling': 'gelding', 'evoo': 'evo', 'dagny': 'danny', 'lafira': 'mafia', '4bottles': 'bottles', 'gabbi': 'rabbi', 'postgame': 'postage', 'alim': 'aim', 'santam': 'santa', 'g2': 'go', 'rhoc': 'hoc', 'mlp': 'map', 'normalizes': 'normalize', 'chery': 'cherry', 'huet': 'hurt', '21k': 'ask', 'kakinada': 'kalinda', 'endvaw': 'endow', 'ddbd': 'did', 'barroga': 'barrage', 'simc': 'sic', 'houtte': 'house', 'sandi74': 'sanding', 'iloilo': 'lolo', 'grandas': 'grandad', 'hakims': 'hairs', 'reisman': 'newsman', 'elen': 'even', 'frkn': 'fran', 'progress60': 'progress', 'niman': 'man', '3yours': 'yours', 'swishers': 'swishes', 'ps35': 'psi', 'nfp': 'nap', 'remainists': 'feminists', 'lavinia': 'latina', 'nosies': 'noises', 'prentis': 'prentiss', 'cownow': 'cowboy', 'caitd3g': 'catdog', 'portel': 'portal', 'rideconnect': 'reconnect', '7on': 'on', 'ffxiv': 'xiv', 'havnt': 'haunt', 'olywa': 'olga', 'tshwana': 'thana', 'zuse': 'use', 'alamos': 'alamo', 'pstd': 'put', 'worldiest': 'worthiest', 'weepin': 'weeping', 'riers': 'rivers', '$vper': 'over', 'days]': 'days', '$billions': 'billions', 'charisee': 'charisma', 'imm': "i'm", 'misma': 'miasma', 'tonio': 'tonic', 'torwood': 'torchwood', 'appeance': 'appearance', 'gumbos': 'gumbo', '60s': 'is', 'ensconce': 'ensconced', 'lapol': 'lapel', 'interesting]': 'interesting', 'chandlery': 'chandler', 'clwb': 'club', 'pres45': 'press', 'to]': 'to', 'cubside': 'subside', 'aviod': 'avoid', 'leece': 'leech', 'kidlet': 'killed', 'offe': 'off', 'kerbside': 'curbside', 'wba': 'wa', 'zalando': 'orlando', 'nhu': 'nu', 'marple': 'marble', 'hithah': 'hitch', 'fsma': 'fema', 'tomrw': 'tour', '$jet': 'jet', 'zuma': 'uma', 'zshq': 'she', 'jusco': 'just', 'almost40': 'almost', 'x5': 'xu', 'raashan': "rada'han", 'gmr8': 'girl', 'putzger': 'putter', 'wuh': 'uh', 'aromatized': 'dramatized', 'bandori': 'candor', 'apho': 'pho', 'muv': 'mum', 'degraff': 'degrade', 'dolan': 'dylan', 'scapes': 'scares', 'lunchbag': 'lunching', 'tolentino': 'valentino', 'salina': 'saliva', 'chinazi': 'china', 'bulding': 'building', 'waga': 'wage', 'ongov': 'onto', 'texlaw': 'texas', 'crosstalk': 'crosswalk', 'indoirs': 'indoors', 'sandakan': 'sandman', 'soutah': 'south', 'glady': 'glad', 'telegraphios': 'telegraphing', 'shiong': 'shing', 'zabiha': 'labia', 'zell': 'tell', '20days': 'days', 'bebuying': 'buying', 'proudaf': 'proud', 'multitasker': 'multitask', 'mnd': 'and', 'adjoin': 'adjoins', 'dayb': 'day', 'uitm': 'uit', 'whinges': 'hinges', 'mbproduce': 'produce', 'deadpans': 'deadpan', 'fishbox': 'fishbowl', 'olivarez': 'alvarez', 'dtric': 'trip', 'tadi': 'taxi', 'mommaj': 'momma', 'gyi': 'gym', 'chutneys': 'chutney', 'continued]': 'continued', 'reasors': 'reasons', 'jps': 'ups', 'ryuhi': 'yuri', 'freedumbs': 'freedoms', 'ps15': 'psi', 'peduto': 'pesto', 'lindi': 'linda', 'grocerries': 'groceries', 'biale': 'bible', 'walkups': 'walkup', 'snell': 'sell', 'quaranti': 'quarantine', 'redis': 'reds', 'pandemy': 'pandey', 'trai': 'train', 'gdsn': 'gun', 'whitepaper': 'wastepaper', 'empl': 'emil', 'blatently': 'blatantly', 'btlol': 'blow', 'cabane': 'cabana', 'tw1': 'two', 'markle': 'marble', 'tarlac': 'tarmac', 'chlorinate': 'chlorinated', 'atdoorsteps': 'doorsteps', 'zwolle': 'swollen', 'alphred': 'altered', '8doll': 'doll', 'schlitter': 'shitter', 'gour': 'your', 'chamal': 'camel', 'lette': 'letter', 'concorder': 'camcorder', 'sb': 'so', 'automob': 'autobot', 'instacat': 'instant', 'ammon': 'ammo', 'rheadd': 'heard', 'fedtech': 'fetch', 'rotf': 'roof', 'karens': 'karen', 'lgp': 'lap', 'yo89': 'you', 'jalepenos': 'jalapenos', 'aoife': 'life', '19minutes': 'minutes', 'thepose': 'these', '5oz': 'coz', 'ecommerc': 'commerce', 'malenadu': 'malena', 'manch': 'match', 'mpqaw': 'paw', 'barna': 'barn', 'mamoth': 'mammoth', 'olx': 'old', 'airb': 'air', 'redwine': 'redline', 'coronaian': 'coronation', 'funzies': 'fuzzies', 'jst': 'just', 'cupmen': 'gunmen', 'hanuman': 'hangman', 'peopley': 'people', 'awschat': 'asshat', 'confort': 'comfort', 'kehe': 'kee', 'paull': 'pull', 'lulla': 'lull', 'infec': 'infect', 'foodt': 'food', 'happyfor': 'happyto', 'kamla': 'kayla', 'gto': 'to', 'sahel': 'shel', 'rootbear': 'footwear', 'kew': 'new', 'copd': 'cold', 'eugenicist': 'eugenics', 'aibpost': 'airport', 'cremini': 'remind', 'medstro': 'metro', 'gdpr': 'gear', 'ricker': 'richer', 'dtd': 'did', 'dashers': 'washers', 'brandnews': 'brandies', 'slcers': 'ulcers', 'tapps': 'tapes', 'crepevine': 'grapevine', 'cdata': 'data', 'amjoy': 'enjoy', 'ddidnt': "didn't", 'domicron': 'omicron', 'dll': 'all', 'cauley': 'called', 'telco': 'tell', 'bostocks': 'stocks', 'psycopath': 'psychopath', 'drdoug': 'drug', 'taca': 'taco', 'nenagh': 'menage', 'countr': 'country', 'stull': 'still', 'fpl': 'fol', 'freezy': 'freeze', '30a': 'a', 'tution': 'tuition', 'ambi': 'ami', 'dadlife': 'deadline', 'anak': 'anal', 'hik': 'his', '13april': 'april', 'wenzel': 'denzel', 'agorz': 'agora', 'to26': 'to', 'underpayment': 'undergarment', 'ejupdate': 'update', 'bklyn': 'klan', 'ultimaker': 'ultimate', 'ovp': 'op', '30hr': 'sehr', 'gcbbo': 'gumbo', 'moisturized': 'moisturizer', 'kona': 'bona', 'chuke': 'choke', 'salada': 'salad', 'kenthink': 'rethink', 'kaitlyn': 'caitlyn', 'ethn': 'ethan', 'masvingo': 'masking', 'oneyear': 'neyer', 'nika': 'nina', 'drats': 'rats', 'pcom': 'prom', 'deadset': 'headset', 'moshodi': 'moshi', 'pnw': 'paw', 'gii': 'gig', 'mankinds': 'mankind', 'cxo': 'ceo', 'nowdid': 'sordid', 'abp': 'app', 'ronus': 'bonus', 'gutfeld': 'gutted', 'danni': 'danny', 'geddit': 'reddit', 'nanas': 'nana', 'sega': 'sea', 'bellpost': 'bedpost', 'txt': 'text', 'coulsdon': 'coulson', 'supurrvisor': 'supervisor', 'pakvsa': 'pasa', 'ps9': 'psi', 'chocolately': 'chocolatey', 'wotnot': "won't", '$goog': 'good', 'puddin': 'pudding', 'lcla': 'lila', 'coorderinate': 'coordinate', 'haere': 'here', 'vtgrown': 'grown', 'kedai': 'media', 'weeee': 'were', 'cipc': 'cip', 'malva': 'mala', 'sanatize': 'sanitize', 'unbeliveable': 'unbelievable', 'organnised': 'organised', 'svacorn': 'scorn', 'shephards': 'shepherds', 'starkist': 'starfish', 'steptoe': 'tiptoe', 'joyc': 'joy', 'wtnh': 'with', 'dewitt': 'hewitt', 'epmads': 'pads', 'eger': 'ever', '2hra': 'ha', 'tirz': 'tire', 'ps2': 'psi', 'alg': 'all', 'castigated': 'castigate', 'denver7': 'denver', 'aswazi': 'swami', 'marmite': 'marine', 'oobleck': 'ofblack', 'cobot': 'robot', 'citt': 'city', 'yesterdat': 'yesterday', 'livetv': 'liveth', 'hurrier': 'hurried', 'wkd': 'wed', 'infinitty': 'infinite', 'perl': 'per', 'geibel': 'rebel', 'keera': 'meera', 'yournumber': 'outnumber', 'khy': 'why', 'utica': 'stick', 'fto': 'to', 'profitabi': 'profitable', 'ciso': 'miso', 'k95s': 'kids', 'gcq': 'get', 'newjobs': 'nutjobs', 'haagan': 'pagan', 'mykoo': 'moo', 'valpak': 'valar', 'paran': 'pagan', 'snapdragon': 'snapdragons', 'peach10': 'peach', 'staywell': 'stairwell', 'bloch': 'block', 'intial': 'initial', '$appl': 'happy', 'marske': 'make', 'marist': 'marxist', 'unts': 'units', 'ripken': 'ripen', 'customrs': 'customers', 'coliform': 'conform', 'cheviot': 'chariot', 'elderflowers': 'elderflower', 'jarrett': 'garrett', 'metropolit': 'metropolis', 'blehhh': 'bleach', 'nky': 'sky', 'mrdelivery': 'delivery', 'dollarsm': 'dollars', 'yw': 'ya', 'gettable': 'eatable', 'noboundaries': 'boundaries', 'sexbot': 'sexist', 'wors': 'work', '12pack': 'pack', 'orderrd': 'ordered', 'starshi': 'starship', 'neighb': 'neighs', 'bos': 'boy', 'cupw': 'cup', 'winfit': 'unfit', 'no2ir': 'noir', 'patricking': 'tricking', 'kamau': 'kama', 'lek': 'let', 'ritualise': 'ritualised', 'tnx': 'tax', '$abus': 'bus', 'hahahaah': 'hahaha', 'restaurantes': 'restaurants', 'orgo': 'orgy', 'irders': 'orders', 'itrms': 'items', 'caugh': 'caught', 'xxox': 'xbox', 'befo': 'bef', 'gorobinsons': 'robinsons', 'scholo': 'school', 'sn': 'in', 'servic': 'service', 'joywe': 'joyce', 'vulne': 'rule', 'rting': 'ring', 'scarb': 'scare', 'washerman': 'fisherman', 'tweats': 'treats', 'jzk': 'jak', 'personel': 'personal', '2buy': 'buy', 'glynis': 'glands', 'tohiy': 'this', 'missourian': 'missouri', 'cline': 'line', 'skippin': 'skipping', 'dfo': 'do', 'f10': 'for', 'gehrig': 'gearing', 'ncr': 'nor', 'vvvv': 'vive', 'solvang': 'solving', '$denn': 'denn', 'keyes': 'eyes', 'jrd': 'jed', 'piti': 'pity', 'parcells': 'parcels', 'starrett': 'starred', 'kangoo': 'tango', '5mm': 'umm', 'penuh': 'punch', 'guddi': 'buddy', 'microwa': 'microwave', 'khaana': 'hana', 'wjrt': 'wart', 'citrusy': 'citrus', 'myfav': 'mynah', 'malasha': 'malaria', 'wmay': 'way', 'privledge': 'privilege', 'tshow': 'show', 'bfast': 'fast', 'wjla': 'will', 'amul': 'maul', 'eac': 'each', 'sanitisers': 'sanitizer', 'insted': 'instead', '30mts': 'emts', 'econoline': 'economize', 'adibcards': 'discards', 'anytype': 'anytime', 'ebery': 'every', '19miles': 'miles', 'patak': 'pack', 'vonns': 'vonas', 'undergroubd': 'underground', 'froo': 'from', '$pixy': 'pixy', 'schumer': 'schemer', 'byd': 'by', 'canvey': 'convey', 'codeca': 'code', 'teamdk': 'team', 'nm': 'no', 'weinberg': 'weiner', 'shameball': 'slimeball', 'unble': 'uncle', 'heybitch': 'haymitch', 'commodies': 'commodity', 'pty': 'pay', 'arlin': 'marlin', 'cta': 'cat', 'mbvans': 'means', 'experien': 'experience', 'faygo': 'fargo', 'goike': 'like', 'lylm': 'lyle', 'apartme': 'apartment', 'hague': 'vague', 'nodiesel': 'diesel', '$gis': 'gis', 'tasker': 'tanker', 'husban': 'husband', '57am': 'am', 'safemoon': 'saewoon', 'tech]': 'tech', 'siny': 'sing', 'catc': 'catch', 'canlab': 'canvas', 'trozang': 'trojan', 'mwp': 'map', 'bokksu': 'books', 'duri': 'yuri', 'momwin': 'domain', 'commected': 'commented', 'phun': 'pun', 'eviden': 'evident', 'gr7rh': 'graph', 'attemp': 'attempt', 'scandless': 'candles', 'shyp': 'ship', 'apapa': 'papa', 'eevery': 'every', 'upgenius': 'genius', 'solame': 'solace', 'emini': 'mini', 'exclusivead': 'exclusive', 'durnk': 'drunk', 'mancos': 'mangos', 'corona]': 'coronal', 'stors': 'story', 'perso': 'person', 'venturer': 'venture', 'bisummit': 'summit', 'youlove': "you've", 'payoura': 'payout', 'chol': 'cool', 'edtech': 'tech', 'sfla': 'sola', 'delmart': 'depart', 'kaitlin': 'caitlin', 'wef': 'we', 'aleviate': 'alleviate', 'clammoring': 'clamoring', 'vignette': 'dinette', 'mar21': 'marry', 'zorach': 'coach', 'gdg': 'gig', 'everywher': 'everywhere', '10wks': 'hawks', 'pula': 'pull', 'ccj': 'can', 'psas': 'pass', '80your': 'your', 'cheeers': 'cheers', 'ovc': 'ove', 'the13': 'the', 'iptv': 'it', 'wtih': 'with', 'sbji': 'ski', 'tomi': 'tomb', 'olsa': 'olga', 'doodh': 'doody', 'orwx': 'oryx', 'mcdonalds': "mcdonald's", '0lq': 'all', 'n95': 'not', 'adt': 'at', 'delouvery': 'delivery', 'yqr': 'yer', 'shmira': 'shirt', 'recirc': 'record', 'bujo': 'brujo', 'knockin': 'knocking', 'shukhur': 'shekhar', 'aoc': 'doc', 'strategery': 'strategy', 'recirculated': 'circulated', 'fluper': 'super', 'apon': 'upon', 'vele': 'vile', 'erbody': 'embody', 'haggerty': 'jaggery', 'maneesh': 'mannish', 'wpg': 'wig', 'shtttt': 'shitty', 'luyanda': 'lucinda', 'dtf': 'def', 'supermom': 'superior', 'urcart': 'cart', 'chillin': 'chilling', 'sblv': 'able', 'tdl': 'til', 'agric': 'again', 'camy': 'came', 'snapzo': 'snap', 'carbz': 'carbs', 'wv': 'we', 'tenterhook': 'tenterhooks', 'waki': 'wake', 'dmcares': 'cares', 'tpn': 'ten', 'casillero': 'caballero', 'hx': 'he', 'edelman': 'elderman', 'spectrumy': 'spectrum', 'partnersh': 'partners', 'randomize': 'randomized', 'lpg': 'leg', 'mozza': 'pizza', 'punnets': 'planets', 'wspa7': 'spa', 'crazyness': 'craziness', 'mazing': 'making', 'sfchinatown': 'chinatown', 'ironwood': 'ironwork', 'adham': 'adam', 'youruber': 'youtube', 'haveee': 'have', 'riz': 'rid', 'fashinoning': 'fashioning', 'reorderered': 'reordered', 'customrr': 'customer', 'swportland': 'portland', '5i': 'i', '60your': 'your', 'sorrounding': 'surrounding', 'plugin': 'plain', 'plaintain': 'plantain', 'upsca': 'upset', '6x': 'ex', 'wearin': 'wearing', 'whoah': 'whoa', 'jack8ss': 'jackass', 'oorah': 'oprah', 'lorie': 'lore', 'freefrom': 'freedom', 'supps': 'supes', 'neveen': 'neven', 'wft': 'wet', 'coronake': 'coronary', '6ls': 'als', 'lect': 'let', 'orderinances': 'ordinances', 'boughton': 'bought', 'mdnd': 'mind', 'chasin': 'chasing', 'angkas': 'angels', 'adricula': 'dracula', 'florals': 'floral', 'expensive$': 'expensive', 'lactase': 'lactose', 'alm': 'all', 'picadilly': 'piccadilly', 'txted': 'texted', 'tlot': 'lot', 'gano': 'gang', 'natl': 'nail', 'brotherr': 'brother', 'wako': 'wake', '00us': 'us', 'bahlali': 'bahubali', 'sequim': 'sequin', 'admini': 'admin', 'apr': 'air', 'bowila': 'bowl', 'okwara': 'kara', 'erk': 'jerk', 'lowbar': 'lower', 'inika': 'iike', 'retroshade': 'retrograde', 'ncl': 'nil', 'liverdad': 'verdad', 'bn': 'in', 'iwork': 'work', 'fanm': 'fan', '$hiph': 'which', 'nationalise': 'nationalist', 'dosent': 'docent', 'beisan': 'began', 'nerul': 'nerve', 'hardpass': 'harass', 'sunchat': 'buncha', 'sentineli': 'sentinels', 'lathay': 'lately', 'chibage': 'cabbage', '9billion': 'billion', '4got': 'got', 'agretti': 'agreat', 'wannabee': 'wannabe', 'stoppages': 'stoppage', 'kwaata': 'kata', 'artificia': 'artificial', 'practic': 'practice', 'gallaway': 'hallway', 'oolala': 'koala', '39am': 'am', 'dendy': 'deny', 'petwalking': 'petaling', 'oushe': 'she', 'shping': 'shaping', 'harrash': 'harrass', 'joyva': 'joy', 'nikon': 'nixon', 'valdemart': 'voldemort', 'rtd': 'red', 'mdpolitics': 'politics', 'ecommerce': 'commerce', 'abeg': 'beg', 'pacifica': 'pacific', 'ealing': 'eating', 'pagination': 'imagination', 'dac': 'day', 'malmo': 'malo', '4x4s': 'axis', 'kiranaa': 'piranha', 'arting': 'acting', 'ekiti': 'exit', 'unmet': 'unmei', 'garnishings': 'garnishing', 'wikibd': 'wiki', 'notiven': 'notice', 'esy': 'easy', 'welfear': 'welfare', 'laelia': 'paella', 'althing': 'anthing', 'alteast': 'atleast', 'shampooer': 'shampooed', 'joanns': 'joanna', 'ridley': 'riley', 'rochelle': 'michelle', 'lufc': 'luff', 'eh12': 'eh', 'ssg': 'sig', 'lcb': 'lab', 'subsitions': 'substations', 'nofilter': 'filter', 'bd': 'be', 'tofoodie': 'foodie', 'flakies': 'flakes', 'now8': 'now', 'enca': 'ence', 'b67': 'be', 'hospt': 'host', 'emanna': 'manna', 'sexys': 'sexy', 'fatboy': 'fanboy', 'ufn': 'fun', 'antibac': 'antiwar', 'epoc': 'epic', 'foodler': 'fooled', 'hartland': 'heartland', 'staffie': 'staff', 'filterd': 'filter', 'complicat': 'complicate', 'gtfo': 'to', 'wnn7': 'want', 'ontime': 'onetime', 'importanton': 'important', 'parkview': 'preview', 'lepas': 'leaps', 'econsultancy': 'consultancy', 'gettas': 'getta', 'rpt': 'rat', 'falmouth': 'fallout', '5p': 'up', 'naresh': 'fresh', 'dsd': 'did', 'bhoot': 'shoot', 'metallicat': 'metallic', 'belvoir': 'revoir', 'funday': 'sunday', 'humanise': 'humanize', 'minimoms': 'minimums', 'askaman': 'shaman', 'infos': 'info', 'tc': 'to', 'berbere': 'barber', 'tartness': 'darkness', 'gayporn': 'rayborn', 'jobto': 'job', 'westjet': 'westen', 'randomised': 'randomized', 'kickin': 'kicking', 'convos': 'convoy', 'chilis': 'chili', 'digitalfb': 'digital', 'onshelf': 'shelf', 'rti': 'ti', 'tameside': 'stateside', 'saavy': 'savvy', 'kevla': 'kevlar', 'mikko': 'miko', 'merce': 'mercy', 'ntoni': 'toni', 'johar': 'johan', 'slcsd': 'sliced', '8p': 'up', 'apn': 'an', 'wewe': 'were', 'opp': 'pop', '10years': 'years', 'extendpsp': 'extends', 'kgs': 'kegs', 'limeades': 'lineages', 'mnf': 'anf', 'fruad': 'fraud', 'vash': 'cash', 'mmhm': 'mom', 'thinfs': 'things', 'srvicea': 'service', 'kull': 'kill', 'dalkeith': 'walketh', 'resister': 'register', 'britannia': 'britannian', 'unionyes': 'unions', 'purell': 'purely', 'godwink': 'hoodwink', 'singup': 'single', 'disguisting': 'disgusting', 'iaa': 'ian', 'resourced': 'resources', 'evejoy': 'every', 'rimbey': 'timber', 'gbagada': 'barada', 'nisall': 'niall', 'floofs': 'floors', 'unvac': 'unpack', 'brushless': 'ruthless', 'nyg': 'nag', 'marcs': 'marks', 'shercock': 'sherlock', 'brah': 'bra', 'shahid': 'said', 'caillum': 'callus', 'sgt': 'set', 'sancrosanct': 'sacrosanct', 'itw': 'it', 'fotonomy': 'autonomy', 'ensu': 'ensue', 'usagi': 'usage', 'caldo': 'aldo', 'baslight': 'gaslight', 'mgmnt': 'moment', 'fraudelent': 'fraudulent', 'simms': 'sims', 'ecrony': 'crony', 'qualy': 'quay', 'happyjo': 'happyto', 'luckey': 'lucky', 'nanorobots': 'nanobots', 'pds': 'ads', 'partea': 'parted', 'fanbase': 'fantasy', 'iznt': 'int', 'fns': 'fans', 'flogger': 'flogged', 'dollarsk': 'dollars', 'firsto': 'first', 'scripti': 'script', 'siomai': 'sigma', 'nfpa': 'napa', '$ape': 'tape', 'inabili': 'inability', 'tcs': 'tis', 'referesh': 'refresh', 'adr': 'air', 'makeitup': 'makeup', 'capeau': 'chapeau', 'terf': 'term', 'brsin': 'brain', 'mvrs': 'mars', 'mwenda': 'agenda', 'baru': 'bar', 'bypa': 'bya', 'cura': 'cure', 'pollutant': 'pollutants', 'othrs': 'others', 'robotech': 'robotic', 'crearte': 'create', 'ipl': 'ill', 'neaither': 'neither', 'gigecon': 'pigeon', 'kamcity': 'amity', 'delover': 'deliver', 'retun': 'return', 'annem': 'anne', 'unsust': 'unjust', 'bulloughs': 'furloughs', 'quarrantine': 'quarantine', 'fanc': 'fancy', 'catharine': 'catherine', 'orap': 'crap', 'foodworks': 'woodwork', 't20': 'the', 'wichita': 'wichitawx', 'napanee': 'japanese', 'agley': 'alley', 'iberia': 'iberian', 'insp': 'ins', 'buspass': 'bypass', 'hsr': 'her', 'ypur': 'your', 'ourchase': 'purchase', 'liesa': 'lies', 'golfcourse': 'ofcourse', 'eddres': 'eddies', 'fcked': 'fucked', 'insom': 'into', 'etcete': 'excite', 'vaishali': 'vishal', 'czon': 'con', '3bd': 'and', 'taiwan': 'taipan', 'dulu': 'dull', 'dianping': 'dancing', 'lynton': 'lyndon', 'camaras': 'cameras', 'speirin': 'spirit', 'walesa': 'wales', 'thisis40': 'thisis', 'pvc': 'pic', '80k': 'ask', 'auck': 'fuck', 'durning': 'during', 'jain': 'join', 'tyoura': 'your', 'osi': 'si', 'thibking': 'thinking', 'grupo': 'group', 'groserias': 'groceries', 'haagen': 'hagen', 'honestbee': 'honeybee', '6px': 'up', 'quebec': 'queen', 'ftsummit': 'summit', 'theconne': 'theone', 'biig': 'big', 'humpers': 'jumpers', 'mmh': 'mah', 'ibmz': 'ibm', 'bofrost': 'boost', '7your': 'your', 'ikoyi': 'koi', 'imme': 'gimme', 'adopter': 'adopted', 'seater': 'sweater', '2]': 'i', 'reachy': 'reach', 'us$': 'us', 'bantu': 'want', 'zev': 'ze', 'branchen': 'branches', 'jupiterco': 'jupiter', 'swiffer': 'sniffer', 'dokay': 'okay', 'unstirred': 'stirred', '10min': 'min', 'lsbu': 'lab', 'fox25': 'fox', 'mircom': 'mirror', 'rawng': 'rang', 'vaxxer': 'baxter', 'gvm': 'gym', 'wagu': 'wage', 'morisons': 'orisons', 'droom': 'room', 'hynes': 'haynes', 'vicpol': 'victor', 'oahf': 'oaf', 'henny': 'penny', 'ginna': 'gonna', 'absolut': 'absolute', 'nd0ms': 'dams', 'corealtors': 'creators', 'granma': 'grandma', 'busin': 'basin', 'abell': 'bell', '4times': 'times', 'shor': 'show', 'fetman': 'hetman', '15bn': 'ibn', 'helber': 'helper', 'finlncill': 'financial', 'istfg': 'stag', 'd20': 'do', 'napier': 'rapier', 'hn': 'in', 'myfood': 'food', 'dewormer': 'deformed', 'botolan': 'boola', 'byo': 'by', 'togher': 'tougher', 'murcan': 'urban', 'headroom': 'bedroom', 'fullfilled': 'fulfilled', 'apts': 'acts', 'hatsoff': 'handoff', '5wks': 'was', 'carvel': 'carved', 'rampantly': 'rampant', 'mj': 'my', 'hooly': 'holy', 'asdawant': 'adamant', 'katc': 'kate', 'manorama': 'panorama', 'kissies': 'kisses', 'exasperation': 'exaggeration', 'pennsylvanian': 'pennsylvania', 'bogside': 'beside', 'autchat': 'outwhat', 'meanin': 'meaning', 'suppor': 'support', 'maloneys': 'baloney', 'gramy': 'gray', 'saida': 'said', 'atieno': 'patient', 'dolefully': 'hopefully', 'semite': 'smite', 'ajlia': 'alia', 'wfuv': 'wuv', 'hayter': 'halter', 'communitys': 'community', 'thyouroid': 'thyroid', 'limi': 'limit', 'kinkora': 'kinor', 'jagrup': 'jaggu', 'entrega': 'entreat', 'saamne': 'same', 'chipman': 'chapman', 'mygofer': 'gofer', 'osseo': 'osso', 'joellen': 'pollen', 'rishi': 'rish', 'kassi': 'kass', '3$': 'i', 'emarketer': 'marketer', 'nyd': 'nod', 'bragger': 'bragged', 'h': 'i', 'delicery': 'delivery', 'donbald': 'donald', 'brrrr': 'error', 'thdm': 'them', 'canclled': 'canceled', 'sodla': 'soda', '06am': 'am', 'yakuzies': "yakuza's", 'ipos': 'ipod', 'yummly': 'yummy', 'tiro': 'tire', 'vanre': 'vance', 'yesr': 'year', 'snuggies': 'snuggles', 'rsa': 'sa', 'highkey': 'higher', '$hfg': 'hug', 'custoner': 'customer', 'wanti': 'want', 'frelard': 'reward', 'msc': 'mic', 'inhad': 'had', '61st': 'just', 'cozi': 'cozy', 'gauranteed': 'guaranteed', 'magodo': 'magog', 'packag': 'package', 'sseum': 'seem', 'quaranteed': 'guaranteed', 'buildboard': 'billboard', 'raych': 'ranch', 'felice': 'feline', 'ughm': 'ugh', 'rellon': 'mellon', 'clustertruck': 'clusterfuck', 'hq': 'he', 'ogun': 'gun', '2strokes': 'strokes', 'bdrm': 'berm', 'hoppin': 'hopping', 'maruti': 'martin', 'bge': 'be', 'fumc': 'fume', 'moring': 'morning', 'nbsp': 'nose', 'psl': 'pal', 'lari': 'lair', 'carrs': 'carry', 'uncredibly': 'incredibly', 'caracas': 'maracas', 'walto': 'waltz', 'frazo': 'crazy', 'ha0': 'had', 'giri': 'girl', 'incentivise': 'incentivize', 'fraine': 'fine', 'miraj': 'mirai', 'oken': 'open', '24h': 'oh', 'staggere': 'staggered', 'hannon': 'cannon', 'repr': 'rear', 'genxer': 'gender', 'mpho': 'pho', '$hrb': 'curb', 'yty': 'ty', 'mapa': 'map', 'usapple': 'sample', 'bramhand': 'farmhand', 'bofa': 'sofa', 'prents': 'parents', 'lifeboathq': 'lifeboat', 'ptbo': 'to', 'natick': 'stick', 'ea20': 'each', 'cainta': 'paint', 'pch': 'ich', 'smallbu': 'small', 'ondo': 'onto', 'pandan': 'panda', 'utr': 'ur', 'homebody': 'somebody', 'helpijg': 'helping', 'gdaughter': 'daughter', 'lignum': 'magnum', 'fayette': 'palette', 'licensee': 'license', 'notfound': 'newfound', 'signin': 'signing', 'unbelievered': 'unbelievers', 'ackerman': 'alderman', 'capchas': 'catches', 'precedented': 'unprecedented', 'banabi': 'banana', 'spinch': 'pinch', 'mckean': 'mean', '90ish': 'wish', 'detoriated': 'decorated', 'nosnacks': 'snacks', 'deleicery': 'delivery', 'twsyl': 'twirl', 'sni': 'sin', 'sedaris': 'sears', 'easyyy': 'easy', 'saba': 'sara', 'cafo': 'cafe', 'comprom': 'comport', 'sippin': 'sipping', 'imgs': 'migs', 'wweraw': 'wwere', 'batac': 'data', 'erradicate': 'eradicate', 'ecm': 'em', 'roch': 'rock', 'daas': 'days', 'votered': 'covered', 'brandwin': 'branding', 'talina': 'talia', 'subsector': 'subsection', 'tnr': 'tar', 'kenai': 'kensi', 'allstay': 'allay', 'udelv': 'del', 'homepod': 'homeboy', 'adtech': 'tech', 'freebag': 'freeway', 'bso': 'so', 'deadmy': 'deadly', 'overlorders': 'overlords', 'panko': 'pinko', 'cabiao': 'cabin', 'clunkier': 'clunker', 'bandra': 'sandra', 'linings]': 'linings', 'fiu': 'fix', 'memaw': 'meemaw', 'thon': 'then', 'uteri': 'teri', 'misbaged': 'massaged', 'bongiwe': 'boogie', 'passwrd': 'password', 'fcbiz': 'fbi', 'maloney': 'baloney', 'bitchea': 'bitches', 'sabiq': 'sadiq', 'newweek': 'newsweek', 'afiari': 'affair', 'r10': 'run', 'ipaja': 'papa', 'wrtie': 'write', 'milanthropy': 'philanthropy', 'scitech': 'switch', 'suhoor': 'school', 'oohh': 'ooh', 'owi': 'own', 'argos': 'argon', 'clearout': 'clearest', 'oh9': 'oh', 'subscibed': 'subscribed', 'goivanti': 'giovanni', 'jh': 'oh', 'tmrrw': 'throw', '$pura': 'pure', 'iskander': 'islander', 'zaroor': 'armor', 'fl11': 'fly', 'goelectric': 'electric', 'hutton': 'button', 'inko': 'into', '2nd': 'and', 'goodread': 'godhead', 'invisibles': 'invisible', 'ceycey': 'cece', 'lrnd': 'land', 'lolipops': 'lollipops', 'clono': 'clone', 'greenling': 'greening', '10tv': 'utv', 'localwr': 'local', 'cookup': 'hookup', 'kisi': 'kiss', 'nan': 'can', 'missoula': 'missouri', 'fucj': 'fuck', 'prepack': 'repack', 'serco': 'servo', 'hasan': 'hasn', 'yachters': 'yachts', 'fayez': 'fayed', 'coronaug': 'coronary', 'pickaroo': 'buckaroo', 'bioled': 'boiled', 'pwa': 'wa', 'buehler': 'butler', 'rtpcr': 'roper', 'psycological': 'psychological', 'sorb': 'sort', 'preggo': 'peggy', 'susah': 'susan', 'logline': 'online', 'tnn': 'ten', 'myourtle': 'myrtle', 'sligo': 'saigo', 'kahlua': 'dahlia', 'firevans': 'fireman', 'vehic': 'vehicle', 'wramc': 'warm', 'bihari': 'binary', 'liquiot': 'liquor', 'yelm': 'yell', 'pupperoni': 'pepperoni', 'stpaul': 'steal', 'ammune': 'immune', 'yead': 'head', 'nampa': 'tampa', 'hashana': 'hahaha', 'dorstep': 'doorstep', 'statment': 'statement', 'advil': 'anvil', 'buc': 'but', 'farnham': 'arkham', 'resid': 'reside', 'zanotto': 'notto', 'akoka': 'aka', 'makinwa': 'making', 'chior': 'choir', 'provin': 'proven', 'ancaps': 'snaps', 'countrary': 'contrary', 'deadlifts': 'deadliest', 'foodwiz': 'foodie', 'laphowe': 'lahore', 'hk': 'he', 'embarassing': 'embarrassing', 'cutesss': 'cutest', 'pansie': 'pansies', 'moravia': 'moravian', 'antib': 'anti', 'sharting': 'starting', 'booomb': 'bomb', 'buju': 'juju', '$atvi': 'sati', 'banyana': 'banana', 'humphreys': 'humphrey', 'estim': 'stim', 'bimble': 'bible', 'commisary': 'commissary', 'pollachi': 'pollack', 'jami': 'jam', 'ps37m': 'psalm', 's4e5': 'she', 'regularise': 'regulars', 'upgradeish': 'upgrades', 'sosjk': 'sock', 'sappho': 'sappy', 'max17': 'max', 'pinhook': 'unhook', 'centercity': 'intercity', 'mcgill': 'chill', 'cleggan': 'clean', 'mspartner': 'partner', 'igl': 'ill', 'tcb': 'tub', 'on22': 'on', 'amazom': 'amazon', 'abbotts': 'abbott', 'trulocal': 'truncal', 'karts': 'parts', 'mnwinter': 'winter', 'zio': 'zip', 'jaba': 'jab', 'kerbey': 'hereby', 'whatsup': 'whats', 'demre': 'demure', 'sheraton': 'shelton', 'patino': 'patio', 'jockamo': 'jockomo', 'mmos': 'moms', 'bika': 'bike', 'redseal': 'reseal', 'deliverysg': 'delivery', 'woollies': 'follies', 'mboyle': 'boyle', 'ktn': 'kin', 'togethers': 'together', 'greenlife': 'greenlit', 'riad': 'read', 'appli': 'apply', 'portlandia': 'portland', 'centralla': 'centrally', '95ph': 'syph', 'jinnie': 'winnie', 'eew': 'new', 'balay': 'balmy', 'ownered': 'ordered', 'coronaon': 'coronation', 'dashi': 'dash', 'alko': 'also', 'hnw': 'how', 'fxbg': 'flag', 'forva': 'fora', 'tuckshop': 'buckshot', 'neuen': 'neven', 'kmt': 'kit', 'uhoused': 'housed', 'smtg': 'smug', 'garron': 'garden', 'they]': 'they', 'kb': 'ke', 'localise': 'localised', 'hmmph': 'hump', 'braco': 'brace', 'coronatx': 'coronary', 'excerise': 'exercise', 'namma': 'gamma', 'supress': 'suppress', 'zachy': 'zach', 'team6': 'team', 'receivd': 'received', 'onroad': 'road', 'busyto': 'busy', 'beechcraft': 'leechcraft', 'hirsch': 'harsh', 'mnarobot': 'nanobot', 'cumulatively': 'cumulative', 'instating': 'insulting', 'alteza': 'alter', 'localnews': 'locales', 'tanooka': 'bazooka', 'adelson': 'nelson', 'csmo': 'como', 'drc': 'dry', 'sandalow': 'sandals', 'igd': "i'd", '12oz': 'coz', 'mgo': 'go', 'okc': 'ok', 'iphone14': 'iphone', 'draeger': 'danger', 'striken': 'strike', 'cultu': 'cult', 'peephole]': 'peephole', 'ahat': 'that', 'afaict': 'fact', 'advnc': 'advice', 'geeky0': 'geeky', 'milb': 'milk', 'lurgie': 'large', 'successfull': 'successful', 'quadcities': 'qualities', 'gruhub': 'grub', 'd': 'i', 'momos': "mom's", 'masc': 'mass', 'aire': 'are', 'dihlogo': 'dialog', 'biabox': 'bimbo', 'zd]': 'do', '27million': 'million', 'stacie': 'statie', 'enc': 'end', 'januar': 'jaguar', 'twenty30': 'twenty', 'discontinu': 'discontinue', 'cuma': 'coma', '20rs': 'cars', 'garnier': 'garner', 'huber': 'uber', 'techiest': 'techies', 'geaux': 'beaux', 'mnps': 'maps', 'sendit': 'send', 'fpf': 'fof', 'msg': 'mug', 'delivary': 'delivery', 'xers': 'hers', 'roits': 'roots', 'proj': 'pro', 'cmb': 'cab', 'tangibly': 'tangible', 'onlike': 'unlike', 'ns': 'is', 'wuhan': 'whan', '33am': 'am', 'continously': 'continuously', 'karina': 'marina', 'smuckers': 'suckers', 'unsquishy': 'squishy', 'hockers': 'hookers', 'refrigerable': 'refrigerate', 'teknocks': 'knocks', 'imigrants': 'immigrants', 'oterwise': 'otherwise', 'aerea': 'area', 'alsoo': 'also', 'voic': 'voice', '6pm': 'up', 'claar': 'clear', '17mo': 'limo', '$sbot': 'shot', 'swangin': 'swingin', 'yoshimoto': 'hashimoto', 'coun': 'count', 'unarm': 'warm', 'ichiro': 'ichido', 'smya': 'soya', '9x': 'ex', 'hunty': 'hunt', 'gmos': 'mos', 'hsm': 'him', 'cville': 'ville', 'banglore': 'bangalore', 'poeople': 'people', 'snackies': 'snakes', 'greenbean': 'greenan', 'cnet': 'net', 'approach2': 'approach', 'bhil': 'bail', 'bndd': 'and', 'mychoice': 'choice', 'dreamily': 'dreaming', 'goinh': 'going', 'tsxv': 'text', 'overpasses': 'overpass', 'arindi': 'grind', 'sanjose': 'santos', 'mperks': 'perks', 'nlng': 'ning', 'x9': 'xu', 'movd': 'move', 'pils': 'piss', '80th': 'with', 'okes': 'ones', 'sbke': 'sake', 'larrikin': 'larkin', '49c': 'doc', 'adichie': 'richie', 'jino': 'jinx', 'myb': 'my', 'towe': 'town', '5get': 'get', 'trekkie': 'trekking', 'ahs': 'as', 'unescessary': 'necessary', 'morricone': 'morrison', 'opportunitys': 'opportunity', 'jiff': 'jeff', '95s': 'is', 'solero': 'bolero', 'feederism': 'feeders', 'daika': 'dark', 'eatons': 'batons', 'dfb': 'dab', 'frictionl': 'friction', 'csnews': 'news', 'breadline': 'deadline', 'naturalista': 'naturalist', 'boarshead': 'boatshed', 'asd': 'and', 'cannellini': 'cannelloni', 'soundson': 'sounds', 'jxn': 'jon', 'moa': 'mom', '1by1': 'by', '$ocdo': 'condo', 'stratification': 'gratification', 'kln': 'kin', 'yamps': 'camps', 'freelocal': 'freeload', 'manishji': 'manish', 'bearish': 'beamish', '30lb': 'bulb', 'bihar': 'bear', '2bottles': 'bottles', 'gaaru': 'guard', 'ttg': 'tag', 'banga': 'bang', 'rocny': 'rocky', 'aaron': 'baron', 'roane': 'roan', 'imfk': 'imf', 'welln': 'well', 'bottler': 'bottle', 'aboutm': 'about', 'worksforme': 'workforce', 'nagpur': 'nagar', 'hemlo': 'hello', 'endpoint': 'midpoint', 'wordering': 'wondering', 'xanjero': 'xander', 'ne65': 'need', 'madison9': 'madison', 'joyh': 'joy', 'unwraps': 'unwrap', 'qd': 'ad', 'livongo': 'living', 'classism': 'classic', 'wcpo': 'lcpo', 'spebid': 'spend', '$gld': 'told', 'ps7': 'psi', 'lmaoo': 'loo', 'houred': 'poured', 'nailah': 'nail', 'trucklife': 'truckle', 'thatover': 'whatever', 'mybodega': 'bodega', 'shery': 'sherry', 'bost': 'most', 'carete': 'carte', '6yo': 'yo', 'mngrown': 'ingrown', 'orr': 'or', 'folkx': 'folks', 'kess': 'less', 'hoya': 'hora', 'beloves': 'beloved', 'afsal': 'fatal', 'fvckin': 'fuckin', 'qghs': 'gas', 'purdue': 'pursue', 'thobile': 'mobile', 'esentials': 'essentials', 'vindman': 'indian', 'shl': 'she', 'trumpster': 'trumpeter', 'noservice': 'service', 'everythin': 'everything', 'tds': 'ads', 'specialities': 'specialties', 'tutting': 'putting', 'zy': 'my', 'blearily': 'clearly', 'duka': 'duke', 'covoid': 'ovoid', 'ygk': 'yak', 'keely': 'kelly', 'sdavis': 'davis', 'carmelo': 'carmela', 'dedicat': 'dedicate', 'dubbo': 'dumbo', 'greencrest': 'greenest', '00p': 'up', 'preform': 'perform', 'perhap': 'perhaps', 'taga': 'tag', 'jpmorgan': 'morgan', 'miamifl': 'miami', 'pilo': 'pilot', 'kimkehr': 'kicker', 'gbishop': 'bishop', 'recyling': 'recycling', 'imout': 'about', 'granholm': 'granola', 'ifas': 'ifs', 'chall': 'call', 'utsc': 'usc', 'hngg': 'high', 'spouseson': 'spouses', 'brainier': 'rainier', 'paksoi': 'parson', 'yesoo': 'yes', 'finalmile': 'finalize', 'pawine': 'pawing', 'mcmullen': 'cullen', 'humorus': 'humorous', 'panera': 'paper', 'ponda': 'pond', 'swpns': 'swans', 'milfam': 'milan', 'mushroomfm': 'mushroom', 'rebelwing': 'rebelling', 'eh51': 'eh', 'champagnat': 'champagne', 'mutih': 'much', 'nocomplaints': 'complaints', 'hmu': 'hum', 'assoc': 'also', 'hixon': 'dixon', 'admist': 'admit', 'takoma': 'aroma', 'degen': 'deen', 'unic': 'unit', 'jobsworth': 'bosworth', 'srry': 'sorry', 'patagonia': 'patagonian', 'afterthought': 'aforethought', 'kabisa': 'labia', 'mwilliams': 'williams', 'expi': 'epi', 'reedus': 'reeds', 'kohls': 'kohl', '5minutes': 'minutes', 'may21': 'maybe', 'food4all': 'football', 'pattee': 'patted', 'phcs': 'pics', 'shungry': 'hungry', 'triskets': 'trinkets', 'everywherez': 'everywhere', 'rwandan': 'wanda', '60mph': 'lymph', 'muerto': 'puerto', 'avalible': 'available', 'milta': 'mita', 'abso': 'also', 'millenni': 'millennia', 'rockette': 'rockettes', 'ateneo': 'stereo', 'sourz': 'sour', 'tanabel': 'taxable', 'myette': 'metre', 'camee': 'came', 'dreambig': 'dreaming', 'foodst': 'foods', 'day3': 'day', 'thinkaboutit': 'thinkabout', 'nanti': 'anti', 'mallot': 'ballot', 'linforder': 'informer', 'zola': 'cola', 'ncrpos': 'crops', 'vegitable': 'vegetable', 'hyfy': 'holy', 'b2k': 'bok', 'rifamily': 'family', 'offboard': 'onboard', 'drogheria': 'progeria', 'tiwari': 'toward', 'rehomer': 'recover', 'witten': 'written', 'bafflingly': 'baffling', 'heggie': 'veggie', 'headass': 'heads', 'syndey': 'sydney', 'cac': 'can', 'upcomming': 'upcoming', 'sheww': 'shew', '8h': 'oh', 'caselaw': 'caseload', 'mcgriddle': 'griddle', 'fresho': 'fresh', 'tassimo': 'cassio', 'surpermarket': 'supermarket', 'masserking': 'mastering', 'lizol': 'lion', 'provisi': 'proviso', 'ps23': 'psi', 'wlst': 'west', 'greentext': 'greenest', 'unpleasynt': 'unpleasant', 'lasgna': 'lasagna', 'yagi': 'yogi', 'quiter': 'quite', 'eatance': 'balance', 'bery': 'very', 'cyar': 'car', 'charl': 'charm', 'sistuation': 'situation', 'tweet]': 'tweet', 'automomous': 'autonomous', 'quarantini': 'quarantine', 'nescafe': 'escape', 'farmigo': 'farming', 'seposen': 'spoken', 'camaree': 'cabaret', 'khamis': 'khakis', 'flywire': 'haywire', 'panadol': 'paradox', 'q13': 'que', 'chanute': 'chante', 'mrna': 'mana', 'scummo': 'scummy', 'toastie': 'taste', 'navage': 'savage', 'basketor': 'basket', 'copolitics': 'politics', 'myamart': 'myheart', 'defaulter': 'defaulted', 'loadind': 'loading', 'ingly': 'tingly', '$ap': 'map', '43p': 'up', 'coverack': 'coverage', 'emeals': 'meals', 'research]': 'research', 'homigo': 'homing', 'yipee': 'yippee', '15m': 'him', 'withi': 'with', 'kibbeh': 'kibosh', 'oncart': 'concert', 'nxggas': 'niggas', 'jackin': 'jacking', 'dunnes': 'dunes', 'ohno': 'ohio', 'grrrr': 'error', 'frens': 'frees', 'recei': 'race', 'cornes': 'corner', 'monti': 'month', 'izzi': 'izzie', 'orderrs': 'orders', 'daywine': 'daytime', 'phcity': 'city', 'hartmans': 'martians', 'gy': 'my', 'iga': 'ga', 'springbank': 'springbok', 'graamm': 'grams', 'bischof': 'bishop', 'mombles': 'mumbles', 'tsbarry': 'starry', 'nagaraj': 'nagara', '1list': 'list', 'fff': 'off', 'artificiality': 'artificially', 'nyo': 'no', 'polygon': 'polaron', 'varna': 'verna', 'bluedot': 'bluest', 'ukjobs': 'jobs', 'waldron': 'caldron', 'glasse': 'glass', 'chrgd': 'child', 'aata': 'data', 'offeri': 'offer', 'paas': 'pass', 'susann': 'susan', 'wali': 'walk', 'onnnn': 'onion', 'veepee': 'teepee', 'bronson': 'brandon', '4pm': 'up', 'encoura': 'encourage', 'taas': 'tabs', 'ykw': 'yaw', '20ft': 'left', 'ceridian': 'meridian', 'smc': 'sec', 'naners': 'niners', 'uprise': 'prise', 'braddy': 'brandy', 'obv': 'ob', 'anticar': 'antiwar', 'malnati': 'magnate', 'sameh': 'same', 'defnintely': 'defnitely', 'malmstrom': 'maelstrom', 'eumin': 'cumin', 'estab': 'stab', 'healthfully': 'healthful', 'caddiz': 'caddie', 'podge': 'lodge', 'prope': 'prove', 'onelxn': 'onel', 'expirin': 'expiring', 'folx': 'folk', 'ithaca': 'thata', 'marawic': 'arabic', 'peapods': 'weapons', 'updf': 'updo', '25kg': 'ekg', 'rescuedog': 'rescued', 'predefined': 'redefined', 'motiv': 'motive', 'fmcg': 'fuck', 'expirience': 'experience', 'obcs': 'orcs', 'funnymy': 'funny', 'jackdaw': 'jackal', 'canby': 'candy', 'buther': 'bother', 'dorito': 'dorian', 'wheez': 'wheel', '80lbs': 'bulbs', 'daythis': 'atthis', 'houshold': 'household', 'kpcb': 'kick', 'ignoranceno': 'ignorance', 'larg': 'large', 'pws': 'pas', 'centra': 'central', 'superhosts': 'superhot', 'worldy': 'world', 'happymy': 'happy', 'k94': 'kid', 'germa': 'german', 'onzies': 'fonzies', 'homr': 'home', 'rohini': 'robin', 'ratass': 'fatass', 'dito': 'dit', 'pamilya': 'family', 'bantins': 'wanting', 'shasta': 'hasta', 'andd': 'and', 'strachan': 'astrakhan', 'impatiens': 'impatient', 'pkdd': 'padd', 'searcy': 'search', 'p18de': 'pride', 'avie': 'vie', 'clg': 'cog', 'buyi': 'buy', 'swt': 'set', 'voad': 'road', 'timepoor': 'timefor', 'alviso': 'also', 'kubs': 'cubs', 'supermarkten': 'supermarket', 'mamaya': 'mama', 'rolser': 'roller', 'incentivises': 'incentives', 'eevee': 'levee', 'avoider': 'avoided', 'lesna': 'lena', 'want4as': 'wants', 'untedu': 'united', 'renaah': 'reach', 'farmway': 'faraway', 'quartine': 'quarantine', '19yo': 'yo', 'freegans': 'freeways', 'auspiciously': 'suspiciously', 'baqi': 'bai', 'buiiding': 'building', 'uikit': 'visit', 'surpriseey': 'surprise', 'sbma': 'sama', 'fmla': 'full', 'sahi': 'sari', '41yours': 'yours', 'tuazon': 'talon', 'magogos': 'maggots', 'cabfare': 'cabbage', 'entrepeneurs': 'entrepreneurs', 'kirkwood': 'firewood', 'richmondva': 'richmond', 'autie': 'auntie', 'santonis': 'antonio', 'sidetable': 'timetable', 'yqg': 'you', 'oyins': 'lying', 'curbsidep': 'curbside', 'alphonso': 'alphonse', 'investme': 'investment', 'damnnnn': 'damning', 'ammt': 'ammo', 'youheb': 'youse', 'tfw': 'tow', 'ybusiness': 'business', 'alldc': 'all', 'patchway': 'pathway', 'lumpia': 'lump', 'wendys': 'wendy', 'mchl': 'much', 'cowner': 'corner', 'chasi': 'chase', 'triflin': 'trifling', 'nyufa': 'nuna', 'gorz': 'gory', 'pva': 'pa', 'betz': 'bet', 'm1': 'my', 'crudites': 'credits', 'vm': 'am', 'cogc': 'cog', 'shloer': 'shoes', 'coronaph': 'coronary', 'listy': 'list', 'wahah': 'ahah', 'weasle': 'weasel', 'lilli': 'lilly', 'bumf': 'bump', 'yasai': 'nasal', 'qrt': 'art', 'durum': 'drum', 'yourk': 'your', 'fwp': 'fop', 'twatty': 'tatty', '88yo': 'yo', 'creandum': 'cranium', '4km': 'him', 'shawt': 'shalt', 'trax': 'tax', '21october': 'october', 'goodfood': 'godhood', 'civies': 'cities', 'larissa1': 'larissa', 'scheidler': 'schneider', 'reguest': 'request', 'tollway': 'hallway', 'seculuded': 'secluded', 'plotline': 'plotting', 'pm': 'am', 'pharmaceutic': 'pharmaceutical', 'amstedam': 'amsterdam', 'thr1ve': 'thrive', 'ballbust': 'ballast', 'chps': 'cops', 'malathi': 'malachi', 'hain': 'hair', 'todlr': 'told', 'classist': 'classiest', 'maxmart': 'mammary', 'aichain': 'chain', 'aseel': 'see', 'rattiest': 'prettiest', 'bggcon': 'bacon', 'apps4se': 'appease', 'hitech': 'hitch', 'totinos': 'toxins', 'syempre': 'sempre', 'empathlife': 'empathize', 'frickity': 'fricking', 'pgold': 'gold', 'radley': 'bradley', 'ousey': 'mousey', 'afganistan': 'afghanistan', 'adun': 'dun', 'wna': 'dna', 'yyyes': 'eyes', 'placon': 'place', 'chumlets': 'crumpets', 'tasmania': 'tasmanian', 'scomicron': 'omicron', 'derv': 'der', 'deliveron': 'deliver', '$aalp': 'scalp', 'vcsummit': 'summit', 'esocial': 'social', 'consi': 'cons', 'fightfor': 'fighter', 'agni': 'ani', 'aab': 'lab', 'sika': 'ska', 'rahui': 'rahul', 'savings10': 'savings', 'moti': 'moth', 'autobuy': 'autopsy', 'asheriffs': 'sheriffs', 'plantasia': 'plantain', 'hadden': 'hidden', 'lynas': 'lines', 'imfino': 'amino', 'veli': 'veil', 'durban': 'urban', 'tts': 'its', 'hhmph': 'hump', 'getnow': 'getyou', 'recorderings': 'recordings', 'rtn': 'run', 'sodie': 'sadie', 'riseup': 'rise', 'vps': 'ups', 'secto': 'sector', 'allyson': 'allison', 'revolut': 'revolt', 'chowca': 'choice', 'reddys': 'ready', 'shss': 'shes', 'kk': 'ok', 'roa': 'road', 'fr8': 'fry', 'armfuls': 'armful', 'screenshots': 'screenshot', 'sleeo': 'sleep', 'groveries': 'groceries', 'mdwoes': 'does', 'rainz': 'rain', 'carie': 'care', 'maciuba': 'marimba', 'fs': 'is', 'jadyn': 'lady', 'parboiled': 'paroled', 'emotes': 'remotes', 'misappropriate': 'misappropriated', 'awesomenew': 'awesomeness', 'basuke': 'base', 'fbh': 'fbi', 'eisenhowe': 'eisenhower', 'expeience': 'experience', 'killearn': 'killer', 't2t': 'tit', 'siders': 'sides', 'pagnell': 'panel', 'whatsaap': 'whatsapp', 'retagged': 'retarded', 'hux': 'hug', 'yo31': 'you', 'behar': 'bear', 'cautio': 'caution', 'naturopathy': 'neuropathy', 'mohali': 'moral', 'panaji': 'kanji', 'ucf': 'uf', 'ferraris': 'ferrari', 'sotp': 'stop', 'today50': 'today', 'cryst': 'crust', 'mcat': 'cat', 'nst': 'not', 'snowbird': 'songbird', 'grp': 'grip', 'kaki': 'khaki', 'deliveryday': 'deliveryman', 'longmont': 'longboat', 'papad': 'papa', 'virion': 'vision', 'harasays': 'harass', 'cadbury': 'asbury', 'qear': 'hear', 'cashewz': 'cashew', 'strtd': 'start', 'momsworld': 'homeworld', 'hadley': 'haley', 'golyve': 'solve', 'potrero': 'porter', '$bapr': 'bar', 'poduncks': 'podunk', 'apk': 'ask', '$acu': 'back', 'referer': 'referee', 'violette': 'violent', 'cklw': 'call', 'ps28': 'psi', 'tashina': 'washing', 'hlg': 'hug', 'vbm': 'vim', 'waalre': 'aware', 'jacobsons': 'jacksons', 'salfish': 'selfish', 'maeko': 'mako', 'gertz': 'hertz', 'imperfect20': 'imperfect', 'paklands': 'garlands', 'cino': 'vino', 'retrired': 'retired', 'curie': 'cure', 'conad': 'conrad', 'toke': 'take', 'gdq': 'do', 'sinh': 'sing', 'selfles': 'selfless', 'dolma': 'dogma', 'chatbot': 'chariot', 'qrted': 'tried', '$e': 'we', 'bobi': 'bob', 'thurston': 'thornton', 'cummins': 'cummings', 'bmb': 'bomb', 'dyhm': 'gym', 'hussle': 'hustle', '4kg': 'ekg', 'businesess': 'businesses', 'vansales': 'vandals', 'instcart': 'instant', 'yipeeee': 'yippee', 'wehappy': 'happy', 'pepto': 'pesto', 'accompying': 'accompanying', 'wants2': 'wants', 'angrys': 'angry', 'freezies': 'freezes', 'ramyeon': 'ramen', 'randoms': 'random', 'nov8th': 'north', '4all': 'all', 'intol': 'into', 'rmrealtor': 'realtor', 'face78': 'face', 'whle': 'whole', 'saare': 'share', 'alimohd': 'almond', 'ps40': 'psi', 'eonomics': 'economics', 'pl19': 'play', 'vapi': 'papi', 'infectivity': 'infertility', 'sope': 'some', 'plsmy': 'play', 'heit': 'hit', 'whts': 'whats', 'cranleigh': 'raleigh', 'snk': 'sink', 'dongle': 'dangle', 'ev1': 'eve', 'urbanisation': 'urbanization', 'harrogate': 'surrogate', 'tude': 'dude', 'streit': 'street', 'techworld': 'theworld', 'hamble': 'humble', 'ceries': 'series', '$tech': 'tech', 'sies': 'sees', 'fingal': 'final', 'lambuk': 'lamb', 'sootired': 'soothed', 'phonepe': 'phone', 'trivially': 'trivial', 'zevia': 'evil', 'kauppa': 'kappa', 'jaisay': 'daisy', 'leadship': 'leadership', 'suxx': 'sux', 'deivery': 'delivery', 'shmoe': 'shoe', 'eulo': 'euro', 'dup': 'up', 'campai': 'campari', 'comox': 'como', 'daus': 'days', 'ps8': 'psi', 'zocalo': 'local', 'ocado': 'condo', 'utahns': 'stains', 'droppa': 'drop', 'mp': 'my', 'sgn': 'son', 'mandis': 'mantis', 'noguilt': 'guilt', 'lunc': 'lunch', 'wcvb': 'wave', 'darrion': 'carrion', 'dafuk': 'dark', 'ricoh': 'rich', 'holme': 'home', 'illi': 'ill', 'newswire': 'newsie', 'homeb': 'home', 'dilligent': 'diligent', 'buch': 'much', 'fox17': 'fox', 'shikon': 'shaken', '80f': 'of', 'bansali': 'banal', 'lawfirm': 'affirm', 'mlezi': 'lei', 'vtp': 'vip', '19th': 'with', 'boujee': 'bounce', 'kano': 'kono', 'byu': 'by', 'payton': 'peyton', 'palli': 'pall', 'suntory': 'story', 'eskin': 'skin', 'etches': 'etched', 'insaid': 'unsaid', 'betwee': 'between', 'badami': 'madam', 'chesler': 'chester', 'faltu': 'felt', 'alisa': 'alias', 'excusee': 'excuse', 'teamsave': 'teammate', 'bowood': 'blood', 'asli': 'ali', 'anticip': 'antics', 'allsec': 'alley', 'mirc': 'mic', 'mek': 'me', 'odf3': 'of', 'bisbee': 'bible', 'pt3': 'pta', 'planb': 'plan', 'celina': 'celine', 'zia': 'via', 'iesocial': 'social', 'linux': 'linus', 'whr': 'who', 'okanagan': 'oranaga', 'cheezits': 'cheerios', 'customapp': 'customary', 'ayoo': 'yoo', 'pandemic]': 'pandemic', 'unka': 'una', 'experienc': 'experience', 'entreprene': 'entrepreneur', 'uncancel': 'cancel', 'yass': 'ass', 'cnsmoms': 'customs', 'istead': 'instead', 'broo': 'broom', 'dmo': 'do', 'choudhury': 'chaudhary', 'opticians': 'optician', 'yomoney': 'money', 'fnc': 'fac', 'diddlin': 'diddling', 'kshs': 'she', 'immmm': 'imam', 'billers': 'killers', 'rozi': 'roz', 'evolu': 'evil', 'tiering': 'tiring', 'omboarded': 'boarded', 'hernando': 'fernando', '2kilometers': 'kilometers', 'richert': 'richest', 'bhasa': 'hasa', 'friaro': 'friar', 'rica': 'rich', 'cllr': 'all', 'machina': 'machine', 'uif': 'if', 'elitists': 'elitist', 'oligarchal': 'oligarchs', 'akatale': 'fatale', 'pineglo': 'pineal', 'pickip': 'pickup', 'morecycle': 'recycle', 'mnc': 'mic', 'mtgo': 'to', 'scadding': 'scalding', 'yaass': 'years', 'groceriesgh': 'groceries', 'yand': 'and', 'roomates': 'roommates', 'kushiwo': 'bushido', 'joco': 'jock', 'cheeto': 'cheetos', 'fadeke': 'fake', 'enugu': 'enough', 'ddn': 'don', 'threfts': 'threats', 'technolog': 'technology', 'shld': 'sold', '03rd': 'hard', 'ritu': 'rite', 'hubz': 'hub', 'firstlight': 'fistfight', 'vmail': 'mail', 'monetise': 'monetize', 'cheesus': 'cheeses', 'eno': 'no', 'lmaooo': 'ladoo', '1k': 'ok', 'weakend': 'weekend', 'foamite': 'famine', 'homesteps': 'homestead', 'matar': 'mater', 'coface': 'face', 'yegcc': 'yegg', '99month': 'month', 'pui': 'put', 'tcdisrupt': 'disrupt', 'oikawa': 'kawa', 'howeever': 'however', 'htt': 'hit', 'hs': 'is', 'ridic': 'ride', 'sidenote': 'denote', 'hout': 'out', 'ftp': 'fop', '3bn': 'ibn', 'dtlive': 'live', 'returnin': 'returning', 'nms': 'nos', '4mins': 'mins', 'prsummit': 'summit', 'egor': 'ego', 'rellenos': "ellen's", 'telemed': 'teemed', 'zos': 'cos', 'ilsang': 'island', 'tmom': 'mom', 'voterid': 'voted', 'canupls': 'canals', 'vmts': 'vets', 'bfa': 'bra', 'kirstyn': 'kirsten', '$food': 'food', 'calabar': 'caliber', '40ft': 'left', 'roza': 'rosa', 'olson': 'orson', 'kn94': 'know', 'p2': 'pa', 'js': 'is', 'mrsking': 'masking', 'gabriela': 'gabriel', 'handi': 'hands', 'reute': 'route', 'fgrs': 'furs', 'paymaya': 'payday', 'powerup': 'power', 'vinai': 'vinci', 'part3': 'part', 'memphistn': 'memphis', 'goong': 'going', 'mygroceries': 'groceries', 'advertisi': 'advertise', '$itri': 'nitro', 'snn': 'son', 'peple': 'people', 'teshie': 'techie', 'ourf9': 'our', 'required]': 'required', 'pajammies': 'jammies', 'delimit': 'limit', 'ytd': 'yid', 'muhh': 'much', '12am': 'am', 'bcz': 'biz', '01may': 'may', 'hingham': 'gingham', '20m': 'him', 'kombat': 'combat', 'decreasy': 'decrease', '18hr': 'sehr', 'ajnara': 'anata', 'cnbc': 'cab', 'goe': 'got', 'chileee': 'chinese', 'jingled': 'jingle', 'tisbury': 'asbury', 'grof': 'grow', 'twi': 'two', 'oslo': 'solo', 'hollaback': 'holdback', 'myhala': 'mala', 'pasttt': 'past', 'negiborhood': 'neighborhood', 'recluses': 'recluse', 'sandeep': 'sanders', 'thomson': 'thompson', 'shownu': 'shown', 'anyaays': 'anyways', 'part2': 'part', 'trialling': 'trilling', 'bajar': 'ajar', 'kindle]': 'kindle', '70m': 'him', 'optimusbt': 'optimist', 'humex': 'home', 'f21': 'for', 'olmito': 'omit', 'ansari': 'sari', 'ugandan': 'ugandans', 'yeey': 'yee', '50x': 'six', 'orderer4d': 'ordered', 'rsgb': 'rob', 'colombi': 'colombia', 'dominios': 'dominion', 'siong': 'sing', 'trac': 'track', 'acquihires': 'acquires', 'oceanex': 'ocean', 'bne': 'be', 'thereee': 'therese', 'nci': 'ni', 'xcd': 'ocd', 'idynamo6': 'dynamo', 'bescom': 'become', 'jamjams': 'jamais', 'kna': 'dna', 'jmp': 'jump', 'walkouts': 'walkout', 'tcv': 'tov', 'stickiness': 'sickness', 'rakhne': 'rake', 'keem': 'keep', 'tonig': 'tonic', 'rushin': 'rushing', 'llamanos': 'llamas', 'fos': 'for', 'impossibl': 'impossible', 'loudon': 'london', 'muricas': 'musical', 'sk7': 'sky', 'tommrw': 'tommy', 'alvera': 'alvers', 'jeesh': 'fresh', 'snakk': 'snake', 'orderained': 'ordained', 'tuju': 'juju', 'npl': 'nil', 'spaghet': 'spaghetti', 'unassign': 'assign', 'wnba': 'nba', 'cusd': 'cuss', 'bayboro': 'bamboo', 'aticles': 'articles', 'pada': 'papa', 'bucko': 'bucks', 'masr': 'mass', 'starstock': 'starstruck', 'aoda': 'soda', 'saveworkers': 'caseworkers', 'refrozen': 'frozen', 'chcking': 'checking', 'nhm': 'nom', 'palomita': 'palomino', 'bst': 'but', 'statt': 'start', 'eastlands': 'wastelands', 'eurusd': 'eurus', 'krakow': 'kraken', 'ishi': 'shi', 'pushkart': 'pushcart', 'sayer': 'safer', 'nelia': 'celia', 'nmecon': 'recon', 'nfkeys': 'keys', '$sant': 'sant', 'cancx': 'can', 'happeninggg': 'happening', 'haili': 'hail', 'downtowncs': "downtown's", 'gmfu': 'gift', 'caturday': 'saturday', 'asynchronous': 'synchronous', 'rotts': 'roots', 'cxld': 'cold', 'backery': 'bakery', 'gasolina': 'gasoline', 'ubere': 'uber', 'huckabee': 'fuckable', 'bf': 'of', 'pelleted': 'deleted', 'fungible': 'tangible', 'morningish': "morning's", 'ecity': 'city', 'rattigan': 'ratting', 'aba': 'ba', 'video]': 'video', 'cccheckup': 'checkup', 'isleta': 'islet', 'ignorancewe': 'ignorance', '1mnth': 'month', 'sotexas': 'texas', 'kenesha': 'keisha', 'medias': 'media', 'liskai': 'lisa', 'ballinger': 'balling', 'ndio': 'dio', 'exposue': 'expose', 'sharpish': 'sharpest', 'cbs': 'cos', 'collabora': 'collaborate', 'baldor': 'balder', 'premades': 'premises', 'adiba': 'alibi', 'wbtv': 'with', 'falce': 'face', 'cxling': 'cling', 'freekin': 'freakin', 'foodlink': 'fooling', 'sikh': 'sigh', 'adicted': 'addicted', 'myland': 'ryland', 'medciens': 'medicines', 'waitingon': 'waiting', 'rodelio': 'rogelio', 'wgsa': 'wasa', 'mnipee': 'sniper', 'kayso': 'also', 'viril': 'viral', 'tsentse': 'tsetse', 'messings': 'messing', 'happynow': 'happyto', 'gmnw': 'gone', 'crus': 'crush', 'aztogether': 'altogether', 'irresponsble': 'irresponsible', 'anyo': 'any', 'communit': 'community', 'kaimey': 'aimed', 'kase': 'case', 'lolz': 'lola', 'hermosa': 'thermos', 'caribbe': 'calibre', 'goofys': 'goofy', 'obnocious': 'obnoxious', 'haggen': 'hagen', 'gorebels': 'rebels', 'rs25': 'rse', 'iverve': 'verve', 'suprmrket': 'supermarket', 'zivo': 'tivo', 'kayq': 'kay', 'aboutif': 'about', 'seabikes': 'seabirds', 'ummmm': 'ummm', 'biyani': 'biryani', 'lolipop': 'lollipop', 'payym': 'pay', '2cust': 'just', 'jobc': 'job', 'multics': 'celtics', 'fummend': 'commend', 'unfollowed': 'followed', 'etd': 'end', 'shlw': 'show', 'mohit': 'rohit', 'yeovil': 'evil', 'excercing': 'exceeding', 'elio': 'eli', 'wesn': 'west', 'whizbox': 'shitbox', '5l': 'al', 'whyme': 'rhyme', 'seb': 'see', 'pitchbook': 'pitchfork', 'dahi': 'dah', 'tyl': 'til', 'suspring': 'spring', 'kuri': 'yuri', 'kla': 'la', 'wtf': 'wif', 'groupe': 'group', 'demat': 'dead', 'pantone': 'anyone', 'khela': 'help', '15mts': 'emts', 'pz': 'pa', 'rayome': 'rhyme', 'kuai': 'kai', 'lexing': 'flexing', 'appnt': 'spent', 'sabse': 'sable', 'genoa': 'geno', 'delievries': 'deliveries', 'mecfs': 'mechs', 'zomato': 'tomato', 'jnu': 'nu', 'cdc': 'do', 'parkridge': 'partridge', 'informatio': 'information', 'over1hr': 'overthe', 'waykart': 'wayward', 'ocadao': 'cacao', 'phnchny': 'punchy', 'thie': 'the', 'exprience': 'experience', 'poptimize': 'optimize', 'etail': 'tail', 'netde': 'need', 'jana': 'jane', 'freebpm': 'freedom', 'whixh': 'which', 'awesomebuy': 'awesomely', 'deyanna': 'deanna', 'sendiks': 'sending', 'kindergartener': 'kindergarteners', 'ampfw': 'ample', 'olace': 'place', 'etcs': 'etc', 'inflati': 'inflate', 'femi': 'semi', 'sayit': 'sayin', 'saste': 'waste', 'newish': 'jewish', '$sing': 'using', 'pechs': 'techs', 'yalantis': 'atlantis', 'r34': 'run', 'beachwood': 'blackwood', 'haynets': 'haynes', 'cielo': 'cell', '$unam': 'tuna', 'aite': 'ate', 'gloveg': 'gloves', 'roskill': 'skill', 'reit': 'rest', 'afia': 'mafia', '20th': 'with', 'ubrs': 'us', 'kenora': 'senora', 'wreckem': 'wrecked', 'uplate': 'plate', 'nyuu': 'yuu', 'funnyi': 'funny', 'uwu': 'wu', 'sanitiser': 'sanitizer', 'sba': 'sea', 'cleani': 'clean', 'circumfuse': 'circumcise', 'mmarkets': 'markets', 'whateva': 'whatever', 'etimad': 'timed', 'homemaking': 'lovemaking', 'grocwries': 'groceries', 'dfw': 'dew', 'dispare': 'spare', 'compamy': 'company', 'luyos': 'guys', 'spounds': 'sounds', 'jc': 'je', 'guarateed': 'guaranteed', 'wheeew': 'where', 'shpt': 'shut', 'helpfull': 'helpful', 'azeri': 'zero', 'substituded': 'substituted', '6$': 'i', 'redcycle': 'recycle', 'fiscus': 'ficus', 'tvquotes': 'quotes', 'teamky': 'team', '1sr': 'is', 'oakhouse': 'outhouse', 'nauseaum': 'nauseam', 'privatised': 'privatized', 'wynonna': 'wonna', 'ansecond': 'second', 'ros': 'cos', 'hort': 'hurt', 'ka01': 'ka', 'elland': 'leland', 'reinke': 'rinse', 'urgrocer': 'grocer', 'golhi': 'gold', 'hosur': 'hour', 'followd': 'follow', '90d': 'and', 'newnan': 'newman', 'npr': 'nor', 'yeaahh': 'yeah', 'coronaai': 'coronary', 'tomrorow': 'tomorrow', 'etsy': 'easy', 'southlondo': 'southland', 'tapwater': 'atwater', 'uwbdn': 'urban', 'davao': 'data', 'takeouts': 'takeout', 'ec2a': 'era', 'ribeyes': "riley's", 'eatmeat': 'oatmeal', 'saaman': 'shaman', 'ondemand': 'demand', 'wwait': 'wait', 'aftwr': 'after', 'prominatly': 'prominently', 'cheif': 'chief', 'postmaids': 'postcards', 'petsitting': 'permitting', 'concer': 'concern', 'kunjee': 'knee', 'mamasab': 'mamas', 'glvs': 'glve', 'projecthills': 'projectiles', 'sujal': 'steal', 'ibadan': 'bada', 'tiff16': 'tiff', 'surreybc': 'surrey', 'peopel': 'people', 'ward3': 'ward', 'leveler': 'leveled', 'degreed': 'degrees', 'posten': 'posted', 'kidd': 'kind', 'indiidual': 'individual', 'onshoring': 'shoring', '2october': 'october', 'terkedu': 'jerked', 'loveand': 'loved', 'pei': 'per', 'dumka': 'dumb', 'namkum': 'wampum', 'mrkt': 'make', 'maira': 'maria', 'typers': 'types', 'misson': 'mission', 'darragh': 'barrage', 'dsa': 'dna', '50ish': 'wish', 'mirecycles': 'recycles', 'lpn': 'lan', 'grovery': 'grocery', 'redcycling': 'recycling', 'fkd': 'fed', '40your': 'your', 'myphx': 'myth', 'pxcat': 'cat', 'murape': 'rape', 'gcfu': 'gift', 'nylag': 'flag', 'ekirana': 'kiran', 'camry': 'carry', 'jeffree': 'jeffrey', 'abyssmally': 'abysmally', 'alertwe': 'alert', 'rgn': 'run', 'shpuld': 'should', '7x': 'ex', 'kaddour': 'ardour', 'bonchak': 'buncha', 'elon': 'melon', 'shamone': 'shame', 'wch': 'ich', 'cochrane': 'cochran', 'decore': 'decode', 'inaantay': 'infantry', 'nicas': 'nicks', 'x64': 'xu', '3cars': 'cars', 'platanos': 'platoons', 'gparents': 'parents', 'dpd': 'did', 'shidou': 'shiyou', 'asg': 'as', 'gatchu': 'watch', '50yo': 'yo', 'ttt': 'tit', 'unvaxd': 'unpaid', 'noata': 'nota', 'treumph': 'triumph', 'crowlers': 'prowlers', 'jever': 'never', 'isaa': 'isa', 'izzay': 'izzy', 'minis': 'minds', 'alcs': 'alms', 'cannibas': 'cannibal', 'xconomy': 'economy', '51st': 'just', 'jac': 'jam', 'insura': 'insure', 'ables': 'able', 'immigrantw': 'immigrants', 'cbridge': 'bridge', 'newtome': 'newcomer', 'goodr': 'good', 'waal': 'wall', 'cmonman': 'conman', 'backroads': 'backwoods', 'accio': 'action', 'sadrifice': 'sacrifice', 'rohnert': 'robert', 'pacifi': 'pacific', 'poopers': 'pooper', 'taal': 'tall', 'orderered': 'ordered', 'infinitude': 'infinite', 'jp': 'up', 'achive': 'active', 'gorder': 'order', 'chauntel': 'channel', 'shri': 'sari', 'migjt': 'might', 'rasht': 'rash', '80m': 'him', 'defend2a': 'defend', 'smexaminer': 'examiner', 'quikk': 'quick', 'stoppedt': 'stopped', 'sapa': 'papa', 'hunkerin': 'hunkering', 'apis': 'apes', 'chana': 'china', '6miles': 'miles', 'proscuitto': 'prosciutto', '2your': 'your', 'r8': 're', 'maddux': 'maddox', 'resourcing': 'resorting', 'ederly': 'elderly', 'wtkr': 'work', 'izs': 'is', 'accorderion': 'accordion', 'techni': 'techno', 'rbob': 'rob', 'chg': 'chi', 'nlwx': 'now', 'fleche': 'leche', 'asos': 'ass', 'tiziana': 'titian', 'stroodles': 'straddles', 'fyndus': 'finds', 'bostonian': 'bosnian', 'volnation': 'violation', 'ltts': 'lots', 'ifclient': 'client', 'yegfood': 'seafood', 'strang': 'strong', '25lb': 'bulb', 'welle': 'well', 'prana': 'prank', 'autopay': 'autopsy', 'azng': 'ang', 'grosso': 'gross', 'ordere': 'order', 'modder': 'madder', 'disorganise': 'disorganised', 'jerseyans': 'jerseys', 'farcry': 'marry', 'cuervo': 'cuerpo', 'japa': 'papa', 'stellina': 'telling', 'vjestica': 'jessica', 'norwich': 'nordic', 'fgs': 'fags', 'temptee': 'tempted', 'marzi': 'mardi', '$ahpi': 'papi', 'betwn': 'been', '4k': 'ok', 'congest': 'contest', 'taiyaki': 'teriyaki', 'bargen': 'barge', 'bc2': 'be', 'togethee': 'together', 'sahur': 'sahir', 'pmg': 'pig', '5w': 'ow', 'introducded': 'introduced', 'ctsc': 'its', 'dtlv': 'doll', 'monthyl': 'monthly', '85ph': 'syph', 'txwx': 'two', 'anant': 'avant', 'momylife': 'mylife', 'marjoram': 'marjorie', 'vogo': 'logo', 'adaquatly': 'adequately', 'usmart': 'smart', 'muscat': 'must', 'shouod': 'should', 'medillin': 'medallion', 'fkt': 'fit', 'pring': 'bring', 'cooley': 'cooler', 'racialized': 'radicalized', 'handz': 'hands', '10am': 'am', 'opennj': 'open', 'lbusd': 'busy', '30mph': 'lymph', 'ssdi': 'ssi', 'reinsurance': 'insurance', 'hooley': 'hooey', 'ias': 'is', 'accou': 'account', 'nigit': 'night', 'b2': 'be', 'sfre': 'sure', 'mhealth': 'health', 'sum1': 'sum', 'nusha': 'push', 'dtdc': 'did', 'bxl': 'bel', 'selfd': 'self', 'sadand': 'stand', 'massoud': 'masseur', 'aca9': 'acai', 'stoomuch': 'stomach', 'msft': 'must', 'neurofen': 'neuron', 'nice06': 'nice', '$juno': 'juno', 'tuckkkers': 'tuckers', 'stockpilling': 'stockpiling', 'heyer': 'meyer', 'embarass': 'embarrass', 'kodi': 'koi', 'tulare': 'flare', 'dabur': 'labor', 'canem': 'cane', 'can95s': 'cans', 'zoomo': 'zoom', 'dollarsph': 'dollars', 'yoplait': 'plait', '16ps': 'cops', 'juicery': 'juicer', '53am': 'am', 'usamazon': 'amazon', 'belon': 'belong', 'callit': 'callie', 'dvcer': 'over', 'briden': 'bride', '$fcel': 'feel', 'newyears': 'fewyears', 'rosoff': 'roof', 'noro': 'nor', 'lacara': 'laura', 'barrera': 'barrel', 'udistrict': 'district', 'utahn': 'utah', 'phoeni': 'phoenix', 'expla': 'explain', 'missin': 'missing', 'aier': 'air', 'cuck': 'fuck', 'lsit': 'sit', 'mindstream': 'mainstream', 'sangrona': 'sangria', 'milla': 'mill', 'kwae': 'kwan', 'mathews': 'matthews', 'mygrocer': 'grocer', 'sads': 'says', 'superette': 'usherette', 'pasig': 'past', 'sharswood': 'hardwood', 'haus': 'has', 'try8ing': 'trying', 'dhruv': 'drug', 'biofeld': 'biofuel', 'musubi': 'musuu', 'socinn': 'social', 'freshdi': 'fresh', 'feelbad': 'freeload', 'rootle': 'rootie', 'blynk': 'blank', 'jacksy': 'jacks', 'brendlen': 'brendan', 'trudel': 'strudel', 'vivi': 'vivid', 'vaxed': 'faxed', 'list]': 'list', 'madh': 'made', 'shutin': 'shut', 'nicci': 'nicki', 'invines': 'invites', 'mar9': 'mark', 'blini': 'blind', 'fulla': 'full', 'liit': 'list', 'luckmy': 'lucky', 'xmastreats': 'mistreats', 'thewell': 'thehell', 'islandny': 'island', 'cmyhoice': 'choice', 'bandn': 'band', 'jaha': 'aha', 'minimim': 'minimum', 'ndii': 'did', 'tteok': 'took', 'hilori': 'hitori', 'wokemob': 'pokemon', 'unkept': 'unkempt', 'ensalz': 'ersatz', 'sambosa': 'samosa', 'iget': 'get', 'spent50': 'spent', 'nasir': 'nadir', 'lanoding': 'landing', '28yo': 'yo', 'conver': 'cover', 'mehn': 'mean', 'sruff': 'stuff', 'mushin': 'muslin', 'cookin': 'cooking', 'reezy': 'breezy', 'carshow': 'cashew', 'sakta': 'santa', 'commiss': 'commits', 'mamdani': 'madman', 'govinda': 'goinna', 'failled': 'filled', 'coborn': 'born', 'kupritz': 'spritz', 'h1n1': 'hand', 'samin': 'satin', 'cookunity': 'community', 'jdon': 'don', 'pajayjays': 'vajayjay', 'seiso': 'seis', 'expa': 'expo', 'otoh': 'ooh', 'qumbu': 'dumb', 'darl': 'dark', 'moodle': 'noodle', 'school]': 'school', 'grimsby': 'grisly', 'beforw': 'before', 'txn': 'ten', 'scoopers': 'scooters', 'co2': 'cop', 'doritos': 'doris', 'onesg': 'ones', 'nodinner': 'dinner', 'rnb': 'rob', 'wip': 'win', 'oura': 'our', 'pulekss': 'pulses', '$amzn': 'damn', 'coburns': 'burns', 'wabc': 'abc', 'painpoints': 'pinpoints', 'thaay': 'thay', 'akho': 'ako', 'cjng': 'cong', 'empowermint': 'empowerment', 'sanne': 'sane', 'luker': 'luke', 'shet': 'she', 'umc': 'um', 'unvaccinated': 'vaccinated', 'businessnews': 'businesses', 'tima': 'time', 'rl': 're', 'yog': 'you', 'cancell': 'cancel', 'baap': 'baa', 'deliverying': 'delivering', 'titas': 'tits', 'runnjng': 'running', 'griib': 'grab', 'healthnut': 'healthful', 'rapido': 'rapid', 'magoos': 'wagons', 'saks': 'says', 'gamification': 'ramifications', 'avleats': 'atleast', 'yumyum': 'yummy', 'fyii': 'faii', 'tamad': 'tamed', 'ohrc': 'orc', 'billa': 'bill', 'gbp': 'gap', 'ecomto': 'combo', 'perbside': 'preside', 'purolator': 'percolator', 'bagong': 'along', 'rece': 'race', 'sould': 'would', 'ifc': 'if', 'r33': 'run', 'kirana': 'kiran', 'pofitec': 'polite', 'call]': 'call', 's2g': 'sig', 'scamsters': 'hamsters', 'baylogic': 'biologic', 'survi': 'suri', 'lbx': 'lex', 'keepsafe': 'keepsake', 'phonecalls': 'phonecall', 'greenways': 'freeways', 'talki': 'talk', 'abs]': 'abs', '35f': 'of', 'dcstrong': 'strong', 'mayur': 'mayor', 'aiart': 'apart', 'cryp': 'cry', 'nolines': 'lines', 'fortinos': 'fortunes', 'goel': 'goes', 'wpgear': 'wear', 'aaand': 'anand', 'honge': 'hone', 'massie': 'massive', 'laina': 'lain', 'dailyne': 'daily', 'pdfs': 'pass', 'jimy': 'jimmy', 'tuk': 'tub', '$amd': 'amd', 'walang': 'waling', 'foreste': 'forest', 'tellpat': 'telepath', 'smss': 'sass', 'rort': 'sort', 'cw': 'cow', 'molope': 'mole', 'wny': 'why', '$dher': 'her', 'delibered': 'delivered', 'mongodb': 'mongols', 'lonnng': 'long', 'aec': 'ace', 'barcamp': 'barca', 'shewins': 'showing', 'fp': 'up', 'thisis34': 'thisis', 'komemame': 'homemade', '8min': 'min', 'jll': 'all', 'malta': 'malt', 'merrick': 'derrick', 'alnighter': 'lighter', 'foid': 'food', 'ahu': 'ah', 'theresss': "there's", '3your': 'your', 'butwhy': 'bitchy', 'hamo': 'ham', 'automatedit': 'automated', 'catherinesh': 'catherine', 'imk': 'ink', 'avilable': 'available', 'mohapi': 'mohair', 'dm': 'do', 'jhand': 'hand', 'plss': 'pass', 'reportedl': 'reported', 'unavailables': 'unavailable', 'beto': 'bet', 'ugadi': 'gai', 'miltonon': 'milton', 'momther': 'mother', 'gsm': 'gym', '$wnw': 'own', '$uavs': 'suave', 'cauda': 'cada', 'gmc': 'my', 'andrzej': 'andrew', 'buca': 'buck', 'alipay': 'alias', 'corona20': 'coronary', 'naias': 'nails', '7gt': 'it', 'huslte': 'hustle', 'ratana': 'katana', 'registerone': 'registering', '1y': 'my', 'waterer': 'watered', 'ostracise': 'ostracize', 'iann': 'inn', 'cheeze': 'cheese', 'left]': 'left', 'theneeds': 'thneed', 'forver': 'forever', '3k': 'ok', 'maskedup': 'makeup', 'columbiasc': 'columbian', 'lovejess': 'loveless', 'avtechnology': 'technology', 'hwasa': 'wasa', 'bossread': 'bossed', 'ukbusiness': 'business', 'veglife': 'veggie', 'blo': 'blow', 'gardentips': 'gardenias', 'spesifically': 'specifically', 'lovethe': 'loveth', 'acatually': 'actually', 'freshne': 'freshen', 'ximb': 'limb', 'neosho': 'nosh', 'genshin': 'gunship', 'helpin': 'helping', 'hotboxed': 'hotbed', '20am': 'am', 'asshats': 'asshat', 'airmiles': 'armies', 'nolunch': 'lunch', 'honeycombers': 'honeycombed', 'sturgis': 'surges', 'broughton': 'brought', 'holi': 'hold', 'fmtnews': 'fitness', 'adhar': 'dear', 'coolish': 'foolish', 'watch]': 'watch', 'qtna': 'tea', 'wibta': 'with', 'yate': 'hate', '2degc': 'dec', 'winestore': 'windstorm', 'smedley': 'medley', 'visi': 'visit', 'givi': 'give', 'lable': 'able', 'impo': 'imp', 'smpd': 'said', 'f22s': 'fits', 'q6': 'qi', 'grocerys': 'grocery', 'peta': 'pet', 'post18': 'post', 'rahe': 'race', 'gep': 'get', 'nobbly': 'wobbly', 'yaad': 'yard', 'giggers': 'niggers', 'shayas': 'says', 'crd': 'card', 'intere': 'intern', 'bayar': 'boyar', 'skokie': 'sookie', 'witn': 'with', 'myer': 'yer', 'demarco': 'marco', 'madinah': 'marina', 'abiola': 'viola', 'raftery': 'rafters', 'onpavement': 'pavement', 'njsen': 'nose', 'weidler': 'wider', 'kurtz': 'kurt', 'ganon': 'canon', 'swla': 'sola', 'hsa': 'has', '4lb': 'all', 'postdoc': 'postop', 'heyo': 'hey', 'filipi': 'filii', 'immedaitely': 'immediately', 'licen': 'linen', 'spoked': 'spoke', 'fendt': 'fend', '$amz': 'am', 'beras': 'bears', 'ventur': 'venture', 'klusters': 'clusters', 'pcprovisions': 'provisions', 'gcs': 'gas', 'darrien': 'darren', '4d': 'ad', 'doshi': 'hoshi', 'speciaali': 'special', 'moretesting': 'retesting', 'randalls': 'randall', 'tinda': 'kinda', 'venutre': 'venture', 'dhec': 'dhe', 'dallam': 'dallas', 'ppl': 'pal', 'temas': 'teams', 'visualai': 'visual', 'dentons': 'centons', 'acmpr': 'camp', 'strted': 'started', 'volution': 'solution', 'gizri': 'girl', 'motb': 'mob', 'sassi': 'sassy', 'underlie': 'underline', 'durino': 'during', 'lbh': 'lah', '3oz': 'coz', 'hazelton': 'hamilton', 'motho': 'motto', 'bagley': 'barley', 'warda': 'ward', 'dlib': 'glib', 'phuck': 'chuck', 'workha': 'work', 'whatev': 'what', 'agdq': 'and', 'claritin': 'clarity', 'barstore': 'baritone', 'dilo': 'kilo', 'scedul': 'schedule', 'reaon': 'reason', 'vaxers': 'papers', 'zappos': 'zapped', 'vul': 'vu', 'goode': 'good', 'ahd': 'and', 'stoor': 'stood', 'tla': 'tea', '6your': 'your', 'tseng': 'tsing', 'livewire': 'likewise', 'changerr': 'changer', 'mcentee': 'mentee', '15k': 'ask', '$ipo': 'lipo', 'familiy': 'family', 'aberdeensd': 'aberdeen', 'notmuch': 'notch', 'chronlaw': 'chronal', 'naah': 'nah', 'localities': 'loyalties', 'njan': 'jan', 'timewarp': 'timecard', 'djsbu': 'desu', 'incepted': 'accepted', 'prabhu': 'raghu', 'snyders': 'snyder', 'presciptions': 'prescriptions', 'makhosi': 'machos', 'wreathes': 'breathes', 'absolutel': 'absolutely', 'tbis': 'this', 'kindom': 'kingdom', 'jakarta': 'dakara', 'gaia': 'gain', 'cashpig': 'cashing', 'cano': 'can', 'wr': 'we', 'coronawin': 'coronation', 'hjf': 'haf', 'baloons': 'balloons', 'untick': 'uptick', 'fwd': 'fed', 'mejiers': 'members', 'zipto': 'into', 'pickedup': 'picked', 'doreos': 'does', 'oten': 'open', 'gravyless': 'graceless', 'mckeen': 'screen', 'instal': 'install', 'praguers': 'prayers', 'caffine': 'caffeine', 'tht': 'the', 'shophard': 'shepherd', 'within60': 'within', 'mudit': 'audit', 'knutson': 'nutso', 'kokind': 'kind', 'f7': 'fu', 'querida': 'queria', 'hsd': 'had', 'news18': 'news', 'technol': 'techno', 'measureable': 'measurable', 'nita': 'rita', 'joyanne': 'joanne', 'woodway': 'doorway', 'conejo': 'condo', 'epf': 'elf', 'rochers': 'rockers', 'sayfie': 'safe', 'beginni': 'begin', 'neibours': 'neighbours', 'gophr': 'gopher', 'gorman': 'german', 'masak': 'mask', 'cartstan': 'cartman', 'stoppin': 'stopping', 'kmov': 'mov', 'u]': 'up', 'mvfb': 'mob', 'peole': 'people', 'logisti': 'logistic', 'heide': 'hide', 'definietly': 'definitely', 'houtech': 'hutch', 'routiene': 'routine', 'wwyd': 'why', 'bbqt': 'but', 'pro3': 'pro', 'chaoyu': 'chaos', 'whoes': "who's", 'completeh': 'complete', 'feeddc': 'feed', 'westbank': 'wetback', 'bizde': 'bide', 'wjth': 'with', 'widescale': 'timescale', 'dique': 'pique', 'rol': 'roll', 'bourgie': 'bougie', 'bigda': 'big', 'esquivel': 'esquire', 'bardez': 'harder', 'ca33': 'can', 'livechat': 'likethat', 'timespan': 'lifespan', 'yyt': 'yet', '25l': 'all', 'mauinews': 'marines', 'immunoc': 'immune', 'grafton': 'graft', 'peleton': 'skeleton', '3month': 'month', 'motorise': 'motorist', 'rundles': 'bundles', 'ps60': 'psi', 'vegease': 'release', 'hodgkin': 'hodgins', 'dlvry': 'dry', 'technologi': 'technology', 'bethune': 'methane', 'transfonum': 'transform', 'whaat': 'what', 'tommorrow': 'tomorrow', 'borks': 'works', 'katch': 'watch', 'opon': 'open', 'cryptocrash': 'cryptograph', '$fare': 'fare', 'trumpgate': 'jumpgate', 'sandwick': 'sandwich', 'delivery01': 'delivery', 'fmnp': 'find', 'hewit': 'hewitt', 'taguig': 'tagging', 'vw': 'vow', 'lykm': 'like', 'dibber': 'jibber', 'amex': 'amen', '21a': 'a', 'trumpers': 'trumpets', 'askar': 'oskar', 'earthgang': 'earthling', 'hahaahah': 'hahaha', 'gaas': 'gas', 'geazone': 'rezone', 'busimess': 'business', 'wdtn': 'with', 'onstorm': 'storm', 'brex': 'bred', 'nanoarch': 'nanotech', 'pkr': 'per', 'balefully': 'carefully', 'youngests': 'youngest', 'betches': 'bitches', 'dsingh': 'singh', 'zoomin': 'zooming', 'stormx': 'storm', 'fedup': 'fed', 'mt': 'it', 'poll]': 'polls', 'wrpt': 'wept', 'yougov': 'youto', 'wordersmith': 'wordsmith', '04am': 'am', 'puentes': 'fuentes', 'bogopa': 'bogota', 'eseries': 'series', 'theranos': 'thermos', 'citirus': 'citrus', 'suvive': 'survive', 'mbrs': 'mars', 'pcc': 'pic', 'allait': 'await', 'us83': 'us', 'aldiusa': 'aldous', 'rnli': 'only', 'oron': 'iron', 'akron': 'apron', 'chutki': 'chuck', 'dbt': 'debt', 'haibo': 'hair', 'synce': 'since', 'pleasyntly': 'pleasantly', 'hospitalists': 'hospitality', 'ayedo': 'aged', 'alcoho': 'alcohol', 'joya': 'joy', 'asper': 'asher', 'momz': 'mom', 'nnpa': 'napa', 'supermarkt': 'supermarket', 'deta': 'data', 'dled': 'died', 'westvan': 'western', 'semiras': 'seminar', 'noggy': 'doggy', 'isee': 'see', 'pdd': 'add', 'snapme': 'snape', 'routin': 'routine', 'mininum': 'minimum', 'adweek': 'aweek', 'fams': 'fans', 'tikiw': 'tiki', 'bongkok': 'bangkok', 'blocks]': 'blocks', 'penrith': 'perish', 'fvcked': 'fucked', 'thung': 'thing', 'somilar': 'similar', 'durust': 'durst', 'eps': 'es', 'noone': 'none', 'plexyglass': 'plexiglass', 'leadlap': 'headlamp', 'frid': 'rid', 'kcr': 'kir', 'diivys': 'drives', '72o': 'to', 'breakin': 'breaking', 'yhou': 'you', 'extensi': 'extend', 'ridelivers': 'delivers', 'heartens': 'hearkens', 'swaggy': 'shaggy', 'today]': 'today', 'skymiles': 'smiles', 'embrun': 'embryo', 'lovethem': 'loveth', 'nowserving': 'observing', 'rikshaws': 'rickshaws', 'hatchie': 'hatch', 'noor': 'door', 'maskson': 'masks', 'cust': 'just', 'montlake': 'montage', 'strano': 'strand', 'regu': 'reg', 'pasco': 'paso', 'dugna': 'dna', 'happyx': 'happy', '10c': 'doc', 'unappreciate': 'unappreciated', 'dockr': 'dock', '20mi': 'mi', 'wedeliver': 'deliver', 'unfollow': 'follow', 'shrule': 'should', 'diapering': 'differing', 'fwy': 'fly', 'alexia': 'alexis', 'coved': 'loved', 'syste': 'system', 'afb': 'aft', '8d': 'ad', 'sadday': 'saddam', 'fowrry': 'sorry', 'kramank': 'kraang', 'strangulate': 'strangulated', 'jodeci': 'jodie', '5cent': 'scent', 'twitchtv': 'twitch', 'owerri': 'terri', 'baray': 'barry', 'poondy': 'pounds', 'pleaseme': 'please', 'tomrrow': 'tomorrow', 'copays': 'cops', 'quarte': 'quarter', 'hpsn': 'hasn', 'saus': 'says', 'askul': 'ask', 'akala': 'kala', 'splc': 'spec', '6th': 'ith', 'nespresso': 'espresso', 'sdlax': 'relax', 'studio5': 'studio', 'muncipality': 'municipality', 'iffin': 'tiffin', 'lalockdown': 'lockdown', 'lovemy': 'lovely', 'hhs': 'his', 'beeutiful': 'beautiful', 'isuzu': 'suru', 'gaya': 'gay', 'newholly': 'wholly', 'acca': 'caca', 'tolive': 'olive', 'expandi': 'expand', 'reqd': 'read', 'pastured': 'pasture', '$psth': 'path', 'bafana': 'banana', 'o2': 'of', 'kryspi': 'krispy', 'airwave': 'airwaves', 'northwich': 'forthwith', 'daytimes': 'daytime', 'construc': 'construct', 'sv': 'so', 'hamites': 'hates', 'maceys': 'maces', 'alphago': 'alpha', 'emergently': 'emergency', 'savest': 'saves', 'kistservs': 'kissers', 'evangelista': 'evangelist', 'elsie': 'else', 'andur': 'and', 'shipopping': 'shopping', 'snl': 'sol', 'encash': 'encase', 'grocio': 'groin', 'chek': 'check', 'gerting': 'getting', 'grandaughter': 'granddaughter', 'powere': 'power', 'ett$': 'etty', 'peepz': 'peep', 'comestics': 'domestics', 'dlvrd': 'lord', 'napex': 'apex', 'reliab': 'relax', 'laxman': 'lawman', 'myua': 'myra', 'jackbox': 'jackpot', 'hedderman': 'elderman', 'planty': 'plenty', 'padua': 'papa', 'pica': 'pick', 'washpost': 'washout', 'carolinian': 'carolina', 'matzot': 'matzo', 'vasai': 'vast', 'offerte': 'offer', 'clendon': 'london', '5]': 'i', 'buut': 'but', 'sttr': 'star', 'iindoors': 'indoors', 'raithe': 'rather', '$wfc': 'itfc', 'baye': 'base', 'carleton': 'carlton', 'maliban': 'taliban', '$lh': 'all', 'coro': 'core', 'asko': 'ask', 'taproom': 'barroom', 'crappie': 'chappie', '$ottv': 'gotta', 'saehee': 'saree', 'steef': 'steel', 'captn': 'capt', 'distributi': 'distribute', 'afterall': 'adderall', 'cn': 'in', 'coron': 'corn', 'reminger': 'reminder', 'dts': 'its', 'birt': 'bit', 'cdle': 'idle', 'puhlease': 'please', 'fenty': 'went', 'msnbc': 'manic', 'lites': 'lives', 'publi': 'public', 'democraps': 'democrats', 'dupage': 'damage', 'katta': 'atta', 'deccan': 'declan', 'themsves': 'themselves', '19march': 'march', 'mntech': 'match', 'ammar': 'amar', 'rego': 'ego', 'lift]': 'lift', 'zhloe': 'chloe', 'fccla': 'fella', 'potoo': 'too', 'mazbox': 'mailbox', 'romanie': 'romance', 'greenlighted': 'greenlight', 'leftie': 'lefties', 'marwadi': 'mardi', 'sumter': 'summer', 'legere': 'degree', 'frc': 'fry', 'garbagecan': 'garbageman', 'restaurt': 'restart', 'mayberry': 'mulberry', 'gaano': 'guano', 'cpchat': 'chat', 'grubl': 'grub', 'sgandc': 'stand', 'homebase': 'homemade', 'crpf': 'cops', 'themsleves': 'themselves', 'cldrink': 'drink', 'careline': 'caroline', 'stagflation': 'stagnation', 'driverles': 'driverless', 'servicio': 'service', 'decolonization': 'colonization', 'wl': 'we', 'mnawami': 'mawari', 'citities': 'cities', 'laff': 'gaff', 'carbo': 'cargo', 'sobe': 'some', 'leaderes': 'leaders', 'hammonds': 'hammond', 'danbury': 'unbury', 'saugus': 'laughs', 'mildliners': 'millinery', 'temperstures': 'temperatures', '27hours': 'hours', 'bstitutions': 'institutions', 'nischay': 'mischa', 'coker': 'cover', 'anong': 'along', 'approvalno': 'approval', 'paym': 'pay', 'dool': 'door', 'ashby': 'ashy', 'omnichan': 'monican', 'rollin2x': 'rolling', 'melton': 'melon', 'simcoe': 'since', 'martyours': 'martyrs', 'brigette': 'brigitte', 'k18': 'kid', '$$': 'i', 'freshpet': 'freshen', 'konke': 'one', 'undoubtably': 'undoubtedly', 'rochford': 'rockford', 'yuge': 'huge', 'esq': 'es', 'winsight': 'insight', 'creem': 'cream', 'twittwr': 'twitter', 'garu': 'guru', 'colli': 'collie', 'web2': 'web', 'roofbox': 'rooftop', 'bernardino': 'bernardo', 'introverts': 'introvert', 'bartells': 'barbells', 'loag': 'long', 'genxers': 'genders', 'teami': 'team', 'sampars': 'samples', 'magaisa': 'malaise', 'stuffer': 'suffer', 'crunch]': 'crunch', 'schwan': 'scan', 'family5': 'family', 'ashrah': 'ashram', 'clamato': 'climate', '$eth': 'meth', 'dusability': 'disability', 'macon': 'bacon', 'ammosexual': 'homosexual', 'schitt': 'shit', 'epco': 'eco', 'mymetro': 'metro', 'mejier': 'meier', 'pawleys': 'valleys', 'slebs': 'slabs', 'fibber': 'fiber', 'wolt': 'wolf', 'aakhir': 'fakir', 'kaleisha': 'keisha', 'novi': 'nova', '14days': 'days', 'wheresrr': 'wherever', 'kubayi': 'dubai', 'payme': 'payne', 'opti': 'optic', 'cecilio': 'cecilia', 'coze': 'come', 'jshbr': 'usher', 'shuuuu': 'shutup', 'cille': 'ville', 'branson': 'brandon', 'botheted': 'bothered', 'dromore': 'promote', 'imnew': 'new', 'aijobs': 'jobs', 'jao': 'jam', 'workingmom': 'workingman', 'gofaone': 'ofone', 'millio': 'million', 'leclue': 'clue', 'galette': 'palette', 'llah': 'blah', 'nov19': 'novel', 'twerkin': 'twerking', 'mg': 'my', 'chicom': 'chico', 'rideshare': 'timeshare', 'zcash': 'cash', 'noirmal': 'normal', 'sumn': 'sun', 'mmorpg': 'morph', 'rohtak': 'rota', 'ivs': 'is', 'condolife': 'condole', 'baju': 'raju', 'opm': 'op', 'carnrite': 'carrie', 'besan': 'began', 'pyclet': 'pocket', 'modelo': 'model', 'mymom': 'mom', 'colleciton': 'collection', 'abuelo': 'abuela', 'zae': 'zap', 'excuss': 'excuse', 'pgp': 'pop', 'fn': 'in', '$tcehy': 'itchy', 'rossen': 'rosen', 'quaffed': 'cuffed', 'granpa': 'grandpa', '1wallet': 'wallet', 'ashburn': 'auburn', 'aquiring': 'acquiring', 'illetrate': 'illiterate', 'nda': 'dna', 'liek': 'like', 'mrg': 'mug', 'lhg': 'leg', 'kenney': 'kenny', 'ncgov': 'ngo', 'n2': 'no', 'marqt': 'mart', 'eocmmerce': 'commerce', 'suuuh': 'such', 'volunteernj': 'volunteer', 'jomari': 'omar', 'ashrams': 'ashram', 'wsi': 'si', 'cookshop': 'bookshop', 'resory': 'resort', 'sylhet': 'sleet', 'mwt': 'met', 'minsk': 'mink', 'gitanes': 'yitanes', 'seany': 'sean', 'ponzis': 'ponzi', 'wnna': 'wanna', 'cyclers': 'cycles', 'fogger': 'forger', 'totd': 'told', 'sese': 'see', 'agonise': 'agonies', 'publife': 'public', 'platinumlv': 'platinum', 'unihop': 'union', 'markeetex': 'marketed', 'injudicious': 'judicious', 'buyjoy': 'buoy', 'cbass': 'class', 'fluxx': 'flux', 'indveng': 'invent', 'meshelly': 'shelly', 'relex': 'relax', 'caray': 'carry', 'atlantans': 'atlanteans', '8nn': 'inn', 'fahid': 'said', 'tippin': 'tipping', 'fhk': 'fuk', 'babyy': 'baby', 'nashvilletn': 'nashville', 'ungendered': 'engendered', 'anyw': 'any', 'friendos': 'friends', 'flirtey': 'flirted', 'lunga': 'lungs', 'shoppingo': 'shopping', 'kratky': 'cranky', 'alyson': 'alison', 'micheals': 'michaels', 'yeayes': 'years', 'kibsons': 'ribbons', 'kren': 'keen', 'm7y': 'my', 'adithya': 'aditya', 'hulu': 'hull', 'heinz': 'hein', 'swamy': 'swamp', 'turant': 'tyrant', 'leftness': 'softness', 'collab': 'collar', 'burstyn': 'burst', 'twimboz': 'wimbo', 'namasteui': 'namaste', 'urgg': 'urge', 'r4today': 'today', 'envi': 'envy', 'urbana': 'urban', 'b6': 'be', 'kimik': 'kimi', 'mca': 'ma', '3rdhour': 'ardour', 'vca': 'via', 'npcs': 'nice', 'vimeo': 'video', 'whateverr': 'whatever', 'buymie': 'budgie', 'hyehouse': 'hothouse', 'weebs': 'weeks', 'grnadparents': 'grandparents', 'bleachy': 'bleach', 'csx': 'csi', 'bikkies': 'bikes', 'severeme': 'severe', 'fuct': 'fuck', 'novembersns': "november's", 'eminem': 'einem', 'accessi': 'access', 'papz': 'papa', 'shash': 'smash', 'glazier': 'glacier', 'albinder': 'binder', 'unres': 'unrest', 'carbonise': 'carbonised', 'couric': 'course', 'rexx': 'rex', 'thelink': 'think', 'tanis': 'tanks', 'tposting': 'posting', 'pfriem': 'prime', 'wfz': 'wiz', 'glorius': 'glorious', 'stockpilers': 'stockpiles', 'visists': 'visits', 'belcamp': 'became', 'xddd': 'did', 'twiggy': 'wiggy', 'bycycle': 'bicycle', 'hunkers': 'hunters', 'flavourful': 'flavorful', 'farrotto': 'garrotte', '1$': 'i', 'expr': 'expo', 'jpow': 'pow', 'kjk': 'kak', '5c': 'ac', 'ymcas': 'ymca', 'brexited': 'exited', 'brandless': 'boundless', 'lakowe': 'lake', 'freeads': 'friends', 'amazonman': 'amazonian', 'nehme': 'name', 'divorc': 'divorce', 'denominate': 'dominate', 'cicis': 'civil', 'tipless': 'topless', 'wholistic': 'holistic', 'wr3': 'wry', 'plf': 'elf', '45p': 'up', 'blease': 'please', 'marivic': 'marvin', 'mre': 'me', 'blinn': 'blind', 'first]': 'first', 'amedore': 'adore', 'finchley': 'finches', '3hr': 'ihr', 'outstation': 'substation', 'affored': 'afford', 'dekat': 'dead', 'lani': 'land', '70p': 'up', 'independant': 'independent', 'extr': 'extra', 'avleat': 'alert', 'xyla': 'lyla', 'shokran': 'shorn', 'nosers': 'losers', 'retailler': 'retailer', 'approlabs': 'approvals', 'strath': 'strata', 'alabbar': 'blabber', 'lccaregiver': 'caregiver', 'makanan': 'kanan', 'drving': 'driving', 'luffly': 'fluffy', 'mse': 'me', 'kdwn': 'down', 'requstng': 'requesting', 'powel': 'power', 'soros': 'sorts', '2the': 'the', 'garlic]': 'garlic', 'dairies': 'fairies', 'zipongo': 'zilong', 'ntd': 'nod', 'boulter': 'boulder', 'unexpecte': 'unexpected', 'vacas': 'vacay', '93the': 'the', 'bgbm': 'baby', 'ebonie': 'bone', 'acadiana': 'arcadian', 'bulgar': 'vulgar', 'mid60s': 'midas', 'bougee': 'bougie', 'zanu': 'zany', 'neilson': 'nelson', 'chroni': 'chronic', 'trav': 'trap', 'ndrf': 'nerf', 'eahwp': 'gawp', 'envo': 'envy', 'kullen': 'sullen', 'grabsg': 'grabs', 'plsd': 'pled', 'omgs': 'mogs', 'alchohol': 'alcohol', 'deltabc': 'delta', 'logon': 'logo', 'beach1': 'beach', 'compe': 'come', 'rechedule': 'reschedule', 'splott': 'spot', '30the': 'the', 'jn': 'in', 'h8': 'he', 'saumik': 'saudi', 'pageviews': 'previews', 'nshit': 'shit', 'aja': 'aka', 'parkg': 'park', 'tec': 'ten', 'jesup': 'jesus', 'respawning': 'spawning', 'bandula': 'bandana', '$sbw': 'saw', 'srvce': 'since', 'lurve': 'lure', 'hecka': 'heck', 'pnl': 'pal', 'senco': 'since', 'godawgs': 'dawgs', 'wardle': 'waddle', 'contractable': 'intractable', 'woollys': 'woolly', 'bmhs': 'boys', 'snackson': 'snacks', '20mins': 'mins', 'dpruto': 'pluto', 'caroberry': 'cranberry', 'xrp': 'are', 'nopancakes': 'pancakes', 'protecti': 'protect', 'mcitot': 'mitt', 'federalway': 'federally', 'martyn': 'martyr', 'femco': 'fetch', 'vik': 'via', 'housemade': 'housemate', 'aadhya': 'radha', 'n10k': 'neck', 'tiired': 'tired', 'pedy': 'peds', 'supe': 'sure', 'freshway': 'freshman', 'deluna': 'deluca', '8pack': 'pack', 'bddu': 'bad', 'mirakl': 'moral', 'yeww': 'yew', 'ewer': 'ever', 'sannys': 'says', 'voluntelling': 'volunteering', 'sheild': 'shield', '19june': 'june', 'wsb': 'web', 'rockland': 'roland', 'kobm': 'kom', 'newsone': 'nelson', 'kimmage': 'image', 'rosh': 'rush', '10er': 'her', 'llf': 'elf', 'conatiners': 'containers', 'doxing': 'doing', 'allston': 'allison', 'mpi': 'mi', 'romare': 'rare', '3weks': 'weeks', 'mcale': 'male', 'maccas': 'marcus', 'wegman': 'woman', 'maketse': 'makes', 'bekah': 'beka', '30et': 'get', 'nutritionis': 'nutritionist', 'keiyser': 'keyser', 'araby': 'arab', 'julietree': 'juliette', 'technologists': 'technologies', 'signally': 'finally', '08am': 'am', 'specialneeds': 'specialness', 'barra': 'barry', 'ocuured': 'occurred', '$amazon': 'amazon', 'temporally': 'temporary', 'amala': 'mala', 'toolate': 'isolate', 'kaley': 'kale', 'fascinateno': 'fascinating', 'hubspot': 'hotspot', 'congressi': 'congress', '$seel': 'see', 'chara': 'charm', 'nulled': 'pulled', 'loker': 'lower', 'vmp': 'amp', 'pooor': 'poor', 'meesh': 'mesh', 'stubbs': 'stubs', 'sfi': 'ski', 'tavros': 'stavros', 'wuse2': 'used', 'donahoe': 'donahue', 'goya': 'gota', '8bn': 'ibn', 'torwards': 'towards', 'increas': 'increase', 'haizt': 'wait', 'cagayan': 'caravan', 'rendre': 'render', 'quadra': 'quarry', 'chcocolate': 'chocolate', 'universee': 'universe', 'krcr': 'kick', 'tomyawk': 'tomahawk', 'hamza': 'mama', 'wario': 'mario', 'freshnuts': 'freshness', 'chatterjee': 'chattered', 'dq': 'do', 'wilco': 'wilcox', 'camas': 'camps', 'hurrrry': 'hurry', 'experince': 'experience', 'donalds': 'donald', '15lb': 'bulb', 'definately': 'definitely', 'sartly': 'partly', 'gll': 'all', 'rebalances': 'rebalance', 'katrin': 'katrina', 'algernon': 'algerian', 'wgxc': 'wax', 'sungai': 'sugar', 'playslack': 'playback', 'rater': 'rather', 'epicure': 'picture', '6in': 'in', 'bavaria': 'bavarian', 'fmb': 'fob', 'teigan': 'began', 'gerdeman': 'german', 'flyering': 'flying', 'interne': 'internet', 'hesistant': 'resistant', 'tldr': 'told', 'gv': 'go', 'ctn': 'can', 'shte': 'she', 'dfwa': 'down', 'minim': 'mini', 'gogators': 'gators', 'dunzo': 'dunno', 'weinstein': 'einstein', 'pasoans': 'asians', 'bawal': 'banal', 'ambar': 'amber', 'withoutout': 'withoutyou', 'zhivago': 'chicago', 'pacman': 'madman', 'frictionless': 'factionless', 'eureeca': 'eureka', 'equivalence': 'equivalency', 'zamato': 'yamato', 'gaddamn': 'goddamn', 'binliners': 'airliners', 'ule': 'use', 'danville': 'danielle', 'cev': 'ce', 'woodbine': 'woodwind', 'clonescript': 'conscript', '7m': 'am', 'deleviry': 'delivery', 'apni': 'ani', 'leu': 'let', 'muja': 'puja', 'rimom': 'room', 'trakop': 'trap', 'onbrand': 'brand', 'aline': 'alone', 'clarita': 'clarity', 'semaine': 'remain', 'emoplyees': 'employees', '$hitheaded': 'hotheaded', 'hayne': 'wayne', 'donovans': 'donovan', 'psws': 'paws', 'zabar': 'bar', 'sandf': 'sand', 'beardie': 'beard', 'serapio': 'seraphim', 'sobys': 'sobs', 'cupon': 'upon', 'ubereats': 'berets', 'informati': 'information', 'alist': 'list', 'wxyz': 'why', '9roll': 'roll', 'morn8ng': 'morning', 'layc': 'lay', 'ngos': 'nos', 'bl': 'be', 'pulix': 'pull', 'elab': 'lab', 'r30': 'run', 'wakeupwi': 'wakeup', 'melawati': 'megawatt', 'ecom': 'com', 'marlena': 'marlene', 'krimpets': 'trumpets', 'padhi': 'paid', 'shallst': 'shallot', 'demonrats': 'democrats', 'dagara': 'dakara', 'nvcc': 'nice', 'rgv': 'rev', 'mesha': 'mesh', 'duggar': 'sugar', 'biltmore': 'baltimore', 'quavo': 'quake', 'flightsd': 'flights', 'dwolla': 'dolla', 'houseshoes': 'horseshoes', 'evins': 'veins', 'zona': 'zone', 'bonch': 'bunch', 'cumshot': 'gunshot', 'hewitts': 'hewitt', 'thinkingif': 'thinking', 'shefani': 'stefan', '12hours': 'hours', 'rebph': 'reach', '$sne': 'she', 'nicc': 'nice', 'defunds': 'defends', 'kta': 'ta', 'minnesotans': 'minnesota', 'tlod': 'told', 'upcycled': 'recycled', 'reallife': 'realize', 'bbe': 'be', 'idled': 'idle', 'ija': 'ja', 'harleston': 'charleston', 'wakingup': 'waking', 'brigman': 'brian', 'provincie': 'province', 'avprogress': 'progress', 'boutgeoise': 'bourgeoise', 'sikhs': 'sighs', 'somik': 'some', 'courrier': 'courier', 'sarnis': 'saris', 'eyeb': 'eyes', 'emaiied': 'emailed', 'edric': 'eric', 'ummed': 'bummed', 'wepickup': 'pickup', 'wek': 'we', 'cfan': 'can', 'relaunched': 'launched', 'antifa': 'anti', 'waltermart': 'watermark', 'leez': 'lee', 'primelike': 'primetime', 'apium': 'opium', 'breaking]': 'breaking', 'merida': 'media', 'anza': 'anna', 'coronatips': 'coronation', 'newmom': 'newton', 'thestore': 'restore', '25cals': 'locals', '5mn': 'amn', 'skidmore': 'widmore', 'ebikes': 'bikes', 'carrera': 'camera', 'typer': 'type', 'salery': 'salary', 'kholo': 'cholo', 'mylanta': 'manta', 'lovetech': 'loveth', 'whitecap': 'whitetip', 'labmates': 'playmates', 'hzy': 'hey', 'mekung': 'making', 'mazon': 'mason', 'uhgv': 'huge', 'jaari': 'jar', 'fm]': 'for', 'galti': 'gate', 'cherk': 'check', 'vunerable': 'vulnerable', 'nowaways': 'nowadays', 'nenad': 'need', 'uske': 'use', '5years': 'years', 'ddr': 'der', 'lchf': 'chef', 'hrm': 'him', 'pagaling': 'paging', 'sanitatized': 'sanitized', 'kasich': 'basic', '8kph': 'syph', 'pp': 'up', 'wooloo': 'voodoo', 'ippozh': 'ippo', '1week': 'week', 'laggy': 'baggy', 'domingo': 'domino', 'haft': 'hat', 'jerami': 'ceramic', 'tomarrow': 'tomorrow', 'allcuts': 'alljust', 'punny': 'funny', 'mullein': 'bulletin', 'toucher': 'touched', 'parkin': 'parking', 'sunfresh': 'suresh', 'ados': 'adds', 'comoany': 'company', 'manchego': 'ranchero', 'loveg': 'love', 'abeba': 'abba', 'anyhoo': 'anyhow', 'exclusi': 'excuse', 'moresales': 'morales', 'zoamto': 'tomato', 'dube': 'due', 'athiest': 'atheist', 'aleutia': 'aleutian', 'abonormal': 'abnormal', 'toptier': 'copier', 'nowak': 'novak', 'htm': 'him', 'bookwork': 'bookworm', 'peerform': 'perform', 'luckfor': 'luckier', 'mbarara': 'barbara', 'boxpoll': 'boxtroll', 'blinkits': 'blankets', '5x': 'ex', 'annemc': 'anne', 'granpaw': 'grandpa', 'capmetro': 'cameto', 'fodor': 'odor', 'deliveryoat': 'deliveryman', 'huesca': 'hues', 'cobain': 'cousin', 'ggered': 'agreed', 'odell': 'dell', 'aprtmnts': 'apartments', 'eeoc': 'ego', 'grippy': 'trippy', 'menon': 'melon', 'olfaction': 'faction', 'sillinessi': 'silliness', 'k95': 'kid', 'utube': 'tube', 'disenfect': 'disinfect', 'cabarrus': 'cabarets', 'ps5s': 'ass', 'freestar': 'freest', 'opulently': 'opulent', 'shenani': 'shebang', 'wowsa': 'wows', 'mangalam': 'mandala', 'nopeople': 'people', 'qhp': 'the', 'awonderfull': 'wonderful', '8rg': 'org', 'wife]': 'wife', 'genexa': 'geneva', 'onken': 'oaken', 'wabbits': 'rabbits', 'repu': 'rep', 'bogost': 'boost', 'wogan': 'woman', 'egregiou': 'egregious', 'smclk': 'small', 'ohel': 'hel', 'bookoo': 'book', 'lane14': 'lane', 'teleducation': 'reeducation', 'raytown': 'rayon', 'wpm': 'wom', 'sakhi': 'saki', 'dollarsb7': 'dollars', 'absocond': 'abscond', 'shapps': 'shapes', 'lrod': 'lord', 'evyb': 'evy', 'lepastu': 'least', 'blog]': 'blog', 'discolor': 'discover', 'momlove': 'oflove', 'edy': 'ed', 'akbanana': 'banana', 'milkis': "milk's", 'sfchronicle': 'chronicle', 'benoey': 'bender', '30yard': 'yard', 'mansfield': 'minefield', 'shaz': 'shag', 'iswear': 'swear', 'dtns': 'dans', 'needey': 'needed', 'nelso': 'nelson', 'dedue': 'deduce', 'shazaam': 'salaam', 'mientras': 'mantras', '25lbs': 'bulbs', 'foodrunner': 'forerunner', 'adhe': 'ache', 'postie': 'pootie', 'fukgin': 'fuckin', 'forsyth': 'forsyte', 'hgtv': 'hit', 'resou': 'rest', 'tenecent': 'tenement', 'sperre': 'serve', 'ghantay': 'ghastly', 'mimac': 'mimic', 'ooey': 'obey', 'khula': 'hula', 'watermill': 'waterfall', 'etip': 'tip', '6p': 'up', 'picksup': 'pickup', 'batavia': 'octavia', 'somrthing': 'something', 'dt9': 'to', 'trashless': 'trackless', 'cooee': 'cooze', 'iight': 'right', 'kttv': 'kit', 'acquirin': 'acquiring', 'cappucino': 'cappuccino', 'roston': 'boston', 'yuxlo': 'yule', 'geezy': 'geeky', 'mcchicken': 'chicken', 'addtional': 'additional', 'jif': 'if', 'psg': 'pig', 'kart': 'part', 'c0nvey': 'convey', 'localnl': 'local', 'wly': 'why', 'marxh': 'march', 'pmpeople': 'people', 'umich': 'mich', 'intw': 'into', 'usp': 'up', 'pci': 'pi', 'devoos': 'demons', 'savingus': 'savings', 'tatmon': 'station', 'outstate': 'upstate', 'srsly': 'truly', 'repackaging': 'packaging', 'jan6': 'jane', 'jd': 'ad', 'pubix': 'pubic', 'pofitech': 'profiteth', 'nonblack': 'ofblack', 'vaunt': 'aunt', 'vnc': 'vic', '8b': 'ab', 'harlene': 'charlene', 'sahin': 'satin', '32nd': 'and', 'canadaus': "canada's", 'marcado': 'arcade', 'kag': 'bag', 'availa': 'avail', 'obike': 'bike', 'unironically': 'ironically', 'arti': 'art', '$atms': 'atms', 'instire': 'inspire', 'gorcery': 'grocery', 'yuss': 'fuss', 'miatake': 'mistake', 'jitse': 'its', 'weekender': 'weekenders', 'dsus2': 'dust', 'jy': 'my', 'tocure2': 'toured', 'sindy': 'windy', 'topsfall': 'topsail', 'gump': 'jump', 'onpoli': 'poli', 'weiser': 'wiser', 'sabzi': 'nazi', 'ablity': 'ability', 'coulnt': 'count', 'as1': 'as', 'hntb': 'into', 'sert': 'set', 'clp': 'cop', 'manomet': 'manet', 'grosheries': 'groceries', 'workarounds': 'workaround', 'chtistmas': 'christmas', 'ereats': 'treats', 'nlaws': 'laws', 'pneu': 'peu', 'hdmc': 'him', 'buurt': 'blurt', 'pdts': 'puts', 'ayamas': 'pajamas', 'sodak': 'soda', 'bokaro': 'board', 'inperson': 'person', 'ghettoest': 'ghettos', 'everyo': 'every', 'wombo': 'womb', 'chitral': 'central', 'calmac': 'calm', 'harbir': 'harbor', '2day': 'day', 'catapillar': 'caterpillar', '14t': 'it', 'stepforder': 'stepford', 'exisistent': 'existent', 'csed': 'used', 'nhi': 'hi', 'dirham': 'dream', 'pain18': 'pain', 'battrened': 'battened', '15b': 'job', 'dehumanise': 'dehumanize', 'fckn': 'can', 'mendy': 'mend', 'laborare': 'elaborate', '91st': 'just', 'copay': 'copy', 'folo': 'fool', 'beseen': 'bemeen', 'oportunity': 'opportunity', 'genuinly': 'genuinely', 'gleamin': 'gleaming', 'seriouly': 'seriously', 'compliancy': 'compliance', 'sanpedro': 'sandro', 'nahh': 'nah', 'commun': 'common', 'breadfast': 'breakfast', 'ks3': 'is', 'vincents': 'vincent', '$aci': 'back', 'padsy': 'pads', 'rekha': 'rehab', 'geeknews': 'meekness', 'khalil': 'khalid', 'scalzi': 'scale', 'eilyn': 'eily', 'stdenys': 'students', 'worderle': 'orderly', 'discontinuity': 'discontinuing', 'eamart': 'smart', 'purefood': 'pureblood', 'kyalami': 'salami', 'worry]': 'worry', 'querido': 'queries', 'benghazi': 'bengali', 'combi': 'comb', 'dogsallowed': 'disallowed', 'mened': 'mend', 'chenin': 'chemin', 'amg': 'am', 'mart21': 'martyr', 'raudhah': 'radha', 'tpm': 'tom', 'phop': 'shop', 'bipass': 'bypass', 'neswagents': "newsagent's", 'yuup': 'yup', 'fics': 'fits', '2ce': 'ice', 'shazzys': 'snazzy', 'bedhead': 'redhead', 'custome': 'customer', 'chandni': 'chandra', 'digitalist': 'digitalis', 'schduld': 'should', 'tvc': 'tac', 'cpil': 'coil', 'bernald': 'bernard', 'sala7ef': 'salade', 'fastsigns': 'assigns', 'ugx': 'ugh', 'acctg': 'act', 'oelke': 'else', 'arizonan': 'arizona', 'bumholes': 'buttholes', 'rapin': 'rain', 'lifesaverz': 'lifesaver', 'needapa': 'needa', 'nostrand': 'strand', 'chep': 'cheap', 'rmn': 'run', 'techbiz': 'techie', 'mnmushrooms': 'mushrooms', 'mvb': 'mob', 'milktea': 'militia', 'nri': 'ni', 'riteway': 'gateway', 'metzgers': 'meters', 'cppd': 'cold', 'arvo': 'arlo', 'stingday': 'stingray', 'nwo': 'no', 'lylli': 'lolli', 'miff': 'riff', 'mobro': 'more', 'amerika': 'america', '2x6': 'ex', 'equi': 'qui', 'wishmaster': 'bushmaster', 'estrada': 'strata', 'palghar': 'palmar', 'croatia': 'croatian', 'bakewells': 'farewells', 'meaderies': 'eateries', 'halva': 'halve', '22n': 'in', 'menorca': 'senora', 'onamission': 'omission', '$fdx': 'fix', 'fullfilment': 'fulfilment', 'vn': 'in', 'vzet': 'vet', 'paltd': 'paled', 'fromtv': 'from', 'capitola': 'capitol', '55am': 'am', 'pffbt': 'offat', 'weeder': 'feeder', 'pergo': 'ergo', '1way': 'way', 'tootired': 'tortured', 'joy70': 'joy', 'foreverr': 'forever', 'foodro': 'food', 'exhorbitant': 'exorbitant', '$prmo': 'pro', 'acce': 'ace', 'ezola': 'ebola', 'qotsa': 'lotsa', 'crewmate': 'cremate', '7ish': 'wish', 'kctl': 'kill', 'minorly': 'minor', 'greers': 'greens', '10saved': 'saved', 'expectmy': 'expect', 'amjad': 'ahead', 'eny': 'any', 'ar15': 'are', 'ahajs': 'has', 'anycart': 'handcart', 'wallahs': 'wallah', 'zipit': 'visit', 'intears': 'tears', 'uspo': 'uso', 'erinarian': 'seminarian', 'pasteurize': 'pasteurized', 'unloa': 'unload', 'haev': 'have', 'r50': 'run', 'undated': 'updated', 'arkit': 'admit', 'vwdealer': 'dealer', 'agreei': 'agree', 'weetos': 'weeks', 'ravenning': 'ravening', 'gadually': 'gradually', 'hsould': 'should', 'avenida': 'agenda', 'basketing': 'marketing', 'blinken': 'blinked', 'sautter': 'shutter', 'gburg': 'burg', 'preplan': 'replay', 'tofi': 'tofu', 'wallas': 'walls', 'thwsw': 'this', 'greeley': 'greely', 'percocet': 'percocets', '50mi': 'mi', 'apr10': 'apron', 'liscence': 'licence', 'ict4d': "it'd", 'yivaan': 'vivian', 'pendeja': 'pendejo', 'kapp': 'app', 'dumbheads': 'dumbhead', 'panedemic': 'pandemic', 'workingwoman': 'workingman', 'swiggys': 'swings', 'pollinator': 'pollinators', 'ttps': 'tips', 'harpur': 'harper', 'nl': 'no', 'childe': 'child', 'imali': 'small', 'winebar': 'vinegar', 'tinn': 'tiny', 'sangrur': 'sangria', 'itisha': 'irish', 'ceburi': 'rebury', 'pymnts': 'pants', 'takeo': 'take', '$fhl': 'ifl', 'labwork': 'labor', 'manytime': 'anytime', 'nancd7u': 'nandu', 'pb': 'pub', 'paham': 'param', 'wordered': 'ordered', 'winable': 'winnable', 'jawa': 'jaw', 'fastpass': 'fatass', 'palea': 'pale', 'steppin': 'stepping', 'nightmareish': 'nightmarish', 'fct': 'fact', 'neandethal': 'neanderthal', 'phobal': 'global', 'natgas': 'niggas', 'tarik': 'tariq', 'addis': 'adds', 'cf64': 'cfo', 'leake': 'leave', 'souq': 'soul', 'sfboy': 'boy', 'asure': 'sure', '90yo': 'yo', 'tieups': 'ties', 'upsers': 'upsets', 'notts': 'notes', 'nazish': 'nazis', 'addtech': 'arktech', 'bezylon': 'babylon', 'accurse': 'accuse', 'fenix': 'felix', 'chodes': 'codes', 'is]': 'is', 'olen': 'open', 'miele': 'mile', 'pso2': 'so', 'fagner': 'wagner', 'numbr': 'number', 'dimlow': 'pillow', 'greco': 'reco', 'shalu': 'shall', 'nbc': 'nba', 'floja': 'flora', 'geekon': 'reckon', 'illai': 'illa', 'gettn': 'gettin', 'flummox': 'lummox', 'mphaso': 'phase', 'clise': 'close', 'ilan': 'plan', 'sajaji': 'safari', 'overbought': 'overwrought', 'achs': 'acts', 'sonicc': 'sonic', 'gobows': 'grows', 'hamesh': 'hamish', 'minecraft': 'mineshaft', 'moka': 'mona', 'modimo': 'modify', 'reasonablely': 'reasonable', 'sandhills': 'handbills', 'day15': 'day', 'skerries': 'series', 'ownwell': 'unwell', 'unnu': 'unni', 'slfd': 'slid', 'figutlre': 'figure', 'lannie': 'annie', 'unles': 'unless', 'yesphx': 'yesh', '2hour': 'hour', 'sandwi': 'sand', 'suisun': 'susan', 'rata': 'data', 'pelik': 'penis', 'retinol': 'retinal', 'nibler': 'nobler', 'ngayon': 'crayon', 'gnc': 'inc', 'restrauants': 'restaurants', 'sportsmom': 'sportsman', 'dellmed': 'yelled', 'logi': 'log', 'coolmix': 'coolie', 'somnia': 'sonia', 'teess': 'teens', 'sfv': 'suv', 'hng': 'hang', 'totalwine': 'totaling', 'dillution': 'dilution', 'wilhelmy': 'wilhelm', 'perscription': 'prescription', 'bolt24': 'bolt', 'sfz': 'sez', 'clapton': 'clayton', 'mathew': 'matthew', 'nemt': 'next', 'haribo': 'harbor', 'walmex': 'walked', 'deontae': 'dental', 'mchs': 'mechs', 'fcuk': 'fuck', 'ibang': 'bang', 'neston': 'newton', 'jerrys': 'jerry', 'dasz': 'dash', 'labourious': 'laborious', 'bramley': 'bradley', 'hyperzod': 'hyperion', 'boichik': 'yoichi', 'sle': 'she', 'southwark': 'southward', 'contxto': 'context', 'the7on8': 'theon', 'handleless': 'handless', 'kitwe': 'kite', 'uda': 'da', 'engloand': 'england', 'chopra': 'copra', 'smb': 'sub', 'dooh': 'door', 'paperwrk': 'paperwork', 'bushland': 'bushman', 'awwkward': 'awkward', 'asadhere': 'adhere', 'causee': 'cause', 'lazada': 'canada', 'croveng': 'proven', 'shiney': 'shine', 'yeehaw': 'seesaw', 'intacart': 'intact', 'shanns': 'shanks', 'help4u': 'help', 'emptive': 'emotive', '30m': 'him', 'satu': 'sat', 'sunbasket': 'sunbaked', 'rethinkri': 'rethink', 'pedx': 'peds', 'unpleasyntly': 'unpleasantly', 'pov': 'pop', 'bpp': 'app', 'durex': 'sure', '10kg': 'ekg', 'dc4c': 'doc', 'cerfew': 'curfew', 'downfalls': 'downfall', 'trimmier': 'trimmer', 'needddd': 'needed', 'inslee': 'inside', 'yassir': 'yasir', 'carona': 'corona', 'androi': 'android', 'nkrak': 'frak', 'camtime': 'captive', 'estelita': 'estella', '44mins': 'mins', 'firequacker': 'firecracker', 'sorso': 'torso', 'rieder': 'rider', 'prking': 'parking', 'gargabe': 'garbage', 'drivinh': 'driving', 'fogo': 'fog', 'ds': 'is', '6b': 'ab', 'preet': 'greet', 'gundersen': 'undersea', 'syesmi': 'semi', 'ecanter': 'decanter', 'jenrick': 'enrich', 'sprin': 'spring', 'gue': 'guy', 'freebirds': 'freebies', 'bronchus': 'branches', '26g': 'big', 'icant': 'cant', 'mallya': 'mall', 'sunlife': 'unlike', 'ctc': 'etc', 'buye': 'buy', 'hangburg': 'hamburg', 'minimalistic': 'animalistic', 'fuccing': 'fucking', 'kiaro': 'piano', 'orderred': 'ordered', '17m': 'him', 'crispr': 'crisp', 'onduty': 'duty', 'gahh': 'gash', 'nimby': 'nimbly', '90cals': 'locals', 'nma': 'ma', 'unplesant': 'unpleasant', 'phakama': 'pajama', 'mh': 'my', 'derping': 'during', 'gramme': 'grammy', 'privelage': 'privilege', '$alb': 'all', '35k': 'ask', 'ihih': 'shih', 'awat': 'away', 'remax': 'relax', 'messina': 'messing', 'onlne': 'online', 'fixthis': 'forthis', 'secks': 'sucks', 'rinu': 'ring', 'redhill': 'refill', 'carfails': 'carpals', 'luckies': 'luckiest', 'qrts': 'arts', 'paulina': 'pauline', 'mexicano': 'mexican', 'yeooooo': 'nooooo', 'borderering': 'bordering', 'topps': 'tops', 'mand': 'and', 'reattempt': 'attempt', 'hak': 'had', 'cult45': 'cult', 'fiore': 'fire', 'marne': 'marine', 'wherls': 'wheels', 'hotdeals': 'hotels', '49am': 'am', 'loveloud': 'lovelorn', 'chela': 'cela', 'awks': 'asks', 'oversharing': 'overhearing', 'idolssa': 'idols', '02s': 'is', 'dmed': 'died', 'eh14': 'eh', 'deliveryi': 'delivery', 'ritas': 'rites', 'loma': 'coma', 'bugaloo': 'bungalow', 'lovefor': 'lover', 'ak15': 'aka', 'teardown': 'beatdown', 'ajah': 'ajar', 'hote': 'home', 'khane': 'shane', 'woc': 'won', 'moishie': 'moshi', 'inkat': 'ink', 'naturelov': 'naturel', 'imreviews': 'reviews', 'ssables': 'stables', 'occuf': 'occur', 'riso': 'risk', 'aox': 'box', 'hygge': 'huge', 'yiguo': 'yago', 'zallat': 'wallet', '$adbe': 'maybe', 'aameen': 'amen', 'cx3': 'can', 'kunywa': 'kurwa', 'rvpark': 'park', 'aahh': 'ahah', 'menugem': 'menage', 'sansolo': 'manolo', 'adks': 'asks', 'ruckzug': 'ruckus', 'wlamart': 'lamar', 'murkomen': 'marksmen', 'dungi': 'dung', 'antiquate': 'antiquated', 'hypermart': 'hypermarket', 'platfo': 'plato', 'frugl': 'frugal', 'providin': 'providing', 'wtng': 'wing', 'b0rked': 'barked', 'shamokin': 'shaolin', 'shj': 'she', 'clairton': 'carton', 'keene': 'keen', 'babyn': 'baby', 'mola': 'mole', ']]': 'i', 'retweets': 'retweet', 'helth': 'health', 'quora': 'quota', 'unbagging': 'nagging', 'deceny': 'decent', 'villig': 'killing', 'shocke': 'shock', 'rm9': 'my', 'roeland': 'roland', 'clifforder': 'clifford', 'newpaper': 'newspaper', 'freshai': 'fresh', 'copenh': 'open', 'shouta': 'shout', 'manang': 'manana', 'chked': 'choked', 'emergancy': 'emergency', 'belagum': 'belgium', 'doorst': 'doors', 'chri$tma$': 'christmas', 'vhaa': 'haa', 'unhappyat': 'unhappy', 'durables': 'durable', 'fyip': 'flip', 'shikena': 'shaken', 'ddays': 'days', 'inequitable': 'inevitable', 'fredos': "fred's", 'mst': 'must', 'rheum': 'them', 'worldha': 'world', 'nigam': 'nizam', 'sbachat': 'sachet', 'transload': 'trainload', '6mos': 'mos', 'phanda': 'panda', 'ballinagh': 'balling', 'yad': 'had', 'chevrolets': 'chevrolet', 'eh32': 'eh', 'securus': 'secures', 'ummies': 'dummies', 'pvr': 'per', 'cylife': 'mylife', '$apha': 'papa', 'inherant': 'inherent', 'ctv': 'ctu', 'biteme': 'bite', 'ca10': 'can', 'homenow': 'somehow', 'neig': 'neil', 'allocator': 'alligator', 'eyesears': 'eyeses', 'demille': 'demise', 'pandore': 'pandora', 'inlaws': 'laws', 'ttm': 'tom', 'sunhat': 'sunset', 'considerately': 'considerate', 'severing]': 'severing', 'sched': 'shed', 'mdha': 'idea', 'soulard': 'solar', 'bubi': 'buti', 'mati': 'math', 'timmies': 'tummies', 'happyxx': 'happy', 'lartey': 'late', 'zoomart': 'goomar', 'dalal': 'halal', 'towson': 'town', '74k': 'ask', 'roes]': 'roes', 'launchday': 'launchpad', 'merrill': 'merrily', '$se': 'use', 'mcfuckin': 'fuckin', 'piccup': 'pickup', 'gips': 'lips', 'org20': 'organ', 'hca': 'ha', 'oncall': 'call', '4wd': 'and', 'cosimo': 'cosmo', 'belieber': 'believer', 'freshco': 'fresco', 'dinn': 'dine', '2lb': 'all', 'arabella': 'isabella', 'repeate': 'repeat', 'pattison': 'partisan', 'ouside': 'outside', 'wfh': 'wah', 'accenture': 'adventure', 'waterkloof': 'waterproof', 'selver': 'silver', '$wmn': 'own', 'gomariners': 'mariners', 'haah': 'hath', 'pt2': 'pta', 'industry40': 'industry', 'marzano': 'marian', 'syourups': 'syrups', 'theek': 'thee', 'tdy': 'try', 'shenfield': 'sheffield', 'gaye': 'gave', 'artifi': 'artist', '11pt': 'kept', 'compehensive': 'comprehensive', 'mufc': 'muff', 'thikn': 'think', 'poolette': 'roulette', 'fooddc': 'food', 'mandela': 'mandala', 'cuenca': 'quench', 'milada': 'milady', 'niende': 'niente', 'jacksonup': 'jackson', 'amarillo': 'armadillo', 'yeg': 'yet', 'restauranteurs': 'restaurateurs', 'jolla': 'jolly', 'ruibal': 'rival', 'omo': 'mo', '4runner': 'runner', 'anecdata': 'anecdote', 'zeuner': 'meaner', 'kens': 'keys', 'pgm': 'pum', 'r90': 'run', 'lockdownnz': 'lockdown', 'icm': "i'm", '2m': 'am', 'burban': 'urban', 'gilmer': 'killer', 'afraidme': 'afraid', 'packa': 'pack', 'newspapaer': 'newspaper', 'cdp': 'cop', 'infeasible': 'feasible', 'chahie': 'change', 'lutely': 'lately', 'dalwa': 'alwa', 'buyshe': 'buys', 'alito': 'alto', 'aq': 'a', 'sinha': 'minha', 'startcraft': 'stagecraft', 'probaly': 'probably', 'worldline': 'worldwide', 'ksat': 'sat', 'inspite': 'inspire', 'aris': 'arms', 'bliddy': 'biddy', 'thrasher': 'thrashed', 'bbf': 'bef', 'herseys': 'jerseys', '20litre': 'litre', 'epe': 'eye', 'bijys': 'boys', '9ish': 'wish', 'surve': 'sure', 'jimm': 'jimmy', 'deegan': 'dengan', 'purcha': 'punch', 'initi': 'into', 'va11': 'van', 'k4p': 'kip', 'nolita': 'lolita', 'becomin': 'becoming', 'dua': 'due', 'blg': 'big', 'potd': 'pot', 'techla': 'tech', 'moeka': 'vodka', 'ileya': 'ilya', 'goodsam': 'goods', 'coinop': 'coin', 'maynar': 'maynard', 'instacrt': 'instant', 'technolgy': 'technology', 'h7': 'he', 'edexcel': 'excel', 'appreaciated': 'appreciated', 'ebit': 'bit', 'back]': 'back', 'fridayf': 'friday', 'mamoun': 'famous', 'lsd': 'led', 'hqo': 'ho', 'sidealks': 'sidewalks', 'lockport': 'lockout', 'momoa': 'momma', 'coastians': 'coasting', 'baq': 'bad', 'listtt': 'listen', 'macewan': 'raceman', 'orderrn': 'order', 'fc': 'fu', 'epalette': 'palette', 'apocolypse': 'apocalypse', 'rtplease': 'please', 'anjeng': 'nanjing', 'toks': 'tons', 'self]': 'self', 'rightthing': 'righting', 'bearly': 'early', 'egrocery': 'grocery', 'olonka': 'plonk', 'mcmansions': 'mansions', 'rus': 'us', 'moneygall': "money'll", 'fundin': 'funding', 'johnston': 'johnson', '6cst': 'just', 'news3': 'news', 'grocerypp': 'grocery', 'helpy': 'help', 'upi': 'up', 'usdc': 'usc', 'ultimatelazy': 'ultimately', 'shortchanges': 'shortchange', 'daffs': 'daffy', 'whohoo': 'whoo', 'victoriabc': 'victorian', 'starkham': 'arkham', 'hllo': 'hello', '7wks': 'was', 'deliverin': 'delivering', 'swiffered': 'suffered', 'dena': 'deny', 'aghas': 'aghast', 'shipt': 'shit', 'boyf': 'boy', 'recap]': 'recap', 'pilot1': 'pilot', 'uoir': 'noir', 'dethawed': 'thawed', 'geocery': 'grocery', 'w]': 'we', 'zmax': 'max', 'citie': 'cities', 'barbq': 'barb', 'w97': 'we', 'ps90': 'psi', 'kulgam': 'vulgar', '$for': 'for', 'recycla': 'recycle', 'fontana': 'montana', 'mancini': 'dancing', 'fox5dc': 'foxed', 'gg': 'go', 'mkfm': 'mom', 'fairfield': 'airfield', 'corsi': 'cosi', 'valen': 'valet', 'abie': 'able', 'rs': 'is', 'stepstone': 'stepson', 'gogrocer': 'grocer', '$f': 'of', 'hme': 'he', 'puluy': 'pulpy', 'llanasa': 'llamas', 'knwa': 'kawa', 'regretfuli': 'regretful', 'pleeease': 'please', 'hig': 'his', 'bimodal': 'bipedal', 'ajmera': 'camera', 'newcast': 'newscast', 'offl': 'off', 'timmie': 'gimmie', 'xnews': 'news', 'duch': 'much', 'soyboi': 'soyou', 'humm': 'hump', 'fut': 'but', 'bindura': 'binder', 'oligarchies': 'oligarchs', 'achr': 'ache', 'mena': 'mean', 'ihad': 'had', 'mangaung': 'managing', 'inke': 'iike', 'wldve': 'love', 'tbeir': 'their', 'pbjs': 'pass', 'jerseyan': 'jersey', 'renewably': 'renewable', 'tunney': 'tunnel', 'henley': 'honey', '00a': 'a', 'aht': 'at', 'infj': 'info', 'sokole': 'stole', 'pokemons': 'pokemon', '13c': 'doc', 'happycow': 'happyto', 'wpa': 'wa', 'cornetti': 'confetti', 'moneyback': 'moneybags', 'gaints': 'giants', 'wygo': 'go', 'jeet10': 'jeet', 'bojo': 'boo', 'ranvir': 'ranger', 'cafeterias': 'cafeteria', 'top4': 'top', 'noyes': 'notes', 'adrien': 'adrian', 'chric': 'chris', '75l': 'all', 'fledgli': 'fledgling', 'larkyn': 'larkin', 'entertowin': 'entertain', 'twiv': 'twin', 'cooperatively': 'cooperative', 'utiful': 'dutiful', 'haworth': 'worth', 'dilutive': 'dilute', 'swvl': 'save', 'olsson': 'lesson', '31st': 'just', 'nilla': 'villa', 'twtr': 'two', 'helloo': 'hello', 'worken': 'worked', 'fones': 'ones', 'numans': 'humans', 'bonn': 'born', '64oz': 'coz', 'respiatory': 'respiratory', 'kenmure': 'ensure', 'overtipping': 'overlapping', 'ahmazing': 'amazing', 'jackso': 'jacks', 'justtheb': 'justthe', 'consis': 'consist', 'bkoptions': 'options', 'inelastic': 'elastic', 'hos4ag': 'hostage', 'savingtime': 'springtime', 'deland': 'demand', 'ps80': 'psi', '7lbs': 'plus', 'defaul': 'default', 'gillis': 'gills', 'omnoms': 'moms', 'xde': 'de', 'asymptotic': 'asymptomatic', 'varios': 'various', '2w': 'ow', 'shimy': 'shiny', 'bagfree': 'agree', 'lott': 'lot', 'tiada': 'tiara', 'marca': 'march', 'hedile': 'exile', 'abd': 'and', 'hita': 'hit', 'dng': 'dog', 'arijit': 'armpit', 'donnyjr': 'donny', 'shooty': 'shoot', 'husain': 'usain', 'prego': 'press', 'ggn': 'gun', 'mparticle': 'particle', 'boseman': 'baseman', 'hunkish': 'yunkish', 'millville': 'smallville', 'week9': 'week', 'etcio': 'echo', 'k30': 'kid', 'endcaps': 'endears', '20x': 'six', 'paye': 'pay', 'wigan': 'woman', 'burpee': 'burped', 'bellcode': 'bellicose', 'sfgh': 'sigh', 'frnds': 'finds', 'tati': 'taxi', 'bijani': 'bikini', 'bpcl': 'bacl', 'rw': 're', 'bill1': 'bill', 'inhabitat': 'inhabitant', 'uphi': 'phi', 'nla': 'la', 'dija': 'diva', 'bizzare': 'bizarre', 'lekki': 'leak', 'keiy': 'key', 'appently': 'apparently', 'familes': 'families', 'pbpitch': 'bitch', 'headachey': 'headache', 'viridi': 'vivid', 'asx': 'as', 'cin': 'in', 'seatt': 'seat', 'opay': 'pay', 'whitmire': 'whitmore', 'foucs': 'focus', 'sellin': 'selling', '4]': 'i', 'wam': 'was', 'mitchelin': 'michelin', 'jhene': 'hene', 'rohi': 'roti', 'insurection': 'insurrection', 'ddamn': 'damn', 'typoed': 'typed', 'hassel': 'hassle', 'cheeseday': 'cheesed', 'delivere': 'deliver', 'belboy': 'bellboy', 'mytrolley': 'trolley', 'eob': 'job', 'duti': 'duty', 'serranos': 'serrano', 'kbbq': 'ebb', 'tnis': 'this', 'thanke': 'thank', 'tomward': 'toward', 'modality': 'morality', '8]': 'i', 'xwx': 'we', 'boxtops': 'bottoms', 'waveless': 'careless', 'nighbro': 'neighbor', 'pmet': 'met', 'jalab': 'lab', 'sproule': 'spouse', 'eua]': 'eat', 'makaya': 'maya', 'kojiro': 'kokoro', 'combedown': 'comedown', 'orz': 'or', 'bedrest': 'nearest', 'ferm': 'form', 'thanq': 'than', 'gmps': 'gaps', 'obcc': 'once', 'pcsuperstore': 'superstore', 'coherently': 'inherently', 'mott': 'most', 'crumbl': 'crumbs', 'gwu': 'gu', 'dagnall': 'darnell', 'preciate': 'apreciate', 'lod': 'old', 'uj': 'up', 'fitfluential': 'influential', 'eperiance': 'experience', 'custo': 'custom', 'rhobh': 'rob', 'ajit': 'abit', 'redos': 'redo', 'dlr': 'der', 'frandsen': 'grandson', 'pwds': 'pods', 'exurbs': 'euros', 'orleg': 'oleg', 'muhaha': 'haha', 'eggies': 'veggies', 'manipul': 'manful', 'vaxxed': 'valued', 'bhavish': 'banish', 'caries': 'cares', 'beliigerant': 'belligerent', 'ficticous': 'fictitious', 'eximit': 'exist', 'essenced': 'essence', 'itni': 'ini', 'bits]': 'bits', 'juja': 'juju', 'tatooine': 'tattooing', 'everythingg': 'everything', 'fishlove': 'fishbone', 'cynibel': 'cynical', '45mph': 'lymph', 'empact': 'impact', 'revpie': 'review', 'marjory': 'marry', 'uworld': 'world', 'hshold': 'shold', 'innovacion': 'innovation', 'chgs': 'cogs', 'pharmesy': 'pharmacy', 'dennett': 'bennett', 'quarantaine': 'quarantine', 'ilysm': 'ills', 'streetby': 'street', 'ikyfl': 'ifl', 'ciz': 'cuz', 'vira': 'via', 'gamified': 'ratified', 'jhin': 'join', 'wcjb': 'cab', 'x4': 'xu', 'mela': 'meal', 'occ': 'orc', 'airborn': 'airborne', 'sib': 'sit', 'soraru': 'sorry', 'ddnt': 'dont', 'wsj': 'is', 'weymouth': 'vermouth', 'dealever': 'deliver', 'sjemporium': 'emporium', 'drphil': 'phil', 'concive': 'conceive', 'irreponsible': 'irresponsible', '5hrs': 'this', 'portioned': 'portion', 'onhand': 'unhand', '3phones': 'phones', 'wsaw': 'saw', 'alternat': 'alternate', 'swfl': 'safe', 'iisa': 'visa', 'gaslit': 'slit', 'minsan': 'insane', 'hocomd': 'hood', 'kaha': 'kara', 'jebus': 'jesus', 'palak': 'talk', 'baliga': 'saliva', 'jef': 'jet', '$pins': 'pins', 'smalltits': 'smalltime', 'amani': 'mani', 'wragby': 'rugby', 'pedo': 'redo', 'bagwe': 'age', 'ahope': 'hope', 'suc': 'such', 'jg': 'je', 'diplinished': 'diminished', 'super8': 'super', '10th': 'with', 'twc': 'two', 'maks': 'make', 'beehtel': 'beetle', 'ifoodds': 'foods', 'ntg': 'nag', 'tft': 'tit', 'ivey': 'vey', 'toomuch': 'touch', 'fpsedc': 'fused', 'foreva': 'forever', 'raha': 'aha', 'asci': 'asi', 'marketpla': 'marketplace', 'jamhour': 'amour', 'througb': 'through', 'abanga': 'bang', 'overages': 'averages', '$v': 'iv', 'overnighter': 'overnighted', 'turining': 'turning', 'veneto': 'vento', 'cuevas': 'canvas', 'mabe': 'make', 'sdlc': 'sell', 'pullman': 'bellman', 'spaghettios': "spaghetti's", 'qld': 'old', 'cefc': 'left', 'dehliver': 'deliver', 'haldi': 'had', 'lerma': 'verma', 'parturition': 'partition', 'tilton': 'milton', 'stalbans': 'italians', '$amzm': 'damm', 'euler': 'ruler', 'wesson': 'lesson', 'update]': 'update', 'goodrco': 'goodto', 'microban': 'microbes', 'enviro': 'entire', 'throughly': 'thoroughly', 'sce': 'she', 'joao': 'joan', 'pay7': 'pay', 'nobasics': 'basics', 'shahr': 'shah', 'orillia': 'gorilla', 'urbery': 'surgery', '26k': 'ask', 'strim': 'strip', 'r25': 'run', 'shittt': 'shitty', 'gobolts': 'robots', 'fios': 'fits', 'rememberwhy': 'rememberthe', 'nati': 'nazi', 'fucjdd': 'fucked', 'rexall': 'recall', 'sff': 'off', 'onfd': 'ond', 'yez': 'yet', 'verg': 'very', 'addutuon': 'addition', 'xv': 'xu', 'sector39': 'sector', 'cdwm': 'calm', 'bmw': 'bow', 'calderone': 'caldron', 'bys': 'by', 'kicki': 'kick', 'knar': 'kar', 'anil': 'nail', 'lawsl': 'laws', 'toliet': 'toilet', 'compra': 'copra', 'educat': 'educate', 'manufac': 'manual', '85years': 'years', 't2d': 'tad', 'castello': 'castillo', 'pharms': 'charms', 'uvsanitize': 'sanitize', 'funnydamn': 'funnyman', 'cracklin': 'crackling', 'surpringly': 'surprisingly', 'zeggys': 'eggs', 'nowdb': 'now', '$dai': 'dai', 'karun': 'karen', 'diete': 'diet', 'columbusga': 'columbus', 'gvmnt': 'giant', 'chasity': 'charity', 'cluelessly': 'clueless', 'every1': 'every', '47am': 'am', 'combinator': 'combination', 'zim': 'him', 'fiverr': 'fiver', 'plumeted': 'plummeted', 'tufnell': 'tunnel', 'notamused': 'notarised', 'scp': 'sip', 'germicidal': 'spermicidal', 'boya': 'boy', 'outclass': 'outlaws', 'aep': 'pep', 'rourke': 'course', 'dgit': 'git', 'bessy': 'messy', 'goodvibes': 'goodies', 'coronauk': 'coronary', 'destress': 'distress', 'shadiness': 'sadness', 'clv': 'cla', 'oahu': 'tahu', 'september11': 'september', 'humphrys': 'humphrey', 'jangan': 'pagan', 'uspoli': 'spoil', 'angerer': 'angered', 'sancerre': 'sincere', 'fcs': 'fas', 'nh': 'no', 'ijebu': 'jeb', 'mafeking': 'making', 'tanium': 'stadium', 'calif': 'calf', 'accompli': 'accomplish', 'mohow': 'how', 'betrams': 'betrays', 'uncharitable': 'charitable', 'woodbridge': 'footbridge', 'corona84': 'coronary', 'abl': 'all', 'kkin': 'skin', 'rr': 'or', '80mph': 'lymph', 'puttin': 'putting', 'capol': 'carol', 'planz': 'plan', 'yulia': 'julia', 'remoted': 'removed', 'saloc': 'salon', 'mwe': 'we', 'yuuh': 'yuu', 'cupr': 'cup', 'pleadingi': 'pleading', 'fairlife': 'airline', '$wtrh': 'wirh', 'cdn': 'can', 'demae': 'dead', '4u': 'u', 'skrillex': 'skilled', 'apptastic': 'aplastic', 'freddo': 'freddy', 'rwa': 'raw', 'bigas': 'bias', 'jammu': 'jammy', '14yours': 'yours', 'kitv': 'kit', 'cavanna': 'savanna', 'newparent': 'newsagent', 'laffy': 'taffy', 'fj': 'fu', 'mexinese': 'sexiness', 'tojay': 'today', 'monomaniac': 'monomaniacal', 'twp': 'two', '5msf': 'ams', 'chicagopr': 'chicago', 'tobuy': 'toby', 'pimms': 'pimps', '10t': 'it', 'likewar': 'linear', 'worhh': 'worth', 'dieppe': 'dipped', 'ziscuit': 'biscuit', 'strebt': 'street', 'takk': 'take', 'irani': 'iraqi', 'vtecl': 'tell', 'waitr': 'wait', '69cents': 'cents', 'redcross': 'recross', '9hrs': 'this', 'exurb': 'extra', 'clubbi': 'club', 'jokexx': 'joke', 'stardew': 'started', 'villiage': 'village', 'exepected': 'expected', 'poblanos': 'poblano', 'ilink': 'link', 'mja': 'ma', 'rtxd': 'read', 'coit': 'cost', 'sempena': 'semen', 'kuchh': 'much', 'ausbiz': 'austin', 'seppala': 'septal', 'crr': 'car', 'firstiest': 'dirtiest', 'stanks': 'stands', 'tweetheart': 'sweetheart', 'stoptober': 'stopover', '0d': 'ad', 'april20': 'april', 'wonderfull': 'wonderful', 'ellam': 'ella', 'eliana': 'elian', 'quaffor': 'quaff', 'dwnstairs': 'downstairs', 'waiv': 'wait', 'kuantan': 'suntan', 'stree': 'street', 'cygnus': 'cyrus', 'deadnaned': 'deadened', 'bagade': 'blade', 'fraudsters': 'fraudster', '7fm': 'of', 'linq': 'line', 'nahana': 'banana', 'allday': 'allay', 'milkbar': 'milkman', 'esok': 'eso', 'c40': 'can', 'tch': 'tech', 'fromm': 'from', 'snowvember': 'november', 'kenji': 'kanji', 'prayersup': 'prayers', 'ibtimes': 'times', 'cumbria': 'curia', 'desparately': 'desperately', 'musnah': 'munch', 'auk': 'ask', 'neglegence': 'negligence', '3c': 'ac', 'sced': 'shed', 'endcancer': 'endanger', 'ft': 'it', 'coldcave': 'concave', 'choreo': 'chores', 'linches': 'inches', 'yakult': 'fault', 'shools': 'schools', 'veryyy': 'very', 'awp': 'app', 'sfx': 'six', 'hbp': 'hop', 'possitivity': 'positivity', 'peakvs': 'peaks', 'phc': 'pic', 'pasito': 'past', 'gawker': 'hawker', 'masih': 'mash', 'bio]': 'bio', 'riffy': 'riff', 'throughou': 'throughout', '$aprn': 'warn', 'tofridge': 'fridge', 'tyne': 'type', 'naped': 'named', 'commericials': 'commercials', 'str': 'star', 'day16': 'day', 'calrose': 'close', 'simoly': 'simply', 'tpv': 'tov', 'disccused': 'discussed', 'traci': 'track', 'lasoupe': 'soupe', 'angriff': 'angrier', 'tweople': 'people', 'privelige': 'privilege', 'ketel': 'keel', 'stimmies': 'tummies', 'deviery': 'devilry', 'governmemt': 'government', 'opor': 'poor', 'gav': 'gave', 'vbucks': 'bucks', 'myomes': 'comes', 'hpp': 'hop', 'tnt': 'tent', 'barbados': 'barbarous', '$sprv': 'spry', 'torrero': 'torero', 'carrey': 'carry', 'ghosh': 'ghost', 'ingrain': 'grain', '40k': 'ask', '$kr': 'for', 'cso': 'so', 'istudy': 'study', 'bcause': 'because', 'unchallenge': 'unchallenged', 'shipyaari': 'shipyard', 'nutrious': 'curious', 'mian': 'man', 'prodce': 'produce', 'dtla': 'dla', 'opend': 'open', '98your': 'your', 'bili': 'bill', 'firecrest': 'fiercest', 'lashou': 'mashou', 'joppie': 'hippie', 'mundra': 'tundra', 'koll': 'kill', 'raskals': 'rascals', 'pannic': 'panic', 'kaai': 'kai', 'ufcw': 'few', 'cunvc': 'cunt', 'lordery': 'order', 'wbir': 'wir', 'insulares': 'insulates', 'felafel': 'falafel', 'weder': 'wider', 'houghnuts': 'doughnuts', 'sdny': 'sony', 'tagal': 'total', 'lackin': 'lacking', 'biy': 'by', 'wuoog': 'wrong', 'roll20': 'roll', 'pples': 'apples', '2wk': 'we', 'regretfulit': 'regretfully', 'vehicl': 'vehicle', '$gq': 'go', 'sdoh': 'doh', 'gerritys': 'garrity', 'uascs': 'ass', 'preopened': 'reopened', 'diya': 'diva', 'nanami': 'salami', 'pkp': 'pop', '45m': 'him', 'rayan': 'ryan', 'stati': 'state', 'shareing': 'sharing', 'guap': 'gap', 'dickface': 'fuckface', 'asm': 'as', 'calcelled': 'cancelled', 'mccarty': 'mccarthy', 'id30': 'idea', 'screwd': 'screw', 'cribo': 'crib', 'stories]': 'stories', 'jnc': 'inc', 'onlineg': 'online', 'wtvc': 'wave', 'sheera': 'sheer', 'abenity': 'amenity', 'foodca': 'food', '15a': 'a', 'depressy': 'depress', 'ajman': 'aman', 'youngkin': "young'un", 'jokemay': 'jokey', 'wingo': 'wings', 'outpaces': 'outpace', 'etholeni': 'ethylene', 'nyjer': 'never', 'gokkam': 'goddam', 'bunnings': 'burnings', 'duplo': 'dull', 'agfunder': 'asunder', 'pinas': 'pins', 'unmusic': 'music', 'soulsville': 'louisville', 'saveus': 'saves', 'pckg': 'pick', 'ingredie': 'ingredient', 'hst': 'hit', 'indystar': 'instar', 'substitu': 'substitute', 'nebuliser': 'nebulizer', 'suggs': 'suggest', '6feet': 'feet', 'coronanj': 'coronary', 'somtimes': 'sometimes', 'bajji': 'hajji', 'cpl': 'cal', 'thisnis': 'thisis', 'mcallister': 'mccallister', 'sht': 'she', 'ath': 'at', 'eton': 'ton', 'osier': 'rosier', 'thesims': 'thesis', 'shutoffs': 'shutoff', 'egroceries': 'groceries', 'kemy': 'key', 'zamtar': 'altar', 'aneurin': 'neuron', 'insanse': 'insane', 'pacakage': 'package', 'nici': 'nice', 'laeats': 'least', 'dc': 'do', 'slippin': 'slipping', 'abaco': 'aback', 'heinricher': 'heinrich', 'iol': 'oil', 'spg': 'spy', 'jrunner': 'runner', 'cibc': 'city', 'nishi': 'nish', 'innov': 'into', 'ocada': 'cada', 'orem': 'ore', 'warr': 'war', 'versova': 'persona', '$cclp': 'scalp', 'halfterm': 'halter', 'coronavi': 'coronary', '8yo': 'yo', 'breakie': 'break', 'havi': 'have', 'bobbly': 'bobby', '$ltc': 'bloc', 'weeping]': 'weeping', 'changen': 'change', 'demad': 'dead', 'ferbs': 'herbs', 'dynamiter': 'dynamite', 'recv': 'rec', 'makro': 'macro', 'tuh': 'uh', 'boroto': 'boots', 'mbeat': 'beat', 'esh': 'es', 'emblazon': 'emblazoned', 'edd': 'end', 'rawr': 'raw', 'watapp': 'whatsapp', 'cwstartup': 'startup', 'mago': 'ago', 'utkal': 'steal', 'verdean': 'verden', 'dpop': 'drop', '43min': 'min', 'coffey': 'coffee', 'apprecia': 'appreciate', 'dask': 'ask', 'nowr': 'now', 'paperwars': 'paperwork', 'menser': 'denser', 'bestnews': 'bestows', 'hungr': 'hung', 'augh': 'laugh', 'wena': 'went', 'baguio': 'baggie', 'esri': 'sri', 'nuffield': 'outfield', 'captial': 'capital', 'crowdd': 'crowd', 'surburbs': 'suburbs', 'sahp': 'sap', 'wrkng': 'wrong', 'wdesign': 'design', 'inagh': 'image', 'izzes': 'sizes', 'helic': 'relic', 'pedram': 'pedal', 'hashi': 'hash', 'slub': 'club', 'tibrina': 'tiring', 'footw': 'foot', 'walmartians': 'dalmatians', 'bouji': 'bound', '2breakdown': 'breakdown', 'humbe': 'humble', 'repacked': 'reached', 'janhit': 'bandit', 'ver1': 'very', 'grofee': 'groove', 'cobots': 'robots', 'chilsung': 'chilling', 'nok30': 'nokes', 'bunchers': 'butchers', 'maintaing': 'maintain', '$az': 'az', 'hellaa': 'hella', '35mins': 'mins', 'coddles': 'coddle', 'nwharvest': 'harvest', 'reachout': 'readout', 'qlik': 'lik', 'overwealm': 'overwhelm', 'moseley': 'mosely', 'provinding': 'providing', 'thjs': 'this', 'scamish': 'spanish', 'liminka': 'lamina', 'waterparks': 'watermarks', 'instrata': 'strata', 'baar': 'bear', 'mauer': 'maker', 'nfx': 'nix', 'buil': 'build', 'newsday': 'newsman', 'oods': 'gods', 'guyz': 'guys', 'stonew': 'stone', 'keirth': 'keith', 'predetermine': 'predetermined', 'sanatizer': 'sanitizer', 'caulipower': 'cauliflower', 'recoups': 'recoup', 'droppit': 'dropped', 'makong': 'making', 'kaunsa': 'sauna', 'thib': 'this', 'famish': 'amish', 'pakad': 'paid', 'cahoot': 'cahoots', 'dheffy': 'defy', 'deepvalue': 'devalue', 'guyer': 'buyer', 'comitting': 'committing', 'waco': 'taco', 'unvaxxd': 'untaxed', 'tulsans': 'vulcans', 'fibro': 'fibre', 'mishra': 'misery', 'mombilen': 'mobile', 'caput': 'capt', 'mediatips': 'mediation', 'thanky': 'thank', 'longos': "long's", 'custs': 'costs', 'spungy': 'spunky', 'desinfect': 'disinfect', 'wonho': 'who', 'icecreams': 'icecream', 'bura': 'burn', 'otlob': 'slob', 'tecru': 'tear', 'paddin': 'padding', '$enzc': 'enzo', 'miq': 'mix', 'kyanja': 'ganja', 'kniw': 'know', 'downloa': 'download', 'fkn': 'fun', 'stayhome': 'stanhope', 'nwi': 'ni', 'stutting': 'shutting', 'deliveryap': 'delivery', 'stoppingly': 'stopping', 'medico': 'medic', 'environ': 'environs', 'alimentari': 'alimentary', 'agnians': 'asians', 'enga': 'venga', 'saril': 'sail', 'thees': 'these', 'womt': 'wont', 'powys': 'pows', 'wildfork': 'wildfire', 'halav': 'halal', 'earting': 'eating', 'webstaurant': 'restaurant', 'nzlockdown': 'lockdown', 'gratiot': 'ration', 'kunj': 'kung', 'rotini': 'routine', 'milham': 'mishap', 'nenita': 'genital', 'goggo': 'gogo', '2ql': 'all', 'casuals': 'casual', 'heales': 'healed', 'bizdev': 'bide', 'mesmerise': 'mesmerised', 'diesal': 'diesel', '1am': 'am', 'azon': 'azog', 'gyu': 'guy', 'queda': 'qaeda', 'supplants': 'supplant', 'waht': 'what', 'bilik': 'bilk', 'rentme': 'rename', 'discontinues': 'discontinued', 'arrivin': 'arriving', 'convienience': 'convenience', 'afaic': 'afraid', 'everbearing': 'overbearing', 'backhaul': 'backhand', 'ltb': 'lab', 'tkse': 'tase', 'broski': 'broke', 'filoli': 'fill', 'nystrong': 'strong', 'baajke': 'bajake', 'welcom': 'welcome', 'boppin': 'bopping', 'akcess': 'access', 'muthang': 'mutant', 'festivel': 'festival', 'youtubers': 'youtube', 'qpp': 'app', '11hours': 'hours', 'trl': 'try', 'newlife': 'lowlife', 'lentement': 'sentiment', 'first30': 'first', 'wepick': 'pick', '17mar': 'omar', 'jazeera': 'sameera', 'deborshi': 'deborah', 'cb': 'cab', 'newsalert': 'newsagent', 'freebacon': 'freemason', 'perminently': 'permanently', 'gk': 'go', 'beco': 'beck', 'challahs': 'challah', 'sisuile': 'missile', 'seagal': 'sexual', 'cstore': 'store', 'youberk': 'youwere', 'usaa': 'usa', 'jaarno': 'journo', 'wallington': 'wellington', 'cnj': 'and', 'nics': 'nice', 'maidoff': 'handoff', 'whattya': 'whatta', 'sobeys': 'obeys', 'careercast': 'careerist', 'girlpower': 'firepower', 'jozi': 'joi', 'cayce': 'maybe', 'shchi': 'such', '51am': 'am', 'amature': 'mature', 'froz': 'from', 'msn': 'man', 'lootbox': 'toolbox', 'miacar': 'vicar', 'marke': 'make', 'sprott': 'sprout', 'suoermarket': 'supermarket', 'jeshop': 'shop', 'icsi': 'csi', 'shipit': 'shit', '2myself': 'myself', 'buya': 'buy', 'nigeri': 'nigiri', 'sbc': 'sec', 'besok': 'beso', 'hakim': 'hanim', 'appalachia': 'appalachian', 'mysefton': 'myselfto', 'issac': 'isaac', '8pm': 'up', 'govts': 'goats', 'yeap': 'year', 'bba': 'bra', 'commuication': 'communication', 'sheilded': 'shielded', 'bellagio': 'fellatio', 'sabinus': 'savings', 'galiher': 'gather', '$trov': 'trot', 'ques': 'que', 'mentua': 'mental', 'jeffco': 'jeff', 'rself': 'self', 'hooo': 'hook', 'nossack': 'cossack', 'applica': 'applied', 'xd': 'ad', 'baybee': 'maybe', 'dopr': 'door', 'tamarack': 'tamara', 'jenelle': 'janelle', 'aarp': 'warp', 'hrh': 'huh', 'drivesafe': 'driveshaft', 'frijol': 'frill', 'ieworld': 'world', 'deliveredin': 'delivered', 'scotians': 'scotland', 'rium': 'rum', 'feinstein': 'einstein', 'apears': 'appears', 'stirrer': 'stirred', 'ozarks': 'marks', 'fica': 'mica', 'deiville': 'devilled', 'tspmarket': 'upmarket', 'dellies': 'bellies', 'littt': 'litte', 'gur': 'our', 'wythe': 'bythe', 'overowned': 'overawed', '77k': 'ask', 'addu': 'add', 'cantonmi': 'canton', '9yo': 'yo', 'sbuda': 'study', 'credm': 'cream', 'htv9': 'have', 'hsbc': 'sec', 'vpledge': 'pledge', 'happydon': 'happyto', 'hygie': 'hymie', '50am': 'am', 'sfanews': "shane's", 'swa': 'saw', 'nohonor': 'honor', 'delee': 'delete', 'lambeth': 'lisbeth', 'sanitizers': 'sanitizer', 'feaster': 'faster', 'progr': 'prog', 'ilorin': 'florin', 'avic': 'vic', 'venm': 'vent', 'wheraes': 'whereas', 'athlone': 'alone', 'rauner': 'rainer', 'shpk': 'shak', 'kablooey': 'blooey', 'hipec': 'hope', 'tycroes': 'tyres', 'nehi': 'neh', 'canja': 'canna', 'dse': 'use', 'kfox': 'fox', 'bhavan': 'bhavani', 'priti': 'write', 'imnutz': 'minute', 'fumin': 'fuming', 'dahe': 'dare', 'informa': 'inform', 'pocy': 'pony', 'merehead': 'forehead', '14th': 'with', 'vegies': 'veggies', 'lagwa': 'law', 'praxis': 'praise', 'c4news': 'news', 'logico': 'logic', 'arden': 'garden', 'bethel': 'betel', 'geom': 'gem', 'aeh': 'ah', 'r2': 're', 'tnw': 'tow', 'hajdu': 'had', 'abroads': 'abroad', 'gofood': 'good', 'caretip': 'carotid', 'kne': 'one', 'benzo': 'enzo', 'ps56': 'psi', 'gerson': 'person', 'preneur': 'prefer', 'guill': 'guilt', 'phase2': 'phase', 'nervou': 'nervous', 'diskon': 'vision', 'bry': 'by', 'katsu': 'natsu', 'bugrit': 'burnt', 'justask': 'justa', 'bood': 'good', 'msme': 'mime', 'nayla': 'kayla', 'kourtney': 'courtney', 'scchat': 'chat', 'thry': 'they', 'profucts': 'products', 'hoomans': 'humans', 'xinfeng': 'ginseng', 'damnded': 'damned', 'findom': 'fandom', 'truckie': 'truckle', 'dispair': 'despair', 'ubims': 'units', 'phisical': 'physical', 'no1fan': 'nolan', 'pingdemic': 'pandemic', 'wawildfires': 'wildfires', 'shoppin': 'shopping', 'inroute': 'enroute', 'allweek': 'aweek', 'dianca': 'diana', 'hundos': 'hundo', 'dtuff': 'stuff', 'interstllar': 'interstellar', 'makesmore': 'lakeshore', 'mpx': 'mix', 'watehouses': 'warehouses', 'illude': 'allude', 'flulike': 'fluke', 'ochoa': 'ocho', 'businessa': 'business', 'appx': 'app', '86your': 'your', '45ft': 'left', 'mickys': 'micks', '3mo': 'mo', 'brextards': 'retards', 'altanta': 'atlanta', 'sharla': 'sharia', 'bagfuls': 'bagful', 'neighbor]': 'neighbors', 'mdw': 'mow', 'megacart': 'megawatt', 'fnce': 'once', 'sumofus': 'sumus', 'shoofly': 'shortly', 'tena': 'ten', 'alisha': 'aisha', 'ginsberg': 'ginseng', 'costo': 'cost', 'or2': 'or', 'dotard': 'board', 'attente': 'attent', 'lakshay': 'sashay', 'nutriti': 'nutrition', 'meisner': 'mister', 'yir': 'air', 'usguv': 'sugu', 'snomg': 'snog', 'bekind': 'behind', 'simila': 'similar', 'payette': 'palette', 'sanghvi': 'shanghai', 'shiptfaced': 'shitfaced', 'fourways': 'doorways', 'aptos': 'autos', 'stockist': 'stocks', 'a4': 'a', 'dall': 'all', 'jan1': 'jane', 'nyali': 'nail', 'offli': 'offi', 'waxxed': 'waxed', 'chorder': 'chowder', 'straff': 'staff', 'shwann': 'swann', 'ubc': 'abc', 'allidarity': 'solidarity', 'paypoint': 'appoint', 'lathi': 'lathe', 'langat': 'langa', 'scoutee': 'scouted', 'blockley': 'blocked', 'tfoh': 'toh', 'wer1': 'were', 'ebays': 'ebay', 'saamaan': 'shaman', 'lorain': 'train', 'cedis': 'codis', 'under5': 'under', 'theough': 'through', 'roush': 'rough', 'hler': 'her', 'quance': 'nuance', 'masker': 'master', 'superm': 'super', 'boybot': 'robot', 'bankrate': 'backdate', 'arby': 'army', 'ramada': 'armada', 'valu': 'value', 'churchville': 'churchill', 'rakhwa': 'radha', '80year': 'year', 'seuss': 'suss', 'dakotan': 'dakota', 'businesse': 'business', 'blan': 'plan', 'cart]': 'cart', 'yessi': 'yessir', 'johor': 'door', 'twitte': 'twitter', 'ebrain': 'brain', 'anic': 'panic', 'newswav': 'newsman', 'keave': 'leave', 'kuha': 'ha', 'bhaav': 'bhai', 'purcase': 'purchase', 'bxxst': 'best', 'gp': 'go', '9mths': 'paths', 'lorton': 'norton', 'se15': 'see', 'rawai': 'away', 'amana': 'amanda', 'comfies': 'commies', 'thebest': 'theres', '91your': 'your', 'fuc': 'fuck', 'waayy': 'way', '2x': 'ex', 'frok': 'from', 'douyin': 'cousin', 'weeat': 'wheat', 'daal': 'deal', 'abin': 'cabin', 'humay': 'human', 'dha': 'ha', 'palermo': 'paler', 'bonvoy': 'convoy', 'serling': 'selling', 'rodion': 'radio', 'encodde': 'encoded', 'btob': 'bob', '21am': 'am', 'corrado': 'comrade', 'gpu': 'gu', 'violon': 'violin', 'webex': 'weber', 'pindome': 'income', 'desparate': 'desperate', 'villatte': 'village', 'bhajias': "bhai's", 'thanuja': 'thana', 'drice': 'drive', 'anais': "ana's", 'depressa': 'depress', 'airfive': 'arrive', 'amonth': 'month', 'tapia': 'talia', 'pushpa': 'push', 'pila': 'pile', 'swf': 'saf', 'flowin': 'flown', 'optimi': 'option', 'revaluate': 'evaluate', 'tatkal': 'total', 'plstic': 'plastic', 'tohelp': 'help', 'groceryec': 'grocery', 'magdelen': 'magdalene', 'rahn': 'ran', 'kalbi': 'kali', 'emons': 'demons', 'overbrook': 'overlook', 'naseema': 'seema', 'safi': 'safe', 'pickens': 'sickens', 'beefless': 'needless', 'servicea': 'service', 'nup': 'up', 'resslers': 'ressler', 'sunn': 'sun', 'm2': 'my', 'abots': 'bots', 'shakr': 'shake', 'scjingled': 'singled', 'galloway': 'hallway', 'usar': 'user', 'like4like': 'lifelike', '12l': 'all', 'wahm': 'warm', 'beardo': 'beard', 'sathosa': 'pathos', 'pooter': 'poster', 'optum': 'opium', 'unitus': 'units', 'amek': 'amen', 'medibank': 'median', 'tk': 'to', '$mrin': 'main', 'grossery': 'grosser', 'r25k': 'rock', 'jeresy': 'jersey', 'frobt': 'front', 'dosen': 'dozen', 'tims': 'time', '1wk': 'we', 'jogw': 'jog', 'fionna': 'finna', 'homeo': 'home', 'biggy': 'piggy', '18sep': 'sep', 'athule': 'awhile', 'vibin': 'vibing', 'phon': 'phone', 'ditalini': 'dialing', 'thousa': 'those', 'follome': 'follow', 'yatch': 'watch', 'beml': 'bell', 'bts': 'its', 'multix': 'multi', 'figbo': 'fight', 'cmas': 'cams', 'lrdn': 'lron', 'sinapore': 'singapore', 'elderl': 'elder', 'redstate': 'restate', 'xox': 'box', 'ughb': 'ugh', 'caffeiene': 'caffeine', 'raimondo': 'raymond', 'allium': 'gallium', 'cometous': 'covetous', '35mm': 'comm', 'generalisation': 'generalization', 'icc': 'ice', 'brazela': 'bravely', 'arron': 'arrow', 'peasynts': 'peasants', 'riga': 'rig', 'actimel': 'active', 'wknds': 'winds', 'unprofess': 'profess', 'moaner': 'meaner', 'c4': 'ca', 'unnec': 'under', 'sodde': 'sodden', 'conveni': 'convent', 'teehee': 'teepee', 'pulte': 'pulse', 'gwagon': 'wagon', 'bagle': 'eagle', 'macklin': 'tackling', 'finns': 'finds', 'fsg': 'fog', '2b': 'ab', 'iest': 'best', 'goleta': 'gotta', 'aesha': 'aisha', 'nkomo': 'komo', 'modhop': 'modo', 'quarantunes': 'quarantines', 'pcsos': 'pesos', 'aiot': 'riot', 'pidi': 'didi', 'recipt': 'recipe', 'reqrd': 'heard', 'rubys': 'ruby', 'unf': 'un', 'degrocery': 'grocery', 'canet': "can't", 'berv': 'perv', '14bn': 'ibn', 'rrpre': 'rare', 'distroyed': 'destroyed', 'luverne': 'laverne', 'pbsam': 'beam', 'pereira': 'persia', 'opticien': 'optician', 'prospectnow': 'prospector', 'nesha': 'neha', 'bevin': 'begin', 'speading': 'speaking', 'kenzee': 'knee', 'stocker': 'sticker', 'uslc': 'usc', 'paren': 'parent', 'hullo': 'hello', 'dtr': 'der', 'allistair': 'alistair', 'shopguy': 'shotgun', 'altu': 'alt', 'oorderer': 'ordered', 'negoci': 'negai', 'taronga': 'throng', 'waah': 'wash', 'pasc': 'past', 'dinglings': 'dangling', 'henni': 'henna', 'tingre': 'tingle', 'lndont': 'dont', 'chhaly': 'chalk', 'epping': 'upping', 'surgi': 'surge', 'thund': 'thud', 'khalsa': 'kalsa', 'newlyn': 'newly', 'nj': 'no', 'aecs': 'aces', 'transmittal': 'transmitter', 'ohhh': 'oh', 'repled': 'replied', 'attdgs': 'attics', 'xxl': 'axl', 'nialle': 'niall', 'gaveocado': 'avocado', 'allincle': 'alliance', 'reemployment': 'employment', 'prasanth': 'peasant', 'thori': 'thorn', 'zwe': 'we', 'softee': 'soften', 'sisak': 'isak', 'livlite': 'lilith', '33rd': 'hard', 'mytil': 'until', 'fuhrers': 'fuhrer', 'yesterda': 'yesterday', '1wonderfull': 'wonderful', 'sucessful': 'successful', 'meetups': 'meets', '28f': 'of', 'harpic': 'heroic', 'chell': 'hell', 'w5': 'we', 'straiton': 'station', 'poggio': 'doggie', '$invest': 'invest', 'pst': 'put', 'pathatic': 'pathetic', 'renderfood': 'tenderfoot', 'coscto': 'costco', 'garbin': 'gain', 'r20': 'run', 'disp': 'dish', 'wallahi': 'wallah', 'heresay': 'heresy', 'ballymore': 'baltimore', '25c': 'doc', 'ilearnt': 'learnt', 'begood': 'good', 'strem': 'stream', 'pavlov': 'parlor', 'rd': 'red', 'cncld': 'could', 'simonian': 'simian', 'dufb': 'dumb', 'innovation]': 'innovation', 'deniro': 'dentro', 'engval': 'naval', 'omnicom': 'omnium', 'pharyngeal': 'laryngeal', 'lml': 'lil', 'fetchit': 'fetch', 'maryanne': 'marianne', 'nosecco': 'prosecco', 'speedfest': 'speediest', 'volunteersf': 'volunteers', 'hcp': 'hop', 'welldone': 'wellcome', 'monkas': 'monks', 'chowky': 'choke', 'eazzy': 'jazzy', 'inva': 'ina', 'wheww': 'whew', 'askit': 'skit', 'nimi': 'kimi', 'keeva': 'keep', 'rocourt': 'court', 'investmen': 'investment', 'yeet': 'meet', 'drrss': 'dress', 'txgiving': 'giving', 'receiv': 'receive', 'pkkaa': 'puka', 'choppas': 'chopped', 'sgst': 'just', 'suceptable': 'acceptable', 'free]': 'free', 'repark': 'remark', 'pand': 'and', 'reta': 'beta', 'salgan': 'salman', '9that': 'that', 'whitewine': 'whitening', 'ganyan': 'banyan', '2moro': 'more', '$dkng': 'ding', 'allt': 'all', 'myrah': 'myrrh', 'siad': 'said', 'sda': 'sea', 'bp': 'be', 'ryuji': 'yuji', 'peli': 'deli', 'obamas': 'obama', 'anns': 'ants', 'automator': 'automaton', 'model3': 'model', 'valkart': 'valiant', 'electricals': 'electrical', 'midamar': 'midair', 'retty': 'pretty', 'carouselsv': 'carousels', 'isang': 'sang', '00s': 'is', 'garbo': 'garb', 'hkmom': 'mom', 'iud': "i'd", 'raedle': 'handle', 'culp': 'cup', 'ekugo': 'ego', 'bakit': 'bait', 'whicc': 'which', 'doggers': 'daggers', 'thingys': 'things', 'telsa': 'tessa', 'esop': 'sop', 'shmowers': 'showers', 'governements': 'governments', 'isii': "i'ii", 'avaiable': 'available', 'punkd': 'punk', 'csw': 'cow', 'dropbear': 'dropper', 'progre': 'prove', 'removed]': 'removed', 'nowplaying': 'downplaying', 'talmbout': 'talkabout', 'elg': 'leg', 'fledge': 'pledge', 'dunstable': 'unstable', 'usuage': 'usage', 'jes': 'yes', 'toyo': 'too', 'jalan': 'alan', 'anz': 'and', 'delever': 'deliver', '23minutes': 'minutes', 'peninsular': 'peninsula', 'fellar': 'fella', 'lochore': 'chore', 'tahday': 'today', 'ffl': 'ifl', 'thole': 'those', 'newtowne': 'newton', 'kdka': 'idea', 'ne37': 'need', '$cryo': 'cryo', 'bakkies': 'babies', 'udhari': 'hari', 'mands': 'hands', 'dsp': 'dip', 'ripley': 'riley', 'bunso': 'buns', 'genied': 'denied', 'speir': 'spear', 'exsisting': 'existing', 'inbetween': 'between', 'investements': 'investments', 'surjit': 'suit', '2kg': 'ekg', 'shoveit': 'shove', 'tq': 'to', 'rifpc': 'rip', 'lojistik': 'logistic', '18month': 'month', 'cleese': 'cheese', 'cuute': 'cute', 'fmcgs': 'focus', 'brooklynn': 'brooklyn', '$swrm': 'swim', 'incred': 'inbred', 'tya': 'ya', 'ther': 'the', 'nitin': 'itin', 'wasiu': 'wasin', 'snapily': 'snail', 'nilje': 'nile', 'hopewell': 'howell', 'ashoka': 'ahsoka', 'tos': 'to', 'permenently': 'permanently', 'fischel': 'fischer', 'someof': 'some', 'brosnan': 'bosnian', 'tohealth': 'health', 'pati': 'path', 'dapg': 'dang', 'emabrgo': 'embargo', 'miraa': 'mirai', 'cw6': 'we', 'longlegs': 'longings', 'renele': 'renege', 'vatsa': 'vats', 'ksu': 'su', 'sententious': 'contentious', 'begrateful': 'grateful', 'hpd': 'had', 'frostbit': 'frostbite', 'cbdt': 'but', 'gubmints': 'submits', 'kourt': 'court', 'cbus': 'bus', '6bn': 'ibn', 'greatread': 'retread', 'rapidcity': 'rapidity', 'gama': 'game', 'kehna': 'keen', 'healthfirst': 'healthiest', 'nypl': 'nypd', 'nwt': 'not', 'furkids': 'forbids', 'amys': 'amy', 'miagro': 'micro', 'wattsup': 'watts', 'gastax': 'pasta', 'dalus': 'talus', 'walsall': 'wassail', 'hha': 'ha', 'rocko': 'rock', 'geri': 'germ', 'chr': 'car', 'uave': 'have', 'd9': 'do', 'malbec': 'maybe', 'indriver': 'driver', 'becounted': 'recounted', 'drs': 'dry', 'abangs': 'bangs', 'superuser': 'superduper', 'armloads': 'carloads', 'gramp': 'grasp', 'achey': 'ache', 'fatima': 'fatwa', 'shareable': 'saleable', 'withing': 'within', 'baconless': 'baroness', 'sandton': 'sanction', 'bombcast': 'bobcat', '1uthank': 'thank', 'severna': 'seven', 'tiangong': 'twanging', 'chd': 'chi', '14p': 'up', 'a2': 'a', 'hanji': 'kanji', 'planetm': 'planet', 'aymptomatic': 'symptomatic', 'chuxing': 'chasing', 'lipsense': 'license', 'remt': 'rest', 'booox': 'botox', 'product]': 'product', 'puzio': 'audio', 'golintu': 'glint', 'disbaled': 'disabled', 'hotrod': 'horror', 'bgn': 'bin', 'wentz': 'went', 'sirji': 'siri', 'f4f': 'fof', 'kavitha': 'tabitha', 'ripoff': 'riff', 'sapol': 'spot', 'mrvn': 'man', 'varney': 'barney', 'partipants': 'participants', 'cabrido': 'cardio', 'noem': 'poem', 'nathalie': 'natalie', 'jasina': 'casino', 'leches': 'leeches', 'getfresh': 'refresh', 'dcommerce': 'commerce', 'kavoor': 'kapoor', '2mos': 'mos', 'workmat': 'workman', 'kwtx': 'wax', 'emer': 'ever', 'pcpt': 'put', 'dwarka': 'dark', 'bestquality': 'bestiality', '4l': 'al', 'kkr': 'kir', '2tym': 'tym', 'sitka': 'sita', 'sumos': 'sums', 'polestar': 'polyester', 'imdly': 'imply', 'givling': 'giving', 'izombie': 'zombie', 'as$wipes': 'swipes', 'rideau': 'idea', 'packagers': 'packages', 'lint]': 'lint', 'eugh': 'ugh', 'zucchi': 'zucchini', 'alochol': 'alcohol', 'yiikes': 'yikes', '22items': 'items', 'glute': 'glue', '20apps': 'apps', 'dsw': 'dew', 'everyplate': 'everyplace', 'mngrocer': 'grocer', 'fs1': 'is', 'zaxby': 'baby', 'car]': 'care', 'menken': 'monkey', 'postmated': 'postdated', 'twitchtbd': 'twitched', '$upld': 'build', 'cincup': 'circus', 'uberl': 'uber', 'coupons]': 'coupons', 'katni': 'latin', 'ummno': 'summon', 'secy': 'sexy', 'poni': 'pony', '15x': 'six', 'valle': 'value', 'll65': 'lli', 'flikpart': 'flippant', 'ripeness': 'likeness', 'crotta': 'cotta', '8hr': 'ihr', 'allinone': 'allison', 'nocode': 'nobody', 'reo': 'red', 'bame': 'came', 'leiliv': 'leslie', 'technology]': 'technology', 'yankiel': 'yankees', 'midflight': 'midnight', 'yegsw': 'yes', 'lupi': 'lui', 'medstar': 'estar', 'bbqs': 'boys', 'singleton': 'simpleton', 'bootup': 'boots', 'grocr': 'grocer', 'lametro': 'metro', 'ps17': 'psi', 'irab': 'grab', 'celosia': 'celia', 'reprioritize': 'deprioritize', 'hotmeals': 'hotels', 'leffe': 'leff', 'thunberg': 'thunder', 'chioma': 'coma', 'allende': 'alleged', 'genisys': 'genius', 'kdnl': 'kind', 'wahmmo': 'ammo', 'hackle': 'tackle', 'ebeano': 'beans', 'malou': 'mahou', 'domme': 'dome', 'bcsm': 'best', 'kheti': 'theta', 'airbuds': 'airbags', 'asso': 'also', 'ameri': 'camera', 'eck': 'neck', 'menma': 'mean', 'aviv': 'avid', 'qeues': 'queues', 'amchit': 'chit', 'nrm': 'arm', 'brio': 'bro', 'ialegis': 'alexis', 'buka': 'yuka', 'neilston': 'nelson', 'cce': 'ice', '1ry': 'try', 'australis': 'australia', 'snd': 'and', 'qne': 'one', 'nationw': 'nation', 'penske': 'pense', 'fwt': 'fit', 'mrpoints': 'points', 'bn3': 'and', 'sfbay': 'say', 'hcl': 'hel', 'tyouranny': 'tyranny', 'bentozy': 'bento', 'jawn': 'dawn', 'luton': 'futon', 'gaddy': 'daddy', 'zoomies': 'roomies', 'shoutouts': 'shootouts', 'moloi': 'moon', 'ddi': 'did', 'cronuts': 'counts', 'rashuns': 'rashes', 'shauntal': 'chantal', 'badtiming': 'baptizing', 'gkids': 'kids', 'homophone': 'homophobe', 'compa': 'coma', 'barthe': 'bathe', 'waisay': 'waist', 'savonna': 'savanna', 'jerzy': 'jerry', 'rkf': 'ref', 'zomb': 'bomb', 'liase': 'lease', 'convenients': 'convenient', 'w12': 'we', 'precipating': 'precipitating', 'livelifeir': 'livelier', 'needin': 'needing', 'uplist': 'uplift', 'copeco': 'cope', 'nioh': 'nigh', 'ampnews': 'dampness', 'whyte': 'white', 'exahsted': 'exhausted', 'sadyou': 'sayyou', 'buy4u': 'buy', 'lockdownug': 'lockdown', 'sufficientl': 'sufficient', 'shapp': 'shape', 'sustainabl': 'sustainable', 'furthr': 'further', 'awarness': 'awareness', 'grabbin': 'grabbing', 'sushant': 'susan', 'wednsday': 'wednesday', 'galway': 'alway', 'itonically': 'ironically', 'admistration': 'administration', 'matuu': 'matsu', '23am': 'am', 'elapses': 'lapses', 'coldean': 'clean', '9to9': 'to', 'oxnard': 'onward', 'gritters': 'critters', 'misha': 'mishap', 'powai': 'yowai', 'realll': 'really', 'bearcom': 'bedroom', 'kosovo': 'kokoro', 'superbox': 'superboy', 'ahole': 'whole', 'zmeyer': 'meyer', 'rxr': 'ror', 'wcnsider': 'consider', 'gammon': 'common', 'laboy': 'labor', 'br3': 'bra', 'lspmk': 'spok', 'playable': 'payable', 'beaguru': 'meguru', 'electriccar': 'electrical', 'osyth': 'south', 'bttf': 'but', 'wnd': 'and', 'mcallen': 'callen', 'mobileal': 'mobile', 'caufield': 'afield', 'baskt': 'basket', 'cumgate': 'curate', 'landlorders': "landlord's", 'uncomfy': 'comfy', 'lolll': 'lolly', 'aznews': 'news', 'bulkwhiz': 'bullwhip', 'kanchan': 'cancan', 'charlottenc': 'charlotte', 'listings]': 'listings', 'cms': 'cos', 'knightswood': 'knighthood', 'expecially': 'especially', 'wellp': 'well', 'writeup': 'write', 'muthi': 'mutha', 'coronahv': 'coronary', 'ojodu': 'jody', 'snowin': 'snowing', 'arb': 'are', 'ientology': 'scientology', 'lol]': 'lola', 'heminway': 'hemingway', 'harissa': 'marissa', 'keysa': 'keys', 'volum': 'volume', 'shemd': "she'd", 'medianerd': 'mediated', 'secutrity': 'security', 'ws': 'is', 'banh': 'bank', 'gdgt': 'get', 'dissociating': 'associating', 'fstec': 'step', 'helmond': 'almond', 'dreets': 'greets', 'staycate': 'stardate', 'uphuma': 'puma', 'tchoo': 'choo', 'mitali': 'mitai', '9$': 'i', 'agenda19': 'agenda', 'greendot': 'greenest', 'timesave': 'timeshare', 'ltl': 'lil', 'frillions': 'trillions', 'mrp': 'map', 'sosiyc': 'sonic', 'nervously]': 'nervously', 'eh30': 'eh', 'autom': 'auto', 'ltch': 'itch', 'gyudon': 'gordon', 'hospit': 'hospital', 'syourian': 'syrian', 'keany': 'kenny', 'btls': "it's", 'tweetmy': 'tweety', 'kina': 'kind', 'bil': 'big', 'crps': 'cops', '$intc': 'into', 'fss': 'ass', 'bles': 'blew', 'streamins': 'streaming', 'patheti': 'pathetic', 'pheww': 'phew', 'unity3d': 'united', 'mealcare': 'malware', 'deere': 'deer', 'funatic': 'lunatic', 'sooq': 'soon', 'banka': 'bank', '52mins': 'mins', 'dammam': 'gamma', 'licious': 'vicious', 'stockton': 'stockman', 'prote': 'prove', 'vusi': 'musi', 'seesh': 'sees', 'emoloyees': 'employees', 'freightnews': 'freighters', 'yourky': 'your', 'bhalf': 'half', 'oban': 'ban', 'usbbc': 'usc', 'damai': 'dalai', 'subhead': 'subbed', 'recvd': 'read', 'ddgrocers': 'grocers', '71your': 'your', 'navi': 'navy', 'theophan': 'thespian', 'specialday': 'specially', 'dollarst': 'dollars', 'cutely': 'acutely', 'snowedin': 'snowed', '20miles': 'miles', 'derasist': 'persist', 'mymtl': 'myth', 'gatwick': 'patrick', 'riverbend': 'riverbed', 'stocke': 'stock', 'costablanca': 'casablanca', 'bacari': 'bakari', 'feckin': 'fuckin', 'haile': 'hail', 'recipie': 'recipe', 'happi': 'happy', 'awb': 'awe', 'peinsider': 'insider', 'theoc': 'theo', 'daikin': 'daikon', 'madrasah': 'madras', 'vdownload': 'download', 'isnt': "isn't", 'gmarket': 'market', '18mon': 'demon', 'kt': 'it', 'e11': 'end', 'v60': 'van', 'tyd': 'tad', 'machale': 'machine', 'kamara': 'tamara', 'gripeo': 'gripe', 'noshowed': 'showed', 'weekendv': 'weekend', 'fetcher': 'fetched', 'cleaing': 'cleaning', 'membershipt': 'membership', 'agaisnt': 'against', 'lifehacker': 'linebacker', 'mathem': 'mayhem', 'voom': 'room', 'kahassle': 'hassle', 'infamo': 'infamy', 'starups': 'startups', 'fortunateso': 'fortunate', 'icus': 'ices', 'valyouation': 'valuation', 'pooler': 'cooler', 'piki': 'pike', 'comminism': 'communism', 'lining20': 'lining', 'tomoz': 'tomo', 'correale': 'corral', 'suna': 'sun', 'tren': 'then', 'dramamine': 'dramatize', '31march': 'march', 'condemed': 'condemned', 'carful': 'careful', 'bambam': 'bamba', 'ural': 'oral', 'hus': 'his', 'infirmed': 'informed', 'msfb': 'mob', 'fffuck': 'fuck', 'basham': 'bash', 'gokart': 'goat', 'fabr': 'far', '9th': 'ith', 'occassionally': 'occasionally', 'shitt': 'shit', 'teambm': 'team', 'dvo': 'do', 'literaly': 'literally', 'undeservedly': 'unreservedly', 'changedi': 'changed', 'amitie': 'auntie', 'jeneil': 'pencil', 'chinatownsf': 'chinatown', 'ajeti': 'jet', 'ksa': 'sa', 'burwell': 'burrell', 'sunkr': 'sunk', 'vorvida': 'corrida', 'longhouse': 'doghouse', 'scrounger': 'scrounge', 'fdn': 'fun', 'lombardi': 'bombard', 'mornins': 'morning', 'rustam': 'custom', 'gameil': 'game', 'kalinga': 'kalinda', 'venezia': 'venetian', 'edgey': 'edge', 'dum': 'due', 'yamhill': 'cahill', 'cbc': 'abc', 'baccwoods': 'backwoods', 'hena': 'hen', 'massine': 'massive', 'wetterer': 'wetter', 'golinks': 'links', 'couryes': 'courses', 'lianhua': 'lingua', 'startupsg': 'startups', 'ponteland': 'portland', 'keefe': 'keep', 'bhh': 'bah', 'leie': 'lie', 'rcd': 'red', 'finlit': 'finest', 'herrick': 'derrick', 'smedium': 'medium', 'fals': 'fall', 'fagin': 'fain', 'dougco': 'dough', 'burk': 'burn', 'newstart': 'restart', 'sfsu': 'sasu', 'trilly': 'frilly', 'safe]': 'safe', 'kochi': 'mochi', 'gamify': 'family', 'hisday': 'midday', 'kieboom': 'kaboom', 'dclivin': 'livin', 'ashmole': 'asshole', 'subst': 'subs', 'hamsteren': 'hamster', 'minneola': 'minnesota', 'tadali': 'tidal', 'fackin': 'fuckin', 'talofa': 'aloft', 'jawapan': 'japan', 'ksdk': 'ask', 'diabled': 'disabled', 'nimbys': 'nimbus', 'fcas': 'cas', 'cialis': 'calls', '46m': 'him', 'nfb': 'nab', 'syst': 'cyst', 'beeb': 'been', 'dudee': 'dude', 'mv': 'my', 'briton': 'britons', 'birla': 'girl', 'homecooking': 'homecoming', 'somoeone': 'someone', 'ohyeah': 'yeah', '7tn': 'to', 'orm': 'or', 'osma': 'osama', 'toshiaa': 'yoshida', 'sdm': 'sum', 'eoe': 'eye', 'dmart': 'smart', 'pulosesi': 'pulses', 'seyi': 'semi', 'timw': 'time', 'uko': 'uno', 'arogya': 'aroma', '7c': 'ac', 'avos': 'vos', 'sequela': 'sequel', 'twitterer': 'twitter', 'sjackson': 'jackson', 'solemates': 'tolerates', 'melbert': 'member', 'sfh': 'suh', 'hibernatey': 'hibernate', 'hwd': 'had', 'mvp': 'map', 'disorderer': 'disordered', 'icymi': 'icy', 'abestof': 'bestow', 'fascs': 'fasts', 'talabat': 'taliban', 'lookig': 'looking', 'barbuat': 'barbra', 'thanku': 'thank', 'vegem': 'seem', 'ufc': 'ufo', '12c': 'doc', 'ochoa6': 'ocho', 'twizzler': 'twizzlers', 'wingstop': 'winston', 'aramark': 'remark', 'tata1mg': 'tatami', 'nhlc': 'chic', 'forey': 'forty', 'daycation': 'vacation', 'ingores': 'ignores', 'momtaz': 'mortal', 'obidients': 'obedient', 'thatcham': 'whatcha', 'pittston': 'piston', 'nossa': 'bossa', 'oraying': 'praying', 'goig': 'going', '4c': 'ac', 'on16': 'on', 'scumy': 'scum', 'allee': 'alley', 'planetx': 'planet', 'hurtin': 'hurting', 'blizz': 'blitz', 'tastyi': 'tasty', 'hcmc': 'him', 'frontliner': 'frontline', 'total10': 'totally', 'ryusen': 'risen', 'nomnom': 'pompom', 'immuno': 'immune', 'beltola': 'bella', 'hlth': 'hath', 'spfc': 'spec', '9am': 'am', 'lh': 'oh', 'quizi': 'quiz', 'balmain': 'bargain', 'barbacoa': 'barbara', 'osja': 'sea', '4hrs': 'this', 'triwest': 'tries', 'sundar': 'sunday', 'seward': 'reward', 'nett': 'next', 'uaenas': 'hyenas', 'kant': 'want', 'nums': 'nuts', 'toonie': 'townie', 'vloggin': 'logging', 'peapo': 'peace', 'wolf15': 'wolf', 'wic]': 'wick', 'dawh': 'dawn', 'isdown': 'down', 'unga': 'una', 'survior': 'survivor', 'panne': 'anne', 'lbma': 'lama', 'lissen': 'listen', 'birdeye': 'birdie', 'fenway': 'runway', 'baihu': 'bath', 'your]': 'your', 'sindi': 'sind', 'gotv': 'got', 'buttt': 'butt', 'sfgiants': 'giants', 'mccabe': 'macabre', 'weeken': 'weekend', 'masterca': 'master', 'bindge': 'binge', 'lcsm': 'last', 'instarip': 'instar', 'methheads': 'meatheads', 'integ': 'intel', 'shamey': 'shame', 'obcm': 'once', 'residen': 'resident', 'scheding': 'scheming', 'retail]': 'retail', 'corderially': 'cordially', 'forster': 'foster', 'zena': 'xena', 'oga': 'yoga', 'fotl': 'fool', 'opppsite': 'opposite', 'pimpri': 'pimp', '$gild': 'gild', 'shiona': 'shine', 'tomorrah': 'tomorrow', 'nodelay': 'delay', 'engi': 'eng', 'starsh': 'stars', 'exurban': 'urban', 'phillie': 'phillip', 'extramile': 'extradite', 'glaslyn': 'gladly', 'abuja': 'abuse', 'progra': 'program', 'inititated': 'initiated', 'chod': 'chop', 'fowey': 'foley', 'stpete': 'state', 'iprights': 'uprights', '87yo': 'yo', 'yego': 'ego', 'summer]': 'summer', 'bleeker': 'bleeder', 'liuna': 'luna', 'paulette': 'palette', 'emplo': 'employ', 'adib': 'adi', 'greenapp': 'greenan', 'verin': 'vein', 'existentially': 'existential', 'rly': 'fly', 'kalau': 'kala', 'newsome': 'awesome', 'br': 'be', 'compairing': 'comparing', 'omya': 'oma', 'hsp': 'hop', 'otg': 'ot', 'multiuse': 'multiple', 'wantto': 'wentto', 'husb': 'hush', 'southerntier': 'southerner', 'graf': 'grab', 'teleworks': 'teleports', 'misinform': 'misinformed', 'chron': 'cheon', 'brughh': 'brought', 'cari': 'care', 'momtogs': 'motors', 'cd': 'ad', '82your': 'your', 'fck': 'fuck', 'funct': 'fuck', 'edutainment': 'detainment', 'alzheimers': "alzheimer's", 'anazon': 'amazon', '52nd': 'and', 'almart': 'almost', 'kui': 'qui', 'dbl': 'del', 'turbotax': 'turbot', '6bh': 'be', 'nagogo': 'magog', 'ardell': 'ordeal', 'news19': 'news', 'aust': 'just', '$clcn': 'clan', 'absa': 'abs', 'webgen': 'wegen', 'beria': 'aeria', 'cvid': 'avid', '57th': 'with', 'tnc': 'tac', 'bwah': 'blah', 'quancy': 'quincy', 'mcdees': 'models', 'temiang': 'timing', 'benifit': 'benefit', 'superworms': 'superiors', 'cabpnada': 'canada', 'toxel': 'towel', 'jda': 'da', 'randpaul': 'randall', 'themselv': 'themself', 'swope': 'swore', 'kapil': 'jail', 'osa': 'ofa', 'gimor': 'minor', 'refunneling': 'refueling', 'twenty20': 'twenty', 'brexi': 'break', 'crocker': 'cracker', 'dontcare': "don'tcare", 'pkge': 'page', 'goud': 'good', 'internation': 'international', 'wholesellers': 'wholesalers', 'noths': 'notes', 'akp': 'app', 'caughing': 'laughing', 'prescirption': 'prescription', 'stallyns': 'stalling', 'walmart': 'palmar', 'imoo': 'moo', 'bgky': 'by', 'haysi': 'hays', 'photoaday': 'photoplay', 'gabaldon': 'abandon', 'fuquay': 'quay', '\\m': 'am', 'totino': 'toting', 'hellmann': 'bellman', 'srvc': 'save', 'tatau': 'tata', 'inki': 'ink', 'reddi': 'redid', 'nko': 'no', 'stresst': 'stress', 'btm': 'bum', 'olathe': 'loathe', 'hf': 'of', 'beindoors': 'indoors', 'ghe': 'the', 'unpacks': 'unpack', 'ecoecting': 'expecting', 'gratitudei': 'gratitude', 'calpol': 'call', 'allergenic': 'allergic', 'shoprites': 'shorties', 'ckart': 'cart', 'gofigure': 'figure', 'haruhi': 'harsh', '7kb': 'job', 'smn': 'son', 'parter': 'partner', 'ecd': 'end', 'delic': 'relic', 'rjm': 'rum', 'googlers': 'googles', 'teamla': 'team', 'aamir': 'amir', 'opene': 'open', 'apclips': 'applies', 'ourfor': 'outfor', 'maramba': 'marimba', 'randals': 'randall', 'shielder': 'shielded', 'nocrowds': 'crowds', 'wdm': 'wom', 'outchea': 'butcher', 'instaca': 'instant', 'khqty': 'kitty', '48th': 'with', 'grocs': 'grows', 'orderination': 'ordination', 'tempera': 'temper', 'matek': 'mate', 'n]': 'no', 'cmh': 'cme', 'findi': 'find', 'caruso': 'cause', 'rawa': 'raw', 'tib': 'tie', 'zaga': 'saga', 'iyong': 'iong', 'arthit': 'artist', 'swiggi': 'swing', 'holderness': 'wilderness', 'dreamin': 'dreaming', 'forb': 'for', 'frenemy': 'enemy', 'dorking': 'working', 'sofinews': 'softness', 'se21': 'see', 'eman': 'man', 'junnie': 'junkie', 'primals': 'primal', '58am': 'am', 'whmis': 'whims', 'reshipped': 'shipped', 'aparantly': 'apparently', 'finport': 'airport', 'alreasy': 'already', 'uyghur': 'uyour', 'suppli': 'supply', 'segmentation': 'pigmentation', '63your': 'your', 'dsv': 'dev', 'smme': 'some', 'huss': 'fuss', 'ppy': 'pay', 'intcomex': 'income', 'wdn': 'win', 'checkouta': 'checkout', 'dickson': 'dicks', 'waitinf': 'waiting', 'hl': 'he', '$selb': 'sell', 'nuther': 'nether', 'ableds': 'able', 'danyo': 'dango', 'bbholly': 'holly', 'errods': 'errors', 'ucpd': 'up', '99loans': 'loans', 'yuo': 'you', 'saveons': 'saves', 'oreders': 'orders', 'serous': 'serious', 'tually': 'tally', 'qwyk': 'wok', 'anipals': 'animals', 'pwople': 'people', 'gati': 'gate', 'bhatia': 'bhai', 'apco': 'paco', 'farro': 'faro', 'mrs': 'mars', 'straiten': 'straighten', 'minimart': 'minimal', 'nomu': 'nom', 'leople': 'people', 'honeynut': 'honeypot', 'cwmbran': 'cambrian', 'freekeh': 'freeze', 'streetwear': 'streetcar', 'redumped': 'dumped', 'liye': 'like', 'auvsi': 'aussi', 'snc': 'sync', 'jun21': 'junk', 'progressu': 'progress', 'rha': 'ha', 'manditory': 'mandatory', 'revy': 'rely', 'foram': 'form', 'karachi': 'mariachi', 'phle': 'pile', 'fg': 'fu', 'keelang': 'keeling', 'doomers': 'boomers', 'parshad': 'arshad', 'gonola': 'gondola', 'dws': 'des', 'evrywhre': 'everywhere', 'win3free': 'winifred', 'propoganda': 'propaganda', 'phdlife': 'midlife', 'octobero': 'october', 'northwes': 'northwest', 'strateg': 'strategy', 'angeleno': 'angeles', 'wyt': 'wet', 'applaudso': 'applauds', 'auro': 'auto', 'uptrend': 'trend', 'prime]': 'prime', 'bpm': 'bum', 'postals': 'postal', 'toonces': 'ounces', 'gorka': 'gonna', 'mairi': 'uairi', 'tomisha': 'trisha', 'fodmap': 'formal', 'ronno': 'ronny', 'powtoon': 'pontoon', 'monodrama': 'melodrama', 'pickupor': 'pickup', 'zypp': 'type', 'omv': 'om', 'accelerative': 'accelerate', 'groser': 'grocer', 'supremists': 'supremacists', 'entrada': 'entrap', 'itlr': 'itl', 'kbc': 'abc', 'msu': 'su', 'bandhan': 'banyan', 'filipina': 'filipino', 'bawiq': 'basic', 'uttan': 'than', 'unbrushed': 'brushed', 'slayy': 'slay', 'kolmol': 'kool', 'oneanother': 'ofanother', 'mngt': 'not', 'bbsr': 'best', 'exascerbate': 'exacerbate', 'bossier': 'dossier', 'tedros': 'tears', 'kgn': 'kin', 'delray': 'delay', 'verizon': 'version', 'chahiye': 'chalice', 'tmz': 'tme', 'awesomehey': 'awesomely', 'rahim': 'ahim', 'safty': 'safety', 'budino': 'buying', 'icanteven': 'canteen', 'monahans': 'monicans', 'mannn': 'manny', 'reductionist': 'reductions', 'lucie': 'lucid', 'nltwitter': 'twitter', 'blain': 'brain', 'aarddr': 'harder', 'improvemen': 'improvement', 'dubiously': 'dubious', 'ohwell': 'howell', 'ronge': 'range', 'saalan': 'salad', 'shareit': 'share', 'bronner': 'bronzer', 'foowid': 'food', 'devos': 'demos', 'redelivery': 'delivery', 'chix': 'chip', 'developme': 'developed', 'nobod': 'nobody', 'plc': 'ply', 'crompton': 'compton', 'otts': 'outs', 'gmai': 'mai', 'examing': 'examine', 'campaing': 'campaign', 'riri': 'kiri', 'collegue': 'college', 'dailycr': 'daily', 'figses': 'files', 'vigalent': 'violent', 'murderd': 'murder', 'bryn': 'bran', 'nzpol': "t'pol", 'googleads': 'googled', 'stylin': 'styling', 'lytle': 'lyle', 'millen': 'miller', 'brot': 'boot', 'bhag': 'bag', 'foght': 'fight', 'ug4ph': 'ugh', 'whs': 'was', 'becel': 'betel', 'nnoo': 'noo', 'ps0': 'psi', 'texmex': 'texted', 'vread': 'read', 'bgnng': 'being', 'pesima': 'prima', 'plonka': 'plonk', 'sharkey': 'share', 'tobs': 'jobs', 'shebeen': 'sheen', 'tbecause': 'because', 'tocure': 'secure', 'rajud': 'raju', 'kalabash': 'calabash', 'fmr': 'for', 'magicpin': 'magician', 'swont': 'wont', 'flipkar': 'flicker', 'tsi': 'si', 'nesf': 'nest', 'pegden': 'pegged', 'bor': 'for', 'gimli': 'girl', 'effers': 'offers', 'deepika': 'deepak', 'foodjustice': 'forjustice', 'fowards': 'towards', 'crustless': 'countless', 'pcmag': 'mag', 'rs20': 'rse', 'studf': 'stuff', 'stappi': 'stamp', 'waffers': 'wafers', 'stt': 'set', 'occam': 'ocean', 'situs': 'sits', 'occassional': 'occasional', 'icin': 'icing', 'motorcyle': 'motorcycle', 'colley': 'volley', 'automatize': 'automatic', 'citylife': 'citywide', 'hawick': 'hack', 'gdso': 'do', 'phonebank': 'phonebook', 'shiptinc': 'shipping', 'kc': 'ke', 'ardc': 'arc', 'colocar': 'color', 'bothata': 'thata', 'enginee': 'engine', 'nwabj': 'swab', 'andfood': 'android', 'suchan': 'sucha', 'bigbask': 'bigass', 'twosday': 'today', 'finaggle': 'finagle', 'tife': 'time', 'hoomins': 'hooking', 'ahaha': 'haha', 'oferzone': 'overdone', 'puregold': 'presold', 'eastbay': 'astray', 'quickwit': 'quickest', 'e17': 'end', 'nething': 'nothing', 'queation': 'question', 'midddleman': 'middleman', 'colectivo': 'collective', 'klevio': 'kevin', 'kilburn': 'wilbur', 'brasstown': 'crosstown', 'kroeber': 'robber', 'terr': 'tear', 'earlie': 'earlier', 'eaf': 'eat', 'arbb': 'arab', 'poppin': 'popping', 'joinery': 'joiner', 'lawerence': 'lawrence', 'outstock': 'buttock', 'tekit': 'test', 'kevin07': 'kevin', 'hamsa': 'hams', 'elate': 'late', 'momming': 'coming', 'sndt': 'snot', 'djungel': 'jungle', 'sarkar': 'sardar', 'roomate': 'roommate', 'goodyear': 'gooder', '$store': 'store', 'joblin': 'goblin', 'myopac': 'myopic', 'aap': 'map', 'docsolution': 'dissolution', 'hospitali': 'hospital', 'getkind': 'getting', 'kunle': 'kune', 'fark]': 'far', 'gandy': 'candy', 'bldng': 'bling', 'resealable': 'repeatable', 'frozens': 'frozen', 'araam': 'aram', 'delson': 'nelson', 'pinon': 'pinot', 'cref': 'crew', 'nosso': 'osso', 'shrunks': 'shrunk', '23miles': 'miles', 'leong': 'long', 'stomach]': 'stomach', 'paylater': 'palate', 'sopa': 'soap', 'ttd': 'tad', 'myajan': 'mayan', 'survery': 'surgery', 'evcst': 'event', 'iterminator': 'terminator', 'flusterating': 'frustrating', 'batc': 'bath', 'rebrands': 'rebrand', 'gobind': 'going', 'brith': 'birth', 'damo': 'damn', 'arkc': 'arc', 'sunnys': 'sunny', 'jul': 'jug', 'prbly': 'truly', 'lazydays': 'paydays', 'questio': 'question', 'christmasy': 'christmas', 'dekalb': 'deal', 'scavino': 'saving', 'frais': 'frail', 'crediblty': 'credibly', '1t': 'it', 'bhiwani': 'bhavani', 'diorg': 'doing', 'newage': 'sewage', 'mst3k': 'mistak', 'lewin': 'lewis', 'nicoleta': 'nicole', 'decra': 'debra', 'abdi': 'andi', 'nonuse': 'house', 'alula': 'alla', 'yende': 'vende', 'alianza': 'alana', 'giga': 'gig', 'harzer': 'harder', 'jio': 'jim', 'twonks': 'twinks', 'einhorn': 'tinhorn', 'airswift': 'airlift', 'joong': 'joon', 'noda': 'soda', 'sacha': 'sasha', 'verdi': 'verde', 'ocda': 'coda', 'itc4': 'itch', 'crist': 'wrist', 'drayman': 'drama', 'streamin': 'streaming', 'jonece': 'once', 'xtian': 'tian', 'doesnt': "doesn't", 'luanne': 'leanne', 'ycot': 'cot', 'egusi': 'eggs', 'sykt': 'set', 'hebden': 'headed', 'milspouse': 'milhouse', 'magogo': 'magog', 'pingwest': 'ingest', 'oopsy': 'oops', 'techne': 'techno', 'gl51': 'glad', 'jumi': 'jump', 'fitspo': 'fits', 'moritz': 'merit', 'bliming': 'blaming', 'thankfulwv': 'thankful', 'ookkkk': 'ookiku', 'jati': 'jai', 'villagefo': 'village', 'inste': 'inte', 'conditon': 'condition', 'ibsaid': 'said', 'drivenmd': 'driven', 'msgd': 'used', 'dainah': 'dinah', 'neccessities': 'necessities', 'cuttack': 'cutback', 'achiev': 'achieve', 'krib': 'rib', 'bouzie': 'bougie', 'pusu': 'push', 'semcents': 'serpents', 'hogu': 'hog', 'lasership': 'leadership', 'stulo': 'still', 'skydome': 'skydive', 'fishsa': 'fish', 'notgood': 'osgood', 'neverr': 'never', '50ph': 'syph', 'digiorno': 'giorno', 'thanh': 'than', 'w21': 'we', 'rli': 'li', 'celebratin': 'celebrating', 'cheeni': 'cheese', 'hellooo': 'hello', 'perrier': 'terrier', 'dubois': 'debris', 'wbhm': 'who', 'charan': 'charge', 'crich': 'rich', 'efc': 'etc', 'dayton': 'dalton', 'bigtits': 'digits', '3min': 'min', 'donlemon': 'doraemon', 'darktrace': 'backtrace', 'permissable': 'permissible', '$jd': 'and', 'moiday': 'monday', 'minot': 'minor', '88c': 'doc', 'kunafa': 'kungfu', 'bestmsp': 'bestest', 'beppu': 'beeps', 'tamaki': 'tanuki', 'rosauers': 'rescuers', 'shwantz': 'shanty', 'anywhwere': 'anywhere', 'hoda': 'soda', 'davie': 'dave', 'awear': 'wear', 'weeknd': 'weekend', 'hardass': 'harass', 'mcadoo': 'meadow', 'whdh': 'what', 'equest': 'request', '78yo': 'yo', '3q': 'iq', 'romania': 'romanian', 'santi': 'santa', 'boujiee': 'bougie', 'tywalk': 'talk', 'lifeison': 'liaison', 'nacy': 'navy', 'larchmont': 'parchment', 'strtups': 'startups', 'sideeye': 'sidney', 'expressnews': 'expresses', '40ps': 'cops', 'awrit': 'await', 'unjabbed': 'nabbed', 'wmart': 'smart', 'baxk': 'back', 'compuses': 'campuses', 'zon': 'on', 'potg': 'pot', 'angmoh': 'angio', 'ahern': 'thern', 'foodart': 'fondant', '$bj': 'be', 'hypercar': 'supercar', 'gibberish]': 'gibberish', 'partener': 'partner', 'teleworking': 'teleporting', 'skeert': 'skeet', 'tpr': 'tar', 'mendon': 'tendon', 'wcco': 'who', 'mpb': 'mob', '28days': 'days', 'frustratd': 'frustrated', 'flushot': 'flush', 'shlub': 'shrub', 'shenzi': "she'ii", 'angkat': 'angst', 'estore': 'store', 'yo71': 'you', 'gr': 'go', 'cps': 'cops', 'jcb': 'job', 'costc': 'cost', '3wks': 'was', 'coronanz': 'coronary', 'visacard': 'discard', 'horsehead': 'forehead', 'domicilio': 'domicile', 'ccac': 'can', 'scenee': 'scene', 'moser': 'loser', 'debbies': 'debbie', 'sylantro': 'cilantro', 'smugi': 'smug', 'nasper': 'jasper', 'cowin': 'coin', 'uninstalls': 'installs', 'supermart': 'supermarket', 'ihub': 'hub', 'wellsies': 'wellies', 'thoreau': 'threat', 'sweatier': 'sweater', '47th': 'with', 'ncnw': 'now', 'mysuru': 'suru', 'crinsights': 'insights', 'october25': 'october', 'justegg': 'juste', 'inves': 'invest', 'jarritos': 'burritos', '1inch': 'inch', 'arlyn': 'early', 'schedulesd': 'scheduled', 'applaudbe': 'applause', 'serives': 'series', '4thedge': 'hedge', 'krwana': 'katana', 'hader': 'harder', 'centricity': 'eccentricity', 'toups': 'tops', 'anywh': 'anyth', 'streetside': 'streetwise', 'mintel': 'intel', 'wurd': 'word', 'aslo': 'also', 'psf': 'psi', 'deliverco': 'deliver', 'virous': 'virus', 'worklife': 'forklift', 'corona21': 'coronary', 'naturalle': 'naturally', 'sharpo': 'sharp', 'phx': 'pox', 'cambios': 'cabins', 'waititu': 'waiting', 'gq': 'go', 'extremly': 'extremely', '95us': 'us', 'clocery': 'closer', 'cleveron': 'clever', 'floury': 'flour', 'vidalon': 'vindaloo', 'dhandha': 'chandra', 'oprl': 'opal', 'scowls': 'scowl', 'bakewell': 'farewell', 'juul': 'just', 'suncart': 'snart', 'dda': 'dad', 'poitier': 'poitiers', 'vickies': 'vickie', 'suckage': 'package', 'sahlen': 'stolen', 'tohospitals': 'hospitals', 'emailsdo': 'emails', 'prov': 'prove', 'sworders': 'orders', 'yeat': 'eat', 'behi': 'bei', 'gobuddy': 'buddy', 'federate': 'federated', 'kimco': 'kick', 'buzzworders': 'buzzwords', 'mortgagor': 'mortgage', 'jpi': 'pi', 'lovewe': 'love', 'bcse': 'base', 'delivery2': 'delivery', 'fufu': 'fugu', 'pinpay': 'pinky', 'kitkat': 'kita', 'zapata': 'papaya', '2q': 'iq', 'twitter10': 'twitter', 'uhall': 'shall', 'forthewin': 'fortheir', 'bakeshop': 'baseship', 'inthose': 'those', 'feasibity': 'feasibility', 'canggu': 'canyou', 'caffiene': 'caffeine', 'meester': 'maester', 'pantie': 'panties', 'faceaging': 'packaging', 'nsra': 'nsa', 'showert': 'shower', 'scandinavia': 'scandinavian', 'thsy': 'they', 'contagous': 'contagious', 'cougs': 'cough', 'itms': "it's", 'metallica': 'metallic', 'rajma': 'raja', 'shelfers': 'shelters', 'honestbe': 'honest', 'unlv': 'onlv', 'sippora': 'support', 'bldg': 'blog', 'rs5': 'rse', 'lyk': 'lik', 'mcelroy': 'mcenroe', 'jilebi': 'jalebis', '30yours': 'yours', 'arcy': 'army', 'hwusc': 'house', 'hr5': 'he', 'mashhad': 'mashed', 'tribeca': 'tribe', 'aleksa': 'ales', 'ratrod': 'ramrod', 'hrt': 'hit', 'habra': 'habla', 'ttrpg': 'tarp', 'tactility': 'activity', 'neonopolis': 'necropolis', 'oomfs': 'comes', 'netflux': 'netflix', 'guelph': 'gulp', 'drg': 'dog', 'vanpoli': 'cannoli', '40ish': 'wish', 'dnc': 'dna', 'trumpis': 'trumps', 'apics': 'pics', '9news': 'news', 'reason2': 'reason', 'breake': 'break', 'fox10': 'fox', 'jbco': 'job', 'apurple': 'purple', 'mingus': 'minus', 'unessential': 'essential', 'blarg': 'blare', 'heylana': 'helena', 'grabcar': 'grammar', 'blm': 'bum', 'semmes': 'sommes', 'askwali': 'alkali', 'eatfresh': 'afresh', 'oorjit': 'orbit', 'sd': 'so', 'kms': 'ams', 'ofcos': 'focus', 'fantas': 'fantasy', 'ps50': 'psi', 'supportri': 'support', 'tshwane': 'shane', 'r22': 'run', 'nycers': 'nicer', 'paduan': 'madman', 'suas': 'seas', 'pinaka': 'pinata', 'cation': 'action', 'tuiniane': 'quinine', 'fml': 'fol', '$wrpt': 'swept', 'incog': 'into', 'bgmea': 'mea', 'aca': 'act', 'wmc5': 'wack', '4nothing': 'nothing', 'makr': 'make', 'sjogren': 'jorgen', 'amt': 'at', 'cpt': 'cut', 'appaulled': 'appalled', 'thesalt': 'thesame', 'iphonega': 'iphone', 'whateve': 'whatever', 'instea': 'instead', 'monisha': 'monica', 'ridiculour': 'ridiculous', 'simion': 'simon', 'vapes': 'tapes', 'whatevah': 'whatever', 'ebon': 'bon', 'nuggie': 'auggie', 'icemaker': 'pacemaker', 'nagashi': 'sagashi', 'ingridient': 'ingredient', 'inam': 'ina', 'taeil': 'tail', 'cvv': 'civ', 'imtiazes': 'imitates', 'macbook': 'facebook', 'jesl': 'jest', 'cp24': 'cpu', 'buywine': 'buying', 'kaasar': 'caesar', 'karad': 'kara', 'hullabo': 'lullaby', 'saylor': 'sailor', 'udemy': 'enemy', 'myakka': 'makea', 'fi3sta': 'fiesta', 'surch': 'such', 'morrisru': 'morris', 'sorola': 'stroll', 'fastenal': 'fasten', '4kgs': 'legs', 'taking10': 'taking', 'moneysaver': 'moneymaker', 'deckup': 'backup', 'sambhar': 'sambar', 'edsel': 'easel', 'collapsable': 'collapsible', 'getitnow': 'getinto', 'personalrisk': 'personalise', 'ledo': 'led', 'vasc': 'vast', 'ad53': 'add', 'kho': 'who', 'morderor': 'murderer', 'sacres': 'sacred', 'seriouness': 'seriousness', 'anywhey': 'anywhere', '15am': 'am', 'dezeen': 'dozen', 'katto': 'kitto', 'frankey': 'frankly', 'decongestion': 'congestion', 'myeloma': 'myeloid', 'accessor': 'accessory', 'eq': 'e', 'ruaka': 'aka', 'resturaunts': 'restaurants', 'zm': 'am', 'dammned': 'damned', '2dw': 'do', 'wsvn': 'when', 'baz': 'bad', 'gardena': 'garden', 'sobrang': 'sobbing', 'bl3': 'bla', 'vathals': 'vitals', 'arran': 'array', 'iyour': 'your', 'sshole': 'asshole', 'gazak': 'gaza', 'tradeoffs': 'tradeoff', 'fareye': 'free', 'cwcf': 'cock', '6yours': 'yours', 'careem': 'career', 'mlas': 'las', 'wenham': 'webcam', 'growliest': 'lowliest', 'rlb': 'rob', 'uthi': 'thi', 'furbys': 'fury', 'buisness': 'business', 'opporutunity': 'opportunity', 'mz': 'my', 'booo': 'book', 'barh': 'bar', 'retailwars': 'retailers', 'grillin': 'grilling', 'matthua': 'matthew', 'artnered': 'partnered', 'riden': 'ride', 'archon': 'anchor', 'ferraro': 'ferrari', 'webvans': 'evans', 'teying': 'trying', 'deprecation': 'depreciation', 'chorley': 'charley', 'dwane': 'diane', 'blatt': 'blast', '2miles': 'miles', 'snr': 'sir', 'bota': 'both', 'commenters': 'commenter', 'utilli': 'still', 'cerelac': 'cereal', 'caren': 'care', 'nouveaux': 'nouveau', 'luzerne': 'laverne', 'mityo': 'mito', 'errthang': 'erything', 'wwjobs': 'jobs', 'immodium': 'ammonium', 'pcid': 'paid', 'famlies': 'families', '$ulta': 'outta', '2i': 'i', 'dietland': 'wetland', 'rsg': 'rig', 'prority': 'priority', 'wmc': 'we', 'gik': 'gig', '12yo': 'yo', 'competit': 'compete', 'folksl': 'folks', 'alejandra': 'alexandra', 'doms': 'does', 'drygoods': 'dragoons', 'voisin': 'visit', 'bartlett': 'barrett', 'accred': 'accrued', 'experiation': 'expedition', 'cojita': 'costa', 'monoline': 'online', 'pizzarias': 'pizzeria', 'ezguy': 'guy', 'shige': 'shine', 'goodform': 'goodfor', 'tonneau': 'tonnes', 'scn': 'son', 'speedfeed': 'speeded', 'kindemic': 'kinetic', 'rmc': 'rec', 'paindad': 'painted', 'viscerally': 'visceral', 'bilo': 'bill', 'assam': 'assay', 'swadesi': 'shades', 'krissy': 'prissy', 'aand': 'and', 'tcobom': 'toom', 'pleps': 'pleas', 'liveline': 'lifeline', 'distractor': 'distract', 'baack': 'back', 'nomanagement': 'management', 'resched': 'reached', 'feedin': 'feeding', 'iherb': 'herb', 'amazont': 'amazons', 'cornersh': 'corners', 'gerba': 'yerba', 'kortes': 'cortes', 'raok': 'rank', 'magento': 'magenta', 'gnv': 'giv', 'grondine': 'grinding', 'idiot]': 'idiot', 'lyndie': 'indie', 'unive': 'unite', 'nowher': 'nowhere', 'nobroker': 'broker', 'criptax': 'cristal', 'biotch': 'bitch', 'fuckign': 'fucking', 'manag': 'manage', 'cc': 'ca', 'engross': 'gross', 'okayy': 'okay', 'rgca': 'rock', 'b66': 'be', 'phase1': 'phase', 'ward4': 'ward', 'aliviate': 'alleviate', 'viber': 'vibe', 'aaya': 'maya', 'jawed': 'jared', 'spryker': 'stryker', 'rimo': 'limo', '9effa': 'jeff', 'commitmenets': 'commitments', 'outnorf': 'outof', 'steiner': 'sterner', 'creepiness': 'creepiest', 'thiefy': 'thief', 'langsung': 'lansing', 'hipp': 'hip', 'shopoing': 'shopping', 'zedify': 'edify', 'giblet': 'goblet', 'rtnd': 'rind', 'newera': 'newer', 'vloggers': 'bloggers', 'rfeif': 'reef', 'fbnw': 'find', 'adapat': 'adapt', 'taxify': 'toxify', '16m': 'him', 'macleod': 'mailed', 'tamar': 'tamer', 'subdash': 'sudah', 'whsts': 'whats', 'mmhmm': 'ummm', 'replyin': 'replying', 'shipget': 'shipment', 'amejn': 'amen', 'paree': 'saree', 'glympse': 'glimpse', 'merced': 'merged', 'panelist': 'panelists', 'samsung': 'sassing', 'lupita': 'capita', 'grrh': 'girl', 'lz': 'la', 'sw14': 'swim', 'bethge': 'bethe', 'shitness': 'shitless', 'alva': 'lava', 'fkkn': 'fun', 'elide': 'slide', 'limang': 'liang', 'mismo': 'miso', 'memorial20': 'memorial', 'nolte': 'note', 'ftav': 'stav', 'nzpost': 'post', 'imane': 'image', 'within1': 'within', 'shipted': 'shipped', 'ehatsapp': 'whatsapp', 'espy': 'spy', 'iknowwhy': 'knowwhy', 'cashingin': 'cashing', 'abc7': 'abc', 'ssd': 'sad', 'debora': 'debra', 'karti': 'kari', 'qualit': 'quality', 'zouch': 'touch', '$boxd': 'box', 'ayaw': 'yaw', 'wsupermarket': 'supermarket', 'karry': 'marry', 'sejour': 'sour', 'ranjit': 'rabbit', 'lgv': 'liv', 'hardiman': 'hardman', 'gaab': 'grab', 'ganaches': 'ganache', 'ciie': 'cite', 'ryman': 'ryan', 'lanark': 'lunar', 'monada': 'monday', 'afang': 'fang', 'ribbboned': 'ribboned', 'byjus': 'bus', 'priceline': 'pipeline', 'anone': 'anyone', '94th': 'with', 'cfs': 'cos', 'fabaliss': 'fatalism', 'ebike': 'bike', 'ponde': 'pond', 'nospoilers': 'spoilers', 'toktok': 'took', 'wwjd': 'word', 'volu': 'vou', 'okwx': 'own', 'jalariya': 'malaria', 'harassement': 'harassment', 'raisinets': 'raisins', 'lobels': 'labels', 'jeon': 'jean', 'folklores': 'folklore', 'terus': 'terms', 'beasley': 'measly', 'live]': 'live', 'crawley': 'crawled', 'randos': 'random', 'phytochemical': 'photochemical', 'theron': 'heron', 'hants': 'wants', 'disenfranchise': 'disenfranchised', 'hono': 'honor', 'aridea': 'idea', 'nona': 'none', 'gtto': 'gotto', 'monang': 'monani', '10minute': 'minute', 'simi': 'kimi', '77th': 'with', '$lyft': 'left', 'furrball': 'furball', 'sfbj': 'sub', 'biomed': 'boomed', 'bringel': 'bringer', 'apar': 'apart', 'rhoderick': 'roderick', 'microcode': 'microscope', 'coverag': 'coverage', 'matchos': 'matches', 'dayum': 'day', 'damnmn': 'damn', 'leenane': 'teenage', '17th': 'with', 'pahwa': 'papa', 'wdw': 'wow', 'endrant': 'entrant', 'shipt2u': 'shiatsu', 'pass]': 'pass', 'hearons': 'herons', 'zvenyu': 'venu', 'reqmt': 'rest', 'suiters': 'suitors', 'tescos': 'tests', 'dunjee': 'bungee', 'passaic': 'passage', 'ncgs': 'nags', 'uddhav': 'buddha', 'dpa': 'dna', 'unders': 'under', 'giesen': 'given', 'servicegh': 'service', 'galus': 'gals', 'laif': 'laid', 'rais': 'raise', 'naahr': 'naar', 'arjay': 'array', 'kerner': 'kernel', 'sitaram': 'sitar', 'gofresh': 'fresh', 'morristown': 'morrison', 'nug': 'bug', 'weis': 'wets', '1w': 'ow', 'shopzed': 'shopped', 'suleman': 'salesman', 'hockwell': 'howell', '$vuzi': 'uzi', 'competi': 'compete', 's$': 'so', 'loree': 'lore', '31c': 'doc', 'malala': 'masala', 'momuu': 'mom', 'moussa': 'mousse', 'loveme': 'love', 'phnom': 'phenom', 'inc4': 'inch', 'vfor': 'for', 'daim': 'aim', 'neke': 'nuke', 'chellam': 'cellar', 'elance': 'glance', 'hoingcto': 'goingto', 'moxley': 'motley', 'ptp': 'pop', 'schoolfee': 'schooled', 'grocery]': 'grocery', 'grinstead': 'instead', 'jeetar': 'nectar', 'aper': 'paper', 'appwhile': 'awhile', 'vacationer': 'vacationed', 'surprizing': 'surprising', 'newhome': 'newcomer', 'itread': 'tread', 'krok': 'kook', 'deagon': 'dragon', 'skynews': 'shyness', 'ttc': 'tac', 'akena': 'arena', 'regul': 'regal', 'licht': 'light', 'metropolish': 'metropolis', 'relabeled': 'labeled', 'ampang': 'amping', 'fmg4u': 'fugu', 'euref': 'sure', 'fintrovert': 'introvert', 'kettering': 'lettering', 'coronanl': 'coronal', 'devpro': 'depo', 'cripps': 'crisps', 'ayours': 'yours', 'biosystems': 'ecosystems', 'hellsites': 'hillsides', 'nz': 'no', '5service': 'service', 'chiswell': 'chisel', 'alfonsi': 'alfonso', 'sapro': 'spare', 'akil': 'akin', 'arepa': 'area', 'whse': 'whose', '88cents': 'cents', 'kaleb': 'kale', 'dilligently': 'diligently', 'deliver4u': 'deliver', 'nowthis': 'knowthis', 'hixson': 'bison', 'dropoff': 'dropout', 'indira': 'india', 'scaffies': 'scabies', 'bs3': 'is', 'unperturbed': 'perturbed', 'manuf': 'manu', 'hyd': 'had', 'faisal': 'final', 'denison': 'venison', 'experation': 'expiration', 'kunwari': 'unwary', 'gatorland': 'garland', 'unausa': 'unjust', 'hdi': 'hi', 'clowd': 'crowd', 'chefvan': 'cheval', '7news': 'news', 'charkop': 'chariot', 'ebe': 'be', 'powe': 'power', 'ahh': 'ah', 'khud': 'thud', 'rankin': 'ranking', 'northvan': 'northern', 'jiodown': 'godown', '10news': 'news', 'nde': 'de', 'xiaozhan': 'xiaoyan', 'mcmuffin': 'muffin', 'reester': 'reenter', 'composters': 'composers', 'sweetinn': 'sweeting', 'floo': 'floor', 'lastmile': 'lasttime', 'cs9': 'csi', 'joyourd': 'joyous', 'quess': 'guess', 'initiativ': 'initiative', 'uumc': 'jump', 'freh': 'free', 'includi': 'include', 'deliveries]': 'deliveries', 'jacinda': 'lucinda', '10pd': 'nypd', 'kxan': 'khan', 'gurantee': 'guarantee', 'idps': 'dips', '16k': 'ask', 'syouringe': 'syringe', 'heavy]': 'heavy', 'zambia': 'zambian', 'occas': 'orcas', 'gaah': 'gash', 'iranu': 'iran', 'amagi': 'magi', 'vt': 'it', 'hii': 'his', '88your': 'your', 'newsfeed': 'newsreel', '6abc': 'abc', 'rivian': 'vivian', 'tiime': 'time', 'callista': 'cellist', 'elli': 'ella', 'earaches': 'earache', 'bidston': 'bison', 'isliye': 'slide', 'groggily': 'groggy', 'tomorr': 'tomorrow', 'bjs': 'bus', 'sifton': 'soften', 'corrin': 'corrine', 'saratoga': 'sarasota', 'rosanna': 'hosanna', 'kiranas': 'piranhas', 'insha': 'inch', 'amtrak': 'astray', 'hsb': 'hub', 'whyq': 'why', 'rahmat': 'hazmat', 'dentsu': 'dents', 'lizzo': 'lizzy', '75min': 'min', 'yeni': 'yen', 'recov': 'recon', 'pj': 'pa', 'destin': 'destiny', 'erryday': 'everyday', 'mindjust': 'andjust', 'zepeto': 'hereto', 'pravin': 'train', 'odsp': 'gods', 'wiltshire': 'wilshire', '48cr': 'secr', 'yaoi': 'yao', 'boynt': 'point', 'sopya': 'soya', 'tiredlessly': 'tirelessly', 'monthend': 'contend', 'matzoh': 'matzo', 'boerum': 'serum', 'atx': 'at', 'hawai': 'hawaii', 'happyas': "happy's", 'wmtw': 'with', 'wtf1': 'wife', 'fcet': 'feet', 'bex': 'be', 'perfectionism': 'perfectionist', 'l3': 'la', 'brolly': 'rolly', '2qts': 'its', 'messa': 'mess', 'cato': 'cat', 'woodard': 'woodward', 'bhive': 'hive', 'zeew': 'zee', 'westland': 'wetland', 'bfl': 'ifl', 'justhinking': 'unthinking', 'appose': 'oppose', '6d': 'ad', 'brillia': 'brillig', 'eeeii': "we'ii", 'ususual': 'unusual', 'tsf2f': 'tiff', 'ocad': 'cad', 'rnaboos': 'taboos', 'hogmanay': 'howmany', 'delayd': 'delay', 'highlig': 'highly', 'glynn': 'lynn', 'ephesus': 'rhesus', 'buckhorn': 'bullhorn', 'newconnect': 'reconnect', 'dollarsmn': 'dollars', 'hru': 'iru', 'ghg': 'gig', 'dopeness': 'hopeless', 'banega': 'bang', 'thinkingan': 'thinking', 'nashu': 'nash', 'postpals': 'postal', 'alwayz': 'always', 'telk': 'tell', 'harrasses': 'harasses', 'raceway': 'railway', 'kati': 'kate', 'russel': 'russell', 'chessy': 'chess', 'piikup': 'pickup', 'quic': 'quit', 'whaddup': 'whaddya', 'hydrox': 'hydro', 'oakwood': 'lockwood', '25yours': 'yours', 'singaporean': 'singapore', 'altenrative': 'alternative', 'graeters': 'greater', 'memang': 'mean', 'collison': 'collision', 'kani': 'kai', 'beaurmont': 'beaumont', 'm2m': 'mom', 'naju': 'naru', 'torned': 'turned', 'inalis': 'nails', 'adeleke': 'delete', 'day25': 'day', 'hunderd': 'hundred', 'palmsnc': 'palms', 'goog': 'good', 'huun': 'hun', 'ubamarket': 'upmarket', 'dhani': 'dani', 'cr': 'or', 'chappal': 'chapel', 'nanded': 'landed', 'fuckcars': 'fuckers', 'illiquid': 'liquid', 'negasi': 'negai', 'blackhawk': 'blackjack', 'supp': 'sup', 'oklisten': 'listen', 'lorder': 'order', 'kaileigh': 'raleigh', 'mdt': 'met', 'r0': 're', 'whole30': 'whole', 'ddot': 'dot', 'lakefield': 'wakefield', 'sastrong': 'strong', 'aftee': 'after', 'milanos': 'milano', 'eustis': 'curtis', 'hoche': 'noche', '3b': 'ab', 'myeni': 'men', 'karasek': 'karate', 'darebin': 'darwin', 'bln': 'bin', 'happy3': 'happy', 'riann': 'rian', 'day1': 'day', 'cofarms': 'confirms', 'sadwilling': 'swilling', 'alleg': 'alley', 'skrt': 'sort', 'alttext': 'attest', 'stayedin': 'stayed', 'bmart': 'smart', 'opposedto': 'opposed', 'weasled': 'weaseled', 'saanich': 'spanish', 'myalgice': 'malice', 'sweaterboy': 'waterboy', 'c3': 'ca', 'individualy': 'individual', 'reimb': 'climb', 'x3l': 'all', 'chelseafc': 'chelsea', 'erruns': 'reruns', 'cabletv': 'cable', 'excelso': 'excels', 'clikat': 'client', 'unisom': 'unison', 'twinings': 'twining', 'porkchop': 'workshop', 'arno': 'ano', 'shd': 'she', 'fuckkin': 'fuckin', 'lovethis': 'loveth', 'lardarse': 'larders', 'naalya': 'natalya', 'mbta': 'meta', 'wuxian': 'julian', 'love4k': 'love', 'topbox': 'toolbox', 'umwe': 'ume', 'chocco': 'choco', 'babcock': 'hancock', 'bever': 'never', 'bced': 'bed', 'gic': 'gig', 'recip': 'recipe', 'antwerp': 'answer', 'hakin': 'akin', 'csn': 'can', 'mansun': 'manson', 'edc': 'ed', 'ps20': 'psi', 'emiliano': 'emilio', 'demarkus': 'remarks', 'appvan': 'appian', 'restros': 'retros', 'omw': 'ow', 'newjob': 'nutjob', 'lgu': 'lug', 'jrr': 'jar', 'hott': 'hot', 'daybed': 'daubed', 'marinette': 'marionette', 'grippers': 'rippers', 'alexapp': 'alexa', 'reats': 'beats', 'sheri': 'sherri', 'roughl': 'rough', 'richmondbc': 'richmond', 'traavel': 'travel', 'pollard': 'collard', 'q7': 'qi', 'eatfix': 'matrix', 'ayat': 'ayah', 'fcuking': 'fucking', 'hazzard': 'hazard', '$snap': 'snap', 'sna': 'sea', 'mels': 'mess', 'blessup': 'bless', 'bastrop': 'bistro', 'ataga': 'stage', 'cowchip': 'coochie', 'bringy': 'bring', 'quar': 'quart', 'epson': 'epsom', 'durga': 'dura', 'balham': 'balsam', '6mm': 'umm', 'ppfft': 'puff', 'meiji': 'mini', 'neuvoo': 'neuro', 'strongbow': 'strongbox', 'monees': 'monies', 'loveseat': 'lovest', 'toocold': 'twofold', 'use1': 'used', 'putnam': 'puta', 'allgood': 'agood', 'valencia': 'valentin', 'arounds': 'around', 'napper': 'rapper', 'wii': 'win', 'sicklife': 'sickle', 'eatgud': 'eated', 'coldwell': 'caldwell', 'parti': 'part', 'thex3': 'the', 'maalat': 'malt', 'custexp': 'custer', 'faithie': 'faith', 'kindn': 'kind', 'jonr': 'jon', 'stofix': 'stoic', '$ndx': 'and', 'supah': 'sudah', 'casue': 'case', 'guber': 'uber', 'inexistent': 'insistent', 'dommes': 'domes', 'eathf': 'each', '37d': 'and', 'providi': 'provide', 'antis': 'ants', 'hakka': 'wakka', 'ohsots': 'shots', 'rufollow': 'follow', 'njwtl': 'nowt', 'cletis': "let's", 'waymo': 'wayto', 'skeds': 'seeds', 'durkan': 'durian', 'fsu': 'fu', 's2': 'so', 'elsalvador': 'salvador', 'decypher': 'decipher', 'wineconne': 'pinecone', 'freegame': 'pregame', 'oup': 'out', 'rg6': 'go', 'asuu': 'asu', 'lovegives': 'lovelies', 'litteri': 'litter', 'launced': 'launched', '2keep': 'keep', 'kitka': 'kita', 'carmeli': 'carmela', 'uyo': 'yo', 'deepcut': 'deepest', 'moana': 'moans', 'rrison': 'prison', 'shopezy': 'shapely', '6mg': 'my', 'acheh': 'ache', '70yours': 'yours', 'compy': 'copy', 'fdic': 'dic', 'wujo': 'who', 'greathorn': 'greenhorn', 'nugs': 'nuts', 'optimom': 'optimum', 'erp': 'era', 'runback': 'unpack', 'gish': 'wish', 'continent]': 'continent', 'ohh': 'oh', 'nurobots': 'robots', 'carrd': 'carry', 'sente': 'sent', 'flocode': 'flood', 'kodwa': 'soda', 'rexdale': 'exhale', 'tyourant': 'tyrant', 'cwar': 'car', 'platfor': 'platform', 'oha': 'oh', 'gms': 'gas', 'anyday': 'anyway', 'peoplecart': 'peopleare', 'sigonas': 'signal', 'thise': 'this', 'eidgah': 'edgar', 'refigure': 'figure', 'luckyaf': 'lucky', 'rvers': 'rivers', 'mmind': 'mind', 'schooldays': 'schoolboys', 'iaweather': 'weather', 'wowser': 'worser', 'sustaindc': 'sustained', 'farepost': 'foremost', '8months': 'months', 'paperd': 'paper', 'ideas4u': 'ideas', 'yesrterday': 'yesterday', 'ezcater': 'cater', 'outdoorist': 'outdoors', 'loven': 'love', 'i9n': 'in', 'appby': 'apply', 'fulfi': 'fulfil', 'geni': 'gene', 'oros': 'pros', 'nvidia': 'vida', 'fuschia': 'fuchsia', 'imprtnt': 'imprint', 'faaaack': 'fatback', 'saibot': 'sabot', 'mellennial': 'millennial', 'ioc': 'doc', 'catapault': 'catapult', 'reactivatted': 'reactivated', 'eramosa': 'ramona', 'geldrop': 'gumdrop', 'demmitt': 'dammit', 'yp': 'up', 'fatique': 'fatigue', 'ukriane': 'ukraine', 'gyncsm': 'syncs', 'cartly': 'partly', 'fy20': 'fy', 'blackcar': 'blacker', 'b2c': 'bac', 'popsy': 'pops', 'cest': 'best', 'ricette': 'ricotta', 'rouxls': 'souls', 'inductively': 'intuitively', 'ofp': 'of', 'gamec': 'game', 'jamaat': 'jamal', 'idgaf': 'idea', 'br1': 'bra', 'jamey': 'james', 'bahut': 'baht', 'ymm': 'yam', 'darlington': 'arlington', 'newsgrit': 'newsprint', 'whissell': 'hisself', 'utpal': 'steal', 'postits': 'posits', 'romes': 'comes', '$nyse': 'nose', 'dawlish': 'danish', 'lamy': 'lady', 'doha': 'dona', 'stressout': 'stressful', 'fauncy': 'fancy', 'phase4': 'phase', 'birden': 'burden', 'senstive': 'sensitive', 'mexicana': 'mexican', 'banza': 'banzai', 'kayode': 'abode', 'clevon': 'clean', 'photojournalism': 'photojournalist', 'airtel': 'intel', 'laredo': 'cared', 'frevert': 'revert', 'yoh': 'you', 'choudary': 'chaudhary', 'carparks': 'carpark', 'imitiaz': 'initial', 'yesoos': 'pesos', 'negligency': 'negligence', 'daamn': 'damn', 'cleves': 'clever', 'toma': 'tomb', 'bookjane': 'bookcase', 'whoohoo': 'hoodoo', 'shotput': 'shout', 'failer': 'failed', 'competetor': 'competitor', '2stop': 'stop', 'sbtf': 'set', 'meron': 'moron', 'dewinton': 'denton', 'schulte': 'schultz', 'nabes': 'names', 'adiwele': 'adele', '9pin': 'pin', 'kizz': 'jizz', 'nbnews': 'news', 'hingry': 'hungry', 'ongoogle': 'google', 'accupressure': 'acupressure', 'bigsur': 'bigger', 'heheh': 'hehe', 'garissa': 'marissa', 'moco': 'mock', 'mych': 'much', 'rcmp': 'ramp', 'mayour': 'mayor', 'raincan': 'rican', 'karim': 'karin', 'sproutbc': 'sprout', 'snf': 'anf', 'reusables': 'reusable', 'supermar': 'superman', 'blom': 'blow', 'wunnah': 'tunnah', 'riovic': 'ironic', 'fuhh': 'much', 'djmel': 'dime', 'granddog': 'grandson', 'kiers': 'piers', 'gman': 'man', 'carshare': 'carthage', 'mmk': 'mak', 'thc': 'the', 'clynk': 'clink', 'unusuable': 'unusable', 'backuards': 'backwards', 'digitalag': 'digital', 'ethne': 'the', 'schdld': 'child', 'hinton': 'hilton', 'duelki': 'duel', 'hardo': 'hard', 'deleb': 'celeb', 'follo': 'follow', 'ssoo': 'shoo', 'feest': 'feet', 'consignee': 'consigned', 'pimplastic': 'simplistic', 'dadli': 'dad', 'angele': 'angel', '2months': 'months', 'xibyte': 'byte', 'isolator': 'isolated', 'jaxon': 'jason', 'oronomn': 'pronoun', 'honestb': 'honest', 'collectio': 'collection', 'midyear': 'mishear', 'doliana': 'dorian', 'l': 'i', 'nbc2': 'nice', 'castiron': 'caution', 'mpos': 'pos', 'bayer': 'layer', 'finbrat': 'fatbrat', 'mausima': 'maxima', 'wakin': 'waking', 'gaffney': 'gaffer', 'lolol': 'loll', 'shtty': 'shitty', 'mcnaught': 'caught', 'shevlin': 'shelling', 't1': 'to', 'nyeats': 'yeats', 'hicutie': 'cutie', 'sainos': 'saints', 'hahah': 'haha', 'protes': 'proves', 'istherel': 'ifthere', 'belluz': 'bell', 'ysu': 'you', 'fria': 'friar', 'jameson': 'james', 'tremblant': 'trembling', 'icym': 'icy', 'diablo3': 'diablo', 'connecti': 'connect', 'keyworders': 'keywords', 'salamat': 'salami', 'holiday25': 'holiday', 'buyable': 'unable', 'thinknthey': 'thinkthey', 'manege': 'manage', 'koh': 'oh', 'afterno': 'after', 'billi': 'bill', 'huqeat': 'heat', 'seriese': 'series', 'neeraj': 'neural', 'x10': 'xu', 'vocs': 'vous', 'efulfilment': 'fulfilment', 'belocal': 'local', 'appsec': 'apple', 'explo': 'expo', 'onetake': 'intake', 'uncorks': 'uncork', 'milltown': 'million', 'jamil': 'jail', 'couped': 'cooped', '$hiti': 'hit', 'bandon': 'abandon', 'warfield': 'garfield', 'swea': 'swear', 'resson': 'reason', 'dmvcd': 'david', 'redmarket': 'remarked', 'caixin': 'cabin', 'constipate': 'constipated', 'jamaa': 'jamal', 'fbpe': 'be', 'abcd': 'abed', 'abutt': 'butt', 'ict': 'it', 'artifical': 'artificial', 'kawawa': 'katana', 'aundh': 'and', 'nhlocal': 'local', 'vampy': 'vamp', 'litcherally': 'literally', 'rs76': 'rse', 'salir': 'sahir', 'gtown': 'town', 'ontrac': 'contract', 'whch': 'which', 'disopred': 'disposed', 'houseless': 'horseless', 'kwn': 'own', 'sheilding': 'shielding', 'rce': 'ice', 'chamisa': 'chemist', 'delivery1': 'delivery', 'fundraisi': 'fundraiser', 'aryo': 'ryo', 'booklking': 'booking', 'stresseat': 'stressed', 'hj': 'he', 'releasy': 'release', 'vicentes': 'licenses', 'networkon': 'network', 'needham': 'needa', 'sauga': 'sauna', 'shopan': 'shop', 'sloss': 'loss', 'febuary': 'february', 'tomake': 'take', 'techiewo': 'techies', 'airtable': 'portable', 'gbcs': 'guys', 'lieutenancy': 'lieutenant', 'campish': 'camping', '$ym': 'gym', 'everything]': 'everything', 'fbr': 'for', 'confrmation': 'confirmation', 'shopify': 'shoplift', 'ffm': 'fam', 'tempe': 'temple', '$dia': 'dia', 'smiler': 'smile', 'bulliten': 'bullied', 'preschooler': 'preschoolers', 'neimbo': 'limbo', 'westover': 'leftover', 'nilgiris': 'nigiri', 'fuek': 'fuck', 'yango': 'tango', 'uae': 'use', 'shiel': 'shield', 'rashan': 'asian', 'essentialism': 'essentials', 'thinke': 'think', 'humza': 'humma', 'peerman': 'german', 'braai': 'break', 'orengo': 'orange', 'repost': 'report', 'fodbox': 'fobo', 'fanclub': 'fansub', 'tubi': 'tube', 'vns': 'vans', 'lassa': 'lass', 'bohol': 'boho', 'ehhe': 'thhe', 'ebfp': 'beep', 'rn': 'in', '8ish': 'wish', 'gurnee': 'gurney', 'weown': 'down', 'umaine': 'maine', 'affordering': 'affording', 'schmoles': 'schmoes', 'apgd': 'and', 'flink': 'link', 'matlab': 'atlas', 'whenu': 'when', 'destractions': 'distractions', 'jenni': 'jenny', 'dealy': 'deal', 'takeou': 'takeout', 'inessential': 'essential', 'bourses': 'courses', 'batoor': 'batter', '$snss': 'sass', 'ps12': 'psi', 'phils': 'phil', 'procures': 'procure', 'gorai': 'goran', '90th': 'with', 'diyas': 'divas', 'kundang': 'funding', 'thyo': 'thy', 'km2': 'my', 'lufa': 'luna', 'hcq': 'he', 'tega': 'tea', 'beidle': 'bridle', 'usitoke': 'stoke', 'covi': 'cove', 'brompton': 'compton', 'nothanks': 'thanks', 'consu': 'cons', 'nailn': 'nail', 'broward': 'forward', 'laurier': 'laurie', 'ropp': 'rope', 'tweeple': 'tweedle', 'waxers': 'waters', 'graeter': 'greater', 'kwathema': 'anathema', 'punkins': 'punking', 'brunsco': 'brunch', 'tnteachers': 'teachers', 'kollok': 'follow', 'closedown': 'comedown', 'essentiala': 'essential', 'pares': 'cares', 'slovenia': 'slovenian', 'betanews': 'beaners', 'homewith': 'comewith', 'ummeed': 'bummed', 'oc': 'of', 'restuarants': 'restaurants', 'sonke': 'sone', 'avl': 'all', 'lundh': 'lunch', 'emz': 'em', 'mrt': 'met', 'lart': 'last', 'canpo': 'capo', 'nuovi': 'nova', 'crowdy': 'crowd', 'trafford': 'afford', 'glawomen': 'lawmen', 'rambly': 'ramble', 'samocat': 'samosas', '$arsn': 'warn', 'freshket': 'freshen', 'bcoz': 'coz', 'maxinweb': 'maxine', 'hemet': 'helmet', 'stevi': 'steve', 'debter': 'deter', 'pediasure': 'pedicure', 'cheetohs': 'cheetahs', 'dedebit': 'deceit', 'waterbury': 'waterboy', 'dunkins': 'dunking', 'fren': 'free', 'lovedfb': 'loved', 'incumbency': 'incumbent', 'bitchmag': 'bitching', 'stowage': 'storage', 'sankalp': 'sandals', 'q3months': 'months', 'thril': 'thrill', 'tbd': 'tad', 'becos': "b'cos", 'penitas': 'penis', 'guarentee': 'guarantee', 'mutoko': 'moloko', 'wahala': 'whale', 'ovtene': 'often', 'qualitytime': 'qualitative', 'anyara': 'anata', 'appt': 'app', 'rubicon': 'rubin', 'us3': 'us', 'wagmi': 'warm', 'tatay': 'tatty', 'khapa': 'hapa', 'ar15s': 'arms', 'rect': 'rest', 'asheville': 'nashville', 'farmstand': 'farmland', 'nodnod': 'nodded', 'symon': 'simon', 'nofaith': 'faith', 'ooops': 'hoops', 'wishaw': 'wish', 'moes': 'does', 'logisticssc': 'logistics', 'tectra': 'tetra', 'brgys': 'brays', 'smuggery': 'surgery', 'badluck': 'padlock', 'boscovs': 'bosoms', 'habilitation': 'rehabilitation', 'raminder': 'reminder', 'levenger': 'revenge', 'sukuma': 'summa', 'kuno': 'kung', 'ilkley': 'likely', 'jobkeeper': 'zookeeper', 'sud': 'sun', '25march': 'march', 'evedything': 'everything', 'arduously': 'arduous', 'idi': 'di', 'hhca': 'ha', 'landscapers': 'landscapes', 'erybody': 'everybody', 'paranoidly': 'paranoid', 'unboil': 'until', 'pog': 'dog', 'blackeyed': 'blackened', 'uniterd': 'united', 'wfrom': 'from', 'bonless': 'boneless', 'hmo': 'ho', 'growup': 'group', 'nr': 'no', 'sorn': 'son', 'blyth': 'blythe', 'emailme': 'email', 'vecinity': 'vicinity', 'mfrazz': 'franz', 'busser': 'busier', 'bottom]': 'bottom', 'tenby': 'ten', 'returnr': 'return', 'frappuccinos': 'frappuccino', 'ayee': 'aye', 'borderen': 'bordered', 'txts': 'tits', 'felllow': 'fellow', 'rabat': 'rat', 'pakodas': 'pagodas', 'ispar': 'spar', 'alerus': 'alerts', 'crohns': 'crowns', 'tryina': 'trying', 'zercular': 'circular', 'commodify': 'commodity', 'corona19': 'coronary', 'apartm': 'apart', 'cwu': 'wu', 'iont': 'dont', 'bukak': 'break', 'wtrh': 'wirh', 'inco': 'into', 'dayly': 'daily', '1st]': 'est', 'vaxd': 'and', 'baku': 'bake', 'upsy': 'ups', 'partn': 'part', 'crafter': 'crater', 'ariz': 'arid', 'skep': 'step', 'dustmen': 'dustman', 'fede': 'feed', 'taing': 'thing', 'okcabs': 'cabs', 'nowlive': 'olive', '7a': 'a', 'warroom': 'barroom', 'margate': 'karate', 'mura': 'aura', 'trustro': 'trust', 'orgs': 'orgy', '4square': 'square', 'sena': 'send', 'ammend': 'amend', 'urm': 'arm', 'carseats': 'carpets', 'kotapet': 'kotae', 'ormedicaid': 'medicaid', 'hata': 'hate', 'waay': 'way', 'meanti': 'meant', '12wks': 'hawks', 'cbs2': 'case', 'bansal': 'banal', 'bunchy': 'bunch', 'langham': 'gingham', 'rektang': 'resting', 'oecd': 'ocd', 'newtown': 'newton', 'members]': 'members', 'altuve': 'alive', 'olay': 'play', 'cuy': 'guy', 'katz': 'kate', 'odem': 'dem', 'abdto': 'auto', 'thqn': 'then', 'immidiately': 'immediately', 'dur': 'our', 'derg': 'der', 'kuya': 'kaya', 'rver': 'over', 'nearly30': 'nearly', 'dhilli': 'chilli', 'pilicies': 'policies', 'pandits': 'bandits', 'philile': 'phillip', 'htc': 'etc', 'muzzy': 'fuzzy', '2year': 'year', 'bookeeper': 'bookkeeper', 'crv': 'cry', 'rcb': 'rob', 'strolli': 'stroll', 'gopats': 'goats', '83rd': 'hard', '48k': 'ask', 'happs': 'happy', 'bayelsa': 'bagels', 'tr$$p': 'trip', 'mealworm': 'metalwork', 'zar': 'car', 'strug': 'strung', 'caskett': 'casket', 'lanae': 'lane', 'jii': 'aii', 'delhites': 'elites', 'ndtv': 'not', 'copout': 'colour', 'freshy': 'fresh', 'sugama': 'sugar', 'dgt': 'dot', 'bernice': 'bernie', 'birmingha': 'birmingham', 'piran': 'iran', 'trecco': 'trench', 'tf': 'to', 'mycousin': 'cousin', 'dedra': 'debra', 'pnt': 'put', 'kro': 'pro', 'tameka': 'camera', 'apf': 'ape', 'lowri': 'lori', 'aramean': 'ramen', 'blasio': 'basic', 'mbz': 'mba', 'cancle': 'cancel', 'myi': 'my', 'isaacs': 'isaac', 'totnes': 'tones', 'bailly': 'billy', 'kerrie': 'berries', 'acceptably': 'acceptable', 'kavan': 'karan', '7eleven': 'eleven', 'washu': 'wash', 'globalize': 'globalized', 'outwest': 'cutest', 'pimentos': 'pimento', 'umay': 'may', 'sunila': 'sunil', 'jarboe': 'carbon', 'fds': 'feds', 'nendo': 'kendo', 'pkpd': 'kid', 'ronde': 'rode', 'glizzies': 'grizzlies', 'dillard': 'willard', 'cf97': 'cfo', 'foodings': 'footings', 'covod': 'good', 'crackin': 'cracking', 'embarressed': 'embarrassed', 'kuroo': 'euros', 'jejely': 'merely', 'chunri': 'chunk', 'byb': 'by', 'zaxbys': 'babys', 'surplussed': 'surpassed', 'notreally': 'normally', 'gulu': 'guru', 'quaran': "qu'ran", 'yy': 'my', 'likeeee': 'likened', 'jalgaon': 'jargon', 'infinitum': 'infinite', 'eatgood': 'agood', 'v8': 've', 'dgf': 'def', 'eyeroll': 'eyeball', 'manzano': 'manana', 'tgus': 'thus', 'indendent': 'independent', 'bautista': 'batista', 'agedp': 'aged', 'unfortunetly': 'unfortunately', 'colonyusa': "colony's", 'aay': 'any', 'weareback': 'bareback', 'hiri': 'hire', 'imapact': 'impact', '2catch': 'catch', 'famers': 'farmers', 'oklaho': 'oklahoma', 'xelnia': 'xenia', 'g14': 'get', 'covac': 'cover', 'youpay': 'yousa', 'duii': 'dui', 'dorkie': 'dookie', 'freshand': 'freshman', 'mixups': 'minus', 'demilitarization': 'militarization', 'namin': 'naming', 'gamily': 'family', 'stempel': 'stepped', 'calander': 'calender', 'ehailing': 'hailing', 'risa': 'risk', '$slbd': 'slid', 'cnas': 'cans', 'mafe': 'make', 'leos': 'less', 'fd': 'fed', 'icd': 'ice', 'mandasa': 'mandala', 'flickr': 'flick', 'shoptal': 'shoal', 'lugrocery': 'grocery', 'smes': 'sees', 'assy': 'ass', 'accomodated': 'accommodated', 'don8ing': 'donning', 'ks': 'is', 'miscount': 'discount', 'ba5': 'bad', 'src': 'arc', 'smhs': 'she', 'corne': 'corner', 'parkand': 'parkland', '$nvda': 'anda', 'rissy': 'risky', 'dench': 'bench', 'shon': 'show', 'reverifying': 'verifying', 'ooshies': 'posies', 'btv': 'bev', 'howick': 'hick', 'sapnio': 'amnio', 'patios': 'patio', 'cshead': 'head', 'muzz': 'buzz', 'iftars': 'stars', 'riband': 'ribald', 'ruebens': 'rubens', 'exition': 'edition', 'aruba': 'aruna', 'kkk': 'kak', 'guardcat': 'guardian', 'wdg': 'wig', 'power10': 'power', 'milennials': 'millennials', 'gwake': 'wake', 'nbflood': 'blood', 'arrives]': 'arrives', 'gamestop': 'gamester', 'aree': 'are', 'ald': 'and', 'papada': 'papaya', 'timeresq': 'timeless', 'rochesterny': 'rochester', 'strarted': 'started', 'kayce': 'maybe', 'babiesto': 'babies', 'w\\': 'we', 'choon': 'coon', 'adcoop': 'scoop', 'sacrifies': 'sacrifices', 'vivaro': 'vicar', 'b29': 'be', 'solarpv': 'solar', 'deining': 'dining', 'recipees': 'recipes', 'mask19': 'mask', 'fsm': 'fam', 'wahrriors': 'warriors', 'nutri': 'nuts', 'suberbs': 'suburbs', 'mober': 'sober', 'difficut': 'difficult', 'zingone': 'ringtone', 'outerbanks': 'outranks', 'lsk': 'ask', 'hubbin': 'rubbing', 'hambre': 'hombre', 'kharkiv': 'khaki', 'ilene': 'irene', 'chambana': 'cabana', 'runnah': 'tunnah', 'bkmagazine': 'magazine', 'viejitas': 'veritas', 'xkzero': 'zero', 'gamrik': 'gambit', 'imba': 'ima', 'hedgies': 'hedges', 'sumbody': 'somebody', 'foodlove': 'foxglove', 'aspersion': 'aspersions', 'backread': 'backed', 'zz': 'ze', 'dillons': 'dillon', 'roark': 'roar', 'explioted': 'exploited', 'worley': 'world', 'heartnews': 'heartless', '1btc': 'etc', 'rupel': 'repel', 'bidfood': 'bigfoot', 'unseasoned': 'seasoned', 'offici': 'office', 'unpadded': 'padded', '4orderers': 'murderers', 'kut': 'but', 'phadke': 'shake', 'a]': 'a', 'militaire': 'military', 'ld': 'old', 'phiona': 'phone', 'servicss': 'services', 'ubers': 'users', 'riks': 'risk', 'noveleta': 'novelty', 'crunchbase': 'crunchable', 'bvnn': 'benn', 'hinterlands': 'hinterland', 'calnan': 'clean', 'scrollin': 'scrolling', 'stamper': 'stamped', 'blackheath': 'blackheads', 'inmouth': 'mouth', 'catboys': 'cowboys', 'nthing': 'thing', 'hospo': 'hope', 'theee': 'there', 'wjxt': 'what', 'kgomo': 'komo', 'watchlab': 'watchman', 'chefate': 'create', 'computin': 'computing', 'kuma': 'uma', 'akila': 'akira', 'subna': 'sun', 'seicho': 'echo', 'kry': 'try', 'amica': 'mica', 'knott': 'knot', 'shittily': 'shitty', 'delevery]': 'delivery', 'sacbee': 'sacred', 'requ': 'req', 'vali': 'valid', 'advertisment': 'advertisement', '7deg': 'idea', 'buggin': 'bugging', 'buyin': 'buying', 'twoogle': 'google', 'irukan': 'truman', 'dorsey': 'horsey', 'passmore': 'passport', 'essec': 'essex', 'ln': 'in', 'kmiz3': 'kizu', 'iromg': 'from', 'safeway': 'safety', 'babyswag': 'babysat', 'topwomen': 'ofwomen', 'goldrush': 'goldfish', 'colonscopy': 'colonoscopy', 'kansans': 'kansas', 'nahii': 'nail', 'empleo': 'temple', 'thevoice': 'thejuice', 'kirtland': 'portland', 'month]': 'months', 'gsbr': 'sir', 'eatsmart': 'outsmart', 'oakham': 'arkham', 'canibalizing': 'cannibalizing', 'jalisa': 'alias', 'izu': 'iru', '45th': 'with', 'sodone': 'somone', 'ahahha': 'haha', '84f': 'of', 'granule': 'granules', 'orgn': 'organ', 'abul': 'abu', 'khsm': 'him', 'pamstores': 'pastures', 'more]]': 'more', 'anki': 'andi', 'jona': 'bona', 'kz': 'ke', 'orderash': 'orders', 'bedforder': 'bedford', 'alok': 'amok', 'mudala': 'medal', 'pheew': 'phew', '1i': 'i', 'binnys': 'bunny', 'edelivery': 'delivery', 'canyaman': 'cayman', 'spd': 'sad', 'tipid': 'timid', 'tlaib': 'train', 'overtone': 'overdone', 'rozanne': 'roxanne', 'pomaa': 'woman', 'twofer': 'tower', 'blt': 'but', 'lanne': 'lane', 'lazboy': 'lazy', 'widdling': 'fiddling', 'jlac': 'lac', 'roundup]': 'roundup', 'mmg': 'mug', 'rmo': 'mo', 'zimm': 'him', 'huffman': 'hoffman', 'shoutdownsa': 'shutdowns', 'secondds': 'seconds', 'cultur': 'culture', 'isucals': 'visuals', 'austudy': 'study', 'gogurts': 'yogurts', 'sonora': 'senora', 'witho': 'with', 'udelivery': 'delivery', 'ingres': 'ingress', 'baasket': 'basket', 'terada': 'trade', 'adpublisher': 'publisher', 'funnymine': 'funnyman', 'liveroo': 'liver', 'myouran': 'mourn', 'qvetched': 'wretched', 'solon': 'soon', 'nengi': 'feng', 'packgages': 'packages', 'ngong': 'gong', 'bradforder': 'bradford', '7up': 'up', 'everr': 'ever', 'buyk': 'buy', 'peggys': 'peggy', 'savi': 'save', 'gaberones': 'cabrones', 'cargill': 'carroll', 'methology': 'mythology', 'jamies': 'james', 'sitz': 'sit', 'yo24': 'you', 'blurrier': 'barrier', 'tchr': 'tchi', 'debatably': 'debatable', '7oz': 'coz', 'lauch': 'laugh', 'sobey': 'obey', 'dhan': 'than', 'obvs': 'obs', 'adversion': 'aversion', 'blockit': 'block', 'peticion': 'petition', 'bbqing': 'being', 'martyour': 'martyr', 'tiffanyld': 'tiffany', 'pardner': 'partner', 'nkataa': 'katana', 'fdx': 'fix', 'ginas': 'gina', 'gartn': 'garth', 'nqaba': 'nab', 'deka': 'deja', 'ladki': 'lady', '5$': 'i', 'wato': 'wat', 'realime': 'realize', 'trincity': 'trinity', 'screemed': 'screamed', 'passle': 'hassle', 'utilit': 'utility', 'esay': 'say', 'lkd': 'led', 'tawanda': 'amanda', 'ckdown': 'down', 'uff': 'off', 'rox': 'box', 'nosleep': 'sleep', 'senioras': 'seniors', 'amrutha': 'martha', 'mrx': 'mix', 'organiza': 'organize', 'cornnelius': 'cornelius', 'unveilin': 'unveiling', 'froud': 'proud', 'flipcart': 'flippant', 'pibb': 'pick', 'liketkit': 'likethis', 'movability': 'mobility', 'lvl1': 'lily', 'beckton': 'beckon', 'distence': 'distance', 'cced': 'iced', 'jeanerette': 'jeanette', 'krne': 'rne', 'thika': 'shika', 'wageva': 'wage', 'deacti': 'death', 'addicti': 'addict', 'udhav': 'hav', 'inneed': 'indeed', 'auc': 'au', 'flatleaf': 'flathead', 'brawnson': 'brandon', 'chspters': 'chapters', 'nascgdr': 'nascar', 'bottlo': 'bottle', 'houstones': 'houston', 'foodtr': 'food', 'sakhe': 'sake', 'blackcap': 'blacktop', 'bck': 'back', 'wegmans': 'germans', 'sternnmm': 'sternum', 'rustan': 'ruslan', 'nph': 'nah', 'usguys': 'guys', 'twatter': 'twitter', 'yakima': 'yakuza', 'bromelain': 'bromeliad', 'cashback': 'flashback', 'tamimi': 'taming', 'shaheen': 'shaken', 'inbtwn': 'inborn', 'cabinents': 'cabinets', 'noc': 'not', 'cantire': 'cantare', 'professionas': 'professional', 'thrir': 'their', 'nshealth': 'health', 'allid': 'allied', 'sumone': 'simone', '8tt': 'att', 'paisano': 'paisans', 'daaling': 'dealing', '95ca': 'ca', 'jrnl': 'jail', 'nodia': 'nadia', 'gtp': 'gap', 'yal': 'ya', 'cavo': 'cave', 'thriftys': 'thrifty', 'infosys': 'informs', 'halee': 'hale', 'ejcting': 'ejecting', 'untipped': 'untapped', 'woodie': 'hoodie', 'whatwedo': 'whatwe', 'elementor': 'elements', 'koharu': 'kawaru', 'punchlines': 'punchline', 'witbier': 'wittier', 'revenu': 'revenue', 'niz': 'ni', 'renames': 'renamed', 'deliveryso': 'delivery', 'baraboo': 'bamboo', 'axschat': 'asshat', 'kadha': 'radha', 'lti': 'lit', 'cajoo': 'major', 'bgcm': 'back', 'psl7': 'pull', 'noma': 'coma', 'difficultly': 'difficulty', 'choccy': 'choice', 'barnsley': 'barley', '\\o': 'to', 'mixmornings': 'midmorning', 'tsamaile': 'tamale', '$jbl': 'jal', 'alserv': 'alert', 'overdraw': 'overdrawn', 'zapps': 'apps', 'lucerne': 'laverne', '9min': 'min', 'princ': 'prince', 'rylee': 'rule', 'lkatz': 'late', 'funnyxx': 'funny', '$spy': 'spy', 'survivers': 'survivors', 'arokya': 'aroma', 'differe': 'differ', 'pfft': 'put', 'ingle': 'single', 'noida': 'noid', 'iep': 'pep', 'andra': 'anda', 'lynwood': 'plywood', 'wtchin': 'watchin', 'korang': 'orang', 'ghbar': 'hear', 'food]': 'food', 'inflatables': 'inflatable', 'nyourr': 'your', 'dahian': 'damian', 'mutahi': 'mutant', 'onev': 'one', 'faceted': 'acted', 'bbackk': 'back', '40mph': 'lymph', 'articulo': 'article', 'missional': 'mission', 'stong': 'strong', 'goshi': 'gosh', 'labas': 'labs', 'belived': 'believed', 'capecod': 'caped', 'oldster': 'older', 'fakin': 'faking', 'karibu': 'caribou', 'instock': 'stock', 'qurban': 'urban', 'matric': 'matrix', 'fifa': 'fife', 'citroen': 'citron', 'tieho': 'tie', 'aramingo': 'framing', 'meteopolis': 'metropolis', 'dv': 'do', 'tuta': 'tuna', 'mulund': 'mound', 'cunniff': 'cunning', 'dollarsmm': 'dollars', 'cocktwat': 'cockmeat', 'hnoa': 'noa', 'yigs': 'pigs', 'ct': 'it', 'picknpa': 'pickup', 'lbloggers': 'bloggers', 'methuen': 'methane', '25natasha': 'natasha', 'bemasked': 'masked', 'kombela': 'tombola', 'sxsw': 'saw', '2build': 'build', 'punnet': 'planet', 'rakoff': 'payoff', 'lville': 'ville', 'pinof': 'pinot', 'grantley': 'granted', 'wakiso': 'waking', 'tbg': 'tag', 'gopuff': 'puff', 'forst': 'first', 'chocula': 'chocolat', 'expe': 'expel', 'airevolution': 'revolution', 'instated': 'instate', 'redelivered': 'delivered', 'helle': 'hell', 'unimog': 'union', 'longday': 'monday', 'butty': 'butt', 'evev': 'even', 'pigtown': 'pigeon', 'sindh': 'sind', '20t': 'it', 'imlazy': 'lazy', 'vehicule': 'vehicle', 'immu': "i'm", 'inkinga': 'inking', 'springwise': 'springtime', 'smu': 'sum', 'bcuz': 'cuz', 'amoun': 'amount', 'lumin': 'lumen', 'kilkee': 'killed', 'these]': 'these', 'centralpa': 'central', 'bsc': 'bac', 'ewowit': 'knowit', 'colla': 'collar', 'willem': 'willed', 'happywe': 'happy', 'cty': 'city', 'amountig': 'amounting', 'thera': 'there', 'fantasti': 'fantastic', 'todays': 'today', 'gstay': 'stay', 'todag': 'today', 'markel': 'market', 'xtremely': 'extremely', 'mediapros': 'mediators', 'vacc': 'vac', 'mankar': 'mandar', 'iwad': 'wad', 'camne': 'came', 'assu': 'ass', 'hilburn': 'wilbur', 'coopcares': 'compares', 'sply': 'spy', 'jcmo': 'jump', 'rudra': 'rude', 'gbbo': 'go', '1mile': 'smile', 'meeee': 'melee', 'bubbeh': 'bubbe', 'bohlin': 'bolin', 'tfg': 'tag', 'rinks': 'drinks', '69p': 'up', 'rolli': 'roll', 'digicel': 'digital', '2be': 'be', 'stp': 'stop', 'illoyal': 'illegal', 'slugfest': 'suggest', 'delv': 'del', '$group': 'group', 'imformative': 'informative', 'carbone': 'carbon', 'basura': 'absurd', 'unshucked': 'unchecked', 'florham': 'floral', 'khit': 'hit', 'gorceries': 'groceries', 'bazar': 'bazaar', 'hebbal': 'herbal', 'garvey': 'harvey', 'siya': 'soya', 'catie': 'cutie', 'otc': 'ot', 'gadkari': 'bakari', 'frooti': 'froot', 'surgury': 'surgery', 'henyu': 'hey', 'gmom': 'mom', 'r5': 're', 'galavanting': 'gallivanting', 'chessman': 'hessian', 'itg': 'it', 'duro': 'euro', 'cupbrds': 'cupboards', 'therfore': 'therefore', 'coinbase': 'coinage', 'kosam': 'roam', 'prepred': 'prepared', 'rotrod': 'rotted', 'cbp': 'cop', '1minread': 'misread', 'gomexican': 'mexican', 'ien': 'in', 'rarified': 'ratified', 'hdpe': 'hope', 'pochick': 'chick', 'spiralized': 'spiraled', 'grdma': 'grandma', 'haats': 'hats', 'cnn': 'can', 'nuun': 'nun', 'boybands': 'bombards', 'mr]': 'mri', 'shoppah': 'shopped', 'af2': 'aft', 'roaump': 'roam', 'alotted': 'plotted', 'fsd': 'fed', 'shakara': 'shakira', 's1': 'so', 'pnps': 'pops', 'squick': 'quick', 'meedo': 'miedo', 'lbps': 'lips', 'hoos': 'hook', 'cj': 'ca', 'assualting': 'assaulting', 'walkthroughs': 'walkthrough', 'population]': 'population', 'probally': 'probably', 'ahiam': 'ahim', 'wartel': 'cartel', '2amazon': 'amazon', 'vabeach': 'beach', 'dako': 'dago', 'fforder': 'order', 'bietjie': 'beetje', 'poggies': 'piggies', 'royalton': 'royalty', 'ingdns': 'inns', 'voods': 'woods', 'cx5': 'can', 'abc15': 'abc', 'oph': 'oh', 'fic': 'fix', 'syngery': 'surgery', '$ing': 'sing', 'laconia': 'laconic', 'sheetcake': 'shortcake', 'doggos': 'dogs', 'pmacem': 'pacem', 'albie': 'allie', 'alphabot': 'alphabet', 'postcarders': 'postcards', 'eriksson': 'frisson', 'q': 'i', 'ageeb': 'agree', 'busineeses': 'businesses', 'iotpl': 'hotel', 'wiggily': 'wiggly', 'houmous': 'humour', 'mth': 'math', 'chine': 'shine', 'hcw': 'how', 'olaney': 'laney', 'dunkirk': 'dunking', 'waffl': 'waffle', 'deu': 'due', 'onwine': 'online', 'gojek': 'goes', 'approvaly': 'approval', 'aspro': 'astro', 'oxymoronic': 'oxymoron', 'porchside': 'portside', 'nogogo': 'gogo', 'flighy': 'flight', 'iterative': 'operative', 'fastly': 'vastly', 'miracling': 'circling', 'terrell': 'terre', 'bussin': 'bussing', 'chametz': 'chalet', 'lastleg': 'castle', 'prev': 'prep', 'spac': 'space', 'iplumped': 'plumped', 's15': 'so', 'npaid': 'paid', 'ndg': 'nag', 'inly': 'only', 'choosen': 'choose', 'slimfast': 'slimmest', 'capolitics': 'politics', 'volle': 'volley', 'carew': 'care', 'krti': 'art', 'wunna': 'wanna', 'doeant': "doesn't", 'mabini': 'making', 'lehubby': 'chubby', 'boltnc': 'bolt', 'tivoli': 'tivo', 'bmto': 'to', 'mililani': 'militant', 'unsnap': 'snap', 'financ': 'finance', 'barq': 'bar', 'chetan': 'thetan', '6working': 'working', 'neft': 'next', 'matlock': 'warlock', 'sbd': 'sad', 'srvcs': 'serves', 'momhood': 'manhood', 'shabnam': 'shabbat', 'mcommerce': 'commerce', 'tokrie': 'tore', 'emas': 'mas', '5per': 'per', 'aiyo': 'hiyo', 'capway': 'away', 'tocacyy': 'tobaccy', 'arkansan': 'arkansas', 'tdynamo': 'dynamo', 'crea': 'area', 'tcl': 'til', '45am': 'am', 'xl8': 'all', 'alri': 'ali', 'freshyo': 'fresh', 'repicking': 'repacking', 'sandt': 'sand', 'mericans': 'americans', 'amazone': 'amazons', 'beltz': 'belt', 'ckd': 'cod', 'vegatables': 'vegetables', 'zomy': 'zoey', 'gobeba': 'gabba', 'sheaux': 'shear', 'rentable': 'reliable', 'berkley': 'berkeley', 'disorderers': 'disorders', 'squally': 'equally', '90f': 'of', 'nytbits': 'nitwits', 'macarons': 'macaroni', 'tessco': 'tess', 'cozmo': 'como', 'perdure': 'perjure', 'katv7': 'kate', 'dinein': 'dining', 'examblues': 'examples', 'advantag': 'advantage', 'rozay': 'today', 'kaba': 'kara', 'chswx': 'chew', 'underprice': 'underwrite', 'dellyman': 'bellman', 'esperando': 'desperado', 'ynwa': 'ya', 'fandb': 'and', 'oshkosh': 'skosh', 'happyi': 'happy', 'mazarro': 'bizarro', 'hdw': 'how', 'gita': 'git', 'outting': 'putting', 'winer': 'wine', 'foreverer': 'forever', '14your': 'your', 'tocook': 'took', 'taz': 'tax', 'greencake': 'greendale', 'apac': 'pac', 'monkees': 'monkeys', 'montanita': 'montana', 'ceucats': 'ducats', 'acqui': 'acquit', 'budtender': 'bartender', 'reon': 'recon', 'ghtoday': 'today', 'entr': 'enter', '04minutes': 'minutes', 'bardon': 'pardon', 'fransisco': 'francisco', 'wholeys': 'wholes', 'nazo': 'nazi', 'lotsh': 'lots', 'magfest': 'safest', 'dogoodr': 'dogwood', 'nexttime': 'nexttimeon', '5and': 'and', 'cartagena': 'cartage', 'oxbridge': 'bridge', 'optimally': 'optically', '11uk': 'fuk', 'cancered': 'canceled', 'everythi': 'everyth', 'gwanda': 'ganda', 'amina': 'amino', 'moochin': 'mooching', 'kosta': 'costa', 'pampa': 'papa', 'naivas': 'nails', 'wooten': 'wooden', '2drop': 'drop', 'maintenace': 'maintenance', 'inc42': 'inch', 'snackthing': 'snatching', 'outsofe': 'outside', 'kslam': 'slam', 'daley': 'dale', 'chik': 'chick', '18f': 'of', 'batsh': 'bath', 'appauling': 'appalling', 'maud': 'mad', 'mvd': 'mad', 'spur15': 'spur', '28h': 'oh', 'irfan': 'iran', 'talkits': 'talkies', 'rejig': 'rig', 'morlock': 'airlock', 'merong': 'wrong', 'dolsot': 'dost', 'distanci': 'distance', 'pharmac': 'pharmacy', 'osmonds': 'almonds', '12yours': 'yours', 'commingle': 'commingled', 'colourist': 'colorist', 'ngara': 'nara', 'hakarat': 'karat', 'mychoppers': 'choppers', 'bina': 'bin', 'besomebody': 'somebody', 'zp': 'up', 'pth': 'path', 'mammo': 'ammo', 'snaa': 'snap', 'daisey': 'daisy', 'angryon': 'angry', 'gooat': 'goat', 'dashleys': 'ashley', 'igotthis': 'getthis', 'kenzie': 'genie', 'just1inc': 'justin', 'mcmuffins': 'muffins', '$delivery': 'delivery', 'requirem': 'require', 'sheraa': 'sherpa', 'banfield': 'infield', 'panelo': 'panel', 'brittney': 'britney', 'singpost': 'signpost', 'tonsilitis': 'tonsillitis', 'cdramas': 'dramas', 'helina': 'helena', 'reginel': 'regional', 'digilist': 'nihilist', 'pkged': 'paged', 'sunfork': 'uncork', 'woggyiest': 'foggiest', '40lbs': 'bulbs', '0v4': 've', 'quikr': 'quirk', 'dempsters': 'dumpsters', 'abnb': 'and', 'splendidum': 'splendid', 'informd': 'inform', 'sdn': 'son', 'pegi': 'peg', 'lowkey': 'looked', 'mrseb': 'arse', 'communites': 'communities', 'blessingwe': 'blessing', 'inalain': 'italian', 'dwl': 'del', 'familias': 'familiar', 'acklen': 'ankle', 'uotnight': 'tonight', 'sfnc': 'sync', 'tip8': 'tip', 'pickily': 'picking', 'rockstars': 'rockstar', 'duhme': 'due', 'tafash': 'trash', 'asf': 'as', 'juwee': 'judge', 'divias': 'divas', 'nta': 'na', 'olia': 'olga', 'cesb': 'ces', 'hmd': 'had', 'unkowable': 'unknowable', '$ko': 'ko', 'gropro': 'grope', 'xevo': 'xev', 'gazillionth': 'gazillion', 'vernors': 'errors', 'ialah': 'salad', 'distros': 'distro', 'rushup': 'pushup', 'twix': 'twin', 'grofers': 'growers', 'mvita': 'vita', 'zients': 'agents', 'aicraft': 'aircraft', 'pogg': 'pong', 'sgtl': 'set', 'heinens': 'heavens', 'vennel': 'kennel', 'indias': 'indian', '50hours': 'hours', 'ago1': 'ago', '1death': 'death', 'mkay': 'may', 'wdy': 'why', 'lambie': 'lame', 'eze': 'eye', 'onkine': 'online', 'batla': 'bala', 'tomorro': 'tomorrow', 'spafy': 'spay', 'bussey': 'busses', 'eatfort': 'effort', 'aggie': 'maggie', 'bebs': 'bees', 'miep': 'miei', 'zasket': 'basket', 'uhn': 'uh', 'shweet': 'sweet', 'pessach': 'peach', 'presiden': 'president', 'witt': 'with', 'ente': 'enter', 'irnz': 'irn', 'weiners': 'wieners', 'rahega': 'raga', 'hesitance': 'hesitancy', 'tgif': 'gif', 'losingit': 'losing', 'nadiya': 'nadia', 'feedingusa': 'feedings', 'audiologist': 'cardiologist', 'charminar': 'charming', 'firstline': 'firsttime', 'brandedup': 'branded', 'odirile': 'virile', 'kungu': 'kung', 'softwar': 'software', 'sharkie': 'sharpie', 'mybusiness': 'business', '7t': 'it', 'southea': 'south', 'wich': 'with', 'craigs': 'craig', 'weeke': 'week', 'insuranc': 'insurance', 'deliveri': 'deliver', 'wakra': 'wakka', 'jaldi': 'jail', 'whycome': 'welcome', 'srorderers': 'sorcerers', 'erxraft': 'errant', '1]': 'i', 'nownow': 'kowtow', 'selecter': 'selected', 'somd': 'some', 'nakid': 'naked', 'emela': 'smell', 'westpack': 'wetback', 'utwu': 'utw', 'joy19': 'joy', 'floor]': 'floor', 'blancato': 'blanco', 'jogg': 'jog', 'adultness': 'dullness', 'doortep': 'doorstep', 'surat': 'sprat', 'unstuffed': 'stuffed', '4wks': 'was', 'uht': 'uh', 'clutchest': 'clutches', 'tenn': 'ten', 'isolators': 'isolates', 'uic': 'vic', 'hubmart': 'hubbard', 'pocari': 'polar', 'beens': 'been', 'woth': 'with', 'washi': 'wash', 'boostered': 'bolstered', 'izel': 'feel', 'kea': 'key', '$bidu': 'bid', 'bmas': 'bias', 'tihs': 'this', 'zorbs': 'orbs', 'd8': 'do', 'gibby': 'gibbs', 'affin': 'affix', 'sickpay': 'sickbay', 'hordering': 'ordering', 'sumthin': 'sustain', 'bbw': 'bow', 'rumy': 'rum', 'hyland': 'ryland', 'dejoy': 'decoy', 'maggies': 'maggie', 'gmhc': 'mic', 'byka': 'bya', 'grumpiest': 'grumpier', 'chikus': 'chicks', 'hottub': 'hotter', 'evetually': 'eventually', 'aip': 'air', 'marinova': 'marina', 'brambling': 'rambling', 'conplaint': 'complaint', 'firstlady': 'firstly', 'fyoure': 'your', 'odyne': 'lodyne', 'ayanna': 'wanna', 'hopely': 'homely', '30atv': 'atv', 'soweto': 'howto', 'district1': 'district', 'bazos': 'bozos', 'ohsu': 'otsu', 'commu': 'comm', 'acadia': 'acacia', 'melas': 'meals', 'indep': 'index', 'ridculous': 'ridiculous', 'chasbo': 'chase', 'ringtons': 'ringtone', 'oopsed': 'hoped', 'malas': 'males', 'legolas': 'legos', 'interconnect': 'interconnected', 'yesan': 'yean', 'nejee': 'need', 'veganish': 'vanish', 'rving': 'ring', 'labne': 'lane', 'magpa': 'magma', 'econo': 'econ', 'meri': 'mere', 'justright': 'outright', 'usd6': 'us', 'q3': 'qi', 'campany': 'company', 'stratospheres': 'stratosphere', 'epri': 'epi', 'reuses': 'refuses', 'ocsp': 'ocp', 'megans': 'means', 'mfin': 'main', 'ahea': 'ahead', 'stitchy': 'stitch', 'starbs': 'starts', 'ecoupon': 'coupon', 'd2cs': 'docs', 'carys': 'cars', 'perioddd': 'period', 'jfcs': 'jobs', 'minte': 'mine', 'edg': 'edge', 'kisero': 'misery', 'hailey': 'hailed', 'orpol': 'pool', 'faenid': 'fend', 'belarus': 'bears', 'umv': 'um', 'cripy': 'crispy', 'acedemia': 'academia', 'givingba': 'giving', 'prout': 'proud', 'momhack': 'mohawk', 'karyl': 'karl', 'blackowned': 'blackened', 'elderlies': 'orderlies', 'tyia': 'tia', 'sofast': 'sofas', '22m': 'him', '2weeks': 'weeks', 'vz': 've', 'sewell': 'swell', 'nanners': 'manners', 'gulay': 'gully', 'ftd': 'fed', 'neccessarily': 'necessarily', 'hurst': 'hurt', 'jolibee': 'jolie', 'luckit': 'lucky', 'ericas': "eric's", 'sailani': 'sailing', 'prati': 'prat', '16yo': 'yo', 'yanny': 'nanny', 'vaco': 'taco', 'esso': 'osso', 'udhampur': 'dhampir', 'egbema': 'enema', 'feul': 'feel', 'rodered': 'ordered', 'portability': 'probability', 'upcycle': 'cycle', 'rezeki': 'reeks', 'tilbury': 'wilbur', 'compan': 'company', 'fintwit': 'nitwit', 'getfood': 'seafood', 'rfac': 'fac', 'golobos': 'globes', 'pikelets': 'piglets', 'jelah': 'yeah', 'myavir': 'mavis', 'beki': 'bei', '$tmst': 'test', 'bjg': 'big', 'dadbod': 'dabo', 'tomball': 'gumball', 'brisker': 'brisket', 'heinla': 'heil', 'idec': 'idea', 'puh': 'put', 'mayoura': 'mayor', 'unprompted': 'prompted', 'mezzop': 'mezzo', 'seaworld': 'seaward', 'starquik': 'starbuck', 'brn': 'born', 'beveridge': 'beverage', 'dollarshr': 'dollars', 'sweet16': 'sweet', 'cousinh': 'cousin', 'menjamin': 'benjamin', 'colchester': 'rochester', 'aug15': 'aught', 's14': 'so', 'atlist': 'list', 'ebates': 'bates', 'quarentine': 'quarantine', 'eamon': 'damon', 'meowmy': 'mommy', 'hailno': 'hail', 'afirmativa': 'affirmative', 'khanya': 'hanya', '1x': 'ex', 'mapprr': 'mapped', 'servis': 'serves', 'spati': 'spat', 'sensitize': 'sensitive', 'bloor': 'blood', 'esol': 'sol', 'rodel': 'model', 'gotchu': 'gotcha', 'ftes': 'fees', 'jeheads': 'beheads', 'noire': 'noise', 'nsars': 'nears', 'hustlin': 'hustling', 'a7': 'a', '$seii': 'semi', 'whops': "who's", 'nocustomer': 'customer', 'decen': 'decent', 'lling': 'lying', 'mmiwg': 'ming', 'kalkti': 'talkto', 'enforcements': 'enforcement', 'fiab': 'fab', 'chaplaincy': 'chaplain', 'transportify': 'transporting', 'lys': 'lays', 'querantine': 'quarantine', 'smollet': 'smaller', 'caffeand': 'caffeine', 'maxxin': 'main', '2days': 'days', 'kca': 'ca', '57p': 'up', 'cbpr': 'car', 'quikart': 'quart', 'prearrange': 'rearrange', 'llo': 'lo', 'katalin': 'natalie', 'sherwin': 'heroin', 'localyeg': 'locale', 'pendle': 'peddle', '15aug': 'smaug', 'evaporative': 'evaporate', 'gua': 'guy', 'gulshan': 'gustan', 'portabello': 'portobello', 'godot': 'got', 'sev': 'see', 'funnydon': 'funnyman', 'rashida': 'rashid', 'nerdi': 'nerd', 'cardano': 'cardio', 'do2': 'do', 'minnesotan': 'minnesota', 'anaheim': 'anthem', 'goodtimes': 'goodies', 'tani': 'tank', 'laughlin': 'laughing', 'mavorg': 'major', 'malaysiaku': 'malaysian', 'profiterole': 'profiteroles', 'mispelling': 'dispelling', 'grocy': 'grody', 'kirabo': 'kira', 'wowow': 'wood', 'ciochat': 'chitchat', 'hclib': 'club', '12f': 'of', 'reflexis': 'reflexes', 'katten': 'kitten', 'marama': 'mama', 'turun': 'turn', 'proptech': 'protect', 'opn': 'on', 'raglan': 'raylan', 'kearny': 'learn', 'trs': 'try', 'jeera': 'meera', 'donbas': 'donnas', 'rer': 'her', 'hornyak': 'horny', 'brenham': 'brenda', 'kitsap': 'kidnap', 'edgars': 'edgar', 'o3o': 'oo', 'palicha': 'alicia', 'darko': 'dark', 'pandaily': 'handily', 'bagan': 'began', 'ksj': 'is', 'bolle': 'belle', 'henforder': 'enforcer', 'brain]': 'brain', 'ftl': 'itl', 'yt': 'it', 'yassss': 'passes', 'develo': 'develop', 'wla': 'la', 'winfrey': 'winfred', 'dsughter': 'daughter', '03may': 'may', 'coupa': 'coup', 'birdstrikes': 'airstrikes', 'svfr': 'over', '1today': 'today', 'sendbird': 'songbird', 'bikita': 'nikita', 'jfk': 'jak', 'mony': 'many', 'custserv': 'custer', 'itmunch': 'munch', 'disappointi': 'disappoint', 'cabellas': 'capella', 'rescale': 'resale', 'kerikeri': 'beriberi', 'modernliving': 'modernizing', 'tkp': 'top', 'saman': 'shaman', 'ezgrab': 'grab', 'septem': 'septum', 'posttag': 'posting', 'streeets': 'streets', 'inshorts': 'shorts', 'farecon': 'recon', 'dejesse': 'defense', 'jow': 'how', 'genji': 'genki', 'kerem': 'were', 'curation': 'duration', 'cream]': 'cream', 'hotmeal': 'hotel', 'saka': 'sake', 'househ': 'house', 'jacey': 'jace', 'ghmc': 'game', 'imediain': 'median', 'jkia': 'kia', 'hapens': 'happens', 'dhaba': 'daba', 'evp': 'eve', 'sheb': 'she', '16may': 'may', 'oshin': 'shin', 'plumetted': 'plummeted', 'friendzz': 'friend', 'goldblum': 'gollum', 'kbo': 'bo', 'shebi': 'sheba', 'diasbetes': 'diabetes', '10miles': 'miles', 'obvi': 'obi', 'tapatio': 'patio', 'jhazoo': 'kazoo', 'cooke': 'cook', 'iconiq': 'iconic', 'rdn': 'run', 'ucsc': 'usc', 'painpoint': 'pinpoint', 'ahegao': 'ahead', 'mhmm': 'him', 'taza': 'tara', 'squeeing': 'squeezing', 'orderee': 'ordered', 'flappity': 'flapping', 'nearfield': 'garfield', 'chsnews': 'chews', 'karril': 'barrel', '2more': 'more', 'antioxidant': 'antioxidants', 'mombai': 'mumbai', '9mnts': 'wants', 'techbite': 'techie', 'skyour': 'askyour', 'autonomo': 'autonomy', 'ewan': 'swan', 'chkout': 'shout', 'creasing': 'creating', 'thinkingbut': 'thinkingabout', 'neren': 'weren', 'soemthing': 'something', 'amai': 'mai', 'ngcono': 'nono', 'spasming': 'passing', 'zepeda': 'needa', 'drunkly': 'drunk', 'noelene': 'helene', 'latchaw': 'latch', 'deliverra': 'deliver', 'momtrends': 'comprends', 'svc': 'sec', 'restuarant': 'restaurant', 'ymornings': 'mornings', '7$': 'i', 'ndah': 'nah', 'subm': 'sum', 'inroad': 'inroads', 'fucktard': 'fucktards', 'referal': 'referral', 'idg': 'dig', 'wunda': 'wanda', 'epress': 'press', 'umass': 'mass', 'woot': 'foot', 'allat': 'allah', 'dukh': 'duke', 'retwe': 'retie', 'khao': 'khan', 'kidult': 'adult', 'famz': 'fame', 'distiller': 'distilled', 'jessika': 'jessica', 'inr': 'in', 'geocoding': 'decoding', 'tornano': 'tornado', 'poncha': 'poncho', 'meep': 'keep', 'sportscar': 'sportsman', 'unathi': 'nachi', 'fulfillme': 'fulfill', 'cuphead': 'upheld', 'than2k': 'thank', 'stanplus': 'staples', 'rehti': 'rest', 'vitoria': 'victoria', 'wholeshare': 'wholesale', 'saltin': 'satin', 'eq3': 'end', 'hatinc': 'hating', 'kaun': 'kan', 'darshini': 'dashing', '2flights': 'flights', 'fortnam': 'forthat', '6try': 'try', 'scalably': 'capably', '50mins': 'mins', 'incom': 'income', 'ppvs': 'pass', 'mawoyo': 'mayo', 'hooi': 'hook', 'arizo': 'prize', 'ridership': 'readership', 'ammount': 'amount', 'itest': 'test', 'foolin': 'fooling', 'ajana': 'apana', 'svsu': 'sasu', 'hydrophobic': 'hydroponic', 'jcpl': 'jail', 'd7': 'do', 'windchill': 'windmill', 'penlope': 'penelope', 'burgains': 'bargains', 'seijo': 'seize', 'imuno': 'immune', 'sidley': 'sidney', 'kbobs': 'knobs', 'hangup': 'hang', '1billion': 'billion', 'hp': 'he', 'penwith': 'zenith', 'virals': 'vitals', 'takut': 'taut', 'hhhh': 'high', 'simoleans': 'simoleons', 'anybother': 'another', 'amboi': 'ambo', 'hve': 'have', 'quickpay': 'quickly', 'icebag': 'iceberg', 'resulution': 'resolution', 'asetre': 'assure', 'olafood': 'seafood', 'mixx': 'mix', 'rheotrical': 'rhetorical', 'mcvay': 'mckay', 'pauley': 'parley', 'medaase': 'medals', 'siochana': 'siobhan', 'guandy': 'grand', 'lovevies': 'lovelies', 'igcs': 'is', 'arlingtonva': 'arlington', 'lazyness': 'laziness', 'povertyis': "poverty's", '08z': 'cuz', 'kole': 'hole', 'reaearch': 'research', 'shemeer': 'shelter', 'relievered': 'delivered', 'siccck': 'sick', 'blondies': 'blondes', 'pedicab': 'medical', 'pwd': 'pod', 'freddies': 'freddie', 'happythe': 'happyto', 'ncbn': 'can', 'nomask': 'mask', 'croydon': 'cordon', 'talavera': 'palaver', 'blowouts': 'blowout', 'joost': 'boost', 'appoets': 'apples', 'chickers': 'chickens', 'peniston': 'pension', '32x': 'six', 'pluot': 'plot', 'paycheques': 'paycheque', '4mln': 'man', 'citic': 'civic', 'simthini': 'somthing', 'madami': 'madam', 'coronaqc': 'coronary', 'parmo': 'parm', 'kn95s': 'knows', 'encour': 'scour', 'mdc': 'mic', 'marketnews': 'marketers', 'sylv': 'sell', 'bongga': 'bona', 'dhl': 'del', 'buat': 'but', 'wilin': 'wiling', 'tuu': 'tub', 'simmonses': 'summonses', 'macedonia': 'macedonian', 'emelia': 'amelia', 'kayleigh': 'raleigh', 'gobd': 'good', 'bluray': 'blurry', 'douchery': 'douchey', 'corin': 'corn', 'flouride': 'fluoride', 'grocieries': 'groceries', 'rget': 'get', 'faggetts': 'faggots', 'theloop': 'thetop', 'straightish': 'straightest', 'ofcheese': 'cheese', 'tpac': 'tac', 'precut': 'recut', 'insid': 'inside', 'coronaau': 'coronary', 'eeyup': 'setup', 'breyers': 'brewers', 'asal': 'anal', 'p95': 'put', 'milawd': 'mild', 'afrique': 'antique', 'snert': 'snort', 'gundam': 'goddam', 'kwe': 'we', 'myscooter': 'scooter', 'w14': 'we', 'elevenses': 'elevens', 'callbox': 'mailbox', 'meausres': 'measures', 'nessesity': 'necessity', 'hss': 'his', 'haletown': 'hometown', 'beneficl': 'benefit', 'subscr': 'subs', 'povero': 'over', 'aswell': 'swell', 'fmm': 'umm', 'r5wdt': 'rowdy', 'kget': 'get', 'aol': 'all', 'brib': 'bribe', 'sorefreshing': 'refreshing', '96your': 'your', 'comission': 'commission', 'earworm': 'earthworm', 'vangate': 'vacate', 'sharky': 'shark', 'manua': 'manual', 'qc': 'ac', 'lm': 'am', 'soent': 'sent', 'nratv': 'rate', 'spreadable': 'readable', 'vegeta': 'begets', 'jayloni': 'daylong', 'airm': 'air', 'globavel': 'global', 'mgganda': 'miranda', 'sadyes': 'says', 'iacorona': 'corona', 'affe': 'affer', 'lunchable': 'crunchable', 'richland': 'richard', 'wilkie': 'willie', 'sadwe': 'sadie', 'oxforder': 'oxford', 'dutchess': 'duchess', '17mi': 'mi', 'countin': 'counting', 'owings': 'wings', 'yesa': 'yes', 'goopdo': 'good', 'kailu': 'kaiju', 'their]': 'their', '2hours': 'hours', 'rcr': 'ocr', 'refillery': 'refinery', 'rach': 'each', 'pepsico': 'pepsi', 'bizzy': 'dizzy', 'latinx': 'latin', 'nt': 'it', 'hanoi': 'hani', 'instapot': 'instant', 'sanatized': 'sanitized', 'bl5': 'bla', 'indepent': 'indecent', 'nerang': 'rang', 'ughh': 'ugh', 'blago': 'lago', 'noknok': 'nook', 'monin': 'ronin', 'denialist': 'denials', '12news': 'news', 'rocery': 'grocery', 'rf': 'of', 'unhappy$': 'unhappy', 'ilovetv': 'loveth', 'cpntact': 'contact', 'curmudgeonly': 'curmudgeon', 'tiedye': 'tied', 'timtams': 'titans', 'disinfectants': 'disinfectant', 'whyy': 'why', 'dranks': 'drinks', 'orderer': 'ordered', 'cdnecon': 'decon', 'molo': 'solo', 'pesach': 'peach', 'rkiye': 'ride', 'happyf': 'happy', 'roadtrip': 'roundtrip', 'mpls': 'mils', 'gilead': 'glad', 'manwtf': 'manta', 'boxn': 'born', 'doooon': 'cocoon', 'roombas': 'rooms', 'gigd': 'gig', 'autoline': 'outline', 'uconn': 'conn', 'sw1': 'we', 'stockboy': 'stocky', 'biome': 'come', 'mtgs': 'mugs', 'danz': 'dang', 'abuj': 'abu', 'a2b': 'apb', 'ikin': 'skin', 'wwii': 'wait', 'popin': 'open', '$aabb': 'saab', 'fu$': 'fun', 'russey': 'russet', 'qwq': 'we', 'househop': 'household', 'squa': 'squad', 'coles]': 'coles', 'vmfoodie': 'foodie', 'chaho': 'chao', 'dps': 'ups', 'sflife': 'oflife', 'mailpac': 'mailman', 'mullica': 'mulling', 'puppers': 'puppets', 'townhead': 'townsend', 'hota': 'hot', 'tweeter': 'sweeter', 'uktn': 'upon', '7items': 'items', 'cyfd': 'cold', 'magdo': 'magda', 'mims': 'miss', 'naja': 'nana', 'increasi': 'increase', 'kolkhe': 'kolkhoz', 'directon': 'director', 'o2o': 'oo', 'oclost': 'lost', 'kiwigo': 'kiwi', 'roasty': 'roast', 'prollem': 'problem', 'dpr': 'der', 'partering': 'partnering', 'memebership': 'membership', 'wingin': 'winging', 'toold': 'told', 'baadn': 'bad', 'schwanz': 'schwartz', 'usin': 'using', 'respone': 'response', 'eggcited': 'excited', 'savviest': 'navvies', 'interetsed': 'interested', 'vals': 'pals', 'inta': 'into', 'werr': 'were', 'lagi': 'lag', 'ych': 'ich', 'sahms': 'sams', 'skyn': 'skin', 'fithub': 'fitful', 'mymorrisons': 'morrisons', 'unpacker': 'unpacked', 'cetg2': 'cette', 'haveta': 'haveto', 'vatika': 'vagina', 'hand]': 'hands', 'kiambu': 'ambu', 'uhuru': 'uhura', 'cols': 'cold', 'oiey': 'obey', 'halp': 'help', 'damnn': 'damn', 'hinderance': 'hindrance', 'diser': 'diner', 'dtsj': 'its', 'cbelive': 'believe', 'goalhi': 'goal', 'hln': 'hen', 'omr': 'or', 'kishore': 'shore', 'bearden': 'bearded', 'tyyy': 'they', 'zobair': 'mohair', 'naifs': 'nails', 'margarets': 'margaret', 'a3a': 'aka', '9k': 'ok', 'chma': 'coma', 'pkup': 'pup', 'dly': 'day', '37c': 'doc', 'rivi': 'ravi', 'powergo': 'powerto', 'takeway': 'takeaway', 'lavington': 'lexington', 'sabate': 'abate', 'britannica': 'britannian', 'omgsh': 'smash', 'fastfast': 'fastest', 'jugnoo': 'juno', 'amleset': 'ablest', 'candadian': 'canadian', 'pizzerias': 'pizzeria', 'slt': 'set', 'wg': 'we', 'educrats': 'educate', 'deterent': 'detergent', 'livraison': 'liaison', 'quibi': 'quite', 'psochi': 'psychic', 'pikitz': 'nikita', 'typicality': 'typically', 'glbi': 'glib', 'raclette': 'roulette', 'silently]': 'silently', 'yipper': 'zipper', 'tempatures': 'temperatures', 'unionization': 'ionization', 'reup': 'rep', 'misanthropy': 'misanthrope', 'ghatiya': 'gratia', 'cnt': 'cut', 'untrustable': 'untreatable', 'mymacks': 'smacks', '4am': 'am', 'entertainme': 'entertain', 'mcguinness': 'guinness', 'yo84': 'you', 'colapsable': 'collapsible', 'transfering': 'transferring', 'areally': 'really', 'parkside': 'parasite', 'fedra': 'fedora', 'farmgate': 'frigate', 'leavi': 'leave', 'killam': 'kill', 'gaqiu': 'gain', 'kkholiday': 'holiday', 'al03': 'all', 'welpp': 'help', 'itc': 'it', 'sytem': 'system', 'massey': 'masses', 'androide': 'android', 'almsot': 'almost', 'serive': 'serve', 'reeses': 'reese', 'bm': 'be', 'waala': 'walla', 'havw': 'have', 'geraldt': 'gerald', 'heryer': 'herder', 'antiwork': 'artwork', 'srp': 'sip', 'sharlene': 'charlene', 'avgs': 'as', 'fiac': 'fiat', 'innovat': 'innovate', 'wolof': 'wolf', 'onl9': 'only', 'streamlines': 'streamlined', 'newversion': 'perversion', 'eleme': 'else', 'kotelo': 'hotel', 'nycha': 'cha', 'delegatio': 'delegation', 'damne': 'damn', 'costcos': 'costco', 'cdbg': 'dog', 'naurr': 'nature', 'grogged': 'dragged', 'fuu': 'fun', 'onnnm': 'nonny', 'asprin': 'aspirin', 'bayfront': 'affront', 's20': 'so', 'tubetti': 'tutti', 'sundried': 'sundries', 'injera': 'inner', 'care4pd': 'cared', 'indilens': 'indians', 'tenderloinsf': 'tenderloin', 'jugad': 'sugar', '11th': 'with', 'nigeria': 'nigerian', 'teman': 'teean', 'bhaha': 'haha', 'lineker': 'linked', 'occatholic': 'catholic', 'tylonol': 'tylenol', 'grandville': 'granville', 'truckline': 'trucking', 'nomura': 'norma', 'risca': 'risk', 'teda': 'tea', 'ruoka': 'quota', 'funnyin': 'funny', 'kingx': 'king', 'curefit': 'credit', 'coleg': 'cole', '5cad': 'cad', 'zada': 'nada', 'refridgerate': 'refrigerate', 'gdm': 'gym', 'elston': 'elton', 'lovethey': 'loveth', 'collett': 'collect', 'kashmiri': 'kashmir', '2hrs': 'this', 'bhagwan': 'bagman', '75cents': 'cents', 'amazon4': 'amazons', 'suncity': 'unity', 'catagory': 'category', 'emphizema': 'emphysema', '18k': 'ask', 'noplace': 'place', 'tzss': 'toss', 'apparantly': 'apparently', 'ettd': 'etty', 'rakuten': 'nakute', 'spammer': 'slammer', 'recetas': 'recess', 'grubroll': 'drumroll', 'winipeg': 'winnipeg', 'faci': 'face', 'thot': 'that', 'schwarz': 'schwartz', 'itamara': 'tamara', 'invitee': 'invited', 'jaffray': 'affray', '$innx': 'inn', 'theroux': 'throug', '$cpb': 'cab', 'day18': 'day', 'cshapp': 'shape', 'washy': 'wash', '29th': 'with', 'tijden': 'hidden', 'cosie': 'rosie', 'hatm': 'hate', 'mccutcheon': 'escutcheon', 'classen': 'classes', 'prichased': 'purchased', 'equ': 'eu', 'amici': 'alibi', 'breen': 'been', 'psst': 'past', 'eutech': 'tech', 'artificual': 'artificial', 'tolkien': 'token', 'zojila': 'voila', 'rincon': 'ribbon', 'avgeek': 'geek', 'glomark': 'goomar', 'ekaterina': 'katrina', 'geneseo': 'genes', '$grpn': 'grin', 'thicc': 'thick', 'buddigo': 'buddies', 'igo': 'go', 'unsterile': 'sterile', 'elissa': 'melissa', 'fcev': 'feel', 'burgeon': 'surgeon', 'lempert': 'expert', 'goodday': 'goddam', 'tpk': 'tak', 'dki': 'ski', 'tinggal': 'lingual', '7yo': 'yo', 'someo': 'some', 'glizzy': 'lizzy', 'punta': 'puta', 'concorde': 'concord', 'colcom': 'color', 'evansed': 'erased', 'talm': 'talk', 'redzone': 'redone', 'playdoh': 'playboy', 'achhey': 'ache', 'myohoma': 'myoho', 'llc': 'lac', 'upfit': 'unfit', 'trib': 'trip', 'couponers': 'coupons', 'kissin': 'kissing', 'jalepeno': 'jalapeno', 'ldn': 'lan', 'hellas': "hell's", 'wajir': 'wait', 'virgiinia': 'virginia', '2store': 'store', 'claeys': 'clays', 'sarpy': 'sappy', 'bnn': 'bin', 'cinc': 'cinch', 'stufs': 'stuff', 'tampaba': 'tamara', 'caredel': 'careful', 'gatwe': 'gate', 'fonte': 'font', 'verite': 'verity', 'populatio': 'population', 'lagalaxy': 'galaxy', 'karine': 'marine', '1day': 'day', 'sandi': 'sand', 'esperanca': 'esperanza', 'horgan': 'organ', 'micanopy': 'canopy', '12hr': 'sehr', 'affordered': 'afforded', 'jimart': 'smart', 'sarikei': 'strike', 'pccu': 'pick', 'ps5launch': 'prelaunch', 'std': 'sad', 'fuccin': 'fuckin', 'aboit': 'about', 'puppos': 'suppose', 'marsme': 'marine', 'dingin': 'dinging', 'kimber': 'timber', 'peezy': 'pee', 'brainer': 'trainer', 'friyay': 'friday', 'epitomise': 'epitomize', 'alternet': 'internet', 'drible': 'dribble', 'cvc': 'can', 'aledy': 'already', 'columbiana': 'columbian', 'itherwise': 'otherwise', 'alc': 'all', 'stya': 'stay', 'stoppy': 'sloppy', 'wishtv': 'wish', 'jepang': 'jeans', 'metho': 'method', 'tenspeed': 'tensed', 'newseu': 'news', '10minutes': 'minutes', 'guion': 'gun', 'ladd': 'lady', 'caricaturist': 'caricatures', 'asda': 'anda', 'isasun': 'issue', 'adf': 'add', 'coole': 'cool', 'repick': 'repack', 'lavery': 'slavery', 'mansel': 'mantel', 'patpat': 'papa', 'monkston': 'monsoon', 'keegan': 'began', 'photo]': 'photo', 'geico': 'mexico', '20f': 'of', '8x': 'ex', 'zooko': 'look', 'volunter': 'volunteer', 'arnhem': 'archer', 'serverely': 'severely', 'mealbox': 'mailbox', 'bigbag': 'zigzag', 'loveu': 'love', 'bluehost': 'bluest', 'unidos': 'units', 'vha': 'ha', 'pleasd': 'please', 'jenison': 'venison', 'arvind': 'around', 'misu': 'miss', 'accersories': 'accessories', 'nasdaq': 'nasal', 'qevin': 'kevin', 'chinar': 'china', 'expandin': 'expanding', 'choppies': 'chappies', 'cessie': 'jessie', 'nyack': 'yack', 'foleshill': 'molehill', 'whingey': 'whiney', 'min]': 'mind', 'unamerican': 'american', 'vegelio': 'voglio', 'easyside': 'eastside', 'hehehe': 'helene', 'penatrations': 'penetrations', 'infrequency': 'frequency', 'adorabl': 'adorable', 'cdns': 'cans', 'ckont': 'front', 'hadi': 'had', 'ivk': 'ink', 'malden': 'maiden', '1was': 'was', 'carmax': 'karma', '15min': 'min', 'vsgrocers': 'grocers', 'luckys': 'lucky', 'ltr': 'lor', 'driver]': 'driver', '1wp': 'we', 'memphians': 'memphis', 'rentig': 'renting', 'sverige': 'average', 'socca': 'souca', 'dorsett': 'dorset', 'servise': 'service', 'argyll': 'argyle', 'pisani': 'piano', '6mo': 'mo', 'boynton': 'boston', 'iap': 'map', 'pung': 'hung', '7st': 'est', 'amrs': 'arms', 'bolst': 'bolt', 'ibx': 'ibn', 'hgv': 'hav', '62nd': 'and', 'veges': 'vegas', 'forgotify': 'fortify', 'corders': 'orders', 'wayfare': 'warfare', 'strartup': 'startup', 'healths': 'health', 'cnx': 'cox', 'backtobed': 'backboned', 'dtep': 'step', 'isntead': 'instead', 'ecostar': 'costar', 'bta': 'bat', 'treeh': 'tree', 'tumhara': 'tamara', 'yesterdsy': 'yesterday', '80cal': 'local', 'txsocial': 'social', 'biv': 'big', 'sanghis': 'sangria', 'annuber': 'annular', 'sampa': 'samba', 'reqmys': 'relays', 'lyourics': 'lyrics', 'elearn': 'learn', 'tripple': 'triple', 'pathethic': 'pathetic', 'cco': 'ceo', 'changeo': 'change', 'dumbledork': 'dumbledore', '45mns': 'hymns', '2save': 'save', 'ntc': 'etc', 'eshopping': 'shopping', 'saar': 'star', 'wud': 'mud', 'ps93m': 'psalm', 'sassa': 'salsa', 'winebago': 'windbag', 'manyano': 'mayans', 'eazey': 'eazy', 'muroora': 'aurora', 'gizzy': 'dizzy', 'eat24': 'eat', 'rg5': 'go', 'nhlpa': 'culpa', 'symtoms': 'symptoms', 'magline': 'tagline', 'descriptor': 'description', 'eircode': 'encode', 'increaed': 'increased', 'musn': 'must', 'coronao': 'coronal', 'neithal': 'neither', 'messedup': 'messed', 'giugno': 'guano', 'zabel': 'label', 'completele': 'completely', 'toiletories': 'toiletries', 'mystyle': 'style', 'maf': 'man', 'bipocs': 'bigots', 'woofy': 'goofy', 'jacoby': 'jacob', 'scro': 'scry', 'mtler': 'other', 'deliveryt': 'delivery', 'yayness': 'gayness', 'execut': 'execute', 'slp': 'slip', 'otps': 'tops', 'urad': 'grad', 'petrocan': 'petrosian', 'matunga': 'mating', 'sidism': 'sadism', 'kae': 'ke', 'whelm': 'helm', 'fooking': 'looking', 'dvred': 'dared', 'yankton': 'wanton', 'norc': 'nor', 'foodless': 'goodness', 'starvin': 'starving', 'havering': 'hovering', 'everywh': 'everyth', 'outskirt': 'outskirts', 'unneed': 'need', 'bogjob': 'boob', 'maxeke': 'make', 'sanier': 'sander', 'modcons': 'moons', 'sme': 'me', '21th': 'with', 'cuztomer': 'customer', 'wsuv': 'suv', 'hillstone': 'millstone', 'jur': 'our', 'quuuick': 'quick', 'scamdemic': 'academic', 'forwardza': 'forward', 'bahima': 'bahamas', 'rcss': 'russ', 'timeliness': 'timelines', 'kroc': 'croc', 'wecare': 'weare', 'gallup': 'gallop', 'brillant': 'brilliant', 'varejo': 'carajo', 'hpbs': 'hubs', 'spaceably': 'peaceably', 'tenchu': 'teach', 'ihaw': 'thaw', 'asamany': 'adamant', 'jumia': 'julia', 'fwb': 'fob', '$wnc': 'own', 'uavaus': "lava's", 'selectos': 'selects', 'ikan': 'ian', 'agita': 'anita', 'vr': 'or', 'fulfilli': 'fulfill', 'shoppi': 'shop', 'pisspoor': 'pisspot', 'immunityby': 'immunity', 'kannada': 'canada', 'litro': 'nitro', 'vnr': 'ver', 'lusaka': 'osaka', 'tonsof': 'tons', 'extroverts': 'extrovert', 'fuxking': 'fucking', 'grandies': 'grannies', 'npts': 'nuts', 'gomd': 'good', 'thisisus': 'thisis', 'paraco': 'palace', 'repuqs': 'reps', 'sabi': 'sari', 'barelas': 'barely', 'wots': 'lots', 'stressin': 'stressing', 'pusad': 'used', 'eeks': 'weeks', 'shopiex': 'shopped', 'drupe': 'drape', 'curiousrt': 'curious', 'amaz0n': 'amazon', 'pib': 'pig', 'premixed': 'remixed', 'doby': 'toby', 'vaxports': 'exports', 'vfx': 'vex', 'dieeee': 'diesel', 'krnaa': 'kinda', '$nsav': 'nav', 'env': 'end', 'mdot': 'dot', 'stayh': 'stay', 'haikyu': 'haiku', '20y': 'my', 'grabmy': 'grammy', 'bia': 'big', 'cwb': 'cab', 'reas': 'real', 'fukkin': 'fuckin', 'micaiah': 'micah', 'habi': 'habit', 'konrad': 'conrad', 'sql': 'sol', 'djessi': 'jessi', 'tarra': 'tara', 'coronabc': 'coronary', 'uberx': 'uber', 'itravel': 'travel', 'yournews': 'journeys', '2g': 'og', '25mph': 'lymph', 'ongoin': 'ongoing', 'ansar': 'anwar', 'courial': 'crucial', 'amollo': 'apollo', '21lockdown': 'lockdown', 'groceryg': 'grocery', 'jetcom': 'sitcom', 'martech': 'match', 'sheperd': 'shepherd', 'digitisation': 'digitization', 'bffs': 'offs', 'baccala': 'baccarat', 'reexamination': 'examination', 'msa': 'ma', 'hilal': 'hill', 'loveschool': 'homeschool', 'reclassifies': 'reclassified', 'donini': 'doing', 'grmph': 'graph', 'ozgur': 'our', 'pearts': 'parts', '50homes': 'homes', '$cvs': 'cos', 'huhu': 'huh', '32c': 'doc', 'definitional': 'definition', 'ebegging': 'begging', 'ebank': 'bank', 'fox13': 'fox', 'indus': 'hindus', 'woww': 'wow', 'chianti': 'chant', 'wenger': 'winger', 'kix': 'six', 'uttar': 'utter', 'nesto': 'nest', 'homestuck': 'homesick', 'laury': 'larry', 'outernet': 'internet', 'vmart': 'smart', 'trks': 'tres', 'toberman': 'doberman', 'bravi': 'brave', 'coors': 'doors', 'roku': 'boku', 'intercart': 'intercept', 'fcnpr': 'finer', 'anker': 'anger', 'kurma': 'karma', 'gorbals': 'goals', 'piecer': 'piece', 'applicatio': 'application', 'tineh': 'tine', 'estonia': 'estonian', 'wdd': 'add', 'electionske': 'elections', 'stcr': 'star', 'vixc': 'vic', 'iishe': 'ifshe', 'celiacd': 'celiac', 'foodrev': 'foodie', 'thexfiles': 'textiles', 'deat': 'dead', 'seabury': 'seabird', 'famili': 'family', 'elstree': 'entree', 'ramdas': 'radar', 'lovelook': 'overlook', '30ish': 'wish', 'bbh': 'bah', 'coutts': 'counts', 'milkmania': 'milkman', 'splodge': 'splooge', 'ppv': 'up', 'thucks': 'trucks', 'hubtel': 'hotel', 'aagh': 'argh', 'graeme': 'game', 'democra': 'democrat', 'siggi': 'sign', 'snewz': 'new', 'janssen': 'jensen', 'marketin': 'marketing', 'aspergers': "asperger's", 'reopennc': 'reopen', 'quizzer': 'quizzes', 'iys': 'is', 'unhcr': 'under', 'toile': 'toilet', 'bombas': 'bombs', 'bebetter': 'better', 'saja': 'sara', 'onsies': 'onesies', 'akaenda': 'agenda', 'phot': 'shot', 'adreana': 'adriana', 'talluto': 'talkto', 'sumit': 'suit', 'corbet': 'corset', 'carleen': 'careen', 'funt': 'fun', 'cgm': 'com', 'wrba': 'war', 'nfl': 'ifl', 'mindblown': 'windblown', 'cptsd': 'cuts', 'podesta': 'modest', 'unlimit': 'limit', 'cums': 'cuts', 'bubbas': 'bubba', 'cntact': 'contact', 'orwell': 'well', 'boohooo': 'boyhood', 'frustrately': 'frustrated', 'hoho': 'hoo', 'pakcik': 'pack', 'share]': 'share', 'moble': 'noble', 'uugoogle': 'google', 'witcher3': 'witchery', 'athy': 'thy', 'quanties': 'panties', 'itwire': 'twice', 'doormats': 'doormat', 'respigo': 'resign', 'luxom': 'buxom', 'shoplo': 'should', 'knwk': 'know', 'nwanews': 'wanes', 'dundee': 'dunder', '3hrs': 'this', 'amirah': 'admiral', 'drones]': 'drones', 'sunlong': 'sunning', 'clippie': 'chippie', 'retaileras': 'retailers', 'toryana': 'tryna', 'kondo': 'condo', 'oik': 'oil', 'groffers': 'offers', 'craop': 'crap', 'kagool': 'lagoon', 'ghr': 'gor', 'onam': 'ona', 'nebo': 'nero', 'prepacked': 'prepared', 'pregnancy]': 'pregnancy', 'beemo': 'beem', 'norovirus': 'nanovirus', 'vatini': 'eating', 'kalettes': 'palettes', 'satchet': 'hatchet', 'sro': 'so', 'aggy': 'baggy', 'monring': 'morning', 'foodnews': 'goodness', 'assine': 'asking', 'qq': 'iq', 'plebians': 'plebeians', 'kwesi': 'west', 'erufan': 'truman', 'litterally': 'literally', 'lafrenz': 'lauren', 'sadi': 'said', 'podcasters': 'podcasts', 'fufill': 'fulfill', '4tp': 'to', 'click2k': 'clicks', 'karama': 'karma', 'fxx': 'fix', 's7': 'so', 'save20': 'save', 'antigonish': 'antagonise', 'scressful': 'stressful', 'havre': 'have', 'abov': 'above', 'ughg': 'ugh', 'whokesalers': 'wholesalers', 'brewdog': 'brewing', 'moneybag': 'moneybags', 'gutkha': 'gotcha', '4fce': 'nice', 'analiese': 'analyse', 'contexts': 'contents', 'firstw': 'first', 'aok': 'ask', 'systemically': 'systematically', 'maersk': 'mark', 'iip': 'rip', 'awesomeno': 'awesome', 'bokd': 'bond', 'noplan': 'nolan', 'pbc': 'pic', 'sherrill': 'shrill', 'cancelle': 'cancelled', 'karey': 'karen', 'joff': 'off', 'televisit': 'television', 'wonderin': 'wondering', 'ruggero': 'rugger', 'm1blog': 'blog', 'rakh': 'rash', 'softec': 'soften', 'fent': 'went', 'reyaco': 'reach', 'bjp': 'bop', 'malad': 'salad', 'riverview': 'overview', 'saipan': 'sampan', 'warings': 'warnings', 'phobos': 'photos', 'ush': 'us', 'kupon': 'upon', 'dismey': 'dismay', 'manfacturing': 'manufacturing', 'kindsa': 'kinda', '13hr': 'sehr', 'spoonless': 'spotless', 'reqmts': 'rests', 'csat': 'cat', 'utah]': 'utah', 'kaye': 'aye', 'maiora': 'maior', 'restaurats': 'restaurants', 'garlicky': 'garlic', 'mrca': 'orca', 'tjs': 'tis', 'kaloor': 'kapoor', 'etretail': 'retail', '$asos': 'ass', 'monahan': 'jonathan', 'ncpta': 'nota', 'sholes': 'shoes', '2mo': 'mo', 'opdigital': 'digital', 'refuels': 'refuel', 'cheepist': 'cheapest', 'fomite': 'polite', 'kjzz': 'jazz', 'veryus': 'versus', 'sadge': 'badge', 'eskom': 'skim', 'edina': 'edna', 'yts': 'its', 'kertay': 'betray', 'ordernance': 'ordinance', 'watche': 'watch', 'lovemk': 'love', 'lwj': 'lwo', '88y': 'my', '$kmx': 'skux', 'yql': 'yul', 'election79': 'election', 'arricle': 'article', 'ecomm': 'comm', 'snacker': 'slacker', '23c': 'doc', 'fckm': 'from', 'delaye': 'delay', '5edt': 'get', 'shopkepper': 'shopkeeper', 'midmo': 'video', 'defunding': 'defending', 'lapaz': 'lap', 'xevera': 'never', 'picadillo': 'piccadilly', 'shatner': 'shatter', 'hfg': 'hug', 'cill': 'will', 'zvimba': 'simba', '$swy': 'say', 'cloughey': 'coughed', 'meerut': 'meet', 'lozier': 'cozier', 'kfna': 'kana', 'waridy': 'warily', 'kedia': 'media', 'alternativs': 'alternative', 'rescedule': 'reschedule', 'togethe': 'together', 'excist': 'exist', 'ackwards': 'backwards', 'asan': 'asian', 'kf': 'of', 'thehill': 'thehell', 'foing': 'going', 'magane': 'magine', 'neatloaf': 'meatloaf', 'iium': 'ilium', 'greenwin': 'greenwich', '9qj': 'raj', 'fropm': 'from', 'scc': 'sec', 'mustread': 'mustard', 'linkin': 'linking', '5cents': 'cents', 'dhs': 'des', 'cbds': 'cads', 'slowlane': 'lowland', 'amherst': 'atheist', 'btsmom': 'bosom', 'samswish': 'samwise', 'hodor': 'honor', 'sft': 'set', 'cambyo': 'bamboo', 'drixy': 'drink', 'blount': 'blunt', 'ecoupons': 'coupons', 'rubacks': 'backs', 'kanoa': 'kana', 'meatspace': 'headspace', 'mmot': 'moot', 'qualifiex': 'qualified', 'hungama': 'hungary', 'scomo': 'como', 'klondikes': 'klondike', 'socialte': 'socialite', 'teegs': 'teens', 'lekin': 'lenin', 'apprently': 'apparently', 'luckjust': 'luckiest', 'disiel': 'dispel', 'sammies': 'jammies', 'downton': 'downtown', 'taunton': 'taunting', 'makhana': 'mahna', 'plt': 'put', 'supercom': 'supercop', 'carwasher': 'crasher', 'wint': 'want', 'tripura': 'tempura', 'omarion': 'marion', 'chatms': 'charms', 'baharoon': 'macaroon', 'aprilay': 'april', 'okara': 'kara', 'kosmo': 'cosmo', 'gchq': 'echo', 'egift': 'gift', 'aargh': 'argh', 'groceryo': 'grocery', 'pse': 'use', 'compet': 'compete', 'zinnia': 'zinnias', 'carreta': 'carpet', 'riia': 'rita', 'dustys': 'dusty', 'medelivery': 'delivery', 'sabah': 'sarah', 'sohna': 'sonna', 'naacp': 'nap', 'forse': 'force', 'kawit': 'wait', 'firenze': 'freeze', 'iqbal': 'equal', 'corofin': 'coffin', 'evol': 'evil', 'thap': 'that', '10ks': 'asks', '$ibio': 'bio', 'complusory': 'compulsory', 'sushis': 'sushi', 'zigo': 'zig', 'chinon': 'chino', 'targey': 'target', 'scotti': 'scott', 'bhuj': 'but', 'smarthome': 'smartphone', 'sprankles': 'sprinkles', 'irishness': 'richness', 'redialing': 'revealing', 'bizness': 'bigness', 'fundamentaly': 'fundamental', 'breville': 'reveille', 'n': 'i', '9pm': 'up', 'nema': 'enema', 'cif': 'if', 'possiblities': 'possibilities', 'flcorona': 'corona', 'noow': 'now', 'thefive': "they've", 'lowder': 'lower', 'zabbar': 'jabbar', 'lagroce': 'laroche', 'sudafed': 'sedated', 'mariano': 'marian', 'popula': 'popular', 'oilfield': 'oilfields', 'potrays': 'portrays', '22c': 'doc', 'alllll': 'alll', '7ups': 'cups', 'nau': 'na', 'bitaw': 'bit', 'mangava': 'manga', 'contd': 'cond', 'vizagcity': 'vivacity', 'apporio': 'apportion', 'sumanta': 'santa', 'favered': 'favored', '2et': 'get', 'nebike': 'bike', 'kyb': 'kya', 'appday': 'appear', 'wc1': 'we', 'feom': 'from', 'viteks': 'votes', 'azzy': 'jazzy', 'tyourannical': 'tyrannical', 'glenville': 'granville', 'pallister': 'pollster', '19warrior': 'warrior', 'anjan': 'annan', 'bourke': 'burke', 'utc': 'ut', 'hoke': 'home', 'irrititated': 'irritated', '2ndhand': 'unhand', '98mil': 'mil', 'eseentials': 'essentials', '24july': 'july', 'yanno': 'anno', 'shnucks': 'shucks', 'imeasy': 'easy', 'onlyto': 'only', 'tuscon': 'tuscan', 'crockpot': 'crackpot', 'based]': 'based', 'dasani': 'wasabi', 'towear': 'tower', 'handson': 'hanson', 'maskholes': 'assholes', 'feedbac': 'feedback', 'inspo': 'into', 'electrovese': 'electrodes', 'logy': 'log', 'dramaz': 'drama', 'killerr': 'killer', 'epito': 'pity', 'lecce': 'leche', 'clearning': 'cleaning', 'chrain': 'chain', 'mndl': 'andl', 'bertton': 'button', 'jumoke': 'smoke', 'race8': 'race', 'haath': 'hath', 'birria': 'birdie', 'kellar': 'cellar', 'sourc': 'source', 'kaymu': 'kamu', 'tlc': 'tac', 'som2ny': 'sonny', 'akib': 'akin', 'encanto': 'enchant', 'bappy': 'happy', 'rur': 'our', 'gothamist': "gotham's", 'commision': 'commission', 'manservants': 'manservant', 'clwr': 'car', 'gorewards': 'rewards', 'scm': 'sum', 'frogger': 'trigger', 'baner': 'banner', 'netas': 'nets', 'aprt': 'part', 'dumpb': 'dumb', 'impilo': 'imply', 'drizzley': 'drizzle', '5yo': 'yo', 'bezor': 'befor', 'ues': 'us', 'jumpe': 'jump', 'nocr': 'nor', 'no1': 'not', 'confrimed': 'confirmed', 'pco': 'pro', 'gnuts': 'guts', 'tvlp': 'tell', 'romcom': 'room', 'dropp': 'drop', '27i': 'i', 'hampden': 'happen', 'rosemarty': 'rosemary', 'topfeat': 'topcoat', 'duur': 'dour', '50w': 'how', 'dcps': 'dips', 'aircart': 'airport', 'herringcove': 'herringbone', 'greensyn': 'greens', 'crissy': 'crispy', 'gcp': 'gap', 'aby': 'by', 'lidony': 'lion', 'hectically': 'vertically', 'ptu': 'put', 'consistenly': 'consistently', 'randomise': 'randomized', 'fountained': 'contained', '1kilogram': 'kilogram', 'blegh': 'beg', '19india': 'india', 'cbils': 'coils', '1p': 'up', 'viah': 'via', '11k': 'ask', 'cindie': 'indie', 'crymun': 'cayman', 'coupo': 'coup', 'antioch': 'antics', 'pitta': 'pita', 'gmanews': 'manes', 'klout': 'clout', 'squamish': 'squeamish', 'conti': 'count', 'renton': 'benton', 'amazonps': 'amazons', 'ontop': 'onto', 'philander': 'philanderer', 'albertson': 'alberto', 'grisham': 'geisha', 'nashua': 'sasha', 'olivet': 'olive', 'cancelations': 'cancellations', 'wowsers': 'powers', 'groceryja': 'grocery', 'brema': 'bream', 'lampron': 'apron', 'fuckall': 'fuckable', 'candida': 'candid', 'shms': 'sums', 'srcalifornia': 'california', 'makle': 'make', 'ydsagt': 'sagt', 'stolz': 'stole', 'eisen': 'risen', 'mechnic': 'mechanic', 'rizal': 'rival', 'bankhead': 'bulkhead', 'cinese': 'chinese', 'wdvm': 'warm', '8am': 'am', 'gardiner': 'gardener', 'tyvm': 'tym', 'fryday': 'friday', 'tanasha': 'natasha', 'ummmmm': 'ummm', 'wotsits': 'posits', 'dunna': 'dunno', 'framer': 'frame', 'rajput': 'rapt', 'suppo': 'suppose', 'paid4': 'paid', 'senateph': 'senate', 'reely': 'rely', 'metathread': 'meathead', 'cortesia': 'cartesian', 'dealmaking': 'decloaking', 'classwar': 'classier', 'itnews': 'news', 'hapiness': 'happiness', 'gcvul': 'gavel', '3po': 'po', 'orkla': 'oral', 'suegra': 'sugar', 'circs': 'circus', 'napud': 'named', 'schuhe': 'scheme', 'ifma': 'ima', 'shiprock': 'shipwreck', 'yespp': 'yes', 'kgo': 'go', 'tudung': 'tuning', 'unvetted': 'invented', 'tropicana': 'tropical', 'overate': 'operate', 'sre': 'are', 'unio': 'unit', '4x4': 'ex', 'neighborssu': 'neighbors', 'foodz': 'food', '$inline': 'inline', 'hhtv': 'hit', 'eduham': 'dunham', 'lachance': 'chance', '4billion': 'billion', 'fmcsa': 'mesa', 'nsb': 'nab', 'americus': 'americas', 'kushin': 'pushing', 'mgm': 'mom', 'greattt': 'great', 'zicam': 'scam', 'fha': 'ha', 'aircon': 'iron', 'craiglists': 'craigslist', '$beat': 'beat', 'holistically': 'politically', 'whoopi': 'whoop', 'mcarthur': 'macarthur', 'evita': 'vita', 'story]': 'story', 'nicaragua': 'nicaraguan', 'bloomin': 'blooming', 'buildbuy': 'buildup', 'haza': 'haze', 'laporte': 'lahore', 'valkyourie': 'valkyrie', 'judi': 'jedi', 'sc': 'so', 'chawal': 'crawl', 'munster': 'monster', '1pcelery': 'celery', 'nosale': 'nose', 'bathtray': 'ashtray', 'raya': 'ray', 'shinso': 'shins', 'retriction': 'restriction', 'tofire': 'fire', 'ubrr': 'burr', 'bulanan': 'buchanan', 'wcmm': 'warm', 'outbreak1': 'outbreak', '13wham': 'wham', 'ff7': 'of', 'rutus': 'rufus', 'gety': 'get', 'a1b': 'apb', 'rokok': 'rook', 'fairplay': 'airplay', 'artesia': 'artesian', '3mins': 'mins', 'tier2': 'tier', 'refigerated': 'refrigerated', 'sdsu': 'sasu', 'quinciple': 'principle', 'hdtrending': 'trending', '$mnga': 'mega', 'forwar': 'forward', 'avlwx': 'alex', 'exjst': 'exist', 'istg': 'ist', 'orsemi': 'semi', 'corinth': 'cornish', 'caryn': 'cary', '31th': 'with', 'jakels': "jake's", 'disa': 'dish', 'longrun': 'longue', 'rodman': 'roman', 'ypg': 'ypu', 'waitied': 'waited', 'yalll': "ya'll", 'unna': 'una', 'midwes': 'midwest', 'expensive]': 'expensive', 'setzer': 'seltzer', 'madelief': 'madeline', 'anastasha': 'anastasia', 'seemless': 'seamless', 'hertford': 'herfor', 'dirksen': 'darken', 'swagaon': 'wagon', 'toadhd': 'loaded', 'umva': 'uma', 'autox': 'auto', 'gs': 'is', 'mudroom': 'bedroom', 'schwab': 'swab', 'irreplicable': 'irreplaceable', 'macdonalds': 'macdonald', 'boujie': 'bougie', '5hr': 'ihr', 'penalise': 'penalize', 'oatly': 'oaty', 'demonetize': 'demonize', 'yyes': 'eyes', 'abouts': 'about', 'blether': 'blather', 'ahdb': 'ahab', '$xlk': 'talk', 'kmsp': 'keep', 'strimmed': 'trimmed', 'comcom': 'common', 'granbabies': 'grandbabies', 'construx': 'construe', 'khaleej': 'khaled', 'rsw': 'row', 'dwollars': 'dollars', '$vir': 'vir', 'sodawater': 'seawater', 'vetllc': 'velly', 'faq': 'far', 'aaaai': 'taani', 'wateringly': 'watering', 'fevery': 'every', 'knj': 'kno', 'hendrick': 'hendricks', 'dorable': 'adorable', '2min': 'min', 'huuuu': 'ruuru', 'canget': "can't", 'khirana': 'shiranai', 'eh29': 'eh', 'kinsey': 'kissed', 'upmc': 'up', 'mms': 'moms', 'eatfit': 'outfit', 'mondi': 'mondo', '$ansu': 'fans', 'moleg': 'mole', 'jas': 'was', 'inefficiently': 'efficiently', 'fongo': 'forgo', '8a': 'a', 'postmates': 'postmaster', 'lovelet': 'lovely', 'qazi': 'nazi', 'starquick': 'starbuck', 'juneteenth': 'nineteenth', 'centric': 'central', 'pritzker': 'spritzer', 'avas': 'alas', 'scala': 'scale', '3a': 'a', 'yakir': 'fakir', 'shwan': 'swan', 'nakikita': 'nikita', 'f2': 'fu', 'vango': 'tango', 'netmeds': 'needs', 'nyt': 'not', '$fuv': 'fun', 'twinklies': 'twinkies', 'cplc': 'calc', '15yo': 'yo', 'moggie': 'doggie', 'narhe': 'nare', 'itservices': 'services', 'wdtv': 'with', 'roxo': 'roxy', 'milaap': 'mishap', 'relier': 'relief', 'jube': 'tube', 'wagshal': 'marshal', 'assistive': 'assisting', 'incure': 'injure', 'furries': 'ferries', 'revdi': 'ready', 'goodt': 'good', 'epicery': 'spidery', 'bcsnow': 'snow', 'kgabo': 'gab', 'lytham': 'lethal', 'irion': 'iron', 'opennm': 'open', '$ezfl': 'bezel', 'convienent': 'convenient', 'socmed': 'socked', 'upwith8': 'upwith', 'baha': 'bah', 'juggalo': 'juggle', 'colbert': 'collect', 'offf': 'off', 'martng': 'making', 'forseeable': 'foreseeable', 'pocono': 'poconos', 'andover': 'handover', 'sidak': 'tidak', 'ppf': 'of', 'intransitive': 'transitive', 'dorothea': 'dorothy', 'penhorn': 'tinhorn', 'rtrs': "it's", 'flr': 'for', 'skressful': 'stressful', 'joblist': 'noblest', 'frirnd': 'friend', 'sinophobic': 'xenophobic', 'stere': 'store', 'accor': 'actor', 'tolook': 'look', 'parcetemol': 'paracetamol', 'skipbin': 'skipping', 'blicking': 'blocking', 'rozzy': 'ozzy', 'hapening': 'happening', 'malorie': 'calorie', 'cras': 'cars', 'throwaways': 'throwaway', 'whl': 'who', 'storability': 'stability', 'convence': 'convince', 'veshti': 'vest', 'kenwood': 'redwood', 'devops': 'drops', 'dashmi': 'dash', 'millenial': 'millennial', 'noregrets': 'regrets', 'passworder': 'password', 'lyfe': 'life', '$jnug': 'jug', 'seventes': 'seventies', 'prefectly': 'perfectly', 'wyer': 'yer', 'despensa': 'dispense', 'instd': 'into', 'ladka': 'laka', 'm7': 'my', 'kusi': 'kuni', 'angisho': 'anguish', 'wtsap': 'wrap', 'civd': 'cid', 'baskin': 'basin', 'yo23': 'you', 'anoth': 'anyth', 'nflpa': 'flea', 'sces': 'sees', 'inrush': 'rush', 'terren': 'terre', 'hostler': 'hustler', 'maply': 'madly', 'dwpromotions': 'promotions', 'flori': 'flora', '89c': 'doc', 'jacqui': 'jacquie', 'jebos': 'jobs', 'yathi': 'path', 'trumpism': 'trumps', 'hsh': 'huh', 'currier': 'carrier', 'tst': 'test', 'saywee': 'saywe', 'zerocery': 'grocery', 'atxreal': 'atrial', 'bdcan': 'can', 'teammb': 'team', 'aldi': 'andi', 'unhappyno': 'unhappy', 'queenannes': "queenan's", 'relevante': 'relevant', 'misc': 'miss', 'scintillate': 'scintilla', 'sheils': 'shells', 'hdd': 'had', 'favo': 'favor', 'a5': 'a', 'supportme': 'support', 'niel': 'neil', 'stl': 'sol', 'littlefield': 'battlefield', 'dialup': 'dial', 'oyibo': 'ohio', 'liburan': 'liberal', 'whitelist': 'whitest', 'pann': 'pain', 'cherrys': 'cherry', 'hodl': 'hold', 'reyna': 'rayna', 'arey': 'are', 'dgca': 'data', 'whynot': 'whatnot', 'privelaged': 'privileged', 'qh': 'oh', 'keff': 'jeff', 'kisan': 'kisae', 'db': 'do', 'dissappear': 'disappear', 'harsha': 'harsh', 'jaana': 'jaan', 'ballon': 'balloon', 'algoma': 'aroma', 'buhler': 'butler', 'volunteerism': 'volunteers', 'mally': 'mall', 'trsnsmission': 'transmission', '16f': 'of', 'janu': 'jane', 'lsetf': 'let', '$aal': 'baal', 'l1s': 'les', '2try': 'try', '0$': 'i', 'livemore': 'livermore', 'purfect': 'perfect', 'iamso': 'also', 'hazan': 'human', 'dhhs': 'this', 'abia': 'asia', 'industr': 'industry', 'paikary': 'primary', 'elogistics': 'logistics', 'exeorience': 'experience', 'nasik': 'lasik', 'ot3': 'ot', 'pan09': 'pants', 'disabi': 'disable', 'cgst': 'cost', 'trendske': 'trends', 'smilin': 'smiling', 'fyinfo': 'info', 'orderering': 'ordering', 'nmb': 'numb', 'yana': 'yank', 'lagta': 'last', 'reputational': 'reputation', 'oled': 'old', 'excitement2': 'excitement', 'eatright': 'alright', 'parikh': 'parish', 'arizona]': 'arizona', 'woow': 'wood', 'pepole': 'people', 'kango': 'tango', 'cdnmedia': 'commedia', 'taufan': 'tarzan', 'way]': 'way', '10lb': 'bulb', 'startrek': 'started', 'bhookhe': 'bookie', '2morrow': 'morrow', 'figurately': 'figuratively', 'delvrd': 'delved', 'growin': 'growing', 'septembre': 'september', 'euv': 'eu', 'delish': 'relish', 'mackay': 'mckay', 'tdem': 'them', 'nmmc': 'name', 'dayyum': 'dadgum', 'chillie': 'chilli', 'lockown': 'lockdown', 'hatud': 'hated', 'truckrs': 'trucks', 'needz': 'need', 'middletown': 'middleton', 'gosnapshot': 'snapshot', 'whoopin': 'whooping', '23may': 'may', 'sjw': 'saw', 'coronavt': 'coronary', 'gpc': 'gpa', 'sayo': 'say', 'ereceipts': 'receipts', 'smawt': 'smart', 'npc': 'nic', 'camaraderi': 'camaraderie', 'bidens': 'widens', 'trolly': 'troll', 'smk': 'smo', 'ltc': 'etc', 'contolled': 'controlled', '40am': 'am', 'puifier': 'purifier', 'cherlyn': 'cherry', 'crossfit': 'crossfire', 'everli': 'ever', 'tfl': 'til', 'pque': 'que', 'happyhi': 'happy', 'decluttering': 'cluttering', 'bugout': 'dugout', 'rejine': 'refine', 'hela': 'help', 'bubur': 'buyer', 'eur': 'our', 'desh': 'desk', 'snotted': 'spotted', 'plateform': 'platform', 'rubia': 'rubin', 'paintworks': 'paintwork', 'makita': 'making', 'vagary': 'salary', 'postulatum': 'postulated', 'loula': 'lola', 'bandaiding': 'bandaging', 'shpit': 'shit', 'wds': 'was', 'madda': 'yadda', '6packs': 'packs', 'appointmen': 'appointment', 'yestersay': 'yesterday', 'shytoshi': 'satoshi', 'digthat': 'ifthat', 'ecvs': 'eyes', 'gamesmaker': 'gamemakers', 'tomy': 'toy', 'fulcher': 'further', 'internetof': 'internet', 'footboy': 'footy', 'vati': 'vat', 'riggz': 'riggs', 'accourding': 'according', 'jerkwad': 'jerked', 'whatapp': 'whatsapp', 'deidentified': 'identified', 'doctober': 'october', 'qg': 'og', 'bellom': 'bellow', 'struggli': 'struggle', 'hypermarkets': 'hypermarket', 'acaba': 'cab', 'qvc': 'que', 'cullman': 'bellman', 'wye': 'we', 'baldry': 'baldy', '7f': 'of', 'callaway': 'hallway', 'hugops': "hugo's", 'flashbox': 'cashbox', 'taim': 'tail', 'ngin': 'gin', 'ledesma': 'edema', '2dy': 'dy', 'chp': 'cop', 'kohler': 'kohled', 'yummies': 'dummies', 'kym': 'gym', 'jackjaw': 'jackal', 'lacto': 'facto', 'afforestation': 'deforestation', 'weejs': 'weeks', 'crc': 'cry', 'adulty': 'adult', 'patricio': 'patricia', 'mybus': 'bus', 'tokree': 'three', 'decorona': 'corona', 'stokvel': 'shovel', '1pinch': 'pinch', 'oconus': 'bonus', 'dunkaroos': 'buckaroos', 'sfgate': 'state', 'corporacion': 'corporation', '3wk': 'we', 'kawo': 'kao', 'gozem': 'golem', 'eriea': 'erica', 'wama': 'mama', 'hereee': 'here', 'mtbs': 'mobs', 'mmj': 'mme', 'stackable': 'trackable', 'fabreeze': 'freeze', 'gcash': 'cash', 'passworders': 'passwords', 'mths': 'myths', 'bwwm': 'swim', 'horeca': 'orca', 'bcferries': 'berries', 'gettiing': 'getting', 'cto': 'to', 'samw': 'same', 'rakhe': 'rake', 'kahan': 'khan', 'nw8': 'we', 'expencive': 'expensive', 'wxia': 'xia', 'timey': 'time', 'normaal': 'normal', 'tamuwa': 'tamara', 'sigeg': 'sieg', 'acb': 'act', 'paltforms': 'platforms', 'tymon': 'tyson', 'noacf': 'coach', 'pudos': 'kudos', 'bigup': 'big', 'backride': 'backside', 'pharamcy': 'pharmacy', 'ealk': 'talk', 'sunbelt': 'sunset', 'ocurrence': 'occurrence', 'okuru': 'kuru', 'aomeone': 'someone', 'nmpol': "t'pol", 'theist': 'heist', 'kathi': 'kathy', 'lfgm': 'legs', 'whittier': 'shittier', 'angier': 'anger', 'latelys': 'lately', 'lolsob': 'lolo', 'socialme': 'social', 'minta': 'mint', 'psilently': 'silently', 'ideliverr': 'deliver', 'igno': 'ino', 'anywho': 'anyway', 'apr5': 'are', 'utrac': 'track', 'bsg': 'big', 'nier': 'nicer', 'coronaus': 'coronary', 'serp': 'perp', 'bloodfeast': 'bloodiest', 'unshowered': 'showered', 'nopes': 'hopes', 'benlai': 'bella', 'bccs': 'back', 'hrcorona': 'corona', 'hmh': 'huh', 'challe': 'calle', 'ojamea': 'james', 'dfc': 'doc', 'fredi': 'fred', 'rs2': 'rse', 'vind': 'find', 'seri': 'sari', 'raynors': 'raptors', 'vlms': 'alms', 'wsib': 'wait', 'tappan': 'taipan', 'spazza': 'spazzy', 'relaunching': 'launching', 'friis': 'fries', 'commercer': 'commerce', 'neighborhoo': 'neighborhood', 'iey': 'key', '83years': 'years', 'myn': 'my', 'callum': 'callus', 'efresh': 'fresh', '2lbs': 'plus', 'psharmi': 'pharma', 'kelley': 'kelly', 'politicsny': 'politics', 'jada': 'jade', 'supervisot': 'supervisor', 'cdnews': 'news', 'onhere': 'ofhere', 'shome': 'some', 'enjoye': 'enjoy', 'germano': 'german', 'softech': 'soften', 'orderinarily': 'ordinarily', 'girasol': 'parasol', 'shoppd': 'shopped', 'ukrayini': 'ukraine', 'ayun': 'yun', 'topoli': 'tools', 'prta': 'puta', '$affirm': 'affirm', 'brazos': 'brains', 'globalgo': 'global', 'atsocial': 'asocial', 'homeworker': 'homework', 'ios5': 'is', 'bader': 'baker', 'cango': 'cargo', 'automagically': 'automatically', 'moval': 'moral', 'seghal': 'sexual', 'retd': 'read', 'mixup': 'mixed', 'upcycling': 'recycling', 'recipies': 'recipes', 'knapp': 'snap', 'menards': 'regards', 'grogery': 'grocery', 'enkil': 'until', 'ingejk': 'ingest', 'bandai': 'banzai', 'fjorder': 'order', 'tblf': 'tell', 'peria': 'peril', 'paproud': 'proud', 'corderon': 'cordon', 'vaal': 'vial', 'michig': 'michi', 'ishopr': 'shop', 'gathari': 'gather', 'apperals': 'appeals', 'file]': 'file', 'bleddy': 'bloody', 'spirtually': 'spiritually', '5min': 'min', 'orenji': 'benji', 'votejoy': 'lovejoy', '8f': 'of', 'refr': 'rear', 'relati': 'relate', 'litmus7': 'litmus', 'm25': 'my', '1min': 'min', 'mtgrix': 'matrix', 'moc': 'mom', 'nowait': 'wait', 'ydy': 'yay', 'brantley': 'bradley', 'badser': 'badger', 'jobposting': 'composting', 'hoth': 'both', 'toowong': 'towing', 'operatio': 'operation', 'nard': 'hard', '48oz': 'coz', 'communally': 'communal', 'dge': 'die', '17am': 'am', 'flood8ng': 'flooding', '$lmwis': 'lewis', 'cartwheelri': 'cartwheels', 'isnmr': "isn't", 'freshies': 'freshen', 'fraser': 'eraser', 'nourishnc': 'nourish', 'kampung': 'camping', 'medici': 'medic', 'aed': 'and', 'adler': 'aller', 'vms': 'vos', 'news4': 'news', 'us98': 'us', '$col': 'col', 'myntra': 'mantra', '2isolate': 'isolate', 'marfos': 'marks', 'olacafe': 'placate', 'betwe': 'bette', 'frigin': 'origin', 'declutter': 'clutter', 'leamington': 'remington', 'ca05': 'can', 'bazer': 'baker', 'expan': 'expand', 'ps75': 'psi', 'bgcs': 'back', 'stevia': 'stevie', 'rushbo': 'rush', 'joannes': 'johannes', 'myour': 'your', 'hlywd': 'lewd', 'supermark': 'supermarket', 'ardyn': 'army', 'bbdaily': 'daily', 'mybelmont': 'belmont', 'prces': 'prices', 'glanville': 'granville', 'kif1a': 'kinda', 'maybethe': 'macbeth', 'freeport': 'report', 'trollies': 'trophies', 'gmef': 'me', 'lha': 'ha', 'relavent': 'relevant', 'amain': 'again', 'ralphs': 'ralph', 'wosm': 'worm', 'vss': 'ass', 'kween': 'keen', 'moley': 'money', 'yoders': 'coders', 'sharjah': 'sarah', '$xall': 'all', 'wahi': 'wah', 'ziply': 'zippy', 'orzo': 'ouzo', 'unthaw': 'thaw', 'ehh': 'heh', 'desperate]': 'desperate', 'zabars': 'bars', 'cucu': 'much', 'easi': 'easy', 'awesm': 'west', 'painsomnia': 'insomnia', 'shirataki': 'shiranai', 'bangin': 'banging', 'hartman': 'cartman', 'elodie': 'lodge', 'wada': 'wade', 'dogtown': 'downtown', 'something]': 'something', 'amts': 'acts', 'inxnews': 'inners', 'hs2': 'is', 'dikki': 'nikki', 'godsent': 'godsend', 'fcsaturday': 'saturday', 'corsight': 'foresight', 'wyckoff': 'kickoff', 'enen': 'even', 'preparer': 'prepared', 'fsa': 'sa', 'calhoun': 'callous', 'uki': 'ski', 'swiggey': 'swagger', 'kesa': 'mesa', 'bizotch': 'bitch', 'atar': 'star', '38th': 'with', '5lbs': 'plus', 'coustmer': 'customer', 'perfor': 'perform', 'oximeter': 'odometer', 'unavaiable': 'unavailable', 'mifa': 'mia', 'addington': 'reddington', 'fortino': 'fortune', 'ifast': 'fast', 'keto': 'veto', 'acc': 'act', '17oz': 'coz', 'parmbir': 'partir', 'girvan': 'given', 'zehrs': 'years', 'zayn': 'zany', 'walkshire': 'wilshire', 'sekhar': 'shekhar', '$wmt': 'wet', 'gomammoth': 'mammoth', 'waale': 'whale', 'aci': 'act', 'tazwiz': 'aziz', 'quintero': 'quieter', 'tbay': 'bay', 'redbox': 'redo', 'certa': 'cert', 'lunchtim': 'lunchtime', 'wakiki': 'waking', 'claygate': 'playmate', 'nexzone': 'neurone', 'telep': 'telex', 'csg': 'cog', 'dz': 'do', 'bulshit': 'bullshit', 'penndot': 'pennant', 'politiciens': 'politicians', 'guapo': 'guano', 'restaur': 'restart', '$hdsn': 'hasn', 'bbc4': 'back', 'ptl': 'pal', 'ganun': 'gun', 'pullup': 'pull', 'borja': 'bora', 'blundertruss': 'blunderbuss', 'shoplyft': 'shoplift', 'rds': 'ads', 'can99': 'can', 'huda': 'hula', 'suburbanite': 'suburbanites', 'azpol': "t'pol", 'betti': 'betty', 'saxby': 'say', 'ir4': 'iru', 'jaun': 'juan', 'calichio': 'calico', 'avlso': 'also', 'brixton': 'braxton', 'trumper': 'trumpet', 'dions': 'lions', 'itr': 'it', 'letafood': 'seafood', 'myp': 'my', 'caritas': 'carnitas', 'pinocchios': 'pinocchio', 'watermelonnn': 'watermelon', 'badola': 'badly', 'coostco': 'costco', 'harrasing': 'harassing', 'unica': 'nice', 'omntwana': 'montana', 'chomsky': 'choosy', 'resubscribe': 'subscribe', 'onc': 'on', 'elederly': 'elderly', 'taazi': 'taani', 'eroski': 'eros', 'eji': 'epi', 'aima': 'aim', 'fraudential': 'prudential', 'thacher': 'teacher', 'butttt': 'butt', 'noticia': 'notice', 'tolu': 'told', 'jlp': 'jap', 'nellis': 'ellis', 'mnb': 'mob', 'topeka': 'opera', 'gugma': 'gum', 'vome': 'come', 'camni': 'cani', 'yfc': 'you', 'dcweed': 'weed', 'altis': 'altos', 'sadas': 'sodas', 'francisc': 'francis', 'rsquo': 'quo', 'comcast': 'coast', 'ohai': 'bhai', 'thwm': 'them', 'hons': 'sons', 'biden': 'widen', 'elapse': 'lapse', 'shazam': 'shaman', 'alphasm': 'alphas', 'amindu': 'mind', 'mtug': 'mug', 'alvior': 'savior', 'cheapie': 'cheap', 'qsrs': 'ass', '80mins': 'mins', 'armload': 'reload', 'passata': 'passage', 'loveour': 'loveyour', 'btaskee': 'basket', 'googleduo': 'googled', 'mempin': 'remain', '25rs': 'cars', 'pheasynt': 'pheasant', 'wsox': 'sox', 'mycroft': 'microt', 'xost': 'most', 'oxo': 'ox', 'lolie': 'louie', 'oakville': 'orville', 'khidma': 'hima', 'hiit': 'hit', 'nival': 'naval', 'salamu': 'salami', 'dropps': 'drops', 'creati': 'create', 'milagro': 'milano', 'phuza': 'pizza', 'orderer]': 'ordered', 'nyers': 'myers', 'tuber': 'tube', 'hmas': 'has', 'fettucine': 'fettuccine', 'chickees': 'chickens', 'haz': 'had', 'tomayto': 'tomato', 'inscart': 'instant', 'gotransit': 'transit', 'ts2': 'tsa', 'gbx': 'be', 'comjng': 'coming', 'regimens': 'regiment', 'redlands': 'wetlands', 'itcompany': 'company', '8billion': 'billion', 'iera': 'era', 'aat': 'at', 'sabki': 'saki', 'begreater': 'greater', 'baberge': 'barge', 'deonde': 'donde', 'tyhini': 'think', 'burdett': 'burnett', '$efx': 'left', 'uuid': 'quid', 'ngisho': 'naisho', 'boyardee': 'boarded', 'daraz': 'dara', 'serval': 'serial', '1yo': 'yo', 'ocassions': 'occasions', 'convinent': 'continent', 'freind': 'friend', 'tallie': 'tallied', 'applian': 'appian', 'inboxed': 'boxed', 'paneras': 'papers', 'tomyto': 'tomato', 'bridgeto': 'bridget', 'ocarina': 'scaring', 'televisits': 'television', 'reiders': 'readers', 'wark': 'work', 'sug': 'sun', 'technolo': 'technology', 'quind': 'quid', 'asiaone': 'asian', 'buzzin': 'buzzing', 'hpa': 'ha', 'memsab': 'mesa', 'nsambya': 'samba', 'v1': 've', 'turo': 'turn', 'newsfront': 'newsprint', 'dreamr': 'dream', '7adults': 'adults', 'grealish': 'realise', 'harada': 'barada', 'tabhi': 'tabi', 'chse': 'case', 'icos': 'cos', 'instax': 'instar', 'followday': 'holloway', 'happywho': 'happyto', 'osp': 'op', 'dxmmo': 'dummy', 'whitegates': 'whitewater', 'crj': 'cry', 'membershipts': 'memberships', 'imsold': 'sold', 'myoning': 'mooning', 'yehey': 'yhey', 'jasleen': 'asleep', 'smeuk': 'speak', 'fleshlight': 'flashlight', 'socialmed': 'socialized', 'sress': 'press', 'brookside': 'broadside', 'kurye': 'kure', 'akleg': 'able', 'reshudle': 'reshuffle', 'perogy': 'proxy', 'donato': 'donate', 'butale': 'butane', 'itjob': 'job', '2hunt': 'hunt', 'incentiv': 'incentive', 'fizzers': 'fingers', 'soops': 'stops', 'phyu': 'phys', 'hrard': 'heard', 'smbiz': 'biz', 'pymt': 'put', 'stayhom': 'stardom', 'givenow': 'given', 'tarka': 'tara', 'aadit': 'audit', 'restraunts': 'restraints', 'primetv': 'prime', 'glugs': 'slugs', 'whns': 'wins', 'astra': 'astray', '2oil': 'oil', 'isolife': 'isolate', 'barrhead': 'barred', 'newss': 'news', 'paedatrician': 'paediatrician', '3rr': 'err', 'strugging': 'struggling', 'prmzd': 'proud', 'teachnology': 'technology', 'mlg': 'mug', 'hygienists': 'hygienist', 'noirs': 'noir', 'oneline': 'online', 'gul': 'guy', '5g': 'og', 'waitsi': 'waits', 'noslip': 'slip', 'recru': 'recur', 'omnom': 'mom', '$tsco': 'disco', 'helpus': 'helps', 'hiebert': 'gilbert', 'bizit': 'bit', 'reeeeee': 'referee', 'bahama': 'bahamas', 'reupped': 'repped', 'ondoor': 'indoor', 'northland': 'northward', 'kid1': 'kids', 'donators': 'doctors', 'lobola': 'lola', 'overpacking': 'overtaking', 'altfi': 'alter', 'acv': 'act', 'afroday': 'friday', 'garbages': 'garbage', '30something': 'something', 'restaurents': 'restaurants', 'keyworder': 'keyword', 'guro': 'guru', 'organis': 'organic', 'intown': 'into', 'obliviously': 'obviously', 'cbf': 'caf', 'wnlyn': 'only', 'fenley': 'finley', 'lastyard': 'bastard', 'tomz': 'tomb', 'arsenio': 'arsenic', 'essentialz': 'essential', 'eggless': 'endless', 'succes': 'success', 'svg': 'sig', 'cheetay': 'cheetah', '6kms': 'arms', 'escarole': 'carole', 'outgo': 'outdo', 'jsb': 'job', 'goproduce': 'produce', 'daya': 'day', '$xrt': 'part', 'obamatn': 'obama', 'cream1': 'cream', 'seva': 'sea', 'kholny': 'holy', '$yelp': 'yelp', 'aqsa': 'asa', 'pwc': 'pic', 'neccessary': 'necessary', 'fy23': 'fy', 'preventer': 'prevented', 'yearslong': 'yearlong', 'skyourockets': 'skyrockets', 'grabfood': 'graboid', 'mashablet': 'washable', 'triscuits': 'biscuits', 'chinotto': 'chotto', 'mannering': 'wandering', 'makework': 'casework', 'entrep': 'entrap', 'blok': 'blow', 'nicastro': 'castro', 'fudes': 'dudes', 'ia04': 'ian', 'mullen': 'sullen', 'respiritory': 'respiratory', 'nicolai': 'nikolai', 'massy': 'mass', 'spor': 'spot', 'appartment': 'apartment', 'boul': 'soul', 'qcommerce': 'commerce', 'on]': 'on', 'ebooks': 'books', 'ecommer': 'comme', 'risker': 'risked', 'zipcar': 'zipper', 'unft': 'unit', 'coteachers': 'teachers', 'gaurds': 'guards', 'codify': 'modify', 'jonno': 'jonny', 'sales6': 'sales', 'sowerby': 'soberly', 'vcfunding': 'funding', 'peopling': 'popping', 'tanzania': 'tanzanian', 'insurances': 'insurance', 'pcr': 'per', 'movi': 'move', 'sanitization': 'sanitation', 'blinkit': 'blanket', 'microds': 'microns', 'theu': 'the', 'livrer': 'liver', '$sea': 'sea', '20c': 'doc', 'foodiez': 'foodie', 'markit': 'market', 'psw': 'paw', 'drayton': 'draytons', 'r7k': 'ask', 'zomg': 'zom', 'percels': 'parcels', 'wooooo': 'nooooo', 'shopmom': 'stepmom', 'naoko': 'nook', 'mashable]': 'washable', 'lidi': 'lid', 'lds': 'lads', 'jfc4': 'jack', 'wdnt': 'want', 'zomata': 'tomato', 'minneso': 'mines', 'everyonr': 'everyone', 'neede': 'need', 'cookstown': 'crosstown', '$cmder': 'cider', '6kg': 'ekg', 'exectn': 'expect', 'tgeir': 'their', 'phor': 'poor', 'antimasker': 'antimatter', 'punekars': 'punkass', '$dash': 'dash', 'medicago': 'medical', 'g2g': 'gig', 'heckuva': 'helluva', 'londis': 'landis', 'lolno': 'lolo', 'dimetap': 'wiretap', 'ikaka': 'aka', 'wwbt': 'what', '18pack': 'pack', 'jasola': 'jason', 'thomaz': 'thomas', 'erick': 'trick', 'wap': 'was', 'rhiannon': 'shannon', 'hubbs': 'hubby', 'mutti': 'mutt', 'upc': 'up', 'imprecision': 'impression', 'news9': 'news', 'detr': 'dear', 'homeschools': 'homeschool', 'anantha': 'samantha', 'moneystr': 'moneys', 'wwv': 'wuv', 'rivarly': 'rivalry', 'mingde': 'mingle', 'montvale': 'montage', 'charo': 'charm', 'pastu': 'past', 'mpp': 'map', 'nd': 'and', 'arbiz': 'orbit', 'hellp': 'help', 'thrille': 'thrilled', '$aker': 'maker', 'alosra': 'flora', 'kearsten': 'kirsten', '1u': 'u', 'shoyld': 'should', 'beerrun': 'overrun', 'castech': 'caltech', 'saranja': 'savanna', 'dangero': 'danger', 'lovesong': 'livelong', 'freeyum': 'freedom', 'milind': 'mind', 'nltech': 'tech', 'amazon]': 'amazons', 'intelligencer': 'intelligence', 'trolley20': 'trolley', 'wtff': 'off', 'stx': 'six', 'chablis': 'charlie', 'floofer': 'floor', 'orherwise': 'otherwise', 'ign': 'in', 'newbury': 'newborn', 'vignesh': 'vines', 'lovegso': 'loves', 'soem': 'some', 'lccc': 'luck', 'cucks': 'bucks', 'measels': 'measles', 'yecommission': 'decommission', 'milkboy': 'milky', 'helfy': 'help', 'px': 'ex', 'loka': 'loca', 'lcao': 'cao', 'dlc': 'doc', 'worki': 'work', 'partage': 'partake', 'punit': 'unit', 'jls': 'jus', 'worktop': 'workshop', 'jefinner': 'beginner', 'greggy': 'groggy', 'nikkei': 'nikki', 'eoy': 'boy', 'nalk': 'talk', 'stra': 'star', 'rockton': 'reckon', 'greenock': 'greenback', 'curbsi': 'curbs', 'interivew': 'interview', 'separaty': 'separate', 'cleanwater': 'clearwater', 'omes': 'comes', 'indyvolved': 'involved', 'lambo': 'lamb', 'baffert': 'affect', 'sammi': 'sammy', 'broasted': 'roasted', 'cornerstore': 'cornerstone', 'apea': 'area', 'ronna': 'gonna', 'uxolo': 'xolo', 'yoplaits': 'plaits', 'vembla': 'verbal', 'clusterf': 'cluster', 'alterph': 'alter', '2morow': 'moron', 'veille': 'ville', 'cryptos': 'crypts', 'secur': 'secure', 'fantisies': 'fantasies', 'lekr': 'lear', 'ganado': 'canada', 'festivus': 'festival', 'rabidly': 'rapidly', 'otehr': 'other', 'cajucom': 'capcom', 'ratin': 'rain', 'tuesdayon': 'tuesdays', '1liter': 'liter', 'texico': 'mexico', 'entreherneur': 'entrepreneur', 'tcmff': 'cuff', 'hosptial': 'hospital', 'dioso': 'dios', 'homefull': 'hopefully', 'trussler': 'trussed', 'norland': 'garland', 'fubar': 'far', 'kewanee': 'swanee', 'amaxon': 'amazon', 'gorillaz': 'gorilla', 'approx': 'approve', 'innout': 'input', 'havinn': 'having', 'leoni': 'leon', 'nyc': 'ny', '30l': 'all', 'aadhar': 'radar', 'odered': 'ordered', 'buttr': 'butt', 'hg1': 'he', '$adrny': 'carny', 'frome': 'from', 'wausau': 'causal', 'macrina': 'marina', 'rehne': 'rene', 'yb67': 'ybe', 'khpes': 'keeps', 'enterprenaur': 'entrepreneur', 'thirtyaf': 'thirty', 'jornada': 'tornado', 'chocs': 'chops', 'lucky15': 'lucky', 'ramens': 'ramen', 'tastiness': 'nastiness', 'magida': 'magda', 'desp': 'deep', 'sahm': 'sam', '35am': 'am', 'trayn': 'train', 'oau': 'ou', 'revote': 'remote', 'francisca': 'francisco', '17mins': 'mins', 'ideliveries': 'deliveries', 'codifiers': 'modifies', 'oliveira': 'oliver', 'lort': 'lot', 'optn': 'open', 'youtuber': 'youtube', 'badal': 'banal', 'checkouts': 'checkout', 'fnb': 'fob', 'phin': 'pain', 'smel': 'smell', 'baliuag': 'baling', 'tsc': 'tac', 'goras': 'gras', 'napaka': 'napalm', 'remeber': 'remember', 'arbutus': 'arduous', 'kesko': 'desk', 'mchughs': 'coughs', 'ddbit': 'debit', 'ndash': 'dash', 'vies': 'lies', 'marconi': 'maroni', 'insidd': 'inside', 'forfree': 'foresee', 'verl': 'very', 'myamole': 'maypole', 'lovee': 'love', 'pt10': 'pta', 'mannone': 'anyone', '4aklondike': 'klondike', 'lold': 'told', 'rajkot': 'racket', 'mousumi': 'susumi', 'koorg': 'door', 'superama': 'superman', 'transitlink': 'transiting', 'bsn': 'bin', 'distrubition': 'distribution', 'anplace': 'anyplace', 'cfc': 'cfo', 'trudgian': 'trudging', 'co3': 'cop', 'dimedwa': 'dimed', 'etrike': 'strike', '30seconds': 'seconds', 'ris': 'is', 'bhd': 'bad', '1lbs': 'plus', 'glazer': 'glazed', 'platfrom': 'platform', 'thelorry': 'theory', 'hanzo': 'hanno', 'supermkt': 'superman', 'busienss': 'business', 'cumberlandon': 'cumberland', '3kms': 'arms', 'rogan': 'organ', 'hhi': 'hi', 'forsale': 'forsake', 'alam': 'alarm', 'akansha': 'akash', 'courier24': 'courier', 'ifresh': 'fresh', 'kron4': 'krone', 'availab': 'available', 'emailer': 'emailed', 'morekind': 'somekind', 'jsi': 'si', 'computerize': 'computerized', 'konsumer': 'consumer', 'pexels': 'peels', 'hubbox': 'hubby', 'aristocats': 'aristocrats', 'coronaes': 'coroner', 'escooters': 'scooters', 'repull': 'pull', 'this]': 'this', 'tnheroes': 'heroes', 'appl': 'apply', 'depan': 'dean', 'bulow': 'blow', 'recientes': 'recites', 'soga': 'soda', 'amart': 'smart', '28th': 'with', 'levashe': 'lease', 'iono': 'ion', '49ers': 'hers', 'chyna': 'china', 'elderies': 'elders', 'haarp': 'harp', 'munchee': 'munched', 'popeyes': 'popeye', 'soaker': 'soaked', 'luchadore': 'luchador', 'man]': 'man', 'ozil': 'oil', 'poundi': 'pounds', 'pikup': 'pickup', 'tigard': 'toward', 'pingot': 'pinot', '$ads': 'lads', 'deman': 'demand', 'anotha': 'anothe', 'slpng': 'sling', 'saama': 'sama', 'loblaws': "lola's", 'appeare': 'appears', 'ozon': 'ozone', 'tomura': 'tour', 'keyon': 'beyond', 'jibilian': 'civilian', 'launc': 'launch', 'dca': 'dna', 'ntr': 'nor', 'awfl': 'awful', 'hngry': 'angry', 'qac': 'mac', 'pancakeday': 'pancaked', 'wesee': 'wesen', 'almalki': 'alkali', 'iyll': "i'll", 'marboro': 'harbor', 'wyvern': 'govern', 'mcjc': 'mic', 'vermonter': 'vermont', 'jasey': 'casey', 'newspa': 'news', 'ngaske': 'gasket', 'shield]': 'shield', 'unber': 'under', 'maduro': 'mature', 'peppa': 'peppy', 'bagi': 'bag', 'trashbags': 'trashcans', 'refe': 'refer', 'pl': 'pal', '$plug': 'plug', 'tennesee': 'tennessee', 'jamu': 'jam', 'gok': 'got', 'deceasy': 'decease', 'tirana': 'tiring', 'tmb': 'tub', 'nurturenj': 'nurture', 'mi6': 'mix', 'skytrain': 'strain', 'stressfull': 'stressful', 'countirng': 'counting', 'spx': 'six', 'speacial': 'special', 'fy18': 'fy', 'abm': 'am', 'aaiden': 'maiden', 'disinfects': 'disinfect', 'inspirewhs': 'inspires', 'eventho': 'event', 'overlorder': 'overloaded', 'haloje': 'alone', 'wbns': 'wins', '$nflx': 'influx', 'cotija': 'ouija', '42nd': 'and', 'a69': 'a', 'cie': 'die', 'establishe': 'establish', 'luto': 'auto', 'groceryid': 'grocery', 'vimto': 'vito', 'monzo': 'mono', 'seamle': 'same', '89b': 'job', 'l]': 'la', 'bakeware': 'beware', 'zoila': 'voila', 'moines': 'mines', 'baras': 'bars', 'reshop': 'reshot', 'cips': 'cops', 'royco': 'rocco', 'autonymous': 'autonomous', 'ggv': 'giv', 'basie': 'base', 'jayein': 'javelin', 'levingston': 'livingstone', 'ri0ts': 'riots', 'ubber': 'rubber', 'shoprite': 'sprite', 'apperently': 'apparently', 'workk': 'work', 'jetta': 'jetty', 'openreach': 'overreach', 'incredyble': 'incredible', 'montco': 'monaco', 'lunden': 'linden', 'vomo': 'homo', 'oneshow': 'onewho', 'shaik': 'shark', 'guam': 'gum', 'bcpoli': 'coli', 'alesha': 'flesh', 'dummpd': 'dump', '6wks': 'was', 'micycle': 'bicycle', 'oddbox': 'outbox', 'glendale': 'greendale', 'lcbos': 'lobos', 'newdawn': 'newman', 'calabasas': 'calabash', 'unintuitive': 'intuitive', 'bull5h': 'bullet', 'norwalk': 'normal', 'lindiwe': 'indie', 'muscato': 'muscatel', 'avram': 'aram', 'ccw': 'cow', 'gogra': 'gonna', 'connec': 'connect', 'risott': 'risotto', '2ish': 'wish', 'xobo': 'hobo', 'ntwrk': 'network', '2da': 'da', 'elxsea': 'else', 'haringey': 'harbinger', 'srrobotics': 'robotics', 'finaly': 'finally', 'muns': 'runs', 'omggg': 'omega', 'gnb': 'gob', 'ukiah': 'kith', 'dilf': 'dill', 'guatemala': 'guatemalans', 'ps42': 'psi', 'aiinnovation': 'innovation', 'kambing': 'gambling', 'cheebs': 'cheers', 'rilliant': 'brilliant', 'maitland': 'mainland', 'gainz': 'gain', 'insomnina': 'insomnia', '58th': 'with', 'bsupportive': 'supportive', 'etanla': 'tanya', 'nowwhat': 'knowwhat', 'namak': 'name', 'sosialize': 'socialize', 'fishco': 'fish', 'fela': 'felt', 'retu': 'ret', 'justina': 'justin', 'deliverys': 'delivery', 'intl': 'into', 'nornal': 'normal', 'yamper': 'tamper', 'nhif': 'this', 'citm': 'city', 'noctobera': 'october', 'rillos': 'kills', 'coronaqa': 'coronary', 'jobz': 'job', 'dhabi': 'dhobi', 'gouger': 'gouge', 'isloo': 'igloo', 'rgis': 'rigs', 'rgnnation': 'ruination', 'wombles': 'wobbles', 'debateable': 'debatable', 'fladged': 'flagged', 'bevis': 'begin', 'bunana': 'banana', 'marylander': 'maryland', 'x3': 'xu', 'kepong': 'keeping', 'claritas': 'clarity', 'dxb': 'dab', 'thoda': 'toda', 'qts': 'its', 'cer': 'her', 'karyana': 'aryans', 'produ': 'proud', 'robelyn': 'robyn', 'aholes': 'holes', 'r35': 'run', 'ekhane': 'enhance', 'suckies': 'suckles', 'contuing': 'counting', 'legon': 'lemon', 'shlep': 'sheep', 'tella': 'tell', 'st8': 'sta', 'geolocation': 'relocation', 'anb': 'and', 'lectoure': 'lecture', 'lansinoh': 'lansing', 'uppp': 'upp', 'allbirds': 'jailbirds', 'trust2': 'trust', 'tataru': 'tatars', 'senoirs': 'seniors', 'arborio': 'arbor', 'ginaury': 'injury', 'raman': 'roman', 'ebresilient': 'resilient', 'masarap': 'mascara', 'agege': 'agree', 'amazingo': 'amazing', 'hobi': 'hobo', 'rahulji': 'rahul', 'amidon': 'maiden', 'puledo': 'puedo', 'magnit': 'magnet', 'mamalady': 'marmalade', 'crankin': 'cranking', 'cga': 'ga', 'morissons': 'orisons', 'jit': 'it', '5kph': 'syph', 'audibles': 'audible', 'repugs': 'refuge', 'nodrip': 'drip', 'tahini': 'tahiti', 'timara': 'tiara', 'cambly': 'family', 'neato': 'neat', 'vinegary': 'vinegar', '5mos': 'mos', 'raena': 'arena', 'kauaa': 'karma', 'tator': 'tutor', 'cotati': 'coat', 'dpro': 'pro', 'ddfree': 'free', 'tubeless': 'tuneless', 'doxx': 'doux', 'newz': 'new', 'donnies': 'donnie', 'gta3': 'stay', 'ugandanews': 'ugandans', 'treat50': 'treat', 'smartwater': 'saltwater', 'mpg': 'mug', 'gtgo': 'gogo', 'belitt': 'belt', 'qurnat': 'burnt', 'ither': 'other', 'kawhi': 'kai', 'sylvania': 'sylvia', 'makoa': 'mako', 'foodlocker': 'footlocker', 'amirite': 'alrite', 'sr': 'so', 'ios8': 'is', 'wedel': 'weel', 'cmall': 'call', 'clt': 'cut', 'encrust': 'entrust', 'sen$e': 'sense', 'amazonin': 'amazonian', 'inprison': 'imprison', 'maurer': 'matter', 'ischeamic': 'ischemic', 'vbank': 'bank', '21m': 'him', 'abacha': 'apache', 'amp]': 'amp', 'sando': 'sand', 'measurably': 'measurable', 'dissenter': 'dissenters', 'wearables': 'wearable', 'ibanking': 'banking', 'privilaged': 'privileged', 'idlf': 'idle', 'corbyn': 'corn', 'ebeneezer': 'ebenezer', 'somnus': 'sounds', 'chicagoil': 'chicago', 'r40': 'run', 'cra': 'car', 'cheezit': 'cheerio', 'kapiti': 'capita', 'mandao': 'mandar', 'belikan': 'belgian', 'mell': 'tell', 'inasal': 'nasal', 'madisonct': 'madison', 'gigcars': 'cigars', 'meegan': 'megan', 'homebound': 'housebound', 'wiating': 'waiting', 'twaz': 'twat', 'nea': 'new', 'kr0': 'are', 'pns': 'pins', 'bvsd': 'best', 'mattel': 'matter', 'spinney': 'spinner', 'hapo': 'halo', 'succesful': 'successful', 'ztb': 'zeb', 'abang': 'bang', 'iveco': 'veto', 'sungei': 'singer', 'idd': 'did', 'ibeju': 'beau', '75th': 'with', 'reques': 'request', 'snuggs': 'slugs', 'clarkson': 'carson', 'evolv': 'evolve', 'hako': 'halo', 'wasent': "was'nt", '4bn': 'ibn', 'dogmeals': 'dogmas', 'wetv': 'wet', '$vz': 've', 'gardy': 'gary', 'dodgem': 'dodge', 'stright': 'straight', 'gopu': 'you', '02am': 'am', 'harrased': 'harassed', 'milkmanit': 'milkman', 'independe': 'independent', 'edchat': 'chat', 'fr': 'for', 'csusb': 'cause', 'chocol': 'choco', 'parterning': 'patterning', 'lixelife': 'lifeline', 'husbnd': 'husband', 'teamup': 'teacup', 'columbo': 'column', 'inact': 'intact', 'keepgoing': 'keeping', 'qurantine': 'quarantine', 'kadoma': 'karma', 'buisnesses': 'businesses', 'macbride': 'mcbride', 'ammenity': 'amenity', 'pville': 'ville', 'orderer0': 'ordered', 'jira': 'lira', 'overies': 'ovaries', 'homesless': 'homeless', 'vssr': 'ussr', 'tkmomentum': 'momentum', 'rockforder': 'rockford', 'maicai': 'magical', 'fssai': 'essay', 'bastids': 'bastards', 'larouche': 'laroche', 'wasfor': 'pastor', 'isidro': 'sido', 'modi': 'mode', 'nyeema': 'enema', 'rathana': 'nathan', 'gomati': 'goat', '5m': 'am', 'bmttx': 'butt', 'fxck': 'fuck', 'applaudmy': 'applaud', 'petsmart': 'outsmart', 'msv': 'mav', 'atdoorstep': 'doorstep', '8ft': 'aft', 'nyy': 'nay', 'tametag': 'nametag', 'followmeto': 'followeth', '$dada': 'dada', 'cmproud': 'proud', 'moncton': 'monitor', 'bgr': 'bar', 'lynk': 'link', 'upd': 'up', 'frye': 'free', 'makoko': 'makoto', 'ga$': 'gas', 'gizmodo': 'gizmo', 'gingerale': 'gingerly', 'questiom': 'question', '1tb': 'to', 'calld': 'call', 'visperas': 'vipers', 'theur': 'their', 'wearedtr': 'wearer', 'abpoli': 'poli', 'kyoura': 'your', 'nost': 'not', 'shipd': 'ship', 'wrk': 'work', 'lj': 'la', 'mydtd': 'myth', 'gerber': 'member', 'urber': 'uber', 'pii': 'pig', 'wern': 'were', 'actua': 'actual', 'hotdrive': 'outdrive', 'bopal': 'opal', 'mallu': 'mall', 'georgann': 'georgian', 'switcharoo': 'switcheroo', 'rutabagas': 'rutabaga', 'shooketh': 'chooseth', 'durin': 'during', '1okc': 'rock', 'ahaaa': 'ahead', 'nini': 'nine', 'undersrand': 'understand', 'oeprate': 'operate', 'kristie': 'kristin', 'raeforder': 'recorder', 'cpgs': 'cogs', 'ninos': 'nines', 'malyali': 'alkali', 'pincodes': 'pinches', 'pokemongo': 'pokemon', 'zavala': 'naval', 'hatuja': 'hakuna', 'ainvest': 'invest', 'wansbury': 'asbury', 'awesomea': 'awesome', 'gweta': 'geta', 'vh': 'oh', 'kumusha': 'yuusha', 'pinktax': 'pinata', 'volmes': 'volumes', 'baybayy': 'barbary', 'issuer': 'issue', 'heinou': 'heinous', 'jerkin': 'jerking', 'alnette': 'annette', 'darriel': 'married', 'wtrf': 'worf', 'stockedup': 'stocked', 'wearerps': 'wearer', 'angryi': 'angry', 'wharehouses': 'warehouses', 'extrs': 'extra', 'skh': 'sky', 'cmts': 'cuts', 'iwu': 'iru', 'distri': 'distro', 'moreov': 'more', 'iit': 'it', 'loadrunner': 'roadrunner', 'hardlines': 'hardliners', 'hm': 'he', 'arrington': 'arlington', 'wlmt': 'what', 'nann': 'nanny', 'jowell': 'powell', 'digitalise': 'digitalis', 'mtkenya': 'kenya', 'freeland': 'freelance', 'nicoll': 'nicole', 'deliberry': 'delivery', 'beps': 'beeps', 'waaf': 'waif', 'hvacs': 'has', 'grocer1': 'grocery', 'lmk': 'lik', 'gtafood': 'seafood', 'outsi': 'outs', 'abiaya': 'aditya', 'juu': 'jug', 'akanaka': 'tanaka', 'wasnot': "wasn't", 'disarray]': 'disarray', 'blursday': 'thursday', 'accommo': 'commo', 'wayy': 'way', 'tennessean': 'tennessee', 'iwish': 'wish', 'hft': 'hit', 'sothat': 'that', 'neigborhood': 'neighborhood', 'fauzi': 'fault', 'unbag': 'bag', 'molvis': 'movie', 'wlox': 'lox', 'kumara': 'kumar', 'qeretail': 'retail', 'm7g': 'mug', 'natch': 'watch', 'newfunding': 'refunding', 'shopeg': 'hope', 'bagless': 'bless', 'ogp': 'op', 'vmt': 'vet', 'pfizer': 'prize', 'brinner': 'bringer', 'modeller': 'modelled', 'zubzero': 'subzero', 'excapades': 'escapades', 'gawwd': 'sawed', 'lora': 'lord', 'haminute': 'minute', 'wfm': 'wom', 'madcity': 'paucity', 'supplyco': 'supply', 'danggit': 'daggit', 'paywalls': 'payrolls', 'kharadi': 'charade', 'philately': 'privately', 'howver': 'however', 'greenpark': 'greenback', 'rinos': 'rings', 'goveggie': 'veggie', 'advt': 'adv', 'lisix': 'six', 'woai': 'wai', 'sametime': 'sometime', 'maaad': 'mad', 'worke': 'work', 'roleplay': 'replay', 'yestrday': 'yesterday', 'episod': 'episode', 'sintra': 'sinatra', 'lettuece': 'lettuce', 'strictness': 'strictest', 'clouderp': 'louder', 'wfmz': 'womb', 'kiflen': 'killed', 'inboxes': 'boxes', 'arkdg': 'ark', 'mvmnt': 'moment', 'omgg': 'egg', 'onted': 'noted', 'matissu': 'marissa', 'potsie': 'potpie', 'rm4': 'my', 'hudd': 'judd', 'joyouride': 'joyride', 'streetart': 'streetcar', 'afi': 'aii', 'show]': 'show', 'ealivery': 'delivery', 'chillll': 'chill', 'mpa': 'map', 'huangpi': 'huang', 'usecase': 'unease', 'svap': 'slap', 'shweta': 'sweat', 'yummi': 'yummy', 'sggov': 'ggot', 'galiano': 'valiant', 'fresca': 'fresco', 'remin': 'remain', 'dekh': 'deh', 'dengerous': 'dangerous', '12k': 'ask', 'vouchering': 'vouching', 'guerin': 'grin', 'rescueph': 'rescue', 'smallbi': 'small', 'grindin': 'grinding', 'daphna': 'daphne', 'airpark': 'airport', 'siwping': 'sipping', 'nje': 'ne', 'delevery': 'delivery', 'ofqual': 'equal', 'arbys': 'army', 'udou': 'uou', 'vrbo': 'rob', 'ackee': 'asked', 'hoggarth': 'hogarth', 'freetea': 'freeze', 'apx': 'ape', 'loi': 'lot', '4immediate': 'immediate', 'ryt': 'rat', 'epublican': 'republican', 'stalinji': 'stalin', 'lul': 'lug', 'zuairia': 'uairi', 'slava': 'slave', 'iechyd': 'itchy', 'worey': 'worry', 'anzac': 'maniac', 'cloi': 'clot', 'nudding': 'pudding', 'butterfies': 'butterflies', 'ej': 'e', 'skilless': 'skinless', 'kalorama': 'panorama', 'wechat': 'what', 'hrs': 'his', 'maraga': 'manage', 'abing': 'aging', 'edmondson': 'edmonds', 'entreprenuer': 'entrepreneur', 'bucs': 'bus', 'pano': 'piano', 'spxl': 'six', 'dronkert': 'drinker', 'avaeats': 'avatars', 'ididit': 'idiot', '90minutes': 'minutes', 'brief]': 'brief', 'foodsta': 'foods', 'veghouse': 'doghouse', 'nunber': 'number', 'phe': 'the', 'slaws': 'laws', 'bahu': 'bah', 'magu': 'mag', 'omarosa': 'marisa', 'rarefy': 'rarely', 'wheeww': 'wheel', 'togovern': 'govern', 'corderes': 'orders', 'chynah': 'china', 'reuable': 'reusable', 'requstd': 'request', 'blk': 'bulk', 'pdc': 'pic', 'h2o2': 'how', '21no': 'no', 'qpr': 'for', 'boboli': 'bobble', 'gapol': 'gaol', 'waititi': 'waiting', 'kwd': 'kid', 'puppa': 'puppy', 'mechinery': 'machinery', 'retweeting': 'retreating', 'bigday': 'midday', 'cabanga': 'cabana', '$gpro': 'pro', 'xyz': 'xy', 'dail': 'jail', 'derwood': 'redwood', 'nrt': 'not', 'sns': 'sons', '79c': 'doc', 'lols': 'lots', 'loveco': 'love', 'ndanga': 'dang', 'yotpo': 'motto', 'ndpp': 'dip', 'pitaxi': 'taxi', 'jevons': 'demons', 'qurantined': 'quarantined', 'geotag': 'gota', 'reala': 'real', 'techn': 'tech', 'ctfcu': 'ctu', 'kwa': 'wa', 'mn': 'in', 's30': 'so', 'bandmate': 'bandmates', 'fmi': 'mi', 'farmiso': 'farming', 'kannur': 'manner', 'blaque': 'plaque', 'coukd': 'could', 'autonomously': 'autonomous', 'unnessary': 'unnecessary', 'destaging': 'staging', 'burnside': 'curbside', 'freebee': 'freebie', 'daan': 'damn', 'hardup': 'hard', 'cv6': 'can', 'kattur': 'matter', 'comokete': 'complete', 'ladipoe': 'adipose', 'epepea': 'peeped', 'khonig': 'khong', 'bogarde': 'bogard', 'farragod': 'tarragon', 'shidded': 'skidded', 'dwg': 'dog', 'bedstuy': 'beastly', 'coulter': 'counter', 'driveby': 'drive', 'spose': 'spoke', 'gl11': 'glad', 'dungaree': 'dungarees', 'hoomom': 'homo', 'indepe': 'indeed', 'mayon': 'mayor', '3am': 'am', 'fjam': 'jam', 'bett': 'best', 'filo': 'film', 'conby': 'coney', 'deptt': 'depth', 'prioritised': 'prioritized', 'undstn': 'unison', 'edn': 'end', 'depute': 'deputy', 'matireal': 'material', 'yemek': 'yemen', 'ndam': 'dam', 'lunds': 'lands', 'wathcing': 'watching', 'foodtow': 'woodrow', 'permanente': 'permanent', 'hertel': 'herel', 'diay': 'day', 'lya': 'lay', 'cuneo': 'cute', 'noexcuses': 'excuses', 'floresit': 'forest', 'bci': 'bi', 'whatevs': "what's", 'app]': 'app', 'greatday': 'greatly', 'courgettes': 'corvettes', 'yourker': 'yorker', 'picc': 'pick', 'hygeine': 'hygiene', 'killinit': 'killing', 'cran': 'can', 'brooklin': 'brooklyn', 'merck': 'mercy', 'jandis': 'janis', 'flautas': 'flats', 'ncf': 'nce', '3hour': 'hour', 'careeb': 'career', 'cptr': 'car', 'itcenter': 'center', 'rathering': 'gathering', 'beckenham': 'beckham', 'itech2': 'tech', 'clr': 'car', 'kemps': 'keeps', 'katanya': 'katana', 'ght': 'get', 'myourna': 'mourn', 'cio': 'ceo', 'usku': 'us', '$crew': 'crew', 'suppliment': 'supplement', 'peggs': 'eggs', 'mabrie': 'marie', 'fernet': 'ferret', 'rishab': 'rehab', 'vidhur': 'vigour', 'lovek': 'love', 'wasatch': 'watch', 'rolle': 'roll', 'fmg': 'fog', 'jappp': 'happy', '85s': 'is', 'shii': 'shit', 'dosanna': 'hosanna', 'reposting': 'reporting', 'earworms': 'earthworms', 'boips': 'boils', 'fms': 'ams', 'realtime': 'mealtime', 'awsiot': 'assist', 'psv': 'psi', 'redacted]': 'redacted', 'dceats': 'death', 'distrbution': 'distribution', 'boise': 'noise', 'sobi': 'sob', 'knwo': 'know', 'mancave': 'manage', 'empting': 'tempting', 'samefood': 'seafood', 'sunway': 'subway', 'brightonma': 'brighton', 'swva': 'siva', 'iftar': 'after', 'sedano': 'sedan', 'delyver': 'deliver', 'hotwife': 'hotwire', 'ppls': 'pals', 'aggrieve': 'aggrieved', 'kheer': 'cheer', 'rhetorically': 'rhetorical', 'buffalfo': 'buffalo', 'lorean': 'korean', 'approvalrt': 'approval', 'nicholson': 'nichols', 'madwed': 'made', 'electy': 'elect', 'wgc': 'we', 'muscatine': 'mescaline', '6ft': 'aft', 'aibn': 'ain', 'hempstead': 'hampstead', 'mdash': 'dash', 'sfl': 'ifl', 's': 'i', 'necess': 'recess', '30min': 'min', 'cons]': 'cons', 'botir': 'both', 'raashn': 'reason', 'atay': 'stay', 'xj': 'xu', 'mozarella': 'mozzarella', '5oys': 'boys', 'jacobi': 'jacob', 'crete': 'create', 'evs': 'es', 'domy': 'dome', 'gameday': 'nameday', 'merritt': 'merit', 'lindt': 'linda', 'bthankful': 'thankful', 'vitalize': 'visualize', 'mazarssa': 'madrassa', '5b': 'ab', 'nyla': 'nya', 'tangelos': 'angels', 'magas': 'mamas', 'kithron': 'kathryn', 'rnt': 'rent', 'o2mur': 'our', 'webdev': 'wedded', 'loyola': 'loyal', 'hurryup': 'hurry', 'lifebelike': 'lifelike', 'ocala': 'call', 'gly': 'guy', 'swys': 'says', 'plotz': 'plot', 'curacao': 'cacao', 'wheeler0': 'wheeler', 'sboe': 'shoe', 'deliverable': 'deliverance', 'trimet': 'trivet', 'slidey': 'slide', 'mufucka': 'fucka', 'morg': 'more', 'convibo': 'convict', 'alipore': 'adipose', 'suff': 'stuff', 'whereareyou': 'whereveryou', 'contagos': 'contains', 'wos': 'was', 'gth': 'goth', 'swco': 'so', '16u': 'you', 'schindler': 'swindler', 'padta': 'pasta', 'amazonlos': 'amazons', 'tsm': 'tom', 'sixty60': 'sixty', 'carbis': 'carbs', 'dier': 'die', 'idleg': 'idle', 'scientect': 'scientist', 'imagen': 'image', 'hfhc': 'huh', 'unfort': 'inform', 'euta': 'esta', 'covd': 'cold', 'nyouree': "your'e", 'wpxi': 'taxi', 'greattful': 'grateful', 'mccc': 'much', 'clackety': 'clickety', 'kimberley': 'kimberly', 'winton': 'wanton', 'topshop': 'teashop', 'tulum': 'tuum', '4food': 'food', 'complainin': 'complaining', 'applauddo': 'applaud', 'shoppe': 'shopped', 'and]': 'and', 'ororama': 'diorama', 'appiah': 'appian', 'justicia': 'justice', 'cameroon': 'cameron', 'freq': 'free', 'ericka': 'erica', 'broughs': 'brought', 'srodek': 'rode', 'boxster': 'booster', 'farrington': 'farmington', 'solut': 'slut', 'unreasonbly': 'unreasonably', 'coparent': 'parent', 'dikop': 'drop', 'udg': 'dug', 'upwork': 'work', 'vidor': 'visor', 'snowmass': 'snowman', 'mccoll': 'mccall', 'businesswire': 'businesslike', 'sophy': 'soppy', 'pv': 'pa', 'simioni': 'simon', 'yegwx': 'yew', 'antalya': 'natalya', 'afari': 'safari', 'joythis': 'ofthis', 'gwx': 'we', 'noriel': 'normal', 'styx': 'sty', 'inteoduction': 'introduction', 'd4ca5': 'decay', 'rebagged': 'bagged', 'feroze': 'froze', 'km$': 'my', '1supermarket': 'supermarket', 'challeng': 'challenge', 'potatoe': 'potato', 'over40': 'over', 'yokado': 'yoko', 'challengi': 'challenge', 'paa': 'pay', 'driversless': 'driverless', 'nyse': 'nose', 'snapcart': 'snapchat', 'cfcs': 'cops', 'dd': 'do', 'btchk': 'back', 'kahn': 'khan', 'pompeno': 'pompey', 'joinin': 'joining', 'limpiest': 'simplest', 'midrand': 'midland', 'naik': 'nail', 'r63': 'run', 'yipeee': 'yippee', 'inst': 'ist', 'deett': 'deets', 'lsg': 'leg', 'anmount': 'amount', 'oyostate': 'prostate', 'fuckton': 'fuckin', 'untcares': 'unawares', 'wildtimes': 'wildfires', 'agraites': 'grates', '6km': 'him', 'qhawe': 'have', 'ruairi': 'uairi', 'hpb': 'hub', 'atxtech': 'attach', 'nmleg': 'leg', 'xax': 'tax', 'clutchh': 'clutch', 'netzero': 'nether', 'nhvr': 'her', 'gopuffed': 'puffed', 'neopets': 'reopens', 'downey': 'downed', 'crefc': 'crew', 'mulitipal': 'municipal', 'goldstream': 'gulfstream', 'additonal': 'additional', 'krogger': 'trigger', 'inexe': 'next', 'itz': 'it', 'fis': 'is', 'tnq': 'tne', 'delive': 'deliver', 'landen': 'landed', 'cwmadison': 'madison', 'schas': 'chas', 'ilwe': 'ifwe', 'umiami': 'miami', 'backpain': 'backpack', 'megain': 'regain', 'syourup': 'syrup', 'belux': 'beaux', '15p': 'up', 'kenedy': 'kennedy', 'tantrumy': 'tantrum', 'egmore': 'elmore', 'leaside': 'seaside', 'endtimes': 'centimes', 'baah': 'bath', '$lvgo': 'logo', 'nauseum': 'nauseam', 'overcomer': 'overcome', 'gardengrocer': 'greengrocer', 'clunkers': 'clunker', 'linez': 'line', 'logsitica': 'logistics', 'nonesense': 'nonsense', 'vto': 'to', 'cem': 'com', 'frowd': 'crowd', 'zzz': 'izz', 'accpt': 'accept', 'rueyling': 'ruling', 'ayesha': 'ayes', 'riskers': 'rikers', 'bfs': 'bus', 'ganne': 'anne', 'lasalle': 'lasagne', 'lardball': 'hardball', 'aventura': 'ventura', '$pso': 'ipso', 'guna': 'gun', 'rigel': 'nigel', 'sangli': 'single', 'huckin': 'fuckin', 'riovi': 'iove', 'shibe': 'shine', 'eceonomy': 'economy', 'upham': 'ham', 'vihar': 'vicar', 'norwin': 'nothin', '13yo': 'yo', 'lovecold': 'lovechild', 'upcharged': 'charged', 'portmore': 'porthole', 'love2': 'love', 'ochi': 'chi', 'thndr': 'their', 'woyare': 'wore', 'tmall': 'small', '9f': 'of', 'mck': 'mock', 'azcentral': 'central', 'doingit': 'doing', '4bd': 'and', 'snhs': 'she', 'sickaf': 'sick', 'lsv': 'liv', 'smartcar': 'smarter', 'grocercies': 'groceries', 'kimetsu': 'keiretsu', 'buying35': 'buying', 'reorderers': 'reordered', 'marquette': 'marquee', 'tesol': 'tell', '36c': 'doc', 'corporately': 'corporate', 'tumse': 'tums', 'phriste': 'phrase', 'fokkol': 'fool', 'dcdelivery': 'delivery', 'abunai': 'adonai', 'quita': 'quite', 'dvc': 'doc', 'ilona': 'llona', 'cotard': 'coward', 'lurpak': 'lurk', 'kenobi': 'knob', 'enactors': 'actors', 'aibusiness': 'business', 'myinay': 'mainly', 'mackie': 'jackie', '7mo': 'mo', 'storewide': 'statewide', 'fintish': 'finish', 'delibots': 'delights', 'shoppes': 'shopped', 'appgrooves': 'approves', 'kyjobs': 'jobs', 'z': 'i', 'frehley': 'freely', 'noobles': 'noodles', 'shitcoin': 'bitcoin', 'notessential': 'nonessential', 'quarentined': 'quarantined', 'ishopper': 'shopper', 'apotea': 'apnea', 'dux': 'due', 'danube': 'dance', 'fredrik': 'frederik', 'timesaving': 'lifesaving', 'joyive': 'motive', 'queenbee': 'queenie', '40p': 'up', 'icta': 'iota', 'legault': 'legal', 'methy': 'meth', 'appea': 'appear', '6ct': 'act', 'bitfeed': 'bitched', 'sadkids': 'skids', 'drocery': 'grocery', '6k': 'ok', 'doorpass': 'doorways', 'brashear': 'crasher', 'biard': 'board', 'opertaing': 'operating', 'reponse': 'response', 'saila': 'sail', 'whizzer': 'whizzed', 'crossedover': 'crossover', 'vber': 'ver', 'boyede': 'bored', '23yo': 'yo', 'nowine': 'nomine', 'ianne': 'anne', 'zesta': 'esta', 'bulwik': 'bulk', 'growing]': 'growing', 'harri': 'harry', 'multip': 'multi', 'rtw': 'row', 'chalav': 'chalk', 'adri': 'ari', '$afety': 'safety', 'libra': 'lira', 'todayy': 'today', 'bott': 'both', 'stanyan': 'tanya', 'diepvries': 'devries', 'oaw': 'saw', 'r': 'i', 'gangtok': 'bangkok', 'kyu': 'yu', 'vuba': 'tuba', 'coalman': 'boatman', 'wm': 'we', 'speyside': 'seaside', 'edh': 'ed', 'shipter': 'shifter', 'becausey': 'because', 'esi': 'es', 'sankey': 'sake', 'buyden': 'burden', 'portabella': 'portobello', 'brugh': 'brush', 'glados': 'gladys', 'devlped': 'helped', 'jollans': 'tollans', 'driveup': 'drive', 'papalit': 'papal', 'willco': 'willgo', 'suggst': 'suggest', 'iwnews': 'news', 'neta': 'net', 'flealand': 'leland', 'dubbs': 'dubs', 'skelly': 'smelly', 'sohungry': 'hungry', 'corndog': 'horndog', 'rawing': 'drawing', 'esf': 'es', 'gafa': 'gala', 'majorkey': 'majority', 'startupaus': 'startups', 'palacol': 'palace', 'healy': 'heavy', 'niamh': 'name', 'repub': 'rub', 'sonsini': 'sensing', 'lpdeal': 'deal', 'ul]': 'ul', 'ndis': 'dis', 'bokae': 'broke', 'bmi': 'mi', 'zenica': 'monica', 'karaage': 'garbage', 'vassel': 'vessel', 'vrrom': 'vroom', 'dkg': 'dog', 'nivea': 'never', 'pleasyntries': 'pleasantries', 'hhhmm': 'uhhmm', 'demings': 'reminds', 'coronans': 'coronary', 'tinga': 'ting', 'mclane': 'plane', 'lucero': 'ulcer', 'foodi': 'food', 'ragland': 'gland', 'latge': 'late', 'needed]': 'needed', 'norderic': 'nordic', 'malkan': 'malian', 'kriam': 'dream', 'loovve': 'love', 'sweett': 'sweet', 'a16z': 'avez', 'gmn': 'gun', 'khata': 'kata', 'swooon': 'swoon', 'tripply': 'trippy', 'saranac': 'satanic', 'cad40': 'cadet', 'hereford': 'herefor', 'houseofl': 'houseful', 'bnes': 'ones', 'blac': 'black', 'sallie': 'callie', 'jorderan': 'jordan', 'gettimg': 'getting', 'ellor': 'ellos', 'andcan': 'indian', 'handsoap': 'handsome', 'mazarians': 'bavarians', 'truro': 'turn', 'lightsabre': 'lightsaber', 'localaos': 'locals', 'naturalgas': 'naturals', 'foodshot': 'bloodshot', 'urile': 'urine', 'lawang': 'laying', 'sonas': "son's", 'sleeved': 'sleeve', 'imani': 'mani', 'answe': 'answer', 'corect': 'correct', 'hamont': 'among', 'horderes': 'orders', 'sctweets': 'streets', '7inches': 'inches', 'kse': 'use', 'flomask': 'flask', 'go2': 'got', 'jewell': 'jewels', 'mtg': 'mug', 'ncaa': 'caa', 'santaco': 'santa', 'timothee': 'timothy', 'sohagi': 'shag', 'lbct': 'but', 'rm3': 'my', 'jaquie': 'jacquie', 'suvs': 'sums', 'benfits': 'benefits', 'hmmpf': 'hump', 'dececco': 'decency', 'mimmick': 'gimmick', 'arthi': 'earth', 'niger': 'nigger', 'brightspot': 'nightspot', '5mins': 'mins', 'may24': 'maybe', 'feetto': 'feet', 'monestery': 'monastery', 'wayn': 'way', 'queerness': 'quietness', 'yellowcabs': 'yellowcake', 'begenerous': 'generous', 'linburn': 'sunburn', 'bowser': 'bowler', 'snoozles': 'noodles', 'mzuzu': 'zuzu', 'tv3': 'the', 'tastebud': 'tasted', 'dp': 'do', 'geeze': 'geese', 'semblancy': 'semblance', 'captcha': 'catch', 'lte': 'let', 'cetma': 'coma', 'warrington': 'washington', 'munje': 'mune', 'peoooe': 'people', 'pugh': 'push', 'dealnews': 'dealers', '15t': 'it', 'pratty': 'pretty', 'topline': 'towline', 'staykind': 'staying', 'deadlinewh': 'deadline', '2minread': 'misread', 'nomy': 'nosy', 'appts': 'apps', 'angularjs': 'angular', 'stade': 'state', 'apolonia': 'apollonian', 'leher': 'lever', 'molong': 'oolong', 'soyl': 'soul', 'jfre': 'are', 'ihg': 'ing', 'patient31': 'patient', 'herby': 'hereby', 'planit': 'planet', 'thespoon': 'teaspoon', 'fanta': 'santa', 'surry': 'sorry', '50rs': 'cars', 'mfm': 'mom', 'safb': 'safe', 'railers': 'raiders', 'ieee': 'ieve', 'whitlam': 'whitman', 'fenrir': 'fender', 'oneamerica': 'ofamerica', 'afforfable': 'affordable', 'maskless': 'massless', 'rellos': 'hellos', 'cabela': 'cabeza', 'bulldawg': 'bulldog', 'innover': 'inner', 'teavana': 'havana', 'npsl': 'nose', 'gnida': 'kinda', 'oldsmar': 'oldman', 'nashik': 'nash', 'bigmess': 'bigness', 'goeb': 'goes', 'soph': 'sop', 'mm': 'my', 'flustercuck': 'clusterfuck', 'wglt': 'wilt', 'advis': 'advise', 'wp': 'we', 'hexham': 'exam', 'image]': 'image', 'cxl': 'cal', 'eligibilit': 'eligibility', 'puente': 'plenty', 'marts': 'parts', 'jabi': 'jab', 'cpap': 'crap', 'milligan': 'mulligan', 'geraldo': 'gerald', 'departmentof': 'department', 'riskless': 'reckless', 'kuj': 'ku', 'honeybears': 'honeybees', 'ottered': 'uttered', 'khosla': 'koala', 'psh': 'push', 'whas': 'what', 'winknews': 'winners', 'actnews': 'actress', 'policano': 'volcano', 'juepier': 'jumper', 'warninks': 'warnings', 'pharmeasy': 'pharmacy', 'n95s': 'news', '21x': 'six', 'sekar': 'sear', 'neel': 'need', 'hallp': 'hall', 'ga02': 'gave', 'damit': 'admit', 'gorgie': 'gorge', 'ndabag': 'teabag', 'fcc': 'fac', 'pinego': 'pine', 'iock': 'rock', 'bhutan': 'human', 'blessedaf': 'blessed', 'ccdoc': 'doc', 'newsok': 'news', 'excalibursg': 'excalibur', 'padnoma': 'panama', 'slco': 'slo', 'bozeman': 'foreman', 'resonders': 'responders', 'unifill': 'unified', 'herocom': 'heroism', 'candideats': 'candidate', 'vanc': 'van', 'mounte': 'mount', 'pusha': 'push', 'ivanka': 'ivana', 'syour': 'your', 'rapidgo': 'rapid', 'radlett': 'raddest', 'dnt': 'dna', 'harwood': 'hardwood', 'nca': 'na', 'valwin': 'vain', 'ditmas': 'dismal', 'cjn': 'can', 'cityam': 'city', 'yegre': "ye're", '1month': 'month', 'phori': 'phone', 'coldish': 'polish', 'drewel': 'drew', 'zep': 'zip', '50acts': 'acts', 'nakuru': 'nature', 'defund': 'defend', 'q4': 'qi', 'comingsoon': 'comingupon', 'eit': 'it', 'foong': 'fong', 'superma': 'superman', 'etauto': 'auto', 'kristal': 'cristal', 'honeymooner': 'honeymooned', 'ragrets': 'regrets', 'whelan': 'when', 'fornula': 'formula', '89your': 'your', 'qultist': 'cultist', 'ayame': 'came', 'portugese': 'portuguese', 'sudan': 'sedan', 'brxassist': 'bassist', '4ths': 'the', 'ckeaners': 'cleaners', 'essentia': 'essential', 'hendo': 'kendo', 'portgual': 'portugal', 'theisen': 'these', 'tita': 'tits', 'carhop': 'chop', '49th': 'with', 'hortons': 'horton', 'rakats': 'rats', 'wd': 'we', '20yours': 'yours', 'dcist': 'list', 'ch': 'oh', 'underpay': 'underway', 'thingamybobs': 'thingamabob', 'pantene': 'panting', 'winowing': 'winnowing', 'wasserman': 'waterman', 'crumlin': 'crumbling', 'loly': 'holy', 'mff': 'off', 'm$': 'my', 'happyso': 'happyto', 'rayleigh': 'raleigh', 'canteloupes': 'cantaloupes', 'selloff': 'sellout', 'inteliot': 'interior', 'fuckong': 'fucking', '95month': 'month', 'bettina': 'betting', 'smm': 'sum', 'chooser': 'choose', 'btween': 'between', 'naija': 'naina', 'stush': 'stash', 'potti': 'potty', 'fuentes01': 'fuentes', 'eeking': 'seeking', '8hours': 'hours', 'enterpreneur': 'entrepreneur', 'hybangalore': 'bangalore', 'linfox': 'info', 'catcall': 'catchall', 'femdomme': 'verdomme', 'jl': 'al', 'stawell': 'swell', 'grovy': 'gravy', 'ventu': 'vent', 'chalta': 'chat', '85yo': 'yo', 'checkins': 'checking', 'basketpk': 'basket', 'paidle': 'paddle', 'kaapo': 'kakapo', 'fatayer': 'father', 'rmvd': 'read', 'joyyou': 'doyou', 'wooster': 'rooster', 'j4u': 'ju', 'utm': 'um', 'zambo': 'mambo', 'hotpoint': 'hotpot', 'trebek': 'tree', 'quezon': 'queen', 'w': 'i', 'swingsets': 'swingers', 'fetchers': 'fetches', 'unbun': 'union', 'sakte': 'sake', 'tauseel': 'tassel', 'paypa': 'papa', 'phsd': 'had', 'fredricka': 'frederick', 'humana': 'human', '2events': 'events', 'togethersf': 'together', 'festi': 'fest', 'cbse': 'case', 'jb': 'job', '13may': 'may', 'hiirt': 'hit', 'waan': 'warn', 'vqcay': 'vacay', '8mar': 'omar', 'backline': 'backing', 'npos': 'nos', 'twt': 'two', 'scroogey': 'scrooge', 'facil': 'fail', 'i6': 'i', 'reily': 'rely', 'californi': 'california', 'bevvies': 'berries', 'cr3': 'cry', 'cca': 'ca', 'rm8': 'my', 'nehru': 'near', 'vit': 'it', '14hour': 'hour', 'mayb': 'maybe', 'jfs': 'ifs', 'kanti': 'anti', 'newsthe': 'nestle', 'dingdang': 'dingding', 'distric': 'district', 'rocketfuel': 'pocketful', 'nxos': 'nos', 'elpaso': 'paso', 'kobo': 'hobo', 'anylist': 'analyst', 'teaneck': 'redneck', 'inve': "i've", 'pli': 'li', 'ndaita': 'data', 'usnews': 'news', 'hongkong': 'honking', 'ato': 'to', '60us': 'us', 'dema': 'demo', 'vch': 'ich', 'aerith': 'earth', 'jenipher': 'decipher', 'pleadingso': 'pleadings', 'starting21': 'starting', '1your': 'your', 'lidl': 'lid', 'ulabox': 'labor', 'mdlz': 'male', 'anent': 'agent', 'reblogging': 'blogging', 'recy': 'rely', 'cks': 'cos', 'exciti': 'excite', 'refubd': 'refund', 'aigues': 'argues', 'kaftan': 'caftan', 'sml': 'sol', 'deliveer': 'deliver', 'tweet50': 'tweet', 'noticias': 'noticing', 'yohr': 'your', 'smtms': 'seems', 'rj': 're', 'horwich': 'herwith', 'understd': 'understa', 'kroeger': 'roger', 'daragh': 'damage', 'trialled': 'trialed', 'bhorta': 'short', 'zongo': 'bongo', 'foodshops': 'bookshops', 'akka': 'aka', 'neighobr': 'neighbor', 'preground': 'foreground', 'foodtec': 'foodie', '$sdon': 'son', 'rstechnology': 'technology', 'sibi': 'tibi', 'peapod': 'weapon', 'yoir': 'your', 'frenemies': 'enemies', 'philade': 'pilate', 'triaging': 'bringing', 'canwnt': "can't", 'immediates': 'immediate', 'everyware': 'everywhere', '$tlt': 'felt', 'hageman': 'caveman', 'denyelle': 'danielle', 'vgi': 'vi', 'dishevel': 'disheveled', 'consumabl': 'consumable', 'stds': 'studs', 'tamari': 'tamara', 'gesd': 'ged', 'harvie': 'have', 'lwalther': 'walther', 'korosec': 'morose', 'lhena': 'lena', 'jumla': 'juma', 'hencing': 'fencing', 'hrcommunity': 'community', 'fok': 'for', 'sparker': 'sparked', 'benchmarked': 'benchmark', 'mymessenger': 'messenger', 'mtia': 'mia', 'shopco': 'shop', 'nutrimart': 'nutriment', 'ubuntu': 'bunt', 'beetches': 'bitches', 'joyi': 'joy', 'igive': 'give', 'ddddd': 'daddy', '4$': 'i', 'pagel': 'page', 'brainerd': 'brained', 'losin': 'losing', 'reshopping': 'shopping', 'kete': 'kite', 'opetition': 'petition', 'aril': 'rail', '0c': 'ac', 'daz': 'day', 'flexibilty': 'flexibility', 'gfejo': 'geo', 'xpress': 'press', '$zm': 'him', 'closter': 'closer', 'fvcks': 'fucks', 'dorritos': 'burritos', 'recieving': 'receiving', 'readmission': 'admission', 'box8': 'box', 'strapline': 'strapping', 'rolltide': 'collide', 'buzzfeed': 'buzzed', 'gfu': 'fu', 'trello': 'tell', 'r45': 'run', 'sw6': 'we', 'similiar': 'similar', 'buybacks': 'buyback', 'aex': 'sex', 'ds9': 'is', 'adriving': 'driving', 'elop': 'flop', 'vela': 'veal', 'novampires': 'vampires', 'alohari': 'aloha', 'mny': 'my', 'cr7': 'cry', 'livelove': 'livelong', 'brower': 'broker', 'iri': 'ii', 'hardik': 'hard', 'stabbs': 'stabs', 'coronapr': 'coroner', 'pahulam': 'paula', 'jalen': 'galen', 'vitas': 'vital', 'brahms': 'brahma', 'overfill': 'overkill', 'shaundy': 'shandy', 'mealkits': 'merkits', '15percent': 'percent', 'plse': 'else', 'challange': 'challenge', 'foodme': 'foodie', 'dejobs': 'jobs', 'lazmart': 'hazmat', 'commodified': 'commodities', 'scarpers': 'scarers', 'pahle': 'pale', 'sevices': 'services', 'nabil': 'nail', 'ksh': 'ash', 'vipkids': "vicki's", 'winamac': 'wingman', 'rampi': 'ramp', 'cravath': 'cravat', 'angiyi': 'angry', 'steups': 'steps', 'heim': 'him', 'roback': 'back', '$cgc': 'cgi', 'beelivery': 'delivery', 'hoisin': 'cousin', 'things]': 'things', 'shlepping': 'schlepping', 'ktul': 'kill', 'rrhi': 'hi', 'bengay': 'bengal', 'linde': 'line', 'hamid': 'amid', '99p': 'up', '2my': 'my', 'superwomen': 'superwoman', 'gj': 'go', 'hont': 'hot', 'timeshar': 'timeshare', 'vanfood': 'manhood', 'mitron': 'matron', 'volsea': 'volley', 'overdrafted': 'overrated', 'gerri': 'gerry', 'mafeki': 'maki', 'teapigs': 'teabags', 'completley': 'completely', 'bothell': 'bother', 'mayonaise': 'mayonnaise', 'ps34': 'psi', 'gnp': 'gap', 'dufferin': 'suffering', 'wangsta': 'gangsta', 'servisa': 'service', 'mccaffrey': 'caffrey', 'youpeople': 'ofpeople', 'whamy': 'whammy', 'accorderi': 'accorded', 'cheapwr': 'cheaper', 'freshnes': 'freshness', 'a1': 'a', 'smooshing': 'smooching', 'fuad': 'fund', 'hedgeend': 'hedged', 'focaccia': 'faccia', 'wangige': 'angie', 'mnt': 'met', 'msp': 'map', 'leman': 'lean', 'denverwx': 'denver', 'sauceit': 'sauce', 'mailmain': 'mailman', '6c': 'ac', 'synomous': 'venomous', 'ph20': 'phil', 'demorats': 'democrats', 'uniqlo': 'unique', 'akure': 'azure', 'microgrants': 'micrograms', 'youxuan': 'youcan', 'drummond': 'drumming', 'harras': 'harris', '8m': 'am', '1ar': 'car', 'stache': 'state', 'adheren': 'adhere', 'ps45': 'psi', 'taman': 'taxman', '2a': 'a', 'ekstrom': 'wistrom', 'alphaone': 'alphonse', 'postmate': 'prostate', 'reimagines': 'reimagine', 'boulderans': 'boulders', 'popul': 'soul', 'lahinch': 'launch', 'leshop': 'shop', 'deboned': 'debone', 'oag': 'bag', 'annou': 'annoy', 'jefrey': 'jeffrey', 'friso': 'frisk', 'christmaseve': 'christmases', 'loship': 'ship', 'rambaua': 'lambada', 'uncaptured': 'captured', 'liesman': 'lifespan', 'tlhogo': 'logo', 'rediff': 'relief', 'pff': 'off', 'muh': 'much', 'conve': 'convey', 'kankar': 'banker', 'ctct': 'cut', 'n1': 'no', 'happie': 'happier', 'optionally': 'rationally', 'squatchy': 'scratchy', 'daugherty': 'daughterto', 'yha': 'ya', 'r47': 'run', 'queenx2': 'queen', 'rvs': 'revs', 'craz': 'crazy', 'yayy': 'yay', 'doodol': 'voodoo', 'deir': 'dear', 'hitraffic': 'traffic', 'ps27': 'psi', 'fatega': 'fate', 'lyz': 'ly', 'shouldv': 'should', 'bushiri': 'hashiri', 'grabph': 'graph', 'desis': 'desks', 'sofancy': 'fancy', 'pandemi': 'pandemic', 'beccles': 'eccles', 'pressers': 'presses', 'ruready': 'already', 'naks': 'naps', 'isama': 'osama', 'visist': 'visit', 'fes': 'few', 'eletric': 'electric', 'p49': 'put', 'uxmatters': 'matters', '0mg': 'my', 'ereading': 'reading', 'pipul': 'pipe', 'goldish': 'goldfish', 'superscoop': 'supercool', 'keifer': 'heifer', 'scaleup': 'scale', 'aimeday': 'someday', 'generationav': 'generational', 'amac': 'mac', 'woobly': 'woolly', '1of': 'of', 'deoderant': 'deodorant', 'digiday': 'digital', 'iterate': 'literate', 'ughhh': 'ugh', 'titi': 'tits', 'vau': 'van', 'uncreative': 'creative', 'miry': 'mary', 'sarecycles': 'recycles', 'sikhaw': 'shaw', 'freshoo': 'fresh', 'tigre': 'tire', 'balanceit': 'balance', 'denlol': 'dental', 'kinga': 'king', 'hcws': 'hows', 'diversecity': 'diversity', 'hoga': 'yoga', 'promotores': 'promotes', 'xer': 'her', 'carcare': 'warfare', 'kueng': 'kung', 'd4livery': 'delivery', '2start': 'start', '3months': 'months', 'huhuh': 'huh', 'mangalore': 'bangalore', 'mcbq': 'mob', 'rmh': 'rah', 'hehee': 'hehe', 'heeee': 'here', 'stollen': 'stolen', 'amqc': 'am', 'camcab': 'cancan', 'mfkas': 'makes', 'suntec': 'hunter', 'hfresh': 'fresh', 'medicins': 'medicine', 'semasa': 'smash', 'viks': 'vics', 'butche': 'butcher', 'gottcha': 'gotcha', 'dingwa': 'dinga', 'unke': 'nuke', '3w': 'ow', 'weho': 'who', 'citiz': 'city', 'csrb': 'curb', 'timesaver': 'lifesaver', 'mckinsey': 'mckinley', '30x': 'six', 'exf': 'ex', 'scrrem': 'scared', 'stroodly': 'strongly', 'complan': 'complain', 'photographe': 'photograph', 'jawdrop': 'dewdrop', 'abanged': 'banged', 'x]': 'xu', 'imold': 'mold', 'kps': 'ups', 'mnl': 'mil', 'willget': 'wallet', 'sfw': 'saw', 'unrefrigerated': 'refrigerated', 'otcs': 'otis', 'thepoint': 'thejoint', 'mauqa': 'maura', 'dori': 'dorm', 'afm': 'am', 'diba': 'dibs', 'precycle': 'recycle', 'palying': 'playing', 'tgi': 'ti', '$tmus': 'thus', 'innotech': 'nanotech', 'haden': 'haven', 'arvada': 'armada', 'projo': 'promo', 'agger': 'anger', 'relie': 'relief', 'godyke': 'dyke', 'momboss': 'combos', 'lupper': 'upper', 'betul': 'betel', 'edu': 'du', 'mcann': 'mann', 'dav': 'day', 'randieri': 'handier', 'emersim': 'emerson', 'naptimes': 'naptime', 'kalki': 'kali', 'unavail': 'naval', '12days': 'days', 'tirred': 'tired', 'mensans': 'means', 'is10': 'is', 'southpark': 'southpaw', 'nieghbor': 'neighbor', 'fram': 'from', 'preske': 'press', 'skald': 'scald', 'oopss': 'oops', 'lordere': 'order', 'pawpa': 'papa', 'gzgrocery': 'grocery', 'tier4': 'tier', 'masepa': 'massa', 'raleys': 'rules', 'doco': 'dock', 'rana': 'ran', 'teronda': 'veranda', 'asgher': 'asher', 'asing': 'asking', 'zambales': 'gambles', 'rackham': 'arkham', 'pissies': 'pisses', 'grt': 'get', 'deming': 'diming', 'hnn': 'inn', 'swaldale': 'swaddle', 'numeracy': 'numeric', 'mirae': 'mirage', 'popup': 'pop', 'reve': 'eve', 'pfp': 'pop', '5mg': 'my', 'myhilo': 'milo', 'genz': 'gene', '\\\\': 'i', 'brittni': 'brittle', 'cobran': 'cobra', 'vday': 'day', 'lrl': 'lil', 'cps5off': 'castoff', 'mobike': 'mobile', 'waifu': 'waif', 'huston': 'houston', 'freds': 'feds', 'shyt': 'shut', 'hogg': 'hog', 'reya': 'rea', '1dj': 'do', 'lastrt': 'last', 'perimeterx': 'perimeter', 'vneck': 'neck', '18th': 'with', 'bootake': 'footage', 'ebucks': 'bucks', 'austal': 'usual', 'delh': 'del', 'unlimted': 'unlimited', 'adulted': 'adult', 'unmc': 'unc', 'smallact': 'smallest', 'webmaster': 'webster', 'sadtp': 'sad', 'overtipped': 'overstepped', 'rsv': 'rev', 'vp': 'up', 'millington': 'wellington', 'morelia': 'cordelia', 'cermak': 'german', 'langar': 'hangar', 'sitti': 'sittin', 'southbronx': 'southrons', 'pando': 'panda', 'shashlik': 'shashi', 'preferentially': 'preferential', 'accredit': 'credit', 'avo': 'ago', 'shoddily': 'shoddy', 'uts': 'us', 'savy': 'say', 'extruder': 'extruded', 'geofence': 'defence', 'uk]': 'uk', 'runniing': 'running', 'gogurt': 'yogurt', 'pyouramids': 'pyramids', 'hubard': 'hubbard', 'matigara': 'marinara', 'accustome': 'accustomed', 'rekt': 'rest', 'sabra': 'sara', 'progresso': 'progress', 'aashish': 'hashish', 'liquir': 'liquor', 'waise': 'raise', 'bluh': 'blue', 'huevona': 'huevos', 'fasher': 'father', 'influencers': 'influences', 'intendo': 'intend', '27your': 'your', 'delcan': 'declan', 'msy': 'my', 'steltzer': 'seltzer', 'kauai': 'kara', '$big': 'big', 'richtung': 'righting', '09am': 'am', 'washouts': 'washout', 'mender': 'tender', 'karabo': 'arab', '20mts': 'emts', 'coldout': 'holdout', 'saveon': 'save', 'rasher': 'rather', 'steepen': 'steeped', 'appleton': 'appletini', 'muft': 'must', 'nw3': 'we', '30day': 'day', 'otr': 'or', 'boxwine': 'boxing', 'uninterupted': 'uninterrupted', 'cnu': 'nu', 'wheatley': 'wheaties', 'politicals': 'political', 'shadi': 'shade', 'contrarily': 'contrary', 'raley': 'rally', 'penney': 'penny', 'fraund': 'fraud', 'thrombotic': 'thrombosis', 'anson': 'arson', 'bzos': 'bros', 'nyda': 'nada', 'fuuuck': 'fuck', 'kvoa': 'koa', 'disabke': 'disable', '$didi': 'didi', 'wisbech': 'lisbeth', 'dispite': 'despite', 'azurine': 'urine', 'flybuys': 'flyboys', 'mommas': 'momma', 'kray': 'pray', 'oho': 'who', 'rm2': 'my', 'djuvec': 'duvet', 'ippadi': 'ippai', 'westview': 'westies', 'sunderland': 'wonderland', 'jyhind': 'behind', 'ods2': 'gods', 'sadana': 'banana', 'iost': 'most', 'bxxsting': 'busting', 'medi': 'media', 'shoppiiing': 'shopping', 'nqunqa': 'nuna', 'inductee': 'inducted', 'facillitate': 'facilitate', '11your': 'your', 'piggily': 'piggly', 'giftcrd': 'gifted', 'lled': 'lied', 'wout': 'out', 'f$': 'fu', 'giphy': 'gipsy', 'kuaidi': 'uairi', 'dville': 'ville', 'adag': 'adam', 'rzusa': 'rush', 'theywon': 'theywant', 'untung': 'unsung', 'khona': 'khong', 'gnss': 'goss', 'ika': 'aka', 'laahd': 'lead', 'angelinos': 'angelina', 'tuedays': 'tuesdays', 'wion': 'win', 'fornthose': 'forthose', 'existance': 'existence', 'swagbucks': 'starbucks', 'binaryic': 'binary', 'panjang': 'panting', 'elfie': 'selfie', 'cpassy': 'classy', '10june': 'june', 'mnts': 'ants', 'unphased': 'phased', 'weapplaud': 'applaud', 'sprouter': 'sprouted', 'dof': 'of', 'buz': 'but', 'unsocialized': 'socialized', 'becoz': 'become', 'generacion': 'generation', 'cabos': 'cabs', 'phoneline': 'shoreline', 'effin': 'effing', 'utili': 'untili', 'usw': 'us', 'garcetti': 'carcetti', 'acidify': 'acidity', 'starbux': 'startup', 'helll': 'hell', 'qty': 'ty', 'okkk': 'okay', 'patra': 'para', '20years': 'years', 'capitec': 'capital', 'lapid': 'laid', 'yebo': 'yeo', 'carlsbad': 'carload', 'luckyme': 'lucky', 'mim': 'him', 'lovegly': 'lovely', 'bestfriend': 'befriend', 'raintime': 'ragtime', 'bren': 'been', 'fym': 'gym', 'cntr': 'into', 'refloat': 'float', 'realale': 'really', 'icart': 'cart', 'barstards': 'bastards', 'avadi': 'avada', 'repurposes': 'repurposed', 'f2h': 'for', 'anot': 'not', 'kor': 'for', 'niaz': 'diaz', 'cognisant': 'cognizant', 'safewa': 'safe', 'dumplin': 'dumpling', 'smokeout': 'stakeout', 'exploi': 'exploit', 'teamrt': 'heart', 'deficult': 'dificult', 'econsave': 'concave', 'ccaas': 'class', 'brinker': 'drinker', 'nolife': 'life', 'mulp': 'mule', 'naugh': 'laugh', 'odr': 'or', 'pvd': 'pod', 'wkly': 'wily', 'tajin': 'takin', 'kariye': 'marine', 'mgmt': 'get', 'bdubs': 'dubs', 'ello': 'hello', 'tirol': 'girl', '$ivv': 'give', '75m': 'him', 'warframe': 'warfare', 'tagine': 'tagline', 'uptil': 'until', 'murd': 'mud', 'bougiest': 'toughest', 'botha': 'both', 'amoke': 'smoke', 'jssue': 'issue', 'uiux': 'six', 'alnasir': 'altair', 'lyourical': 'lyrical', 'myhustle': 'hustle', 'deplorables': 'deplorable', 'elkhart': 'eckhart', 'resells': 'resell', 'poha': 'poca', 'bookr': 'book', 'havok': 'havoc', 'entrepre': 'entree', 'tailers': 'trailers', 'makerbus': 'makers', 'njoy': 'enjoy', 'terribble': 'terrible', 'approv': 'approve', 'wldnt': 'want', 'statup': 'status', 'mx': 'my', 'jgm': 'jam', 'hmv': 'hav', 'ctd': 'cod', 'meijier': 'messier', 'tw2': 'two', 'defelice': 'defence', 'apricart': 'apricot', 'txlege': 'allege', 'horas': 'horns', 'henster': 'hester', 'carrell': 'carroll', 'engel': 'angel', 'etal': 'metal', 'petshop': 'potshot', 'dacia': 'dacha', 'maligai': 'malign', 'toya': 'toy', 'wc': 'we', 'trickels': 'trickles', 'dbanyan': 'banyan', 'coaste': 'coast', 'ltrs': 'lars', 'r2d2': 'red', 'ebu': 'bu', 'soupergirl': 'supergirl', 'gettig': 'getting', 'baroud': 'around', 'barganier': 'bargained', 'brasil': 'basil', 'ctamember': 'camembert', 'recenty': 'recently', 'b]': 'be', 'empaynado': 'empanada', 'ecad': 'cad', '85your': 'your', 'usanews': 'sinews', 'boulia': 'coulda', 'pgemployee': 'employee', 'timonium': 'titanium', 'tupelo': 'tuxedo', 'joplin': 'join', 'apke': 'ape', 'nootropics': 'nootropic', 'levu': 'levy', 'thaaa': 'thata', 'islandia': 'island', 'nuroai': 'kurai', 'fusian': 'fusion', 'ppod': 'pod', 'bonzo': 'bono', 'wec': 'we', 'iloveyou': 'loveyou', 'obgyns': 'organs', 'garuda': 'garda', 'createher': 'breather', 'melville': 'neville', 'brokened': 'brokered', 'generat': 'general', 'htx': 'hex', 'sausalito': 'causality', 'crabbit': 'rabbit', 'eithe': 'either', 'reupping': 'repping', 'slovak': 'soak', 'unpackage': 'package', 'ahahah': 'haha', 'kerby': 'derby', 'lcg': 'leg', 'transfereed': 'transferred', 'sufferring': 'suffering', 'fbjam': 'beam', 'anothers': 'another', 'culineer': 'cylinder', 'romesco': 'romeo', 'epa': 'era', 'balik': 'malik', 'loneng': 'long', 'conseq': 'consec', 'yaself': 'myself', 'matara': 'matata', 'unrequested': 'requested', 'oll': 'all', 'datamining': 'detaining', 'mn01': 'mnie', 'jhola': 'hola', 'tripnation': 'urination', 'bihh': 'bish', 'conciders': 'considers', 'xl': 'al', 'mile12': 'miles', 'rasies': 'raises', 'dest': 'best', 'foodbooking': 'goodlooking', 'bsod': 'sod', 'ohoho': 'photo', 'amigas': 'amigos', 'loto': 'lot', 'ojesh': 'ones', 'wznae': 'wine', 'hrc': 'arc', 'tseen': 'seen', 'vika': 'via', 'tama': 'mama', 'regionus': 'regions', 'usfhero': 'usher', 'tilden': 'hidden', 'arnews': 'news', 'coronanyc': 'coronary', 'cnp': 'cop', 'rous': 'vous', 'danestone': 'sandstone', 'hagley': 'haley', 'wms': 'was', '22yo': 'yo', 'mna': 'man', 'alwz': 'alwa', 'jezuz': 'jesus', 'approacheth': 'approached', 'loulou': 'lulu', 'jobcuts': 'bobcats', 'interwebs': 'interweb', 'lakesun': 'lakes', 'pressels': 'presses', 'spadina': 'sparing', 'texa': 'tea', 'terfs': 'terms', 'cristela': 'cristina', 'substitue': 'substitute', 'gompyo': 'gimpy', 'godbless': 'godless', 'wreslting': 'wrestling', 'arev': 'are', 'ubud': 'bud', 'mfa': 'ma', 'dvds': 'dads', 'yo73': 'you', 'hdmi': 'hemi', '60yours': 'yours', 'pertol': 'petrol', '15mph': 'lymph', 'jerz': 'jerk', 'kj': 'ke', 'jate': 'hate', 'masjid': 'majid', 'carfare': 'warfare', 'upselling': 'upwelling', 'teamcld': 'teamed', 'muna': 'tuna', 'bernstein': 'einstein', 'sosusa': 'sous', 'cmdr': 'car', 'gigger': 'bigger', 'maintaun': 'maintain', 'ada30': 'adapt', 'woodlawn': 'woodland', 'eveleth': 'everyth', 'thisissg': 'thisis', 'wll': 'all', 'baahar': 'bazaar', 'over3': 'over', 'ralf': 'half', 'grofe': 'grove', 'wmata': 'mata', 'iee': 'see', 'rak': 'ran', 'bruhh': 'brush', 'brom': 'from', 'plesure': 'pleasure', 'estrategico': 'strategic', 'explainer': 'explained', 'wbz': 'wiz', 'bryla': 'bra', 'christee': 'christen', 'itvalue': 'value', 'rupco': 'repo', 'deal]': 'deal', 'sathe': 'bathe', 'sarc': 'arc', 'adei': 'dei', 'mccomb': 'mccombs', 'kora': 'fora', 'dallastx': 'ballast', 'alonso': 'alonzo', 'nfm': 'nom', 'fekkin': 'fuckin', 'perturb': 'perjury', 'cxchat': 'chat', 'kielbasas': 'kielbasa', 'expected2': 'expected', 'chutiye': 'chute', 'morrions': 'morrisons', 'jhark': 'shark', 'chch': 'much', 'tca': 'tea', 'forsure': 'fortune', 'ligne': 'line', 'winbig': 'winning', 'techcr': 'teacher', 'tsao': 'tsar', 'dcfood': 'food', 'pharamacies': 'pharmacies', 'kailua': 'laila', 'camsites': 'campsites', 'maalot': 'ballot', 'posession': 'possession', 'duror': 'juror', 'iniatives': 'natives', '$deo': 'deo', 'kron': 'iron', 'queing': 'queuing', 'monacan': 'monican', 'cont': 'count', 'whizzy': 'whizz', 'sheradon': 'sheldon', 'theg': 'the', 'respo': 'repo', 'roally': 'really', 'tellya': 'telly', 'marianos': 'marines', 'forgeon': 'forget', 'gbr': 'gor', 'achill': 'chill', 'foldable': 'solvable', 'immma': 'mama', 'mrman': 'merman', 'hommus': 'hummus', 'intrapreneur': 'entrepreneur', 'unfi': 'unfit', 'scaleable': 'saleable', 'maule': 'male', 'pgrs': 'pars', 'bronxites': 'bronies', 'kayl': 'karl', 'erra': 'era', 'cahr': 'car', 'pmdd': 'padd', 'vieew': 'view', 'hatchin': 'hatching', 'partington': 'arlington', 'okemos': 'memos', 'appto': 'apply', 'squizz': 'quiz', 'ettes': 'estes', 'epidemiologists': 'epidemiologist', 'tr': 'to', 'braais': 'brain', 'mysanity': 'insanity', 'tenk': 'ten', 'joysign': 'consign', 'nx': 'no', 'itrtg': 'trig', '$weyl': 'well', 'aaysee': 'hayseed', 'givin': 'given', 'consindered': 'considered', 'tysm': 'tym', 'pwede': 'puede', 'bestdel': 'besides', 'ortlieb': 'outlier', 'keekee': 'keeper', 'frou': 'from', 'fastbag': 'fasting', 'potatey': 'potato', 'creamation': 'cremation', 'bbkat': 'beat', 'treet': 'street', 'mxm': 'mom', 'apartmt': 'apartment', 'hamps': 'camps', 'finnity': 'dignity', 'beaulieu': 'beauties', 'i9': 'i', '55th': 'with', 'tassc': 'ass', 'comitment': 'commitment', 'morro': 'morrow', 'appo': 'app', 'nologic': 'logic', 'mashable': 'washable', 'night2': 'night', 'jorts': 'sorts', 'omani': 'omni', 'isha': 'isa', 'ppr': 'per', 'tcha': 'tha', 'emerg': 'emerge', 'ubable': 'unable', 'oblast': 'blast', 'bandla': 'banda', 'atten': 'attend', 'begger': 'bigger', 'ecto': 'echo', 'walphie': 'ralphie', 'stoppd': 'stopped', 'disinfecter': 'disinfected', 'pffizer': 'officer', '15lbs': 'bulbs', 'matabolism': 'metabolism', 'anythingz': 'anything', '39z': 'cuz', 'combe': 'come', 'dabri': 'dari', 'doubl': 'doubt', 'nida': 'vida', 'vernia': 'hernia', '$chwy': 'chewy', 'scoby': 'scooby', 'craigie': 'prairie', 'roboto': 'robot', 'bern': 'been', 'addon': 'addons', 'assigne': 'assigned', 'usbusiness': 'business', 'hoa': 'how', 'fistfull': 'fistful', 'pymts': 'puts', 'again]': 'again', 'alondra': 'lndra', '8l': 'al', 'lnr': 'lor', 'egger': 'eager', 'nutrit': 'outfit', 'wgtn': 'with', 'gofrugal': 'frugal', 'midstream': 'mainstream', 'britany': 'brittany', 'eca': 'era', 'notideal': 'notional', '7park': 'park', 'weebly': 'weekly', 'statists': 'stations', 'marthene': 'earthen', 'mealime': 'mealtime', 'arrton': 'arrow', 'yuyan': 'yuan', 'noice': 'nice', 'yonca': 'gonna', '$xly': 'only', 'coronamx': 'coronary', 'kotlin': 'goblin', 'trippin': 'tripping', 'ttump': 'trump', 'kayo': 'mayo', 'shaplist': 'shoplift', 'vizag': 'vital', 'applaude': 'applause', '3but': 'but', 'nabe': 'name', 'hangups': 'hangs', 'ungrat': 'unwrap', 'daaaam': 'salaam', 'thow': 'how', 'alomost': 'almost', 'liers': 'lies', 'itv': 'it', 'lovefl': 'love', 'griess': 'griefs', 'kuaf': 'kung', 'analyti': 'analytic', 'brapa': 'bapa', 'trialing': 'trailing', 'acohol': 'alcohol', 'stawberries': 'strawberries', 'birista': 'barista', 'saludtues': 'salutes', '3ish': 'wish', '5qt': 'it', 'franklintn': 'franklin', 'transmision': 'transmission', 'plasa': 'plasma', 'sextion': 'section', 'hullaboo': 'hullabaloo', 'abc13': 'abc', 'yeay': 'year', 'rbc': 'rec', 'phvendors': 'vendors', 'lovejust': 'havejust', 'gwinett': 'gwyneth', 'shadipur': 'shadier', 'uui': 'qui', 'applaudno': 'applaud', 'keke': 'eke', 'waseem': 'seem', '2go2': 'got', 'shopp': 'shop', 'approvalshe': 'approvals', 'sachs': 'sacks', 'phar': 'par', 'zww': 'oww', 'lamont': 'lament', 'fairmac': 'tarmac', 'reutersjp': 'reuters', 'liay': 'lay', 'gullane': 'gallant', 'wulf': 'wolf', 'robots1': 'robots', 'galt': 'salt', 'metra': 'metro', 'sfit': 'sit', 'codiant': 'radiant', 'ohokay': 'okay', 'sssh': 'sash', '$mck': 'back', 'vlogs': 'logs', 'newsgang': 'newsman', 'induviduals': 'individuals', 'pundoq': 'undo', 'indiep': 'indie', 'szas': 'seas', 'explaination': 'explanation', 'govrnment': 'government', 'eyournd': 'yound', 'nalini': 'alibi', 'armagh': 'armada', 'byob': 'bob', 'biscoff': 'scoff', 'yanna': 'wanna', 'pssble': 'possible', 'yoself': 'yourself', 'voddy': 'toddy', 'aji': 'aii', 'mcmerchant': 'merchant', 'brar': 'bear', 'genpie': 'genie', 'corporator': 'corporate', 'wdbo': 'do', 'mcmillon': 'macmillan', '70your': 'your', 'grabon': 'grab', 'wavah': 'watch', 'hassl': 'hassle', 'costco]': 'costco', 'orrville': 'orville', 'clar': 'car', 'butterfinger': 'butterfingers', 'ashlee': 'ashley', 'grr8': 'girl', 'manipal': 'animal', 'zomoto': 'tomato', 'pehly': 'rely', 'jta': 'ta', '11c': 'doc', 'rweff': 'reef', 'zyada': 'yada', 'emylou': 'employ', 'norderies': 'orderlies', 'micropa': 'micro', 'panner': 'manner', 'plushies': 'pushes', 'spidering': 'shivering', '27k': 'ask', '7min': 'min', 'stondon': 'london', '6days': 'days', 'husbamd': 'husband', 'ecq': 'eco', 'whil': 'while', 'fckng': 'fucking', 'mcdo': 'modo', 'helllla': 'helluva', 'seasonally': 'personally', 'honeybuns': 'honeybunny', 'uidesign': 'design', 'savefood': 'seafood', 'retur': 'return', 'restaraunt': 'restaurant', 'cargob': 'cargo', 'wowa': 'wow', 'geary': 'gear', 'livemint': 'livening', 'ldreporter': 'reporter', 'knysna': 'nyssa', 'clampers': 'campers', 'cbts': 'cuts', 'miti': 'mini', 'fifo': 'fife', 'chernin': 'heroin', '5million': 'million', 'smishes': 'smashes', 'benne': 'benny', 'accorder': 'accorded', 'funnyuse': "funny's", 'tuking': 'taking', 'swamman': 'shaman', 'acceptor': 'accept', 'bbreaking': 'breaking', 'deathful': 'healthful', 'fournado': 'tornado', 'procrastinator': 'procrastination', 'skinsuit': 'swimsuit', 'screamin': 'screaming', 'momfy': 'mommy', 'skyla': 'skylar', 'paymnt': 'payment', '2parties': 'parties', 'sucralose': 'sucrose', 'milkwoman': 'milkman', 'calgary': 'calvary', 'bakorder': 'border', 'ukrai': 'kurai', '6qq': 'iq', 'carside': 'carbide', 'saudis': 'saudi', 'sfmoms': 'moms', 'sbs': 'subs', 'northfork': 'southfork', 'bhookh': 'book', 'youjesus': 'ofjesus', 'nawf': 'naff', 'mausam': 'madam', 'happyno': 'happyto', 'john$on': 'johnson', 'safeways': 'sideways', 'glasto': 'last', 'bellini': 'telling', 'shakenup': 'shakeup', 'redi': 'red', 'frends': 'friends', 'eaze': 'ease', 'qkids': 'kids', 'shaler': 'shaker', 'heydan': 'heyday', 'sinc': 'since', 'cv': 'ca', 'ofcs': 'orcs', 'itachi': 'tachi', 'doyoung': 'young', '$gs': 'is', 'md': 'my', 'helenm': 'helen', 'bawbag': 'baba', 'halstead': 'halted', 'coborns': 'colors', '15your': 'your', 'ungloveg': 'unloved', 'vivan': 'vivian', 'snoonu': 'soon', 'teers': 'tears', 'msging': 'making', 'incldg': 'include', 'bdays': 'days', 'chevis': 'checks', 'cri': 'cry', 'sweed': 'sweet', 'milpitas': 'militias', 'eaasy': 'easy', 'themill': "them'll", 'mclean': 'clean', 'khakhras': 'chakras', 'quebecer': 'queerer', 'distractedly': 'distracted', 'blvd': 'bled', 'discounter': 'discounted', 'sataday': 'saturday', 'mashkor': 'masher', 'alevie': 'alive', 'billionares': 'billionaires', 'imagazine': 'magazine', 'waisted': 'waited', '48hours': 'hours', 'evrydy': 'everyday', 'ftf': 'fof', 'level5': 'level', 'vompletely': 'completely', 'm33': 'my', 'rbw': 'row', 'al7': 'all', 'icna': 'ina', 'flexibil': 'flexible', 'cocid': 'couid', 'bagal': 'bagel', 'alessandra': 'alessandro', 'caretackers': 'caretakers', '13th': 'with', '10lbs': 'bulbs', 'hapur': 'happy', 'yababa': 'yabba', 'nvleg': 'leg', 'shoin': 'shown', 'christoph': 'christophe', 'april27': 'april', 'emart': 'smart', 'bigplan': 'biplane', 'floridians': 'floridian', 'fhey': 'they', 'birchwood': 'torchwood', 'clackity': 'clarity', 'delvit': 'devil', 'condit': 'conduit', 'ker': 'her', 'unthawed': 'thawed', 'pamdemic': 'pandemic', 'kimy': 'kim', 'intersting': 'interesting', 'becki': 'beck', 'televists': 'televised', 'instacrat': 'instagram', 'sabd': 'said', 'refusers': 'refuses', 'graciela': 'gracia', 'bb': 'be', 'dcp': 'dip', '12your': 'your', 'kencko': 'gecko', 'ogui': 'oui', 'mccoys': 'mccoy', 'melaka': 'melba', 'slepp': 'sleep', 'toodles': 'noodles', 'cpag': 'crag', 'gulivery': 'delivery', 'worders': 'orders', 'szura': 'sura', 'masketeer': 'musketeer', 'vaca': 'vac', 'semira': 'seminar', 'whynews': 'whines', 'rrt': 'art', 'shakin': 'shaking', 'reli': 'rely', 'baniya': 'banana', 'bizu': 'biz', 'perron': 'person', 'chromatica': 'chromatic', 'bangor': 'banger', 'monys': 'monks', 'deleivery': 'delivery', 'startin': 'starting', 'appreciatively': 'appreciative', 'wral': 'wrap', 'dogood': 'dogwood', 'inmar': 'near', 'nrv': 'nav', 'toronto': 'pronto', 'mtm': 'mom', 'gfx': 'of', 'figma': 'sigma', 'evng': 'eng', 'bluej': 'blue', 'hdb': 'hub', 'quiqup': 'quip', 'tastyfo': 'tasty', 'ck3': 'can', 'vcu': 'vou', 'alexandriava': 'alexandria', 'pickup]': 'pickup', 'regalado': 'regaled', 'pinoy': 'pinky', 'jaen': 'jean', 'son38': 'son', 'hhis': 'this', 'heelies': 'wheelies', 'edt': 'eat', 'bhe': 'the', 'deamat': 'defeat', 'enou': 'nou', 'grindszn': 'grandson', 'tahanan': 'thana', 'odi': 'odd', 'tideland': 'ireland', 'mcg': 'mug', 'iby': 'by', 'foresaken': 'forsaken', 'uplb': 'upl', 'chrisps': 'crisps', '12noon': 'noon', 'ntrepreneur': 'entrepreneur', 'molino': 'moving', 'yout': 'you', 'imfpa': 'ima', '4sept': 'sept', 'porteiro': 'porter', '15th': 'with', 'frigg': 'frig', 'gaudets': 'gardens', 'tulis': 'tulips', 'lesley': 'wesley', 'jadhav': 'radha', 'panchayath': 'panchayat', 'containmnt': 'containment', 'innocenz': 'innocent', 'harriman': 'harridan', 'mcd': 'mad', 'awal': 'away', 'hardwon': 'pardon', 'tikkun': 'tikka', 'wirecard': 'timecard', 'q1': 'qi', 'manority': 'majority', 'thal': 'that', 'merkel': 'market', 'bisnow': 'snow', 'whhhhy': 'whyphy', 'vc': 've', 'sgela': 'sell', 'thous': 'thou', 'aphysical': 'physical', 'jtag': 'tag', 'sahana': 'shana', 'washesha': 'washes', 'purani': 'punani', 'bochum': 'chum', 'bumfuck': 'unfuck', 'hbc': 'hoc', 'worderplay': 'wordplay', 'olo': 'old', 'croton': 'proton', 'yesrs': 'years', 'waivered': 'wavered', 'ounje': 'ounce', 'rockefellas': 'rockefellers', 'anecdotally': 'anecdotal', 'freeproduce': 'reproduce', 'kigu': 'kizu', 'kaufmann': 'kaufmans', 'iles': 'miles', 'cavite': 'cavity', '$whsi': 'whyi', 'sensex': 'sense', 'fitbits': 'titbits', 'misgiving': 'misgivings', 'athome': 'those', 'fundraiaing': 'fundraising', 'avoide': 'avoid', 'gratitudewe': 'gratitude', 'rohlik': 'rollin', 'camello': 'camel', 'idjit': 'idjits', 'channel]': 'channel', 'kneebu': 'knees', 'yus': 'us', 'dettol': 'detail', 'witbank': 'wetback', 'handup': 'hands', 'lyfts': 'lifts', 'freestone': 'freestyle', 'only]': 'only', 'repricing': 'replacing', 'foodprint': 'footprint', 'alp': 'all', 'panrty': 'party', 'mypoints': 'points', 'krisps': 'crisps', 'haat': 'heat', 'fatlil': 'fall', 'chaiko': 'chair', 'nruo': 'no', 'nosophobia': 'homophobia', 'emcq': 'each', 'applevis': 'apples', 'thogo': 'togo', 'understook': 'understood', 'teh': 'the', 'first10': 'first', 'legbourne': 'melbourne', 'day55': 'day', 'gordery': 'order', 'concrn': 'concern', 'bpa': 'bra', '$dis': 'dis', 'mirin': 'marin', 'rixa': 'rita', 'shamers': 'shames', 'fammle': 'family', 'tyoure': 'your', 'probert': 'robert', 'mres': 'tres', 'xpected': 'expected', 'loqsea': 'lose', 'anche': 'ache', 'parkhouse': 'warehouse', 'pandoro': 'pandora', 'rhony': 'phony', 'qotd': 'qot', 'dreadlocs': 'dreadlocks', 'cbcnl': 'cecil', 'meetup': 'meet', 'warranteed': 'warranted', '4everyone': 'everyone', 'craisins': 'raisins', 'fap': 'far', 'citymall': "city'll", 'bizzarro': 'bizarro', 'jenwe': 'sense', 'govenors': 'governors', '1c': 'ac', 'freeand': 'friend', 'hahaaha': 'hahaha', 'kaarthik': 'karthik', '8mos': 'mos', 'yaji': 'yami', 'guaran': 'guard', 'happyfox': 'happyto', 'dronedek': 'droned', 'lettin': 'letting', 'allready': 'already', 'r17': 'run', 'cplife': 'life', 'effy': 'iffy', '7off': 'off', 'budlight': 'sunlight', 'autono': 'auto', 'autonomou': 'autonomous', 'moonprism': 'moonrise', 'comany': 'company', 'industrialize': 'industrialized', 'mol': 'mom', 'ladue': 'laude', 'gotdamn': 'goddamn', 'basha': 'bash', 'jadi': 'jade', 'udderly': 'suddenly', 'sapang': 'saying', 'hofstra': 'nostra', 'nadler': 'nader', 'quetta': 'questa', 'okw': 'ok', 'stohr': 'stor', 'rossuers': 'roosters', 'kimmi': 'kimi', 'tmo': 'to', 'frederico': 'frederick', 'sharship': 'starship', 'mab': 'man', 'hubro': 'hurt', 'yeen': 'been', 'onev1': 'one', 'ucuza': 'scuba', 'cema': 'coma', 'dischem': 'dishes', 'sitaraman': 'starman', 'urben': 'urban', 'pukerua': 'puerta', 'wheather': 'whether', 'wams': 'was', 'flynx': 'flynn', 'dukhan': 'duncan', 'xinan': 'xian', 'sfc': 'sec', 'izishop': 'bishop', 'lesson1': 'lesson', 'sqft': 'soft', 'westmont': 'weston', 'sawda': 'sawa', 'hallelu': 'halle', 'deliverie': 'deliveries', 'smiff': 'stiff', 'ibeica': 'becca', 'sweetdreams': 'sweetbreads', 'aguados': 'guidos', 'fhese': 'these', 'spacs': 'space', '2wipe': 'wipe', 'ghettoo': 'ghetto', 'bangal': 'banal', '$bbby': 'baby', 'oarnges': 'oranges', 'mortsr': 'mortar', 'bharta': 'chart', '82yo': 'yo', 'lorn': 'born', 'hase': 'have', 'ps14': 'psi', 'carillion': 'trillion', 'linn': 'line', 'ekaaro': 'skaar', 'anny': 'any', 'magots': 'maggots', 're84b': 'rehab', 'wintrust': 'entrust', 'camero': 'camera', 'mgnt': 'mint', 'nmu': 'mu', 'finnah': 'finna', 'dvlp': 'help', 'wlwt': 'what', 'skyourocket': 'skyrocket', 'sendy': 'send', 'autopilot1': 'autopilot', 'downp': 'down', 'withno': 'with', '$roo': 'roo', 'aza': 'aka', 'dragoncon': 'dragoon', 'autovan': 'autobahn', 'mejer': 'meter', 'amiright': 'alright', 'mfb': 'mob', 'mushies': 'tushies', 'mspark': 'spark', 'paperbags': 'paperback', 'ashima': 'ashita', '72hours': 'hours', '$sbry': 'sory', 'kaylas': 'kayla', 'vg': 've', 'hotnews': 'hotness', 'lvg': 'leg', 'spf': 'spy', 'lytton': 'button', 'raramo': 'carajo', 'embarassment': 'embarrassment', 'wwd': 'wed', 'talkn': 'talk', 'hsve': 'have', 'clickit': 'clicks', 'idahome': 'idaho', 'hornying': 'horning', 'scotia': 'scotch', 'gojuice': 'juice', 'umlazi': 'umami', 'gyps': 'gypsy', 'ethicist': 'ethics', 'kapoya': 'kappa', 'sjx': 'six', 'treatin': 'treating', 'stthomas': 'thomas', 'borje': 'bore', 'ad87': 'add', 'targetd': 'target', 'love1s': 'loves', '$inc': 'zinc', 'iwc': 'ic', 'mcsltd': 'misled', 'pubg': 'pub', 'baidu': 'said', 'stms': 'sums', 'ekhono': 'kono', 'curside': 'cursive', 'dangit': 'daggit', 'yela': 'yell', 'unita': 'unit', 'leonora': 'leonard', 'pinellas': 'patellas', '7kw': 'how', 'malasa': 'males', 'toole': 'tools', '60k': 'ask', 'eyydu': 'eyed', 'nqr': 'nor', 'lfg': 'leg', 'riov': 'riot', 'letewa': 'leeway', 'gern': 'germ', 'thesims4': 'thesis', 'bartle': 'battle', 'mepolitics': 'politics', '$wfm': 'swim', 'teamlocked': 'deadlocked', 'bball': 'ball', 'bridgeport': 'bridgework', 'sociald': 'social', 'suppl': 'supply', 'numbnut': 'numbnuts', 'mangwao': 'mango', 'shambolic': 'symbolic', 'thouseel': 'housel', 'mlwb': 'club', 'nbc4': 'nice', 'corndogs': 'condoms', '69c': 'doc', 'all4': 'all', 'unsplash': 'splash', 'brielle': 'briefly', 'wuld': 'would', 'palmart': 'palmar', 'lmfoa': 'foa', 'salockdown': 'lockdown', 'vehi': 'veh', 'mkoma': 'lkoma', 'bww': 'bow', 'belil': 'bell', 'puso': 'push', 'pckge': 'page', 'nidhish': 'yiddish', 'forders': 'orders', 'horridly': 'horribly', 'remoter': 'remote', 'nlbs': 'nibs', '7days': 'days', 'phonetap': 'phonetic', 'enfield': 'infield', 'livinn': 'living', 'kanwal': 'manual', 'polmont': 'belmont', 'gambia': 'gambit', 'repeatability': 'respectability', 'memoed': 'memory', 'unneccessary': 'unnecessary', 'flareup': 'flare', 'rahsia': 'russia', 'byee': 'bye', 'soflo': 'solo', 'remus': 'emus', 'fishlife': 'fishwife', 'nochance': 'chance', 'aleve': 'alive', 'cultiva': 'cultivate', 'germino': 'german', 'sellable': 'syllable', 'slpcares': 'scares', 'cheesies': 'cheeses', 'ps31': 'psi', 'youthful': 'mouthful', 'clicky': 'clicks', 'chea': 'cheap', 'guh': 'guy', '2grilled': 'grilled', 'wohoo': 'whoo', 'expections': 'expecting', 'zumzum': 'zuzu', 'isekai': 'sekai', 'science]': 'science', 'fitternity': 'fraternity', '1hour': 'hour', 'watchout': 'without', '19am': 'am', 'myloc': 'melon', 'rwdsu': 'rods', 'q10': 'que', 'eicher': 'either', 'hmb': 'hub', 'pyme': 'pyre', 'racquel': 'raquel', 'magruders': 'marauders', 'airbnb': 'airing', 'ctagegory': 'category', 'sla': 'sea', 'exu': 'ex', 'inh': 'in', 'iladies': 'ladies', 'premarket': 'remarked', 'karwane': 'karate', 'genovia': 'genovian', 'octoberx': 'october', 'memwho': 'memo', '2mph': 'amp', 'nemlig': 'nellie', 'andriod': 'android', 'trynna': 'tryna', 'adel': 'del', 'embersed': 'embedded', 'wpmi': 'mi', 'iebc': 'ice', 'mileend': 'milked', 'vaccianted': 'vaccinated', 'goucher': 'voucher', 'schmip': 'ship', 'whtsapp': 'whatsapp', 'evwn': 'even', 'parcs': 'parts', 'bcorp': 'corp', 'windchills': 'windmills', 'commitee': 'committee', 'shipsx': 'ships', 'dundrum': 'humdrum', 'shitcago': 'chicago', 'chandrika': 'chandra', 'm3': 'my', 'angelenos': 'angeles', 'mbare': 'bare', 'contactus': 'contacts', 'zumbo': 'jumbo', 'insidem': 'inside', 'marawi': 'maria', 'bodeega': 'bodega', 'rasu': 'rash', 'avoca': 'avoid', 'karte': 'karate', 'ayoko': 'yoko', 'adnan': 'annan', 'delft': 'deft', 'democratize': 'democratic', 'getters': 'letters', 'refrigerati': 'refrigerate', '45minutes': 'minutes', 'nmhm': 'mom', 'wegood': 'good', 'wdnes': 'wines', 'logyn': 'logan', 'singal': 'signal', 'foodbiz': 'foodie', 'pensively': 'pensive', 'troung': 'aroung', 'resteraunts': 'restraints', 'mtoag': 'tag', 'thenine': 'thine', 'ehlers': 'cheers', 'luddite': 'laddie', '3p': 'up', 'fiton': 'futon', 'aapni': 'papi', 'fiving': 'living', 'bangui': 'bang', 'audlem': 'audrey', 'seminole': 'seminal', 'oway': 'way', 'milkma': 'milkman', 'benatural': 'natural', 'visine': 'visit', 'seffner': 'suffer', 'scanalyzer': 'analyzer', 'ferenchik': 'frenchie', 'rlpartners': 'partners', 'jagoffs': 'jagoff', 'bekas': 'beaks', 'abadir': 'nadir', 'vinod': 'vino', 'slush15': 'slush', 'hungup': 'hung', 'etf': 'et', 'morpeth': 'morph', 'snackie': 'snake', 'parr': 'part', 'daphnia': 'daphne', 'wfla': 'will', 'lenor': 'tenor', 'aisling': 'ailing', 'telli': 'tell', 'risha': 'trisha', 'bicester': 'leicester', 'bathrm': 'bathroom', 'egrocer': 'grocer', 'zima': 'ima', 'qr': 'or', 'crispies': 'krispies', 'no2sea': 'nose', 'burien': 'buried', 'mandel': 'mantel', 'marya': 'mary', 'miils': 'mills', 'interquest': 'interest', 'grociers': 'grocers', 'bekar': 'bear', 'dayyy': 'day', 'schill20': 'schiller', 'succorng': 'succor', 'woodz': 'woods', 'gameplan': 'gameplay', 'pricele$$': 'priceless', 'ssa61': 'ssa', 'chs': 'cos', 'whinning': 'winning', 'buji': 'buti', 'no10': 'not', 'aerons': 'aprons', 'kuget': 'kugel', 'socialcues': 'socialites', 'airmax': 'airman', 'georderi': 'reorder', 'oregonian': 'gregorian', 'maypopne': 'maypole', 'halka': 'halfa', 'aftr': 'after', 'mktg': 'met', 'elswood': 'elwood', 'paprikash': 'paprika', 'forebode': 'forbade', 'stns': 'sons', 'sanetizer': 'sanitizer', 'lesigh': 'leigh', 'stockup': 'stickup', 'megadeth': 'leadeth', 'bueller': 'mueller', 'lovewfh': 'loveth', 'saffola': 'scaffold', 'vlogmas': 'dogmas', 'magiccon': 'magician', 'cowtown': 'downtown', 'kur': 'our', 'ugvs': 'us', 'jitney': 'kidney', 'propagator': 'propagate', 'betch': 'bitch', 'swaad': 'said', '20packs': 'packs', 'dejah': 'deja', 'cld': 'old', '90k': 'ask', 'coeliacs': 'celiac', 'howuch': 'howmuch', 'swfs': 'was', 'addl': 'add', 'canberra': 'cranberry', '10off': 'off', '13m': 'him', 'nucor': 'nor', 'mtn': 'man', 'seekin': 'seeking', 'peticao': 'pelican', 'lvngbook': 'logbook', 'ooawwe': 'howwe', '32am': 'am', 'szopi': 'stop', 'box]': 'box', 'nylander': 'slander', 'bodda': 'badda', 'yonk': 'monk', 'idtheft': 'theft', 'mostl': 'most', 'xtremes': 'extremes', 'sohail': 'shall', 'halelujiah': 'hallelujah', 'sketty': 'pretty', 'specifie': 'specific', 'muthafxckas': 'muthafuckas', 'batjoke': 'ajoke', 'exeption': 'exception', 'webby': 'webb', 'fintree': 'entree', 'gdoc': 'doc', '$burg': 'burg', 'isro': 'iso', 'manav': 'mana', 'abbi': 'rabbi', 'dropof': 'drop', 'copps': 'cops', 'contries': 'countries', 'gahmen': 'game', '29c': 'doc', 'casi': 'case', 'logiq': 'logic', 'verzuz': 'versus', '75p': 'up', 'reprehensibly': 'reprehensible', 'mabond': 'bond', '39weeks': 'weeks', 'techtips': 'techies', 'westborn': 'western', 'sweatsuit': 'sweatshirt', 'engano': 'engine', 'monon': 'moon', 'lilne': 'line', 'wmbg': 'wing', 'grapely': 'gravely', 'greycard': 'keycard', 'hotpod': 'hotpot', 'wapda': 'wanda', 'hallowe': 'hallowed', 'europe1': 'europe', 'ising': 'using', 'ph17': 'phil', 'se1': 'see', 'thatike': 'thati', 'sh': 'so', 'realmvp': 'realm', 'pennines': 'pennies', '$demae': 'edema', 'yara': 'yard', 'bpusa': 'busy', 'januaryff': 'january', 'ccp': 'cop', 'minimun': 'minimum', 'mesilla': 'medulla', 'deliviees': 'delivers', 'fazio': 'fabio', 'iolated': 'isolated', 'soraya': 'sorry', 'june1st': "june's", 'watercooler': 'watercolor', 'sside': 'side', 'nerufa': 'nebula', 'wholely': 'wholly', 'restaura': 'restaurant', 'babaji': 'baba', 'redund': 'refund', 'meyerland': 'neverland', 'bigtech': 'biotech', 'stw': 'saw', 'momy': 'mom', 'phonics': 'phonies', 'ndemic': 'endemic', 'foxnews': 'foxes', 'ngicela': 'nicely', 'wildalis': 'willis', 'desthtrap': 'deathtrap', 'babby': 'baby', 'flav': 'flat', 'onlined': 'online', 'tiggy': 'piggy', 'cazorla': 'carla', 'iste19': 'isten', 'seko': 'seo', 'quaritine': 'quarantine', 'iplaw': 'play', 'alayna': 'alana', 'andro': 'andre', 'magat': 'maga', 'carkeys': 'cares', 'rajeev': 'raees', 'technolodge': 'technology', 'acr': 'car', 'w1': 'we', 'moneyi': 'money', 'atton': 'anton', 'gard': 'hard', 'stooned': 'stoned', 'ibs': 'is', 'wegetit': 'getit', 'bitcoi': 'bitcoin', 'tosave': 'save', 'riverhead': 'overhead', 'aditional': 'additional', 'p': 'i', 'recycler': 'recycled', 'bhut': 'but', 'myblue': 'blue', 'sephora': 'senora', 'pickuo': 'pickup', 'payoh': 'pay', 'courtnie': 'courting', 'aaa': 'aka', 'bentwater': 'meltwater', 'caramelise': 'caramelised', 'croptop': 'crypto', '$dvax': 'dax', 'sng': 'sing', 'krumbah': 'rumba', '2y': 'my', 'nabo': 'nab', 'winit': 'want', 'instag': 'instar', 'showery': 'shower', 'pinner': 'dinner', 'etrade': 'trade', 'refief': 'relief', 'appde': 'apple', 'foodhall': 'football', 'undoorpants': 'underpants', 'conrade': 'comrade', 'boricua': 'boric', 'lipread': 'spread', 'emrs': 'ears', 'adnoc': 'doc', 'shiot': 'shit', 'fruitveg': 'fruiting', 'coronas': 'coronal', 'pu$$ies': 'puppies', 'sunak': 'sunk', 'smokin': 'smoking', 'striberg': 'strikers', 'inequ': 'inept', 'scentsy': 'scents', 'sfrs': 'sirs', 'tavishi': 'lavish', 'redseer': 'redeem', 'hr': 'he', 'appalachians': 'appalachian', 'darrel': 'barrel', 'certajnly': 'certainly', 'bauers': 'bakers', 'candi': 'candy', 'pibk': 'pick', 'sector67': 'sector', 'smoothen': 'smoother', 'isd': 'is', 'bitcher': 'bitches', 'detwiler': 'detailed', 'tfit': 'fit', 'ct13': 'ctu', 'schmaling': 'schmeling', 'tiktok': 'tiptoe', 'wtb': 'web', 'magno': 'mango', 'virga': 'virgo', 'indepedent': 'independent', 'meaux': 'beaux', 'livs': 'live', 'cbx': 'cox', 'eris': 'eric', 'tegeka': 'teresa', 'utd': 'und', 'urgh': 'urge', 'ourchases': 'purchases', 'dobbies': 'hobbies', 'migos': 'amigos', 'goodlife': 'godlike', 'mochela': 'mocha', '50lb': 'bulb', 'khari': 'khaki', 'emmaus': "emma's", 'uhmm': 'umm', 'rhoda': 'rhode', 'chemex': 'cheer', 'milkshaked': 'milkshake', 'fishkin': 'fishing', 'pitman': 'hitman', 'agoddess': 'goddess', 'instnt': 'instant', 'housto': 'houston', 'przez': 'prez', 'fubaring': 'fearing', 'hfx': 'hex', 'eastpark': 'eastward', 'medicalim': 'medical', 'physiically': 'physically', 'paylook1': 'palooka', 'yosender': 'yonder', 'otiende': 'tiene', 'plann': 'plan', 'wenk': 'went', 'pcevolution': 'revolution', 'svf': 'saf', 'vacillate': 'oscillate', 'pona': 'pony', 'comparably': 'comparable', 'khair': 'hair', 'kke': 'ke', 'grassfed': 'grassed', 'rolos': 'rolls', 'kfla': 'kola', 'stymy': 'stay', 'junco': 'bunco', 'nimb': 'limb', 'exci': 'each', 'fuggin': 'bugging', 'driscolls': 'driscoll', 'mawmaw': 'mama', 'caddo': 'caddy', 'pilsen': 'pissed', 'resuse': 'refuse', 'leandro': 'leaner', 'verryyy': 'verry', 'wheee': 'where', 'faller': 'fallen', 'phoen': 'phone', 'refurnish': 'refurbish', 'holebas': 'holes', 'tsu': 'tu', 'tattie': 'tattle', 'meatsack': 'seatback', 'afterwork': 'masterwork', 'viktoria': 'victoria', 'wishes2u': 'wishes', 'plier': 'pier', 'listerr': 'lister', 'colloquy': 'soliloquy', 'ww2': 'wwe', 'shem': 'she', 'groupsis': 'groups', 'yumyun': 'ramyun', 'aaii': 'aii', 'journal]': 'journal', '19rs': 'cars', 'torta': 'sorta', 'skyting': 'skating', 'weevolve': 'evolve', 'reseller': 'seller', 'transillion': 'transition', 'centro': 'centre', 'schiff': 'chief', 'uli': 'li', 'snext': 'next', 'mnthy': 'months', 'cyclinder': 'cylinder', '$innd': 'find', 'foodbrain': 'forebrain', 'wkend': 'wend', 'pernah': 'perish', '9fm': 'of', 'arup': 'aru', 'haney': 'honey', 'conamor': 'connor', 'otsaw': 'saw', 'frm': 'from', 'palatka': 'palate', 'seiyu': 'shiyu', 'nnamdi': 'nami', 'plair': 'pair', 'unconditions': 'conditions', 'minetil': 'mineral', 'alameda': 'blamed', 'excepti': 'except', 'lonliness': 'loneliness', 'teleconf': 'telecom', 'thankf': 'thank', 'thermastat': 'thermostat', 'labou': 'labor', 'trush': 'trust', 'composta': 'compost', 'power2do': 'powered', 'bubly': 'bully', 'largets': 'largest', 'tech24': 'tech', 'coronalk': 'coronal', 'donettes': 'donates', 'stef': 'step', 'coronamb': 'coronary', 'cov19': 'cover', 'meegs': 'meets', '84yo': 'yo', 'bbstar': 'star', '$mgyour': 'meyour', 'bulwick': 'bullock', 'cucci': 'gucci', 'tasia': 'asia', 'raipur': 'rapper', '60yo': 'yo', 'cna': 'can', 'dst': 'dust', 'bfly': 'fly', '5your': 'your', 'culligan': 'mulligan', 'wf': 'of', 'tiwa': 'tina', '26c': 'doc', 'asylu': 'asylum', 'excutives': 'executives', 'diann': 'diane', 'amal': 'anal', 'bagay': 'baggy', 'zomick': 'comic', '2pm': 'up', 'vax': 'van', 'payup': 'layup', 'kerbstone': 'keystone', 'qol': 'ol', 'soriano': 'syrian', 'disobediance': 'disobedience', 'saken': 'taken', 'hanta': 'santa', 'shatto': 'thatto', 'philco': 'phil', 'trsns': 'trans', 'beshear': 'behead', 'sasol': 'season', 'linley': 'finley', 'bcif': 'scif', 'lgh': 'ugh', 'ajusshi': 'ajussi', 'dctech': 'tech', 'anit': 'unit', 'irerefer': 'reefer', 'outlander': 'outlanders', 'aowa': 'awa', 'pomo': 'poo', 'allin': 'allen', 'denali': 'deal', 'blackhorse': 'blockhouse', 'contee': 'congee', '3heavy': 'heavy', 'mfpa': 'map', 'wrongun': 'wrong', 'creamees': 'creates', 'mckee': 'mcgee', 'freshcar': 'freshman', 'ismael': 'israel', 'chaq': 'chat', 'frio': 'trio', 'technologie': 'technologies', 'here4u': 'here', 'timbeta': 'timber', 'cde': 'de', 'pilotin': 'piloting', '$dcix': 'dci', 'ise16': 'ise', '85th': 'with', 'esports': 'sports', 'fortnightly': 'fortnight', 'wssh': 'wish', 'backwardness': 'awkwardness', 'itreat': 'treat', 'tnam': 'team', 'kato': 'kate', 'vaporub': 'vapor', 'fortnite': 'fortunate', 'paega': 'page', 'dencity': 'density', 'uthiru': 'third', 'ferrer': 'ferret', 'brumit': 'bruit', 'ashould': 'should', 'cofirmed': 'confirmed', 'yellen': 'yelled', '44st': 'just', 'salestax': 'salesman', 'employes': 'employee', 'enviornment': 'environment', 'nadu': 'nada', 'examin': 'examine', 'tooo': 'too', 'rajat': 'raja', 'litetally': 'literally', 'cusotmers': 'customers', 'zone4': 'zone', 'us50': 'us', 'sbutd': 'but', 'grieveances': 'grievances', 'bogan': 'began', 'bedecent': 'decent', 'anitviral': 'antiviral', 'venha': 'vena', 'koivisto': 'koibito', 'mtp': 'map', 'cct': 'cut', 'saq': 'say', 'zymgs': 'hymns', 'happyrt': 'happy', 'abad': 'bad', '$bce': 'be', 'nrdc': 'nude', 'monhu': 'money', '40days': 'days', 'flaskes': 'flashes', 'muncie': 'uncle', 'croesus': 'cross', 'ingorg': 'ignore', 'xp': 'up', 'everyon': 'everyone', '4qn': 'in', 'd1e': 'die', 'thha': 'tha', 'recordering': 'reordering', 'ugma': 'uma', 'salm': 'calm', 'qanon': 'canon', 'us6': 'us', 'creackers': 'crackers', 'viennetta': 'vendetta', 'polyam': 'polar', 'funfetti': 'confetti', 'approvaluse': 'approvals', 'lafy': 'lady', 'couriering': 'cornering', '17days': 'days', 'trang': 'rang', 'toyin': 'toying', 'wht': 'what', 'lajobs': 'jobs', 'vietnames': 'vietnamese', 'technoolgy': 'technology', 'cfl': 'ifl', 'haig': 'hang', 'pattys': 'patty', 'ctrl': 'curl', 'downtownla': 'downtown', 'amc': 'am', 'tyt': 'tit', 'keppa': 'kappa', 'hawley': 'haley', 'amaveni': 'maven', 'turnitup': 'turnip', 'whingers': 'whiners', 'eagame': 'game', 'bk5': 'be', 'spamming': 'slamming', 'cnns': 'cans', 'requi': 'require', 'azarcon': 'axarion', 'openlab': 'openly', '$codx': 'code', 'tecnology': 'technology', 'maz': 'man', 'naudain': 'maudlin', 'rawat': 'away', 'gwest': 'guest', 'samed': 'same', 'neff': 'jeff', 'nonkyc': 'honky', 'innovatio': 'innovation', 'duanes': 'dunes', 'intersperse': 'interspersed', 'mgrocery': 'grocery', 'ozick': 'pick', 'trekker': 'trekked', 'phanatic': 'fanatic', 'dailey': 'daily', 'onlline': 'online', 'poing': 'going', 'corvid': 'forbid', 'solimo': 'solid', 'orive': 'drive', 'sanfey': 'safe', 'cuomo': 'como', 'retailng': 'retailing', 'tremdously': 'tremendously', 'werklu': 'weekly', 'frustration1': 'frustration', 'koehler': 'kosher', '$ddl': 'add', 'mboga': 'yoga', 'refrigerato': 'refrigerator', 'dishi': 'dish', 'coffeeup': 'coffee', 'weeze': 'wheeze', 'sourdo': 'sounds', 'optim': 'optic', 'mhh': 'mah', 'skal': 'seal', 'eston': 'estoy', 'nena': 'xena', 'ect': 'eat', 'ninyile': 'nitrile', 'emow': 'mow', 'caz': 'can', 'xfinity': 'affinity', 'omuka': 'omega', 'clackin': 'clacking', 'ps25': 'psi', 'nitesh': 'notes', 'westdene': 'westen', 'yongest': 'youngest', 'abott': 'about', 'realy': 'really', 'gola': 'gold', 'detractor': 'detractors', 'thato': 'that', 'danja': 'dana', 'katingle': 'tingle', 'intvu': 'into', 'pyaza': 'plaza', '4pt5': 'apt', 'lfp': 'lap', 'inquirer': 'inquire', 'bishnu': 'vishnu', 'sealtest': 'smartest', 'delrey': 'degree', 'heriot': 'hero', 'curbsid': 'curbside', 'qatar': 'water', 'abuelito': 'abuelita', 'evts': 'eats', 'skcc': 'such', 'ztrip': 'trip', 'madurai': 'samurai', 'cng': 'ing', 'palcus': 'places', 'scanta': 'santa', 'ipsum': 'issue', 'whyyy': 'why', 'commoditized': 'commodities', 'shela': 'shell', 'sethu': 'seth', 'ellio': 'elliot', 'kenn': 'keen', 'bryson': 'prison', 'tiga': 'toga', 'wareroom': 'wardroom', 'kier': 'pier', 'greeno': 'green', 'wingfield': 'minefield', 'winter20': 'winter', 'pts': 'its', 'exo': 'ex', 'witholding': 'withholding', 'blackpilled': 'blackballed', '22your': 'your', 'emirate': 'emigrate', 'po1': 'pop', 'taite': 'taste', 'marchmont': 'parchment', 'sometyms': 'sometimes', 'warung': 'wrung', 'rpd': 'red', 'pll': 'all', 'addy': 'add', 'ket': 'get', '70something': 'something', 'penta': 'pent', 'pozole': 'poole', '90mins': 'mins', 'yo3': 'you', 'dahra': 'dara', 'exercice': 'exercise', 'mandemic': 'pandemic', 'redhook': 'redwood', 'alomar': 'altar', '1b': 'ab', 'iowan': 'rowan', 'riyal': 'royal', 'justme': 'juste', 'mistakea': 'mistake', 'schwans': 'scans', 'etn': 'ten', 'pushkaro': 'pushcart', 'ewic': 'epic', 'logis': 'logic', 'palmbay': 'palmar', 'campgrou': 'campground', 'alivia': 'olivia', 'chup': 'cup', 'leakin': 'leaking', 'frigo': 'frig', 'tl': 'to', 'ivrs': 'ives', 'mallorca': 'mallory', 'zulzi': 'multi', 'baybe': 'maybe', 'rainx': 'rain', 'ab5': 'abs', 'wcsh6': 'wish', 'ziglar': 'cigar', 'curiousai': 'curious', 'sikhism': 'schism', 'klown': 'known', 'bedraggle': 'bedraggled', 'creche': 'create', 'yummlys': 'yummy', 'leed': 'need', 'ojinna': 'jenna', 'ecentric': 'eccentric', 'yey': 'yet', 'hmg': 'hug', 'wapow': 'weapon', 'kuuza': 'ruuzu', 'cortishae': 'cortisone', 'splosion': 'explosion', '1f': 'of', 'sworderfish': 'swordfish', 'huffington': 'huntington', 'kagin': 'karin', 'umb': 'dumb', 'horderer': 'ordered', 'ps10m': 'psalm', 'ipt': 'it', 'jklive': 'live', 'tinnnie': 'winnie', 'msmes': 'mimes', '4mths': 'paths', 'oklahoman': 'oklahoma', 'ians': 'fans', 'quenchit': 'quench', 'rcn': 'run', 'studen': 'student', 'cop26': 'cops', '17a': 'a', 'sanchar': 'anchor', '04th': 'with', 'janata': 'anata', 'flexe': 'flee', 'umd': 'mud', 'themselve': 'themselves', 'renz': 'rent', 'atozgo': 'togo', 'r65': 'run', 'youro': 'your', 'justno': 'just', 'bearfaced': 'barefaced', '20l': 'all', 'arcadey': 'arcade', 'unprescribed': 'prescribed', 'ranma': 'rama', 'adsnity': 'sanity', 'nabbit': 'rabbit', 'tera': 'tea', 'dh20': 'dhe', 'degil': 'devil', 'levien': 'levied', 'dasa': 'data', 'intelcom': 'intercom', 'bebopping': 'bopping', 'mejia': 'media', 'letswork': 'network', 'clippinger': 'clipping', 'suppermarket': 'supermarket', 'groen': 'green', 'pulle': 'pull', 'aaj': 'raj', 'pdq': 'pda', 'theboy': 'thejoy', 'bidar': 'bear', 'supermrkts': 'supermarkets', '83your': 'your', 'twitt': 'twist', 'foia': 'foil', 'outsources': 'outsource', 'alibaba': 'alabama', 'speth': 'seth', 'greystone': 'keystone', 'lanka': 'lana', 'afforderable': 'affordable', 'kuenda': 'kinda', 'tegan': 'began', 'pdt': 'put', 'usd': 'us', 'cz': 'ca', 'nationalday': 'nationality', 'hanbury': 'unbury', 'mingy': 'ming', 'd1': 'do', 'ezadar': 'radar', 'opportuntity': 'opportunity', 'kh': 'oh', 'kbit': 'bit', 'vs': 'is', '$abs': 'tabs', 'rigby': 'rugby', 'rdchat': 'chat', 'counth': 'count', 'betocrat': 'democrat', '1yes': 'eyes', 'folate': 'late', 'effor': 'effort', 'sadaf': 'sad', 'cobham': 'cobra', 'chitki': 'chick', 'dvp': 'dip', 'definatly': 'defiantly', 'giullian': 'gillian', 'w3': 'we', 'parisi': 'parish', 'sdv': 'suv', 'mintues': 'minutes', 'barse': 'base', '41minutes': 'minutes', 'reciept': 'receipt', 'radious': 'radius', 'jokin': 'join', 'tamzin': 'tampon', 'hopfield': 'hayfield', 'mla': 'la', 'reuseable': 'reusable', 'hlala': 'lala', 'premade': 'remade', 'stokvels': 'shovels', 'restraunt': 'restraint', 'chngd': 'change', 'gaja': 'gala', 'gerr': 'gear', 'owino': 'owing', 'arguement': 'argument', 'untai': 'unti', 'deliverii': 'deliver', 'sinema': 'cinema', 'rusk': 'risk', 'nejire': 'entire', '4pk': 'up', 'wooaw': 'woman', 'upsell': 'unsell', 'cathay': 'cathy', 'anydate': 'mandate', 'bblf': 'able', 'cantabrian': 'candarian', 'bromley': 'trolley', 'tacon': 'bacon', 'rocey': 'rocky', 'prepper': 'pepper', 'diehl': 'die', 'bakin': 'baking', 'woolie': 'coolie', 'upto': 'unto', 'vogmask': 'dogmas', 'bako': 'bake', 'benhur': 'bender', 'syska': 'sasha', 'yuyu': 'yuku', '10minut': 'minut', 'combini': 'combine', 'idfk': 'if', 'neolix': 'felix', 'vvv': 'vve', 'yegdt': 'yet', 'coloredge': 'colored', 'skoop': 'scoop', 'oyeti': 'yeti', 'paramore': 'paramour', 'nyana': 'nana', 'inclded': 'included', 'becsuse': 'because', 'storytime': 'storyline', 'recieved': 'received', 'frontend': 'fronted', 'ohk': 'oh', 'adoni': 'adonai', '5mi': 'mi', 'kayden': 'hayden', 'savoz': 'savor', 'bie': 'be', 'lingard': 'lizard', 'workaholism': 'workaholic', 'lwapeng': 'leaping', 'notley': 'motley', 'kabhi': 'rabbi', 'waterrights': 'watertight', 'lahaina': 'lasagna', 'zariye': 'marine', 'malaki': 'malaria', 'foredt': 'forest', '3x': 'ex', 'crut': 'cut', 'politcs': 'politics', 'checkmark': 'checkmate', 'ddriver': 'driver', 'ssakia': 'saki', 'lifee': 'life', '85b': 'job', 'intstant': 'instant', 'pradhan': 'radha', 'chk': 'chi', 'laca': 'lack', 'kpop': 'pop', 'bighearts': 'bighearted', '5yours': 'yours', 'mukul': 'mutual', 'soulbite': 'soulmate', 'flore': 'flare', 'healthsd': 'health', 'inverness': 'governess', 'tailwin': 'tailwind', 'manggis': 'manages', 'oldthink': 'outthink', 'kuii': 'kuni', 'kahsmir': 'kashmir', 'dolo': 'doll', 'townhomes': 'townhouse', 'umang': 'mang', 'shipra': 'ship', 'itworks': 'works', 'broadwest': 'broadest', 'becauae': 'because', 'thre': 'the', 'tm': 'to', 'walj': 'walk', 'lankan': 'lawman', '25mins': 'mins', 'marti': 'martin', 'wellocks': 'bollocks', 'fuggem': 'bugger', 'adcart': 'apart', 'nmdoh': 'doh', 'agrr': 'agro', 'getbring': 'getting', 'lln': 'lan', 'vegis': 'vegas', 'swdc': 'side', 'distributio': 'distribution', 'exerciser': 'exercise', 'mastercl': 'master', 'useing': 'using', 'withtop': 'withthe', 'tsaka': 'taka', 'dollarso': 'dollars', 'tomm': 'tomb', 'avilez': 'vile', 'mongaloid': 'mongoloid', 'niffler': 'differ', 'sypolt': 'spot', 'spoonie': 'spoon', 'addrsss': 'address', 'xdak': 'dak', 'plis': 'plus', 'resturant': 'restaurant', 'codem': 'code', 'tikhat': 'that', 'nosense': 'nonsense', 'jonezin': 'jonesing', 'binch': 'bunch', 'skare': 'share', '$amc': 'am', 'tmc': 'tac', 'ndly': 'idly', 'chahe': 'chase', 'marchon': 'march', 'robison': 'robinson', 'stansfield': 'stanfield', 'fidji': 'fide', 'capitalismed': 'capitalism', 'simoens': 'simons', '2were': 'were', 'totake': 'take', 'madd': 'made', 'nomby': 'nobby', 'sidders': 'bidders', 'hmstore': 'history', 'calicut': 'calcium', 'thk': 'the', 'mathing': 'matching', 'kjp': 'kip', 'earlham': 'arkham', 'n3': 'no', 'zzst': 'zest', 'yknow': 'know', 'chocolateday': 'chocolatey', '$etek': 'week', 'rberg': 'berg', 'lyt': 'let', 'mohalle': 'morally', 'nickled': 'nicked', 'dd2': 'do', 'f00l': 'feel', 'lyanne': 'leanne', 'condescnding': 'condescending', '1o': 'to', 'lfo': 'lo', 'yeasted': 'feasted', 'preggos': 'preggers', 'slad': 'glad', 'williamson': 'williams', 'gurgaon': 'surgeon', 'breaux': 'beaux', 'messi': 'mess', 'throigh': 'through', 'beheamoth': 'behemoth', 'avila': 'evil', 'wacha': 'dacha', 'strategized': 'strategize', 'wilocal': 'local', 'freegle': 'freeze', 'dfnews': 'news', 'phev': 'thev', 'valente': 'valence', 'parkrun': 'parkour', 'to6': 'to', 'eisai': 'essay', 'projectbtg': 'projecting', '10sseries': 'tusseries', 'sowwy': 'sorry', 'graphes': 'grapes', 'menominee': 'nominee', 'chiptole': 'chipotle', 'opines': 'pines', 'marketplac': 'marketplace', 'distance]': 'distance', 'epidsode': 'episode', 'durrr': 'hurry', 'juda': 'judo', 'jogads': 'roads', 'expl': 'expel', 'enuff': 'snuff', 'kingged': 'rigged', 'permisn': 'permian', 'cjlist': 'list', 'abn': 'an', 'aimlow': 'allow', 'dieu': 'die', 'mandalay': 'mandala', 'tomsa': 'tomas', 'fliplart': 'flippant', 'bix': 'big', 'tryyto': 'tryto', 'portola': 'portal', 'picup': 'pickup', 'krogering': 'rogering', 'hootsuite': 'holosuite', 'kiplinger': 'kipling', 'planne': 'plane', '9april': 'april', 'sefton': 'sexton', 'govans': 'groans', 'peabody': 'parody', 'tweezer': 'tweezers', 'reactjs': 'reacts', 'tiddy': 'tidy', '20day': 'day', 'makenzie': 'mackenzie', 'workwise': 'workwith', 'minimu': 'minimum', 'ldnont': 'dont', 'fuckn': 'fuck', '70ish': 'wish', 'lugansk': 'luxans', 'mcewan': 'mean', 'niy': 'ni', 'bcstorm': 'storm', 'lordert': 'order', 'alon': 'along', 'chainging': 'changing', 'cheezels': 'cheeses', 'aem': 'am', 'attie': 'attic', 'technos': 'techno', 'breck': 'break', 'mudi': 'mud', 'tayler': 'taller', 'beverlyma': 'beverly', 'domipart': 'dominant', 'imax': 'max', 'bju': 'bu', 'informatics': 'information', 'methot': 'method', 'stimulized': 'stimulated', 'minutes1': 'minutes', 'packy': 'pack', 'jeebus': 'jesus', 'shakey': 'shake', 'messily': 'easily', 'elfil': 'exfil', 'bcflood': 'blood', 'mochae': 'mocha', 'afforders': 'afforded', 'vehcle': 'vehicle', 'stratus': 'status', 'omanga': 'manga', 'goodz': 'good', 'laggard': 'haggard', 'westmount': 'westbound', 'populati': 'populate', '$chif': 'chief', 'thgt': 'that', 'defe': 'defy', 'rotabi': 'rotate', '20something': 'something', '5hours': 'hours', 'lovie': 'love', 'pressies': 'presses', 'gtg': 'gig', 'orderen': 'ordered', '4m': 'am', 'weh': 'we', 'freemakan': 'freeman', 'neccesary': 'necessary', 'bham': 'beam', 'bpd': 'bad', 'bariga': 'barge', '12hot': 'shot', 'sun7': 'sun', 'collmbia': 'columbia', 'sik': 'sit', 'kasia': 'asia', 'logotype': 'logotypes', 'nassauer': 'massager', 'hebel': 'rebel', 'bukoto': 'koto', 'oomyfriend': 'boyfriend', 'linked2': 'linked', 'ksb': 'usb', 'conservas': 'conserves', 'wadsworth': 'wordsworth', 'samakal': 'samara', 'byebish': 'blemish', 'sumer': 'summer', 'ha9': 'had', 'cargills': 'carmilla', 'steves': 'steve', 'aleen': 'alien', 'caprabo': 'carajo', '$nh': 'unh', 'sros': 'pros', 'makati': 'maketh', 'lamour': 'labour', '$clis': 'clip', 'hyman': 'human', 'yolo': 'solo', 'fox44': 'fox', 'grantor': 'grant', 'talkd': 'talk', 'halfoff': 'handoff', '2ah': 'ah', 'leshy': 'fleshy', 'oaka': 'oak', 'nofood': 'food', 'swit': 'sit', 'partnershi': 'partnership', 'hyder': 'hyper', 'outsoon': 'outdoor', 'uhg': 'uh', 'hb': 'he', 'weow': 'wow', 'medellin': 'medallion', 'cdnpoli': 'cannoli', 'ajain4': 'again', 'berea': 'beret', 'wcnc': 'want', 'newstage': 'restage', 'xwo': 'two', 'yakoyo': 'yahoo', 'neighourhood': 'neighbourhood', 'bressi': 'press', 'daybreak8': 'daybreak', 'offy': 'off', 'patreon': 'patron', 'mcintosh': 'macintosh', 'ultas': 'altar', 'becime': 'become', 'zhen': 'when', 'farq': 'far', 'sogood': 'osgood', 'iamgdc': 'iambic', 'p8': 'pa', 'zapp': 'app', 'gatik': 'gate', 'dior': 'door', 'noproblems': 'problems', 'dik': 'did', 'jandy': 'candy', 'multai': 'multi', 'esential': 'essential', 'parkhead': 'parked', 'downlo': 'downl', 'ttu': 'tu', 'ribeye': 'riley', 'wtoc': 'toc', 'swith': 'with', 'traves': 'travel', 'plsrt': 'part', 'nspoli': 'spoil', 'minimom': 'minimum', 'uragan': 'dragon', 'townhall': 'downhill', 'maintainance': 'maintenance', '2cents': 'cents', 'majalah': 'masala', 'blankies': 'blankie', 'cnsidred': 'considered', 'cosplayer': 'cosplay', 'besos': 'pesos', 'viabox': 'viable', 'ubereat': 'bereft', 'cpi': 'chi', 'britbox': 'shitbox', 'aeresol': 'aerosol', 'gulfgate': 'sulfate', 'lemonaid': 'lemonade', 'reopenings': 'reopening', 'tallaro': 'taller', 'amesbury': 'asbury', 'ordererly': 'orderly', 'degular': 'regular', '4dec': 'dec', 'accomodating': 'accommodating', 'issa': 'isa', 'cento': 'cents', 'mhm': 'mom', 'supperr': 'supper', 'waverley': 'waverly', '$tgt': 'tit', 'wella': 'well', 'ero': 'hero', 'poat': 'boat', 'gandparents': 'grandparents', 'giuliani': 'gillian', 'us25': 'us', 'flatrate': 'flatmate', 'flavio': 'flavor', 'asma': 'alma', 'lvrj': 'lord', 'scrapple': 'scrape', 'maff': 'muff', 'azn': 'an', 'dns': 'dna', 'iframes': 'frames', 'comey': 'come', 'springday': 'springy', 'chking': 'choking', 'generalise': 'generalize', 'ruiru': 'ruuru', '$icart': 'cart', 'bcfood': 'blood', 'lovebut': 'lovest', 'houseworker': 'housework', 'barvecue': 'barbecue', 'cranwell': 'cromwell', 'seabit': 'seat', 'citi': 'city', 'afib': 'fib', 'multco': 'molto', 'beyween': 'between', 'cityish': "city's", 'foodbev': 'foodie', '20lb': 'bulb', 'xcited': 'excited', 'yourkers': 'yorkers', 'socity': 'society', 'intracity': 'intercity', 'tantalus': 'tantalum', 'cpn': 'can', 'evitos': 'editor', 'zeba': 'zebra', 'najja': 'ninja', 'loutre': 'louvre', 'cheftina': 'cheating', 'mlo': 'lo', 'mc': 'my', '6]': 'i', 'eudora': 'aurora', 'khou': 'thou', 'lovedm': 'loved', 'nthome': 'those', 'sov': 'so', '19yours': 'yours', 'redirection': 'direction', 'yandex': 'landed', 'drivey': 'drive', 'thx': 'the', 'nesil': 'neil', 'fornight': 'fortnight', 'sampai': 'sampan', 'rnz': 'rne', 'lefoods': 'foods', 'wepartner': 'partner', 'naki': 'nazi', 'some1': 'some', 'hona': 'bona', 'applaudask': 'applauds', 'ugane': 'gave', 'unshopped': 'shopped', 't]': 'to', 'vprotect': 'protect', 'kvj': 'kid', 'everythng': 'everything', 'thehague': 'teague', 'capit': 'capita', 'ignoramosus': 'ignoramus', 'ivm': "i'm", 'ss': 'is', '30sec': 'sec', 'adobo': 'adobe', 'walknow': 'wallow', 'prioritises': 'priorities', 'individu': 'individual', 'yo87': 'you', 'incent': 'invent', 'eprnews': 'renews', 'sanghi': 'sang', '9c': 'ac', 'franchisee': 'franchise', 'packetsup': 'packets', 'webdevelop': 'redevelop', 'gwt': 'get', 'harrassment': 'harassment', 'paypal': 'papal', 'satur': 'satyr', 'motherfcker': 'motherfucker', 'deloveering': 'delivering', 'noplastic': 'plastic', 'beli': 'bell', 'vup': 'up', 'bassaam': 'balsam', 'azzes': 'asses', 'maggi': 'maggie', 'taskrabbit': 'jackrabbit', 'becouse': 'because', 'pakki': 'paki', 'unatag': 'anata', 'urfa': 'urea', 'auam': 'adam', 'roids': 'roads', 'kipnis': 'kinds', 'enam': 'exam', 'gunging': 'gunning', 'iim': 'him', 'bss': 'ass', 'abilene': 'abalone', '$shyf': 'shy', 'dvva': 'diva', 'vasool': 'vassal', 'shopwings': 'showings', 'wowwww': 'powwow', 'cabify': 'cabin', 'qrting': 'trying', 'heffner': 'heifer', 'petronize': 'patronize', 'lawley': 'lawyer', '7pm': 'up', 'smooze': 'snooze', 'majora': 'major', 'weather]': 'weather', 'owyn': 'own', 'noosa': 'noose', 'femail': 'email', 'tuyo': 'tuo', 'entilted': 'entitled', 'ryb': 'rob', 'tbc': 'tac', 'destuction': 'destruction', 'iqan': 'ian', 'pavlo': 'pablo', 'eaton': 'eaten', 'seasi': 'seas', 'grumpily': 'grumpy', 'ecdu': 'end', 'domot': 'doot', 'flixton': 'clinton', 'gccares': 'cares', 'amr': 'am', 'accomodate': 'accommodate', 'vba': 'via', 'uscg': 'usc', 'gloo': 'glow', 'uncongested': 'uncontested', 'kmn': 'kin', 'tbr': 'tar', 'jmeter': 'meter', 'nese': 'nose', 'bykins': 'brains', 'roaman': 'roman', 'gethired': 'gathered', 'larue': 'large', 'inlove': 'love', 'uncluttered': 'cluttered', 'mutuals': 'mutual', 'dictafone': 'dictaphone', 'channelise': 'channels', 'kdo': 'do', 'nno': 'no', 'moneyuk': 'money', 'whshere': 'where', 'conciliator': 'conciliatory', 'tka': 'tea', 'kaufland': 'kaufmans', 'bringmea': 'bringer', 'gobert': 'robert', 'tuks': 'turks', 'peap': 'leap', 'hoomin': 'hooking', 'nothingt': 'nothing', 'hailin': 'hailing', 'shiddd': 'shield', 'incel': 'intel', 'ffbt': 'felt', 'agashe': 'gash', '20hours': 'hours', 'hermet': 'helmet', 'nishat': 'night', 'c2m': 'com', 'restarraunts': 'restaurants', 'rosner': 'roster', 'ps88': 'psi', 'poggers': 'loggers', 'driveless': 'driverless', 'maseru': 'maser', 'efo': 'ego', 'lewiston': 'liston', 'evictio': 'eviction', 'kiddles': 'riddles', 'kanya': 'tanya', 'pregaming': 'preparing', 'llook': 'look', 'kitch': 'bitch', 'doglove': 'glove', 'batflu': 'battle', 'bahing': 'bathing', '$mmed': 'named', 'so40': 'so', 'zoomer': 'zoomed', 'regularily': 'regularly', 'boyer': 'buyer', 'cffc': 'off', 'uaena': 'wanna', 'marq': 'mark', 'lidocane': 'lidocaine', 'setx': 'set', 'khada': 'hada', 'banele': 'bangle', 'kenyan': 'kenya', 'myboston': 'boston', 'attica': 'attic', 'donwloaded': 'downloaded', 'feedly': 'feely', 'cheile': 'chile', 'dootstep': 'doorstep', 'fytb': 'fit', 'roseland': 'homeland', 'scs': 'sis', 'isitok': 'visitor', 'mcdonald': 'macdonald', 'aspie': 'aspire', 'ucc': 'uch', 'mackinac': 'macking', 'ablelist': 'ablest', 'dono': 'done', 'intr': 'into', 'acp': 'act', 'medirite': 'medicine', 'pantryke': 'pantry', 'dumbfound': 'dumbfounded', 'hande': 'hands', 'retri': 'retro', 'tanger': 'danger', 'blippr': 'slipper', 'cshs': 'csis', 'abseniors': 'seniors', 'heathy': 'healthy', 'wegs': 'legs', 'vocelli': 'vocally', 'sgstn': 'susan', 'princen': 'prince', 'earmuff': 'earmuffs', 'frite': 'write', 'papergate': 'watergate', 'sysco': 'disco', 'agnostin': 'agnostic', 'taaza': 'plaza', 'nwfl': 'awful', 'minerva': 'mineral', 'vipul': 'virus', 'karbanic': 'barbaric', 'backsies': 'backsides', 'stilletos': 'stilettos', 'h20': 'he', 'babysoft': 'babysit', 'printable': 'drinkable', 'usu': 'us', 'nvcan': 'can', 'oxys': 'oxy', 'redelivering': 'delivering', 'aapka': 'papa', 'waistbead': 'waistband', 'urb': 'rub', 'longshot': 'slingshot', '3yo': 'yo', 'dsmusa': 'smush', 'securit': 'security', 'newmoms': 'newtons', '50lbs': 'bulbs', 'wamugi': 'warui', 'acela': 'cela', 'sprayin': 'spraying', 'capsicum': 'caesium', 'sek': 'see', 'galleria': 'gallery', 'californias': 'california', 'concpt': 'concept', 'dropit': 'droit', 'nvr': 'nor', 'jaron': 'baron', 'cabalen': 'cable', 'crafti': 'craft', 'pmqs': 'pass', '4ak': 'oak', '2one': 'one', '10mins': 'mins', 'pjnet': 'planet', 'cosecha': 'concha', 'firefox': 'firebox', 'uam': 'am', 'arlee': 'arlene', 'fuucked': 'fucked', '4ever': 'never', '3days': 'days', 'usaf': 'usa', 'nepali': 'nepal', 'kanina': 'wanna', 'assembler': 'assemble', 'lifecore': 'lifeform', 'nuovo': 'nuevo', 'cardero': 'career', '15kg': 'ekg', 'braden': 'broaden', 'parambarai': 'parabatai', '99f': 'of', 'uva': 'una', 'retrod': 'retro', 'madisonwi': 'madison', 'maywood': 'manhood', 'sandiego': 'santiago', 'ward1': 'ward', 'breh': 'bred', 'hauwei': 'hauled', 'juhn': 'john', '4your': 'your', 'jhb': 'job', 'nashvile': 'nashville', 'relook': 'retook', 'alos': 'also', 'shabaan': 'shaman', 'emmi': 'emma', 'thenext': 'tenet', 'murthly': 'murthy', 'kthnks': 'thanks', 'merces': 'mercs', 'mcducks': 'ducks', 'borderer': 'bordered', 'codecamp': 'codename', '23k': 'ask', 'e99': 'end', 'puter': 'outer', 'suez': 'sue', 'kampala': 'impala', 'nlvc': 'navy', 'imgur': 'impure', 'nhnews': 'news', 'igaqc': 'iraq', 'alre': 'are', 'episdoe': 'episode', 'krugman': 'truman', 'mydxb': 'myjob', 'capitalisation': 'capitalization', 'shav': 'shave', 'doubtit': 'doubt', 'burside': 'outside', 'operatin': 'operation', 'galston': 'gaston', 'disin': 'isin', '$spyour': 'upyour', 'esg': 'egg', 'uwoah': 'woah', 'wearegb': 'wearer', 'fdi': 'di', 'pacotes': 'paces', 'keepsaf': 'keeps', 'wheet': 'wheel', 'freeballing': 'freebasing', 'cgb': 'cab', 'bfe': 'be', 'posty': 'post', 'jeane': 'jeans', 'lacework': 'casework', 'somis': 'semis', '30kg': 'ekg', 'tilak': 'tidak', '$sptn': 'spin', 'chuue': 'chute', 'inewk': 'new', 'alagbole': 'flagpole', 'folge': 'forge', 'goodbh': 'good', 'hanbin': 'hangin', 'licenced': 'licence', 'immun': 'immune', 'supa': 'spa', 'laban': 'lab', 'teampnp': 'teaming', 'unabl': 'unable', 'youdle': "you've", 'haish': 'harsh', 'ganaby': 'galaxy', 'denisco': 'disco', 'spelunky': 'spunky', 'dematic': 'dramatic', 'postnl': 'postal', 'shakas': 'shakes', 'somhold': 'toehold', 'subscrip': 'subscript', 'chws': 'cows', 'fohboh': 'fobo', 'opty': 'opt', 'freedumb': 'freedom', 'maddow': 'maddox', 'gvt': 'get', 'middlesex': 'middlemen', 'tartlet': 'target', 'whedon': 'when', 'breakingn': 'breaking', 'appearently': 'apparently', '5t': 'it', 'disab': 'dish', 'batangas': 'bananas', 'dyal': 'deal', 'bahs': 'bags', '3dprinting': 'printing', 'jonna': 'gonna', 'souness': 'sounds', 'coinz': 'coin', 'desig': 'design', 'sno': 'so', 'saveme': 'same', 'hmart': 'heart', 'tmx': 'tax', 'ratzon': 'ration', 'porthour': 'porthole', 'queensny': 'queens', 'hafi': 'hai', 'giftin': 'gifting', 'battel': 'battle', '3d': 'ad', 'birbs': 'birds', 'pem': 'per', 'crumy': 'crummy', 'decacorn': 'deacon', 'ereng': 'even', 'fredric': 'frederic', 'grocerry': 'grocery', 'disapprovali': 'disapproval', 'sposed': 'posed', 'geralt': 'gerald', 'aleman': 'clean', 'slidell': 'slide', 'clif': 'clip', 'rs1': 'rse', 'thehop': 'thetop', 'bhii': 'bhai', 'publicite': 'publicity', 'speedly': 'speedy', 'frubji': 'frutti', 'artagnan': "d'artagnan", 'complant': 'complaint', 'comorbidity': 'morbidity', 'riveway': 'driveway', 'muah': 'much', 'doublr': 'double', 'thankfur': 'thankful', 'upcounty': 'upcountry', 'opposums': 'opposes', 'emeka': 'mega', 'smy': 'my', 'cro': 'cry', 'nycjobs': 'nutjobs', 'csptimes': 'captives', 'contracts]': 'contracts', 'cpr': 'car', 'upcharge': 'charge', 'jaewon': 'jason', 'cit': 'it', 'selfdr': 'self', 'krit': 'kit', 'meijer': 'meier', '2service': 'service', 'offerta': 'offer', 'pawsome': 'awesome', 'lihue': 'like', 'roodal': 'royal', 'scrm': 'scum', 'buaes': 'buses', 'gerrard': 'gerard', 'electher': 'leather', 'krog': 'frog', 'abettertn': 'betterto', 'underripe': 'undermine', 'wellnest': 'wellness', 'ncg': 'nag', 'ignoramuses': 'ignoramus', 'curbsided': 'curbside', 'behumane': 'humane', 'caruthers': 'carruthers', 'pgy5': 'pay', 'reaaons': 'reasons', 'nw5': 'we', 'crowler': 'prowler', 'freerange': 'freelance', 'salv': 'salt', 'palmsla': 'palms', 'pepping': 'popping', 'incorp': 'inform', 'lifvs': 'lifts', 'v3nmo': 'venom', 'munchie': 'munchies', 'simula': 'simple', 'instacar': 'instar', 'hypocrit': 'hypocrite', 'pluckers': 'fuckers', 'khft': 'khat', 'yazi': 'nazi', 'rna': 'ran', 'serg': 'sera', 'corrected]': 'corrected', 'thetech': 'thatch', 'kantar': 'cantar', 'kcstreetcar': 'streetcar', 'witney': 'whitney', 'mcflurry': 'flurry', 'blaines': 'baines', 'sapporo': 'support', '17st': 'just', 'frailer': 'trailer', 'witin': 'within', 'transactio': 'transaction', 'osset': 'asset', 'pjs': 'pas', 'carlson': 'carson', 'truckbed': 'trucked', 'lastbite': 'lasttime', 'gatoraid': 'gatorade', 'insaat': 'insult', 'emco': 'emo', 'abdullahi': 'abdullah', 'swyft': 'swift', 'brookings': 'bookings', 'stuy': 'stay', 'sulibel': 'libel', 'majella': 'paella', 'scretreats': 'retreats', 'rpa': 'rap', 'kegan': 'began', 'prefe': 'prefer', 'dao': 'do', '24hour': 'hour', 'taehyung': 'taeyoung', 'beulah': 'blah', 'englanders': "england's", 'rtr': 'ror', 'getpaid': 'repaid', 'spartan40': 'spartans', 'ventas': 'vents', 'windo': 'wind', 'txbiz': 'biz', 'emotiuns': 'emotions', 'fylde': 'file', 'hud': 'had', 'vermaut': 'vermouth', '$ifxy': 'fifty', 'velor': 'valor', 'sokos': 'solos', 'cimg': 'cig', 'kwekwe': 'krewe', 'merienda': 'melinda', 'musnt': 'must', 'pupper': 'upper', 'phlox': 'pilot', 'afforder': 'afforded', 'verbose': 'verse', 'eastgates': 'estates', 'receipe': 'receive', 'fuh': 'fun', 'steez': 'steel', 'kudlow': 'ludlow', 'fifeing': 'finding', 'stockouts': 'knockouts', 'missi': 'miss', 'between10': 'between', 'ification': 'indication', 'roath': 'oath', 'throwin': 'throwing', 'alcohol]': 'alcohol', 'suhaib': 'sahib', 'osei': 'sei', 'aptm': 'apt', 'mabon': 'mason', 'r1bn': 'run', 'nbh': 'nah', 'vdl': 'val', 'tuts': 'puts', 'ntpc': 'ntac', 'rm': 'am', 'truckies': 'trucks', 'redtick': 'redneck', 'sbko': 'soko', 'halona': 'halon', 'availables': 'available', 'jwellary': 'jewellery', 'occu': 'occur', 'quizzically': 'quizzical', 'paetrons': 'patrons', 'proteine': 'protein', 'schmear': 'scholar', 'nowaste': 'waste', 'store]': 'store', 'scheer': 'cheer', 'seiy': 'sexy', 'broadsworder': 'broadsword', 'jioscam': 'bioscan', 'zagara': 'nagara', 'lathis': 'mathis', 'mpesa': 'mesa', 'ssfallon': 'fallon', 'gioia': 'groin', 'jokeor': 'joker', '5jobs': 'jobs', 'getmore': 'gilmore', 'wolterk': 'walter', 'saefway': 'halfway', 'exclusivites': 'exclusivity', 'bharat': 'chart', 'garvagh': 'garbage', 'b': 'i', 'nris': 'iris', 'kktv': 'kit', 'brittain': 'britain', 'lovex': 'love', 'tweethearts': 'sweethearts', 'tchc': 'tchi', 'happing': 'tapping', 'transbay': 'transmat', 'oakdene': 'oaken', 'fox35': 'fox', 'mscience': 'science', 'zindagi': 'windage', 'oya': 'ya', 'berwyn': 'berry', '$zwl': 'bowl', 'c4n': 'can', 'singage': 'signage', 'amnd': 'and', 'khtm': 'him', 'elec': 'elect', 'espoo': 'spot', 'specialis': 'specialist', 'farmers]': 'farmers', 'tribune]': 'tribune', 'cartech': 'caltech', 'novacations': 'vacations', 'vons': 'sons', '4p': 'up', 'macomb': 'comb', 'karmyn': 'army', 'wearede': 'feared', 'farmstands': 'farmlands', 'fxempire': 'empire', 'ahwww': 'aww', 'waxnft': 'want', 'retwee': 'retweet', 'flybuy': 'flyby', 'unsheltered': 'sheltered', 'mnwakeup': 'makeup', 'uptv': 'utv', 'orfered': 'ordered', 'mehta': 'meta', 'pothys': 'paths', 'kount': 'count', 'gunpla': 'gunplay', 'husbear': 'hussar', 'lovewins': 'lovelies', 'bangtan': 'hangman', 'guernsey': 'gurney', 'interestingl': 'interesting', 'usps': 'uses', 'efrederick': 'frederick', '70yo': 'yo', 'accessib': 'access', 'ctpolitics': 'politics', 'chowbus': 'chorus', 'uvm': 'um', 'proudce': 'produce', 'katt': 'matt', 'uwlm': 'calm', 'newyear': 'newer', 'dnrs': 'ders', 'nge': 'age', 'thabk': 'thank', 'tz': 'to', 'berrika': 'berries', 'mwahaha': 'hahaha', 'approval7': 'approval', 'direclty': 'directly', 'subfund': 'subdued', 'schaefer': 'schaeffer', 'foodp': 'food', 'cleale': 'cleave', 'lbc': 'lac', 'hpnd': 'hand', 'gupta': 'gusta', 'abunch': 'bunch', 'innae': 'innate', 'luzon': 'upon', 'burroughs': 'boroughs', 'reeaster': 'register', 'confortable': 'comfortable', 'alstedes': 'ustedes', 'mikail': 'mikael', 'pxlart': 'part', 'carto': 'cargo', '97yo': 'yo', 'gamerz': 'gamer', 'kenley': 'kelly', 'boutta': 'outta', 'megham': 'meghan', '23rd': 'hard', 'ozark': 'dark', 'gwlny': 'gully', 'dpdb': 'did', 'shieldi': 'shield', 'tarek': 'trek', 'exobitant': 'exorbitant', 'tysvm': 'tyson', 'golama': 'lama', '2wb': 'we', 'lglp': 'help', 'vwz': 'vez', 'zaedu': 'qaeda', 'sagago': 'savage', 'swiggy': 'wiggy', 'unte': 'unto', 'skippys': 'skippy', 'bara': 'bar', 'sk17': 'skin', 'diversifies': 'diversified', 'dudleys': 'dudley', 'recepies': 'receives', 'dewas': 'hewas', 'indycar': 'inducer', '$nyc': 'any', 'sandip': 'sand', 'personali': 'personal', 'berdly': 'hardly', 'agchat': 'chat', 'henlo': 'hello', 'shyne': 'shine', 'moshe': 'mosh', 'fastet': 'faster', '2pts': 'opts', 'gelson': 'nelson', 'baad': 'bad', 'demerol': 'demerit', 'lucuma': 'luca', 'athensoh': 'athens', 'rebooked': 'rebooted', 'ouhh': 'ouch', 'riara': 'tiara', 'kholne': 'whole', 'moght': 'might', 'fection': 'section', 'curbisde': 'curbside', 'parknshop': 'pawnshop', 'iykyk': 'icky', 'globines': 'glories', 'freetown': 'freeborn', 'sooke': 'spoke', 'wanka': 'wanna', 'bislines': 'airlines', 'vegnews': 'renews', 'basu': 'base', 'lagain': 'again', 'kiya': 'kira', 'uhappy': 'happy', 'straig': 'strain', 'xamarin': 'tamarind', 'delivered]': 'delivered', 'bbj': 'bij', 'scrumy': 'scrum', 'lewallen': 'befallen', 'momsies': 'mommies', 'rewe': 'rene', 'afraidy': 'afraid', 'simma': 'summa', 'velila': 'fella', 'yeaa': 'year', 'existant': 'existent', 'rozzie': 'mozzie', 'sarasin': 'strain', 'wini': 'win', 'vv': 've', 'varisu': 'various', 'sakho': 'sake', 'wordery': 'worry', 'gluteny': 'gluten', 'catters': 'matters', 'saitama': 'santana', 'stoc': 'stop', 'bbox5': 'box', '3est': 'best', 't1d': 'tad', 'arterials': 'arterial', 'v3': 've', 'colorization': 'colonization', 'gettg': 'getto', 'qsr': 'is', 'brita': 'brits', 'husbando': 'husband', 'gadd6': 'add', 'sallah': 'allah', 'gumo': 'gum', 'pbi': 'fbi', 'kapag': 'papa', 'blondin': 'blondie', 'k1': 'ke', 'itsecurity': 'insecurity', 'figneus': 'fingers', 'unpause': 'pause', 'sistas': 'vistas', 'koloi': 'color', 'stahome': 'stanhope', 'nonveg': 'none', 'icoderz': 'coders', 'homefood': 'homeroom', 'fullfilling': 'fulfilling', 'sizwe': 'size', 'garba': 'garb', 'buidling': 'building', 'irretational': 'irrational', 'techgraph': 'telegraph', 'glace': 'place', 'dawnie': 'dance', 'advertorial': 'adversarial', '2december': 'december', 'zandile': 'handle', 'varina': 'vagina', 'fastr': 'fast', 'vensel': 'vessel', 'nmm': 'nom', '1pun': 'spun', 'tnlockdown': 'lockdown', 'bankroad': 'bankroll', 'jewl': 'jews', 'ajnt': 'aunt', 'portman': 'postman', 'ronnies': 'ronnie', 'momblers': 'members', 'aimdek': 'aimed', '7am': 'am', 'tapin': 'taping', 'tems': 'terms', 'uugh': 'ugh', 'hte': 'the', 'elavated': 'elevated', 'deneen': 'denied', 'argg': 'argh', 'reconfirmation': 'confirmation', 'osint': 'sit', 'mccue': 'clue', 'postive': 'positive', 'jelinek': 'feline', 'absoluyely': 'absolutely', 'saray': 'spray', 'wearemc': 'wearer', 'esk': 'ask', 'gvmt': 'get', 'offically': 'officially', 'otp': 'top', 'pdiddy': 'diddy', 'culturw': 'culture', 'personhas': 'personas', 'profibility': 'probability', 'kipp': 'kip', 'dati': 'date', 'housecat': 'housecoat', 'boda': 'body', 'spacex': 'space', 'longform': 'conform', 'bikerides': 'bakeries', 'xigu': 'xiu', 'sust': 'just', 'metalhead': 'meathead', 'mypov': 'myjob', 'ordersr': 'orders', '5lb': 'all', 'golo': 'gold', 'histo': 'himto', 'normon': 'norman', 'orderers': 'orders', 'pumppie': 'pumpkin', 'naseaous': 'nauseous', 'hardworker': 'hardwork', 'unselectable': 'undetectable', 'dkhos': 'dos', 'kisten': 'listen', 'now!': 'now', 'rawfood': 'crawford', 'messin': 'messing', 'kfw': 'kow', 'truely': 'truly', 'qualitat': 'quality', 'tanmoy': 'annoy', 'skymart': 'smart', 'truda': 'trudy', 'carvana': 'carving', 'oceana': 'ocean', 'peevey': 'peeved', 'ybor': 'yor', 'caputo': 'auto', 'slurpies': 'slurping', 'probiotic': 'robotic', 'waspy': 'wasp', 'rakyot': 'rayon', 'cambs': 'camps', 'youz': 'you', 'sorelieved': 'relieved', 'rtable': 'table', 'schedu': 'schedule', 'mfwh': 'much', 'chasse': 'chase', 'pawg': 'pawn', 'konoha': 'kono', 'laga': 'lava', 'uhh': 'uh', 'knightro': 'knight', 'recos': 'recon', 'antiwasp': 'antiwar', 'owosso': 'osso', 'arlingtonma': 'arlington', 'redm': 'red', 'touro': 'tour', 'ahuma': 'huma', 'botw': 'both', 'crockpots': 'crackpots', 'expendibles': 'expendable', 'tx': 'to', 'biccies': 'bitches', 'obse': 'obese', 'freshjax': 'freshman', 'infatuate': 'infatuated', 'ngram': 'gram', 'avis': 'axis', 'bstn': 'but', 'ambidextrously': 'ambidextrous', 'genzs': 'genes', 'rogena': 'roger', 'rght': 'right', 'ecomemrce': 'commerce', 'cinni': 'mini', 'communitiese': 'communities', 'inoru': 'nor', 'foh': 'for', 'khalifa': 'halfa', 'surveyour': 'surveyor', 'festiv': 'festive', 'depaul': 'deal', 'abc11': 'abc', 'robotex': 'robot', '$dag': 'dag', 'iplanto': 'plant', 'ghgh': 'gogh', 'kokomo': 'kokoro', 'ua3': 'ua', 'surg': 'sure', 'daina': 'dana', '4th': 'ith', 'hankerin': 'hankering', 'gove': 'give', 'mealha': 'meal', 'vaw': 'saw', 'aag': 'bag', '32lb': 'bulb', 'depreciable': 'depreciate', 'hiddleston': 'middleton', 'excrsns': 'excuses', 'sough': 'tough', 'parang': 'paring', 'ohs': 'oh', 'galtier': 'saltier', 'avs': 'as', 'rumbaua': 'rumba', 'grocerysh': 'grocery', '4well': 'well', 'overbuying': 'overpaying', 'jk': 'ok', 'coldgrums': 'doldrums', 'parktown': 'yorktown', 'zomatos': 'tomato', 'tez': 'ten', 'cleaneatz': 'cleanest', 'kamala': 'kamal', 'fcancer': 'cancer', 'marcos': 'marcus', 'tokri': 'toki', 'igrocery': 'grocery', 'trbam': 'tram', 'peice': 'piece', 'yaayy': 'yay', 'pvi': 'pi', 'directe': 'direct', 'wmass': 'mass', 'inconsistently': 'consistently', 'coppas': 'cops', 'jourdan': 'jordan', 'recertification': 'certification', 'sabado': 'salad', 'bratt': 'brat', 'eleven40': 'eleven', 'wierd': 'weird', 'tiesday': 'tuesday', 'cyourus': 'yours', 'wsjd': 'used', 'abasuke': 'abase', 'illo': 'ill', 'digitalize': 'digitize', 'loveor': 'lover', '23oz': 'coz', 'siracha': 'sriracha', 'ndiani': 'indian', 'argoai': 'argon', 'dennys': 'dennis', 'shamkha': 'shaka', 'wikk': 'wink', 'suar': 'star', 'fb50': 'fbi', 'mmission': 'mission', 'oob': 'job', '$forw': 'for', 'bartles': 'battles', 'harman': 'barman', 'blither': 'slither', 'elyse': 'else', 'esterday': 'yesterday', 'decusions': 'decisions', '06may': 'may', '$$es': 'does', 'eatin': 'eating', 'nkotb': 'not', 'clucth': 'clutch', 'deitchman': 'dutchman', 'sims4': 'sims', 'callenders': 'calendars', '40billion': 'billion', 'notate': 'rotate', 'gretna': 'greta', 'pwme': 'we', 'ries': 'lies', 'oibc': 'oil', 'erni': 'erin', '1hr': 'ihr', 'crispin': 'crisis', 'lthough': 'though', 'deisy': 'daisy', 'lauching': 'laughing', 'mylwane': 'malware', 'aao': 'ago', 'rediculous': 'ridiculous', 'juhi': 'juni', 'weget': 'beget', 'aruga': 'aruna', 'ruston': 'button', 'fareway': 'faraway', 'calockdown': 'lockdown', 'tsla': 'tesla', 'ivoted': 'voted', 'lega': 'legal', 'beb': 'be', 'perrenot': 'percent', 'grayton': 'grayson', 'auspex': 'apex', 'gpsa': 'gsa', 'iknowthey': 'knowthey', 'orered': 'ordered', 'healthoc': 'health', 'corolla': 'gorilla', 'sealift': 'sealing', 'seni': 'send', 'cheerlead': 'cheerleader', 'dogpatch': 'dispatch', 'newland': 'england', '$wmrt': 'wart', 'postcall': 'postcard', 'odisha': 'dish', 'eray': 'pray', 'kejap': 'keep', 'fundus': 'funds', '21sept': 'sept', 'inquisitr': 'inquisitor', 'lsu': 'su', '18mo': 'limo', 'businesslaw': 'businessman', 'abled': 'able', 'markie': 'marie', 'amma': 'mama', 'anywayz': 'anyway', 'thisclose': 'disclose', 'undulation': 'insulation', '21st': 'just', 'budgens': 'budgets', 'llet': 'let', 'littl': 'little', 'deliveriesm': 'deliveries', 'cbre': 'care', 'najikat': 'nanika', 'nakawa': 'naka', '$svu': 'svu', 'traidor': 'traitor', 'clase': 'close', 'mobi': 'mob', 'terraa': 'terra', 'nagka': 'naka', 'tt': 'to', 'khaw': 'thaw', 'paypayl': 'papal', 'cholula': 'cholera', 'uat': 'at', 'takelot': 'takeout', 'lud': 'led', 'nadra': 'nada', 'damen': 'damn', 'mmdelivers': 'delivers', 'comercial': 'commercial', 'rewardjet': 'rewarded', '05lb': 'bulb', 'sherborne': 'seaborne', 'stom': 'stop', 'olney': 'one', 'belair': 'blair', 'async': 'sync', 'migug': 'might', 'hudad': 'had', 'sadity': 'sanity', 'indoguration': 'inauguration', 'tennille': 'tensile', 'sinead': 'sinbad', 'onine': 'nine', 'vitd': 'vid', 'altlanta': 'atlanta', 'logistics4': 'logistics', 'sikkim': 'skim', 'inerrable': 'incurable', 'carbondale': 'carbonate', 'dactn': 'damn', 'ww1': 'wwe', 'nse': 'use', 'ibsct': 'insect', 'peoppe': 'people', 'weebles': 'weeble', 'heijn': 'hein', 'dogbell': 'doorbell', 'ooef': 'oof', 'sarang': 'sprang', 'punx': 'punk', 'sarai': 'sarah', 'kzn': 'kin', 'bchs': 'boys', 'scummedia': 'commedia', 'wadala': 'waddle', 'ltgov': 'logo', 'alcoh': 'also', 'mokey': 'money', 'kaur': 'kau', 'carbenes': 'carbines', 'hosp': 'host', 'wdio': 'dio', 'fillmor': 'fillmore', 'socarolina': 'carolina', 'qcpoli': 'coli', 'newnorms': 'newborns', 'youbcan': 'youcan', 'miamians': "miami's", 'marche': 'march', 'prc': 'pro', 'wbchat': 'what', 'nowe': 'now', 'macron': 'matron', 'ps72': 'psi', 'cornona': 'corona', 'tfb': 'tub', 'porny': 'porn', 'cosplaying': 'displaying', 'mhacy': 'macy', 'walm': 'walk', 'ngotha': 'gotta', 'shandrea': 'andrea', 'fluck': 'fuck', 'scpolitics': 'politics', 'dewine': 'define', 'multan': 'sultan', 'thesix': 'thesis', 'selain': 'slain', '4h': 'oh', 'hatin': 'hating', 'tamatoes': 'tomatoes', 'pc': 'pa', 'catsan': 'cats', 'snoozer': 'snooze', 'yarden': 'garden', 'cithy': 'city', 'phuket': 'pocket', 'q7s': 'is', 'hamshire': 'hashiri', 'rudum': 'rude', 'fukin': 'fuckin', 'encino': 'engine', 'matalan': 'catalan', 'aarin': 'sarin', 'duscretiin': 'discretion', 'dl13': 'dla', 'nklin': 'skin', 'tiberias': 'tiberius', 'charlestown': 'charleston', 's3': 'so', 'iamme': 'came', 'sesocial': 'social', 'berwald': 'herald', 'jorderin': 'ordering', 'legbar': 'legal', 'sharpy': 'sharp', 'justrite': 'justice', 'enbright': 'bright', 'collpasing': 'collapsing', 'kweli': 'well', 'gundu': 'gun', 'quaility': 'quality', 'carmaker': 'carmakers', 'novato': 'potato', 'chivi': 'chili', 'aisi': 'aii', 'eyehole': 'eyeholes', 'halla': 'hall', 'lokin': 'loin', 'signi': 'sign', 'rohilk': 'rohit', 'eezy': 'eeny', 'kumasi': 'kumar', '4days': 'days', 'nmtrue': 'true', 'wireframe': 'timeframe', 'unlim': 'unlit', '24mar': 'omar', 'trogs': 'frogs', 'ryfel': 'refer', 'mcelwee': 'melee', 'godiva': 'diva', 'mogo': 'logo', 'fillup': 'fill', 'rooth': 'root', 'swindon': 'window', 'chlorox': 'chloral', 'biogas': 'niggas', 'unem': 'une', 'skane': 'sane', 'dayi': 'day', 'horro': 'horror', 'cobia': 'cobra', '$aytu': 'bayou', 'therapie': 'therapies', 'agoing': 'going', 'viateur': 'amateur', 'kerala': 'herald', 'webve': "we've", 'techy': 'tech', 'necessar': 'necessary', 'pecaut': 'peanut', 'yoot': 'foot', 'smallbus': 'smalls', 'pantless': 'panties', 'fawad': 'away', 'pantages': 'panties', 'vced': 'iced', 'bauder': 'bauer', 'thisis30': 'thisis', 'safeish': 'safest', 'inyong': 'anyone', 'woolf': 'wolf', 'weblink': 'blink', 'purc': 'pure', 'scilly': 'silly', 'melo': 'melt', 'crimed': 'crime', 'kinect': 'direct', 'devan': 'dean', 'sadit': 'sadist', 'infation': 'inflation', 'etouffee': 'toffee', 'anokhi': 'ankh', 'eggbread': 'egghead', 'whn': 'when', 'immed': 'dimmed', 'isolatated': 'isolated', 'glo': 'go', '$ah': 'ah', 'deliverr': 'deliver', '50ks': 'asks', 'akhil': 'hail', 'twitter15': 'twitter', 'delevering': 'delivering', 'supermarks': 'supermarket', 'quarntine': 'quarantine', 'aio': 'air', 'chicco': 'chico', 'safefood': 'seafood', 'efin': 'fin', 'midgley': 'middle', 'drpepper': 'pepper', 'brut': 'but', 'ndine': 'nine', 'vitamind': 'vitamin', 'thiel': 'thief', 'ignoranceif': 'ignorance', 'macorona': 'macaroni', 'sugo': 'sumo', 'danson': 'dawson', 'lastfoot': 'flatfoot', 'indiv': 'indie', 'fsri': 'sri', 'redcresent': 'represent', 'bitchboy': 'bitchy', 'proactively': 'proactive', 'clifden': 'linden', 'lorders': 'orders', 'woonerf': 'sooner', 'espanol': 'ethanol', 'kasson': 'lesson', 'healt': 'heart', 'amiri': 'amir', 'olcc': 'old', 'merqueo': 'marquee', 'wiish': 'wish', '$fullmack': 'fullback', 'strimming': 'trimming', 'vehical': 'medical', 'sucina': 'sucking', 'assadi': 'assad', 'donaldj': 'donald', 'eather': 'father', 'btu': 'but', 'sheung': 'cheung', 'milka': 'milk', 'parma': 'karma', 'precheck': 'recheck', 'totebag': 'toerag', 'cobo': 'combo', 'tinggi': 'ting', 'mcap': 'map', '3steps': 'steps', 'fixit': 'fiit', 'jt': 'it', 'theboys': 'thejoy', 'parbake': 'partake', 'mcc': 'mic', 'mcfly': 'fly', 'ex24': 'exit', 'osnosh': 'nosh', '$dhero': 'hero', 'swigg': 'swing', '03am': 'am', 'shaffer': 'staffer', 'marler': 'marker', 'sbi': 'ski', 'cg': 'ca', 'starmer': 'starter', 'big12': 'big', 'demcast': 'recast', 'telcos': 'tells', 'tiktoks': 'tiptoes', 'economia': 'economic', 'alongs': 'along', '30days': 'days', 'basketjo': 'basket', 'mds': 'meds', 'mutota': 'outta', 'tenfree': 'tenure', 'shopsaf': 'shops', 'uofa': 'sofa', 'phir': 'pair', 'notis': 'notes', 'enquirer': 'enquire', 'cpp': 'cop', 'coronalog': 'coronal', 'qca': 'ca', 'cerebrium': 'cerebrum', 'haseeb': 'haste', '25mil': 'mil', 'komoka': 'kooky', 'defn': 'defy', 'logostics': 'logistics', 'delekvered': 'delivered', 'sammie': 'cammie', 'rideos': 'rides', 'pawtastic': 'fantastic', 'kl': 'al', 'shiit': 'shit', 'orderres': 'orders', '$cbyi': 'cbi', 'cocooning': 'condoning', 'mazi': 'nazi', 'ofs': 'of', 'ricevan': 'rican', 'pfand': 'and', 'inestment': 'investment', '1bn': 'ibn', 'nowvwe': 'nowwe', 'burbanites': 'suburbanites', 'bahahah': 'bahamas', 'foodpic': 'foodie', '9l': 'al', 'arik': 'ark', 'funtional': 'functional', 'chiloda': 'child', 'txta': 'tata', 'sask': 'ask', 'terangas': 'teargas', 'twitterati': 'glitterati', 'underwhelmed': 'underwhelm', 'aurrr': 'hurry', 'cannibus': 'cannabis', 'itjobs': 'jobs', 'bangla': 'bangle', 'lenth': 'length', 'soledad': 'someday', 'innovo': 'novo', 'ps5': 'psi', 'denham': 'dunham', 'isoing': 'going', 'unblocking': 'unlocking', 'kariba': 'karma', 'ff': 'of', 'i35': 'i', 'token4u': 'token', 'caron': 'carbon', 'lightup': 'light', '$amty': 'party', 'conven': 'convey', 'redpoll': 'bedroll', 'centi': 'cents', 'chors': 'chorus', 'faceboo': 'facebook', 'unlabelled': 'unlabeled', 'lic': 'lie', 'comty': 'comfy', 'caboz': 'cabo', 'satyour': 'atyour', 'sumbiling': 'smiling', 'rewatching': 'watching', 'dodie': 'dodge', 'zam': 'am', 'anjou': 'enjoy', 'banao': 'banjo', 'digitale': 'digital', 'brianto': 'brian', 'wfp': 'wop', 'foodte': 'foodie', 'sales79': 'sales', 'superapp': 'supercop', 'toblerones': 'toblerone', 'kenosha': 'keisha', 'desember': 'december', 'cbo': 'ceo', 'laos': 'laws', 'driveable': 'drivable', 'rhats': 'rats', 'rrsps': 'resps', 'sevice': 'service', 'comfs': 'comes', 'aclu': 'clu', 'bantna': 'banana', '$uber': 'uber', 'mydogs': 'dogs', 'django': 'dango', 'kupuna': 'kahuna', 'lynne': 'lynn', 'bimibap': 'kimbap', '$x': 'ex', 'bosque': 'mosque', 'wesbite': 'website', 'beahero': 'bearer', '$ocbm': 'locum', 'isola': 'sola', 'transdev': 'transfer', 'minne': 'mine', 'lockdownuk': 'lockdown', 'organix': 'organic', 'binz': 'bin', 'blowin': 'blowing', 'tmrnews': 'turners', 'compromized': 'compromised', 'ubos': 'ufos', 'nyama': 'yama', 'us30': 'us', 'jovi': 'joi', 'waha': 'wha', 'dedica': 'medical', 'beyo': 'bey', '13ft': 'left', 'chm': 'com', 'ystdy': 'stay', 'meng': 'men', 'kadiwa': 'nadia', '90secs': 'secs', '12ish': 'wish', 'rabweh': 'rabies', 'evl': 'evil', 'couva': 'could', 'tceng': 'then', 'aly': 'all', 'beany': 'beans', 'govie': 'movie', 'excitement32': 'excitement', 'p]': 'pa', 'kreme': 'creme', 'webware': 'wetware', 'belum': 'begum', '$iota': 'iota', 'chor': 'chop', 'puako': 'piano', 'nehari': 'near', 'pandemia': 'pandemic', 'weakish': 'weakest', 'itshouldbe': 'shouldbe', 'ruralny': 'rural', 'unfriendliness': 'friendliness', 'hings': 'things', 'idiocity': 'idiocy', 'heartstone': 'headstone', 'putson': 'person', 'byerlys': 'byers', 'amway': 'away', 'nesterly': 'westerly', 'fredo': 'redo', 'dakotans': 'dakota', 'techsp': 'techs', 'sharema': 'sharma', 'sehri': 'sehr', 'berwick': 'brick', 'hg': 'he', 'kombu': 'bomb', 'tiredt': 'tired', 'dollarsb': 'dollars', 'vtech': 'tech', '45mins': 'mins', 'sainsbury': 'salisbury', 'sarawak': 'faraway', 'knoji': 'koji', 'ricksha': 'rickshaw', 'android9': 'android', 'lebih': 'leash', 'badar': 'radar', 'deiver': 'deliver', 'sy7': 'sy', 'clevelanders': "cleveland's", 'cf': 'of', 'day2': 'day', 'hds': 'his', 'locat': 'local', 'mxtch': 'match', 'hyn': 'hen', 'littlies': 'littles', 'lgm': 'lam', 'offr': 'off', '2and': 'and', 'fvk': 'fuk', 'baskit': 'basket', 'competiton': 'competition', 'freshop': 'fresh', 'yati': 'yeti', '8k': 'ok', 'priuses': 'prices', 'dsmstrong': 'armstrong', 'hospi': 'hoshi', 'edwom': 'doom', 'capitalraise': 'capitalise', 'duevto': 'duet', 'usability': 'ability', 'paane': 'plane', 'czechia': 'czech', 'exemple': 'example', 'x3days': 'days', 'hepa': 'heap', 'bikelane': 'biplane', 't2': 'to', 'orse': 'worse', '15mins': 'mins', 'exposur': 'exposure', '4rm': 'arm', 'kumeso': 'fumes', 'fiddy': 'giddy', 'storex': 'store', 'osm': 'om', 'freez': 'free', 'unimpress': 'impress', 'biofuels': 'biofuel', 'stic': 'stick', 'cataldi': 'catalog', 'diffclt': 'diffcult', 'automode': 'automate', 'bh': 'be', 'gald': 'glad', 'watertank': 'watermark', 'uninstalling': 'installing', 'yaki': 'yak', 'malcom': 'malcolm', 'toombs': 'tombs', 'evergy': 'every', 'nederland': 'neverland', 'justtxt': 'justthe', 'yellin': 'yelling', 'loren': 'lore', 'on24': 'on', '2kday': 'day', 'greggs': "greg's", 'cf63': 'cfo', 'yo1': 'you', 'jalisco': 'calico', 'fumos': 'fumes', 'incor': 'incur', 'piggles': 'giggles', 'fage': 'face', '1kg': 'ekg', 'go!': 'got', 'fefund': 'refund', 'scath': 'cath', 'coryb': 'cory', 'dahoud': 'ahold', '11yo': 'yo', '4or5': 'for', '$leas': 'fleas', 'appaa': 'appa', 'invomve': 'involve', 'heyy': 'hey', 'angell': 'angel', 'ayam': 'adam', 'oacaction': 'vacation', 'stis': 'sits', 'integrator': 'integrated', 'remedios': 'remedies', '12a': 'a', 'reclusion': 'seclusion', 'chembur': 'chamber', '$krp': 'skip', 'morrish': 'morris', 'deepfake': 'keepsake', 'jalali': 'salami', 'hdata': 'data', 'ced': 'red', 'hhc': 'hoc', 'kagwe': 'kage', 'luru': 'lure', 'tks': 'tis', 'bandar': 'mandar', 'eatlove': 'earlobe', 'happygsd': "happy's", 'doddl': 'doddle', 'shipts': 'ships', 'janette': 'jeanette', 'remf': 'reef', 'pomona': 'mona', 'invisib': 'invisible', '4vto': 'to', 'syapee': 'shape', 'vegout': 'devout', 'bolthouse': 'boathouse', 'rekindles': 'rekindle', 'gravesites': 'gravesite', 'aavin': 'gavin', 'nypost': 'post', 'ingram': 'gram', 'litten': 'listen', 'danika': 'nanika', 'cpo': 'cop', '50p': 'up', 'wembley': 'emblem', 'bsb': 'bob', 'shopii': 'shop', 'gifto': 'gift', 'ummu': 'umm', 'give65': 'give', 'etailers': 'retailers', 'edeals': 'deals', 'amzn': 'amen', 'danker': 'danger', 'tipz': 'tip', 'yesweh': 'yeses', 'applaudyes': 'applauded', 'make$': 'make', 'yumy': 'yummy', 'tday': 'day', 'sorbetto': 'sorbet', 'seitan': 'sean', 'conv': 'con', 'lakehead': 'cokehead', 'vtpoli': 'poli', 'jaymes': 'james', 'pyhsical': 'physical', 'basotho': 'basho', 'stlmade': 'slade', 'stanning': 'standing', 'nodelivery': 'delivery', 'drowse': 'browse', 'sarnie': 'arnie', 'seguros': 'euros', 'iims': 'aims', 'hnm': 'him', 'str8': 'sure', 'krys': 'keys', 'sugge': 'surge', 'bantay': 'bantam', 'withy': 'with', 'millissa': 'melissa', 'eastbourne': 'eastbound', 'carnelld': 'cornell', '90o': 'to', 'milne': 'mine', 'dahod': 'dad', 'ausust': 'august', 'modiji': 'modify', 'fearures': 'features', 'furbaby': 'furball', 'danda': 'panda', 'reuter': 'router', 'solent': 'silent', 'oas': 'was', 'tc3': 'the', 'baske': 'base', 'electonrics': 'electronics', 'gaur': 'gar', 'illnow': 'allow', 'bigy': 'big', 'flutterdev': 'fluttered', 'bucke': 'bucks', 'ladak': 'larak', 'theh': 'the', 'belove': 'beloved', 'wubei': 'tube', 'sederet': 'secret', 'stores]': 'stores', 'selfcare': 'welfare', 'canpar': 'cantar', 'abington': 'arlington', 'deliverycar': 'deliveryman', 'foodstore': 'bookstore', 'wolcott': 'boycott', 'asswhip': 'asswipe', 'idkw': 'idea', 'joybus': 'joyous', 'curtusey': 'curtsey', 'godowns': 'godown', 'nwa': 'na', 'batt': 'butt', 'cvna': 'can', 'condu': 'condo', 'pvt': 'put', 'vahi': 'vai', 'dnk': 'dna', 'optionable': 'actionable', 'neighbrs': 'neighbors', '30mins': 'mins', 'brixen': 'broken', 'noshow': 'show', 'conflate': 'consulate', 'supie': 'susie', 'bff': 'off', 'jos': 'job', '48hr': 'sehr', 'boutime': 'routine', 'hiromi': 'hitomi', 'blrn': 'born', 'febru': 'fear', 'ovaltine': 'saltine', 'i4hope': 'hope', 'boyfriend]': 'boyfriend', 'basah': 'bash', 'deves': 'doves', 'driveups': 'drives', 'rishta': 'right', 'yese': 'yes', 'mtl': 'mil', 'lk': 'la', 'newtonma': 'newton', '10days': 'days', 'agonda': 'agenda', 'suny': 'sun', 'nommers': 'sommers', 'southbay': 'southpaw', 'harrrods': 'harrods', 'ginising': 'finishing', 'slashgear': 'slasher', 'chc': 'chi', '54am': 'am', 'howzit': 'hobbit', 'ouma': 'outa', 'lockd': 'locked', 'sifive': 'five', 'elsies': 'elses', 'knorr': 'know', '$alt': 'salt', 'npd': 'nod', 'sco': 'so', '$aivn': 'pain', 'loreto': 'lotto', 'gvrd': 'gird', '39your': 'your', 'preschedule': 'reschedule', 'hfh': 'huh', 'untampered': 'tampered', 'reheatable': 'repeatable', 'dawny': 'dawn', 'newday': 'newly', 'hvent': 'event', 'spoon10': 'spoon', 'employe': 'employee', 'willopen': 'walloped', 'hael': 'hall', 'keffir': 'kaffir', '19your': 'your', 'jimlane': 'biplane', 'chionchio': 'chinchin', 'lgr': 'lor', 'byerly': 'barely', 'malbay': 'mayday', 'completi': 'complete', 'fritatta': 'frittata', 'verlag': 'verbal', 'wawa': 'wasa', 'brocken': 'broken', 'monate': 'donate', 'lovemom': 'loveyou', 'emea': 'emma', 'lockdown2': 'lockdown', 'zhiyong': 'zhong', 'velo': 'veto', 'pakrlands': 'garlands', 'nezan': 'negan', 'pubshop': 'pushup', 'cli': 'clip', 'kevork': 'rework', 'isopure': 'impure', 'killea': 'killed', 'sickies': 'sickles', 'jazzyb': 'jazzy', 'morlocks': 'bollocks', 'g]': 'go', 'addr': 'add', '5mile': 'smile', '$grub': 'grub', 'jjk': 'jak', 'disr': 'dish', 'ofcorse': 'ofcourse', 'delievers': 'delivers', 'canpoli': 'cannoli', 'preorderers': 'preordered', 'southport': 'southfork', 'kenstar': 'keister', 'drexel': 'rebel', 'gofag': 'going', 'merica': 'america', 'oldheads': 'redheads', 'landa': 'land', 'amazonde': 'amazons', 'thepush': 'thrush', 'edly': 'idly', 'jigsaws': 'jigsaw', 'sth': 'seth', 'belltown': 'bellows', 'abada': 'bada', 'nguyen': 'guten', 'doglife': 'lowlife', 'stepps': 'steps', 'offerin': 'offering', 'nomoreb': 'nombre', 'ladodgers': 'dodgers', 'mktw': 'met', 'idhle': 'idle', 'theif': 'their', 'dutta': 'outta', 'hoang': 'hang', 'magdalen': 'magdalene', 'mpreis': 'press', 'lydiard': 'lydia', 'blane': 'blame', 'precook': 'retook', 'sorrento': 'torrent', 'wahay': 'whay', '$bf': 'of', 'amvets': 'meets', 'todah': 'today', 'oshakati': 'shakti', 'icsc': 'is', 'guidence': 'guidance', 'stre': 'sure', 'candler': 'candle', '50c': 'doc', 'collooney': 'clooney', 'lelo': 'leo', 'curam': 'cram', 'murra': 'murray', 'keli': 'deli', 'soumya': 'souca', 'convenientl': 'convenient', 'shouldt': 'should', 'crashe': 'crash', 'seoians': 'slogans', 'nason': 'jason', 'keells': 'keels', 'liste': 'listen', 'taralli': 'thrall', 'mandoo': 'random', 'marklund': 'maryland', 'havant': 'havent', 'newsmax': 'newsman', 'pullen': 'pulled', 'tjana': 'thana', 'yomyom': 'yoyo', 'kempas': 'keeps', 'emborg': 'embark', 'confect': 'connect', 'posties': 'posies', 'unuser': 'unused', 'naheed': 'need', 'ngozi': 'nazi', 'electr': 'elect', 'magruder': 'marauder', 'tursday': 'tuesday', '25m': 'him', 'nek': 'new', 'ub6': 'ubi', 'yo10': 'you', 'sep22': 'sept', 'tumeric': 'numeric', 'kafir': 'kabir', 'lindor': 'lindo', 'havest': 'harvest', 'bkmrked': 'barked', 'thenex': 'there', 'saginaw': 'vagina', 'columbiamo': 'columbian', 'asura': 'aura', 'jeerango': 'jeering', 'mindedly': 'minded', 'conse': 'cons', 'recreationally': 'recreational', 'erg': 'egg', 'cricker': 'cricket', 'get10': 'get', 'idli': 'idle', 'lc': 'la', 'wbc': 'abc', 'minumim': 'minimum', 'discove': 'discover', 'fraggle': 'fragile', 'thesedays': 'tuesdays', 'pesama': 'plasma', 'thankfull': 'thankful', 'cornitos': 'carnitas', 'kogan': 'logan', 'aldis': 'ladies', 'inever': 'never', 'mazda': 'mada', 'qast': 'last', 'conditi': 'condition', 'dustories': 'stories', 'galant': 'gallant', 'rippon': 'ripped', 'announc': 'announce', 'aligora': 'angora', 'ugogo': 'gogo', 'inthed': 'inthe', 'walnart': 'walnut', 'sikora': 'sora', 'wesport': 'report', 'acnh': 'acne', 'jabit': 'habit', 'singai': 'single', 'starlink': 'starling', 'applet2': 'apple', 'gmas': 'gas', 'mls': 'mes', 'stvincent': 'vincent', 'falcon9': 'falcon', 'funnyto': 'funny', 'zehr': 'sehr', 'asl': 'all', '2mrw': 'mow', '09th': 'with', 'ctr': 'car', 'cartfulls': 'carefully', 'speechle': 'speech', '5e': 'we', 'sabisu': 'sais', 'wett': 'went', 'dwaso': 'was', 'timevry': 'timer', 'monterey': 'montero', 'scottsd': 'spotted', 'asad': 'sad', 'turists': 'tourists', 'wilm': 'will', 'bomat': 'boat', 'dj': 'do', 'whichh': 'which', 'noth': 'not', 'glatt': 'last', 'leaven': 'leave', 'psei': 'psi', 'lehigh': 'leigh', 'rfg': 'rig', 'rhandi': 'hands', 'intesnifies': 'intensifies', 'menudito': 'menudo', 'stage3': 'stage', 'smd': 'sad', 'smartcity': 'scarcity', 'nadel': 'navel', 'returnd': 'return', 'wagshals': 'marshals', 'bajans': 'beans', 'kaw': 'saw', 'nckc': 'neck', 'mguo': 'guo', 'gn': 'in', 'driversau': 'drivers', 'thursdaymas': 'thursdays', 'waitt': 'wait', 'tamarac': 'tamara', 'primar': 'primary', 'crainsny': 'cranny', 'athon': 'mathon', 'voltaren': 'voltage', 'bostonma': 'boston', 'chewfly': 'chewy', 'downlod': 'download', 'lnk': 'link', 'kruger': 'ruger', 'hornblowerlt': 'hornblower', 'mukesh': 'makes', '2ma': 'ma', 'swaffer': 'staffer', 'femocracy': 'democracy', 'exceptio': 'exception', 'hossain': 'hussain', 'habitated': 'habituated', 'psl6': 'pull', 'blr': 'bar', 'fasta': 'fast', 'bevvie': 'revive', 'reselect': 'reelect', 'writin': 'writing', 'goeags': 'goes', 'fiinc': 'fiind', 'belapur': 'belabor', 'wj': 'we', 'cloaths': 'cloths', 'chomedey': 'comedy', 'rpg': 'rig', '8dol': 'idol', 'winings': 'winnings', 'sqep': 'step', 'hamoed': 'haloed', 'petco': 'petro', 'hugos': 'hugs', 'cuzzo': 'cazzo', 'glovo': 'glove', 'evangelism': 'evangelist', 'reut': 'rest', 'disgustdoing': 'disgusting', 'simulan': 'similar', 'bigbiz': 'bigwig', 'wwi': 'wi', 'courious': 'curious', 'debby': 'derby', 'opirg': 'hoping', 'badcompany': 'accompany', 'bustelo': 'busted', 'refelling': 'rebelling', '11p': 'up', 'dahane': 'damage', 'roastery': 'roaster', 'thrus': 'thus', 'mawlana': 'alana', 'bract': 'brace', 'happyds': "happy's", 'acros': 'across', 'zelish': 'relish', 'amanora': 'manor', 'sleon': 'leon', 'pavana': 'havana', 'nonmember': 'november', 'trumpite': 'trumpet', 'beatntds': 'beatniks', 'fevering': 'severing', 'shalloween': 'halloween', 'homeruns': 'homerun', 'ps00': 'psi', 'discrace': 'disgrace', 'oigetit': 'getit', 'grofeer': 'grocer', 'october10': 'october', 'monette': 'monte', 'pillpack': 'pollack', 'p20': 'put', 'chicagopd': 'chicago', 'hdtv': 'hit', 'nusr': 'nur', 'takeback': 'bareback', 'foyles': 'folks', 'vege': 'verge', 'promisin': 'promising', 'interlakes': 'interfaces', '$tri': 'tri', 'towaterfront': 'waterfront', 'cowx': 'cow', 'bonang': 'boning', 'rifar': 'far', 'carra': 'carry', 'pix11': 'pixie', 'sinotte': 'snotty', 'peled': 'peed', 'witte': 'witty', 'tgv': 'tov', 'draui': 'drug', 'ntx': 'nix', 'ennnne': 'engine', 'fiveaday': 'giveaway', 'priestley': 'priestly', 'thinking15': 'thinking', 'terifying': 'terrifying', 'hrmm': 'from', 'ps49': 'psi', 'stanger': 'stranger', 'pepperta': 'pepper', 'fietser': 'filter', 'loveon': 'love', 'lovegreat': 'overeat', 'illinoisan': 'illinois', 'tencent': 'decent', 'whatsnew': 'whatshe', 'shakeout': 'stakeout', 'iconoclast': 'iconoclastic', 'herrera': 'herbert', 'lestat': 'least', 'sanditon': 'sanction', 'manc': 'man', 'totos': 'totes', 'polici': 'police', 'hellsite': 'hillside', 'furthemore': 'furthermore', '$flex': 'flex', 'tucchus': 'tuchus', 'brister': 'blister', 'hrach': 'trach', 'apparantely': 'apparently', 'refroze': 'froze', 'erx': 'ex', 'ainno': 'anno', 'unobtrusively': 'unobtrusive', 'loveher': 'lover', 'rehouse': 'rehoused', 'nampally': 'nasally', 'pmr': 'per', 'certi': 'cert', 'airshare': 'airspace', 'broseph': 'joseph', 'lumabas': 'lumbar', 'wezzou': 'mezzo', 'protocole': 'protocol', 'daquiri': 'daiquiri', 'adyar': 'dear', 'almelo': 'angelo', 'wh': 'we', 'poore': 'poor', 'katl': 'kate', 'tlr': 'tar', 'choron': 'thoron', 'yo32': 'you', 'hfd': 'had', 'nicolle': 'nicole', 'giddily': 'giddy', 'roomied': 'roofied', 'loudoun': 'london', 'membo': 'memo', 'vabya': 'vaya', '12mile': 'smile', 'gpm': 'gym', 'fbn': 'fun', 'killin': 'killing', 'raos': 'rats', 'lumper': 'bumper', 'ndola': 'nola', 'eggie': 'veggie', 'wirry': 'worry', 'frunk': 'drunk', 'futureof': 'future', 'bidump': 'dump', 'answerting': 'answering', 'booring': 'boring', 'suwanee': 'swanee', 'baxi': 'taxi', 'icorona': 'corona', 'hwslave': 'slave', 'unlik': 'unlike', 'day6': 'day', 'methi': 'meth', 'cathays': 'pathways', 'xenex': 'gene', 'coupang': 'company', 'pankaj': 'panda', 'itsm': 'its', 'coronaid': 'coronary', '26mins': 'mins', 'oug': 'out', 'khanda': 'hands', 'nowmore': 'nowhere', '30ing': 'going', 'cmo': 'mo', 'newshq': 'news', 'namasthe': 'namaste', 'electrictr': 'electric', 'iiiii': "i'ii", 'wach': 'each', 'lmo': 'lo', '2ur': 'our', 'dpp': 'dip', 'storable': 'storage', 'newsoms': 'newsies', 'tapal': 'papal', 'bethnal': 'lethal', 'vegged': 'begged', 'get30': 'get', 'eataliano': 'italiano', 'flappin': 'flapping', 'moolee': 'fooled', 'sofi': 'soft', 'glitched': 'glitches', 'nn': 'in', 'saboo': 'taboo', '87rt': 'part', 'ohioans': 'onions', 'fluish': 'flush', 'dene': 'done', 'eggsla': 'eggs', 'remarketing': 'marketing', 'bkn': 'bin', 'barr': 'bar', 'singalong': 'signaling', 'anitra': 'anita', 'drenthe': 'breathe', 'celeriac': 'cleric', 'marcona': 'marina', 'growcer': 'grocer', 'hume': 'home', 'klar': 'klan', 'commercialmt': 'commercial', 'covaids': 'cowards', 'yamcha': 'yamaha', 'custmer': 'customer', 'lechon': 'lesson', 'resp': 'rest', 'lallabi': 'lullaby', 'mandatorily': 'mandatory', 'indomie': 'income', 'itttt': 'titty', 'jank': 'bank', '6months': 'months', 'rooker': 'hooker', 'grabe': 'grab', 'altgirl': 'altair', 'poggerz': 'jogger', 'importas': 'imports', 'hadd': 'had', 'emba': 'emma', 'werei': 'were', 'colonizer': 'colonize', 'dopson': 'dobson', 'rs40': 'rse', 'sleepingwe': 'sleeping', 'helotes': 'heroes', 'natura': 'natural', 'airbnbs': 'airbags', 'reaarange': 'rearrange', 'kremes': 'trees', 'transp': 'trans', 'ryde': 'ride', 'wptv': 'with', '2c': 'ac', 'jamuna': 'sauna', 'harressing': 'harnessing', 'vikex': 'like', 'ocs': 'cos', 'mts': 'its', 'latism': 'latest', 'excutive': 'executive', 'locgov': 'logo', 'nonno': 'nonny', 'vmas': 'mas', 'save50': 'save', 'ps61': 'psi', 'juneau': 'bureau', '20b': 'job', 'curren': 'current', 'missles': 'misses', 'doombar': 'goombah', 'tonshop': 'nonstop', 'marana': 'marina', 'aiight': 'alight', 'htown': 'town', 'truckee': 'trucker', 'marshallese': "marshall's", 'lillet': 'fillet', 'bestof': 'bestow', 'geldard': 'gerard', 'tsk': 'ask', 'hermitude': 'servitude', 'vaxx': 'van', 'tje': 'the', 'loveif': 'loveit', 'mofos': 'moos', 'nielson': 'nelson', 'impt': 'imp', 'northwood': 'northward', 'grcoery': 'grocery', 'browny': 'brown', 'cambie': 'cabbie', 'smugtry': 'sultry', 'areyvut': 'areyou', 'diabetiic': 'diabetic', '1mj': 'my', 'alro': 'also', 'yhoo': 'hoo', 'gerty': 'gerry', '1nok': 'nok', 'onow': 'know', 'dromes': 'drones', 'wimborne': 'airborne', 'mpotsa': 'spots', 'boojie': 'boogie', 'sfn': 'son', 'tsakane': 'tsukande', 'varsha': 'harsh', 'managedit': 'managed', 'senpex': 'sense', 'dumy': 'duty', 'winkler': 'winkle', 'khatra': 'katra', 'taringa': 'taking', 'o1x': 'ox', '$trend': 'trend', 'primeday': 'primed', 'ad12': 'add', 'rieger': 'ringer', 'paidl': 'paid', 'goc': 'got', 'kuzo': 'kumo', 'augusto': 'august', 'lifegoals': 'lifeboats', 'texturally': 'textural', 'ng': 'no', 'dependedon': 'depended', 'bethuna': 'bethany', 'wowe': 'owe', 'firefigh': 'firefight', 'custumer': 'customer', 'a1a': 'aka', 'intentioned': 'intention', 'aslong': 'along', 'ozgrocery': 'grocery', 'punkin': 'punking', 'reemerges': 'reemerge', 'samford': 'sanford', 'r9': 're', 'cana': 'can', 'purchasd': 'purchase', 'bueracratic': 'bureaucratic', 'yousoon': 'monsoon', 'packidge': 'package', 'opebi': 'open', 'tgat': 'that', 'tgod': 'god', 'j2': 'je', 'just4u': 'just', '7with': 'with', 'mangu': 'mango', '20ish': 'wish', 'redeliver': 'deliver', '7b': 'ab', 'awesom': 'awesome', 'farrin': 'darrin', 'irshan': 'orphan', 'erith': 'edith', 'delicatessens': 'delicatessen', 'rela': 'real', 'hellman': 'bellman', 'bralette': 'brunette', 'hellmart': 'hallmark', 'wtv': 'wuv', 'agr': 'air', 'kuro': 'euro', 'oisix': 'six', 'kunya': 'kenya', 'vx': 'ex', 'ktla': 'kola', 'platz': 'plate', 'exhange': 'exchange', 'flipk': 'flip', 'wearetmh': 'heareth', 'gts': 'its', 'gossett': 'grossest', 'grenada': 'grenade', '8$': 'i', 'nga': 'na', 'hypocrits': 'hypocrite', 'coronay': 'coronary', 'scipol': 'school', 'kyiv': 'kid', 'bwahaha': 'hahaha', 'keepin': 'keeping', 'giftbags': 'dirtbags', 'enyways': 'anyways', 'legba': 'legal', 'lmfao': 'lao', 'nothank': 'nothing', 'yehuda': 'yasuda', 'unenforceable': 'enforceable', '2bucks': 'bucks', 'pyeha': 'yeah', 'ademola': 'areola', 'unviable': 'enviable', 'upose': 'pose', 'gado': 'ado', 'psms': 'poms', 'obando': 'band', 'us10': 'us', 'unpleasingly': 'unpleasantly', 'notsold': 'outsold', 'agp': 'ago', 'tarriffs': 'tariffs', 'teleology': 'theology', 'adityaji': 'aditya', 'isolte': 'isolate', 'qt': 'it', 'flockin': 'flocking', 'kingsley': 'kingly', 'sanit': 'saint', 'abdel': 'abel', 'khar': 'har', 'lyssa': 'alyssa', 'hvdsa': 'visa', 'rossiter': 'rooster', 'chage': 'change', 'paeds': 'pads', 'indebt': 'indent', '$plce': 'place', 'meltingly': 'melting', 'proterra': 'progeria', 'anoteera': 'another', 'np': 'no', 'kish': 'wish', 'bunnik': 'bunny', 'babyback': 'payback', 'kazi': 'nazi', 'picaroo': 'picard', 'imli': 'imi', 'burnie': 'bernie', 'saker': 'sake', 'ebidta': 'debita', 'hotay': 'hotly', 'fxsignals': 'signals', 'myclub': 'club', 'swerte': 'swerve', 'throe': 'three', 'sbdelivers': 'delivers', 'fhnow': 'know', 'subrosa': 'sucrose', 'footcare': 'footage', 'unionizing': 'ionizing', 'hoem': 'home', 'wallst': 'walls', 'giirl': 'girl', '$gevo': 'evo', 'cheapos': 'cheapo', 'ovr': 'or', 'fla': 'fly', 'sagex3': 'sage', 'ruto': 'auto', 'mandawee': 'mandate', 'nanowebs': 'nanomeds', 'fau': 'far', 'facetimed': 'facetime', 'karn': 'warn', 'homeware': 'homeward', 'babbitt': 'rabbit', 'leander': 'leader', 'malaysianews': 'malaysians', '0pm': 'up', 'etonian': 'estonian', 'babka': 'baba', 'cbr': 'car', 'rsdp': 'red', 'pubsub': 'pursue', 'snaxx': 'snake', 'ruzich': 'rich', 'reveive': 'receive', 'mranderson': 'anderson', 'nw': 'no', 'halps': 'helps', 'avlgov': 'algo', 'ohlone': 'phone', 'b2b': 'bob', 'affectively': 'effectively', 'piday': 'day', 'adm': 'am', 'pic2': 'pick', 'elearning': 'learning', 'dirhams': 'dreams', 'butuan': 'button', 'tmra': 'tara', 'ahhaha': 'hahaha', 'adele25': 'adele', 'tejas': 'teas', 'kzkdi': 'kiki', '00am': 'am', 'homemom': 'homeroom', 'sarili': 'sail', 'priveleged': 'privileged', 'metto': 'motto', 'cmos': 'cos', 'hutto': 'tutto', 'remai': 'remain', 'recourses': 'recourse', 'fanks': 'fans', 'pti': 'pit', 'a10': 'a', 'pgh': 'ugh', 'pt': 'it', '00m': 'him', 'ngmi': 'nami', 'regardin': 'regarding', 'bullshxt': 'bullshit', 'smft': 'soft', 'uparcel': 'parcel', 'ihaven': 'haven', 'podcaster': 'podcast', 'genrator': 'generator', 'doobage': 'footage', 'thurs': 'thus', 'sahas': 'sagas', 'acl': 'all', 'neill': 'neil', '3times': 'times', 'aocs': 'docs', 'payp': 'pay', 'cowrkers': 'coworkers', 'linlin': 'yinlin', 'committment': 'commitment', 'durags': 'drags', 'repurchase': 'purchase', 'bathong': 'bathing', 'apag': 'apa', 'sportsing': 'sporting', 'nuru': 'guru', 'mself': 'myself', 'approvali': 'approval', 'orderr': 'order', 'agonizes': 'agonies', 'kommissar': 'commissar', 'corderless': 'cordless', 'lyft': 'left', 'shisui': 'shui', 'woolsey': 'wolsey', 'parentals': 'parental', 'gisele': 'aisle', 'machakos': 'machos', 'rv': 're', 'a$sholes': 'assholes', 'funyons': 'funyuns', 'indata': 'data', 'e3': 'e', 'tingrams': 'diagrams', 'parl': 'part', 'sevreal': 'several', 'yll': 'all', 'websi': 'webs', 'lezzoo': 'mezzo', 'foodwars': 'woodward', 'kiwayu': 'kawaru', 'thenew': 'there', '70lbs': 'bulbs', 'tmsg': 'test', 'trusst': 'trust', 'tba': 'tea', 'ily4l': 'ill', 'awf': 'awe', 'stringently': 'stringent', 'peachtea': 'peaches', 'hwp': 'hop', 'upperclass': 'underclass', 'refu': 'ref', 'yoursel': 'yourself', 'kendriya': 'kendra', 'khbrn': 'horn', 'jatein': 'eaten', 'egen': 'even', 'baloc': 'bloc', 'exelon': 'melon', 'halfish': 'hagfish', 'chowk': 'chow', 'sivuma': 'sigma', 'nhcs': 'this', 'prefential': 'preferential', 'myhero': 'hero', 'osd': 'old', 'ps36': 'psi', 'cle': 'clue', 'islavista': 'islamists', 'boy3': 'boy', 'barnabas': 'bananas', 'karol': 'carol', 'makesse': 'makes', 'airhugs': 'airbags', 'ifat': 'fat', 'cssn': 'can', 'worc': 'work', 'iceys': 'ices', 'frito': 'fritz', 'chatbots': 'chariots', 'recorderer': 'recorder', 'mpgs': 'mugs', 'fromnhs': 'fronts', 'lssc': 'last', 'bnd': 'and', 'agaianst': 'against', '88th': 'with', 'hrnews': 'news', 'delf': 'self', 'marchents': 'merchants', 'kod': 'kid', 'inaccessibility': 'accessibility', 'tpe': 'the', 'techc': 'tech', '$rst': 'irst', 'tavua': 'tava', 'warmness': 'wariness', 'checkin': 'checking', 'snotter': 'spotter', 'thela': 'theta', '$ma': 'ma', 'distt': 'visit', 'shinewe': 'chinese', 'fromma': 'froma', 'ayana': 'apana', 'witham': 'witha', 'guestimate': 'guesstimate', 'lond': 'long', 'mncs': 'mics', 'abdoo': 'audio', 'dnd': 'and', 'umer': 'user', 'tupo': 'tuo', 'tix': 'six', 'anniv': 'annie', 'yxecc': 'exec', 'garten': 'garden', 'keluar': 'cellar', 'consid': 'consider', 'lfa': 'la', 'bleww': 'blew', 'opportuniy': 'opportunity', 'naij': 'nail', 'neenah': 'needa', 'fkg': 'fog', 'wesr': 'wear', 'any1': 'any', 'dayy': 'day', 'eateasy': 'eaters', 'ambul': 'ambu', 'barnett': 'barrett', 'tunku': 'tank', 'kinna': 'kinda', 'brandable': 'bankable', 'comorbid': 'morbid', 'norcott': 'boycott', 'mamoyo': 'mambo', '10oz': 'coz', 'tryng': 'trying', 'charmin': 'charming', 'citicbs': 'cities', 'minsider': 'insider', 'level3': 'level', 'djlife': 'life', 'nytimber': 'timber', 'svdp': 'stop', '1rs': 'irs', 'mabigat': 'habitat', 'hyeree': 'here', 'evcharging': 'charging', 'pelamin': 'plain', 'tonightno': 'tonight', 'jimjams': 'imams', 'shedder': 'shudder', 'lachine': 'machine', 'nieghbours': 'neighbours', 'notif': 'notify', 'chaitali': 'chantal', 'resole': 'resolve', 'cannibalizd': 'cannibalize', 'cl': 'al', 'deliver4': 'deliver', 'nocontact': 'contact', 'gorham': 'gotham', 'yyjonathan': 'jonathan', 'rebreath': 'breath', 'truvy': 'truly', 'trofi': 'troi', 'lanham': 'lana', 'overr': 'over', 'csl': 'cal', 'frieght': 'freight', 'itemz': 'item', 'uncg': 'ung', 'askmehow': 'somehow', 'pfas': 'peas', '$m': 'am', 'khk': 'kak', 'wook': 'look', 'sarwar': 'salwar', '1q': 'iq', 'wholey': 'whole', 'pln': 'plan', 'khive': 'hive', 'mci': 'mi', 'wcmh': 'with', 'bahrain': 'brain', 'memeber': 'member', 'kwh': 'we', 'phantasy': 'phantasm', 'breslin': 'resin', 'newo': 'new', 'murrsuit': 'pursuit', 'kfp': 'kip', 'inxink': 'inning', 'frontpage': 'frontage', 'friendo': 'friend', 'kadut': 'kaput', 'dr': 'do', 'slcpd': 'slept', 'kolonial': 'colonial', 'giserman': 'german', 'service1': 'service', 'no2idling': 'noodling', 'vcmcguire': 'mcguire', 'vapin': 'vain', '1st': 'est', 'keeler': 'keeper', 'realfood': 'seafood', 'intaking': 'taking', 'hammersi': 'hammers', 'appcost': 'accost', 'pola': 'pole', '99c': 'doc', 'amtul': 'awful', 'fashoro': 'ashore', 'mrr': 'err', 'haaate': 'hamate', 'dors': 'does', 'vqa': 'via', 'timenow': 'timeyou', 'especi': 'expect', 'maren': 'mare', 'trul': 'true', 'xf': 'of', 'unemp': 'enemy', 'unhoused': 'housed', 'offshoring': 'offspring', 'outremont': 'outermost', 'free2oc': 'freedom', 'asc': 'as', 'ieagles': 'eagles', 'refreshings': 'refreshing', 'authorlife': 'authorize', 'nebext': 'next', 'braaing': 'braking', 'onyaka': 'naka', '$lmt': 'let', 'preg': 'prep', 'taxscam': 'taxicab', 'wuwei': 'wei', '$ttoo': 'too', 'maximo': 'maxim', 'incld': 'uncle', 'reastaurant': 'restaurant', 'tehy': 'they', 'malapit': 'malaria', 'tseleng': 'selena', 'chiliza': 'chili', 'lmt': 'let', 'productively': 'productive', 'depisit': 'deposit', 'ugyurs': 'augurs', 'xfer': 'fer', 'cooki': 'cook', 'bevmo': 'beam', 'looooot': 'lookout', 'hombound': 'compound', '2ppl': 'apply', 'menulog': 'menudo', 'pref': 'prep', 'ehcp': 'help', 'despensas': 'dispenses', 'cabi': 'cab', '$cognet': 'cogent', 'blackhead': 'blockhead', 'societ': 'society', 'certificarte': 'certificate', 'hooper': 'hooker', 'apos': 'apes', 'tunisia': 'tunisian', 'smartbags': 'smartass', 'jugak': 'sugar', 'cominh': 'coming', 'jocom': 'room', 'deligh': 'delight', 'shutt': 'shut', 'c7': 'ca', '$joby': 'job', 'participat': 'participate', 'ck': 'ok', 'prodschool': 'preschool', 'ch1ck': 'check', 'hisband': 'husband', 'zakat': 'aka', 'provincially': 'provincial', 'merchandiser': 'merchandise', 'spiffup': 'spiffy', 'valuin': 'valuing', 'baer': 'bear', 'toolkits': 'toolkit', 'watever': 'whatever', 'caaneo': 'canned', 'spendi': 'spend', 'flatiron': 'elation', '1of2': 'of', 'week]': 'week', 'shula': 'hula', 'karein': 'karen', 'gomora': 'gooru', 'tasleem': 'asleep', 'lemak': 'leak', 'furst': 'first', 'kro2go': 'drongo', 'thisisu': 'thisis', 'podis': 'pods', 'nzenews': 'renews', 'viero': 'vitro', 'rafeeq': 'rafael', 'thruuu': 'thrust', 'agressive': 'aggressive', 'arive': 'drive', 'florescent': 'fluorescent', 'technichal': 'technical', 'mdlf8': 'milf', 'jobsite': 'website', 'ccps': 'cops', '4x': 'ex', 'rewatch': 'rematch', 'dec26': 'deck', 'senai': 'sekai', 'angryto': 'angry', 'jumpi': 'jump', 'hotones': 'stones', 'shaam': 'sham', 'xborderer': 'bordered', 'asalam': 'salam', 'y2k': 'yak', 'essentails': 'essentials', 'mayalin': 'marlin', 'daytona': 'dalton', 'famazon': 'amazon', 'ccm': 'com', '48f': 'of', 'aweh': 'awe', 'hyperpure': 'hypercube', 'abent': 'agent', 'sif': 'if', 'dk': 'do', 'aaaa': 'laaa', 'milkmen': 'milkman', 'hungryy': 'hungry', 'halifax': 'halfa', 'pushi': 'push', 'djssn': 'doesn', 'announ': 'announce', 'disjoint': 'discount', 'ullah': 'allah', 'acostathe': 'apostate', 'redressal': 'redress', 'myourderer': 'murderer', 'itask': 'task', 'supmag': 'summat', 'etnowe': 'ettore', 'reschedules': 'reschedule', 'stips': 'ships', 'approval98': 'approval', 'bosch': 'botch', 'ps97': 'psi', 'cummimgs': 'cummings', 'suncrest': 'unrest', 'agin': 'again', 'customners': 'customers', '$rarx': 'rare', 'trashh': 'trash', 'kaity': 'katy', 'envoirnment': 'environment', 'deviced': 'device', 'bfresh': 'fresh', 'pocs': 'pops', 'doppio': 'oppo', 'calmdown': 'clampdown', 'astoria': 'astrid', 'taskers': 'tankers', 'nmed': 'need', 'numpties': 'empties', 'hubbly': 'humbly', 'neccesity': 'necessity', 'certo': 'herto', 'naye': 'name', 'groceris': 'groceries', 'jhatka': 'whatta', 'mija': 'mia', 'alandur': 'slander', '50minute': 'minute', 'puttur': 'putter', 'rifardh': 'ricardo', 'karwao': 'karwa', 'runn': 'run', 'objectif': 'objectify', 'ivolunteers': 'volunteers', 'hoas': 'has', 'jeeee': 'jewel', 'newk': 'new', 'soci': 'sock', 'cr5': 'cry', 'rqst': 'rest', 'aircond': 'abscond', 'ephrata': 'ephraim', 'prsn': 'run', 'ladyj': 'lady', 'borgess': 'burgess', 'biztech': 'biotech', 'vampira': 'vampire', 'toshiba': 'yoshida', 'untill': 'until', 'primorderial': 'primordial', 'defunded': 'defended', 'aiuto': 'auto', 'temasek': 'tease', 'dlivrd': 'lived', 'sve': 'she', 'panal': 'panel', '2deliver': 'deliver', 'cockface': 'fuckface', 'omegasons': 'megatons', 'accretive': 'secretive', 'locl': 'local', 'emoped': 'eloped', 'mymart': 'smart', 'appwali': 'appeal', 'fello': 'fell', 'fauci': 'fuck', 'belive': 'believe', 'britton': 'written', '4hr': 'ihr', 'silay': 'silly', 'sumpin': 'jumping', 'whiteroses': 'whiterose', 'fw': 'few', 'redfx': 'red', 'niice': 'nice', 'food52': 'food', 'endwell': 'dwell', 'chinta': 'china', 'kelli': 'kelly', 'torbay': 'today', 'zango': 'tango', 'quietnext': 'quietest', 'nuggy': 'buggy', 'baniyas': 'bananas', 'murah': 'mural', 'comfree': 'coffee', 'oddle': 'oddly', 'koch': 'loch', 'gf': 'of', 'wsaz': 'waz', 'gebuinely': 'genuinely', 'supplie': 'supplies', 'doppleganger': 'doppelganger', 'kermits': 'permits', 'wishh': 'wish', 'nwh': 'nah', 'gracery': 'grocery', '25min': 'min', 'mfam': 'mam', 'strawber': 'stranger', 'mpn': 'man', 'coronatime': 'coronation', 'packit': 'packet', 'compny': 'company', 'babybel': 'babble', 'aleeve': 'sleeve', 'neu': 'new', 'widnes': 'wines', 'thelist': 'thesis', 'minimizer': 'minimize', 'mbuyane': 'butane', 'onfleet': 'fleet', 'khusta': 'musta', 'edutaining': 'detaining', 'enriquez': 'enrique', 'disorderered': 'disordered', 'goodchild': 'godchild', 'tlkn': 'take', 'nbfcs': 'narcs', 'heckin': 'checking', 'fishdom': 'wisdom', 'redgroup': 'regroup', 'stoerre': 'store', 'kiag5': 'king', 'bagby': 'baby', '$nby': 'by', 'wingey': 'winged', 'bangi': 'bang', 'rfect': 'rest', 'sggc': 'sign', 'ferrero': 'ferret', 'whalberg': 'whaler', 'oola': 'cola', 'baazar': 'bazaar', 'toxicgf': 'toxic', 'dt': 'it', 'enuf': 'neuf', 'saels': 'sales', 'whanau': 'hana', 'reataurants': 'restaurants', 'jugaad': 'jugged', 'tulipea': 'tulips', 'reen': 'been', 'mblogger': 'blogger', 'mph': 'mah', 'xplain': 'explain', 'clancey': 'clancy', '$jtakeaway': 'takeaway', 'hsu': 'su', 'acurracy': 'accuracy', 'askd': 'ask', 'mbalwa': 'balsa', 'gasley': 'galley', 'kavak': 'kayak', 'lbt': 'let', 'sbfood': 'blood', 'worrell': 'sorrel', 'kealan': 'alan', 'neglige': 'negligee', 'dollarsmo': 'dollars', 'thks': 'this', 'bixi': 'bidi', 'theives': 'thieves', 'breadlines': 'headlines', '10day': 'day', 'eburg': 'burg', 'vamsi': 'vast', 'arh': 'are', 'cudnt': 'cunt', 'p2p': 'pop', 'bajaj': 'baja', 'watchnow': 'watchdog', 'pili': 'pile', 'busi': 'busy', 'solacres': 'solace', 'eld': 'old', 'extrem': 'extreme', 'mannerless': 'masterless', 'skpoli': 'spoil', 'madelyne': 'madeline', 'drawbrige': 'drawbridge', 'thelas': 'tells', 'sakye': 'sake', 'alimentation': 'alienation', 'connex': 'conned', 'bingos': 'bingo', 'saban': 'satan', '2nite': 'unite', '$end': 'end', 'marvy': 'marry', '$ua': 'tua', 'panakes': 'pancakes', 'hindf': 'hind', 'inuit': 'innit', 'dorah': 'dora', 'toria': 'tori', 'usrc': 'usc', 'crostini': 'crossing', 'labiz': 'labia', 'angryit': 'angry', 'eephillips': 'phillips', 'baesler': 'baser', 'a5lt': 'alt', 'kanta': 'santa', 'cosworth': 'bosworth', 'tricities': 'atrocities', '8qr': 'for', 'fleshing': 'flashing', 'farah': 'sarah', 'compies': 'copies', 'nilam': 'nizam', 'gawas': 'galas', 'greatful': 'grateful', 'wsyour': 'isyour', 'braum': 'bram', 'annotate': 'annotated', 'katale': 'fatale', 'kaise': 'raise', 'walgett': 'wallet', 'kila': 'kill', 'stvpid': 'stupid', 'razan': 'raza', 'mtns': 'mans', 'peper': 'paper', 'chungha': 'chung', 'abraxas': 'abraham', '20and': 'and', '69m': 'him', 'mch': 'much', 'lovein': 'lovin', 'rioter': 'rioters', '$fb': 'of', 'lvh': 'lvy', 'kitchenettes': 'kitchenette', 'bhilai': 'chili', 'flyered': 'altered', 'saif': 'said', '40hour': 'hour', 'narrati': 'narrate', 'vcs': 'vics', 'berta': 'beta', 'mico': 'mice', 'unforseen': 'unforeseen', 'mhpd': 'had', 'rankage': 'manage', 'kisasi': 'kiss', 'noita': 'nota', 'cbs5': 'case', 'moreton': 'morton', 'unattempted': 'attempted', 'dambu': 'ambu', 'webtoon': 'weston', 'pil': 'oil', 'arkq': 'ark', '18c': 'doc', 'ecb': 'ebb', 'caretacke': 'caretaker', 'jammin': 'jamming', 'sunridge': 'sunrise', 'mylowes': 'myles', 'fack': 'back', 'themesh': 'themes', 'watchi': 'watch', 'appnts': 'pants', '$bx': 'be', 'pcmc': 'pump', 'shwans': 'swans', 'unorganised': 'unorganized', 'fullstack': 'fullback', 'okudah': 'judah', 'ugv': 'ugh', 'courtice': 'courting', 'capiche': 'cache', 'mkm': 'mom', 'sapp': 'app', 'orate': 'rate', '$muln': 'mule', '16b': 'job', 'yoikes': 'yokes', 'conversationally': 'conversational', 'islandeats': 'islanders', 'gunter': 'hunter', '$vet': 'vet', 'hvny': 'any', 'diverless': 'driverless', '2pmcst': 'upmost', 'kise': 'kiss', 'marseille': 'marseilles', 'fareal': 'areal', '2cd': 'ocd', 'raaz': 'razz', 'touchless': 'touches', 'autogas': 'autos', 'usana': 'sana', '10bis': 'nobis', 'cashmeet': 'cashmere', 'hydroxy': 'hydro', 'ngrr': 'near', 'kcen': 'keen', 'syouria': 'syria', 'patna': 'panna', 'groceie': 'grocery', 'immidiate': 'immediate', 'fairpay': 'fairway', 'nft': 'not', 'esajee': 'saree', 'baler': 'baker', 'hwys': 'whys', 'pindi': 'xindi', 'menas': 'means', 'stimulas': 'stimulus', 'glymr': 'gym', 'gramma': 'gamma', 'advs': 'adds', 'reliefirs': 'reliefs', 'kon': 'on', 'huvaa': 'human', 'olado': 'old', 'prorata': 'prorate', 'surtax': 'sorta', 'maricopa': 'maricon', 'alene': 'alone', 'q11': 'que', 'skipgoo': 'skidoo', 'clarks': 'clark', 'arnot': "arn't", 'tomw': 'tomb', '1er': 'her', 'skk': 'sky', 'boonton': 'boston', 'publix': 'public', 'lickin': 'licking', 'voluntee': 'volunteer', 'shopwrong': 'shopworn', 'heehee': 'heeded', 'foxhill': 'foxhole', 'techto': 'techno', 'trumbull': 'turnbull', 'premie': 'premier', 'bowen': 'bowel', 'bridgend': 'bridged', 'truckin': 'trucking', 'hfhwe': 'ifwe', 'bbq': 'be', 'gelsons': 'reasons', 'promisong': 'promising', 'mtv': 'mav', 'virtrual': 'virtual', 'nantwich': 'sandwich', 'christingle': 'christine', 'brighstone': 'brimstone', 'vrzmb': 'crumb', 'cnet]': 'net', 'belgaum': 'belgium', 'aish': 'wish', 'noticable': 'noticeable', 'chernila': 'hernia', 'ql': 'al', 'smms8': 'smash', '5live': 'live', 'howev': 'howe', 'ocnj': 'on', 'maontained': 'maintained', 'chaulk': 'chalk', 'lenta': 'lent', 'exprnce': 'expense', 'vallet': 'wallet', 'bruin': 'brain', 'dolf': 'golf', 'himbo': 'limbo', 'urbx': 'rub', 'nokia': 'nova', 'kegalle': 'legally', 'ttinfo': 'info', 'hbu': 'hub', 'buhay': 'buy', 'preceptor': 'receptor', 'accha': 'alpha', 'soreen': 'screen', 'indv': 'ind', 'bhaktha': 'bantha', 'irthe': 'inthe', 'kplc': 'kill', 'plu': 'plus', 'bawali': 'banal', 'bvunzai': 'banzai', 'sherships': 'starships', 'northside': 'portside', 'glutenous': 'glutinous', 'entreprenues': 'entrepreneur', 'vgtbl': 'vital', 'kwanza': 'wanna', 'rccg': 'ring', 'getjag': 'geta', 'empj': 'emp', 'idlies': 'idles', 'chroniclife': 'chronicle', 'russi': 'russia', 'emhired': 'hired', 'buhera': 'buyer', 'firsttimer': 'firsttime', 'ifpti': 'ifi', 'tuneups': 'tunes', 'bcs': 'bus', 'avle': 'able', 'sbak': 'soak', 'mh12': 'mhat', 'consiquences': 'consequences', 'cuboid': 'couid', 'bantase': 'fantasy', 'daou': 'darou', 'expectin': 'expecting', 'nlw': 'now', 'himalayas': 'himalayan', 'itam': 'item', 'kakkad': 'wakka', 'nbd': 'nod', 'delavan': 'delvian', 'gvbot': 'got', 'fauchi': 'faucet', 'isoro': 'sort', 'muckups': 'fuckups', 'x0': 'xu', 'bursaries': 'burglaries', 'applauda': 'applaud', 'paylook': 'payload', 'uci': 'uni', 'postma': 'postman', '12aug': 'smaug', 'ch4news': 'chews', 'hamm': 'harm', 'unv': 'un', 'woudln': 'wouldn', 'atherton': 'thereon', 'bq': 'be', 'biggby': 'bigger', 'b31': 'be', 'aalis': 'calls', 'lottie': 'hottie', 'mmf': 'mme', 'freshlly': 'freshly', 'isabela': 'isabella', 'tootin': 'tooting', 'sdbn': 'seen', 'tali': 'talk', 'tastic': 'tactic', 'uping': 'using', 'fooda': 'food', 'somalia': 'somalian', 'lorena': 'lorna', 'draftkings': 'drafting', 'icom': 'com', 'dumfries': 'dummies', 'munggo': 'mango', 'albo': 'also', 'iver': 'over', 'diegans': 'vegans', 'groveland': 'groveling', 'hispi': 'his', 'calgarians': 'bulgarians', 'individaul': 'individual', 'kuz': 'cuz', 'poders': 'powers', 'transgenero': 'transgender', 'sike': 'like', 'lax4u': 'lax', 'usgov': 'uso', 'throughs': 'through', 'delver': 'deliver', 'gambell': 'gamble', 'dicaprio': 'sicario', 'youessential': 'nonessential', 'shuana': 'shauna', 'sunbury': 'sunburn', 'jeni': 'jedi', 'eggos': 'eggs', 'wayer': 'water', 'mcr': 'mar', 'caiden': 'maiden', 'obx': 'box', 'dlar': 'dear', 'wegot': 'begot', 'eski': 'ski', 'okleg': 'oleg', 'happyu': 'happy', 'piri': 'kiri', 'risch': 'rich', 'h2o': 'ho', 'esmond': 'desmond', 'baramati': 'basmati', 'gdc': 'do', 'dnr': 'dna', 'umoba': 'mob', '13cabs': 'cabs', 'leeann': 'leann', 'nitb': 'nit', 'decentralize': 'decentralized', 'vigoroni': 'vigorous', 'nsians': 'asians', 'immunosu': 'immundus', 'reddit]': 'reddit', '$goev': 'goes', 'incidentlayy': 'incidentally', 'sanza': 'santa', 'waant': 'want', 'sotu': 'sou', 'stident': 'student', 'convi': 'convo', 'purshases': 'purchases', 'ssc': 'sec', 'wbl': 'wil', 'naww': 'aww', 'findomuk': 'findout', 'sgv': 'suv', 'opensea': 'opened', 'hopin': 'hoping', 'ecn': 'en', 'eigth': 'eight', 'osf': 'of', 'hd': 'he', 'existe': 'exist', '2hr': 'ihr', 'uitemplates': 'templates', 'baraka': 'barada', 'openpr': 'opener', 'ramadhan': "rada'han", 'computers]': 'computers', 'eatreal': 'lateral', 'timax': 'time', 'mgk': 'mak', 'offca': 'offa', 'yeshua': 'joshua', 'oatcake': 'pancake', '10k': 'ask', 'trivialise': 'trivialize', 'traufler': 'trailer', 'bw': 'be', 'birb': 'bird', 'cmpnies': 'companies', 'kolsch': 'kitsch', 'competiti': 'competition', 'truckily': 'luckily', 'deepmind': 'deeming', 'cs': 'is', 'eme': 'me', 'icgc': 'ice', 'trinis': 'things', 'dikha': 'dish', 'mutal': 'metal', 'impuls': 'impulse', 'dinnedr': 'dinner', 'gratitudeto': 'gratitude', 'yako': 'yak', 'arugil': 'frugal', '2oth': 'both', 'mulaney': 'laney', 'noisettes': 'boisette', 'r1': 're', 'irctc': 'arctic', 'sjempo': 'tempo', 'vacinity': 'vicinity', 'mmr': 'mar', 'despacs': 'despair', 'gbd': 'god', 'huga': 'huge', 'dte': 'die', 'up]': 'up', 'uob': 'job', 'collabs': 'collars', 'cabrini': 'cabin', 'takoradi': 'takoyaki', 'cmbo': 'combo', 'zonal': 'tonal', 'pakets': 'packets', 'erran': 'errand', 'layne': 'lane', 'cldnt': 'clint', 'roda': 'road', 'incompetant': 'incompetent', 'bigbox': 'bimbo', 'meaann': 'mean', 'expedites': 'expedite', 'neckbones': 'backbones', 'ecu': 'eu', 'hubster': 'hunter', 'zoomers': 'boomers', 'whitep': 'white', '50share': 'share', 'katies': 'katie', 'mwc': 'mic', 'walimi': 'alibi', 'healthest': 'healthiest', 'siwggy': 'piggy', 'mycoop': 'scoop', 'emro': 'euro', 'marvella': 'marcella', 'reser': 'refer', '2br': 'for', 'hanzi': 'hani', 'cm12': 'cme', 'maugat': 'caught', 'gela': 'gel', 'b2field': 'field', 'tomor': 'tumor', 'ozzie': 'izzie', '26th': 'with', 'nrf': 'nerf', 'mfers': 'myers', 'smugery': 'surgery', 'inhouse': 'house', 'eyy': 'eye', 'eoc': 'doc', 'sanger': 'danger', 'austrea': 'austria', 'organiq': 'organic', 'bahn': 'barn', 'chisme': 'chime', '3xl': 'axl', 'vemag': 'veal', 'pell': 'tell', 'wfhlife': 'oflife', 'autoship': 'authorship', 'cuaron': 'carbon', '1el': 'el', 'rakhlo': 'rashly', 'qjo': 'quo', 'harrod': 'harrow', 'hbates': 'hates', 'doawnload': 'download', 'baarish': 'parish', 'woopie': 'poopie', 'sombra': 'sombre', 'tweeps': 'sweeps', 'dailogue': 'dialogue', 'stricr': 'strict', 'stressfu': 'stressful', 'qirc': 'girl', 'freshmart': 'freshman', 'scrunchies': 'scrunchie', 'carlow': 'carlos', '59c': 'doc', 'kanban': 'kanan', 'ghb': 'gob', 'latour': 'labour', 'eternalst': 'eternal', 'hoylman': 'hoffman', '4b': 'ab', 'servi': 'serve', 'investinwe': 'investing', '25k': 'ask', 'gelabah': 'geladas', 'ondp': 'ond', 'allegs': 'alleys', 'guaca': 'guava', 'savingup': 'saving', 'stroes': 'stores', 'gousto': 'gusto', 'lete': 'let', 'mckenney': 'mckinley', '60f': 'of', 'daycares': 'daycare', 'ekulu': 'skull', 'dpmo': 'demo', 'wooftown': 'oftown', 'diamon': 'diamond', 'natale': 'natalie', 'wajid': 'walid', 'uschina': 'china', 'howmstever': 'howsoever', 'aake': 'take', '8lbs': 'plus', 'quaranting': 'quarantine', 'dufflet': 'duffle', 'sbn': 'son', 'wargaming': 'warming', 'bsafe': 'safe', 'rtgs': 'rags', 'reqs': 'revs', 'then]': 'then', 'timewars': 'timers', 'divi': 'dive', 'reshopped': 'shopped', 'medicince': 'medicine', 'houstontx': 'houston', 'maaay': 'many', 'pulau': 'play', 'neverbuy': 'neverbe', '47days': 'days', 'wcw': 'wow', 'chicest': 'choicest', 'staywarm': 'stalwart', 'nogmo': 'logo', 'trumpists': 'trumpets', 'dusabled': 'disabled', 'raincoast': 'raincoat', 'nxt': 'not', 'loveleeds': 'loveless', 'wdyt': 'what', 'grocelly': 'grocery', 'frustrationi': 'frustration', 'mngr': 'nor', 'wtae': 'tae', 'wrac20': 'wrack', 'competently': 'competent', 'colborne': 'cologne', 'usally': 'usually', 'barrie': 'barrier', 'pasabuy': 'passably', 'ssr': 'sir', '55lincoln': 'lincoln', 'chronical': 'chronic', 'atyodo': 'todo', 'asaani': 'asian', 'senthil': 'entail', 'possbly': 'possibly', 'bhakri': 'bhai', 'kozmo': 'komo', 'unhelpfully': 'unhelpful', 'commandes': 'commander', 'hygenic': 'hygienic', 'rpm': 'rum', 'teen18': 'teen', 'wbcs': 'was', 'loopnet': 'looped', 'bloomerg': 'bloomers', 'siavash': 'slavish', 'technocally': 'technically', 'bannas': 'bananas', 'hubbytes': "hubby's", 'aobut': 'about', 'unbc': 'unc', 'goodnews': 'goodness', 'pineville': 'neville', 'dccobbs': 'dobbs', 'shopman': 'showman', 'ecommere': 'commerce', 'vl': 'al', 'mineola': 'mineral', 'bozotus': 'boots', 'scalable': 'capable', 'gorderon': 'gordon', 'd4': 'do', 'skrewballs': 'screwballs', 'lovenot': 'loveyou', 'appok': 'apply', 'karyn': 'karen', 'mibutler': 'butler', 'webshop': 'teashop', 'reciveing': 'receiving', 'porfa': 'porta', 'kanpur': 'kaput', 'nilsson': 'wilson', 'broccolis': 'broccoli', 'dgaf': 'deaf', 'lelia': 'leila', 'heartrate': 'hearthat', 'ineptness': 'neatness', 'datalake': 'database', 'altman': 'altar', 'srk': 'ark', 'soria': 'sorta', 'wonderinf': 'wondering', 'drvanta': 'devant', 'raskob': 'jakob', 'hyderabadis': 'hyderabad', 'pixer': 'pier', 'tiss': 'miss', 'bsi': 'si', 'skonie': 'stone', 'petwants': 'peasants', 'cmcg': 'cock', 'ctown': 'town', 'reebok': 'reek', 'belx': 'bell', 'checker60': 'checkers', 'subsc': 'subs', 'ambridge': 'cambridge', '8c': 'ac', 'journee': 'journey', 'ica': 'ice', 'yumm': 'yummy', 'southhams': 'southpaws', 'catra': 'cara', 'mbezi': 'mei', 'forin': 'florin', 'kisumu': 'rizumu', 'kitcken': 'kitchen', 'undramatic': 'dramatic', 'isol': 'idol', 'plano': 'plan', 'keeley': 'keeled', 'unvaxxed': 'untaxed', 'kooda': 'good', '40quid': 'liquid', 'berman': 'german', 'delivrng': 'delivering', 'finserv': 'finger', 'placemats': 'placemat', 'haskell': 'skell', 'cooltech': 'caltech', 'biscottini': 'biscotti', 'ndp': 'nap', 'daman': 'damn', 'ystday': 'stay', 'shopway': 'showy', 'wers': 'were', 'onfile': 'office', 'frankfort': 'frankfurt', 'bpo': 'bro', 'mutsu': 'jutsu', 'ezpass': 'pass', 'lobbtists': 'lobbyists', 'appmade': 'appease', 'amost': 'most', 'eshop': 'shop', 'bheed': 'bleed', 'bcsub': 'club', 'pressur': 'pressure', 'egbeda': 'embed', 'clorox': 'claro', 'impare': 'impure', 'virtural': 'virtual', 'efficiacy': 'efficacy', 'katelyn': 'lately', 'contactas': 'contacts', 'randle': 'handle', 'isfree': 'free', 'fele': 'feel', 'get15': 'get', 'shopinng': 'hoping', 'twimbo': 'wimbo', 'q2': 'qi', 'raila': 'rail', 'iashine': 'machine', 'jaden': 'jade', 'savana': 'savant', 'viplav': 'villa', 'manten': 'mantel', 'invester': 'invested', 'fransico': 'frantic', 'results]': 'results', 'righr': 'right', 'loveta': 'loveth', 'moment]': 'moment', '2litres': 'litres', 'portlanders': 'outlanders', 'forbes]': 'forbes', 'jata': 'data', 'blacksburg': 'blackburn', 'deliveres': 'delivered', 'creditc': 'credit', 'atmost': 'almost', 'shaq': 'shag', 'croix': 'crois', 'rxs': 'res', 'ak47': 'aka', 'carme': 'came', 'kingori': 'signori', 'wasteless': 'tasteless', 'cbsnews': 'cashews', '80p': 'up', 'mohale': 'morale', 'prestie': 'prestige', 'dcurbside': 'curbside', 'csrs': 'cars', 'naver': 'never', 'wtp': 'wop', 'kinsa': 'kinda', 'mohana': 'mohan', 'hv': 'he', 'wtvm': 'warm', 'duanne': 'duane', 'nsw]': 'now', 'zagana': 'banana', 'autonomousve': 'autonomous', 'kommunity': 'community', 'article]': 'article', 'dus': 'us', 'byw': 'by', 'tza': 'tea', 'vake': 'take', 'fwm': 'fam', 'allll': 'alll', 'dowgin': 'down', 'agv': 'ago', 'happend': 'happened', 'wengine': 'engine', 'whippy': 'hippy', 'camt': 'came', 'eow': 'how', 'cdls': 'call', 'crieff': 'chief', 'accp': 'act', '92f': 'of', 'foldonf': 'folding', 'greenway': 'freeway', 'twinsies': 'twinkies', 'michiana': 'michigan', 'moule': 'mouse', 'gamzee': 'game', 'mysql': 'myself', 'chaorce': 'chance', 'imu': 'im', 'glebe': 'globe', 'osmos': 'cosmos', '27lb': 'bulb', 'adl': 'all', 'sendin': 'sending', 'oja': 'ofa', 'nabed': 'named', 'copp': 'cops', 'skagit': 'skit', 'krisp': 'crisp', 'shada': 'shade', '7support': 'support', 'wking': 'king', 'roport': 'report', 'crunchie': 'scrunchie', 'plennya': 'plenty', 'mpc': 'mic', '$unfi': 'unfit', 'umrah': 'sarah', 'mackin': 'macking', 'paypigs': "papi's", '$drip': 'drip', 'inarguably': 'arguably', 'abinav': 'alina', 'contini': 'continue', 'terres': 'torres', 'mcds': 'meds', 'autho': 'author', 'snackish': 'snacks', 'coronama': 'coronary', 'arundel': 'asunder', '3california': 'california', 'plaform': 'platform', 'aggress': 'address', 'fypshi': 'yoshi', 'bhi': 'hi', 'butugh': 'butch', 'daiya': 'day', 'cvd': 'cod', 'ms4': 'is', '70k': 'ask', 'carnbee': 'carnage', 'sorad': 'sora', 'evoos': 'evos', 'targer': 'target', 'holiday20': 'holiday', '7tharch': 'starch', 'freshh': 'fresh', 'musiccity': 'musicality', 'yonks': 'monks', 'lipase': 'lips', 'gella': 'fella', 'cushwa': 'cushy', 'specia': 'special', 'shuki': 'shui', 'wose': 'lose', 'hashtags': 'hashtag', 'nonseasonal': 'unseasonal', 'galsshopper': 'grasshopper', 'honeycombs': 'honeycomb', 'fuxkking': 'fucking', 'dwb': 'dab', 'melb': 'melt', 'hatteras': 'matters', 'grabbit': 'rabbit', 'wtheck': 'check', 'jarr': 'jar', 'risau': 'risk', 'utlize': 'utilize', 'dandaro': 'dander', 'assunta': 'assent', 'salsas': 'salsa', 'usf': 'us', 'bogos': 'bogus', 'toughie': 'tough', 'blindswine': 'blindside', 'a11y': 'any', 'nicholls': 'nicholas', 'takaya': 'papaya', 'jhansi': 'jeans', 'thestew': 'these', 'bigrep': 'bigger', 'bdcs': 'back', 'yef': 'yet', '9days': 'days', 'cori': 'core', 'netflixed': 'netflix', 'itmes': 'times', 'getirs': 'gets', 'jlt': 'jet', 'rs58': 'rse', 'soldout': 'holdout', 'biketo': 'liketo', 'buybeer': 'buyer', 'i\\': 'i', 'tlkng': 'thing', '5vare': 'are', 'dvorak': 'dork', '$nakd': 'naked', 'bleedding': 'bleeding', 'mlse': 'else', 'incongruent': 'congruent', 'tylor': 'taylor', '1br': 'for', 'farsan': 'farhan', 'managua': 'managed', 'pennine': 'penning', 'gaute': 'gate', 'skte': 'site', 'victoriapbs': 'victorians', 'tapmart': 'apart', 'skang': 'sang', 'ttv': 'tte', 'didabled': 'disabled', 'mcsweeney': 'sweeney', 'crestline': 'wrestling', 'pmc': 'pic', 'omigosh': 'amigos', 'mixie': 'pixie', 'citizn': 'citizen', 'f2f': 'fof', 'frand': 'grand', 'zimmerman': 'cimmerian', 'copco': 'coco', 'eggulf1': 'engulf', 'tcm': 'tom', 'catpic': 'attic', 'keeo': 'keep', 'noz': 'not', 'aayga': 'saga', 'chec': 'check', 'farmbox': 'firebox', 'prohana': 'propane', '30your': 'your', 'hala': 'half', 'marna': 'maria', 'againa': 'again', 'food2u': 'food', 'nambale': 'gamble', 'dabi': 'dab', 'numpty': 'humpty', 'l6x': 'lex', 'regrann': 'regain', 'melania': 'melanie', 'anotherside': 'otherside', 'shps': 'ships', 'afnt': 'aunt', 'zenik': 'penis', 'jv': 'je', 'zines': 'lines', 'migwo': 'might', 'icema': 'iceman', 'tollgate': 'tolerate', 'jhds': 'this', 'rm5': 'my', 'wnated': 'wanted', 'posti': 'post', 'pk': 'ok', 'changin': 'changing', 'lula': 'hula', 'benef': 'beef', 'shiok': 'shook', 'fuckig': 'fucking', 'stelara': 'stellar', 'boebert': 'bomber', 'ksn': 'kin', 'shap': 'ship', 'gae': 'gave', 'primark': 'primary', 'pengest': 'longest', 'roscoes': 'roscoe', 'tiby': 'tiny', 'latraffic': 'traffic', 'freerides': 'freebies', 'icici': "i'ii", 'jure': 'sure', 'yekkh': 'yeah', 'wfaa': 'faa', 'roaddo': 'road', 'millon': 'million', 'yaz': 'ya', '19dec': 'dec', 'yunus': 'runs', '21c': 'doc', 'jispit': 'visit', 'sz': 'so', 'gianna': 'goanna', 'momsy': 'mommy', 'chronsome': 'chromosome', 'ps30': 'psi', 'zhe': 'the', 'imc': 'im', 'orl': 'or', 'farmlaws': 'farmland', 'braixen': 'brain', '30mile': 'smile', 'cdv': 'civ', 'chandleraz': 'chandler', 'jgor': 'igor', 'stonehouse': 'storehouse', 'expre': 'expire', 'suplies': 'supplies', 'oinkfest': 'infest', 'vend': 'end', 'lorra': 'lorry', 'youknow': "y'know", 'reactnati': 'reactant', 'wetu': 'wet', '10mts': 'emts', 'cashless': 'careless', 'coscos': 'cosmos', 'habituate': 'habituated', 'cockatiel': 'cocktail', 'batche': 'batch', 'navistar': 'savitar', '65yours': 'yours', 'discusse': 'discuss', 'exorbitantly': 'exorbitant', 'chappelles': 'chappelle', 'happyah': 'happy', 'ssri': 'sari', 'ropeable': 'loveable', 'voice]': 'voice', 'longhaul': 'longhand', 'peth': 'path', 'rbg': 'rig', 'jaz': 'jam', 'odyody': 'doody', 'dro': 'do', 'lut': 'but', 'ctleg': 'leg', 'goramble': 'gamble', 'winco': 'winch', 'wbu': 'bu', 'sch': 'such', 'guesse': 'guess', 'makara': 'dakara', 'pttering': 'uttering', 'kmanews': 'manes', 'neverbiden': 'neverbeen', 'newcar': 'near', 'approvala': 'approval', 'urbanag': 'urban', 'fre': 'are', 'astream': 'stream', 'buidl': 'build', 'tapi': 'tape', 'mif': 'if', 'adani': 'dani', 'internati': 'internal', 'onhealth': 'health', 'rva': 'va', 'bttls': 'battle', 'weightloss': 'weightless', 'popmaster': 'postmaster', '24ish': 'wish', '1paper': 'paper', 'funtime': 'untie', 'toyota': 'tooth', 'chud': 'thud', 'roky': 'rocky', 'waitlisted': 'waitlist', 'whearhouse': 'wheelhouse', 'ssdai': 'soda', 'olio': 'polio', 'kanje': 'kanji', 'rven': 'even', 'ncc': 'nce', 'kuddos': 'kudos', 'bellwood': 'bollywood', 'naspers': 'jaspers', 'ezgo': 'ego', 'devs': 'des', 'stonk': 'stone', 'mrm': 'mom', 'magkaroon': 'macaroon', 'juhu': 'juju', 'xg': 'xu', 'accessues': 'accesses', 'wildd': 'wild', 'toojay': 'today', 'imperitive': 'imperative', 'skeeze': 'sneeze', 'timeframes': 'timeframe', 'favuorite': 'favorite', 'pdp': 'pop', '$icoa': 'cocoa', 'ibra': 'bra', 'cbreview': 'review', 'eds': 'ends', 'ghad': 'had', 'stll': 'still', 'settin': 'setting', 'sardis': 'saris', 'ajh': 'ah', 'leonida': 'leonid', '9jan': 'jan', 'modernas': 'modern', 'rqstd': 'rest', 'leadtime': 'bedtime', 'mohalla': 'morally', 'mazimba': 'marimba', 'birdge': 'bridge', 'hawaiin': 'hawaiian', 'fomog': 'food', 'ganyang': 'ganging', 'slyyk': 'slay', 'etspecials': 'specials', 'neart': 'heart', 'wpuld': 'would', 'villamor': 'villager', 'getall': 'tall', 'kavall': 'naval', 'quine': 'quite', 'bbcaq': 'bobcat', 'heathcare': 'healthcare', 'wkrn': 'warn', 'pansari': 'pasar', 'mercari': 'mercury', 'wkg': 'wig', 'northmead': 'northmen', 'evtech': 'tech', 'jsforbes': 'forbes', 'uty': 'duty', 'unmelted': 'melted', 'smarcar': 'smarter', 'passadena': 'pasadena', 'profi': 'profit', 'bchealth': 'health', 'recep': 'recap', 'opado': 'pad', 'mrf': 'mri', 'bamforder': 'camcorder', 'pa5': 'pay', 'winetime': 'lifetime', 'ntbha': 'tha', 'fox5': 'fox', 'asile': 'aside', 'behin': 'behind', 'chillis': 'chills', 'lates': 'late', 'elseland': 'lsland', 'acwri': 'acre', 'eastentials': 'essentials', 'ubx': 'ubi', 'temporari': 'temporary', 'caulflower': 'cauliflower', 'hamlin': 'hauling', 'blamers': 'blames', 'beare': 'bear', 'mayne': 'maybe', 'ofputting': 'putting', 'sodexo': 'rodeo', 'exposu': 'expose', 'shoukd': 'should', 'ex15': 'exit', 'gqp': 'gap', 'emmental': 'mental', 'bayonne': 'bayonet', 'acha': 'ache', 'artois': 'artis', 'bocage': 'became', 'eos': 'cos', 'tw': 'to', 'momble': 'mumble', 'yarmouth': 'badmouth', 'madelene': 'madeline', 'killeen': 'killed', 'deliveredt': 'delivered', 'skrts': 'sorts', 'panpipe': 'bagpipe', 'krwa': 'kurwa', 'tdp': 'top', 'interesti': 'interest', 'duprey': 'prey', 'symptimatic': 'symptomatic', 'abolishice': 'abolishing', 'wearorange': 'rearrange', 'apmc': 'am', 'kproad': 'proud', 'freegan': 'freeman', 'ofidy': 'tidy', 'walaa': 'walla', 'duplicator': 'duplicate', 'mortadela': 'mortadella', 'doreal': 'dorsal', 'ameen': 'amen', 'whitty': 'shitty', 'restricti': 'restrict', 'exn': 'en', 'newy': 'new', 'eventua': 'eventual', 'brannigan': 'rannian', 'adw': 'add', 'partime': 'wartime', 'grabbys': 'grabby', 'eataly': 'early', 'sila': 'silk', 'residentes': 'residents', 'cd4': 'do', '4mail': 'mail', 'fitt': 'fit', 'bonell': 'lonely', 'semidey': 'smiley', 'sharaf': 'sharif', 'caiding': 'aiding', 'pbr': 'per', 'bareilles': 'marseilles', 'wats': 'was', 'clinkies': 'clinics', 'natio': 'nation', 'reenie': 'weenie', 'colback': 'comeback', '33c': 'doc', 'nyp': 'nap', 'rethinkpink': 'rethinking', 'slingtv': 'sling', 'restaurent': 'restaurant', 'streaming]': 'streaming', 'jahmal': 'jamal', 'retailte': 'retail', 'shnuggle': 'snuggle', 'ozharvest': 'harvest', 'pmafc': 'mac', 'lettan': 'letter', 'achillies': 'achilles', 'gerwig': 'gerbil', 'punya': 'puny', '$crsm': 'cram', 'kcnext': 'next', 'urs': 'us', 'bigggg': 'bigger', 'tvseries': 'series', 'mazzle': 'muzzle', 'vap': 'map', 'arabind': 'rabid', 'oget': 'get', 'disgruntle': 'disgruntled', 'kep': 'keep', 'pompeo': 'pompey', '9v': 'iv', 'codementor': 'dementor', 'lifelabs': 'lifeless', 'erum': 'drum', 'dhabas': 'habeas', 'hoopty': 'booty', 'foodni': 'food', 'lapho': 'lap', 'arnside': 'inside', 'southfield': 'outfield', 'lodi': 'lode', 'mgmstrong': 'armstrong', 'mitsuwa': 'itsuka', 'steamies': 'steadies', 'reshape21': 'reshape', '40th': 'with', 'compl': 'compel', 'corderen': 'ordered', 'somthng': 'somthing', 'tadhg': 'tag', 'parcelly': 'parcel', 'inpressed': 'impressed', 'burpees': 'rupees', 'choccie': 'choice', 'waldman': 'walkman', 'scsnews': 'scenes', 'hubley': 'hurley', 'colgate': 'collate', 'graphql': 'graphic', 'hhsc': 'his', 'pulla': 'pull', 'campagn': 'campaign', 'hg3': 'he', 'meesho': 'mesh', 'melati': 'meat', 'panadura': 'pandora', 'brevard': 'bread', 'hologrids': 'holograms', 'redzones': 'redone', 'hild': 'hold', 'ftc': 'etc', 'cavani': 'caving', 'zaka': 'aka', 'venyu': 'venu', 'instcon': 'winston', 'amazo': 'amaze', 'disrupti': 'disrupt', 'powless': 'powers', 'priviliged': 'privileged', 'thackeray': 'thackery', 'mrc': 'mic', 'avaliable': 'available', 'sjv': 'suv', 'tech2': 'tech', 'staks': 'stars', 'sinong': 'strong', 'outweird': 'outward', 'takde': 'take', 'yaya': 'yay', 'eaterla': 'lateral', 'plender': 'plunder', 'jamas': 'james', 'carryout': 'carrot', 'availabilit': 'availability', 'wps': 'was', 'joyrt': 'sort', 'rollups': 'rolls', 'tonkean': 'token', 'spent99': 'spent', 'aayush': 'ambush', 'travellife': 'travelling', 'estabby': 'stabby', 'karna': 'karma', 'bloomber': 'bloomer', 'granada': 'grand', 'fex': 'few', 'tld': 'told', 'simu': 'sim', 'asymptomaric': 'asymptomatic', 'towelly': 'totally', 'pomelo': 'polo', 'sawers': 'sewers', 'bayern': 'barn', 'loveamazing': 'lovemaking', 'fm': 'am', 'parklands': 'parkland', 'harrasment': 'harassment', 'crumbs]': 'crumbs', 'appmts': 'apples', 'icann': 'ican', 'bawang': 'bang', 'tbqh': 'tech', 'sf': 'of', 'autoblog': 'autobot', 'fyp': 'fop', 'pma': 'ma', 'enger': 'enter', 'landsby': 'lands', 'barg': 'bag', 'naigaon': 'saigon', 'elmira': 'lira', 'wsm': 'ism', 'aliz': 'ali', 'warna': 'wanna', 'indust': 'induct', 'uvu': 'vu', 'commuity': 'community', 'eatables': 'eatable', 'lockdownsa': 'lockdown', 'fsbo': 'fobo', 'furthers': 'further', 'nuggs': 'eggs', 'fuuck': 'fuck', 'g11': 'get', 'landi': 'land', '10m': 'him', 'nipawin': 'niacin', 'nighas': 'nights', 'maakt': 'make', 'laurentians': 'laurentian', 'notspon': 'notion', 'wamart': 'smart', 'rtve': 'rave', 'yandi': 'andi', '6weeks': 'weeks', 'obtenga': 'tenga', 'goodbox': 'goodbye', 'usaid': 'said', 'zumri': 'yuri', 'manuever': 'maneuver', 'midnite': 'minute', 'releaf': 'release', 'hornet]': 'hornets', 'cocreativity': 'creativity', 'livacm': 'lilac', 'rocher': 'richer', 'ukama': 'kama', 'berners': 'burners', 'chumly': 'chummy', '4people': 'people', 'flavo': 'flavor', 'idpol': 'idol', 'clemson': 'lesson', 'upscaling': 'scaling', 'yambot': 'bamboo', 'ytown': 'town', 'pikt': 'pit', 'postcodes': 'postcode', 'skokehouse': 'smokehouse', 'lovework': 'homework', 'thr': 'the', 'pitso': 'pits', 'compled': 'complex', 'wtfock': 'stock', 'olloo': 'ollow', 'speakup': 'speak', '42yo': 'yo', 'vension': 'version', 'dayam': 'dalam', 'adsa': 'ads', 'frmr': 'fror', 'footlong': 'footling', 'pps': 'pops', 'x1': 'xu', 'ec2': 'eco', 'vpd': 'vid', 'tklein': 'klein', 'niajax': 'ninja', 'holton': 'hilton', 'stg': 'stag', 'samy': 'say', 'timmoy': 'timmy', 'highline': 'highland', 'seshing': 'meshing', 'mafers': 'makers', 'jacksontn': 'jackson', 'peabod': 'period', '$asps': 'gasps', 'gyud': 'gud', 'boms': 'boys', 'targe': 'large', 'toron': 'torn', 'submissives': 'submissive', 'dmoreno': 'moreno', 'vitek': 'vite', 'serbia': 'serbian', 'dacoco': 'coco', 'wrongdoor': 'wrongdoer', 'shole': 'whole', 'sumac': 'human', 'beleaving': 'leaving', '$mf': 'imf', 'assis': 'assist', 'torr': 'tour', '66b': 'job', 'monday]': 'monday', 'minute5': 'minutes', '22th': 'with', 'mng': 'mug', 'becom': 'become', 'dltr': 'door', 'seside': 'beside', 'saras6': 'sarah', '$free': 'free', 'sanforder': 'sanford', 'grammi': 'grammy', 'etobto': 'toto', 'sfd': 'sad', 'thearchers': 'teachers', 'pokego': 'poker', 'whatcom': 'whatcha', 'plcd': 'pled', 'permal': 'dermal', 'vfpartners': 'partners', 'everidge': 'eldridge', 'happyty': 'happyto', 'a73f': 'auf', 'timestamps': 'timestamp', 'apz': 'ape', 'unhidden': 'hidden', 'shib': 'shit', 'thanksging': 'thanking', 'palaspa': 'palapa', '7services': 'services', 'zlee': 'flee', '05th': 'with', 'rlu': 'flu', 'donetsk': 'donuts', 'simpl': 'simple', 'mccafe': 'cafe', 'zarina': 'marina', 'chawk': 'chalk', 'softwares': 'software', 'flipkat': 'flippant', 'singledad': 'singled', 'maavu': 'mauve', 'carecore': 'hardcore', 'bilioni': 'billion', 'ili': 'ill', '2under2': 'under', 'sejadi': 'sekai', 'waahh': 'watch', 'togle': 'ogle', 'sbtw': 'saw', 'helpout': 'helpful', 'bemuse': 'bemused', 'tangina': 'angina', 'amwec': 'amen', 'eh28': 'eh', 'groceey': 'grocery', '5th': 'ith', 'fofever': 'forever', 'mazak': 'manak', 'neela': 'needa', 'pmo': 'pro', 'undected': 'infected', 'dlozi': 'doze', 'lucky13': 'lucky', 'thejournal': 'thejourney', 'others]': 'others', 'boothe': 'booth', 'bomba': 'bomb', 'evolut': 'evolve', 'inds': 'finds', 'fanart': 'apart', 'libshits': 'dipshits', 'goodrich': 'goodish', 'pymble': 'humble', 'krqe': 'kree', 'kutya': 'outta', 'burpy': 'bury', 'veds': 'feds', 'lft': 'let', 'slootman': 'footman', 'barselo': 'barely', '$inpx': 'jinx', 'ruralvt': 'rural', 'longingi': 'longing', 'namibia': 'namibian', 'wagaira': 'nagara', 'lbs': 'labs', 'lt': 'it', 'beckie': 'become', 'lohri': 'lori', 'kaypoh': 'kapo', 'md]': 'my', 'imfirmed': 'informed', 'immobi': 'immobile', 'geekin': 'geeking', 'rh': 'oh', 'carolpa': 'carol', '20lbs': 'bulbs', 'seiu': 'sei', 'lifeftime': 'lifetime', 'sangh': 'sang', 'winie': 'wine', 'omm': 'mom', 'brodie': 'broke', 'ginsters': 'sisters', 'available]': 'available', 'xs': 'is', 'kerja': 'era', 'selly': 'sell', 'jimin': 'jiminy', 'fritos': 'frits', 'shariffa': 'sheriff', '4ft': 'aft', 'safet': 'safe', 'arriv': 'arrive', 'partnerining': 'partnering', 'omnicron': 'omicron', 'teni': 'ten', 'repairer': 'repaired', 'zaria': 'maria', 'imperfec': 'imperfect', 'grocries': 'groceries', 'philippin': 'philippine', 'cctv': 'cut', 'yeungling': 'youngling', 'bpc': 'bac', 'mimspr': 'miser', 'permanen': 'permanent', 'atrias': 'atrial', 'chaiye': 'chaise', 'marrissa': 'marissa', 'kdmc': 'doc', 'parc': 'part', 'miltown': 'midtown', 'avijit': 'visit', 'medievil': 'medieval', 'unsorted': 'sorted', 'airbonded': 'airbender', 'sawmills': 'sawmill', 'manahan': 'manana', 'lata': 'late', 'eglinton': 'clinton', 'lorem': 'lore', 'overwelming': 'overwhelming', 'daojia': 'dahlia', 'asti': 'anti', 'rosso': 'ross', '20june': 'june', 'idris': 'iris', 'screwin': 'screwing', 'jamat': 'jamal', 'westlands': 'wetlands', 'brno': 'bro', 'arat': 'art', 'iwin': 'win', 'portlandme': 'portland', 'unavailble': 'unavailable', 'retailnews': 'retainers', 'cm': 'am', 'roshan': 'rohan', '1million': 'million', 'lapeer': 'later', 'dcs': 'des', 'yeye': 'eye', 'aerpost': 'airport', 'oreos': 'ores', 'floof': 'floor', '2cm': 'him', 'tharoor': 'harbor', 'witmer': 'wither', 'ards': 'arms', 'grn': 'gun', 'wizkid': 'wicked', 'irmo': 'iro', 'tms': 'tis', 'roemer': 'remer', 'catrriona': 'carrion', 'melrosema': 'melrose', 'equis': 'equip', 'ramai2': 'remain', 'messup': 'mess', 'kefer': 'refer', 'yanga': 'yang', 'airlink': 'airline', 'accomodates': 'accommodates', 'dlia': 'dia', 'towner': 'owner', 'braggy': 'baggy', 'medcines': 'medicines', 'sfo': 'so', 'canwick': 'carsick', 'takkel': 'take', 'cf62': 'cfo', 'appnew': 'apple', 'tesco': 'teach', 'kog': 'dog', 'superfood': 'supercool', 'grampi': 'gramps', 'akeshi': 'apeshit', '6pr': 'for', 'specialgift': 'specialist', 'kipos': 'kilos', 'xiaomi': 'xiaoyi', 'ff6': 'of', 'cryptid': 'cryptic', 'bemine': 'being', 'leisured': 'leisure', 'tensely': 'densely', 'sherene': 'serene', 'thnk': 'think', 'aminnn': 'amino', 'chandan': 'cancan', 'nurser': 'nurse', 'cronch': 'crunch', 'samad': 'salad', 'loveland': 'homeland', 'footprin': 'footprint', 'acquistions': 'acquisitions', 'suhn': 'sun', 'f5': 'fu', 'privatise': 'privatize', 'tiler': 'tiger', 'iba': 'ba', 'renaud': 'renard', 'sje': 'she', 'sparerib': 'spareribs', 'shitton': 'shitty', 'exposere': 'exposure', 'takeour': 'takeout', 'cait': 'wait', 'masdar': 'mandar', 'muertos': 'puerto', 'tossco': 'toss', 'crt': 'cut', 'valero': 'alert', 'calmart': 'calmer', '6am': 'am', 'anchorssa': 'anchors', 'hooglian': 'hooligan', 'roker': 'poker', 'allstate': 'allocate', 'marena': 'arena', 'lla': 'la', 'padron': 'pardon', 'lardons': 'pardons', 'orsered': 'ordered', 'macys': 'mays', 'jff': 'off', 'dadar': 'radar', 'tokey': 'token', 'z28': 'zip', 'thrrefore': 'therefore', 'softskin': 'goatskin', 'ota': 'ofa', 'sebo': 'seo', 'nytimes': 'times', 'jezos': 'jesus', 'moyale': 'morale', 'approvalcc': 'approval', 'strasner': 'strainer', 'am72': 'am', 'kathleenr': 'kathleen', 'kg': 'keg', 'quintal': 'quintals', 'openforce': 'enforce', 'eh5': 'eh', 'dlm': 'dam', 'rggplants': 'eggplants', 'nielsen': 'nelson', 'italia': 'italian', 'mnas': 'mans', 'hobart': 'heart', 'znkhr': 'ankh', 'yeahh': 'yeah', 'grocini': 'growing', 'sunrun': 'outrun', 'dhania': 'mania', 'whitehouse': 'whorehouse', 'atill': 'still', 'fug': 'fun', 'silkwood': 'silkworm', 'love5': 'love', 'bahay': 'away', 'alchol': 'alcohol', 'fasho': 'basho', 'dlv': 'dev', 'sagar': 'sugar', 'lmg': 'leg', 'waqt': 'want', 'etac': 'evac', 'grocry': 'grocery', 'krave': 'brave', 'pbloggers': 'bloggers', 'tvmysteries': 'mysteries', 'ifsc': 'ifs', 'gunge': 'gung', 'beandip': 'bandit', 'mayours': 'mayors', 'stori': 'story', '2x2': 'ex', 'diffe': 'differ', 'olver': 'over', 'samos': "sam's", 'sela': 'sell', 'hema': 'hem', 'platformer': 'platform', '1farmers': 'farmers', 'manora': 'manor', 'nickles': 'pickles', 'defra': 'debra', 'landover': 'handover', '10x': 'six', 'sringr': 'bring', 'nissan': 'pissant', 'csp': 'cop', 'bydrone': 'drone', 'hermiting': 'permitting', 'minigame': 'mitigate', 'trie': 'tried', 'vtc': 'vic', 'onmarketing': 'marketing', 'plutocracy': 'autocracy', 'magsabi': 'wasabi', 'kpis': 'kris', 'holladay': 'holiday', 'zui': 'qui', '30mi': 'mi', 'centos': 'cents', '1km': 'him', 'someons': 'someone', 'yourselves': 'ourselves', 'oann': 'ann', 'use5': 'used', 'zappa': 'kappa', 'uniprot': 'unibrow', 'evn': 'even', 'fresh20': 'fresh', 'flsirs': 'flies', 'zile': 'file', 'crirically': 'critically', '15days': 'days', 'msnnow': 'minnow', 'breyer': 'brewer', 'antigua': 'antique', 'priven': 'driven', 'frikin': 'frickin', 'reade': 'ready', 'porg': 'port', 'lymington': 'remington', 'lefrank': 'frank', 'trigg': 'trig', 'terrys': 'terry', 'purego': 'pure', 'thisis29': 'thisis', 'agm': 'am', 'twirm': 'twirl', 'partneri': 'partner', 'wls': 'was', 'mellisa': 'mellish', 'grange': 'range', 'taobao': 'taboo', 'karnal': 'carnal', 'bando': 'band', 'giv4': 'give', 'confucius': 'confucian', 'wristwatches': 'wristwatch', 'marsha': 'marshal', 'motortrade': 'motorcade', 'autons': 'futons', 'wlg': 'wig', 'proveen': 'proven', 'awardees': 'awarded', 'amazob': 'amazon', 'macca': 'mecca', 'moschino': 'mooching', 'jsh': 'ash', 'tommorow': 'tomorow', 'flatbreads': 'flatbread', 'tobin': 'toxin', 'snaggle': 'snuggle', 'seedrs': 'seeds', 'elia': 'ella', 'inman': 'incan', 'sh1t': 'shut', 'tessie': 'jessie', 'adelai': 'delay', 'bsf': 'bef', 'gettign': 'getting', 'kztv': 'kit', '$pzza': 'pizza', 'nodapl': 'nodal', 'kumail': 'mail', '99th': 'with', 'tunnocks': 'unlocks', 'jouns': 'joins', 'lagos': 'logos', 'lucknow': 'lknow', 'frnd': 'find', 'msee': 'see', 'sfdc': 'side', 'therapyeh': 'therapy', 'meru': 'mere', 'heyward': 'heard', 'lactaid': 'lactic', 'lapile': 'pile', 'frazier': 'crazier', '3km': 'him', 'cktz': 'cut', 'ensur': 'ensure', 'kalher': 'father', 'pirogies': 'pierogies', '34wk': 'hawk', 'garecha': 'garcia', 'digiliant': 'vigilant', 'clermont': 'clement', 'ube': 'be', 'misael': 'mikael', 'discussin': 'discussing', 'keurig': 'keri', 'fajr': 'far', 'ece': 'eye', 'mwa': 'ma', 'wusa9': 'susan', 'rtclient': 'client', 'forder': 'order', 'doxxed': 'doomed', 'emad': 'mad', 'menabytes': 'megabytes', 'dickhaus': 'dickheads', 'sorrytory': 'sorryfor', 'tiff18': 'tiff', 'unavoided': 'avoided', 'hvor': 'vor', 'benetan': 'beneath', 'equina': 'equine', 'donavon': 'donation', 'availbility': 'availability', 'anxthen': 'andthen', 'pengar': 'beggar', 'prodcts': 'products', 'diabeti': 'diabetic', 'gifs': 'gift', 'technion': 'techno', 'sovereingty': 'sovereignty', 'resecheduled': 'rescheduled', 'reciepts': 'receipts', 'safdar': 'sardar', 'inwood': 'wood', 'communitea': 'community', 'utrecht': 'strech', 'doodash': 'doodah', '10ish': 'wish', 'seawooda': 'seafood', 'retailnew': 'retained', 'pythian': 'python', 'busine': 'busing', 'grosa': 'gross', 'crownc': 'crown', 'buynow': 'burrow', 'hafiz': 'hair', '16th': 'with', 'autonomus': 'autonomous', 'bringi': 'bring', 'adver': 'advert', 'fle': 'fly', 'rfp': 'rip', 'biddle': 'middle', 'technie': 'techie', 'wifr': 'wife', 'psfk': 'ask', 'adsum': 'assume', 'kot': 'not', 'sadler': 'sadder', 'homicron': 'omicron', 'garlitos': 'carlito', 'vermoth': 'vermouth', 'parton': 'pardon', 'darkplaces': 'workplaces', 'delivin': 'deliver', 'dayao': 'day', 'boohoo': 'voodoo', 'caister': 'canister', 'nanshee': 'banshee', 'sitto': 'itto', 'abyssmal': 'abysmal', 'tostrengthen': 'strengthen', 'teslas': 'tesla', 'saww': 'saw', 'bse': 'be', 'kanal': 'canal', 'ponmo': 'ponyo', 'liviri': 'living', 'snipper': 'sniper', 'ayour': 'your', 'thankgod': 'thanked', 'dgrqk': 'dark', 'uski': 'ski', 'whalesub': 'whales', 'kambucha': 'kombucha', 'wget': 'get', 'yeaah': 'yeah', 'businessnew': 'businesses', 'berlino': 'berlin', 'uganda': 'ganda', 'ktown': 'known', 'ambien': 'ambient', 'dls': 'des', 'transpor': 'transport', 'karissa': 'marissa', 'buforder': 'border', 'webstore': 'restore', 'ehn': 'en', 'quintin': 'quentin', 'longread': 'longed', 'distributive': 'distribution', 'chorlton': 'charlton', 'gardein': 'garden', 'inorderinate': 'inordinate', 'juug': 'jug', 'becasue': 'because', 'ngoku': 'nook', 'stco': 'sto', 'slas': 'slap', 'cxinsights': 'insights', 'apsley': 'apply', 'ahuja': 'aura', 'aataa': 'data', 'thnx': 'thanx', 'hardcopy': 'hardcore', 'mfo': 'mo', 'gng': 'gang', 'vayal': 'vaya', 'pulsee': 'pulse', 'subarus': 'subaru', 'reccomending': 'recommending', 'onemoney': 'anemone', 'anectodote': 'anecdote', '35b': 'job', 'hnt': 'hit', 'crackberry': 'cranberry', 'ivend': 'svend', 'tevin': 'kevin', 'iph': 'ich', '10the': 'the', 'yoooo': 'soooo', 'leticia': 'felicia', 'samay': 'sammy', 'captiva': 'captive', '$yndx': 'lynx', 'swtor': 'stor', 'shppy': 'sappy', 'polokwa': 'polka', 'myoura': 'your', 'vaxing': 'waxing', 'cnntech': 'caltech', 'nv': 'no', 'baig': 'big', 'flexi': 'flex', 'evry1': 'very', 'brien': 'brief', 'moncler': 'monster', 'ver6': 'very', 'razer': 'razor', 'yonge': 'yong', 'kamil': 'tamil', 'banchan': 'banyan', 'stomper': 'stomped', 'vpp': 'app', 'lstwitter': 'twitter', 'gcal': 'goal', 'disrn': 'dirt', 'zpcs': 'ups', 'seville': 'neville', 'grk': 'ark', 'nio': 'no', 'fasholy': 'rashly', 'newwest': 'newest', 'haller': 'taller', 'lumsden': 'lumen', '1fetch': 'fetch', 'pudo': 'judo', '3weeks': 'weeks', 'fancisco': 'francisco', 'mbvo': 'move', 'sahd': 'said', 'ogd': 'old', '$psix': 'six', 'delux': 'deluxe', 'hoverboard2': 'hoverboard', 'tpt': 'tit', 'swpl': 'sell', 'dcop': 'drop', 'checkpost': 'checkpoint', 'anjel': 'angel', 'everever': 'wherever', 'waqas': 'was', '52am': 'am', 'swcaa': 'swear', 'sector47': 'sector', 'flapol': 'flavor', 'protractor': 'protector', 'tantilized': 'tantalized', 'uniport': 'uniform', 'doola': 'boola', 'neuman': 'newman', 'dain': 'damn', 'manipur': 'manicure', 'difft': 'diff', '45min': 'min', 'wk': 'we', '20buttons': 'buttons', 'bhushan': 'bushman', 'kashif': 'bashir', 'scandie': 'candle', 'jamy': 'jam', 'reauthorized': 'authorized', 'guarant': 'guaranty', 'anmer': 'anger', 'reasor': 'reason', 'airboats': 'airboat', 'zombs': 'bombs', 'nlfood': 'blood', 'ekdum': 'odum', 'csuite': 'suite', 'tashify': 'testify', 'speedread': 'speeded', 'selectable': 'delectable', 'worrh': 'worry', 'saundrea': 'saunders', 'hairul': 'hair', 'ndx': 'nix', 'kalwa': 'alwa', 'anjuman': 'inhuman', '$clsk': 'cask', 'techra': 'tech', 'relnews': 'renews', 'ballman': 'bellman', 'nagraj': 'nara', 'liffts': 'lifts', 'dyour': 'your', 'loui': 'loud', 'shouldi': 'should', 'maaa': 'mama', 'tcnn': 'can', 'myesh': 'mesh', '16mins': 'mins', 'nixcraft': 'aircraft', 'grocersies': 'groceries', 'finee': 'fine', 'hilside': 'hillside', 'pureplay': 'purely', 'qub': 'rub', 'ppal': 'pal', 'maponya': 'madonna', 'bwtakecare': 'takecare', 'musial': 'musical', 'resturaunt': 'restaurant', 'imore': 'more', 'enti': 'anti', 'munsey': 'money', '$orcl': 'force', 'cambourne': 'tambourine', 'outschool': 'outshoot', 'udinese': 'chinese', 'paska': 'pasta', 'useconomy': 'economy', 'arendse': 'amends', 'cei': 'chi', 'acertained': 'ascertained', 'uncomf': 'income', 'soz': 'so', '6month': 'month', '66th': 'with', 'mangement': 'management', '\\3': 'i', '40c': 'doc', 'waisting': 'waiting', 'cheerie': 'cheerio', 'lgbt': 'get', '60th': 'with', 'assholeish': 'assholes', 'steffi': 'stuff', 'aldies': 'ladies', 'schitts': 'shitty', 'tamebay': 'nameday', '11aug': 'smaug', 'pcd': 'pod', '4my': 'my', 'exercieses': 'exercises', 'jetson': 'person', 'wanta': 'want', 'apc': 'ape', 'malabar': 'salazar', 'carrington': 'arlington', 'retutned': 'returned', 'danna': 'wanna', 'yday': 'day', 'lokhu': 'lakhs', 'aligarh': 'oligarch', 'aeroplan': 'aeroplane', 'gomart': 'smart', 'hausa': 'hasa', 'maximom': 'maximum', '3cubed': 'cubed', 'gymea': 'game', 'secmash': 'smash', 'petani': 'retain', 'shoppage': 'stoppage', 'sssoo': 'shoo', 'excl': 'excel', 'uneconomic': 'economic', 'wikimedia': 'wikipedia', 'baklawa': 'baklava', '3maura': 'maura', 'mex': 'me', 'ricci': 'rich', 'teqzest': 'request', 'nayi': 'nazi', 'arl': 'are', 'malus': 'males', 'punecall': 'phonecall', 'goatmeat': 'oatmeal', 'fanfic': 'panic', 'mello': 'hello', '7bn': 'ibn', 'averagely': 'average', 'ulundi': 'lundy', 'grcery': 'grocery', 'prueba': 'rumba', 'dfps': 'dips', 'wkyt': 'what', 'smood': 'stood', 'nbpoli': 'poli', 'helly': 'hell', 'wisk': 'wish', 'distrib': 'disturb', '10may': 'may', 'cldn': 'clan', 'telyport': 'teleport', 'ikeja': 'ikea', 'geetika': 'geeta', 'ener': 'ever', 'withown': 'without', 'mykuya': 'yuya', 'adjacency': 'adjacent', 'utk': 'ut', 'tralee': 'travel', 'includ': 'include', 'siloam': 'slam', 'nisa': 'visa', '96th': 'with', 'smarta': 'smart', 'idai': 'dai', 'hkw': 'how', 'tomight': 'tonight', 'disenfected': 'disinfected', 'sva': 'sea', 'coa': 'cop', 'dris': 'drip', 'legalizeit': 'legalize', 'kamothe': 'anothe', 'mudase': 'madame', 'caremonger': 'warmonger', 'mboya': 'moya', 'r95s': 'runs', 'cheri': 'cher', 'essentialser': 'essentials', 'normatively': 'normative', 'wiild': 'wild', 'automaker': 'automakers', 'coeliac': 'celiac', 'unlivable': 'unlikable', '02nd': 'and', 'annviersary': 'anniversary', 'sjgds': 'signs', 'proav': 'prove', 'backe': 'back', 'getorg': 'georg', 'awi': 'aii', 'trukey': 'turkey', 'splutter': 'splatter', 'cubbyholed': 'cubbyhole', 'reiterates': 'reiterate', 'futu': 'tutu', 'refrigerat': 'refrigerate', 'bonners': 'banners', 'xoby': 'toby', 'marginalise': 'marginalize', 'gegen': 'geen', 'nov': 'not', 'unint': 'unit', 'd2d': 'did', 'garening': 'gardening', 'alviere': 'algiers', 'tradies': 'trades', 'debasio': 'debased', 'link]': 'link', 'greyson': 'grayson', 'qfc': 'of', 'arpu': 'aru', '6mph': 'amp', 'r70': 'run', 'mndf': 'and', 'naa': 'na', 'bhaaii': 'bhai', '5kms': 'arms', 'gagan': 'pagan', 'yxe': 'ye', '12min': 'min', 'movemen': 'movement', 'mefap': 'mean', 'symoe': 'some', 'ibd': "i'd", 'outter': 'outer', 'simo': 'limo', '4oz': 'coz', 'rmb': 'rob', 'hurrica': 'hurricane', 'leseng': 'losing', 'willnt': 'willnot', 'sensous': 'sensors', 'pamps': 'camps', 'omel': 'omen', 'scania': 'scan', 'justs': 'just', 'changling': 'changing', 'jubliant': 'jubilant', 'commerc': 'commerce', 'zuper': 'super', 'phd': 'pod', 'nongoer': 'longer', 'trupik': 'truck', 'e4': 'e', 'khi': 'hi', 'boleh': 'both', 'lbr': 'lor', 'pimo': 'pimp', 'poenta': 'polenta', 'neuer': 'never', 'retrail': 'retail', 'wtfs': 'was', 'ps92': 'psi', 'trudo': 'trudy', 'pdf': 'edf', 'outbrea': 'outbreak', 'carnell': 'cornell', 'kiehle': 'kindle', '67minutes': 'minutes', 'reshelve': 'resolve', 'mannnn': 'manner', 'aaall': 'atall', 'mesg': 'mess', 'dhal': 'deal', 'palava': 'palapa', 'cyclades': 'cycles', 'oldham': 'olam', 'ny50': 'ny', 'carefu': 'careful', 'gavina': 'gavin', 'e1': 'e', 'answ': 'anew', 'anzisha': 'aisha', 'gtline': 'line', 'thikard': 'third', 'elo': 'ego', 'jaysin': 'jason', 'uperer': 'upper', 'sipp': 'sip', 'psa': 'spa', 'psyop': 'stop', 'ranka': 'rank', 'minimises': 'minimise', 'salu': 'salt', 'faughter': 'daughter', '54f': 'of', 'lig': 'big', '$gfoo': 'goo', 'voicey': 'voice', 'garlandusa': 'garlands', 'petworth': 'wentworth', 'wifelife': 'wildlife', 'metz': 'met', 'rhs': 'res', 'kundan': 'kinda', 'kaliisa': 'kalinda', 'harga': 'hara', 'orisa': 'oris', 'ontheway': 'theway', 'apauling': 'hauling', 'overshoe': 'overshot', 'poignantly': 'poignant', 'manfriend': 'manfred', 'myina': 'mina', 'jmo': 'mo', 'hojati': 'ojai', 'midvale': 'middle', 'carlin': 'marlin', 'layou': 'layout', 'tafe': 'take', 'nobueno': 'bueno', 'depts': 'depth', 'perogies': 'pierogies', 'quickse': 'quickie', 'faders': 'fades', 'evoluti': 'evolution', 'schertz': 'schmutz', 'yo85': 'you', '$ozon': 'soon', 'defs': 'defy', 'gibler': 'gibber', 'kherson': 'person', 'kcbae': 'kebab', '25am': 'am', 'abes': 'ages', 'douro': 'dour', 'sidcup': 'hiccup', 'fnm': 'fam', 'lowfat': 'lowest', 'mgulkd': 'mould', '$agc': 'ago', 'utilisation': 'utilization', 'webt': 'went', 'perwyn': 'person', 'mbga': 'mega', 'joppaa': 'oppa', 'eceryone': 'everyone', 'utlwa': 'outlaw', '7kg': 'ekg', 'vulnerabl': 'vulnerable', 'kyc': 'kya', 'stevenson': 'stevens', 'uncrease': 'increase', 'unocha': 'mocha', 'egso': 'ego', 'xcover': 'cover', 'beiscoming': 'becoming', 'momof3': 'momo', 'affordera': 'afforded', 'drovre': 'drove', 'ols': 'old', 'dhaniya': 'hanya', 'redefinition': 'definition', 'biebers': 'bieber', 'localsmes': 'locales', 'pancit': 'pandit', 'guyug': 'guys', 'krantz': 'grant', '3mf': 'imf', 'backla': 'backl', 'beshak': 'beak', 'reliantcu': 'reliant', '4that': 'that', 'trailblazer': 'trailblazers', 'senio': 'senior', 'knoq': 'know', 'brudi': 'rudi', 'krita': 'rita', 'cmpny': 'many', 'fuckbag': 'fucking', 'oloye': 'love', 'woburn': 'burn', 'carebridge': 'cartridge', 'signaturewas': 'signatures', 'nebraskan': 'nebraska', 'boholshop': 'bookshop', 'watchung': 'watching', 'prople': 'people', '11mo': 'limo', 'ambler': 'gambler', 'spartanash': 'spartans', 'bookmarking': 'bookmaking', 'agasin': 'again', 'kisrah': 'kira', 'cdsl': 'call', 'nickeled': 'nickel', 'pretip': 'pretty', 'atcos': 'tacos', 'wealt': 'dealt', 'buffa': 'buff', 'ayy': 'any', 'ducey': 'dicey', 'thokar': 'thor', 'hbb': 'hub', 'futher': 'father', 'mataro': 'mature', 'trailblaze': 'trailblazers', 'atrans': 'trans', 'applaudit': 'applaud', 'bunche': 'bunch', 'alredy': 'already', 'barnet': 'barnes', 'hrhour': 'hour', 'staci': 'stack', 'overbuy': 'overly', 'youwont': 'youwant', 'service]': 'service', 'bignews': 'bigness', 'kop': 'top', 'bs': 'is', 'latam': 'late', 'creselly': 'cruelly', 'normalise': 'normalize', 'kannan': 'kanan', 'bagboy': 'baby', 'pplease': 'please', 'steller': 'teller', 'kapa': 'papa', 'vfw': 'vow', 'lazyaf': 'lazy', 'deltav': 'delta', 'getty': 'petty', 'ipd': "i'd", 'offstreet': 'offscreen', 'cestoni': 'centons', 'maritimes': 'maritime', 'const': 'cost', 'exce': 'excel', 'trendhunt': 'trenchant', 'mybay': 'myway', 'fullfill': 'fulfill', 'mosy': 'most', 'tipthem': 'tiptoe', 'ringingly': 'ringing', 'mizzy': 'dizzy', 'sbeautiful': 'beautiful', 'erlema': 'elena', '12months': 'months', 'hlen': 'hen', 'autologous': 'autonomous', 'deamed': 'dreamed', 'poblacion': 'oblation', 'seeder': 'seeker', 'domesticsub': 'domestics', 'luxory': 'luxury', 'cda': 'da', 'techhouse': 'teahouse', 'csart': 'cart', 'feelies': 'seelies', 'pcps': 'pops', 'surchar': 'surcharge', 'codemotion': 'commotion', 'stockers': 'stickers', 'loosing': 'looking', 'gmb': 'gob', 'vfd': 'vid', 'elroy': 'leroy', 'riight': 'right', '2mm': 'umm', 'uwm': 'um', 'cdw': 'cow', 'coteda': 'voted', 'paisola': 'payola', 'variou': 'various', 'sanebox': 'sandbox', 'nostress': 'stress', 'cny': 'any', '$hit': 'hit', 'commoditised': 'commodities', 'mohamad': 'mohammad', 'srs': 'sis', 'fermenti': 'ferment', 'chioptle': 'chipotle', 'ankeny': 'antony', 'joini': 'join', 'rochmn': 'rohan', 'bhopal': 'choral', 'crazy$': 'crazy', 'pasqual': 'casual', 'kamryn': 'karen', '46items': 'items', 'sheltie': 'shelter', 'neva': 'eva', 'oxon': 'oxen', 'snapseed': 'snapped', 'scotian': 'scotland', 'kail': 'jail', 'chasney': 'chaney', 'zoba': 'soba', 'csuinc': 'suing', 'defi': 'defy', 'tulsan': 'tulsa', 'lv': 'la', 'marrian': 'martian', 'atp': 'at', 'rrr': 'err', 'ruidoso': 'guidos', 'headpat': 'headway', 'gropper': 'grouper', 'nps': 'ups', 'upcharges': 'charges', 'usua': 'usual', 'urdu': 'uru', 'tentang': 'tenting', '4imprint': 'imprint', 'stgeorge': 'george', 'prepear': 'prefer', 'jwfoods': 'woods', 'dedusctible': 'deductible', 'birdbox': 'firebox', 'langhorne': 'longhorn', 'unfrosted': 'frosted', 'bsv': 'bev', 'madero': 'made', 'kitchn': 'kitchen', 'tempos': 'tempo', 'ppt': 'put', 'boostrap': 'bootstrap', 'kainis': 'kinds', 'jhf': 'the', 'kmc': 'my', 'issit': "isn't", 'hornbeam': 'moonbeam', 'chokey': 'choke', 'ckhalil': 'khalid', 'manoj': 'manor', 'saucedo': 'sauced', 'agenci': 'agency', 'payouroll': 'payroll', 'homeline': 'homeland', 'xc': 'xu', 'nhpolitics': 'politics', 'committ': 'commit', 'esistme': 'epistle', 'karoi': 'kari', 'gustado': 'gustavo', 'ifmk': 'if', 'mlb': 'mob', 'cmco': 'coco', 'ciaz': 'cia', 'phillps': 'phillips', 'mezza': 'mezzo', 'saveco': 'save', 'presebt': 'present', 'paschim': 'achim', 'amreading': 'reading', 'curbsite': 'curbside', 'tootle': 'tootles', 'nopants': 'pants', 'olipop': 'lollipop', 'flavered': 'flavored', 'sherbert': 'sherbet', '$mdb': 'mob', 'bezo': 'bozo', 'fucksake': 'fuckface', 'uxdesign': 'design', 'spendin': 'spending', 'irvine': 'irving', 'recharge1': 'recharge', 'wemans': 'womans', 'grocieres': 'groceries', 'aski': 'ask', 'hoti': 'hot', 'gpay': 'pay', 'govnt': 'goint', 'tumm': 'tummy', 'possil': 'fossil', 'manflu': 'manful', 'keenan': 'keen', 'doorgani': 'doorman', '5kg': 'ekg', 'driftless': 'drifters', 'freaken': 'freaked', 'throught': 'thought', 'equitably': 'equitable', 'vernonbc': 'vernon', 'otv': 'ot', 'sotired': 'tired', 'washingtondc': 'washington', 'mcmansion': 'mansion', 'fecta': 'feta', 'covip': 'cover', 'siegel': 'siege', 'unbagged': 'bagged', 'zoney': 'money', 'podsters': 'posters', 'stapl': 'stall', 'ericsson': 'frisson', 'iteam': 'team', '$wba': 'wa', 'ethereum': 'ethereal', 'kubi': 'kuni', 'klms': 'alms', 'rockitap': 'rockstar', 'autononomous': 'autonomous', 'reatil': 'retail', 'pfizers': 'fibers', 'ekl': 'el', 'tambaram': 'tamara', 'splenda': 'spend', 'unionizes': 'unionized', 'hvac': 'evac', 'duhh': 'duh', 'kth': 'ith', 'jrm': 'arm', 'jrs3': 'just', 'csto': 'sto', 'afroze': 'froze', 'dillions': 'millions', '$heps': "he's", 'reylo': 'real', 'rieds': 'rides', 'birra': 'birth', 'k3b3': 'kobe', 'volker': 'older', 'booknerd': 'booked', 'roh': 'oh', 'uct': 'cut', '25million': 'million', 'tgk': 'tak', 'nuckles': 'knuckles', 'excitementi': 'excitement', 'pgn': 'pin', 'take3': 'take', 'quaratine': 'quarantine', 'supermarkers': 'supermarkets', 'mazeed': 'marked', 'dsps': 'dips', 'deseret': 'desert', 'blech': 'bleach', 'uvsage': 'usage', 'quot': 'quit', 'yayo': 'yay', 'fukumura': 'yukimura', 'shia': 'shit', 'cbd': 'cod', 'thel': 'the', 'onevim': 'onetime', 'estero': 'ester', 'tarike': 'trike', 'leecious': 'precious', 'tomtastic': 'bombastic', 'milknmac': 'milkman', 'genx': 'gene', 'wooho': 'wooh', 'innovates': 'innovate', 'delusionally': 'delusional', 'nutritioninv': 'nutritionist', 'rootbeer': 'rooter', 'bipoc': 'bio', 'peosta': 'post', 'indusitries': 'industries', 'venda': 'vena', 'cofounding': 'confounding', 'doncaster': 'lancaster', 'feofast': 'feast', 'experiencia': 'experience', 'soliman': 'solemn', 'drunmettes': 'brunettes', 'harare': 'aware', 'keegs': 'keeps', 'applic': 'apply', 'cota': 'coat', 'upcycles': 'cycles', '28a': 'a', 'convered': 'covered', 'jonkler': 'joker', 'myke': 'make', 'partida': 'parties', 'thorougly': 'thoroughly', 'expierence': 'experience', 'moneyon': 'money', 'innercity': 'intercity', 'mkosi': 'most', 'apping': 'tapping', 'mamak': 'mama', 'wvxu': 'wax', 'nfc': 'nic', 'whther': 'whether', 'yest': 'best', 'accordering': 'according', 'sameday': 'someday', 'base10': 'based', 'spean': 'speak', 'cathe': 'bathe', '9m': 'am', 'selig': 'seeing', 'shehla': 'sheila', 'communtiy': 'community', 'ersary': 'essay', 'eki': 'ski', 'gogadgets': 'gadgets', 'courgette': 'corvette', 'billerica': 'ballerina', 'sunshade': 'sunshine', 'asali': 'asami', 'sayg': 'say', 'cxunplugged': 'unplugged', 'p3': 'pa', 'talktalk': 'backtalk', 'mcgregor': 'gregor', 'omaxe': 'omae', 'saha': 'sara', 'agustd': 'abused', 'hathaway': 'pathway', 'dogos': 'dogs', 'isolation]': 'isolation', '31bn': 'ibn', 'hearten': 'hearted', 'cdk1': 'cake', 'wegoo': 'wagon', 'ozfeed': 'feed', 'noexcuse': 'excuse', 'shandong': 'standing', 'pso2na': 'sonna', 'disorganize': 'disorganized', 'aiw': 'air', 'trave': 'travel', 'schuman': 'human', 'taky': 'take', 'tfs': 'ifs', 'bql': 'bel', 'eskies': 'skies', 'bartok': 'barton', 'inliding': 'including', 'newham': 'wham', 'skittle': 'skittles', 'leavesn': 'leaves', 'cst': 'cut', 'filum': 'film', '$tmo': 'atmo', 'commercia': 'commercial', 'nhii': 'thii', '2yours': 'yours', 'vsu': 'su', 'dsc': 'doc', 'eto': 'to', 'castile': 'castle', 'usi': 'us', 'bartley': 'barley', 'atltech': 'attach', 'axios': 'axis', 'studder': 'shudder', 'idgi': 'digi', 'wallmart': 'hallmark', 'harmattan': 'sarmatian', 'councellor': 'counsellor', 'chari': 'chair', 'recieves': 'receives', 'hwvr': 'have', 'lockable': 'lickable', 'lumpur': 'lump', '11am': 'am', 'nootric': 'nitric', 'bcsd': 'best', 'osos': 'sos', 'thema': 'them', 'mandeville': 'vaudeville', 'conds': 'bonds', 'understanda': 'understand', 'koeman': 'woman', 'adkom': 'doom', 'automatee': 'automated', 'fiiine': 'fine', 'automizing': 'authorizing', 'advertisng': 'advertising', 'ammounts': 'amounts', 'caas': 'cars', 'bcce': 'bocce', 'langon': 'langdon', 'anadai': 'andi', 'occasi': 'occasion', 'tendys': 'tends', 'wray': 'way', 'prosp': 'prop', 'ecocup': 'coup', 'vidcast': 'podcast', 'powerbi': 'power', 'maryann': 'mayans', 'sniffly': 'sniffle', 'kaminfo': 'amino', 'galp': 'gal', 'nodejs': 'nodes', 'tulane': 'plane', 'nocorona': 'corona', 'ba8': 'bad', 'useage': 'usage', 'simce': 'since', 'luquore': 'liquor', 'chanko': 'thank', 'harina': 'marina', 'vanwa': 'vanya', 'grocerie': 'groceries', 'agbaby': 'baby', 'bcom': 'boom', 'yumminess': 'yummiest', 'saniti': 'sanity', 'kiddos': 'kiddo', 'nerfed': 'perfed', 'auburnal': 'auburn', 'pantr': 'pants', 'diffce': 'dice', 'yeaterdsy': 'yesterday', 'petito': 'petit', 'idt': 'it', 'republicday': 'republican', 'justeat': 'justa', 'metropolita': 'metropolitan', 'leftism': 'leftist', 'swanzey': 'swanky', 'locavore': 'locator', 'resos': 'rests', 'cccfa': 'cocoa', 'bramalea': 'bramble', 'buer': 'beer', 'amaro': 'amar', 'wbo': 'who', 'upfrom': 'from', 'gratitudeno': 'gratitude', 'ede': 'eye', 'atemians': 'athenians', 'hyg': 'hug', 'lipa': 'lips', 'pagar': 'pagan', 'etier': 'ether', 'vawbc': 'vac', 'fedelity': 'fidelity', 'wtbf': 'wolf', 'cramazon': 'amazon', 'zillow': 'pillow', 'taowards': 'towards', 'experi': 'expert', 'gigwork': 'legwork', 'cardenas': 'cardinal', 'fked': 'feed', 'wkyc': 'why', 'safarians': 'samaritans', 'ktb': 'kob', 'verrado': 'serrano', 'buzzworder': 'buzzwords', 'segari': 'sugar', 'khan99': 'khan', 'madera': 'madea', 'zaman': 'aman', 'kcmo': 'kumo', 'wxceot': 'except', 'molina': 'polina', '$gme': 'me', 'jumbly': 'humbly', 'mgrs': 'mars', 'repot': 'report', 'groc': 'grow', 'inaws': 'gnaws', 'eloned': 'cloned', 'czusa': 'cause', 'saval': 'naval', 'iceburg': 'iceberg', 'fall19': 'fall', 'anonline': 'online', 'mssgs': 'miss', 'cstat': 'stat', 'paperboard': 'waterboard', 'twixes': 'twines', 'maan': 'man', 'kahin': 'karin', 'richardson': 'richardsons', 'ncbeer': 'beer', 'geog': 'grog', 'liard': 'liar', 'fkng': 'feng', 'icity': 'city', 'samarit': 'smart', 'brocolli': 'broccoli', 'spoonies': 'ponies', 'disabilit': 'disability', 'raeburn': 'return', '$ms': 'ams', 'easterbasket': 'wastebasket', 'etiquettes': 'etiquette', '5a': 'a', 'dougy': 'dough', 'sanam': 'salam', 'barbad': 'barbed', '2litre': 'litre', 'addinf': 'adding', 'champlain': 'chaplain', 'suprise': 'surprise', 'msgs': 'mugs', 'itna': 'ina', 'starkman': 'starman', 'bhool': 'whool', 'enappd': 'snapped', 'luann': 'leann', 'gonchar': 'gotcha', 'sadface': 'surface', 'neelix': 'felix', 'lbvs': 'love', 'brexit': 'credit', 'jagung': 'raging', 'bualumni': 'alumni', 'ukasian': 'asian', 'whow': 'how', 'getsine': 'getting', 'kitsi': 'kits', 'grocerkey': 'grocery', 'dinosaured': 'dinosaur', 'unfriending': 'befriending', 'psu': 'su', 'mbogas': 'bogus', 'burnet': 'burned', 't9n': 'ten', 'wapo': 'capo', 'pongal': 'portal', '18ct': 'fact', 'norca': 'orca', 'electricia': 'electrician', '38c': 'doc', 'maaddox': 'maddox', 'localogy': 'locally', 'retailt': 'retail', 'dred': 'died', 'edsa': 'edna', 'servicesgh': 'services', 'futbol': 'futon', 'mtcomm': 'comm', 'fogy': 'fog', 'amazedi': 'amazed', 'oo0o': 'ooo', 'finsub': 'fansub', 'aggies': 'baggies', 'odering': 'ordering', 'altamira': 'altair', 'valas': 'alas', 'wellcare': 'welfare', 'ogmund': 'mound', 'mendelson': 'henderson', 'breonka': 'brenda', 'kyourie': 'kyrie', '6a': 'a', 'werrick': 'derrick', 'xtra': 'extra', 'tourons': 'turns', 'dinne': 'dinner', 'scifi': 'scif', 'brindle': 'bridle', 'denki': 'genki', 'zucc': 'much', 'ingui': 'input', 'softw': 'soft', '30k': 'ask', 'percen': 'percent', 'eamonn': 'among', 'opk': 'ok', 'yhu': 'you', 'mbo': 'mob', 'herenow': 'hereto', 'myan': 'man', 'notor': 'motor', 'increasy': 'increase', 'tvp': 'top', 'pkt': 'put', 'medline': 'midline', 'catch22': 'catch', 'kamo': 'camo', 'fokkus': 'folks', 'kamm': 'damm', 'tasmay': 'tasty', 'abhi': 'ahi', 'scheels': 'cheers', 'peruzzi': 'pezzi', 'samparn': 'sampan', 'd70': 'do', 'kohei': 'kosher', 'irc': 'arc', 'malton': 'milton', 'falgsc': 'false', 'dundas': 'funds', 'herbacide': 'herbicide', 'bigmac': 'sigma', 'exhaused': 'exhausted', 'alloo': 'allow', 'yeezy': 'needy', 'raksha': 'daksha', 'binny': 'bunny', '50m': 'him', 'ldv': 'liv', 'cih': 'chi', 'lfrom': 'from', 'overtapped': 'overlapped', 'got7': 'got', 'happywtf': 'happyto', '$asnd': 'and', 'durango': 'during', 'roel': 'roll', 'palmwine': 'palming', 'durag': 'drag', '24hs': 'oohs', 'arepas': 'areas', '59th': 'with', 'cuke': 'cute', 'b3': 'be', 'tangelo': 'angelo', 'pge': 'age', 'juuust': 'just', 'trussell': 'russell', 'mealboxes': 'mailboxes', 'flippin': 'flipping', 'waterfor': 'waterloo', 'evangelicalism': 'evangelicals', 'foodballet': 'footballer', 'tgiving': 'giving', 'morng': 'morn', 'yoin': 'join', 'iamai': 'mama', 'shopin': 'chopin', 'sipho': 'siphon', 'yogourt': 'yogurt', 'pf': 'of', 'ghante': 'chante', 'pdps': 'pops', 'ptowels': 'towels', 'ehackings': 'hacking', 'sulit': 'suit', 'odette': 'datte', 'nflorist': 'florist', 'fastco': 'fast', 'andrid': 'android', 'bupkiss': 'bumpkins', 'akl': 'all', 'aar': 'car', 'ngl': 'nil', '21days': 'days', 'marqeta': 'market', 'hmm': 'him', 'rz': 're', 'qwp': 'we', 'mcenany': 'meany', 'afreen': 'free', '50th': 'with', 'othrr': 'other', 'mdevine': 'devine', 'bopis': 'boris', 'jims': 'jim', 'jjajangmyeon': 'jjajangmyun', 'oem': 'poem', 'kigal': 'kill', '40kg': 'ekg', 'supposer': 'supposed', 'unbruised': 'bruised', '3lb': 'all', 'roseburg': 'rosebuds', '7out': 'out', 'atul': 'azul', 'sanpada': 'canada', 'psgrs': 'pages', 'a14': 'a', 'allyship': 'ladyship', 'aug22': 'aught', 'selfishess': 'selfishness', 'mydubai': 'dubai', 'consumables': 'consumable', 'ukufa': 'lkuma', 'ridi': 'ride', 'tanna': 'wanna', 'dn': 'in', 'albertan': 'albert', 'wipin': 'wiping', 'jtr': 'jar', 'sibe': 'side', 'mamata': 'matata', 'sprnd': 'spend', 'trecking': 'tracking', 'denature': 'denture', 'maraming': 'marrying', 'wtop': 'stop', 'macha': 'macho', 'papillion': 'pavilion', '18dec': 'dec', 'retentive]': 'retentive', 'alanis': "alan's", 'memb': 'memo', 'enywhere': 'anywhere', 'whebn': 'when', 'mypillow': 'pillow', 'assitance': 'assistance', 'chattery': 'chatter', 'birungo': 'bingo', 'yerr': 'year', 'c19': 'can', 'throughput': 'throughout', 'go15': 'got', 'stelter': 'shelter', 'bldgs': 'blogs', 'unflavored': 'flavored', 'ttw': 'tow', 'rsvp': 'rip', 'h1': 'he', 'pinterest': 'interest', 'img': 'im', 'uml': 'um', 'busness': 'business', 'jarringly': 'jarring', 'nesr': 'near', 'fursona': 'persona', 'zalora': 'valor', 'abliest': 'ablest', 'uninstall': 'install', 'strategys': 'strategy', 'davis09': 'davis', 'dbd': 'did', 'lele': 'lee', 'notip': 'not', 'mlevu': 'menu', 'irder': 'order', 'fairwage': 'fairway', 'halton': 'hilton', 'krna': 'kana', 'scurred': 'scarred', 'capmac': 'tarmac', 'remittancezw': 'remittance', 'crapton': 'carton', 'backpay': 'backpack', 'browne': 'brown', 'weg': 'we', 'andrelax': 'andrea', 'baithe': 'bathe', 'postmating': 'prostrating', 'bosasa': 'bossa', 'livingston': 'livingstone', 'escide': 'side', 'aeg': 'beg', 'mccartney': 'mccarthy', 'homestayers': 'homesteaders', '27minutes': 'minutes', 'niro': 'nitro', 'mga': 'ma', 'bojangles': 'bangles', 'wingnuts': 'wingnut', 'londoner': 'londoners', 'trumpf': 'trump', 'sodoku': 'sudoku', '6m': 'am', 'schaal': 'shall', 'pavillions': 'pavilions', 'thn': 'the', 'giftings': 'gifting', 'nac': 'na', 'goodpr': 'gooder', 'cuyapo': 'capo', 'sorree': 'soiree', 'continence': 'confidence', 'sickday': 'sickbay', 'groceryd': 'grocery', 'clemente': 'clement', 'opex': 'open', 'tusimple': 'simple', 'ledroit': 'detroit', 'leev': 'lee', 'bvi': 'bi', 'prescripti': 'prescription', 'eloy': 'ploy', 'urself': 'ourself', 'eldon': 'elton', 'jgh': 'ugh', 'dropsak': 'drops', 'dg': 'do', 'soamy': 'soapy', 'noped': 'hoped', 'redcard': 'record', 'yenna': 'henna', 'drvr': 'door', 'of2': 'of', 'klang': 'slang', 'primorus': 'priors', 'sbarnard': 'barnyard', 'cinderllas': 'cinderella', 'nomilk': 'milk', 'zw': 'ow', 'bandung': 'banding', 'burnley': 'burned', 'electricty': 'electricity', 'neue': 'nee', 'scbwi': 'scowl', 'veja': 'deja', 'bsa': 'bra', 'fayeda': 'fayed', 'unshapely': 'shapely', 'theorigin': 'theorizing', 'ariya': 'aria', 'quoodo': 'quod', 'alllat': 'afloat', 'franzia': 'frantic', 'r2r': 'ror', 'chickn': 'chicken', '$iwm': 'him', 'saic': 'said', 'dieepic': 'dietetic', 'screaming]': 'screaming', 'horr': 'hour', '24seven': 'seven', '3inches': 'inches', 'shntings': 'stings', 'ebusiness': 'business', 'serie': 'serve', 'afika': 'alike', 'lorderdd': 'ordered', 'laval': 'naval', 'aaah': 'ayah', 'oladash': 'coladas', 'bepositive': 'positive', 'bruns': 'runs', 'pd': 'ad', 'ciw15': 'ciwan', 'gh': 'go', 'beachwear': 'beachhead', 'anyy': 'any', 'albigna': 'align', 'lavash': 'lavish', 'uomo': 'homo', 'cartken': 'cartel', 'beloit': 'belong', 'j': 'i', 'yupp': 'yup', 'tizer': 'tiger', 'schl': 'she', 'yayvo': 'yay', 'beuseful': 'useful', 'winc': 'win', 'tpir': 'upir', 'obe': 'be', 'oufff': 'off', 'ph': 'oh', 'brbr': 'brer', 'goodful': 'roomful', 'mujhe': 'much', 'yyj': 'you', 'coasto': 'coast', 'boere': 'bore', 'vols': 'vous', 'mapoli': 'maple', 'ufika': 'erika', 'wjcl': 'will', 'famil': 'family', 'overhired': 'oversized', 'abreathing': 'breathing', '$sfm': 'sum', 'vishi': 'wish', 'rfh': 'rah', 'ready5': 'ready', 'igtv': 'it', 'teslin': 'tellin', 'papamoa': 'pajama', 'hzbus': 'bus', 'gbviews': 'views', 'toycat': 'tomcat', 'jsyk': 'ask', 'bidu': 'bid', 'mimimom': 'minimum', 'stuffies': 'stuffis', 'merchand': 'merchant', 'tmobile': 'mobile', 'safpa': 'safe', 'somenody': 'somebody', 'cambro': 'camaro', 'expensi': 'expense', 'dacapo': 'capo', 'fmc': 'fac', 'adeeg': 'deep', 'brechin': 'breaching', 'snkrs': 'stars', 'tarhan': 'tarzan', 'myheb': 'mayhem', 'mixology': 'biology', 'bbnow': 'know', 'deilvery': 'delivery', 'gnut': 'nut', 'leasy': 'least', 'differebt': 'different', 'besm': 'best', 'sooji': 'shoji', 'tomoro': 'totoro', 'bnh': 'bah', 'azad': 'and', 'molotovs': 'molotov', 'jise': 'rise', 'ekta': 'esta', 'alzheimer': "alzheimer's", 'bahaha': 'hahaha', 'distancers': 'distances', 'mariella': 'marcella', 'myburb': 'suburb', 'pandemicusa': 'pandemics', 'gampaha': 'yamaha', 'ptts': 'puts', 'commodores': 'commodore', 'rps': 'ups', 'coocked': 'cooked', 'bayy': 'baby', 'fraudalert': 'fraudulent', '24m': 'him', 'kelco': 'kelso', 'medway': 'midway', 'b2me': 'be', 'schnucks': 'schmucks', 'acount': 'count', 'ribas': 'ribs', 'brennaman': 'brennan', 'conditionally': 'conditional', 'guffaw': 'guffaws', 'telone': 'alone', 'patan': 'pagan', 'magdalenrd': 'magdalene', 'kr': 'or', 'deathmatch': 'deathwatch', 'ny16': 'ny', 'stater': 'state', 'genauso': 'genius', '$slv': 'sly', 'monstera': 'monster', '00h': 'oh', 'inddors': 'indoors', 'heaney': 'heavy', 'rthday': 'today', 'prac': 'pray', 'newsnow': 'newsboy', 'trival': 'trial', 'prd': 'pro', 'agad': 'aged', 'sanctimony': 'sanction', 'yatim': 'aim', 'inflatio': 'inflation', 'crafton': 'craft', 'delieveries': 'deliveries', 'litterbox': 'letterbox', 'redondo': 'redone', 'thundery': 'thunder', '2have': 'have', 'hartlands': 'garlands', 'thinkingi': 'thinking', 'aldrin': 'adrian', 'sasuke': 'assume', 'aasma': 'mama', 'sizov': 'size', 'shopkpr': 'shopper', 'addto': 'add', 'longlife': 'lowlife', 'vna': 'van', 'whonare': 'whore', 'kadish': 'radish', 'sizo': 'size', 'easyday': 'payday', 'helados': 'heads', 'sworder': 'order', 'catharines': 'catherine', 'sumthing': 'somthing', 'groceryuk': 'grocery', 'slc': 'sly', 'cristen': 'christen', 'genderism': 'genders', 'uncharged': 'uncharted', 'karni': 'karin', 'pleasee': 'please', 'here]': 'here', 'portsmith': 'wordsmith', 'norge': 'forge', 'jol': 'job', 'd6': 'do', 'karlie': 'earlier', 'hfc': 'hoc', 'suppies': 'supplies', 'probs': 'probe', 'deku': 'desu', 'onsale': 'insane', 'logisitcs': 'logistics', 'ellwood': 'elwood', 'dayenu': 'dayyou', 'coving': 'coming', 'aoba': 'soba', 'social20': 'social', 'pincode': 'income', 'yanggang': 'gangbang', 'lantidi': 'lantian', 'henckles': 'heckles', 'bathinda': 'bathing', 'popsugar': 'popular', 'maralit': 'morality', 'jgrocer': 'grocer', 'wih': 'with', 'goverment': 'government', 'pnb': 'pub', 'handwrite': 'handwrote', 'dra': 'draw', 'nnnn': 'none', 'outport': 'outpost', 'ingalls': 'inhales', '$rbz': 'orb', 'rifood': 'food', 'yg': 'ya', 'well]': 'well', 'cucuy': 'lucky', 'lubis': 'luis', 'pakwaan': 'padawan', 'truee': 'true', 'bizzes': 'buzzes', 'under20': 'under', 'adisha': 'aisha', 'civvie': 'civvies', 'wishbox': 'wishbone', 'sah': 'say', 'oner': 'one', 'kwai': 'kai', 'dietitian': 'dietician', 'bottleshop': 'battleship', 'hwnm': 'him', 'fadness': 'madness', 'techlife': 'techie', 'orki': 'ori', 'sloo': 'slow', 'million$': 'million', 'in09': 'in', 'kida': 'kids', 'faved': 'saved', 'severn': 'seven', 'aek': 'ask', 'redd': 'read', 'shofar': 'shower', 'epo': 'ego', 'wondaland': 'woodland', 'lider': 'liver', 'poeces': 'pieces', 'mabelle': 'belle', 'outsid': 'outside', 'liberalist': 'liberalism', 'smtlive': 'strive', 'cmc': 'cme', 'foodd': 'food', 'knw': 'know', 'esperanto': 'esperanza', 'rdc': 'rec', 'coopback': 'comeback', 'cornership': 'ownership', 'asst': 'ass', 'groundlink': 'grounding', 'bergmann': 'herrmann', 'pickme': 'pickle', 'sarnies': 'carnies', 'rania': 'mania', 'vaccinator': 'vaccination', 'palomas': 'paloma', 'mzansi': 'means', 'wader': 'water', 'heros': 'hero', 'stepup': 'step', 'ciel': 'cel', 'nanuet': 'banquet', 'perla': 'pela', 'xenophobia': 'xenophobic', 'dist': 'list', 'ktk': 'kak', 'day13': 'day', 'peckham': 'beckham', '80kg': 'ekg', 'unfreakin': 'freakin', 'delr': 'dear', 'peterhead': 'letterhead', 'thingsi': 'things', 'welathy': 'wealthy', 'shahi': 'shah', 'entriely': 'entirely', 'eeew': 'eee', 'engrained': 'ingrained', 'batavian': 'bavarian', 'todoor': 'door', 'mazar': 'maar', 'kiakia': 'khaki', 'nung': 'hung', 'uniq': 'unit', 'fres': 'free', 'founders]': 'founders', 'beelivered': 'delivered', 'gaisano': 'paisans', 'inhave': 'inhale', 'sampson': 'simpson', 'riemer': 'remer', 'biotin': 'biting', 'kmm': 'kim', 'nido': 'nino', 'trlt': 'tilt', 'whitby': 'whitey', 'kendiri': 'kendra', 'pala': 'palm', 'occorona': 'corona', 'ronne': 'ronnie', 'foreve': 'forever', 'iodize': 'iodine', '$takeaway': 'takeaway', 'rxturned': 'returned', 'tatem': 'totem', 'virtualized': 'visualized', 'wroth': 'worth', 'akong': 'along', 'rambut': 'rabbit', 'qanda': 'panda', 'tchrs': 'this', '$sby': 'by', 'geneia': 'geneva', 'njed': 'need', 'parkchat': 'backchat', 'coburg': 'court', 'couth': 'mouth', 'ucd': 'und', 'ajuda': 'aura', 'ewelina': 'eyeline', 'wework': 'rework', 'etx': 'et', 'accorderance': 'accordance', 'norwood': 'wormwood', '75c': 'doc', 'dtc': 'doc', 'didis': 'didi', 'wpp': 'app', 'kpu': 'pu', 'delboy': 'delay', 'cmarix': 'matrix', 'mrng': 'ming', 'vbc': 'vic', '95th': 'with', 'sani': 'sand', 'mayhaps': 'mayhap', 'n5': 'no', 'iibtv': 'gibt', 'mwhaha': 'haha', 'tfff': 'tiff', 'roder': 'order', 'masuk': 'mask', 'rwchat': 'what', 'meijrr': 'mejor', 'whoopsies': 'whoopsie', 'eckert': 'ecbert', 'nhn': 'non', 'bettr': 'better', 'work]': 'work', 'shrm': 'sham', 'swansons': 'seasons', 'nust': 'just', 'btexclusive': 'exclusive', 'goole': 'goose', 'jelmer': 'elmer', 'corderia': 'cordelia', 'grylls': 'grills', 'psawers': 'papers', 'iphoney': 'phoney', 'sfgmc': 'sigma', 'zika': 'ziva', 'edm': 'ed', 'laya': 'lay', 'mbfree': 'free', 'sinbad2': 'sinbad', 'floc': 'flow', 'arizon': 'arizona', 'roadyo': 'ready', 'chuchu': 'church', 'nuggies': 'buggies', 'fabien': 'fabian', 'may1st': 'mayst', 'vxx': 'vex', 'lothian': 'lotion', 'decatur': 'decanter', 'funs': 'fun', 'inforced': 'informed', 'ncpol': 'cool', 'sitars': 'stars', 'mcm': 'mom', 'izard': 'wizard', 'rayy': 'ray', 'gorderers': 'murderers', 'hffi': 'offi', 'cowmeat': 'combat', 'fmart': 'smart', 'spinoff': 'spiff', 'taquito': 'taquitos', 'powerade': 'powered', '$hawkeyet': 'hawkeye', 'stantec': 'started', 'abernat': 'aberrant', 'offbrand': 'offhand', 'laundrie': 'laundries', 'bflo': 'bolo', 'bria': 'bra', 'faves': 'faces', 'ozymy': 'ofmy', 'malnourish': 'malnourished', 'emtpy': 'empty', 'placitas': 'plaits', 'me1': 'me', 'catforder': 'camcorder', 'robi': 'rob', '4yours': 'yours', 'keanna': 'deanna', 'duba': 'dub', 'thxs': 'this', 'pdl': 'pal', 'anythint': 'anything', 'directtv': 'direct', 'anyt': 'any', 'rtinsights': 'insights', 'bebest': 'behest', 'motitom': 'bottom', 'bbmp': 'bump', '4ps': 'ups', 'obsm': 'obs', 'dl': 'do', 'tlabs': 'tabs', 'wisd': 'wish', 'pledger': 'pledge', 'competeters': 'competitors', 'dbids': 'bids', 'gday': 'day', 'mobili': 'mobile', 'webinar': 'seminar', 'pleadingpls': 'pleadings', 'smdh': 'such', 'thoughout': 'throughout', 'phonlo': 'phono', 'frigidly': 'rigidly', 'munroe': 'monroe', 'nybiz': 'biz', 'intension': 'intention', 'm4all': 'mall', 'unionizers': 'unionized', 'bwaha': 'bwana', 'mushkin': 'muslin', 'retweet]': 'retweet', 'borris': 'boris', 'kondena': 'konna', 'infeld': 'infield', 'execsuite': 'execute', 'promoed': 'promoted', 'saltman': 'salman', 'almaden': 'laden', 'ahould': 'should', 'repertoires': 'repertoire', 'strugg': 'strung', 'assesup': 'asses', 'aglaw': 'aglow', 'lowco': 'loco', 'frictio': 'friction', 'ppi': 'pi', 'akshath': 'asshat', 'arrngent': 'arrogant', 'stellatos': "stella's", 'mgt': 'met', 'jayda': 'jade', 'stonks': 'stones', 'ffvii': 'faii', 'theeee': 'there', 'migros': 'mirror', 'possiblity': 'possibility', 'pdi': 'di', 'smartpost': 'smartest', 'economi': 'economic', 'dunhams': 'dunham', 'onoshi': 'hoshi', 'cteph': 'steph', 'tedfled': 'trifled', 'rukh': 'rush', 'excitementso': 'excitements', 'viplist': 'vilest', 'ninad': 'nina', 'awoof': 'woof', 'decluttered': 'cluttered', 'rold': 'told', 'snakey': 'snake', 'rbst': 'rest', 'po64': 'poor', 'nvega': 'vega', 'azzi': 'aziz', 'protea': 'protean', 'niyes': 'nines', 'nkandla': 'nanda', '2l': 'al', 'momas': "mom's", 'mjepta': 'meta', 'lio': 'lie', 'braveman': 'caveman', 'tudes': 'dudes', 'gunviolence': 'nonviolence', 'crp': 'cop', 'r12': 'run', 'winers': 'winners', 'kernals': 'kernels', '22hr': 'sehr', 'ifthh': 'ifthe', 'mitiigated': 'mitigated', 'rendang': 'rending', 'partici': 'parties', 'kiarash': 'tiaras', 'nopatience': 'patience', 'digitalm': 'digital', 'cbj': 'cbi', 'mtwashington': 'washington', 'megz': 'mega', 'tobago': 'tobacco', 'whethe': 'whether', 'boii': 'boil', 'pyaz': 'paz', 'kartwheels': 'cartwheels', 'menlo': 'men', 'phark': 'park', 'askari': 'safari', 'lquentis': 'quentin', 'ableg': 'able', 'digorno': 'giorno', 'watsup': 'wassup', '20min': 'min', 'mmplove': 'improve', 'arete': 'arewe', 'kasi': 'kai', 'barrington': 'arlington', 'ajout': 'about', 'suany': 'sunny', 'carplay': 'carly', 'janiceh': 'janice', 'kushiin': 'cushion', 'paravada': 'parapara', 'chouji': 'choi', 'fawkes': 'fakes', 'brawner': 'brawler', 'mayward': 'wayward', 'budd': 'buddy', 'i3': 'i', '80yo': 'yo', 'purs': 'puts', 'efg': 'egg', 'fairshot': 'earshot', 'mcmillan': 'macmillan', 'itlog': 'log', 'radi': 'radio', 'rauch': 'ranch', 'pipelineiq': 'pipeline', 'doggo': 'doggy', '$upport$': 'support', 'investin': 'investing', 'hydrogenate': 'hydrogenated', '6g': 'og', 'lebelle': 'belle', 'trichy': 'tricky', 'zcj': 'act', 'stockfish': 'rockfish', 'tuuu': 'tutu', 'unhe': 'une', 'marva': 'maria', 'angeliki': 'angelic', 'trends]': 'trends', 'ehandel': 'handel', '30c': 'doc', 'dmv': 'dev', '$mtsi': 'emts', 'clou': 'cloud', 'deliever': 'deliver', 'lgus': 'gus', 'pamplets': 'pamphlets', 'mysiak': 'myriad', 'frieken': 'fricken', 'bfc': 'bac', 'rasband': 'husband', 'establi': 'establish', 'launch5': 'launch', 'stire': 'store', 'chqbook': 'chinook', 'thanet': 'thane', 'sifana': 'diana', 'vue': 'due', 'minsfor': 'minor', 'k12': 'kid', 'ajo': 'ago', 'masterpass': 'masterless', 'newlow': 'fellow', 'ucsb': 'usb', 'tblightning': 'lightning', 'beihn': 'bein', 'dissussing': 'discussing', 'etico': 'tico', 'dapat': 'apart', 'kiyaa': 'kinda', 'nangabe': 'wannabe', '4ir': 'air', 'remainers': 'retainers', 'vikrant': 'vibrant', 'welchs': 'welch', 'clps': 'cops', 'haya': 'hay', 'hipaa': 'hip', 'adela': 'adele', 'karne': 'karen', 'aholic': 'ahold', 'wapol': 'weapon', 'wildin': 'wilding', 'binmen': 'bitten', 'bigdeal': 'ideal', 'centeral': 'central', 'wilmington': 'wellington', 'clhaeges': 'changes', 'lefrak': 'leak', 'coppa': 'oppa', 'lancs': 'lands', 'afedex': 'fedex', 'getready': 'retread', 'eod': 'end', 'tufc': 'tuft', 'terp': 'term', 'paywall': 'payroll', 'dta': 'data', 'giglife': 'girlie', 'shippin': 'shipping', 'tozo': 'too', 'havannah': 'savannah', 'pannama': 'panama', 'can$': 'can', 'boardroom]': 'boardroom', 'mtlebanon': 'lebanon', 'waitingg': 'waiting', 'abailable': 'available', 'licken': 'licked', 'gw1': 'we', 'finistere': 'minister', 'choky': 'choke', 'joule': 'joules', 'nlp': 'nap', 'tailbacks': 'tailback', 'hannity': 'vanity', 'delayed]': 'delayed', 'woulde': 'would', 'mukono': 'mono', 's8': 'so', 'hamden': 'harden', 'phata': 'phat', 'chalo': 'chalk', 'adelaida': 'adelaide', 'stoneage': 'storage', 'gri': 'grip', 'dbe': 'be', 'louche': 'douche', 'kpvi': 'kevin', 'dgtr': 'door', 'hummart': 'summary', 'resturants': 'restaurants', 'c02': 'can', 'premake': 'remake', 'supermaket': 'supermarket', 'ikr': 'ike', 's18': 'so', 'nw1': 'we', 'newdad': 'needed', 'sxswi': 'sushi', 'sheetz': 'sheet', 'ucingo': 'using', 'askham': 'arkham', 'otl': 'oil', '27c': 'doc', 'maltesers': 'maltese', 'sarl': 'sail', '12th': 'with', 'delmar': 'dear', 'lpc': 'lac', 'psumac': 'puma', 'jhedic': 'medic', 'se9': 'see', 'karoo': 'kazoo', 'pbj': 'be', 'groceri': 'grocery', 'discovercvs': 'discovers', 'virar': 'viral', 'caterin': 'catering', 'amke': 'make', 'lalaland': 'lalala', 'nxa': 'na', 'cravenly': 'heavenly', 'joris': 'boris', 'ebma': 'emma', 'warby': 'wary', 'shieleded': 'shielded', 'fttb': 'fit', 'iguess': 'guess', 'essen': 'lessen', 'clough': 'cough', 'letsgo': 'lets', 'sgf': 'saf', 'awakish': 'awash', 'stickk': 'stick', 'gamedev': 'gamed', 'rivas': 'rival', '10aug': 'smaug', 'interenet': 'internet', 'ableism': 'atheism', 'hangry': 'angry', 'digitalt': 'digital', 'edmame': 'edamame', 'catsit': 'cast', 'thier': 'their', 'lade': 'made', 'trem': 'them', 'caserta': 'camera', 'ucare': 'care', 'stn': 'son', 'bouta': 'bout', 'pablano': 'poblano', 'equipmemt': 'equipment', 'bahar': 'bear', 'electoryte': 'electorate', 'waraira': 'warbird', 'lst': 'let', 'kefir': 'refer', 'jokr': 'joke', 'sotuation': 'situation', 'bohot': 'boot', 'pullin': 'pulling', 'soper': 'super', 'iter': 'item', 'dcf': 'def', 'amazonfc': 'amazons', 'popmarket': 'upmarket', 'lotogether': 'together', 'bgc': 'bac', 'zoya': 'soya', 'bianchi': 'branch', 'arlesey': 'arlene', 'bdsm': 'best', 'malaking': 'talking', 'thenk': 'think', 'gensan': 'german', 'nsfas': 'seas', 'salut': 'salt', 'wutzu': 'putz', 'nlg': 'nag', 'shanna': 'hanna', 'criteron': 'criterion', 'discov': 'disco', 'inlegis': 'inlets', 'wafoyo': 'wahoo', 'taru': 'tar', 'boofhead': 'bonehead', 'ps1': 'psi', 'uncrosser': 'uncrossed', '9b': 'ab', 'ceod': 'ceo', 'rlrt': 'let', '$acb': 'back', 'quq': 'que', '1h': 'oh', 'deisel': 'diesel', 'bith': 'with', 'needsl': 'needs', 'vegi': 'veg', 'cusco': 'cisco', 'hanlon': 'halon', 'puckup': 'pickup', '18your': 'your', 'benis': 'penis', 'pullas': 'pulls', 'platte': 'plate', 'fordero': 'order', 'isse': 'issue', 'upd8d': 'upped', 'bernies': 'berries', 'blumming': 'bumming', '$twoh': 'two', 'pckt': 'put', 'iine': 'fine', 'cipating': 'creating', 'seonds': 'seconds', 'muthafuckin': 'muthafucka', 'assing': 'asking', 'jeramie': 'ceramic', 'doxxing': 'doing', 'leshow': 'show', 'rsglobal': 'global', 'grocefy': 'grocery', 'illidan': 'lillian', 'getver': 'getter', 'altoona': 'alton', 'giggy': 'piggy', 'scaries': 'scares', 'dumbarses': 'dumbasses', 'gago2': 'ago', 'ihsaan': 'isaac', 'ifwt': 'ifwe', 'niv': 'ni', 'livving': 'living', 'casses': 'cases', 'wpvi': 'wave', 'refind': 'remind', 'cofc': 'coff', 'cricut': 'circus', 'ferrell': 'farrell', 'hogi': 'hog', 'panindia': 'panini', 'gmv': 'giv', 'surburbia': 'suburbia', 'lka': 'la', 'kareem': 'career', 'tham': 'that', 'utpol': 'upon', 'wfd': 'wed', 'induben': 'induce', 'complai': 'complain', 'nwhw': 'who', 'detailers': 'retailers', 'guarenteed': 'guaranteed', 'excercise': 'exercise', 'totchos': 'touches', '4students': 'students', 'lvmh': 'lame', 'grtting': 'getting', 'intune': 'intone', 'skilz': 'skill', 'tw4t5': 'twat', 'camrose': 'compose', 'hopi': 'hope', 'shmona': 'simon', 'rstech': 'tech', 'techbot': 'techno', 'thouse': 'those', 'e]': 'e', 'tinnie': 'winnie', 'nerdone': 'overdone', 'groxery': 'grocery', 'babai': 'baba', 'trmpy': 'trampy', 'toug': 'tough', 'sodaa': 'soda', 'fullll': 'full', 'orbach': 'reach', 'wwmt3': 'wwith', 'karmita': 'karma', 'pachter': 'painter', 'lockdow': 'lockdown', 'flakely': 'blakely', 'kingdim': 'kingdom', 'botad': 'bad', 'skipcart': 'skincare', 'unrolling': 'enrolling', 'wuplo': 'upl', 'ds2': 'is', 'loveso': 'loves', 'ostentatiously': 'ostentatious', 'unedible': 'inedible', 'mishearing': 'misleading', 'drivi': 'drive', 'meijin': 'mein', 'nanue': 'nante', 'habbibs': 'habits', 'tovah': 'torah', 'alot': 'lot', 'ol9': 'old', 'cosplays': 'cosplay', 'sabino': 'sabine', 'yev': 'yet', 'foror': 'forom', 'halafu': 'half', 'refinitiv': 'definitive', 'wowyar': 'boyar', 'apl': 'all', 'dokan': 'down', 'hoge': 'home', 'chimbly': 'chilly', 'demino': 'domino', 'shab': 'stab', 'convid': 'convict', 'whrv': 'what', '$unh': 'unh', 'lpt': 'let', 'temprite': 'termite', 'koli': 'koi', 'tabla': 'table', 'broda': 'broad', 'rubix': 'rubin', 'optio': 'option', 'lueders': 'murders', 'geuda': 'gouda', 'brothe': 'brother', 'happioness': 'happiness', 'stravagant': 'extravagant', 'otage': 'stage', 'chocka': 'chock', 'loggi': 'long', 'nomakeup': 'makeup', 'matterz': 'matter', 'isbpl': 'isabel', 'urmston': 'preston', 'chair]': 'chair', 'marcela': 'marcel', 'weezy': 'weepy', 'gadfly': 'badly', 'ostly': 'mostly', 'tenaday': 'tenancy', 'rossland': 'roland', 'fashonista': 'fashionista', 'yesplease': 'displease', 'corderial': 'cordial', 'vancity': 'vanity', 'servicefee': 'servicemen', 'onsen': 'onset', 'shr': 'she', 'usmc': 'usc', 'blmanga': 'manga', 'galee': 'glee', 'redditors': 'editors', 'frenz': 'frenzy', 'ikeda': 'ikea', 'bz': 'be', 'myanmar': 'mandar', 'bradgate': 'graduate', 'resu': 'rest', 'shyft': 'shift', 'sabulao': 'tabula', 'verdes': 'verses', 'overshopping': 'overstepping', 'urusan': 'urban', 'maxx': 'max', '90c': 'doc', 'closeby': 'closely', 'opac': 'opal', 'yerg': 'yer', 'expoited': 'exploited', 'nomc': 'nom', 'wayfarer': 'warfare', 'truno': 'trunk', 'spendy': 'spend', 'beekind': 'behind', 'nb': 'no', 'lakeline': 'lifeline', 'dula': 'dull', 'smileys': 'smiles', 'fiweh': 'five', 'replen': 'reply', 'reimberse': 'reimburse', '4ish': 'wish', 'chamath': 'hamate', 'tneducators': 'educators', 'lavka': 'lava', 'zee5': 'zee', 'funnyno': 'funny', 'amrish': 'amish', 'rrif': 'arif', 'prescriptive': 'prescription', 'phiri': 'shiri', 'mashup': 'makeup', 'day7': 'day', 'snapebt': 'snape', 'imaginatively': 'imaginative', 'delt': 'felt', 'eltham': 'lethal', 'kholi': 'whole', 'c5': 'ca', 'quickbuy': 'quickly', 'dcavar': 'caviar', 'egads': 'gas', 'facebookads': 'facebooked', 'natoli': 'natalie', 'albanese': 'lebanese', 'adorbs': 'adores', 'doitright': 'downright', 'whurfm': 'wharf', 'jeezo': 'jeez', 'buloh': 'blow', '$wdi': 'di', 'marionos': "marion's", 'zepto': 'kept', 'firts': 'first', 'followin': 'following', 'sm': 'so', 'dlg': 'dog', 'greengb': 'green', '5days': 'days', 'pavan': 'pagan', 'congr': 'conga', 'lakeland': 'leland', 'havasu': 'harass', 'bandhu': 'band', 'provo': 'prove', 'ww': 'we', 'shud': 'shut', 'copius': 'copies', 'chendani': 'chennai', 'eyeopening': 'reopening', 'digue': 'die', 'menstral': 'menstrual', 'armenia': 'armenian', 'forhead': 'forehead', 'isolati': 'isolate', 'hhaybo': 'shambo', 'letseat': 'retreat', 'clemm': 'clem', 'postgres': 'postures', 'malteser': 'maltese', 'attestive': 'attentive', 'siaya': 'saya', 'autonom': 'autonomy', 'kuang': 'kung', 'warly': 'early', 'arlingtontx': 'arlington', 'phisho': 'phish', 'g13': 'get', 'tyoures': 'yours', 'congrtn': 'congrats', 'nvsnap': 'snap', 'desc': 'desk', 'my3': 'my', '$srax': 'sax', 'lach': 'each', 'georganics': 'organics', 'aati': 'anti', 'jm': 'am', 'uncornered': 'uncovered', 'fooduse': 'foods', 'gby': 'by', 'nmes': 'names', 'jamaal': 'jamal', 'censhare': 'censure', 'toor': 'too', 'balaji': 'salami', 'bleurgh': 'burgh', 'sqsp': 'stop', 'cartfr': 'carter', 'meowsie': 'meows', 'dekar': 'dear', '70f': 'of', 'itsow': 'its', 'telcom': 'telecom', 'prosy': 'pros', 'b$': 'be', 'ingles': 'singles', 'localbus': 'locals', '25gig': 'gig', 'comtmnt': 'commit', 'believin': 'believing', 'lowpoly': 'lowly', 'toploader': 'uploader', 'gfm': 'gym', 'hounews': 'houses', 'morans': 'moans', 'discombobulate': 'discombobulated', 'benjies': 'bennies', 'syfy': 'say', 'perennials': 'perennial', 'dumbarse': 'dumbass', 'brandis': 'brands', 'supersol': 'superior', 'tlou2': 'thou', 'prearrangement': 'rearrangement', 'konga': 'conga', 'approxima': 'approximate', 'taloja': 'talons', 'dtn': 'don', 'producti': 'product', 'journaler': 'journal', 'numbe': 'number', 'easy7': 'easy', 'dinty': 'dirty', 'markyate': 'marinate', 'outfam': 'outta', 'bitchin': 'bitching', 'whitmer': 'whiter', 'hoggy': 'doggy', 'ramc': 'ram', 'graziano': 'grazing', 'ctfd': 'cold', 'mutare': 'mutate', 'charlsley': 'charley', 'illy': 'silly', 'marlay': 'parlay', 'enough$': 'enough', 'zhombe': 'home', 'edlp': 'help', 'tso': 'to', 'praire': 'praise', 'reccomend': 'recommend', 'begi': 'beg', 'blv': 'bla', 'kqa': 'ka', 'hardie': 'hardier', 'instac': 'instar', 'veeb': 'verb', 'hitchin': 'hitching', 'rogi': 'yogi', 'gfy': 'guy', 'refrig': 'refrain', 'joongang': 'poontang', 'usaba': 'sara', 'giroud': 'proud', 'creggs': 'eggs', 'slushies': 'slushie', 'amtwt': 'matt', 'handydart': 'handcart', 'tomtoming': 'bottoming', 'chauvin': 'chain', 'municipa': 'municipal', 'delist': 'desist', 'ypal': 'pal', 'tillie': 'willie', 'betuel': 'betel', 'dmblogs': 'blogs', 'leomar': 'leopard', '$sapx': 'sap', 'mcinnis': 'minnie', 'whtc': 'what', 'tnf': 'anf', 'lewerd': 'lower', 'debish': 'jewish', 'presc': 'press', 'rtih': 'with', '97th': 'with', 'gofund': 'found', 'weeett': 'werert', 'stayin': 'staying', 'gromit': 'profit', 'fyc': 'fy', '60minutes': 'minutes', 'srd': 'sad', '56b': 'job', 'dranken': 'drunken', 'pasteurise': 'pasteurized', 'storese': 'stores', 'reee': 'reel', 'viles': 'miles', 'overit': 'overt', 'musher': 'muster', 'becautious': 'cautious', 'eastland': 'wasteland', 'onboarded': 'boarded', 'juuls': 'souls', 'yaiba': 'yabba', '3boxes': 'boxes', 'sonething': 'something', 'receip': 'receipt', 'maka': 'make', 'verona': 'verone', 'nospam': 'spam', 'wellerman': 'elderman', 'marton': 'martin', 'hove15': 'hover', '5lba': 'alba', 'batte': 'battle', 'seasonto': 'reasonto', 'dwnload': 'download', 'otipy': 'tiny', 'fod': 'for', '$bnews': 'news', 'skday': 'say', 'cuhs': 'cuts', 'tlise': 'elise', 'sevier': 'sever', 'sendme': 'send', 'g51': 'get', 'bantwal': 'banal', 'chowtime': 'showtime', 'noshit': 'shit', 'peobably': 'probably', 'mtpol': 'tool', 'nowell': 'powell', 'mamy': 'many', 'taji': 'taxi', 'aws': 'was', 'bl7': 'bla', 'naivasha': 'natasha', '24hours': 'hours', 'uppies': 'puppies', 'nuthin': 'nothin', 'humilate': 'humiliate', 'uavs': 'uav', 'newbitch': 'bewitch', 'dayjob': 'ajob', 'vaneisha': 'vanish', 'megacycle': 'megacycles', 'jevne': 'jeune', 'solivita': 'olivia', 'cote': 'come', 'andhra': 'anders', 'boonstra': 'bootstrap', 'prescheduled': 'rescheduled', 'peret': 'beret', 'uuuu': 'guru', 'sansitise': 'sensitive', 'locsl': 'local', 'applaudto': 'applaud', 'lancia': 'lance', 'gorton': 'gordon', 'ttntt': 'tent', 'failover': 'allover', 'cattv': 'catty', '$l': 'al', '$arnews': 'harness', 'diel': 'die', 'bethan': 'ethan', 'wrangell': 'wrangle', 'comfirmed': 'confirmed', 'ratcliffe': 'radcliffe', 'crystar': 'crystal', 'boyds': 'boys', 'geezz': 'geez', 'deloveery': 'delivery', 'cyouril': 'cyril', 'dhyari': 'hari', 'bqexc': 'bec', 'wftv': 'with', 'colima': 'climb', 'vespa': 'vest', 'zont': 'dont', 'egc': 'egg', 'mussee': 'mussel', 'euan': 'yuan', 'megamart': 'megawatt', 'donal': 'donald', '3pint': 'pint', 'choswn': 'chosen', 'yoongi': 'young', 'geelong': 'feeling', 'orfer': 'order', 'rwot': 'root', 'paisaje': 'passage', 'atlest': 'atleast', 'meyen': 'meyer', 'ecomerce': 'commerce', 'elden': 'elder', 'draegers': 'rangers', 'utnaa': 'tuna', 'phcc': 'pick', 'successul': 'successful', 'deeplea': 'deeply', 'rsgc': 'rage', 'emoloyer': 'employer', 'bbcan': 'can', 'whiffle': 'wiffle', 'befre': 'before', 'kpm': 'kim', 'manda': 'panda', 'consumptn': 'consumption', '4mor': 'amor', 'accomodation': 'accommodation', 'partenr': 'partner', 'woodman': 'woodsman', 'awtwb': 'aww', 'heds': "he's", 'arla': 'area', 'servoces': 'services', 'biuk': 'bilk', 'rp': 'up', 'winterpeg': 'wintering', 'countryfile': 'countryside', 'emaile': 'email', 'zsl': 'zsa', 'tebow': 'throw', 'ncrsystem': 'ecosystem', 'appaulse': 'applause', 'jeddah': 'judah', '$rtp': 'drop', 'rockwood': 'lockwood', 'delightedly': 'delighted', 'putney': 'puny', 'resumimg': 'resuming', 'uwinipeg': 'winnipeg', 'resi': 'rest', 'supportively': 'supportive', 'btwn': 'been', 'individua': 'individual', 'us5': 'us', 'yonow': 'know', 'jwhd': 'who', 'schunck': 'chuck', 'slurpees': 'slurpee', 'stitt': 'stint', 'arrrr': 'arrow', 'orwildfires': 'wildfires', 'gbowo': 'grow', 'scsu': 'csu', 'finzels': 'fingers', 'lalege': 'allege', 'thullu': 'hull', 'gruopon': 'groupon', 'nexgen': 'ningen', 'mikkelson': 'mikaelsons', 'hlophe': 'hope', 'someome': 'someone', 'greenle': 'greenie', 'sugo2': 'sugar', 'wrt': 'art', 'osun': 'sun', 'ulendo': 'lend', 'hourish': 'nourish', 'roorkee': 'rookie', 'myeg': 'meg', 'ate68': 'ate', 'pham': 'ham', 'schwoerer': 'schroeder', 'shenley': 'shelley', 'bishh': 'bish', 'dyin': 'dying', 'ayub': 'club', 'lomas': 'comas', 'venmoing': 'vending', 'bestprices': 'bestrides', 'buyblack': 'buyback', 'austintx': 'austin', 'sbry': 'sory', 'dismissively': 'dismissive', 'smartpay': 'smartly', 'muchi': 'much', 'woodhouse': 'roadhouse', 'avery16': 'avery', 'asean': 'asian', 'rotondi': 'rotund', 'catmas': 'cats', 'sarahs': 'sarah', 'avers': 'avert', 'develope': 'developed', 'semiya': 'seiya', 'plandemic': 'pandemic', 'impreza': 'impress', 'noairfreight': 'airfreight', 'agaiin': 'again', 'inclusivity': 'exclusivity', 'jeroen': 'heroes', 'eftpos': 'ethos', 'gobox': 'good', 'crm': 'cry', '1hub': 'hub', 'babyfaced': 'barefaced', 'floridays': "florida's", 'hiload': 'load', 'i]': 'i', 'debi': 'debt', 'zealan': 'zealand', 'ingat': 'ingot', 'skool': 'stool', 'wjo': 'who', 'summatime': 'summertime', 'compliants': 'complaints', 'foodtown': 'boomtown', 'nbc7': 'nice', '27th': 'with', 'another30': 'another', 'kahit': 'wait', 'anythng': 'anything', 'dmx': 'dex', '01st': 'just', '8th': 'ith', 'coldcuts': 'conducts', 'koha': 'kohl', 'happyoh': 'happy', 'medl': 'meal', 'milkfat': 'milkman', 'cspnet': 'spent', 'hindy': 'handy', 'ibdmoms': 'idioms', 'dialy': 'daily', 'sk': 'so', 'workloads': 'workload', 'espana': 'expand', 'sweeet': 'sweet', 'bairds': 'birds', 'tnbeer': 'beer', 'suppoes': 'suppose', 'tn': 'to', 'pcexpress': 'express', 'afs': 'as', 'ceejay': 'deejay', '$prpm': 'prom', 'modris': 'morris', 'gatech': 'watch', 'dictors': 'doctors', 'gj4ites': 'genites', 'curbstomp': 'curbstone', 'jkjk': 'jerk', 'nidoshi': 'hiroshi', 'geniusden': 'geniuses', 'veni': 'vent', 'shtuff': 'stuff', 'luciana': 'lucian', '87yours': 'yours', 'bhej': 'the', 'guntur': 'guitar', 'gahlan': 'gallant', 'soending': 'sending', 'honesbee': 'honeybee', 'ncstate': 'state', 'go4me': 'come', 'malika': 'malik', 'authorites': 'authorities', 'kouncil': 'council', 'h2': 'he', 'availabilty': 'availability', 'goodnights': 'goodnight', '$tlsa': 'salsa', 'eastsidesa': 'eastside', 'iwf': 'if', 'deliveroo': 'deliver', 'birr': 'bird', 'mearns': 'means', 'mologistics': 'logistics', 'm50i': 'mi', 'materisls': 'materials', 'oscott': 'scott', 'predi': 'pedi', 'dintsi': 'dings', 'tinternet': 'internet', 'spuriously': 'seriously', 'yakes': 'makes', 'messge': 'message', 'halili': 'hall', 'crosstrek': 'crossbred', 'cnn]': 'can', '12lbs': 'bulbs', 'jual': 'dual', 'ksnv': 'kind', 'nobrand': 'brand', 'scho': 'echo', 'greetin': 'greeting', 'mondaaay': 'monday', 'muhith': 'judith', 'holliston': 'collision', 'linkedin': 'linked', '27m': 'him', 'artemy': 'artery', 'py': 'my', '8ths': 'the', 'vlasic': 'basic', 'ponderosa': 'ponderous', 'transformat': 'transform', 'scooplet': 'scooped', 'bagh': 'bag', 'tndom': 'random', 'paapa': 'papa', 'recimo': 'recipe', 'buzillion': 'bazillion', 'ducation': 'education', 'psyops': 'stops', 'lodtc': 'lot', 'guidlines': 'guidelines', 'appsolution': 'absolution', '34oz': 'coz', 'durries': 'hurries', 'dammika': 'dammit', 'teenmoms': 'tenemos', 'fynirs': 'fairs', 'menso': 'mens', 'bcefa': 'beef', 'nonexistant': 'nonexistent', 'myal': 'meal', 'caligra': 'calibre', 'mzinga': 'ming', 'significa': 'significant', 'mya': 'my', 'frieden': 'friend', 'delivey': 'deliver', 'lp': 'up', 'gratituded': 'gratitude', 'water]': 'water', 'artics': 'antics', 'rathwr': 'rather', 'daam': 'dam', 'demic': 'demi', 'braless': 'bless', 'rexghing': 'reaching', 'hwut': 'hut', 'reinfected': 'infected', 'chaldal': 'chantal', 'kneads': 'knead', 'holgate': 'collate', '$csco': 'disco', 'scritch': 'scratch', 'mbok': 'bok', 'florio': 'florid', '$shop': 'shop', 'awesomeon': 'awesome', 'xm': 'am', 'b9': 'be', 'visitacion': 'visitation', 'argenta': 'argent', 'alexys': 'alexis', 'tekel': 'teel', 'googl': 'google', 'bbc1': 'back', 'fluz': 'flu', 'fdr': 'for', 'czk': 'czy', 'everyine': 'everyone', 'surprisey': 'surprise', 'genna': 'gonna', 'rcslearn': 'relearn', 'sociopat': 'sociopath', 'messager': 'message', '$leyat': 'bleat', 'alluzo': 'allure', 'luddites': 'buddies', 'sheepdip': 'sheepdog', 'th': 'the', 'alabaru': 'alabama', 'gno': 'no', 'corktown': 'yorktown', 'fers': 'fears', 'latcham': 'latch', 'ondc': 'ond', 'toru': 'tour', 'coypu': 'cops', 'limeade': 'lineage', 'cideries': 'miseries', 'ipnews': 'news', 'teng': 'ten', 'kaupapa': 'kappa', 'bentrim': 'benthic', 'goign': 'going', 'autoimm': 'autism', 'ncbloggers': 'bloggers', 'joinme': 'join', 'scamzone': 'calzone', 'dlvr': 'door', 'c': 'i', 'phine': 'phone', 'doz': 'do', 'futaba': 'futari', 'jormall': 'normal', 'baggers': 'buggers', 'sanitise': 'sanitize', 'kamuda': 'lambda', 'poway': 'noway', 'ghen': 'when', 'headedness': 'headdress', 'jok': 'job', 'fckd': 'find', 'ayouro': 'your', 'rs19': 'rse', 'misreads': 'misread', 'kudineer': 'mutineer', 'frightclub': 'nightclub', 'chex': 'chew', 'neet': 'need', 'flgiveaway': 'giveaway', 'tarky': 'tacky', 'ekasi': 'easy', 'fitn': 'fit', 'obc': 'orc', 'petstore': 'restore', 'covina': 'coming', 'agreenew': 'greener', '2he': 'the', 'piz': 'pig', 'dvt': 'dot', 'mindster': 'minister', 'mcshane': 'methane', 'm': 'i', 'congregant': 'congregants', 'appology': 'apology', 'aiken': 'liken', 'frippery': 'slippery', '85p': 'up', 'oundle': 'bundle', 'sendle': 'senile', 'ipoh': 'ipod', 'cf24': 'cfo', 'contactl': 'contact', 'instructred': 'instructed', 'punchered': 'punched', 'ofday': 'oday', 'maskhole': 'asshole', 'rudgers': 'rodgers', 'donething': 'domething', 'gmorning': 'morning', 'downstater': 'downstate', 'jlockett': 'locket', 'reapeat': 'repeat', 'ngel': 'angel', 'indigeneous': 'indigenous', 'hongi': 'hong', 'maskup': 'markup', 'yitt': 'mitt', 'kucky': 'lucky', 'ltn': 'lan', 'ryebrook': 'rebook', '4bill': 'bill', 'laue': 'late', 'goodjob': 'goodfor', 'hordere': 'order', 'jewels]': 'jewels', 'competin': 'competing', 'excdpt': 'except', 'joylit': 'jolt', 'piperg': 'piper', 'harrington': 'arlington', 'prot': 'pot', 'teledocs': 'telecoms', 'samaan': 'sampan', 'socializ': 'socialize', 'simpy': 'simply', 'epass': 'pass', 'cayer': 'layer', 'centralise': 'centralize', 'vail': 'jail', 'btr': 'bar', 'wck': 'wack', 'walkaway': 'walkway', 'shanny': 'shanty', 'ittwt': 'itto', 'letseats': 'retreats', 'jiomart': 'xiomara', 'stoli': 'stole', 'elkind': 'elkins', 'vaccinati': 'vaccinate', 'suprt': 'spurt', 'damb': 'damn', 'techpro': 'techno', 'drvrs': 'dress', 'rheu': 'rhe', 'corby': 'corny', 'kopit': 'kept', 'centrist': 'dentist', 'carepackage': 'repackage', 'lucah': 'lucas', 'marg': 'mark', 'tostones': 'stones', 'tiredso': 'tired', 'biab': 'bias', 'nofrills': 'frills', 'pirchase': 'purchase', 'dripfeed': 'drifted', 'mayani': 'mayans', 'buku': 'boku', 'kristina': 'cristina', 'raas': 'rats', 'llmf': 'lame', 'gedders': 'genders', 'noxtech': 'notch', '60m': 'him', '85c': 'doc', 'taglines': 'tagline', 'dvs': 'des', 'mokulele': 'ukulele', 'pluh': 'plus', 'seattleish': "seattle's", 'plant18': 'plant', 'smashin': 'smashing', 'gisd': 'gist', 'mnth': 'month', 'unta': 'unto', 'oriana': 'organ', 'oviedo': 'video', 'layton': 'clayton', 'gvg': 'gig', 'tizz': 'jizz', 'hazlo': 'halo', 'hfmp': 'hump', 'combinatio': 'combination', 'itsel': 'itself', 'fillion': 'million', 'rdash': 'rash', 'pems': 'poems', 'greentech': 'greenwich', 'ebutler': 'butler', 'poors': 'poor', 'shugar': 'sugar', 'ranty': 'panty', 'gdn': 'gun', 'helplines': 'helpline', 'csra': 'cara', 'bizero': 'zero', 'pwedeng': 'sweden', 'paseo': 'paso', 'coworking': 'working', '$spot': 'spot', 'alberton': 'alberto', 'evolutonary': 'evolutionary', 'quantine': 'quarantine', 'benifical': 'beneficial', 'moraine': 'morning', 'purrp': 'purr', 'beni': 'bend', 'marsden': 'garden', 'smthn': 'stun', '$cat': 'cat', 'kre': 'are', 'howeice': 'hospice', 'asun': 'sun', 'spamazon': 'amazon', 'jayde': 'jade', 'subsists': 'subsist', 'botana': 'botany', 'whsmith': 'smith', 'spoilage': 'spillage', 'bcstx': 'best', 'nra': 'na', 'listerine': 'listening', 'socalled': 'called', 'univeral': 'universal', 'buhari': 'bhai', 'coresight': 'foresight', 'wheaton': 'weapon', 'angeline': 'angelina', 'dickh': 'dick', '3year': 'year', 'reliefri': 'relief', '1a': 'a', 'accorders': 'accorded', 'homefront': 'forefront', 'crossie': 'cross', 'duale': 'dual', 'g32': 'get', 'urselfs': 'ourself', 'teton': 'ten', 'saintsfc': 'saints', 'hortible': 'horrible', 'norzli': 'nozzle', 'fomo': 'homo', '3things': 'things', 'dhaka': 'shaka', 'chos': 'chose', 'sdg2': 'sign', 'sketchboo': 'sketchbook', 'grreat': 'great', 'sess': 'less', 'greensboro': 'queensboro', 'intrvu': 'intro', 'breathaly': 'breathable', 'veras': 'vegas', 'alem': 'ale', 'kwik': 'klik', 'chezein': 'wherein', 'jannah': 'hannah', '6pages': 'pages', 'toco': 'too', 'instant15': 'instant', 'treater': 'treated', 'pelotons': 'protons', 'bonney': 'bonnet', 'shihtzu': 'shiatsu', 'tooter': 'hooter', 'bigdat': 'begat', 'mugga': 'muggy', 'joyit': 'join', 'amadeu': 'made', '4a': 'a', 'spirou': 'spiro', 'pym': 'gym', 'ufcs': 'ufos', 'relas': 'relax', 'quantum17': 'quantum', 'gubbins': 'nubbins', 't3sco': 'disco', 'foodland': 'woodland', '$nbio': 'bio', 'quicksent': 'quickest', 'chmaers': 'cheers', 'eatsbox': 'hatbox', 'kirane': 'kiran', 'contrac': 'contract', 'robtest': 'contest', 'century21': 'century', 'uhhmmm': 'uhhmm', '30am': 'am', 'daftie': 'date', 'cr17': 'cry', 'gilpin': 'gillian', 'smores': 'stores', 'asnd': 'and', 'prestonians': 'estonians', 'recurve': 'receive', 'pron': 'iron', 'prezzy': 'pretty', 'dubey': 'due', 'cabelas': 'cameras', 'abena': 'arena', 'ntinda': 'kinda', 'bourdain': 'ordain', '23f': 'of', 'cudifference': 'difference', '2chemo': 'chemo', 'smt': 'set', 'foodness': 'goodness', 'servingtn': 'serving', 'qoq': 'coq', 'quartantine': 'quarantine', 'dikoloi': 'nikolai', 'deutsch': 'deutsche', 'bedeo': 'been', 'downland': 'download', 'cx': 'ex', 'northam': 'north', 'deddy': 'daddy', 'villarin': 'villain', 'plantians': 'plantains', 'oodee': 'order', 'quil': 'quit', 'westgate': 'estate', 'gimme10': 'gimme', 'listuk': 'listen', 'totalin': 'totaling', 'inmintently': 'imminently', 'groccery': 'grocery', 'ntm': 'nom', 'garag': 'garage', 'nogroceries': 'groceries', 'kelat': 'eat', 'instructio': 'instruction', 'crazzzy': 'crazy', 'stockworkers': 'stockbrokers', 'executiv': 'executive', 'fuul': 'full', 'edenpark': 'denmark', 'hoboken': 'hooked', 'tipped20': 'tipped', 'cancld': 'cancel', 'andher': 'andhe', 'katawan': 'katana', 'tallinn': 'talking', 'tnews': 'news', 'eish': 'wish', 'curethe': 'curate', 'treate': 'treat', 'dami': 'damn', 'familie': 'families', 'beber': 'beer', 'link]]': 'link', 'glfeel': 'lfeel', 'calvert': 'culvert', 'albertans': 'algerians', 'unlicked': 'unlocked', 'fb': 'fu', 'convenientte': 'convenient', 'kentuck': 'kentucky', 'one97': 'one', 'mlrs': 'mars', 'bdo': 'do', 'hifi': 'ifi', 'provied': 'provide', 'fcss': 'fuss', 'paranai': 'paranoid', 'cpzs': 'cops', 'piccard': 'picard', 'vivek': 'vive', 'kaia': 'kara', 'pkgs': 'pigs', 'fentynol': 'fentanyl', '5am': 'am', 'toomey': 'doomed', 'realme': 'realm', 'requestb': 'request', 'presid': 'preside', 'nandos': 'nando', 'camber': 'chamber', 'ready2go': 'readyto', 'deferral': 'referral', 'lacap': 'local', 'oit': 'it', 'canc': 'can', 'togeek': 'geek', 'lenovo': 'lenore', 'hagw': 'hag', 'checkitout': 'checkout', 'mbcaptain': 'captain', 'homefree': 'homered', 'ifnot': 'isnot', 'stess': 'steps', 'xna': 'dna', 'afforder]': 'afforded', 'chand': 'hand', 'swonderfull': 'wonderful', 'sahab': 'sahib', 'oev': 'rev', 'naned': 'named', 'fwend': 'fend', 'obgyn': 'obey', 'cuh': 'cut', 'baajaar': 'bazaar', 'restau': 'rest', 'dogee': 'doge', 'backends': 'backend', 'machiavelli': 'machiavellian', 'balsamics': 'balsamic', 'retailne': 'retain', 'gogreen': 'green', 'ideaa': 'idea', 'baaloody': 'bloody', 'daays': 'days', 'peppie': 'people', 'tassles': 'hassles', 'pointman': 'postman', 'fingest': 'finest', 'augusta': 'august', 'pickley': 'pickled', 'shami': 'shame', 'asfuck': 'fuck', 'puria': 'curia', 'baatein': 'beaten', 'tutup': 'tutu', 'arsoft': 'soft', 'rogel': 'roger', 'akshay': 'sashay', '2p': 'up', 'npay': 'pay', 'tjmax': 'tax', '1mbb': 'mob', 'hopefulky': 'hopefully', 'pex': 'sex', 'meno': 'men', '2d': 'ad', 'despisers': 'despises', 'seaforth': 'seaworthy', 'addre': 'adore', 'ranney': 'range', 'provigo': 'proviso', '20your': 'your', 'ireally': 'really', 'homesa': 'homes', 'markstey': 'marketed', 'test3': 'test', 'digitalad': 'digital', 'residencial': 'residential', 'hygien': 'hygiene', 'pn': 'in', '40lb': 'bulb', 'aita': 'rita', 'brewster': 'browser', 'tremorfa': 'tremors', 'copyouright': 'copyright', 'shouldnt': "shouldn't", 'ausa': 'aura', 'texaco': 'teach', 'lennard': 'leonard', 'ces20': 'ces', 'fadzi': 'fade', 'sedona': 'second', 'lvtf': 'let', 'iman': 'man', 'spca': 'spa', 'uti': 'ti', 'kinkos': 'kinks', 'lotr': 'lot', 'ustin': 'justin', 'wissched': 'wished', 'avcato': 'vacate', 'jennifers': 'jennifer', 'fakhri': 'faker', 'zure': 'sure', 'prepareai': 'prepared', '24oz': 'coz', 'capi': 'cap', 'nrtc': 'not', '$afrm': 'warm', 'deanwood': 'deadwood', 'skaut': 'start', 'mco': 'mo', 'accts': 'acts', 'tingum': 'tongue', 'blackcat': 'blackout', 'lrca': 'loca', 'shreiking': 'shrieking', 'dropoffs': 'dropouts', 'rydges': 'ridges', 'sugoi': 'sugi', 'coru': 'core', 'medicati': 'medicate', 'coronad': 'coronal', 'suzuka': 'suzuki', 'unip': 'unit', 'teladoc': 'celadon', 'gtoceries': 'groceries', 'sibo': 'silo', 'panny': 'penny', 'ashoks': 'ashok', 'ngt': 'not', 'cheezecake': 'cheesecake', 'fx': 'fix', 'millionare': 'millionaire', 'scv': 'suv', 'greenbelt': 'greenest', 'ncga': 'naga', 'laboura': 'labour', 'slavitt': 'slaving', 'chrystia': 'christian', 'pavillon': 'pavilion', 'supporing': 'supporting', 'theview': 'review', 'bida': 'bid', 'yig': 'big', 'fba': 'fbi', 'adecco': 'deco', 'birx': 'bird', 'mfn': 'man', 'pixelon': 'pigeon', 'succulento': 'succulent', 'coronadc': 'coronado', 'gdp': 'gap', 'vendome': 'venom', 'woyld': 'would', 'smartgro': 'smarter', 'armands': 'armbands', 'schenley': 'shelley', 'andfill': 'landfill', 'misred': 'missed', 'reu': 'red', 'brumhour': 'rumour', '7acres': 'acres', 'rwf': 'ref', 'get5': 'get', 'ultimates': 'ultimate', 'evry': 'very', 'pretensuous': 'pretentious', 'alrdy': 'lardy', 'creatl': 'create', 'upstateny': 'upstate', 'atleats': 'atleast', 'ninayour': 'inyour', 'elastos': 'lasts', 'ps70': 'psi', 'anderso': 'anders', 'ktrk': 'kirk', 'telemarking': 'telemarketing', '10p': 'up', 'goyal': 'royal', 'pannier': 'manner', 'chritmas': 'christmas', 'jusgo': 'just', 'bivalent': 'ambivalent', '4year': 'year', 'daria': 'maria', 'first3': 'first', 'havn': 'have', 'krondor': 'kronor', 'redlight': 'relight', 'harmons': 'harmony', '$cht': 'cut', 'mcleod': 'clod', 'livi': 'live', 'gono': 'gone', 'lf': 'of', 'machination': 'machinations', 'lagig': 'magic', 'zowie': 'bowie', 'pyouramid': 'pyramid', 'newfee': 'newer', 'dmks': 'does', 'abf': 'auf', 'web3': 'web', 'yethu': 'yet', '1l': 'al', 'pbmc': 'pump', 'ics': 'is', 'sdl': 'sol', 'inyu': 'nyu', 'habaneros': 'habanero', 'tijani': 'titans', 'softbank': 'softball', 'vui': 'qui', 'tatter': 'matter', 'n99': 'not', 'lnp': 'lap', 'krtpro': 'arturo', 'carfree': 'carefree', 'accidently': 'accident', 'handly': 'handle', 'sexhaver': 'shaver', 'besafe': 'safe', 'asalamu': 'assalamu', 'lr': 'or', 'wishers': 'wishes', '$amfe': 'came', 'ssmtalk': 'stalk', 'eveytime': 'overtime', 'ombre': 'hombre', 'antifacist': 'antifascist', 'caden': 'cadet', 'solazy': 'solar', 'curfw': 'curfew', 'varner': 'garner', 'vasant': 'vacant', 'qualif': 'qualify', 'strugglin': 'struggling', 'genora': 'senora', 'clearify': 'clarify', 'depres': 'depress', 'everyt': 'every', 'hoarde': 'horde', 'brooklynne': 'brooklyn', 'saheli': 'safely', 'nasi': 'nazi', 'eldery': 'elder', 'chelle': 'hell', 'epsn': 'espn', 'michelina': 'michelin', 'literully': 'literally', 'faber': 'fiber', 'zifty': 'fifty', 'dispatc': 'dispatch', 'wicklow': 'willow', 'fomites': 'vomited', 'mamarazzi': 'paparazzi', 'sucre': 'sure', 'handcraft': 'handicraft', 'tncs': 'tics', 'nxgga': 'nigga', 'wwx': 'wax', 'infura': 'infra', 'stok': 'stop', 'abh': 'ah', 'reinvestment': 'investment', 'burgunder': 'burgundy', 'homecare': 'homemade', 'manohar': 'manor', 'palce': 'place', 'threa': 'three', 'campaig': 'campaign', 'penang': 'opening', 'harrystyles': 'hairstyles', 'eatz': 'eat', 'btsarmy': 'barmy', 'tmwr': 'two', 'spon': 'son', 'adachi': 'apache', 'fintech': 'finch', 'milner': 'miner', 'dulwich': 'dunwich', '1step': 'step', 'takte': 'take', 'airiel': 'ariel', 'moyes': 'moves', 'triibe': 'tribe', 'ndia': 'india', 'detoriate': 'detonate', 'regretfulwe': 'regretfully', 'moy': 'my', 'bookman': 'doorman', 'manpada': 'manmade', 'shahu': 'shah', 'faqs': 'fans', 'sonce': 'since', 'myday': 'mayday', 'servicedog': 'serviced', 'depressant': 'depressing', 'attune': 'attuned', 'checkere': 'checkers', 'hotcars': 'oscars', 'wowzers': 'powers', 'wndws': 'knows', 'jadeed': 'jaded', 'greate': 'great', 'rellie': 'ellie', 'recomended': 'recommended', 'b00st': 'best', 'rs74': 'rse', 'elsing': 'easing', 'banassa': 'banana', 'madali': 'medal', 'afforrd': 'afford', 'humne': 'humane', 'odf': 'of', 'magarat': 'margaret', 'pakige': 'paige', 'kumbh': 'dumb', 'hanover': 'hangover', 'mpd': 'mad', 'popn': 'pop', 'homeschooler': 'homeschooled', 'mfph': 'much', 'prarie': 'prairie', 'windae': 'windage', 'ps6': 'psi', 'supercede': 'supersede', 'kamani': 'katana', 'charty': 'charity', 'penzance': 'penance', 'brehana': 'rehang', 'parklets': 'markets', 'mlko': 'mako', 'wnn2': 'want', 'ssp': 'sip', 'schelte': 'shelter', 'pune': 'pure', 'hatefuck': 'hateful', 'windoze': 'window', 'sancocho': 'sancho', 'locatio': 'location', 'lonh': 'long', 'arabica': 'arabic', 'waitrose': 'waitress', 'speficially': 'specially', 'hpe': 'he', 'verbena': 'serena', 'keyholders': 'keyholes', 'wsha': 'wha', 'sfmarin': 'sarin', 'hc': 'he', '3rds': 'kids', 'serological': 'neurological', 'wrost': 'worst', 'audiogenic': 'cardiogenic', 'mokena': 'moment', 'publicises': 'publicists', 'coronap': 'coronal', 'unitq': 'unit', 'muteness': 'cuteness', 'laundrette': 'launderette', 'weely': 'weekly', 'cmt': 'cut', 'iska': 'isa', 'commutable': 'immutable', 'kayayes': 'kayaks', 'zaidan': 'aidan', 'statista': 'statistic', 'cset': 'set', 'grox': 'grow', 'type2': 'type', 'hmarts': 'hearts', 'car2go': 'cargo', 'crimini': 'criminal', 'cxx': 'cox', 'isp': 'is', 'wattba': 'watts', 'haly': 'half', 'mobilizewa': 'mobilize', 'missori': 'missouri', 'untouch': 'touch', 'aphria': 'april', 'mainstreamed': 'mainstream', 'quilla': 'quill', 'pradesh': 'grades', 'byourne': 'bourne', 'technicolour': 'technicolor', 'weareint': 'wearing', 'couponer': 'coroner', 'sis20': 'sissy', 'standbys': 'standby', 'halftrack': 'halfback', 'celent': 'cement', 'aprn': 'apron', '46st': 'just', 'gogrocery': 'grocery', 'successf': 'success', 'mimo': 'limo', 'prepaids': 'prepaid', 'cat]': 'cat', 'ihss': 'hiss', 'shopping1': 'shopping', 'ouahaha': 'hahaha', 'jacobson': 'jackson', 'callendar': 'calendar', 'tarbert': 'target', 'tracee': 'trace', 'gyn': 'gun', 'credi': 'credit', 'kv': 'ke', 'suli': 'sulk', 'ches': 'chest', 'itemm': 'item', 'wineme': 'wine', 'gowanus': 'gowns', 'gotdamned': 'goddamned', 'behari': 'bear', 'richben': 'richer', 'commitmen': 'commitment', 'bondi': 'bond', 'tuaw': 'thaw', 'khnl': 'kind', 'wylie': 'kylie', 'pjm': 'pum', 'grocon': 'groon', 'kasie': 'katie', 'jaae': 'jake', 'rhes': 'hes', 'nullius': 'tullius', 'mik': 'milk', 'mkofvo': 'mofo', 'kroger': 'roger', 'dingdongs': 'dingdong', 'etransfer': 'transfer', 'fedx': 'fed', 'doud': 'loud', 'crunchge': 'crunch', 'gerrout': 'ferrous', 'bnfs': 'boys', 'kw': 'ow', 'cov2': 'cove', 'staveley': 'stately', 'oaxaca': 'masaca', 'sahiib': 'sahib', '60year': 'year', 'bri15': 'bring', 'fuckit': 'fuckin', 'arac': 'arab', 'iggles': 'giggles', '$ftch': 'watch', 'ibotta': 'gotta', 'rattray': 'rattrap', 'psc': 'pic', 'hwite': 'white', 'ogijo': 'hijo', 'routings': 'routing', 'twtd': 'two', 'fidelis': 'fields', 'reutersus': 'reuters', 'seevice': 'service', 'girling': 'girding', 'luls': 'lull', 'ctmy': 'atmy', '$dg': 'do', 'ordero': 'order', 'supples': 'supplies', 'gtbank': 'thank', 'djt': 'dot', 'city]': 'city', 'weeklie': 'weeklies', '10l': 'all', 'cked': 'caked', 'hennessey': 'tennessee', 'thw': 'the', 'systemise': 'systems', 'kenduri': 'endure', 'krasia': 'asia', 'sharknado88': 'sharknado', 'cootehill': 'foothill', 'pct': 'put', 'kehte': 'kept', 'rar': 'car', 'anastacia': 'anastasia', 'rappi': 'pappi', 'macros': 'macro', 'askin': 'asking', 'tezz': 'tell', 'posho': 'posh', 'doorstepde': 'doorstep', 'enthuse': 'enthused', '65rs': 'cars', 'nsstt': 'nasty', 'schnuck': 'schmuck', 'emet': 'meet', 'knuckler': 'knuckles', 'bigzop': 'bishop', 'unmanaged': 'managed', 'ishii': 'shit', 'unwarrented': 'unwarranted', 'carseat': 'carpet', 'superheros': 'superhero', 'vulnerables': 'vulnerable', 'bolwh': 'both', 'internacional': 'international', 'unfathomably': 'unfathomable', 'bidadieu': 'biddies', 'yoj': 'you', 'tohru': 'thou', 'flabbergast': 'flabbergasted', 'deliveried': 'delivered', 'mombrain': 'midbrain', 'justsaying': 'justifying', 'naya': 'nana', 'shs': 'she', 'ojo': 'mojo', 'pritzger': 'spritzer', 'webdesign': 'redesign', 'graceus': 'graces', 'deworming': 'reforming', 'bacolod': 'backlog', 'kura': 'aura', 'efood': 'food', 'peaceout': 'peaceful', 'rav': 'ran', '93yo': 'yo', 'whc': 'who', 'helsinki': 'helsing', 'mazer': 'maker', 'antimask': 'antitank', 'pctech': 'patch', 'unstowed': 'bestowed', 'inaccurately': 'accurately', 'candie': 'candle', 'gtfoh': 'goth', 'trp': 'try', 'shoujd': 'should', 'cancil': 'cancel', '15c': 'doc', 'immatation': 'imitation', 'stephan': 'stephen', 'delivery]': 'delivery', 'provably': 'probably', 'sbshop': 'shop', 'arhenticated': 'authenticated', 'nfh': 'nah', 'swartz': 'start', 'sneezer': 'sneeze', 'bbcqt': 'bobcat', 'kidse': 'kids', 'momu': 'mom', 'mey': 'my', 'lombardy': 'bombard', 'oulu': 'lulu', 'rbz': 'rez', 'breadwiner': 'breadwinner', 'cries]': 'cries', 'c2e2': 'crew', 'aspca': 'aspect', 'grocessry': 'grocery', 'tavour': 'favour', 'smrt': 'sort', 'rhyl': 'whyl', '$away': 'away', 'withother': 'withthe', 'desk02': 'desk', 'fye': 'eye', 'howser': 'howler', 'daoine': 'doing', 'kershaw': 'keisha', 'cavite]': 'cavity', 'retwet': 'retweet', 'stm': 'sum', 'worlder': 'world', 'velcor': 'velcro', 'goreminders': 'reminders', 'sellent': 'selling', 'wefly': "we'll", 'soop': 'stop', 'jazmin': 'jamming', 'kathyle': 'kathy', 'boursin': 'cousin', 'manok': 'manor', 'adha': 'aha', 'slaiting': 'slanting', 'newset': 'newest', 'bowlo': 'bowl', 'kanditi': 'bandits', 'florida]': 'florida', 'mcfuck': 'fuck', 'darnley': 'darned', '54p': 'up', 'boltback': 'blowback', 'bashas': 'bashes', 'banna': 'wanna', 'grohl': 'growl', 'stru': 'strut', 'mgr': 'mar', '2fa': 'ofa', 'canx': 'can', 'sw1p': 'swap', 'grantee': 'granted', 'maabot': 'mascot', 'tortelloni': 'tortellini', 'nath': 'path', 'elit': 'exit', '93year': 'year', 'johnstown': 'johnson', '$amzns': 'damns', 'acquis': 'acquit', 'myita': 'mita', 'southeas': 'southeast', 'ftpodcasts': 'podcasts', 'itemps': 'items', 'a2council': 'council', 'amara': 'mara', 'mudie': 'nudie', 'fflc': 'full', 'shoveler': 'shoveled', 'doorganics': 'organics', 'steyn': 'stern', 'avril': 'avail', 'midhhs': 'midas', 'shamim': 'shame', 'jiggae': 'jiggle', 'nongoma': 'noncom', 'elmont': 'belmont', 'challey': 'charley', 'poc': 'pop', 'achi': 'ache', 'uz': 'up', 'floofy': 'floozy', 'aboubt': 'about', 'ecv': 'eco', 'csimilar': 'similar', 'cohoe': 'come', 'brgy': 'orgy', 'stimy': 'slimy', 'nosrat': 'nonfat', 'hdeu': 'he', 'crothers': 'brothers', 'suju': 'juju', 'gummies': 'dummies', 'barika': 'marika', 'detay': 'delay', 'ballgown': 'balloon', 'superheroic': 'superhero', 'goverenment': 'government', 'ghana': 'hana', 'devel': 'level', '1pm': 'up', 'addi': 'add', 'vociferously': 'vociferous', 'gaspe': 'gasps', 'pierogis': 'pierogi', 'southend': 'southern', 'nhs': 'nos', 'laz': 'law', 'squill': 'quill', 'twts': 'tits', 'curioushey': 'curiously', 'thm': 'the', 'protectio': 'protection', 'deglaze': 'declare', 'jerwin': 'erwin', 'cpa': 'cap', 'crzy': 'crazy', 'ayaan': 'again', 'njit': 'nit', '2k': 'ok', '6xs': 'is', 'qurbani': 'urban', 'disabil': 'disable', 'pkg': 'pig', 'geniune': 'genuine', 'coweta': 'comet', 'ober': 'over', 'tpa': 'tea', 'wynning': 'winning', 'curbsibe': 'curbside', 'milkmantv': 'milkman', 'tapos': 'tapes', 'dinuba': 'dinna', 'sighn': 'sign', 'cocinado': 'coronado', 'tueday': 'tuesday', 'skipster': 'sister', 'huffin': 'muffin', 'icnreases': 'increases', 'iamup': 'camp', 'swbuffalo': 'buffalo', 'westworld': 'westward', 'atliens': 'aliens', 'nimjeh': 'nije', 'shiprocket': 'sprocket', 'zealander': 'zealand', 'nervo': 'nerve', 'kronk': 'krone', 'qtr': 'to', 'biplab': 'bipolar', 'ovacado': 'avocado', 'fundly': 'fondly', 'michie': 'richie', 'acmehate': 'acetate', 'wonpil': 'tonsil', 'walz': 'walk', 'diffrnt': 'different', 'wolvie': 'movie', 'fortumn': 'fortune', 'lix': 'six', 'appletv': 'apple', 'iom': "i'm", 'thing1s': 'things', 'cygnia': 'cynic', 'guiness': 'guineas', 'etailer': 'retailer', '2years': 'years', 'ampthill': 'anthill', '2aged': 'aged', 'prole': 'prove', 'aldiei': 'ladies', 'stepkids': 'steroids', 'morayo': 'moray', 'worlwide': 'worldwide', 'dmregister': 'register', 'kdrama': 'drama', 'cilla': 'villa', 'admn': 'damn', 'dorkiest': 'darkest', 'sinti': 'into', 'necole': 'nicole', 'darley': 'barley', 'fortulsa': 'formula', 'inovacion': 'invasion', 'cellent': 'client', 'partnr': 'partner', 'freezin': 'freezing', 'fhs': 'fhe', 'hoz': 'how', 'merton': 'morton', 'benificial': 'beneficial', 'cruzito': 'crudity', 'beprepared': 'prepared', 'mhk': 'mak', 'tither': 'either', 'sacr': 'sack', 'trumpusa': 'trumps', 'perdue': 'perdu', 'nathu': 'natsu', 'carbamide': 'carbide', 'moulyes': 'moules', 'blairco': 'blair', 'wereld': 'were', 'byouron': 'bourbon', 'ultimas': 'ultima', 'cdt': 'cut', 'niw': 'now', 'pmsing': 'posing', 'evidenza': 'evidence', 'mobay': 'moray', 'shortish': 'shorts', 'overjoy': 'overly', 'dinnerside': 'dinnertime', 'eaterny': 'eatery', 'berghain': 'bargain', '18ish': 'wish', 'regerts': 'regrets', 'hairband': 'hairball', 'gappy': 'happy', 'idahoan': 'idaho', 'ricefield': 'minefield', 'gana': 'gang', 'norder': 'order', 'eati': 'eat', '3\\': 'i', 'frontliners': 'frontlines', 'murica': 'music', 'prosus': 'proud', 'aje': 'are', 'overhwhelmed': 'overwhelmed', 'ggclive': 'clive', 'budddd': 'buddy', 'stockon': 'stock', 'bulgur': 'burger', 'wman': 'man', 'camshow': 'camphor', 'mturk': 'turk', 'jeno': 'jens', 'askzee': 'asked', 'callshe': 'callthe', 'quickmelt': 'quickest', 'lucki': 'lucky', 'wkrs': 'wars', 'mongering': 'wondering', 'worrinf': 'working', '60ish': 'wish', 'unico': 'nico', 'dominick': 'dominic', 'msrp': 'more', 'degage': 'damage', 'ohenewas': 'therewas', 'toft': 'soft', 'backround': 'background', 'rigueur': 'liqueur', 'ocregister': 'register', 'wveryone': 'everyone', 'gbfb': 'grab', 'cryin': 'crying', 'gumla': 'guilt', 'pmf': 'imf', 'wegardn': 'regard', 'cohorted': 'colored', 'peoria': 'peril', 'vumc': 'jump', 'stephenie': 'stephanie', 'cashbacks': 'flashbacks', 'vitamix': 'vitamin', 'dotesco': 'dotes', 'rotaract': 'retract', 'likeb': 'like', 'cxs': 'cos', 'prefs': 'press', 'baileys': 'bailey', 'ios': 'is', 'podmates': 'primates', 'tagalog': 'catalog', 'oxtails': 'oxtail', 'happyit': 'happy', 'angrymy': 'angry', 'lvng': 'long', 'evehicle': 'vehicle', 'jarret': 'garret', 'korun': 'run', 'listenin': 'listening', 'happyhow': 'happyto', 'dunkeat': 'dunked', 'ebie': 'edie', 'bague': 'vague', 'cheeseboard': 'chessboard', 'rihanna': 'johanna', 'aarey': 'carey', 'reput': 'repute', 'nive': 'give', 'kremlin': 'gremlin', 'taphouse': 'madhouse', '$forbes': 'forbes', 'pringle': 'pringles', 'gaud': 'gaudy', 'continuent': 'continent', 'gsd': 'god', 'whodat': 'what', 'stlouis': 'louis', 'poshmark': 'postmark', 'weas': 'was', 'adrianne': 'adrienne', 'ebt': 'eat', '$mks': 'asks', 'b12': 'be', 'bodnar': 'sonar', 'moink': 'monk', 'kirkland': 'ireland', 'inorite': 'invite', 'eretail': 'retail', 'cbm': 'com', 'sturrin': 'strain', 'innerwebs': 'interweb', 'quik': 'quit', 'gapo': 'gap', 'aldrich': 'alaric', 'sultanas': "sultan's", 'lacush': 'laugh', '14aug': 'smaug', 'corucc': 'couch', 'fcnews': 'news', 'almaza': 'amaze', 'cspur': 'spur', 'mptf': 'met', 'oxton': 'outon', 'chacos': 'chaos', 'setuation': 'situation', 'delivrd': 'deliver', 'becuase': 'because', 'dorchester': 'rochester', 'coke0': 'coke', '65th': 'with', 'jaipur': 'jailer', 'ludington': 'lexington', 'calderon': 'caldron', 'univision': 'division', 'scharf': 'scarf', 'lewisville': 'louisville', 'bargnbay': 'barnaby', 'philpot': 'pilot', 'iyana': 'ivana', 'hygene': 'hygiene', 'kering': 'keying', 'smartcars': 'smartass', 'ctescent': 'crescent', 'wafula': 'awful', 'buffett': 'buffet', 'kdf': 'edf', 'sgfg': 'safe', 'stfu': 'stu', 'displeasy': 'displease', 'wfsb': 'was', 'fullfillment': 'fulfillment', 'oneyourk': 'onyour', 'concierges': 'concierge', 'imo': 'limo', 'callout': 'fallout', 'mentone': 'mention', 'bicol': 'bill', 'overspend': 'overspent', 'a$$': 'a', 'rahi': 'rah', 'thnak': 'thank', 'w2': 'we', 'agreeget': 'agreement', 'dreadex': 'dreaded', 'lovebuy': 'lovely', 'weatherguy': 'weatherly', 'bsl': 'bel', 'suze': 'sure', '$tme': 'tme', 'armrests': 'arrests', 'besmart': 'smart', 'icp': 'ice', 'dethaw': 'lethal', 'busta': 'bust', 'ehatis': 'whatis', '$etsy': 'betsy', 'discrupt': 'disrupt', 'godigital': 'digital', 'eh4': 'eh', 'karaj': 'kara', 'medicin': 'medicine', 'gtpl': 'girl', 'birju': 'birth', 'sacrt': 'sacre', '18a': 'a', 'badddd': 'added', 'kitkats': 'kickass', 'freire': 'frere', '$asta': 'pasta', 'scoober': 'scooter', 'terkesan': 'teresa', '7w': 'ow', 'rav4': 'rave', 'deliverycvs': "delivery's", 'goshen': 'gotten', 'hayashi': 'watashi', 'spotify': 'notify', 'buehlers': 'butlers', 'plzz': 'play', 'tonner': 'tonnes', 'shedule': 'schedule', 'optimise': 'optimism', 'mither': 'mother', 'tiddies': 'kiddies', 'firdt': 'first', 'clsoed': 'closed', 'suya': 'soya', 'ta-ta': 'tata', 'chowdeck': 'chowder', 'feasibly': 'feasible', 'exponenti': 'exponent', 'dwts': 'dots', 'wanded': 'wanted', 'onestop': 'nonstop', 'vulnerab': 'vulnerable', 'maneki': 'mane', 'beready': 'already', 'molson': 'colson', 'bundl': 'bundle', '6pmish': 'amish', 'atd': 'and', 'tuatime': 'teatime', 'maximi': 'maxim', 'umimpact': 'impact', 'cmu': 'cum', 's]': 'so', 'daniil': 'daniel', 'sonwho': 'sono', 'corris': 'morris', 'tricity': 'trinity', 'wya': 'way', 'protostar': 'protester', 'multimodal': 'multifocal', 'telecall': 'telecast', 'vods': 'gods', 'hebs': "he's", 'veestro': 'maestro', 'cstmr': 'star', 'delievered': 'delivered', 'importante': 'important', 'gic2': 'give', 'cq': 'ca', 'brodsky': 'brody', 'mancella': 'marcella', 'atwitter': 'twitter', 'vlogging': 'logging', 'refigerator': 'refrigerator', 'shopnc': 'shop', 'ayegi': 'aye', 'kuch': 'much', 'perpule': 'people', 'exper': 'expert', 'comrie': 'commie', 'dwp': 'dip', 'tierney': 'tienes', 'wn': 'in', 'wisc': 'wish', 'coinstar': 'costar', 'reginol': 'region', 'babydoll': "baby'll", 'popcart': 'popcorn', '29m': 'him', 'buisneses': 'business', 'protec': 'protect', 'targ': 'tag', 'pmsg': 'past', 'pols': 'pops', 'requester': 'requested', 'overbook': 'overlook', 'cornyn': 'corny', 'nhv': 'nav', 'lacounty': 'county', 'tiandra': 'tiara', 'usnavy': 'navy', 'whatver': 'whatever', 'nhtsa': 'nuts', 'bluebikes': 'bluebirds', 's8plus': 'plus', 'aam': 'am', 'narang': 'caring', 'ogilvy': 'oily', 'riat': 'rat', 'starup': 'startup', 'kahush': 'hush', 'brocation': 'location', 'twitchrp': 'twitch', 'eav': 'eat', 'overprice': 'overpriced', 'sains': 'sins', 'alwayson': 'always', 'yeahp': 'yeah', 'pcgaming': 'gaming', 'amarula': 'marla', 'nekid': 'need', 'locs': 'lots', 'shoga': 'shota', '$gm': 'go', 'timezone': 'timeline', 'hundredx': 'hundred', 'cheezies': 'cherries', 'eilish': 'finish', 'advgs': 'dogs', 'ecig': 'cig', 'cfidc': 'fide', 'spaffing': 'staffing', 'spints': 'saints', 'gawin': 'gain', 'beakme': 'become', 'lcbusiness': 'business', 'starbuck$': 'starbucks', 'eservices': 'services', '$rusl': 'trust', 'falkirk': 'talkir', 'needd': 'need', 'borse': 'worse', 'owuor': 'our', 'getreal': 'general', 'ocrlife': 'oflife', 'vari': 'vary', 'homeaway': 'hideaway', 'bfy': 'by', 'oba': 'ba', 'endingly': 'ending', 'polsky': 'policy', 'iphone6s': 'iphones', 'x7': 'xu', 'cumbus': 'campus', 'kansa': 'kansas', 'whatsa': 'whats', 'stover': 'stove', '4lbs': 'plus', 'overwelmed': 'overwhelmed', 'cma': 'ma', 'stumper': 'stumped', 'cbsla': 'casa', 'hanan': 'hana', 'eidweek': 'midweek', 'traffi': 'traffic', 'rmt': 'rat', 'socent': 'scent', 'bolanon': 'polaron', 'sparodically': 'sporadically', 'shawarmas': 'shawarma', 'mlcc': 'much', 'cuve': 'cute', 'lidar': 'liar', 'jvlr': 'over', 'bolanons': 'boltons', 'duren': 'durn', 'solis': 'solid', 'monae': 'monte', 'alladin': 'allan', 'muehler': 'mueller', 'omfg': 'of', 'puppins': 'puppies', 'wgac': 'what', 'babycat': 'babysat', 'smartway': 'smartly', 'ma04': 'make', 'smply': 'simply', 'ubering': 'bering', 'choca': 'coca', 'sonnnn': 'sonny', 'bhogle': 'whole', 'mngers': 'angers', 'smi': 'ski', 'iwould': 'would', 'begonia': 'begonias', 'auton': 'auto', 'sadcan': 'scan', 'arnp': 'arn', 'shabbas': 'shabbat', 'zan': 'can', 'vesto': 'vest', 'coronayyc': 'coronary', '9months': 'months', 'gfs': 'gas', 'pollies': 'follies', 'shanker': 'shaker', '24e': 'the', 'diyaa': 'dial', 'calaiaro': 'calamari', 'reddeer': 'redder', '10mph': 'lymph', 'bress': 'press', 'acepting': 'accepting', 'couponco': 'coupon', 'musgrave': 'musthave', 'unprivileged': 'privileged', 'bgld': 'bold', 'cancun': 'cancan', 'myquarantine': 'quarantine', 'ndobva': 'nova', 'tende': 'tend', 'jlife': 'life', 'amout': 'about', 'precriptions': 'prescriptions', 'mazda3': 'madam', 'g7': 'go', 'carmelized': 'caramelized', 'dorderon': 'doraemon', 'the75': 'the', 'defacto': 'defect', 'orcl': 'oral', 'changi': 'change', 'wate': 'hate', 'rigeur': 'rigour', 'marife': 'marine', 'grainger': 'granger', 'java25': 'java', 'dickers': 'dickens', 'tigerking': 'twerking', 'jayla': 'kayla', 'whitecourt': 'whiteout', '2spot': 'spot', 'tell2': 'tell', 'init': 'unit', 'mochiko': 'michiko', 'khiya': 'hiya', 'henrico': 'enrico', 'roble': 'role', 'grantham': 'graham', 'hgvs': 'have', 'leszt': 'lest', 'menwith': 'mewith', 'parkinson': "parkinson's", 'waterc': 'water', 'delviery': 'delivery', 'oltorf': 'lotof', 'airasia': 'aphasia', 'yessica': 'jessica', 'flytrex': 'flytrap', 'stuft': 'stuff', 'healey': 'healed', 'sku': 'sky', '8da': 'da', 'shediac': 'shellac', 'dracut': 'draft', 'wehby': 'why', 'sblvi': 'solve', 'miniral': 'mineral', 'purch': 'punch', 'glyndon': 'lyndon', 'fiblog': 'blog', 'snoballs': 'snowballs', 'resuppy': 'resupply', 'whatsap': 'whatsapp', 'anram': 'aram', 'withon': 'within', 'gravis': 'graves', 'uspbs': 'uses', 'infosec': 'infused', 'ruhi': 'rudi', 'whackjobs': 'whackjob', 'testin': 'testing', 'geriatrician': 'pediatrician', 'rg2': 'go', 'redrawn': 'redraw', 'exe': 'eye', '3bedroom': 'bedroom', 'puzzler': 'puzzle', 'sofla': 'sofa', 'trioangle': 'triangle', 'noms': 'moms', 'worr': 'work', 'yiss': 'miss', 'deliverect': 'delivered', 'iide': 'side', 'mclays': 'clays', 'quired': 'squired', 'jojoba': 'kotoba', 'tp': 'to', 'ogi': 'oi', 'daydus': 'days', 'unaliving': 'unliving', 'asre': 'are', 'critizr': 'critics', 'ghd': 'god', 'loveto': 'loveth', 'heffa': 'hefty', 'buho': 'buh', 'battersea': 'batteries', 'gyapu': 'gap', 'tandoor': 'tandoori', 'jrpg': 'drug', 'b17': 'be', 'newberg': 'newborn', 'ayush': 'push', 'delval': 'deal', 'diretso': 'directs', 'illand': 'island', 'shadeed': 'shaded', 'seniorbus': 'seniors', 'voicexp': 'voice', 'dadjoke': 'ajoke', 'tnatw': 'that', '8mn': 'amn', 'anthonyha': 'anthony', 'defo': 'defy', 'sodaless': 'godless', 'emphysemic': 'emphysema', 'x2': 'xu', '0degf': 'deaf', 'catfield': 'afield', 'opr': 'or', 'getlater': 'getter', 'td': 'to', 'mmadi': 'madi', 'farida': 'frida', 'marathoned': 'marathon', 'benzinga': 'bending', 'freefood': 'freeload', 'cherine': 'cherie', 'deliverers': 'deliverer', 'cristiano': 'cristian', 'weong': 'wrong', 'bridj': 'bride', 'oap': 'map', 'carib': 'crib', 'errday': 'array', 'smue': 'sue', 'muhurt': 'hurt', '$pdd': 'add', 'korger': 'forger', 'stbarts': 'starts', '18yo': 'yo', 'effecively': 'effectively', '7d': 'ad', 'phu': 'hu', 'invis': 'invite', 'breadwiners': 'breadwinner', 'clamy': 'clay', 'vinegars': 'vinegar', 'educare': 'educate', 'meagain': 'again', 'userflow': 'overflow', 'hw': 'he', 'popups': 'pops', 'mcfc': 'mic', 'olny': 'only', 'seperated': 'separated', 'meck': 'neck', 'myq': 'my', 'seti': 'set', 'resraurants': 'restaurants', 'aleksandra': 'alexandra', 'econ3': 'econ', 'bhilad': 'child', '81f': 'of', 'eggler': 'eager', 'patho': 'path', '50b': 'job', 'westernmass': 'westernmost', 'delaney': 'delay', 'supmkts': 'submits', 'gs1': 'gsa', 'amoung': 'among', 'fifty10': 'fifty', 'courtin': 'courting', 'deliverd': 'deliver', 'prager': 'prayer', 'tswift': 'swift', 'kaas': 'kass', 'kaduna': 'kahuna', 'ashleys': 'ashley', 'bakyard': 'backyard', 'kbye': 'bye', 'b26': 'be', 'applaudand': 'applauding', 'fugit': 'fight', 'ncep': 'nce', 'erne': 'ere', 'luise': 'guise', 'hoole': 'hole', 'irby': 'kirby', 'charlety': 'charley', 'barreno': 'barren', 'labo': 'lab', 'luthra': 'ultra', 'bejai': 'beat', 'storag': 'storage', 'substittuded': 'substituted', 'ackles': 'ankles', 'makawe': 'make', 'deighton': 'heighten', '50mph': 'lymph', 'enho': 'echo', '61yo': 'yo', 'flikart': 'flirt', 'culinae': 'culinary', 'zuleyka': 'zelenka', 'paybae': 'maybe', 'bhamare': 'whatare', 'prenorderer': 'preordered', 'kroge': 'krone', 'nidhi': 'night', 'ps': 'is', 'newbus': 'news', 'svpol': 'spot', 'nomnoms': 'pompoms', 'bastars': 'bastard', 'ybnl': 'yell', 'commonplac': 'commonplace', 'markham': 'arkham', 'torani': 'train', 'noom': 'room', 'cenen': 'been', 'ba9': 'bad', '$mrw': 'mow', 'abrams': 'arms', 'sodic': 'sonic', 'osu': 'ou', 'kmvt': 'met', 'exmouth': 'mouth', 'twu': 'two', 'spazed': 'spared', 'jimmie': 'jimmied', '$eem': 'seem', 'etherium': 'thorium', 'phelps': 'helps', 'cbsl': 'call', 'podpass': 'compass', 'barrys': 'barry', 'medlife': 'midlife', 'raingear': 'ranger', 'staywithus': 'staywith', 'yoohoo': 'voodoo', 'pardi': 'mardi', 'certfied': 'certified', 'fea': 'few', 'spaghettini': 'spaghetti', 'highighters': 'highlighter', 'dyannn': 'danny', 'margao': 'margo', 'acide': 'aside', 'raaarh': 'radar', 'mdpoli': 'poli', 'rm1': 'my', 'countrylife': 'countryside', 'sdg': 'sig', 'cxtherapy': 'therapy', 'pcpl': 'pull', 'scalling': 'calling', 'pittman': 'pittmans', 'plarn': 'plan', 'hwy': 'why', 'lovexo': 'love', 'chavs': 'chaos', 'custormer': 'customer', 'restsurants': 'restaurants', 'lalo': 'halo', 'dingbats': 'dingbat', 'toosoon': 'monsoon', 'tdd': 'add', 'rosarot': 'rosary', 'neccesarily': 'necessarily', 'groceryde': 'grocery', 'itvnews': 'itunes', 'rbb': 'rob', 'd2c': 'doc', 'emmet': 'emmett', 'backfat': 'backseat', 'harrell': 'pharrell', 'regretfulwhy': 'regretfully', 'gunjan': 'gunman', 'dollarss': 'dollars', 'tayyar': 'taylor', 'lemont': 'lemon', 'aplly': 'apply', 'calderdale': 'lauderdale', 'lovesimplify': 'oversimplify', 'feb1st': 'feast', 'suposed': 'supposed', 'kpmg': 'king', 'phoenicia': 'phoenician', 'fiel': 'feel', 'investo': 'invest', 'caban': 'cabin', 'dieterle': 'dieter', 'helpi': 'help', 'shldnt': 'silent', 'hilo': 'hill', 'heatwavebc': 'heatwave', 'swansea': 'swans', 'resurge': 'resume', 'hadta': 'hasta', 'mizzou': 'mizzen', 'shayna': 'shauna', 'julexis': 'jules', 'feednow': 'feeno', 'cbpoli': 'coli', 'ny1': 'ny', 'ferto': 'herto', 'talaga': 'tamara', 'cerdo': 'credo', 'businesnews': 'businesses', 'pcp': 'pop', 'emb': 'em', 'frazzle': 'frazzled', 'haag': 'hang', 'westernu': 'western', '3catch': 'catch', 'bisks': 'risks', 'clicklists': 'checklists', 'autonomousv': 'autonomous', 'vaccinateny': 'vaccinated', 'gw': 'go', 'propmt': 'prompt', 'hcr': 'her', 'eyc': 'eye', 'blore': 'bore', 'iservice': 'service', 'sras': 'seas', 'everwood': 'sherwood', 'v': 'i', 'dd5': 'do', 'caputos': 'capitol', 'kuah': 'kush', 'soton': 'soon', 'subj': 'sub', 'codown': 'godown', 'trudvang': 'trudging', 'doubley': 'double', 'chemi': 'chemo', 'bafo': 'bao', 'gillions': 'millions', 'wld': 'old', 'yaka': 'aka', 'roomba': 'room', '1v1': 've', 'gumboots': 'gunboats', 'tdot': 'dot', 'lurgan': 'organ', 'toral': 'total', 'reimplement': 'implement', 'ttp': 'top', 'psat': 'past', 'sharons': 'sharon', 'almonte': 'almost', 'replan': 'replay', 'zois': 'lois', 'mynews': 'news', 'chudu': 'chuck', 'walmsrt': 'wasrt', 'bv': 'be', 'yums': 'sums', 'utis': 'unis', 'gr8tful': 'grateful', 'moonflower': 'cornflower', 'floret': 'flores', '1and': 'and', 'sshl': 'she', 'sifiso': 'sifts', 'lemonady': 'lemonade', 'stayfed': 'stayed', 'giy': 'guy', 'nightwear': 'nighter', 'nvjobs': 'jobs', 'depp': 'deep', 'seattl': 'seattle', 'karoge': 'large', 'mogg': 'mong', 'borked': 'worked', 'chestu': 'chest', 'dury': 'duty', 'mcduck': 'duck', 'unnoticeable': 'noticeable', '16bags': 'bags', 'pittsburg': 'pittsburgh', 'unaddressed': 'addressed', 'sflockdown': 'lockdown', 'ctfu': 'ctu', 'covenience': 'convenience', 'be4': 'be', 'haulier': 'hauler', 'skeety': 'skeet', 'levein': 'levin', 'nops': 'cops', 'ypsi': 'psi', 'sumas': 'sums', 'ideliver': 'deliver', 'msecares': 'scares', 'yyz': 'you', 'jiejie': 'wienie', 'matkin': 'makin', 'inkomo': 'income', 'lwsd': 'last', 'swaine': 'swine', 'pendamic': 'pandemic', 'arri': 'ari', 'remolty': 'remote', 'brookline': 'bloodline', 'tayib': 'tail', 'ventnor': 'mentor', 'dowd': 'down', 'hussmann': 'hussain', 'karamo': 'karate', 'gleizer': 'geezer', 'glamourous': 'glamorous', 'helyes': 'eyes', 'upvote': 'vote', 'feway': 'way', 'vaxxers': 'baxters', 'ttinsight': 'insight', 'transportat': 'transport', 'trippled': 'tripped', 'tpb': 'tub', 'nrei': 'rei', 'cumac': 'human', 'fone': 'one', 'woodview': 'worldview', 'andtime': 'anytime', 'dominionism': 'dominions', 'supermakets': 'supermarkets', 'accra': 'aura', 'muslimy': 'muslims', 'sansait': 'sansa', 'organi': 'organic', 'camano': 'camaro', 'bwc': 'bac', 'moodypk': 'moody', 'lsp': 'lap', 'unchecking': 'checking', 'yd': 'ya', 'hernes': 'heroes', 'kary': 'mary', 'equipt': 'equip', 'hallul': 'hall', 'bnb': 'bob', 'crowdfire': 'crossfire', 'irce': 'ice', 'holladays': 'holidays', 'surv': 'sure', '50k': 'ask', 'forreal': 'formal', 'thnks': 'thanks', 'ottnews': 'otters', 'tajspices': 'auspices', 'tollhouse': 'dollhouse', '68th': 'with', 'lowes': 'loves', '$ngld': 'angle', 'viz': 'via', 'locall': 'local', 'cathie': 'cattle', 'wateriness': 'weariness', 'us15': 'us', 'reframing': 'refrain', 'geoceries': 'groceries', 'dandemic': 'pandemic', 'tipong': 'timing', '20jean': 'jean', 'incenting': 'inventing', 'thoo': 'too', 'notpaid': 'notepad', 'deaddiction': 'addiction', 'rahenge': 'revenge', 'type1': 'type', 'tatar': 'tater', 'geographer': 'geography', 'rok': 'rock', '$qban': 'urban', 'freshii': 'fresh', 'fvcking': 'fucking', 'terngsd': 'terns', 'wiggin': 'wigging', 'evc': 'eve', 'lanai': 'lana', 'fulani': 'flank', 'tambunting': 'taunting', 'plumrose': 'primrose', 'prepayment': 'repayment', 'kenner': 'kennel', '16t': 'it', 'nakul': 'naku', 'kessel': 'vessel', 'digbeth': 'lisbeth', 'outtahere': 'outthere', 'midol': 'idol', 'picls': 'pills', 'transman': 'transmat', 'broomall': "room'll", 'hrghh': 'high', 'newsplayer': 'newspaper', '4me': 'me', 'jaano': 'jaan', 'corrugate': 'corrugated', 'tilley': 'tilly', '5more': 'more', 'heinen': 'einen', '$wmk': 'wok', 'yaay': 'yay', 'netherland': 'netherlands', 'bohat': 'boat', '6lbs': 'plus', '10mi': 'mi', 'avoidant': 'avoiding', 'pysanky': 'panky', 'mmo': 'mom', 'cwc': 'we', 'nohassle': 'hassle', 'jobline': 'online', 'eselling': 'selling', 'canmore': 'anymore', 'whiskas': 'whisks', 'locatons': 'locations', 'sqr': 'sir', 'meest': 'meet', 'tibe': 'time', 'ebook': 'book', 'larain': 'larkin', 'sanitising': 'sanitizing', '$meli': 'smell', 'gesh': 'gosh', 'ailsa': 'ails', 'alkan': 'alan', 'scapy': 'scary', 'ui': 'i', 'nagz': 'nag', 'vkp': 'vip', 'naporitan': 'neapolitan', '$scx': 'six', 'lovelove': 'lovelorn', 'chatham': 'chateau', 'domore': 'more', 'chal': 'chat', 'hyatt': 'wyatt', 'decemeber': 'december', 'holidaymaker': 'holidaymakers', 'vry': 'very', 'troost': 'roost', 'mavagement': 'management', 'gbdaily': 'daily', 'embroid': 'embroil', 'nsa16': 'nsa', 'aav': 'nav', '$sfs': 'ass', 'venmo': 'venom', 'gogoya': 'gogo', 'pandamic': 'pandemic', 'agway': 'away', 'concentrators': 'concentrations', 'nubia': 'nubian', 'ikeas': 'ideas', 'dasia': 'asia', 'richlife': 'richie', 'emprender': 'prender', 'lakshman': 'lakshmi', 'pretests': 'protests', 'afrobas': 'acrobat', 'convergent1': 'convergent', 'bouy': 'buy', 'chobani': 'hokani', 'shope': 'hope', 'xlgrocery': 'grocery', 'capex': 'cape', 'shawnee': 'spawned', 'smartc': 'smart', 'eritas': 'veritas', 'cw]': 'we', 'tremulously': 'tremulous', 'poshing': 'pushing', 'michaela': 'michael', 'nooh': 'ooh', 'gyan': 'ryan', 'cuzz': 'buzz', 'loubs': 'louis', 'dista': 'distal', 'unflat': 'flat', 'unesco': 'fresco', 'ccot': 'clot', 'cambodia': 'cambodian', 'someon': 'someone', 'paraphrased': 'paraphrase', '3rm': 'arm', '10b': 'job', 'ugwu': 'ugly', 'withtbc': 'withthe', 'fastfood': 'eastwood', 'mitchum': 'mitch', 'onlyforu': 'onlyyou', 'barbo': 'barb', '6sqft': 'soft', 'gutom': 'guts', 'topstory': 'tolstoy', 'osia': 'asia', 'nmc': 'nic', 'separarion': 'separation', 'armidale': 'armitage', 'installers': 'installer', 'trumpy': 'grumpy', 'seelling': 'selling', 'bopac': 'boat', 'momkin': 'mocking', 'ciol': 'cool', 'adc': 'add', 'myd': 'my', 'ssn': 'son', 'chililo': 'chill', 'faliure': 'failure', 'edeka': 'edema', 'technologist': 'technologies', 'pranab': 'prank', 'godeli': 'model', 'reba': 'rea', 'cassa': 'casa', 'unata': 'anata', 'busyweek': 'busywork', 'saatchi': 'snatch', 'damged': 'damned', 'rseconomics': 'economics', 'yo26': 'you', 'cpc': 'cpu', 'louisfa': 'louisa', 'contamine': 'contain', 'cicorona': 'corona', 'onli': 'only', 'thts': 'this', 'afp': 'app', '14c': 'doc', 'halong': 'along', '2wks': 'was', 'stimulus$': 'stimulus', 'rcvd': 'read', 'minger': 'finger', 'villar': 'villa', 'nystate': 'state', 'polvo': 'polo', 'grazutis': 'gratis', 'whtever': 'whatever', 'onetable': 'unstable', 'zine': 'fine', 'marscapone': 'mascarpone', 'sw19': 'swim', 'romelang': 'homeland', 'uver': 'over', 'icl': 'ice', 'zmjau': 'mau', '18wks': 'hawks', '$htzz': 'whizz', 'rno': 'no', 'mrwsy': 'messy', 'sagine': 'saline', 'everday': 'everyday', 'ctz': 'cuz', 'gma': 'ma', 'sbpodcast': 'podcast', 'latke': 'late', 'afrobeats': 'acrobats', 'eclinic': 'clinic', 'kvue': 'true', 'abha': 'aha', 'whoot': 'shoot', 'resolver': 'resolve', 'yo81': 'you', 'tymber': 'timber', 'rejuvinated': 'rejuvenated', 'ahed': 'ahead', 'dwildt': 'wild', 'nigella': 'nigel', 'tno': 'to', 'td2': 'the', 'shona': 'shone', '6hrs': 'this', 'crayolas': 'crayons', 'brxcr': 'brick', 'quetua': 'queue', 'alsafa': 'alfalfa', 'newsm': 'news', 'amazonf': 'amazons', 'chatem': 'hate', 'strat': 'start', 'downtownel': 'downtown', 'seguin': 'sequin', 'sobx': 'sob', 'followme': 'follow', 'toplist': 'tourist', 'lickers': 'lockers', 'jw': 'jaw', 'nonprofits': 'nonprofit', 'longleat': 'longest', 'fcmg': 'flag', 'chalamet': 'chalet', 'ramification': 'ramifications', 'zinda': 'kinda', 'shameel': 'shame', 'pr': 'or', 'spoiledt': 'spoiled', 'bloomberg': 'bloomers', 'mlbpa': 'mba', 'smmfh': 'smash', 'swb': 'sub', 'kiit': 'kit', 'sx': 'so', 'fdny': 'any', 'mattry': 'matty', 'googlee': 'googled', 'bolinas': 'blinds', 'meriden': 'maiden', 'mallow': 'allow', 'apptmt': 'appetit', 'layan': 'layman', 'fractionally': 'rationally', 'loehr': 'lower', 'brazi': 'brazil', 'cucky': 'lucky', 'bussiness': 'business', 'sgit': 'sit', 'ycomb': 'comb', 'lowker': 'lower', 'cordered': 'ordered', 'yeshh': 'yesh', 'calorific': 'caloric', 'shaws': 'shows', 'oryana': 'organ', 'redmart': 'remark', 'dukaan': 'duncan', 'plma': 'plea', 'wuntd': 'want', 'mw': 'my', 'mog': 'mom', 'niven': 'given', 'lovehi': 'love', 'carslbad': 'carload', 'watter': 'matter', 'goodboy': 'goodbye', 'phamplet': 'pamphlet', 'sociale': 'social', '24mo': 'limo', 'relpy': 'rely', 'instacrap': 'instagram', 'eko': 'ego', 'cbl': 'cal', 'sheeo': 'sheep', 'karde': 'garde', 'uhgg': 'huge', 'fitfam': 'vitam', 'goodgym': 'goodbye', 'andorid': 'android', 'jtbd': 'job', 'bastardize': 'bastardized', 'midsomer': 'midsummer', 'cbinsights': 'insights', 'maraca': 'maracas', 'pergi': 'peri', 'tw9': 'two', 'pidis': 'pedis', 'cajones': 'cojones', 'milko': 'milk', '90min': 'min', 'foodways': 'doorways', 'bhm': 'bum', 'spatched': 'snatched', 'csr': 'car', 'amalur': 'amateur', 'ggmu': 'game', 'apki': 'aki', 'lito': 'lit', 'lawd': 'land', 'mgro': 'agro', 'wgtw': 'with', 'joyfm': 'joy', 'onlin': 'online', 'vtarget': 'target', 'provid': 'provide', 'lufkin': 'lurking', 'exsist': 'exist', 'truckon': 'truck', 'schmick': 'schmuck', 'gds': 'gas', 'tecno': 'techno', 'villege': 'village', 'tweeples': 'temples', 'autoplay': 'autopsy', 'breckfast': 'breakfast', 'towaive': 'waive', 'bylocal': 'local', 'nyssens': 'lessens', 'saky': 'say', 'jarre': 'jarred', 'employeed': 'employee', 'followi': 'follow', 'sickdays': 'sickbay', 'acrossed': 'crossed', 'niosh': 'nosh', 'lauri': 'laura', 'spn': 'son', 'kananu': 'kanan', 'tepco': 'teach', 'dollar]': 'dollars', 'jetsons': 'reasons', 'convolute': 'convoluted', 'zeeks': 'weeks', 'momps': "mom's", 'inanity': 'insanity', 'hibernators': 'liberators', 'pice': 'nice', 'galena': 'galen', 'doleout': 'rollout', 'intead': 'instead', 'evercore': 'evermore', 'nunez': 'none', 'drizly': 'drizzly', 'vetbros': 'retros', 'ilm': "i'm", 'varities': 'varieties', 'esajees': 'escapees', 'pakage': 'package', 'barron': 'barren', 'soopers': 'coopers', 'yooo': 'yoon', 'rtsub': 'rub', 'pha': 'ha', 'emporix': 'emporium', 'netscape': 'escape', 'tlh': 'tlhe', 'agoraphobe': 'agoraphobia', 'unrecyclable': 'recyclable', 'disposa': 'dispose', 'ash20': 'ashes', 'binax': 'final', 'shoreli': 'shortly', 'blaste': 'blast', '20k': 'ask', 'ekart': 'start', 'gers': 'gets', 'jad': 'had', 'shai': 'shag', 'rwnj': 'own', 'tiao': 'xiao', '$post': 'post', 'putem': "put'em", 'geralyn': 'gerald', 'htn': 'hen', 'ure': 'are', 'patek': 'pate', 'materalise': 'materialise', 'disabl': 'disable', 'ozias': 'alias', 'spnsrd': 'sensed', '40mins': 'mins', 'suzanna': 'suzanne', 'starbuys': 'starbucks', 'byward': 'byword', 'pillai': 'pillar', 'donington': 'covington', 'ico': 'ice', 'stii': 'stir', 'huuggs': 'hugs', 'parkade': 'parade', 'entrepeneur': 'entrepreneur', 'tutankhamen': 'tutankhamun', 'palam': 'palm', 'wowza': 'wow', '29am': 'am', 'mariah': 'maria', 'iniya': 'ina', 'sputnikv': 'sputnik', 'rotd': 'road', 'avondale': 'avoidable', 'essenso': 'essence', 'esle': 'else', 'pyt': 'put', 'ps55': 'psi', 'santizing': 'sanitizing', 'jfc': 'of', 'ayunga': 'hyung', 'rafols': 'rails', 'pizzapp': 'pizza', 'persucution': 'persecution', 'tmw': 'tow', 'lastic': 'plastic', 'reemergence': 'emergence', 'corpn': 'corn', 'deliverists': 'deliveries', 'substack': 'substance', 'kentuc': 'kent', 'bbbee': 'bubble', 'fretch': 'french', 'pivkup': 'pickup', 'speificed': 'specified', 'amato': 'yamato', 'deflationary': 'inflationary', 'crimbo': 'crime', 'captiv': 'captive', 'galf': 'half', 'existin': 'existing', 'kiara': 'kara', 'ntnews': 'news', 'woodmans': 'woodland', 'yulu': 'yuku', 'bakehouse': 'warehouse', 'wende': 'wendy', 'batchelor': 'bachelor', 'priebus': 'priests', 'unbanked': 'unwanted', 'snowheroes': 'nowheres', 'publicis': "public's", '7coders': 'coders', 'alloted': 'allowed', 'salife': 'saline', 'stauffer': 'staffer', 'rozelle': 'gazelle', 'fishtown': 'fishbowl', 'cerberus': 'cerebrum', 'tradi': 'trade', 'chillest': 'chilled', 'carbonmade': 'carbonate', '4mo': 'mo', 'tnk': 'tank', 'sharpless': 'sharpness', 'mha': 'ha', 'altarac': 'altar', 'tarhas': 'areas', 'neko': 'nero', 'pissel': 'pissed', 'maeghan': 'meghan', 'umasked': 'masked', 'intertoons': 'intentions', 'pisa': 'piss', 'stahp': 'stamp', 'cudtomers': 'customers', 'mequon': 'melon', 'techjob': 'techno', 'patiala': 'partial', 'jesterking': 'pestering', 'rockaways': 'rockaway', 'chalega': 'cholera', 'igg': 'egg', 'healthly': 'healthy', 'burnaby': 'barnaby', 'jonaya': 'jonas', 'squiggled': 'squiggle', 'dilouge': 'dialogue', 'ankaa': 'ankle', 'incentivised': 'incentivized', 'rumi': 'rum', 'rakib': 'raki', 'nhave': 'have', 'precuations': 'precautions', 'binan': 'bian', 'grumbly': 'grumble', 'ulez': 'lez', 'gh8': 'the', 'burrata': 'burrito', 'slackin': 'slacking', 'relatedly': 'belatedly', 'dayparts': 'departs', 'sosa': 'soda', 'deadass': 'badass', 'noba': 'nova', 'mmb': 'mob', 'greenville': 'granville', 'asab': 'arab', 'ultras': 'ultra', 'iotchat': 'gotcha', 'sundeep': 'sundae', 'leadership]': 'leadership', '1m': 'am', '14hr': 'sehr', 'norml': 'normal', 'hceapest': 'cheapest', 'vicinato': 'vicinity', 'joog': 'jog', '6mar': 'omar', '12years': 'years', 'coderood': 'codeword', 'gne': 'one', 'sinar': 'sonar', 'needs24': 'needs', 'colse': 'close', 'mewgulf': 'engulf', 'dafuq': 'daft', 'trud': 'true', 'cxnext': 'next', '$inds': 'finds', 'alabang': 'alabama', 'stiffler': 'stiffer', 'youch': 'touch', 'jazmyn': 'jazzy', 'prdering': 'ordering', 'knoppers': 'coppers', 'exacerbation': 'exacerbating', 'revali': 'real', 'cashcow': 'cashew', 'mkoo': 'moo', 'montoring': 'monitoring', 'nyani': 'nani', 'individualize': 'individualized', 'roeing': 'rowing', 'prolifer': 'provider', 'prescr': 'press', 'massag': 'massage', 'krta': 'kata', 'rg': 're', 'impl': 'imply', 'followus': 'follows', '9hours': 'hours', '$svok': 'tuvok', 'bixby': 'baby', 'coatue': 'coat', 'mlkday': 'midday', 'vara': 'para', '42min': 'min', '8oz': 'coz', 'locations]': 'locations', 'hestitant': 'hesitant', 'bajrang': 'bajrangi', 'souper': 'super', 'localbiz': 'localize', 'childfree': 'children', 'joann': 'joan', 'a9': 'a', 'bouvier': 'courier', 'khernia': 'hernia', 'cannnot': 'cannot', 'tmprd': 'third', 'ledbury': 'rebury', 'ubuquitous': 'ubiquitous', 'dtwn': 'down', 'phenix': 'phoenix', 'moreso': 'mores', 'outsize': 'outside', 'chacha': 'coach', 'xsx': 'is', 'reishi': 'relish', 'fxckin': 'fuckin', 'superbowl': 'supercool', 'bikea': 'bike', 'walki': 'walk', 'gopf': 'golf', 'sms': 'sums', 'recs': 'rec', 'avacado': 'avocado', 'calum': 'calm', 'wiwx': 'with', 'veganday': 'veranda', 'checkbox': 'checkbook', 'trista': 'tristan', 'nhcc': 'nice', 'californie': 'california', 'monta': 'month', 'tj': 'to', 'rebrandings': 'rebranding', 'unfixed': 'unfired', 'gresham': 'graham', 'inyo': 'into', 'redmarke': 'remarks', 'incl': 'inch', 'effusively': 'effusive', 'norridge': 'porridge', 'wic': 'win', 'rtui': 'tui', 'afrim': 'from', 'inari': 'nazi', 'ps11': 'psi', 'lapt': 'last', 'refrigera': 'refrigerate', 'chidi': 'chili', 'tapinto': 'taping', 'evoucher': 'voucher', 'shopright': 'stoplight', 'seafoam': 'seafood', 'ps3': 'psi', 'barbour': 'harbour', 'sault': 'fault', 'o7': 'of', 'barasat': 'barista', 'sussex': 'sussed', 'hemank': 'human', 'coustomer': 'customer', 'h2p': 'hop', 'dman': 'man', 'lordership': 'leadership', 'redic': 'medic', 'kandy': 'candy', 'dumbaf': 'dumb', 'pik': 'pick', 'waitlists': 'waitlist', 'furthe': 'further', 'haber': 'saber', 'acro': 'acre', 'umms': 'mums', 'manhmo': 'macho', 'zifi': 'ifi', 'nester': 'pester', 'futseg': 'fuse', 'hmmm': 'ummm', 'cbgo': 'go', 'businessplan': 'businessman', 'waiting4': 'waiting', 'chiuaua': 'chihuahua', 'beastiest': 'beasties', 'gofa': 'sofa', 'cleare': 'clear', 'paymt': 'part', 'foc': 'for', 'lauer': 'later', 'urruela': 'ursula', 'bialy': 'billy', 'pixar': 'pilar', '7job': 'job', 'senary': 'sentry', 'hazardously': 'hazardous', 'roseline': 'baseline', 'wpb': 'web', '8lb': 'all', 'jaja': 'raja', 'turdeau': 'bureau', 'ganito': 'janitor', 'themselfs': 'themself', 'previously]': 'previously', 'bacs': 'back', 'atrisk': 'risk', 'newsbox': 'newsboy', 'farros': 'farrow', 'dozjuly': 'ofjuly', 'intra': 'intro', 'pmnt': 'pint', 'idontwanna': "don'twanna", 'gentrifiers': 'gentrified', 'starz': 'start', 'wjhl': 'will', 'mfg': 'mug', 'tob': 'to', 'oeuvres': 'oeuvre', 'trucken': 'trucker', 'invisage': 'envisage', 'mobikes': 'mobiles', 'waicy': 'way', 'europ': 'euros', 'gittin': 'gettin', 'uarantine': 'quarantine', 'ts1': 'tsa', 'kylo': 'kilo', 'temescal': 'mescal', 'sehe': 'she', 'vala': 'gala', 'divyesh': 'dives', 'goldenera': 'goldeneye', 'morneau': 'moreau', 'shumer': 'summer', 'trvld': 'told', 'pssg': 'past', 'sargenti': 'sargent', 'immart': 'impart', 'fedjo': 'fed', 'rpts': 'rats', 'phoh': 'pooh', 'togetherky': 'together', 'shluld': 'should', 'provecho': 'prosecco', 'kalama': 'karma', 'aiova': 'above', 'lb': 'la', '3hours': 'hours', 'pepcid': 'pencil', 'se18': 'see', 'fuente': 'fuentes', 'baat': 'beat', 'tacoma': 'coma', 'canabis': 'cannabis', 'letahrgic': 'lethargic', 'acunia': 'acacia', 'wrcb': 'grab', 'rolller': 'roller', 'sharpton': 'sharpen', 'lmad': 'lead', 'agustin': 'austin', 'hacmve': 'have', 'mihlali': 'milli', 'pressley': 'pressed', 'dsm': 'dam', 'payc': 'pay', 'aiman': 'airman', 'gyz': 'gym', 'techbytes': 'terabytes', 'wabt': 'want', 'expatbs': 'expats', 'murathe': 'mutate', 'typa': 'type', 'havta': 'havea', 'morderen': 'ordered', 'adk': 'ask', 'megastore': 'megastar', 'luckasz': 'lucas', 'wapas': 'tapas', 'ludy': 'lady', 'ableist': 'ablest', 'sigh]': 'sight', 'frontrange': 'frontage', 'saanse': 'sense', 'colruyt': 'corrupt', '2maro': 'smart', 'negroe': 'negro', 'kah': 'ah', 'unifor': 'uniform', 'sleepyit': 'sleepy', 'priveledge': 'privilege', 'kiso': 'kiss', 'everson': 'emerson', 'robron': 'robot', 'bmc': 'bac', 'laiken': 'liken', 'sunoco': 'bunco', 'bruda': 'bruja', 'coquito': 'poquito', 'revamps': 'revamp', 'm4m': 'mom', 'confusedmy': 'confused', 'technews': 'techies', 'dominantly': 'dominant', 'delvery': 'delivery', 'parenting5': 'parenting', 'introspect': 'retrospect', 'camillus': 'camilla', 'nashv': 'nash', 'egrove': 'grove', 'thawe': 'thaw', 'macai': 'machi', '44am': 'am', 'dfcs': 'docs', 'don0s': 'dons', 'silvis': 'silvio', 'eliminati': 'eliminate', 'arghh': 'argh', 'fairshop': 'airship', 'mcmaster': 'master', 'moz': 'mom', 'consum': 'consume', 'illinoi': 'illinois', 'lipstik': 'lipstick', 'csas': 'cass', '5hat': 'that', '5d': 'ad', 'exectuive': 'executive', 'motorize': 'motorized', 'questionably': 'questionable', 'funtimes': 'aunties', 'underspent': 'underwent', 'cbg': 'cog', 'kha': 'ha', 'brg': 'big', 'swer': 'swear', 'teddington': 'reddington', 'acclimation': 'acclamation', 'kpc': 'up', 'hyvee': 'have', 'towboat': 'rowboat', 'ksleg': 'leg', 'mkot': 'moot', 'ootb': 'oot', 'notbored': 'motored', 'heartworm': 'earthworm', 'mely': 'rely', 'ordererlies': 'orderlies', 'minset': 'mindset', 'notifs': 'notify', 'detailst': 'details', 'flove': 'love', 'nger': 'anger', 'kuruman': 'truman', 'yiga': 'yoga', 'cfa': 'ofa', 'girll': 'girl', 'aapi': 'papi', 'verdura': 'ventura', '80rs': 'cars', 'ooc': 'ooh', 'mbps': 'maps', 'digg': 'dig', '60min': 'min', 'unrefundable': 'nonrefundable', 'jiayous': 'jealous', 'festoo': 'festoon', 'fari': 'far', 'tarea': 'area', 'urbina': 'urban', 'kops': 'cops', 'popu': 'pop', 'wyld': 'wild', 'nagpapa': 'nagara', 'marchent': 'marching', 'paybyphone': 'payphone', 'hitchens': 'kitchens', 'wante': 'want', 'ss10': 'ssa', 'jdcom': 'com', 'preorderered': 'preordered', 'wendover': 'handover', 'swpdx': 'swede', 'kalua': 'kala', 'nmtdt': 'nott', 'afta': 'aft', 'livelyhood': 'livelihood', 'kynli': 'kyle', 'affleck': 'affect', 'gardai': 'garda', 'freinds': 'friends', 'funko': 'funky', 'estaban': 'esteban', 'primefail': 'primeval', 'rosato': 'potato', 'preta': 'greta', 'mgn': 'man', 'beunique': 'unique', 'boh': 'both', 'bhpd': 'had', 'ntnn': 'ten', 'cuffe': 'cuff', 'fayose': 'false', 'ussd': 'used', 'normalyn': 'normal', 'kaff': 'gaff', 'boutto': 'outto', 'spazas': 'sparks', 'myine': 'mine', 'cheerily': 'cheering', 'parlait': 'parfait', 'misa': 'miss', '1for': 'for', 'tol': 'to', 'uklockdown': 'lockdown', 'ynr': 'yer', 'thanl': 'than', 'tammi': 'tammy', 'wrentching': 'wrenching', 'shutins': 'shouting', 'hg2': 'he', '4pints': 'pints', 'donkado': 'dorado', 'bloop': 'blood', 'minn': 'mind', 'marination': 'imagination', 'lafc': 'lac', 'oopsie': 'sophie', 'requsting': 'requesting', 'lyf': 'ly', 'malout': 'about', 'smet': 'set', 'diagon': 'dragon', 'bsln': 'been', 'estaz': 'esta', 'ubered': 'bored', 'nako': 'nano', 'tsb': 'tub', 'ponderi': 'ponder', 'neighbors]': 'neighbors', 'wollies': 'follies', 'everquest': 'everest', 'adora': 'adore', 'jesu': 'jesus', 'masbia': 'mafia', 'pademic': 'pandemic', 'nicolson': 'colson', 'stevil': 'stevie', 'strack': 'track', 'proje': 'prove', 'resuply': 'resupply', 'd3': 'do', 'perog': 'pero', 'bc': 'be', 'bridgeton': 'bridget', 'subscribtion': 'subscription', 'ikk': 'ink', 'coleus': "cole's", 'rahma': 'rama', 'conpany': 'company', 'panvel': 'panel', 'salemma': 'salesman', 'purveyours': 'purveyors', 'javi': 'jai', 'wccb': 'cab', 'gronks': 'drinks', 'freeplay': 'freely', 'livepd': 'lived', 'pleasy': 'please', 'sarf': 'surf', 'delcares': 'declares', 'd]': 'do', 'gtube': 'tube', 'myplayer': 'player', 'ricola': 'nicola', 'uodate': 'update', 'sutta': 'outta', 'worksites': 'worksite', 'luminate': 'laminate', 'stepint': 'stepping', 'zuck': 'fuck', 'birdseye': 'birdseed', 'paani': 'paan', '$t': 'it', 'autodrive': 'outdrive', 'crowland': 'rowland', 'ashapp': 'shape', 'mvideo': 'video', 'neighbo': 'neighbor', 'fuxk': 'fuck', 'wadadli': 'waddle', 'ths': 'the', 'weelo': 'weel', 'payed': 'played', '15h': 'oh', 'mecq': 'mech', 'theymy': 'they', 'selfhood': 'seafood', 'iaharris': 'harris', '12p': 'up', 'codogno': 'cologne', 'doore': 'door', 'shouter': 'shooter', 'thinkinga': 'thinking', '18yours': 'yours', 'bullsh': 'bulls', 'sq': 'so', 'aquire': 'acquire', '30p': 'up', 'digita': 'digital', 'baconised': 'agonised', 'xiaoqu': 'xiaoyu', 'riro': 'rio', 'yummers': 'summers', 'nanak': 'nana', 'awg': 'awe', 'berkshire': 'berkshires', 'chedder': 'cheddar', 'endlini': 'ending', 'didactics': 'didactic', 'essent': 'assent', '5bn': 'ibn', 'fleek': 'fleet', 'tagum': 'tag', 'vashi': 'ashi', 'republik': 'republic', 'lawren': 'lauren', 'weks': 'weeks', 'qnytjing': 'anything', 'bordereaux': 'bordeaux', 'anglesey': 'angles', 'greenbay': 'greenback', 'ligering': 'lingering', 'styourofoam': 'styrofoam', 'strawberrie': 'strawberries', 'pkwy': 'play', 'cepat': 'cept', 'eren': 'even', 'hardwar': 'hardware', 'buckner': 'buckler', 'groceria': 'grocery', 'brahmin': 'brahmins', 'nyclife': 'mylife', 'garg': 'gang', 'kath': 'path', 'wfo': 'who', 'neph': 'neh', 'capetown': 'camptown', 'esls': 'eels', 'currentl': 'current', 'nhp': 'nap', 'nyap': 'nap', 'landersph': 'landers', 'chewse': 'cheese', 'johnathan': 'jonathan', 'trudon': 'tendon', 'f': 'i', 'celis': 'cells', 'ssot': 'shot', 'nanaimo': 'nanimo', 'sleeting': 'sleeping', 'gcc': 'get', 'rakyat': 'rayyan', 'unglove': 'glove', 'aigh': 'high', 'philo': 'phil', 'mrd': 'mad', 'imus': 'mus', 'swy': 'say', 'masauti': 'masti', '$baba': 'baba', 'ky': 'my', 'brummie': 'rommie', 'commited': 'committed', 'vadams': 'adams', 'rachana': 'rachael', 'whaaat': 'what', 'alang': 'along', 'bmvo': 'mo', 'nahi': 'nazi', 'minimisation': 'minimization', 'sheiding': 'shedding', 'cantact': 'contact', 'recommendat': 'recommend', 'abule': 'able', 'obvioiusly': 'obviously', 'ionbiz': 'ionic', '92yo': 'yo', 'dru': 'drug', 'amee': 'amen', 'londyn': 'london', 'nwarkansas': 'arkansas', 'qaum': 'quam', 'gve': 'give', 'delhivery': 'delivery', '7cm': 'him', 'skipton': 'siphon', 'ize': 'ice', 'bouncin': 'bouncing', 'torcen': 'torben', 'gps': 'gas', 'news]': 'news', 'roxbury': 'robbery', 'refurb': 'return', 'urbantz': 'urban', '18m': 'him', 'eeep': 'keep', 'novelte': 'novelty', 'amman': 'aman', 'mrand': 'grand', 'xoxox': 'xoxo', 'pm]': 'my', 'krolia': 'frolic', 'workhard': 'orchard', 'blessin': 'blessing', 'brotato': 'potato', 'femenine': 'feminine', 'fipola': 'bipolar', 'fall1': 'fall', 'tylerites': 'tolerates', 'ukheatwave': 'heatwave', 'mri]': 'mri', 'challan': 'challah', 'grabpay': 'grabby', 'crysis': 'crisis', 'motza': 'mota', 'fanya': 'tanya', 'ebitda': 'debita', 'outlooks': 'outlook', 'sepa': 'sea', 'urbanism': 'organism', 'screan': 'screen', 'aramex': 'armed', 'krishn': 'krishna', 'icard': 'card', 'meagan': 'megan', 'choys': 'chops', 'caret': 'care', 'pmcs': 'pics', 'erlier': 'earlier', 'thinkingl': 'thinking', 'meatza': 'meat', 'haricot': 'haricots', 'yourkshire': 'yorkshire', 'mollie': 'collie', 'pdx': 'pox', 'uws': 'us', 'autonews': 'atones', 'cba': 'cab', 'mccbiz': 'mccain', '7mins': 'mins', 'dotnet': 'botnet', 'rett': 'rest', 'pawel': 'panel', 'ps18': 'psi', 'molic': 'colic', '$glpg': 'glug', 'bitttch': 'bitch', 'iteams': 'teams', 'unsubscribed': 'subscribed', 'techvu': 'tech', 'samarth': 'smart', 'dokey': 'donkey', 'zysola': 'sola', '2provide': 'provide', 'gonoliers': 'gondolier', 'wolff': 'wolf', 'doorand': 'doorman', 'mfking': 'making', 'rrb': 'rob', 'grogo': 'grog', 'myalo': 'malo', 'hubbie': 'hubble', 'bpost': 'post', 'foodora': 'fedora', 'bcus': 'bus', 'philipp': 'philip', 'shorehead': 'forehead', 'sendial': 'sundial', 'lak': 'law', 'darwinism': 'darwinian', 'rtto': 'itto', 'metabol': 'metal', 'mec': 'me', 'evnews': 'news', 'jennica': 'jessica', 'sacrifising': 'sacrificing', 'sbb': 'sub', 'myouriad': 'myriad', 'yisroel': 'israel', 'keyani': 'keying', 'cretch': 'wretch', 'outspeed': 'outspend', '$saex': 'sex', 'plicy': 'policy', 'pyo': 'pro', 'niiro': 'nitro', 'noncense': 'nonsense', 'jofail': 'jail', 'sherrif': 'sherri', 'primula': 'primal', 'telos': 'tells', 'enployees': 'employees', 'ocdo': 'odo', 'agee': 'agree', 'yaf': 'ya', 'picku': 'pick', 'pfmagazine': 'magazine', 'katsuki': 'kabuki', 'canteloupe': 'cantaloupe', 'eroego': 'forego', 's16': 'so', 'centr': 'center', 'sexuales': 'sexual', 'karta': 'karma', 'auspoi': 'aussi', '$xpev': 'expel', '4them': 'them', 'smartlink': 'startling', 'groceru': 'grocery', 'brava': 'brave', 'pabili': 'pail', 'theyour': 'toyour', 'wakad': 'waked', 'checkr': 'check', 'summerside': 'summertime', 'diferent': 'different', 'fiveguys': 'wiseguys', 'ary': 'are', 'rie': 'die', 'pompano': 'company', 'brae': 'brave', 'stmarks': 'starks', 'clonescripts': 'conscripts', 'fridg': 'fridge', 'casco': 'cisco', 'minnis': 'minnie', 'solgar': 'solar', 'stadens': 'stamens', 'kushner': 'usher', '$news': 'news', 'scitua': 'shita', 'payee': 'payer', 'msks': 'asks', 'wafaa': 'wanna', 'wttg': 'with', 'homeqart': 'homeward', 'ohpol': 'pool', 'socio': 'socko', 'fortnums': 'fortunes', 'thumburg': 'thumbing', 'vist': 'visit', 'ke2': 'key', 'assosicates': 'associates', 'bulloch': 'bullock', 'iterally': 'literally', 'socal': 'local', 'nbrs': 'news', 'stingersup': 'stingers', 'vonurable': 'honorable', 'ridiculo': 'ridicule', 'kirkby': 'kirby', 'suburbish': 'suburbs', 'gotgame': 'gotham', 'flaggy': 'faggy', 'jemp': 'jump', 'amediting': 'editing', 'selfs': 'self', 'sefalana': 'sealant', 'optionality': 'nationality', 'moneyweb': 'moneyed', 'pastorsam': 'pastoral', 'deliverw': 'deliver', 'harleys': 'harley', 'zakwan': 'akan', 'shopex': 'hope', 'novid': 'navid', 'ceasy': 'easy', 'suckin': 'sucking', 'wegotus': 'legatus', 'segway': 'subway', 'zlatan': 'latin', 'soma': 'some', 'rbh': 'rah', 'partech': 'patch', 'bizop': 'bishop', 'arrino': 'arrive', 'acando': 'cando', '68yo': 'yo', '16lb': 'bulb', '4ward': 'award', 'betweeb': 'between', 'f1': 'fu', 'porkbelly': 'potbelly', 'rsl': 'rel', 'quesha': 'questa', 'rchd': 'read', 'frightning': 'frightening', 'exp': 'ex', 'newlands': 'wetlands', 'conecuh': 'concur', 'disgustor': 'disgust', 'howcan': 'showman', 'arcorona': 'corona', 'higly': 'highly', 'ngmedia': 'media', 'mohala': 'moral', 'oms': 'moms', 'morbi': 'morbid', '35weeks': 'weeks', 'upbonus': 'bonus', 'udy': 'judy', 'gopalam': 'gopal', 'isolat': 'isolate', 'bissous': 'missus', 'everytime': 'overtime', 'chicag': 'chicago', 'dispirit': 'spirit', 'scnews': 'screws', 'jeezus': 'jesus', 'leclerc': 'lecher', 'umtata': 'mutate', 'lede': 'led', '12ft': 'left', 'hawaiieats': 'hawaiians', '2do': 'do', 'sfxlabs': 'slabs', 'barkfest': 'darkest', 'bandmom': 'random', 'incentivizes': 'incentivize', 'king5': 'king', 'nutz': 'nuts', 'technica': 'technical', 'nightstands': 'nightstand', 'wkow': 'wow', 'weloveyou': 'loveyou', 'stepout': 'spout', 'saharty': 'salary', 'paety': 'party', 'bnpl': 'ball', 'hdropcy': 'dropsy', 'basf': 'base', 'puting': 'putting', 'rowena': 'rowing', 'bazzar': 'bazaar', 'vaxid': 'valid', 'eir': 'air', 'livhud': 'lived', 'shelv': 'shell', 'spod': 'spot', 'realnews': 'realness', 'hpbasket': 'basket', 'sugbo': 'sugar', 'virtua': 'virtual', 'houzz': 'house', 'eotr': 'for', 'storw': 'story', 'flatbush': 'flatfish', 'shavi': 'shave', 'definetly': 'definetely', 'dukan': 'duran', 'ruk': 'run', 'subcription': 'subscription', 'quacker': 'quicker', 'esacalation': 'escalation', 'groovered': 'hoovered', 'epu': 'epi', 'isitme': 'site', 'harcourt': 'harbour', 'film4': 'film', '$fubo': 'fobo', 'gimac': 'imac', 'caledon': 'cameron', 'cincy': 'cinch', 'paypals': 'papal', 'dayoff': 'payoff', 'eastend': 'eaten', 'cottager': 'cottage', '07am': 'am', 'kalen': 'karen', 'prop22': 'proper', 'vegg': 'egg', 'quarantained': 'quarantined', 'siha': 'sha', 'lemonande': 'lemonade', 'fth': 'ith', 'taskrabbits': 'jackrabbits', '3m': 'am', 'greeners': 'greener', 'edenred': 'denied', 'monnies': 'monies', 'trumpist': 'trumpet', 'wtoday': 'today', 'mi5': 'mix', 'jangga': 'manga', 'w42st': 'worst', 'ctmr': 'come', 'denge': 'dense', 'yardwork': 'hardwork', 'bestfriends': 'befriends', 'essentals': 'essentials', 'inexeter': 'exeter', 'anonzi': 'anon', 'numeris': 'numeric', 'leipold': 'leopold', 'algos': 'algo', 'elderlaw': 'elderman', '5x5': 'ex', 'gifted]': 'gifted', 'satx': 'sat', 'synergise': 'energise', 'e4i': 'epi', 'fundaising': 'fundraising', 'urianger': 'ranger', 'hatfield': 'hayfield', 'unctuously': 'unctuous', 'supportnz': 'support', 'nagendra': 'agenda', 'momgasm': 'orgasm', 'mtnews': 'news', 'ofering': 'offering', 'vuln': 'turn', '45degrees': 'degrees', 'makkah': 'maketh', 'rustans': 'russians', 'jamel': 'camel', '$og': 'dog', 'coronaab': 'coronary', 'oos': 'cos', 'sche': 'she', 'patie': 'patio', 'yc': 'ya', 'barocas': 'barons', 'alessi': 'less', 'a11': 'a', 'banyak': 'banyan', 'ffs': 'ifs', 'fabulessly': 'fabulously', 'flooz': 'floor', '75yo': 'yo', '3mil': 'mil', 'dehli': 'deli', '60mins': 'mins', 'cgnze': 'gaze', 'unifil': 'until', 'athir': 'ather', 'pesimo': 'primo', 'avacados': 'avocados', 'eif': 'if', '20oz': 'coz', 'fera': 'fear', 'oblidged': 'obliged', 'redcliff': 'radcliffe', 'uuu': 'uou', 'lurgy': 'lucky', 'thear': 'their', '4million': 'million', 'muli': 'mule', 'barbuda': 'barbed', 'quarintine': 'quarantine', 'ballentine': 'valentine', 'yashar': 'washer', 'ubmarket': 'upmarket', 'hermited': 'permitted', 'bejeebus': 'bejeezus', 'presse': 'press', 'mdhhs': 'myths', 'johnstone': 'johnson', 'kobma': 'coma', 'meknubs': 'menus', 'quarantimes': 'quarantines', 'portkey': 'porter', 'kul': 'ku', 'hinews': 'sinews', 'asado': 'asada', 'proseccos': 'prosecco', 'jaye': 'jake', 'mure': 'more', 'moher': 'mother', '99cent': 'decent', 'tmi': 'mi', 'vanful': 'manful', 'lcs': 'les', 'adahlia': 'dahlia', 'ktvu': 'tu', 'kp': 'up', 'pauwi': 'pawn', 'nijiya': 'nikita', 'prerecord': 'rerecord', 'yaas': 'yams', 'matcha': 'match', 'droneup': 'drone', 'dele': 'del', 'estrogan': 'estrogen', 'faristas': 'baristas', 'panathur': 'panther', 'ronney': 'rodney', 'cdl': 'cal', 'brrr': 'burr', 'qaoow': 'avow', 'e4f': 'elf', 'atas': 'alas', 'ps4': 'psi', 'fixins': 'fixing', 'necessery': 'necessary', 'gopros': 'pros', '4wk': 'we', 'tweakn': 'tweak', 'parttime': 'wartime', 'platonicly': 'platonic', 'ramai': 'rama', 'turesday': 'tuesday', 'whidbey': 'whiskey', '2xs': 'is', 'plmarketing': 'marketing', 'attackt': 'attack', 'mavotv': 'makoto', 'o]': 'of', 'global18': 'global', 'suceessul': 'successful', 'cagr': 'car', 'craftbeer': 'craftier', 'wastefully': 'tastefully', 'othef': 'other', 'absol': 'also', 'wdym': 'why', 'schw': 'scow', 'oloyin': 'loin', 'boaw': 'boat', 'booy': 'body', 'ipas': 'pas', 'determinant': 'determining', 'carapps': 'crappy', 'blattler': 'battle', 'duma': 'dumb', 'amdg': 'amd', '2$': 'i', 'deshaun': 'shaun', 'klutch': 'clutch', 'mailday': 'railway', 'hahaa': 'haha', 'aparts': 'apart', 'usiness': 'business', 'indygo': 'indigo', 'onct': 'once', 'benicia': 'felicia', 'rdeats': 'death', 'zedelivery': 'delivery', 'wara': 'war', '$tsla': 'tesla', '$googl': 'google', 'unlove': 'unloved', 'teally': 'really', 'sanibel': 'daniel', 'freshpass': 'trespass', 'leighton': 'lighten', 'chaura': 'chakra', 'nevadan': 'nevada', 'unretrieved': 'retrieved', 'boatle': 'bottle', 'stockage': 'stockade', 'ennio': 'ennis', 'timesfor': 'timefor', 'forgeo': 'forget', 'rts': 'its', 'ssaw': 'saw', 'nessie': 'jessie', 'belethe': 'delete', 'whiteman': 'whitman', 'power20': 'power', 'kadai': 'kadam', 'havenots': "haven't", 'viraj': 'viral', 'vaccin': 'vaccine', 'magde': 'made', 'croupy': 'croup', 'ghanta': 'giant', 'hoeft': 'heft', 'gottn': 'gotta', 'toyosi': 'toys', 'soff': 'off', 'disabed': 'disabled', 'foos': 'food', 'kaito': 'wait', 'aute': 'ate', 'asusual': 'usual', 'recommende': 'recommend', 'yokart': 'yogurt', 'ramdev': 'raided', 'myworld': 'world', 'bigthink': 'birthing', 'easierchef': 'masterchef', 'shouldld': 'should', 'twin33s': 'twins', 'newstv': 'news', 'gst': 'get', 'multiplier': 'multiplied', 'econol': 'cool', 'hadiyes': 'ladies', 'exc': 'ex', 'koproud': 'proud', 'presente': 'present', 'haate': 'hate', 'inum': 'num', 'vudu': 'kudu', 'vedder': 'redder', 'pbe': 'be', 'bg': 'be', 'houstonia': 'houston', 'mozz': 'more', 'basti': 'baste', 'schwann': 'swann', 'myajc': 'mac', 'kechar': 'nectar', 'calaykum': 'alaykum', 'kitchener': 'kitchen', 'foodbox': 'toolbox', 'suport': 'support', 'ukudla': 'uvula', 'fox31': 'fox', 'es6': 'es', 'univers': 'universe', 'urbe': 'urge', 'howrah': 'hoorah', 'achoo': 'choo', 'tifa': 'tiff', 'kiryat': 'kira', 'fttp': 'stop', 'freshd': 'fresh', 'funnyit': 'funny', 'edkaplan': 'kaplan', 'reshedule': 'reschedule', 'fateh': 'fate', 'peopod': 'people', 'ewg': 'egg', 'durham': 'dunham', 'deservingly': 'deserving', 'gyouro': 'your', 'lolies': 'lilies', 'hhaha': 'haha', '9lb': 'all', 'ysm': 'yam', 'couono': 'count', '1mg': 'my', 'swensen': 'sense', 'kycc': 'kick', 'thrillist': 'thrilling', 'rollouts': 'rollout', 'dlave': 'slave', 'spossed': 'supossed', 'raices': 'races', 'saini': 'saint', 'upcs': 'ups', 'yforall': 'forall', 'racisty': 'racist', 'vender': 'tender', 'dettah': 'death', 'kn95': 'know', 'glich': 'glitch', 'buliran': 'julian', 'feed]': 'feed', 'aapl': 'all', 'azur': 'azure', 'clairol': 'claire', 'motiva': 'motive', 'direc': 'direct', 'ww3': 'wwe', 'indorians': 'andorians', 'carrrier': 'carrier', 'taptop': 'laptop', 'exton': 'elton', 'gatineau': 'gateau', 'dodgie': 'dodge', 'healthness': 'healthiest', 'okurr': 'our', 'midern': 'modern', 'quarantineli': 'quarantine', 'purina': 'during', 'mariupol': 'marisol', 'marshmellow': 'marshmallow', 'jollof': 'follow', 'positiv': 'positive', '3pm': 'up', 'zelle': 'belle', 'uipath': 'path', 'spi': 'spit', 'rerail': 'retail', 'widener': 'wiener', 'stfg': 'stag', 'medtech': 'melech', 'zanily': 'family', '7]': 'i', 'kuh': 'uh', '8your': 'your', 'golaam': 'goddam', 'suprema': 'supreme', 'papam': 'papa', 'edtwt': 'edit', 'mke': 'me', 'g5qs': 'guys', 'pattisons': 'partisans', 'broyle': 'broyles', 'denna': 'donna', '$hewa': 'chew', 'torylies': 'tories', 'bedale': 'beadle', 'ballina': 'balling', 'curr': 'cure', 'carless': 'careless', 'dicknose': 'diagnose', 'mwn': 'man', 'gl': 'go', 'abhij': 'abhor', '50cents': 'cents', 'flemington': 'remington', 'uzima': 'prima', 'rajnasta': 'canasta', 'probierts': 'roberts', 'fux': 'fun', 'rx': 're', 'read]': 'ready', 'lifb': 'life', 'rosca': 'rosa', 'ekhaya': 'kaya', 'aburi': 'burn', 'katyisd': 'katniss', 'restaraunts': 'restaurants', 'balla': 'ball', 'conta': 'conga', 'owaisi': 'waist', 'banplastic': 'aplastic', 'gowda': 'gouda', 'watsapp': 'whatsapp', 'uwah': 'wah', 'doners': 'donors', 'goteddy': 'teddy', 'coppell': 'copper', 'douchebro': 'douchebag', 'opentable': 'pentacle', 'tetiana': 'tatiana', 'bismol': 'pistol', 'plantt': 'planet', 'ambode': 'abode', 'bbm': 'bum', 'lokoja': 'lakota', 'procu': 'prove', 'overcrowd': 'overgrown', 'mln': 'man', 'notakeout': 'takeout', 'vistana': 'distant', 'beryl': 'meryl', 'convice': 'convince', 'arpico': 'apricot', 'bbc': 'bac', 'toasties': 'tastes', 'losang': 'losing', 'apartmen': 'apartment', '32oz': 'coz', 'lvwt': 'let', '3]': 'i', 'yyc': 'you', 'anazing': 'amazing', 'gc': 'go', 'surfside': 'subside', 'zimbos': 'bimbos', 'uktech': 'tech', 'availbl': 'available', 'marlton': 'marlon', 'coronav': 'coronal', 'coloradan': 'colorado', 'ealry': 'early', 'lxc': 'lac', 'carena': 'arena', 'gerd': 'herd', 'ebo': 'ego', 'nomas': 'nomads', 'thrifting': 'drifting', 'implem': 'simple', 'evedintly': 'evidently', 'bauman': 'barman', 'smuggly': 'smuggle', 'yaa': 'ya', 'apna': 'anna', 'percik': 'permit', 'subluxed': 'subdued', 'electricvans': 'electricians', 'coas': 'cops', 'scrutized': 'scrutinized', 'imran': 'ikran', 'wholesomely': 'wholesome', 'shonen': 'shone', 'renegotiation': 'negotiation', 'serivce': 'service', 'tshirts': 'shirts', 'scranned': 'scanned', 'kalam': 'salam', 'gatos': 'gates', 'testing]': 'testing', 'okada': 'okay', 'cooly': 'cool', 'borth': 'both', '$weed': 'weed', 'ngah': 'nah', 'dapps': 'apps', 'robbe': 'robbed', 'door]': 'door', 'iirc': 'girl', 'adhd': 'add', 'sugarloaf': 'sugarcoat', 'mememe': 'lemme', 'telecomm': 'telecom', 'tayag': 'tag', 'pedos': 'pesos', 'jj': 'je', 'twincity': 'trinity', 'mcs': 'mes', 'shortlived': 'shortlisted', 'nw6': 'we', 'markfed': 'marked', 'psn': 'pin', '$shrm': 'sham', 'lakemont': 'lakefront', 'flipp': 'flip', 'rateone': 'ration', 'ex1': 'ex', 'novemberff': 'november', 'showin': 'showing', 'crengland': 'england', 'areaish': 'areas', 'visayas': 'visas', 'crapiest': 'craziest', '2help': 'help', 'mno': 'no', 'rosans': 'romans', 'grenadian': 'grenadine', 'alanag': 'alana', 'pmfc': 'puff', 'upsi': 'ups', 'zina': 'ina', 'armys': 'army', 'selfmade': 'selfsame', 'jmoe': 'joe', 'takesman': 'salesman', 'tarkov': 'markov', '$ndyn': 'andy', 'pressie': 'press', 'intiative': 'initiative', 'jvt': 'jet', 'wowowow': 'powwow', 'applaudxx': 'applaud', 'tid': 'did', 'tomtea': 'totem', 'duckweed': 'dickweed', 'yasss': 'ass', 'diecast': 'recast', 'bhandar': 'hangar', 'heyned': 'helped', 'disrup': 'disrupt', 'tarjay': 'tray', 'dh': 'do', 'medhat': 'mewhat', '$avisx': 'lavish', 'nre': 'are', 'stelle': 'stella', 'labriola': 'variola', 'alahai': 'alpha', 'hugz': 'huge', 'adolfo': 'adolf', 'aas': 'was', 'deliveryby': 'delivery', 'sewa': 'sea', 'dph1': 'dah', 'karo': 'kara', 'fiacre': 'face', 'ende': 'end', 'rhonny': 'ronny', 'chee': 'thee', 'rma': 'ma', 'bombest': 'bombs', 'copur': 'coeur', 'reorderer': 'reordered', 'bgm': 'bum', 'exhilarate': 'exhilarated', 'abcnews': "acne's", 'classpass': 'classless', 'weserve': 'deserve', 'dayquil': 'daqui', 'flagg': 'flag', 'stusents': 'students', 'wuxia': 'julia', 'deliverered': 'delivered', 'tues': 'ties', 'fortmyers': 'formers', 'ashleigh': 'sleigh', 'amik': 'amid', 'exorcized': 'exorcised', 'kinegar': 'vinegar', 'ugc': 'ugh', 'everybo': 'every', 'f2bq': 'fbi', 'fortnum': 'fortune', 'caccia': 'faccia', 'buychamber': 'bedchamber', 'verley': 'very', 'sandwhiches': 'sandwiches', 'kulp': 'pulp', 'eggo': 'eggs', 'continente': 'continent', 'wheew': 'wheel', 'anim': 'aim', 'kain': 'pain', 'bol': 'boy', 'ecsf': 'else', '2mins': 'mins', 'trea': 'treat', 'royton': 'rotten', 'wygd': 'word', '$compq': 'comp', '7ft': 'aft', 'jeebies': 'jerries', '17bn': 'ibn', 'getir': 'gettir', 'funnygo': 'funny', 'vogels': 'vowels', 'roil': 'roll', 'startupuk': 'startup', 'centeva': 'center', 'decaro': 'dear', 'rabbiter': 'rabbit', 'gugs': 'guys', 'rebag': 'rebar', 'mckinney': 'mckinley', 'rescuedule': 'reschedule', 'brum': 'bum', 'vdoc': 'doc', 'kofi': 'koi', '9kg': 'ekg', 'shoply': 'should', '44k': 'ask', 'nfeld': 'need', 'marginall': 'marginal', 'puten': 'guten', 'bvma': 'boma', '45ish': 'wish', 'mandals': 'sandals', 'php': 'pop', 'swabi': 'swab', 'amzl': 'amyl', 'haaland': 'garland', 'akak': 'aka', 'ottawa': 'otta', 'saveman': 'caveman', 'd3ath': 'death', 'verka': 'vera', 'topchef': 'touched', 'amwriting': 'writing', 'lpve': 'love', 'djk': 'dak', 'msde': 'made', 'amab': 'arab', 'unrolls': 'unroll', 'woodside': 'roadside', 'er]': 'era', 'speedstar': 'speedster', 'ntuthu': 'tutu', 'vaughan': 'vaughn', 'rnd': 'and', 'commadation': 'commendation', 'sabziya': 'sabrina', 'laist': 'last', 'nomo': 'homo', 'mnr': 'mar', 'uefa': 'left', 'paypig': 'paying', 'itired': 'tired', 'aesop': 'also', 'juicero': 'juicer', 'avai': 'avail', '6mins': 'mins', 'worldall': "world'll", 'boymom': 'bottom', '$ups': 'cups', 'klm': 'kim', 'perrone': 'perrine', 'lohia': 'logic', 'colden': 'golden', 'logiciel': 'logical', 'bacck': 'back', 'guzman': 'gunman', 'bulgaria': 'bulgarian', 'springboro': 'springboard', 'snellville': 'smallville', 'iteasm': 'team', 'yho': 'who', 'orig': 'rig', 'motivaton': 'motivation', 'shrimal': 'serial', '2ys': 'ys', 'catania': 'satanic', 'cochin': 'cousin', 'recieve': 'receive', 'dormont': 'dormant', 'dronned': 'drowned', 'abv': 'abs', 'langata': 'gangsta', 'upas': 'ups', 'solong': 'oolong', 'camu': 'came', 'retaill': 'retail', '12packs': 'packs', 'forebearance': 'forbearance', 'impressiv': 'impressive', 'nsfw': 'now', 'jphn': 'john', 'walmar': 'palmar', 'sct': 'set', 'maketime': 'maritime', 'asin': 'sin', 'jjs': 'jus', 'feeke': 'feel', 'ouid': 'quid', 'bcos': 'cos', 'harbolt': 'harbor', 'jayson': 'jason', 'ncwabe': 'cabe', '5pp': 'app', 'gautam': 'bantam', 'vmy': 'my', 'makai': 'maki', 'asyura': 'assure', 'playstation5': 'playstation', 'epos': 'egos', 'andrex': 'andrew', 'igm': "i'm", 'varinder': 'grinder', 'botd': 'both', 'vanua': 'vanya', 'rehri': 'rear', 'ance': 'once', 'foodx': 'food', 'strores': 'stores', 'healthw': 'health', 'pte': 'ate', 'charites': 'charities', 'bereave': 'bereaved', 'hiphop': 'bishop', 'swe': 'we', 'lofoten': 'often', 'polution': 'solution', 'firsr': 'first', 'waxman': 'taxman', 'perspex': 'perplex', 'taika': 'tacka', 'plcb': 'place', 'phononic': 'photonic', 'w00t': 'what', 'lllll': 'lull', 'vagasil': 'vagisil', 'busines': 'business', 'vccircle': 'circle', 'lbfe': 'life', 'margs': 'marks', 'grcc': 'grab', 'v2': 've', 'gillard': 'willard', 'randazzo': 'ragazzo', 'columbians': 'colombians', 'odie': 'die', 'extracash': 'extracts', 'infamously': 'infamous', 'columbuswi': 'columbus', 'gaynor': 'mayor', 'ilocos': 'locos', 'ovehaul': 'overhaul', 'jebel': 'rebel', 'overstock': 'overtook', 'infilled': 'unfilled', 'butto': 'butt', 'sthg': 'stag', 'waialea': 'wailed', 'lkke': 'like', 'wefast': 'least', 'payless': 'painless', 'olympia': 'olympic', 'accoutrement': 'accoutrements', 'intiki': 'intake', 'retzler': 'rattler', 'koek': 'kook', 'mfgs': 'mugs', 'forshame': 'forsake', 'licka': 'lick', 'npa': 'na', 'africas': 'african', 'nk': 'no', 'ivr': 'ivy', 'nephrology': 'neurology', 'sudbury': 'suburb', 'shasha': 'sasha', 'wplus': 'plus', 'orderinary': 'ordinary', '3pl': 'upl', 'purefit': 'profit', 'sayu': 'say', '25th': 'with', 'scottla': 'scott', 'pavlova': 'pavlovian', 'ferntree': 'entree', 'farmacy': 'pharmacy', '5pm': 'up', 'travelt': 'travel', 'sinces': 'since', 'gridley': 'grilled', 'tila': 'till', 'orrico': 'rico', 'corcoran': 'corporal', 'townhome': 'townhouse', 'tamiflu': 'tamil', 'literlly': 'literally', 'sourav': 'source', 'doorste': 'doorstep', 'unsafely': 'safely', 'applicat': 'applicant', 'v3cube': 'cube', 'ggx': 'ggo', 'forestation': 'molestation', 'cerilli': 'chilli', 'chaval': 'cheval', 'jub': 'job', 'coldones': 'condones', 'merza': 'mercy', 'ivery': 'very', 'bianco': 'bianca', 'freshbyt': 'freshly', 'bbd': 'bad', 'kevins': 'kevin', 'kokal': 'okay', 'fewk': 'few', 'mistily': 'mostly', '9your': 'your', '5y': 'my', 'nytsb': 'nuts', '$ba': 'ba', 'pfa': 'pa', 'commandent': 'commandant', 'rej': 'red', 'emailiing': 'emailing', 'gametime': 'sometime', 'shopbs3': 'shops', 'decad': 'dead', 'norwell': 'powell', 'boxtober': 'october', 'eleventy': 'eleventh', 'continue1': 'continue', 'wayve': 'wave', 'k7': 'ke', 'dronfield': 'cornfield', 'recogniz': 'recognize', 'usvi': 'us', '50ft': 'left', 'tamy': 'tame', 'lynley': 'lonely', 'tumanze': 'humane', 'fleejack': 'flapjack', 'usalak': 'salad', 'ancaster': 'lancaster', 'nuri': 'yuri', 'vicnews': 'views', 'crunchgear': 'cruncher', 'pm2': 'my', 'welcombe': 'welcome', 'sab': 'say', 'coll': 'call', 'monika': 'monica', 'ogg': 'egg', 'kedah': 'yeah', 'srv': 'suv', 'gopro': 'pro', 'n20': 'not', 'stj': 'sta', 'allots': 'allows', 'aybo': 'ambo', 'previo': 'previa', 'pearmageddon': 'armageddon', 'ahha': 'aha', 'birfday': 'birthday', 'boxe': 'box', 'pgs': 'pigs', 'tamsin': 'basin', 'eligable': 'eligible', 'aeon': 'neon', 'eic': 'epic', 'distancinguk': 'distancing', 'maidstone': 'sandstone', 'susanna': 'susannah', 'krte': 'kite', 'ps1m': 'sum', 'marxism': 'marxist', 'spaceo': 'space', 'sl': 'so', 'justi': 'just', 'survivalkit': 'survivalist', 'fl': 'fly', 'aiya': 'aida', 'matildas': 'matilda', 'overcomplicated': 'overcomplicate', 'nalang': 'along', 'darthie': 'archie', 'bangaluru': 'bangalore', 'cristopher': 'christopher', '$chno': 'echo', 'naiinis': 'paninis', 'healthmc': 'health', 'ideate': 'idea', 'chockered': 'checkered', 'unbothered': 'bothered', 'mamamoo': 'yamamoto', 'larsson': 'larson', 'joeys': 'joys', 'tanlines': 'canines', 'brix': 'brig', 'mgs': 'mugs', 'expectorant': 'expectant', 'sotasty': 'tasty', 'mcconnell': "o'connell", 'garri': 'garry', 'yanko': 'yank', 'ddp': 'dip', 'appdly': 'apply', 'recordered': 'reordered', 'hollister': 'holster', 'bahlsen': 'ballsed', 'deliveriing': 'delivering', '$riot': 'riot', 'hmrs': 'hers', 'mh02': 'mhat', 'grumped': 'trumped', 'weibo': 'weird', 'lafd': 'land', 'dassit': 'assist', 'fruitnet': 'fruited', 'cyprium': 'cyprus', 'bearwood': 'deadwood', 'sorry]': 'sorry', 'urewera': 'brewery', 'mtr': 'mar', 'maana': 'mana', 'dosomething': 'something', 'palapas': 'palapa', 'murphys': 'murphy', 'awsome': 'awesome', 'domt': 'dont', 'unnatended': 'unattended', 'broceries': 'groceries', 'geogian': 'georgian', 'egumball': 'gumball', 'seevices': 'services', 'vacseal': 'vassal', 'wyff': 'off', 'vido': 'video', 'lysol': 'sol', '2share': 'share', '80ish': 'wish', 'miah': 'mia', 'hoose': 'house', 'ramadan': 'armada', 'zatanna': 'katana', '23sept': 'sept', 'kwenzo': 'enzo', 'practo': 'tractor', 'bund': 'band', 'buffalonews': 'buffaloes', 'crackingup': 'cracking', 'hanyang': 'hanging', 's12': 'so', 'twilio': 'emilio', 'iiot': 'idiot', 'senryu': 'henry', 'andhi': 'andi', 'grindr': 'grind', 'visted': 'visited', '14july': 'july', 'cruisin': 'cruising', 'sendi': 'send', 'remainer': 'remained', 'tunein': 'tune', 'pitmad': 'pitied', 'gilet': 'filet', 'walkable': 'valuable', 'delivr': 'deliver'}

In [100]:
def correct_spellings(text, corrected):
    words = text.split(' ')
    new_words = []
    for word in words:
        if word in corrected:
            new_words.append(corrected[word])
        else:
            new_words.append(word)
    return ' '.join(new_words)
            
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x:correct_spellings(x, corrected_words))        

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 361480.56it/s]


In [101]:
print(count_link(data), count_i(data), count_my(data))

195869 220145 89060


### Remove non-english words

In [102]:
import nltk

In [103]:
#find non-english words
non_english = []
eng_words = set(nltk.corpus.words.words())
words = ' '.join(data['tweet']).split()
for i in tqdm(words):
    if i.lower() not in eng_words and i.lower() not in non_english:
        non_english.append(i)
len(non_english)        

100%|█████████████████████████████| 6610274/6610274 [00:20<00:00, 322165.51it/s]


34028

In [104]:
import nltk

eng_words = set(nltk.corpus.words.words())
keep = ['timelsots','timeslot','convenience','efficient','dissapoint','convenient','inconvenience','substitute',
       'recycling', 'recycle', 'pandemic','omnichannel', 'covid', 'curbside','hyperlink','biodegradable', 'hyperlink']

def remove_nonenglish_words(sent,eng_words,keep):
    words = [w for w in nltk.wordpunct_tokenize(sent) if w.lower() in eng_words or w.lower() in keep]
    return ' '.join(words)

In [105]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x:remove_nonenglish_words(x,eng_words,keep))
print(data['tweet'][5003])
print(data['tweet'][5006])

100%|███████████████████████████████| 368413/368413 [00:02<00:00, 155129.88it/s]

my wife stop grocery store curbside pickup they put your ordered your trunk contact she she saw ton boomer go store protection all
they also say it probably get food restaurant curb side versus grocery shopping right now less exposure they close also worker watch hand curbside delivery


### Find words related to Covid

In [106]:
def replace_curb(text):
    text = text.replace('curb side', 'curbside')
    return text

In [107]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x: replace_curb(x))

100%|██████████████████████████████| 368413/368413 [00:00<00:00, 1488164.99it/s]


In [108]:
def replace_covid(text):
    text = text.replace('corona', 'covid')
    return text

In [109]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x: replace_covid(x))

100%|██████████████████████████████| 368413/368413 [00:00<00:00, 1496361.95it/s]


In [110]:
words = ' '.join(data['tweet']).split()

In [111]:
len(data)

368413

In [112]:
data['Category'].value_counts()

Consumer         8377
Advertisement    8092
Name: Category, dtype: int64

In [113]:
data.to_csv('Cleaned_data_for_classification.csv', index = False)